# 🌊 Exploring Copernicus Marine Data in Google Colab

**An interactive notebook for retrieving and exploring Copernicus Marine datasets, created by Náttúrufræðistofnun.**

---

### Overview
This notebook connects to **Copernicus Marine Service** to retrieve and list all available datasets.
Using Python, it fetches dataset descriptions automatically and generates markdown documentation.

The script logs in Copernicus Marine using credentials and queries Copernicus Marine API for metadata.

Find a full list of available products here: https://data.marine.copernicus.eu/products

### How to Use
1. Open this notebook in Google Colab or another cloud platform.
2. Register a Copernicus Marine account here: https://data.marine.copernicus.eu/register
3. Insert valid credentials in section Login to access the data.
4. Run the provided code cells in sequence (do not skip cells) to explore dataset descriptions and available services, download, process and display them.

| Try the code via free cloud platforms: | [![Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/lmi/Copernicus/blob/master/Marine/Overview/ConnectMarine.ipynb)

---

### Attribution
- **License**: This notebook is published under **CC BY 4.0**.
- **Citation**: "*Credits: Náttúrufræðistofnun*"
- **Author**: Marco Pizzolato
- **Data Sources**: [Copernicus Marine Service](https://marine.copernicus.eu/)
- **Created in**: Python

---

**This notebook provides an automated way to explore and document Copernicus Marine datasets, making ocean data more accessible.**


## Install and Import

In [26]:
!pip install -q copernicusmarine pandas tabulate Pillow mdutils leafmap rioxarray xarray netCDF4 rasterio pyproj localtileserver

In [9]:
import os
import requests
from PIL import Image
from mdutils.mdutils import MdUtils
from mdutils.tools import Html

## Login to Copernicus Marine

Register a Copernicus Marine account here: https://data.marine.copernicus.eu/register

In [ ]:
import copernicusmarine as cm
import getpass

username = input("Enter your Copernicus Marine username: ")
password = getpass.getpass("Enter your Copernicus Marine password: ")

cm.login(username=username, password=password)

## Fetch Catalog

In [12]:
catalog = cm.describe()

Fetching products:  64%|██████▍   | 194/303 [00:16<00:13,  8.05it/s]WARNING:urllib3.connectionpool:Connection pool is full, discarding connection: s3.waw3-1.cloudferro.com. Connection pool size: 10

Fetching products:  97%|█████████▋| 295/303 [00:24<00:00, 28.08it/s]WARNING:urllib3.connectionpool:Connection pool is full, discarding connection: s3.waw3-1.cloudferro.com. Connection pool size: 10

Fetching products:  99%|█████████▊| 299/303 [00:24<00:00, 23.49it/s]WARNING:urllib3.connectionpool:Connection pool is full, discarding connection: s3.waw3-1.cloudferro.com. Connection pool size: 10

Fetching catalogue 1: 100%|██████████| 2/2 [00:36<00:00, 18.13s/it]


In [13]:
# Inspect the structure of the first product and its datasets
first_product = catalog.products[0]
print(first_product.model_dump())
print(first_product.datasets)

{'title': 'Antarctic Sea Ice Extent from Reanalysis', 'product_id': 'ANTARCTIC_OMI_SI_extent', 'thumbnail_url': 'https://documentation.marine.copernicus.eu/IMG/ANTARCTIC_OMI_SI_extent.png', 'description': '**DEFINITION**\n\nEstimates of Antarctic sea ice extent are obtained from the surface of oceans grid cells that have at least 15% sea ice concentration. These values are cumulated in the entire Southern Hemisphere (excluding ice lakes) and from 1993 up to real time aiming to:\ni) obtain the Antarctic sea ice extent as expressed in millions of km squared (106 km2) to monitor both the large-scale variability and mean state and change.\nii) to monitor the change in sea ice extent as expressed in millions of km squared per decade (106 km2/decade), or in sea ice extent loss/gain since the beginning of the time series as expressed in percent per decade (%/decade; reference period being the first date of the key figure b) dot-dashed trend line, Vaughan et al., 2013)). For the Southern Hemis

## Make Markdown File

In [14]:
# Initialize the Markdown file
md_file = MdUtils(file_name='Copernicus_Marine_Products_and_Datasets')

# Iterate through products
for product in catalog.products:
    product_title = product.title
    product_id = product.product_id
    description = product.description
    thumbnail_url = product.thumbnail_url

        # Add thumbnail image if available
    if thumbnail_url:
        # Resize the image to 100x100 pixels using HTML
        img_tag = Html.image(path=thumbnail_url, size='300x200')
        md_file.new_paragraph(img_tag)

    # Add an empty line after the image
    md_file.new_paragraph('\n')

    # Add a header for the product
    md_file.new_header(level=1, title=product_title)

    # Add product ID
    md_file.new_paragraph(f'**Product ID:** {product_id}')

    # Add description
    md_file.new_paragraph(f'**Description:** {description}')

    # Add an empty line after the image
    md_file.new_paragraph('\n')

In [15]:
# Create the Markdown file
md_file.create_md_file()

## Display the created overview

In [16]:
from IPython.display import display, Markdown

In [17]:
# Display the generated markdown file at the end of the notebook
with open("Copernicus_Marine_Products_and_Datasets.md", "r") as file:
    display(Markdown(file.read()))






<img src="https://documentation.marine.copernicus.eu/IMG/ANTARCTIC_OMI_SI_extent.png" width="300" height="200"/>



# Antarctic Sea Ice Extent from Reanalysis


**Product ID:** ANTARCTIC_OMI_SI_extent

**Description:** **DEFINITION**

Estimates of Antarctic sea ice extent are obtained from the surface of oceans grid cells that have at least 15% sea ice concentration. These values are cumulated in the entire Southern Hemisphere (excluding ice lakes) and from 1993 up to real time aiming to:
i) obtain the Antarctic sea ice extent as expressed in millions of km squared (106 km2) to monitor both the large-scale variability and mean state and change.
ii) to monitor the change in sea ice extent as expressed in millions of km squared per decade (106 km2/decade), or in sea ice extent loss/gain since the beginning of the time series as expressed in percent per decade (%/decade; reference period being the first date of the key figure b) dot-dashed trend line, Vaughan et al., 2013)). For the Southern Hemisphere, these trends are calculated from the annual mean values.
The Antarctic sea ice extent used here is based on the “multi-product” (GLOBAL_MULTIYEAR_PHY_ENS_001_031) approach as introduced in the second issue of the Ocean State Report (CMEMS OSR, 2017). Five global products have been used to build the ensemble mean, and its associated ensemble spread.

**CONTEXT**

Sea ice is frozen seawater that floats on the ocean surface. This large blanket of millions of square kilometers insulates the relatively warm ocean waters from the cold polar atmosphere. The seasonal cycle of the sea ice, forming and melting with the polar seasons, impacts both human activities and biological habitat. Knowing how and how much the sea ice cover is changing is essential for monitoring the health of the Earth as sea ice is one of the highest sensitive natural environments. Variations in sea ice cover can induce changes in ocean stratification and modify the key rule played by the cold poles in the Earth engine (IPCC, 2019).  
The sea ice cover is monitored here in terms of sea ice extent quantity. More details and full scientific evaluations can be found in the CMEMS Ocean State Report (Samuelsen et al., 2016; Samuelsen et al., 2018).
 
**CMEMS KEY FINDINGS**

With quasi regular highs and lows, the annual Antarctic sea ice extent shows large variability until several monthly record high in 2014 and record lows in 2017, 2018 and 2023. Since the year 1993, the Southern Hemisphere annual sea ice extent regularly alternates positive and negative trend. The period 1993-2023 have seen a slight decrease at a rate of -0.28*106km2 per decade. This represents a loss amount of -2.4% per decade of Southern Hemisphere sea ice extent during this period; with however large uncertainties. The last quarter of the year 2016 and years 2017 and 2018 experienced unusual losses of ice. The year 2023 is an exceptional year and its average has a strong impact on the whole time series.

Note: The key findings will be updated annually in November, in line with OMI evolutions.

**DOI (product):** 
https://doi.org/10.48670/moi-00186

**References:**

* IPCC Special Report on the Ocean and Cryosphere in a Changing Climate. (2019). In H. O. Pörtner, D. C. Roberts, V. Masson-Delmotte, P. Zhai, M. Tignor, E. Poloczanska, K. Mintenbeck, A. Alegría, M. Nicolai, A. Okem, J. Petzold, B. Rama, & N. M. Weyer (Eds.), IPCC Intergovernmental Panel on Climate Change: Geneva, Switzerland. https://www.ipcc.ch/srocc/
* Samuelsen et al., 2016: Sea Ice In: The Copernicus Marine Environment Monitoring Service Ocean State Report, issue 1, Journal of Operational Oceanography, 9, 2016, http://dx.doi.org/10.1080/1755876X.2016.1273446.
* Samuelsen et al., 2018: Sea Ice. In: The Copernicus Marine Environment Monitoring Service Ocean State Report, issue 2, Journal of Operational Oceanography, 11:sup1, 2018, DOI: 10.1080/1755876X.2018.1489208.
* Vaughan, D.G., J.C. Comiso, I. Allison, J. Carrasco, G. Kaser, R. Kwok, P. Mote, T. Murray, F. Paul, J. Ren, E. Rignot, O. Solomina, K. Steffen and T. Zhang, 2013: Observations: Cryosphere. In: Climate Change 2013: The Physical Science Basis. Contribution of Working Group I to the Fifth Assessment Report of the Intergovernmental Panel on Climate Change [Stocker, T.F., D. Qin, G.-K. Plattner, M.Tignor, S.K. Allen, J. Boschung, A. Nauels, Y. Xia, V. Bex and P.M. Midgley (eds.)]. Cambridge University Press, Cambridge, United Kingdom and New York, NY, USA, pp. 317–382, doi:10.1017/CBO9781107415324.012.





<img src="https://documentation.marine.copernicus.eu/IMG/ANTARCTIC_OMI_SI_extent_obs.png" width="300" height="200"/>



# Antarctic Monthly Sea Ice Extent from Observations Reprocessing


**Product ID:** ANTARCTIC_OMI_SI_extent_obs

**Description:** **DEFINITION**

Sea Ice Extent (SIE) is defined as the area covered by sufficient sea ice, that is the area of ocean having more than 15% Sea Ice Concentration (SIC). SIC is the fractional area of ocean surface that is covered with sea ice. SIC is computed from Passive Microwave satellite observations since 1979. 
SIE is often reported with units of 106 km2 (millions square kilometers). The change in sea ice extent (trend) is expressed in millions of km squared per decade (106 km2/decade). In addition, trends are expressed relative to the 1979-2022 period in % per decade.
These trends are calculated (i) from the annual mean values; (ii) from the September values (winter ice loss); (iii) from February values (summer ice loss). The annual mean trend is reported on the key figure, the September (maximum extent) and February (minimum extent) values are reported in the text below.
SIE includes all sea ice, except for lake and river ice.
See also section 1.7 in Samuelsen et al. (2016) for an introduction to this Ocean Monitoring Indicator (OMI).

**CONTEXT**

Sea ice is frozen seawater that floats at the ocean surface. This large blanket of millions of square kilometers insulates the relatively warm ocean waters from the cold polar atmosphere. The seasonal cycle of sea ice, forming and melting with the polar seasons, impacts both human activities and biological habitat. Knowing how and by how much the sea-ice cover is changing is essential for monitoring the health of the Earth (Meredith et al. 2019). 

**CMEMS KEY FINDINGS**

Since 1979, there has been an overall slight increase of sea ice extent in the Southern Hemisphere but a sharp decrease was observed after 2016. Over the period 1979-2022, the annual rate amounts to +0.02 +/- 0.05 106 km2 per decade (+0.18% per decade). Winter (September) sea ice extent trend amounts to +0.06 +/- 0.05106 km2 per decade (+0.32% per decade). Summer (February) sea ice extent trend amounts to -0.01+/- 0.05 106 km2 per decade (-0.38% per decade). These trend estimates are hardly significant, which is in agreement with the IPCC SROCC, which has assessed that ‘Antarctic sea ice extent overall has had no statistically significant trend (1979–2018) due to contrasting regional signals and large interannual variability (high confidence).’ (IPCC, 2019). Both June and July 2022 had the lowest average sea ice extent values for these months since 1979. 

**Figure caption**

a) The seasonal cycle of Southern Hemisphere sea ice extent expressed in millions of km2 averaged over the period 1979-2022 (red), shown together with the seasonal cycle in the year 2022 (green), and b) time series of yearly average Southern Hemisphere sea ice extent expressed in millions of km2. Time series are based on satellite observations (SMMR, SSM/I, SSMIS) by EUMETSAT OSI SAF Sea Ice Index (v2.2) with R&D input from ESA CCI. Details on the product are given in the corresponding PUM for this OMI. The change of sea ice extent over the period 1979-2022      is expressed as a trend in millions of square kilometers per decade and is plotted with a dashed line on panel b).

**DOI (product):** 
https://doi.org/10.48670/moi-00187

**References:**

* Meredith, M., M. Sommerkorn, S. Cassotta, C. Derksen, A. Ekaykin, A. Hollowed, G. Kofinas, A. Mackintosh, J. Melbourne-Thomas, M.M.C. Muelbert, G. Ottersen, H. Pritchard, and E.A.G. Schuur, 2019: Polar Regions. In: IPCC Special Report on the Ocean and Cryosphere in a Changing Climate [H.-O. Pörtner, D.C. Roberts, V. Masson-Delmotte, P. Zhai, M. Tignor, E. Poloczanska, K. Mintenbeck, A. Alegría, M. Nicolai, A. Okem, J. Petzold, B. Rama, N.M. Weyer (eds.)]. In press.
* IPCC, 2019: IPCC Special Report on the Ocean and Cryosphere in a Changing Climate [H.-O. Pörtner, D.C. Roberts, V. Masson-Delmotte, P. Zhai, M. Tignor, E. Poloczanska, K. Mintenbeck, A. Alegría, M. Nicolai, A. Okem, J. Petzold, B. Rama, N.M. Weyer (eds.)]. In press.
* Samuelsen, A., L.-A. Breivik, R.P. Raj, G. Garric, L. Axell, E. Olason (2016): Sea Ice. In: The Copernicus Marine Service Ocean State Report, issue 1, Journal of Operational Oceanography, 9:sup2, s235-s320, DOI: 10.1080/1755876X.2016.1273446





<img src="https://mdl-metadata.s3.waw3-1.cloudferro.com/metadata/thumbnails/ARCTIC_ANALYSISFORECAST_BGC_002_004.jpg" width="300" height="200"/>



# Arctic Ocean Biogeochemistry Analysis and Forecast


**Product ID:** ARCTIC_ANALYSISFORECAST_BGC_002_004

**Description:** The operational TOPAZ5-ECOSMO Arctic Ocean system uses the ECOSMO biological model coupled online to the TOPAZ5 physical model (ARCTIC_ANALYSISFORECAST_PHY_002_001 product). It is run daily to provide 10 days of forecast of 3D biogeochemical variables ocean. The coupling is done by the FABM framework.

Coupling to a biological ocean model provides a description of the evolution of basic biogeochemical variables. The output consists of daily mean fields interpolated onto a standard grid and 40 fixed levels in NetCDF4 CF format. Variables include 3D fields of nutrients (nitrate, phosphate, silicate), phytoplankton and zooplankton biomass, oxygen, chlorophyll, primary productivity, carbon cycle variables (pH, dissolved inorganic carbon and surface partial CO2 pressure in seawater) and light attenuation coefficient. Surface Chlorophyll-a from satellite ocean colour is assimilated every week and projected downwards using a modified Ardyna et al. (2013) method. A new 10-day forecast is produced daily using the previous day's forecast and the most up-to-date prognostic forcing fields.
Output products have 6.25 km resolution at the North Pole (equivalent to 1/8 deg) on a stereographic projection. See the Product User Manual for the exact projection parameters.

**DOI (product):**   
https://doi.org/10.48670/moi-00003

**References:**

* Ardyna, M., Babin, M., Gosselin, M., Devred, E., Bélanger, S., Matsuoka, A., and Tremblay, J.-É.: Parameterization of vertical chlorophyll a in the Arctic Ocean: impact of the subsurface chlorophyll maximum on regional, seasonal, and annual primary production estimates, Biogeosciences, 10, 4383–4404, https://doi.org/10.5194/bg-10-4383-2013, 2013.
* Yumruktepe, V. Ç., Samuelsen, A., and Daewel, U.: ECOSMO II(CHL): a marine biogeochemical model for the North Atlantic and the Arctic, Geosci. Model Dev., 15, 3901–3921, https://doi.org/10.5194/gmd-15-3901-2022, 2022.





<img src="https://mdl-metadata.s3.waw3-1.cloudferro.com/metadata/thumbnails/ARCTIC_ANALYSISFORECAST_PHY_002_001.jpg" width="300" height="200"/>



# Arctic Ocean Physics Analysis and Forecast


**Product ID:** ARCTIC_ANALYSISFORECAST_PHY_002_001

**Description:** The operational TOPAZ5 Arctic Ocean system uses the HYCOM model and a 100-member EnKF assimilation scheme. It is run daily to provide 10 days of forecast (average of 10 members) of the 3D physical ocean, including sea ice with the CICEv5.1 model; data assimilation is performed weekly to provide 7 days of analysis (ensemble average).

Output products are interpolated on a grid of 6 km resolution at the North Pole on a polar stereographic projection. The geographical projection follows these proj4 library parameters: 

proj4 = "+units=m +proj=stere +lon_0=-45 +lat_0=90 +k=1 +R=6378273 +no_defs" 

**DOI (product):**
https://doi.org/10.48670/moi-00001

**References:**

* Sakov, P., Counillon, F., Bertino, L., Lisæter, K. A., Oke, P. R. and Korablev, A.: TOPAZ4: an ocean-sea ice data assimilation system for the North Atlantic and Arctic, Ocean Sci., 8(4), 633–656, doi:10.5194/os-8-633-2012, 2012.
* Melsom, A., Counillon, F., LaCasce, J. H. and Bertino, L.: Forecasting search areas using ensemble ocean circulation modeling, Ocean Dyn., 62(8), 1245–1257, doi:10.1007/s10236-012-0561-5, 2012.





<img src="https://mdl-metadata.s3.waw3-1.cloudferro.com/metadata/thumbnails/ARCTIC_ANALYSISFORECAST_PHY_ICE_002_011.jpg" width="300" height="200"/>



# Arctic Ocean Sea Ice Analysis and Forecast


**Product ID:** ARCTIC_ANALYSISFORECAST_PHY_ICE_002_011

**Description:** The Arctic Sea Ice Analysis and Forecast system uses the neXtSIM stand-alone sea ice model running the Brittle-Bingham-Maxwell sea ice rheology on an adaptive triangular mesh of 10 km average cell length. The model domain covers the whole Arctic domain, including the Canadian Archipelago and the Bering Sea. neXtSIM is forced with surface atmosphere forcings from the ECMWF (European Centre for Medium-Range Weather Forecasts) and ocean forcings from TOPAZ5, the ARC MFC PHY NRT system (002_001a). neXtSIM runs daily, assimilating manual ice charts, sea ice thickness from CS2SMOS in winter and providing 9-day forecasts. The output variables are the ice concentrations, ice thickness, ice drift velocity, snow depths, sea ice type, sea ice age, ridge volume fraction and albedo, provided at hourly frequency. The adaptive Lagrangian mesh is interpolated for convenience on a 3 km resolution regular grid in a Polar Stereographic projection. The projection is identical to other ARC MFC products.


**DOI (product):** 

https://doi.org/10.48670/moi-00004

**References:**

* Williams, T., Korosov, A., Rampal, P., and Ólason, E.: Presentation and evaluation of the Arctic sea ice forecasting system neXtSIM-F, The Cryosphere, 15, 3207–3227, https://doi.org/10.5194/tc-15-3207-2021, 2021.





<img src="https://mdl-metadata.s3.waw3-1.cloudferro.com/metadata/thumbnails/ARCTIC_ANALYSISFORECAST_PHY_TIDE_002_015.jpg" width="300" height="200"/>



# Arctic Ocean Tidal Analysis and Forecast


**Product ID:** ARCTIC_ANALYSISFORECAST_PHY_TIDE_002_015

**Description:** The Arctic Ocean Surface Currents Analysis and Forecast system uses the HYCOM model at 3 km resolution forced with tides at its lateral boundaries, surface winds sea level pressure from the ECMWF (European Centre for Medium-Range Weather Forecasts) and wave terms (Stokes-Coriolis drift, stress and parameterisation of mixing by Langmuir cells) from the Arctic wave forecast. HYCOM runs daily providing 10 days forecast. The output variables are the surface currents and sea surface heights, provided at 15 minutes frequency, which therefore include mesoscale signals (though without data assimilation so far), tides and storm surge signals. 

**DOI (product):**   
https://doi.org/10.48670/moi-00005




<img src="https://mdl-metadata.s3.waw3-1.cloudferro.com/metadata/thumbnails/ARCTIC_ANALYSIS_FORECAST_WAV_002_014.jpg" width="300" height="200"/>



# Arctic Ocean Wave Analysis and Forecast


**Product ID:** ARCTIC_ANALYSIS_FORECAST_WAV_002_014

**Description:** The Arctic Ocean Wave Analysis and Forecast system uses the WAM model at 3 km resolution forced with surface winds and boundary wave spectra from the ECMWF (European Centre for Medium-Range Weather Forecasts) together with tidal currents and ice from the ARC MFC forecasts (Sea Ice concentration and thickness). WAM runs twice daily providing one hourly 10 days forecast and one hourly 5 days forecast. From the output variables the most commonly used are significant wave height, peak period and mean direction.

**DOI (product):**  
https://doi.org/10.48670/moi-00002




<img src="https://mdl-metadata.s3.waw3-1.cloudferro.com/metadata/thumbnails/ARCTIC_MULTIYEAR_BGC_002_005.jpg" width="300" height="200"/>



# Arctic Ocean Biogeochemistry Reanalysis


**Product ID:** ARCTIC_MULTIYEAR_BGC_002_005

**Description:** The TOPAZ-ECOSMO reanalysis system assimilates satellite chlorophyll observations and in situ nutrient profiles.  The model uses the Hybrid Coordinate Ocean Model (HYCOM) coupled online to a sea ice model and the ECOSMO biogeochemical model. It uses the Determinstic version of the Ensemble Kalman Smoother to assimilate remotely sensed colour data and nutrient profiles. Data assimilation, including the 80-member ensemble production, is performed every 8-days. Atmospheric forcing fields from the ECMWF ERA-5 dataset are used

**DOI (product):**  
https://doi.org/10.48670/moi-00006

**References:**

* Simon, E., Samuelsen, A., Bertino, L. and Mouysset, S.: Experiences in multiyear combined state-parameter estimation with an ecosystem model of the North Atlantic and Arctic Oceans using the Ensemble Kalman Filter, J. Mar. Syst., 152, 1–17, doi:10.1016/j.jmarsys.2015.07.004, 2015.





<img src="https://mdl-metadata.s3.waw3-1.cloudferro.com/metadata/thumbnails/ARCTIC_MULTIYEAR_PHY_002_003.jpg" width="300" height="200"/>



# Arctic Ocean Physics Reanalysis


**Product ID:** ARCTIC_MULTIYEAR_PHY_002_003

**Description:** The current version of the TOPAZ system - TOPAZ4b -  is nearly identical to the real-time forecast system run at MET Norway. It uses a recent version of the Hybrid Coordinate Ocean Model (HYCOM) developed at University of Miami (Bleck 2002). HYCOM is coupled to a sea ice model; ice thermodynamics are described in Drange and Simonsen (1996) and the elastic-viscous-plastic rheology in Hunke and Dukowicz (1997). The model's native grid covers the Arctic and North Atlantic Oceans,  has fairly homogeneous horizontal spacing (between 11 and 16 km). 50 hybrid layers are used in the vertical (z-isopycnal). TOPAZ4b uses the Deterministic version of the Ensemble Kalman filter (DEnKF; Sakov and Oke 2008) to assimilate remotely sensed as well as temperature and salinity profiles. The output is interpolated onto standard grids and depths for convenience. Daily values are provided at all depths and surfaces momentum and heat fluxes are provided as well. Data assimilation, including the 100-member ensemble production, is performed weekly. Sea ice thickness prior to 2010 has been post-processed by an AI-based procedure. 

**DOI (product):**   
https://doi.org/10.48670/moi-00007




<img src="https://mdl-metadata.s3.waw3-1.cloudferro.com/metadata/thumbnails/ARCTIC_MULTIYEAR_PHY_ICE_002_016.jpg" width="300" height="200"/>



# Arctic Ocean Sea Ice Reanalysis


**Product ID:** ARCTIC_MULTIYEAR_PHY_ICE_002_016

**Description:** The Arctic Sea Ice Reanalysis system uses the neXtSIM stand-alone sea ice model running the Brittle-Bingham-Maxwell sea ice rheology on an adaptive triangular mesh of 10 km average cell length. The model domain covers the whole Arctic domain, from Bering Strait to the North Atlantic. neXtSIM is forced by reanalyzed surface atmosphere forcings (ERA5) from the ECMWF (European Centre for Medium-Range Weather Forecasts) and ocean forcings from TOPAZ4b, the ARC MFC MYP system (002_003). neXtSIM assimilates satellite sea ice concentrations from Passive Microwave satellite sensors, and sea ice thickness from CS2SMOS in winter from October 2010 onwards. The output variables are sea ice concentrations (total, young ice, and multi-year ice), sea ice thickness, sea ice velocity, snow depth on sea ice, sea ice type, sea ice age, sea ice ridge volume fraction and sea ice albedo, provided at daily and monthly frequency. The adaptive Lagrangian mesh is interpolated for convenience on a 3 km resolution regular grid in a Polar Stereographic projection. The projection is identical to other ARC MFC products.

**DOI (product):**   
https://doi.org/10.48670/mds-00336

**References:**

* Williams, T., Korosov, A., Rampal, P., and Ólason, E.: Presentation and evaluation of the Arctic sea ice forecasting system neXtSIM-F, The Cryosphere, 15, 3207–3227, https://doi.org/10.5194/tc-15-3207-2021, 2021.





<img src="https://mdl-metadata.s3.waw3-1.cloudferro.com/metadata/thumbnails/ARCTIC_MULTIYEAR_WAV_002_013.jpg" width="300" height="200"/>



# Arctic Ocean Wave Hindcast


**Product ID:** ARCTIC_MULTIYEAR_WAV_002_013

**Description:** The Arctic Ocean Wave Hindcast system uses the WAM model at 3 km resolution forced with surface winds and boundary wave spectra from the ECMWF (European Centre for Medium-Range Weather Forecasts) ERA5 reanalysis together with ice from the ARC MFC reanalysis (Sea Ice concentration and thickness). Additionally, in the North Atlantic area, surface winds are used from a 2.5km atmospheric hindcast system. From the output variables the most commonly used are significant wave height, peak period and mean direction.

**DOI (product):**   
https://doi.org/10.48670/moi-00008




<img src="https://documentation.marine.copernicus.eu/IMG/ARCTIC_OMI_SI_Transport_NordicSeas.png" width="300" height="200"/>



# Sea Ice Area/Volume Transport in the Nordic Seas from Reanalysis


**Product ID:** ARCTIC_OMI_SI_Transport_NordicSeas

**Description:** **DEFINITION**

Net sea-ice volume and area transport through the openings Fram Strait between Spitsbergen and Greenland along 79°N, 20°W - 10°E (positive southward); northern Barents Sea between Svalbard and Franz Josef Land archipelagos along 80°N, 27°E - 60°E (positive southward); eastern Barents Sea between the Novaya Zemlya and Franz Josef Land archipelagos along 60°E, 76°N - 80°N (positive westward). For further details, see Lien et al. (2021).

**CONTEXT**

The Arctic Ocean contains a large amount of freshwater, and the freshwater export from the Arctic to the North Atlantic influence the stratification, and, the Atlantic Meridional Overturning Circulation (e.g., Aagaard et al., 1985). The Fram Strait represents the major gateway for freshwater transport from the Arctic Ocean, both as liquid freshwater and as sea ice (e.g., Vinje et al., 1998). The transport of sea ice through the Fram Strait is therefore important for the mass balance of the perennial sea-ice cover in the Arctic as it represents a large export of about 10% of the total sea ice volume every year (e.g., Rampal et al., 2011). Sea ice export through the Fram Strait has been found to explain a major part of the interannual variations in Arctic perennial sea ice volume changes (Ricker et al., 2018). The sea ice and associated freshwater transport to the Barents Sea has been suggested to be a driving mechanism for the presence of Arctic Water in the northern Barents Sea, and, hence, the presence of the Barents Sea Polar Front dividing the Barents Sea into a boreal and an Arctic part (Lind et al., 2018). In recent decades, the Arctic part of the Barents Sea has been giving way to an increasing boreal part, with large implications for the marine ecosystem and harvestable resources (e.g., Fossheim et al., 2015).

**CMEMS KEY FINDINGS**

The sea-ice transport through the Fram Strait shows a distinct seasonal cycle in both sea ice area and volume transport, with a maximum in winter. There is a slight positive trend in the volume transport over the last two and a half decades. In the Barents Sea, a strong reduction of nearly 90% in average sea-ice thickness has diminished the sea-ice import from the Polar Basin (Lien et al., 2021). In both areas, the Fram Strait and the Barents Sea, the winds governed by the regional patterns of atmospheric pressure is an important driving force of temporal variations in sea-ice transport (e.g., Aaboe et al., 2021; Lien et al., 2021).

**DOI (product):** 
https://doi.org/10.48670/moi-00192

**References:**

* Aaboe S, Lind S, Hendricks S, Down E, Lavergne T, Ricker R. 2021. Sea-ice and ocean conditions surprisingly normal in the Svalbard-Barents Sea region after large sea-ice inflows in 2019. In: Copernicus Marine Environment Monitoring Service Ocean State Report, issue 5, J Oper Oceanogr. 14, sup1, 140-148
* Aagaard K, Swift JH, Carmack EC. 1985. Thermohaline circulation in the Arctic Mediterranean seas. J Geophys Res. 90(C7), 4833-4846
* Fossheim M, Primicerio R, Johannesen E, Ingvaldsen RB, Aschan MM, Dolgov AV. 2015. Recent warming leads to a rapid borealization of fish communities in the Arctic. Nature Clim Change. doi:10.1038/nclimate2647
* Lien VS, Raj RP, Chatterjee S. 2021. Modelled sea-ice volume and area transport from the Arctic Ocean to the Nordic and Barents seas. In: Copernicus Marine Environment Monitoring Service Ocean State Report, issue 5, J Oper Oceanogr. 14, sup1, 10-17
* Lind S, Ingvaldsen RB, Furevik T. 2018. Arctic warming hotspot in the northern Barents Sea linked to declining sea ice import. Nature Clim Change. doi:10.1038/s41558-018-0205-y
* Rampal P, Weiss J, Dubois C, Campin J-M. 2011. IPCC climate models do not capture Arctic sea ice drift acceleration: Consequences in terms of projected sea ice thinning and decline. J Geophys Res. 116, C00D07. https://doi.org/10.1029/2011JC007110
* Ricker R, Girard-Ardhuin F, Krumpen T, Lique C. 2018. Satellite-derived sea ice export and its impact on Arctic ice mass balance. Cryosphere. 12, 3017-3032
* Vinje T, Nordlund N, Kvambekk Å. 1998. Monitoring ice thickness in Fram Strait. J Geophys Res. 103(C5), 10437-10449





<img src="https://documentation.marine.copernicus.eu/IMG/ARCTIC_OMI_SI_extent.png" width="300" height="200"/>



# Arctic Sea Ice Extent from Reanalysis


**Product ID:** ARCTIC_OMI_SI_extent

**Description:** **DEFINITION**

Estimates of Arctic sea ice extent are obtained from the surface of oceans grid cells that have at least 15% sea ice concentration. These values are cumulated in the entire Northern Hemisphere (excluding ice lakes) and from 1993 up to the year 2019 aiming to:
i) obtain the Arctic sea ice extent as expressed in millions of km square (106 km2) to monitor both the large-scale variability and mean state and change.
ii) to monitor the change in sea ice extent as expressed in millions of km squared per decade (106 km2/decade), or in sea ice extent loss since the beginning of the time series as expressed in percent per decade (%/decade; reference period being the first date of the key figure b) dot-dashed trend line, Vaughan et al., 2013). These trends are calculated in three ways, i.e. (i) from the annual mean values; (ii) from the March values (winter ice loss); (iii) from September values (summer ice loss).
The Arctic sea ice extent used here is based on the “multi-product”  (GLOBAL_MULTIYEAR_PHY_ENS_001_031) approach as introduced in the second issue of the Ocean State Report (CMEMS OSR, 2017). Five global products have been used to build the ensemble mean, and its associated ensemble spread.

**CONTEXT**

Sea ice is frozen seawater that floats on the ocean surface. This large blanket of millions of square kilometers insulates the relatively warm ocean waters from the cold polar atmosphere. The seasonal cycle of the sea ice, forming and melting with the polar seasons, impacts both human activities and biological habitat. Knowing how and how much the sea ice cover is changing is essential for monitoring the health of the Earth as sea ice is one of the highest sensitive natural environments. Variations in sea ice cover can induce changes in ocean stratification, in global and regional sea level rates and modify the key rule played by the cold poles in the Earth engine (IPCC, 2019).  
The sea ice cover is monitored here in terms of sea ice extent quantity. More details and full scientific evaluations can be found in the CMEMS Ocean State Report (Samuelsen et al., 2016; Samuelsen et al., 2018).

**CMEMS KEY FINDINGS**

Since the year 1993 to 2023 the Arctic sea ice extent has decreased significantly at an annual rate of -0.57*106 km2 per decade. This represents an amount of -4.8 % per decade of Arctic sea ice extent loss over the period 1993 to 2023. Over the period 1993 to 2018, summer (September) sea ice extent loss amounts to -1.18*106 km2/decade (September values), which corresponds to -14.85% per decade. Winter (March) sea ice extent loss amounts to -0.57*106 km2/decade, which corresponds to -3.42% per decade. These values slightly exceed the estimates given in the AR5 IPCC assessment report (estimate up to the year 2012) as a consequence of continuing Northern Hemisphere sea ice extent loss. Main change in the mean seasonal cycle is characterized by less and less presence of sea ice during summertime with time.  

Note: The key findings will be updated annually in November, in line with OMI evolutions.

**DOI (product):** 
https://doi.org/10.48670/moi-00190

**References:**

* IPCC Special Report on the Ocean and Cryosphere in a Changing Climate. (2019). In H. O. Pörtner, D. C. Roberts, V. Masson-Delmotte, P. Zhai, M. Tignor, E. Poloczanska, K. Mintenbeck, A. Alegría, M. Nicolai, A. Okem, J. Petzold, B. Rama, & N. M. Weyer (Eds.), IPCC Intergovernmental Panel on Climate Change: Geneva, Switzerland. https://www.ipcc.ch/srocc/
* Samuelsen et al., 2016: Sea Ice In: The Copernicus Marine Environment Monitoring Service Ocean State Report, issue 1, Journal of Operational Oceanography, 9, 2016, http://dx.doi.org/10.1080/1755876X.2016.1273446.
* Samuelsen et al., 2018: Sea Ice. In: The Copernicus Marine Environment Monitoring Service Ocean State Report, issue 2, Journal of Operational Oceanography, 11:sup1, 2018, DOI: 10.1080/1755876X.2018.1489208.
* Vaughan, D.G., J.C. Comiso, I. Allison, J. Carrasco, G. Kaser, R. Kwok, P. Mote, T. Murray, F. Paul, J. Ren, E. Rignot, O. Solomina, K. Steffen and T. Zhang, 2013: Observations: Cryosphere. In: Climate Change 2013: The Physical Science Basis. Contribution of Working Group I to the Fifth Assessment Report of the Intergovernmental Panel on Climate Change [Stocker, T.F., D. Qin, G.-K. Plattner, M.Tignor, S.K. Allen, J. Boschung, A. Nauels, Y. Xia, V. Bex and P.M. Midgley (eds.)]. Cambridge University Press, Cambridge, United Kingdom and New York, NY, USA, pp. 317–382, doi:10.1017/CBO9781107415324.012.





<img src="https://documentation.marine.copernicus.eu/IMG/ARCTIC_OMI_SI_extent_obs.png" width="300" height="200"/>



# Arctic Monthly Mean Sea Ice Extent from Observations Reprocessing


**Product ID:** ARCTIC_OMI_SI_extent_obs

**Description:** **DEFINITION**

Sea Ice Extent (SIE) is defined as the area covered by sufficient sea ice, that is the area of ocean having more than 15% Sea Ice Concentration (SIC). SIC is the fractional area of ocean that is covered with sea ice. It is computed from Passive Microwave satellite observations since 1979. 
SIE is often reported with units of 106 km2 (millions square kilometers). The change in sea ice extent (trend) is expressed in millions of km squared per decade (106 km2/decade). In addition, trends are expressed relative to the 1979-2022 period in % per decade.
These trends are calculated (i) from the annual mean values; (ii) from the March values (winter ice loss); (iii) from September values (summer ice loss). The annual mean trend is reported on the key figure, the March and September values are reported in the text below.
SIE includes all sea ice, but not lake or river ice.
See also section 1.7 in Samuelsen et al. (2016) for an introduction to this Ocean Monitoring Indicator (OMI).

**CONTEXT**

Sea ice is frozen seawater that floats at the ocean surface. This large blanket of millions of square kilometers insulates the relatively warm ocean waters from the cold polar atmosphere. The seasonal cycle of sea ice, forming and melting with the polar seasons, impacts both human activities and biological habitat. Knowing how and by how much the sea ice cover is changing is essential for monitoring the health of the Earth. Sea ice has a significant impact on ecosystems and Arctic communities, as well as economic activities such as fisheries, tourism, and transport (Meredith et al. 2019).

**CMEMS KEY FINDINGS**

Since 1979, the Northern Hemisphere sea ice extent has decreased at an annual rate of -0.51 +/- 0.03106 km2 per decade (-4.41% per decade).  Loss of sea ice extent during summer exceeds the loss observed during winter periods: Summer (September) sea ice extent loss amounts to -0.81 +/- 0.06 106 km2 per decade (-12.73% per decade). Winter (March) sea ice extent loss amounts to -0.39 +/- 0.03 106 km2 per decade (-2.55% per decade). These values are in agreement with those assessed in the IPCC Special Report on the Ocean and Cryosphere in a Changing Climate (SROCC) (Meredith et al. 2019, with estimates up until year 2018). September 2022 had the 11th lowest mean September sea ice extent. Sea ice extent in September 2012 is to date the record minimum Northern Hemisphere sea ice extent value since the beginning of the satellite record, followed by September values in 2020.

**Figure caption**

a) The seasonal cycle of Northern Hemisphere sea ice extent expressed in millions of km2 averaged over the period 1979-2022 (red), shown together with the seasonal cycle in the year 2022 (green), and b) time series of yearly average Northern Hemisphere sea ice extent expressed in millions of km2. Time series are based on satellite observations (SMMR, SSM/I, SSMIS) by EUMETSAT OSI SAF Sea Ice Index (v2.2) with R&D input from ESA CCI. Details on the product are given in the corresponding PUM for this OMI. The change of sea ice extent over the period 1979-2022 is expressed as a trend in millions of square kilometers per decade and is plotted with a dashed line in panel b).

**DOI (product):** 
https://doi.org/10.48670/moi-00191

**References:**

* Samuelsen, A., L.-A. Breivik, R.P. Raj, G. Garric, L. Axell, E. Olason (2016): Sea Ice. In: The Copernicus Marine Service Ocean State Report, issue 1, Journal of Operational Oceanography, 9:sup2, s235-s320, DOI: 10.1080/1755876X.2016.1273446
* Meredith, M., M. Sommerkorn, S. Cassotta, C. Derksen, A. Ekaykin, A. Hollowed, G. Kofinas, A. Mackintosh, J. Melbourne-Thomas, M.M.C. Muelbert, G. Ottersen, H. Pritchard, and E.A.G. Schuur, 2019: Polar Regions. In: IPCC Special Report on the Ocean and Cryosphere in a Changing Climate [H.-O. Pörtner, D.C. Roberts, V. Masson-Delmotte, P. Zhai, M. Tignor, E. Poloczanska, K. Mintenbeck, A. Alegría, M. Nicolai, A. Okem, J. Petzold, B. Rama, N.M. Weyer (eds.)]. In press.





<img src="https://documentation.marine.copernicus.eu/IMG/ARCTIC_OMI_TEMPSAL_FWC.png" width="300" height="200"/>



# Arctic Freshwater Content from Reanalysis


**Product ID:** ARCTIC_OMI_TEMPSAL_FWC

**Description:** **DEFINITION**

Estimates of Arctic liquid Freshwater Content (FWC in meters) are obtained from integrated differences of the measured salinity and a reference salinity (set to 34.8) from the surface to the bottom per unit area in the Arctic region with a water depth greater than 500m as function of salinity (S), the vertical cell thickness of the dataset (dz) and the salinity reference (Sref). Waters saltier than the 34.8 reference are not included in the estimation. The regional FWC values from 1993 up to real time are then averaged aiming to:
* obtain the mean FWC as expressed in cubic km (km3) 
* monitor the large-scale variability and change of liquid freshwater stored in the Arctic Ocean (i.e. the change of FWC in time).

**CONTEXT**

The Arctic region is warming twice as fast as the global mean and its climate is undergoing unprecedented and drastic changes, affecting all the components of the Arctic system. Many of these changes affect the hydrological cycle. Monitoring the storage of freshwater in the Arctic region is essential for understanding the contemporary Earth system state and variability. Variations in Arctic freshwater can induce changes in ocean stratification. Exported southward downstream, these waters have potential future implications for global circulation and heat transport.  

**CMEMS KEY FINDINGS**

Since 1993, the Arctic Ocean freshwater has experienced a significant increase of 423 ± 39 km3/year. The year 2016 witnessed the highest freshwater content in the Artic since the last 24 years. Second half of 2016 and first half of 2017 show a substantial decrease of the FW storage. 

Note: The key findings will be updated annually in November, in line with OMI evolutions.

**DOI (product):**
https://doi.org/10.48670/moi-00193

**References:**

* G. Garric, O. Hernandez, C. Bricaud, A. Storto, K. A. Peterson, H. Zuo, 2018: Arctic Ocean freshwater content. In: Copernicus Marine Service Ocean State Report, Issue 2, Journal of Operational Oceanography, 11:sup1, s70–s72, DOI: 10.1080/1755876X.2018.1489208





<img src="https://mdl-metadata.s3.waw3-1.cloudferro.com/metadata/thumbnails/BALTICSEA_ANALYSISFORECAST_BGC_003_007.jpg" width="300" height="200"/>



# Baltic Sea Biogeochemistry Analysis and Forecast


**Product ID:** BALTICSEA_ANALYSISFORECAST_BGC_003_007

**Description:** This Baltic Sea biogeochemical model product provides forecasts for the biogeochemical conditions in the Baltic Sea. The Baltic forecast is updated twice daily from a 00Z production proving a 10 days forecast and from a 12Z production providing a 6 days forecast. Different datasets are provided. One with daily means and one with monthly means values for these parameters: nitrate, phosphate, chl-a, ammonium, dissolved oxygen, ph, phytoplankton, zooplankton, silicate,  dissolved inorganic carbon, dissolved iron, dissolved cdom, hydrogen sulfide, and partial pressure of co2 at the surface. Instantaenous values for the Secchi Depth and light attenuation valid for noon (12Z) are included in the daily mean files/dataset. Additionally a third dataset with daily accumulated values of the netto primary production is available.  The product is produced by the biogeochemical model ERGOM (Neumann et al, 2021) one way coupled to a Baltic Sea set up of  the NEMO ocean model, which provides the Baltic Sea physical ocean forecast product (BALTICSEA_ANALYSISFORECAST_PHY_003_006). This biogeochemical product is provided at the models native grid with a resolution of 1 nautical mile in the horizontal, and with up to 56 vertical depth levels. The product covers the Baltic Sea including the transition area towards the North Sea (i.e. the Danish Belts, the Kattegat and Skagerrak).

**DOI (product):**   
https://doi.org/10.48670/moi-00009




<img src="https://mdl-metadata.s3.waw3-1.cloudferro.com/metadata/thumbnails/BALTICSEA_ANALYSISFORECAST_PHY_003_006.jpg" width="300" height="200"/>



# Baltic Sea Physics Analysis and Forecast


**Product ID:** BALTICSEA_ANALYSISFORECAST_PHY_003_006

**Description:** This Baltic Sea physical model product provides forecasts for the physical conditions in the Baltic Sea. The Baltic forecast is updated twice daily from a 00Z production proving a 10 days forecast and from a 12Z production providing a 6 days forecast. Several datasets are provided: One with hourly instantaneous values, one with daily mean values and one with monthly mean values, all containing these parameters: sea level variations, ice concentration and thickness at the surface, and temperature, salinity and horizontal and vertical velocities for the 3D field. Additionally a dataset with 15 minutes (instantaneous) surface values are provided for the sea level variation and the surface horizontal currents, as well as detided daily values. The product is produced by a Baltic Sea set up of the NEMOv4.2.1 ocean model. This product is provided at the models native grid with a resolution of 1 nautical mile in the horizontal, and with up to 56 vertical depth levels. The area covers the Baltic Sea including the transition area towards the North Sea (i.e. the Danish Belts, the Kattegat and Skagerrak). The ocean model is forced with Stokes drift data from the Baltic Sea Wave forecast product (BALTICSEA_ANALYSISFORECAST_WAV_003_010). Satellite SST, sea ice concentrations and in-situ T and S profiles are assimilated into the model's analysis field.

**DOI (product):**  
https://doi.org/10.48670/moi-00010




<img src="https://mdl-metadata.s3.waw3-1.cloudferro.com/metadata/thumbnails/BALTICSEA_ANALYSISFORECAST_WAV_003_010.jpg" width="300" height="200"/>



# Baltic Sea Wave Analysis and Forecast


**Product ID:** BALTICSEA_ANALYSISFORECAST_WAV_003_010

**Description:** This Baltic Sea wave model product provides forecasts for the wave conditions in the Baltic Sea. The Baltic forecast is updated twice daily from a 00Z production proving a 10 days forecast and from a 12Z production providing a 6 days forecast. Data are provided with hourly instantaneous data for significant wave height, wave period and wave direction for total sea, wind sea and swell, the Stokes drift, and two paramters for the maximum wave. The product is based on the wave model WAM cycle 4.7. The wave model is forced with surface currents, sea level anomaly and ice information from the Baltic Sea ocean forecast product (BALTICSEA_ANALYSISFORECAST_PHY_003_006). The product grid has a horizontal resolution of 1 nautical mile. The area covers the Baltic Sea including the transition area towards the North Sea (i.e. the Danish Belts, the Kattegat and Skagerrak).

**DOI (product):**   
https://doi.org/10.48670/moi-00011




<img src="https://mdl-metadata.s3.waw3-1.cloudferro.com/metadata/thumbnails/BALTICSEA_MULTIYEAR_BGC_003_012.jpg" width="300" height="200"/>



# Baltic Sea Biogeochemistry Reanalysis


**Product ID:** BALTICSEA_MULTIYEAR_BGC_003_012

**Description:** This Baltic Sea Biogeochemical Reanalysis product provides a biogeochemical reanalysis for the whole Baltic Sea area, inclusive the Transition Area to the North Sea, from January 1993 and up to minus maximum 1 year relative to real time. The product is produced by using the biogeochemical model ERGOM one-way online-coupled with the ice-ocean model system Nemo. All variables are avalable as daily, monthly and annual means and include nitrate, phosphate, ammonium, dissolved oxygen, ph, chlorophyll-a, secchi depth, surface partial co2 pressure and net primary production. The data are available at the native model resulution (1 nautical mile horizontal resolution, and 56 vertical layers).

**DOI (product):**

https://doi.org/10.48670/moi-00012




<img src="https://mdl-metadata.s3.waw3-1.cloudferro.com/metadata/thumbnails/BALTICSEA_MULTIYEAR_PHY_003_011.jpg" width="300" height="200"/>



# Baltic Sea Physics Reanalysis


**Product ID:** BALTICSEA_MULTIYEAR_PHY_003_011

**Description:** This Baltic Sea Physical Reanalysis product provides a reanalysis for the physical conditions for the whole Baltic Sea area, inclusive the Transition Area to the North Sea, from January 1993 and up to minus maximum 1 year relative to real time. The product is produced by using the  ice-ocean model system Nemo. All variables are avalable as daily, monthly and annual means and include sea level, ice concentration, ice thickness, salinity, temperature, horizonal velocities and the mixed layer depths. The data are available at the native model resulution (1 nautical mile horizontal resolution, and 56 vertical layers).

**DOI (product):**   
https://doi.org/10.48670/moi-00013




<img src="https://mdl-metadata.s3.waw3-1.cloudferro.com/metadata/thumbnails/BALTICSEA_MULTIYEAR_WAV_003_015.jpg" width="300" height="200"/>



# Baltic Sea Wave Hindcast


**Product ID:** BALTICSEA_MULTIYEAR_WAV_003_015

**Description:** **This product has been archived** 



This Baltic Sea wave model multiyear product provides a hindcast for the wave conditions in the Baltic Sea since 1/1 1980 and up to 0.5-1 year compared to real time.
This hindcast product consists of a dataset with hourly data for significant wave height, wave period and wave direction for total sea, wind sea and swell, the maximum waves, and also the Stokes drift. Another dataset contains hourly values for five air-sea flux parameters. Additionally a dataset with monthly climatology are provided for the significant wave height and the wave period. The product is based on the wave model WAM cycle 4.7, and surface forcing from ECMWF's ERA5 reanalysis products.  The product grid has a horizontal resolution of 1 nautical mile. The area covers the Baltic Sea including the transition area towards the North Sea (i.e. the Danish Belts, the Kattegat and Skagerrak). The product provides hourly instantaneously model data.

**DOI (product):**
https://doi.org/10.48670/moi-00014




<img src="https://documentation.marine.copernicus.eu/IMG/BALTIC_OMI_HEALTH_codt_volume.png" width="300" height="200"/>



# Baltic Sea Cod Reproductive Volume from Reanalysis


**Product ID:** BALTIC_OMI_HEALTH_codt_volume

**Description:** **DEFINITION**

The cod reproductive volume has been derived from regional reanalysis modelling results for the Baltic Sea BALTICSEA_MULTIYEAR_PHY_003_011 and BALTICSEA_MULTIYEAR_BGC_003_012. The volume has been calculated taking into account the three most important influencing abiotic factors of cod reproductive success: salinity > 11 g/kg, oxygen concentration > 2 ml/l and water temperature over 1.5°C (MacKenzie et al., 1996; Heikinheimo, 2008; Plikshs et al., 2015). The daily volumes are calculated as the volumes of the water with salinity > 11 g/kg, oxygen content > 2 ml/l and water temperature over 1.5°C in the Baltic Sea International Council for the Exploration of the Sea subdivisions of 25-28 (ICES, 2019).

**CONTEXT**

Cod (Gadus morhua) is a characteristic fish species in the Baltic Sea with major economic importance. Spawning stock biomasses of the Baltic cod have gone through a steep decline in the late 1980s (Bryhn et al., 2022). Water salinity and oxygen concentration affect cod stock through the survival of eggs (Westin and Nissling, 1991; Wieland et al., 1994). Major Baltic Inflows provide a suitable environment for cod reproduction by bringing saline oxygenated water to the deep basins of the Baltic Sea (BALTIC_OMI_WMHE_mbi_bottom_salinity_arkona_bornholm and BALTIC_OMI_WMHE_mbi_sto2tz_gotland). Increased cod reproductive volume has a positive effect on cod reproduction success, which should reflect an increase of stock size indicator 4–5 years after the Major Baltic Inflow (Raudsepp et al., 2019). Eastern Baltic cod reaches maturity around age 2–3, depending on the population density and environmental conditions. Low oxygen and salinity cause stress, which negatively affects cod recruitment, whereas sufficient conditions may bring about male cod maturation even at the age of 1.5 years (Cardinale and Modin, 1999; Karasiova et al., 2008). There are a number of environmental factors affecting cod populations (Bryhn et al., 2022).

**KEY FINDINGS**

Typically, the cod reproductive volume in the Baltic Sea oscillates between 200 and 400 km³. There have been two distinct periods of significant increase, with maximum values reaching over 1200 km³, corresponding to the aftermath of Major Baltic Inflows (BALTIC_OMI_WMHE_mbi_bottom_salinity_arkona_bornholm and BALTIC_OMI_WMHE_mbi_sto2tz_gotland) from 2003 to 2004 and from 2016 to 2017. Following a decline to the baseline of 200 km³ in 2018, there was a rise to 800 km³ in 2019. The cod reproductive volume hit a second peak of 800 km³ in 2022 and has since stabilized at 600 km³. However, Bryhn et al. (2022) report no observed increase in the spawning stock biomass of the eastern Baltic Sea cod.

**DOI (product):** 
https://doi.org/10.48670/moi-00196

**References:**

* Cardinale, M., Modin, J., 1999. Changes in size-at-maturity of Baltic cod (Gadus morhua) during a period of large variations in stock size and environmental conditions. Vol. 41 (3), 285-295. https://doi.org/10.1016/S0165-7836(99)00021-1
* Heikinheimo, O., 2008. Average salinity as an index for environmental forcing on cod recruitment in the Baltic Sea. Boreal Environ Res 13:457
* ICES, 2019. Baltic Sea Ecoregion – Fisheries overview, ICES Advice, DOI:10.17895/ices.advice.5566 Karasiova, E.M., Voss, R., Eero, M., 2008. Long-term dynamics in eastern Baltic cod spawning time: from small scale reversible changes to a recent drastic shift. ICES CM 2008/J:03
* MacKenzie, B., St. John, M., Wieland, K., 1996. Eastern Baltic cod: perspectives from existing data on processes affecting growth and survival of eggs and larvae. Mar Ecol Prog Ser Vol. 134: 265-281.
* Plikshs, M., Hinrichsen, H. H., Elferts, D., Sics, I., Kornilovs, G., Köster, F., 2015. Reproduction of Baltic cod, Gadus morhua (Actinopterygii: Gadiformes: Gadidae), in the Gotland Basin: Causes of annual variability. Acta Ichtyologica et Piscatoria, Vol. 45, No. 3, 2015, p. 247-258.
* Raudsepp, U., Legeais, J.-F., She, J., Maljutenko, I., Jandt, S., 2018. Baltic inflows. In: Copernicus Marine Service Ocean State Report, Issue 2, Journal of Operational Oceanography, 11:sup1, s106–s110, DOI: 10.1080/1755876X.2018.1489208
* Raudsepp, U., Maljutenko, I., Kõuts, M., 2019. Cod reproductive volume potential in the Baltic Sea. In: Copernicus Marine Service Ocean State Report, Issue 3, Journal of Operational Oceanography, 12:sup1, s26–s30; DOI: 10.1080/ 1755876X.2019.1633075
* Westin, L., Nissling, A., 1991. Effects of salinity on spermatozoa motility, percentage of fertilized eggs and egg development of Baltic cod Gadus morhua, and implications for cod stock fluctuations in the Baltic. Mar. Biol. 108, 5 – 9.
* Wieland, K., Waller, U., Schnack, D., 1994. Development of Baltic cod eggs at different levels of temperature and oxygen content. Dana 10, 163 – 177.
* Bryhn, A.C.., Bergek, S., Bergström,U., Casini, M., Dahlgren, E., Ek, C., Hjelm, J., Königson, S., Ljungberg, P., Lundström, K., Lunneryd, S.G., Ovegård, M., Sköld, M., Valentinsson, D., Vitale, F., Wennhage, H., 2022. Which factors can affect the productivity and dynamics of cod stocks in the Baltic Sea, Kattegat and Skagerrak? Ocean & Coastal Management, 223, 106154. https://doi.org/10.1016/j.ocecoaman.2022.106154





<img src="https://documentation.marine.copernicus.eu/IMG/BALTIC_OMI_OHC_area_averaged_anomalies.png" width="300" height="200"/>



# Baltic Sea Ocean Heat Content Anomaly (0-300m) from Reanalysis


**Product ID:** BALTIC_OMI_OHC_area_averaged_anomalies

**Description:** **DEFINITION**

The method for calculating the ocean heat content anomaly is based on the daily mean sea water potential temperature fields (Tp) derived from the Baltic Sea reanalysis product BALTICSEA_MULTIYEAR_PHY_003_011. The total heat content is determined using the following formula:

HC =  ρ * cp * ( Tp  +273.15).

Here, ρ and cp represent spatially varying sea water density and specific heat, respectively, which are computed based on potential temperature, salinity and pressure using the UNESCO 1983 polynomial developed by Fofonoff and Millard (1983). The vertical integral is computed using the static cell vertical thicknesses sourced from the reanalysis product BALTICSEA_MULTIYEAR_PHY_003_011 dataset cmems_mod_bal_phy_my_static, spanning from the sea surface to the 300 m depth. Spatial averaging is performed over the Baltic Sea spatial domain, defined as the region between 13° - 31° E and 53° - 66° N. To obtain the OHC annual anomaly time series in (J/m2), the mean heat content over the reference period of 1993-2014 was subtracted from the annual mean total heat content.
We evaluate the uncertainty from the mean annual error of the potential temperature compared to the observations from the Baltic Sea (Giorgetti et al., 2020). The shade corresponds to the RMSD of the annual mean heat content biases (± 35.3 MJ/m²) evaluated from the observed temperatures and corresponding model values. 
Linear trend (W/m2) has been estimated from the annual anomalies with the uncertainty of 1.96-times standard error.

**CONTEXT**

Ocean heat content is a key ocean climate change indicator. It accounts for the energy absorbed and stored by oceans. Ocean Heat Content in the upper 2,000 m of the World Ocean has increased with the rate of 0.35 ± 0.08 W/m2 in the period 1955–2019, while during the last decade of 2010–2019 the warming rate was 0.70 ± 0.07 W/m2 (Garcia-Soto et al., 2021). The high variability in the local climate of the Baltic Sea region is attributed to the interplay between a temperate marine zone and a subarctic continental zone. Therefore, the Ocean Heat Content of the Baltic Sea could exhibit strong interannual variability and the trend could be less pronounced than in the ocean.

**KEY FINDINGS**

The ocean heat content (OHC) of the Baltic Sea exhibits an increasing trend of 0.3±0.1 W/m², along with multi-year oscillations. This increase is less pronounced than the global OHC trend (Holland et al. 2019; von Schuckmann et al. 2019) and that of some other marginal seas (von Schuckmann et al. 2018; Lima et al. 2020). The relatively low trend values are attributed to the Baltic Sea's shallowness, which constrains heat accumulation in its waters. The most significant OHC anomaly was recorded in 2020, and following a decline from this peak, the anomaly has now stabilized at approximately 250 MJ/m².

**DOI (product):** 
https://doi.org/10.48670/mds-00322

**References:**

* Garcia-Soto C, Cheng L, Caesar L, Schmidtko S, Jewett EB, Cheripka A, Rigor I, Caballero A, Chiba S, Báez JC, Zielinski T and Abraham JP (2021) An Overview of Ocean Climate Change Indicators: Sea Surface Temperature, Ocean Heat Content, Ocean pH, Dissolved Oxygen Concentration, Arctic Sea Ice Extent, Thickness and Volume, Sea Level and Strength of the AMOC (Atlantic Meridional Overturning Circulation). Front. Mar. Sci. 8:642372. doi: 10.3389/fmars.2021.642372
* Fofonoff, P. and Millard, R.C. Jr UNESCO 1983. Algorithms for computation of fundamental properties of seawater. UNESCO Tech. Pap. in Mar. Sci., No. 44, 53 pp., p.39. http://unesdoc.unesco.org/images/0005/000598/059832eb.pdf
* Giorgetti, A., Lipizer, M., Molina Jack, M.E., Holdsworth, N., Jensen, H.M., Buga, L., Sarbu, G., Iona, A., Gatti, J., Larsen, M. and Fyrberg, L., 2020. Aggregated and Validated Datasets for the European Seas: The Contribution of EMODnet Chemistry. Frontiers in Marine Science, 7, p.583657.
* Holland E, von Schuckmann K, Monier M, Legeais J-F, Prado S, Sathyendranath S, Dupouy C. 2019. The use of Copernicus Marine Service products to describe the state of the tropical western Pacific Ocean around the islands: a case study. In: Copernicus Marine Service Ocean State Report, Issue 3. J Oper Oceanogr. 12(suppl. 1):s43–s48. doi:10.1080/1755876X.2019.1633075
* Lima L, Peneva E, Ciliberti S, Masina S, Lemieux B, Storto A, Chtirkova B. 2020. Ocean heat content in the Black Sea. In: Copernicus Marine Service Ocean State Report, Issue 4. J Oper Oceanogr. 13(suppl. 1):s41–s48. doi:10.1080/1755876X.2020.1785097.
* von Schuckmann K, Le Traon P-Y, Smith N, Pascual A, Djavidnia S, Gattuso J-P, Grégoire M, Nolan G. 2019. Copernicus Marine Service Ocean State report. J Oper Oceanogr. 12(suppl. 1):s1–s123. doi:10.1080/1755876X.2019.1633075.
* von Schuckmann K, Storto A, Simoncelli S, Raj RP, Samuelsen A, Collar A, Sotillo MG, Szerkely T, Mayer M, Peterson KA, et al. 2018. Ocean heat content. In: Copernicus Marine Service Ocean State Report, issue 2. J Oper Oceanogr. 11 (Suppl. 1):s1–s142. doi:10.1080/1755876X.2018.1489208.





<img src="https://documentation.marine.copernicus.eu/IMG/BALTIC_OMI_SI_extent.png" width="300" height="200"/>



# Baltic Sea Ice Extent from Observations Reprocessing


**Product ID:** BALTIC_OMI_SI_extent

**Description:** **DEFINITION**

Sea ice extent is defined as the area covered by sea ice, that is the area of the ocean having more than 15% sea ice concentration. Sea ice concentration is the fractional coverage of an ocean area covered with sea ice. Daily sea ice extent values are computed from the daily sea ice concentration maps. All sea ice covering the Baltic Sea is included, except for lake ice. The data used to produce the charts are Synthetic Aperture Radar images as well as in situ observations from ice breakers (Uiboupin et al., 2010). The annual course of the sea ice extent has been calculated as daily mean ice extent for each day-of-year over the period October 1992 – September 2014. Weekly smoothed time series of the sea ice extent have been calculated from daily values using a 7-day moving average filter.

**CONTEXT**

Sea ice coverage has a vital role in the annual course of physical and ecological conditions in the Baltic Sea. Moreover, it is an important parameter for safe winter navigation. The presence of sea ice cover sets special requirements for navigation, both for the construction of the ships and their behavior in ice, as in many cases, merchant ships need icebreaker assistance. Temporal trends of the sea ice extent could be a valuable indicator of the climate change signal in the Baltic Sea region. It has been estimated that a 1 °C increase in the average air temperature results in the retreat of ice-covered area in the Baltic Sea about 45,000 km2 (Granskog et al., 2006). Decrease in maximum ice extent may influence vertical stratification of the Baltic Sea (Hordoir and Meier, 2012) and affect the onset of the spring bloom (Eilola et al., 2013). In addition, statistical sea ice coverage information is crucial for planning of coastal and offshore construction. Therefore, the knowledge about ice conditions and their variability is required and monitored in Copernicus Marine Service.

**KEY FINDINGS**

Sea ice in the Baltic Sea exhibits a strong seasonal pattern. Typically, formation begins in October and can persist until June. The 2022/23 ice season saw a relatively low maximum ice extent in the Baltic Sea, peaking at around 65,000 km². Formation started in November and accelerated in the second week of December. The ice extent then remained fairly stable and below the climatological average until the end of January. From February to the second week of March, the extent of sea ice steadily grew to its maximum of 65,000 km², before gradually receding. The peak day for sea ice extent varies annually but generally oscillates between the end of February and the start of March (Singh et al., 2024). The past 30 years saw the highest sea ice extent at 260,000 km² in 2010/11. Despite a downward trend in sea ice extent in the Baltic Sea from 1993 to 2022, the linear trend does not show statistical significance.

**DOI (product):** 
https://doi.org/10.48670/moi-00200

**References:**

* Eilola K, Mårtensson S, Meier HEM, 2013. Modeling the impact of reduced sea ice cover in future climate on the Baltic Sea biogeochemistry. Geophysical Research Letters, 40, 149-154, doi:10.1029/2012GL054375
* Granskog M, Kaartokallio H, Kuosa H, Thomas DN, Vainio J, 2006. Sea ice in the Baltic Sea – A review. Estuarine, Coastal and Shelf Science, 70, 145–160. doi:10.1016/j.ecss.2006.06.001
* Hordoir R., Meier HEM, 2012. Effect of climate change on the thermal stratification of the Baltic Sea: a sensitivity experiment. Climate Dynamics, 38, 1703-1713, doi:10.1007/s00382-011-1036-y
* Uiboupin R, Axell L, Raudsepp U, Sipelgas L, 2010. Comparison of operational ice charts with satellite based ice concentration products in the Baltic Sea. 2010 IEEE/ OES US/EU Balt Int Symp Balt 2010, doi:10.1109/BALTIC.2010.5621649
* Vihma T, Haapala J, 2009. Geophysics of sea ice in the Baltic Sea: A review. Progress in Oceanography, 80, 129-148, doi:10.1016/j.pocean.2009.02.002





<img src="https://documentation.marine.copernicus.eu/IMG/BALTIC_OMI_SI_volume.png" width="300" height="200"/>



# Baltic Sea Ice Volume from Observations Reprocessing


**Product ID:** BALTIC_OMI_SI_volume

**Description:** **DEFINITION**

The sea ice volume is a product of sea ice concentration and sea ice thickness integrated over respective area. Sea ice concentration is the fractional coverage of an ocean area covered with sea ice. The Baltic Sea area having more than 15% of sea ice concentration is included into the sea ice volume analysis. Daily sea ice volume values are computed from the daily sea ice concentration and sea ice thickness maps. The data used to produce the charts are Synthetic Aperture Radar images as well as in situ observations from ice breakers (Uiboupin et al., 2010; https://www.smhi.se/data/oceanografi/havsis). The annual course of the sea ice volume has been calculated as daily mean ice volume for each day-of-year over the period October 1992 – September 2014. Weekly smoothed time series of the sea ice volume have been calculated from daily values using a 7-day moving average filter.

**CONTEXT**

Sea ice coverage has a vital role in the annual course of physical and ecological conditions in the Baltic Sea. Knowledge of the sea ice volume facilitates planning of icebreaking activity and operation of the icebreakers (Valdez Banda et al., 2015; Boström and Österman, 2017). A long-term monitoring of ice parameters is required for design and installation of offshore constructions in seasonally ice covered seas (Heinonen and Rissanen, 2017). A reduction of the sea ice volume in the Baltic Sea has a critical impact on the population of ringed seals (Harkonen et al., 2008). Ringed seals need stable ice conditions for about two months for breeding and moulting (Sundqvist et al., 2012). The sea ice is a habitat for diverse biological assemblages (Enberg et al., 2018).

**KEY FINDINGS**

In the Baltic Sea, the ice season may begin in October and last until June. The maximum sea ice volume typically peaks in March, but in 2023, it was observed in April. The 2022/23 ice season saw a low maximum sea ice volume of approximately 14 km³. From 1993 to 2023, the annual maximum ice volume ranged from 4 km³ in 2020 to 60 km³ in 1996. There is a statistically significant downward trend of -0.73 km³/year (p=0.01) in the Baltic Sea's maximum sea ice volume. Recent trends indicate a decrease in sea ice fraction and thickness across the Baltic Sea, particularly in the Bothnian Bay, as reported by Singh et al. (2024).

**DOI (product):** 
https://doi.org/10.48670/moi-00201

**References:**

* Boström M, Österman C, 2017, Improving operational safety during icebreaker operations, WMU Journal of Maritime Affairs, 16, 73-88, DOI: 10.1007/s13437-016-0105-9
* Enberg S, Majaneva M, Autio R, Blomster J, Rintala J-M, 2018, Phases of microalgal succession in sea ice and the water column in the baltic sea from autumn to spring. Marine Ecology Progress Series, 559, 19-34. DOI: 10.3354/meps12645
* Harkonen T, Jüssi M, Jüssi I, Verevkin M, Dmitrieva L, Helle E, Sagitov R, Harding KC, 2008, Seasonal Activity Budget of Adult Baltic Ringed Seals, PLoS ONE 3(4): e2006, DOI: 0.1371/journal.pone.0002006
* Heinonen J, Rissanen S, 2017, Coupled-crushing analysis of a sea ice - wind turbine interaction – feasibility study of FAST simulation software, Ships and Offshore Structures, 12, 1056-1063. DOI: 10.1080/17445302.2017.1308782
* Sundqvist L, Harkonen T, Svensson CJ, Harding KC, 2012, Linking Climate Trends to Population Dynamics in the Baltic Ringed Seal: Impacts of Historical and Future Winter Temperatures, AMBIO, 41: 865, DOI: 10.1007/s13280-012-0334-x
* Uiboupin R, Axell L, Raudsepp U, Sipelgas L, 2010, Comparison of operational ice charts with satellite based ice concentration products in the Baltic Sea. 2010 IEEE/ OES US/EU Balt Int Symp Balt 2010, DOI: 10.1109/BALTIC.2010.5621649
* Valdez Banda OA, Goerlandt F, Montewka J, Kujala P, 2015, A risk analysis of winter navigation in Finnish sea areas, Accident Analysis & Prevention, 79, 100–116, DOI: 10.1016/j.aap.2015.03.024





<img src="https://documentation.marine.copernicus.eu/IMG/BALTIC_OMI_TEMPSAL_Stz_trend.png" width="300" height="200"/>



# Baltic Sea Subsurface Salinity trend from Reanalysis


**Product ID:** BALTIC_OMI_TEMPSAL_Stz_trend

**Description:** **DEFINITION**

The subsurface salinity trends have been derived from regional reanalysis and forecast modelling results of the Copernicus Marine Service BAL MFC group for the Baltic Sea (product reference BALTICSEA_MULTIYEAR_PHY_003_011). The salinity trend has been obtained through a linear fit for each time series of horizontally averaged (13 °E - 31 °E and 53 °N - 66 °N; excluding the Skagerrak strait) annual salinity and at each depth level.

**CONTEXT**

The Baltic Sea is a brackish semi-enclosed sea in North-Eastern Europe. The surface salinity varies horizontally from ~10 near the Danish Straits down to ~2 at the northernmost and easternmost sub-basins of the Baltic Sea. The halocline, a vertical layer with rapid changes of salinity with depth that separates the well-mixed surface layer from the weakly stratified layer below, is located at the depth range of 60-80 metres (Matthäus, 1984). The bottom layer salinity below the halocline depth varies from 15 in the south down to 3 in the northern Baltic Sea (Väli et al., 2013). The long-term salinity is determined by net precipitation and river discharge as well as saline water inflows from the North Sea (Lehmann et al., 2022). Long-term salinity decrease may reduce the occurrence and biomass of the Fucus vesiculosus - Idotea balthica association/symbiotic aggregations (Kotta et al., 2019). Changes in salinity and oxygen content affect the survival of the Baltic cod eggs (Raudsepp et al, 2019; von Dewitz et al., 2018).

**KEY FINDINGS**

The subsurface salinity from 1993 to 2023 exhibits distinct variations at different depths. In the surface layer up to 25 meters, which aligns with the average upper mixed layer depth in the Baltic Sea, there is no discernible trend. The salinity trend increases steadily from zero at a 25-meter depth to 0.04 per year at 70 meters. The most pronounced trend, 0.045 per year, is found within the extended halocline layer ranging from 70 to 150 meters. It is noteworthy that there is a slight reduction in the salinity trend to 0.04 per year between depths of 150 and 220 meters. Although this decrease is minor, it suggests that salt transport into the extended halocline layer is more pronounced than into the deeper layers. The Major Baltic Inflows are responsible for the significant salinity trend of 0.05 per year observed in the deepest layer of the Baltic Sea.   

**DOI (product):** 
https://doi.org/10.48670/moi-00207

**References:**

* von Dewitz B, Tamm S, HoÈflich K, Voss R, Hinrichsen H-H., 2018. Use of existing hydrographic infrastructure to forecast the environmental spawning conditions for Eastern Baltic cod, PLoS ONE 13(5): e0196477, doi:10.1371/journal.pone.0196477
* Kotta, J., Vanhatalo, J., Jänes, H., Orav-Kotta, H., Rugiu, L., Jormalainen, V., Bobsien, I., Viitasalo, M., Virtanen, E., Nyström Sandman, A., Isaeus, M., Leidenberger, S., Jonsson, P.R., Johannesson, K., 2019. Integrating experimental and distribution data to predict future species patterns. Scientific Reports, 9: 1821, doi:10.1038/s41598-018-38416-3
* Matthäus W, 1984, Climatic and seasonal variability of oceanological parameters in the Baltic Sea, Beitr. Meereskund, 51, 29–49.
* Sandrine Mulet, Bruno Buongiorno Nardelli, Simon Good, Andrea Pisano, Eric Greiner, Maeva Monier, Emmanuelle Autret, Lars Axell, Fredrik Boberg, Stefania Ciliberti, Marie Drévillon, Riccardo Droghei, Owen Embury, Jérome Gourrion, Jacob Høyer, Mélanie Juza, John Kennedy, Benedicte Lemieux-Dudon, Elisaveta Peneva, Rebecca Reid, Simona Simoncelli, Andrea Storto, Jonathan Tinker, Karina von Schuckmann and Sarah L. Wakelin. 2018. Ocean temperature and salinity. In: Copernicus Marine Service Ocean State Report, Issue 2, Journal of Operational Oceanography, 11:sup1, s5–s13, DOI:10.1080/1755876X.2018.1489208
* Raudsepp, U., Maljutenko, I., Kõuts, M., 2019. 2.7 Cod reproductive volume potential in the Baltic Sea. In: Copernicus Marine Service Ocean State Report, Issue 3
* Väli G, Meier HEM, Elken J, 2013, Simulated halocline variability in the baltic sea and its impact on hypoxia during 1961-2007, Journal of Geophysical Research: Oceans, 118(12), 6982–7000, DOI:10.1002/2013JC009192





<img src="https://documentation.marine.copernicus.eu/IMG/BALTIC_OMI_TEMPSAL_Ttz_trend.png" width="300" height="200"/>



# Baltic Sea Subsurface Temperature trend from Reanalysis


**Product ID:** BALTIC_OMI_TEMPSAL_Ttz_trend

**Description:** **DEFINITION**

The subsurface temperature trends have been derived from regional reanalysis results for the Baltic Sea (product references BALTICSEA_MULTIYEAR_PHY_003_011). Horizontal averaging has been done over the Baltic Sea domain (13 °E - 31 °E and 53 °N - 66 °N; excluding the Skagerrak strait). The temperature trend has been obtained through a linear fit for each time series of horizontally averaged annual temperature and at each depth level. 

**CONTEXT**

The Baltic Sea is a semi-enclosed sea in North-Eastern Europe. The temperature of the upper mixed layer of the Baltic Sea is characterised by a strong seasonal cycle driven by the annual course of solar radiation (Leppäranta and Myrberg, 2008). The maximum water temperatures in the upper layer are reached in July and August and the minimum during February, when the Baltic Sea becomes partially frozen (CMEMS OMI Baltic Sea Sea Ice Extent, CMEMS OMI Baltic Sea Sea Ice Volume). Seasonal thermocline, developing in the depth range of 10-30 m in spring, reaches its maximum strength in summer and is eroded in autumn. During autumn and winter the Baltic Sea is thermally mixed down to the permanent halocline in the depth range of 60-80 metres (Matthäus, 1984). The 20–50 m thick cold intermediate layer forms below the upper mixed layer in March and is observed until October within the 15-65 m depth range (Chubarenko and Stepanova, 2018; Liblik and Lips, 2011). The deep layers of the Baltic Sea are disconnected from the ventilated upper ocean layers, and temperature variations are predominantly driven by mixing processes and horizontal advection. A warming trend of the sea surface waters is positively correlated with the increasing trend of diffuse attenuation of light (Kd490) and satellite-detected chlorophyll concentration (Kahru et al., 2016). Temperature increase in the water column could accelerate oxygen consumption during organic matter oxidation (Savchuk, 2018).

**KEY FINDINGS**

Analysis of subsurface temperatures from 1993 to 2023 indicates that the Baltic Sea is experiencing warming across all depth intervals. The temperature trend in the upper mixed layer (0-25 m) is approximately 0.055 °C/year, decreasing to 0.045 °C/year within the seasonal thermocline layer. A peak temperature trend of 0.065 °C/year is observed at a depth of 70 m, aligning with the base of the cold intermediate layer. Beyond this depth, the trend stabilizes, closely matching the 0.065 °C/year value. At a 95% confidence level, it can be stated that the Baltic Sea's warming is consistent with depth, averaging around 0.06 °C/year. Notably, recent trends show a significant increase; for instance, Savchuk's 2018 measurements indicate an average temperature trend of 0.04 °C/year in the Baltic Proper's deep layers (>60m) from 1979 to 2016.

**DOI (product):** 
https://doi.org/10.48670/moi-00208

**References:**

* Chubarenko, I., Stepanova, N. 2018. Cold intermediate layer of the Baltic Sea: Hypothesis of the formation of its core. Progress in Oceanography, 167, 1-10, doi: 10.1016/j.pocean.2018.06.012
* Kahru, M., Elmgren, R., and Savchuk, O. P. 2016. Changing seasonality of the Baltic Sea. Biogeosciences 13, 1009–1018. doi: 10.5194/bg-13-1009-2016
* Leppäranta, M., Myrberg, K. 2008. Physical Oceanography of the Baltic Sea. Springer, Praxis Publishing, Chichester, UK, pp. 370
* Liblik, T., Lips, U. 2011. Characteristics and variability of the vertical thermohaline structure in the Gulf of Finland in summer. Boreal Environment Research, 16, 73-83.
* Matthäus W, 1984, Climatic and seasonal variability of oceanological parameters in the Baltic Sea, Beitr. Meereskund, 51, 29–49.
* Savchuk, .P. 2018. Large-Scale Nutrient Dynamics in the Baltic Sea, 1970–2016. Frontiers in Marine Science, 5:95, doi: 10.3389/fmars.2018.00095





<img src="https://documentation.marine.copernicus.eu/IMG/BALTIC_OMI_WMHE_mbi_bottom_salinity_arkona_bornholm.png" width="300" height="200"/>



# Baltic Sea Major Baltic Inflow: bottom salinity from Reanalysis


**Product ID:** BALTIC_OMI_WMHE_mbi_bottom_salinity_arkona_bornholm

**Description:** **DEFINITION**

Major Baltic Inflows bring large volumes of saline and oxygen-rich water into the bottom layers of the deep basins of the Baltic Sea- Bornholm basin, Gdansk basin and Gotland basin. The Major Baltic Inflows occur seldom, sometimes many years apart (Mohrholz, 2018). The Major Baltic Inflow OMI consists of the time series of the bottom layer salinity in the Arkona basin and in the Bornholm basin and the time-depth plot of temperature, salinity and dissolved oxygen concentration in the Gotland basin (BALTIC_OMI_WMHE_mbi_sto2tz_gotland). Bottom salinity increase in the Arkona basin is the first indication of the saline water inflow, but not necessarily Major Baltic Inflow. Abrupt increase of bottom salinity of 2-3 units in the more downstream Bornholm basin is a solid indicator that Major Baltic Inflow has occurred.
The subsurface temperature trends have been derived from regional reanalysis results for the Baltic 

**CONTEXT**

The Baltic Sea is a huge brackish water basin in Northern Europe whose salinity is controlled by its freshwater budget and by the water exchange with the North Sea (e.g. Neumann et al., 2017). The saline and oxygenated water inflows to the Baltic Sea through the Danish straits, especially the Major Baltic Inflows, occur only intermittently (e.g. Mohrholz, 2018). Long-lasting periods of oxygen depletion in the deep layers of the central Baltic Sea accompanied by a salinity decline and the overall weakening of vertical stratification are referred to as stagnation periods. Extensive stagnation periods occurred in the 1920s/1930s, in the 1950s/1960s and in the 1980s/beginning of 1990s Lehmann et al., 2022). Bottom salinity variations in the Arkona Basin represent water exchange between the Baltic Sea and Skagerrak-Kattegat area. The increasing salinity signal in that area does not indicate that a Major Baltic Inflow has occurred. The mean sea level of the Baltic Sea derived from satellite altimetry data can be used as a proxy for the detection of saline water inflows to the Baltic Sea from the North Sea (Raudsepp et al., 2018). The medium and strong inflow events increase oxygen concentration in the near-bottom layer of the Bornholm Basin while some medium size inflows have no impact on deep water salinity (Mohrholz, 2018). 

**KEY FINDINGS**

Time series data of bottom salinity variations in the Arkona Basin are instrumental for monitoring the sporadic nature of water inflow and outflow events. The bottom salinity in the Arkona Basin fluctuates between 11 and 25 g/kg. The highest recorded bottom salinity value is associated with the Major Baltic Inflow of 2014, while other significant salinity peaks align with the Major Baltic Inflows of 1993 and 2002. Low salinity episodes in the Arkona Basin mark the occasions of barotropic outflows of brackish water from the Baltic Sea. In the Bornholm Basin, the bottom salinity record indicates three Major Baltic Inflow events: the first in 1993, followed by 2002, and the most recent in 2014. Following the last Major Baltic Inflow, the bottom salinity in the Bornholm Basin rose to 20 g/kg. Over the subsequent nine years, it has declined to 16 g/kg. The winter of 2023/24 did not experience a Major Baltic Inflow.

**DOI (product):**
https://doi.org/10.48670/moi-00209

**References:**

* Lehmann, A., Myrberg, K., Post, P., Chubarenko, I., Dailidiene, I., Hinrichsen, H.-H., Hüssy, K., Liblik, T., Meier, H. E. M., Lips, U., Bukanova, T., 2022. Salinity dynamics of the Baltic Sea. Earth System Dynamics, 13(1), pp 373 - 392. doi:10.5194/esd-13-373-2022
* Mohrholz V, 2018, Major Baltic Inflow Statistics – Revised. Frontiers in Marine Science, 5:384, doi: 10.3389/fmars.2018.00384
* Neumann, T., Radtke, H., Seifert, T., 2017. On the importance of Major Baltic Inﬂows for oxygenation of the central Baltic Sea, J. Geophys. Res. Oceans, 122, 1090–1101, doi:10.1002/2016JC012525.
* Raudsepp, U., Legeais, J.-F., She, J., Maljutenko, I., Jandt, S., 2018. Baltic inflows. In: Copernicus Marine Service Ocean State Report, Issue 2, Journal of Operational Oceanography, 11:sup1, s13–s16, doi: 10.1080/1755876X.2018.1489208





<img src="https://documentation.marine.copernicus.eu/IMG/BALTIC_OMI_WMHE_mbi_sto2tz_gotland.png" width="300" height="200"/>



# Baltic Sea Major Baltic Inflow: time/depth evolution S,T,O2 from Observations Reprocessing


**Product ID:** BALTIC_OMI_WMHE_mbi_sto2tz_gotland

**Description:** "_DEFINITION_'

Major Baltic Inflows bring large volumes of saline and oxygen-rich water into the bottom layers of the deep basins of the central Baltic Sea, i.e. the Gotland Basin. These Major Baltic Inflows occur seldom, sometimes many years apart (Mohrholz, 2018). The Major Baltic Inflow OMI consists of the time series of the bottom layer salinity in the Arkona Basin and in the Bornholm Basin (BALTIC_OMI_WMHE_mbi_bottom_salinity_arkona_bornholm) and the time-depth plot of temperature, salinity and dissolved oxygen concentration in the Gotland Basin. Temperature, salinity and dissolved oxygen profiles in the Gotland Basin enable us to estimate the amount of the Major Baltic Inflow water that has reached central Baltic, the depth interval of which has been the most affected, and how much the oxygen conditions have been improved. 

**CONTEXT**

The Baltic Sea is a huge brackish water basin in Northern Europe whose salinity is controlled by its freshwater budget and by the water exchange with the North Sea (e.g. Neumann et al., 2017). This implies that fresher water lies on top of water with higher salinity. The saline water inflows to the Baltic Sea through the Danish Straits, especially the Major Baltic Inflows, shape hydrophysical conditions in the Gotland Basin of the central Baltic Sea, which in turn have a substantial influence on marine ecology on different trophic levels (Bergen et al., 2018; Raudsepp et al.,2019). In the absence of the Major Baltic Inflows, oxygen in the deeper layers of the Gotland Basin is depleted and replaced by hydrogen sulphide (e.g., Savchuk, 2018). As the Baltic Sea is connected to the North Sea only through very narrow and shallow channels in the Danish Straits, inflows of high salinity and oxygenated water into the Baltic occur only intermittently (e.g., Mohrholz, 2018). Long-lasting periods of oxygen depletion in the deep layers of the central Baltic Sea accompanied by a salinity decline and overall weakening of the vertical stratification are referred to as stagnation periods. Extensive stagnation periods occurred in the 1920s/1930s, in the 1950s/1960s and in the 1980s/beginning of 1990s (Lehmann et al., 2022).

**KEY FINDINGS**

The Major Baltic Inflows of 1993, 2002, and 2014 (BALTIC_OMI_WMHE_mbi_bottom_salinity_arkona_bornholm) present a distinct signal in the Gotland Basin, influencing water salinity, temperature, and dissolved oxygen up to a depth of 100 meters. Following each event, deep layer salinity in the Gotland Basin increases, reaching peak bottom salinities approximately 1.5 years later, with elevated salinity levels persisting for about three years. Post-2017, salinity below 150 meters has declined, while the halocline has risen, suggesting saline water movement to the Gotland Basin's intermediate layers. Typically, temperatures fall immediately after a Major Baltic Inflow, indicating the descent of cold water from nearby upstream regions to the Gotland Deep's bottom. From 1993 to 1997, deep water temperatures remained relatively low (below 6 °C). Since 1998, these waters have warmed, with even moderate inflows in 1997/98, 2006/07, and 2018/19 introducing warmer water to the Gotland Basin's bottom layer. From 2019 onwards, water warmer than 7 °C has filled the layer beneath 100 meters depth. The water temperature below the halocline has risen by approximately 2 °C since 1993, and the cold intermediate layer's temperature has also increased from 1993 to 2023. Oxygen levels begin to drop sharply after the temporary reoxygenation of the bottom waters. The decline in 2014 was attributed to a shortage of smaller inflows that could bring oxygen-rich water to the Gotland Basin (Neumann et al., 2017) and an increase in biological oxygen demand (Savchuk, 2018; Meier et al., 2018). Additionally, warmer water has accelerated oxygen consumption in the deep layer, leading to increased anoxia. By 2023, oxygen was completely depleted below the depth of 75 metres.

**DOI (product):** 
https://doi.org/10.48670/moi-00210

**References:**

* Lehmann, A., Myrberg, K., Post, P., Chubarenko, I., Dailidiene, I., Hinrichsen, H.-H., Hüssy, K., Liblik, T., Meier, H. E. M., Lips, U., Bukanova, T., 2022. Salinity dynamics of the Baltic Sea. Earth System Dynamics, 13(1), pp 373 - 392. doi:10.5194/esd-13-373-2022
* Bergen, B., Naumann, M., Herlemann, D.P.R., Gräwe, U., Labrenz, M., Jürgens, K., 2018. Impact of a Major inflow event on the composition and distribution of bacterioplankton communities in the Baltic Sea. Frontiers in Marine Science, 5:383, doi: 10.3389/fmars.2018.00383
* Meier, H.E.M., Väli, G., Naumann, M., Eilola, K., Frauen, C., 2018. Recently Accelerated Oxygen Consumption Rates Amplify Deoxygenation in the Baltic Sea. , J. Geophys. Res. Oceans, doi:10.1029/2017JC013686|
* Mohrholz, V., 2018. Major Baltic Inflow Statistics – Revised. Frontiers in Marine Science, 5:384, DOI: 10.3389/fmars.2018.00384
* Neumann, T., Radtke, H., Seifert, T., 2017. On the importance of Major Baltic Inﬂows for oxygenation of the central Baltic Sea, J. Geophys. Res. Oceans, 122, 1090–1101, doi:10.1002/2016JC012525.
* Raudsepp, U., Maljutenko, I., Kõuts, M., 2019. Cod reproductive volume potential in the Baltic Sea. In: Copernicus Marine Service Ocean State Report, Issue 3
* Savchuk, P. 2018. Large-Scale Nutrient Dynamics in the Baltic Sea, 1970–2016. Frontiers in Marine Science, 5:95, doi: 10.3389/fmars.2018.00095





<img src="https://mdl-metadata.s3.waw3-1.cloudferro.com/metadata/thumbnails/BLKSEA_ANALYSISFORECAST_BGC_007_010.jpg" width="300" height="200"/>



# Black Sea Biogeochemistry Analysis and Forecast


**Product ID:** BLKSEA_ANALYSISFORECAST_BGC_007_010

**Description:** BLKSEA_ANALYSISFORECAST_BGC_007_010 is the nominal product of the Black Sea Biogeochemistry NRT system and is generated by the NEMO 4.2-BAMHBI modelling system. Biogeochemical Model for Hypoxic and Benthic Influenced areas (BAMHBI) is an innovative biogeochemical model with a 28-variable pelagic component (including the carbonate system) and a 6-variable benthic component ; it explicitely represents processes in the anoxic layer.
The product provides analysis and forecast for 3D concentration of chlorophyll, nutrients (nitrate and phosphate), dissolved oxygen, zooplankton and phytoplankton carbon biomass, oxygen-demand-units, net primary production, pH, dissolved inorganic carbon, total alkalinity, and for 2D fields of bottom oxygen concentration (for the North-Western shelf), surface partial pressure of CO2 and surface flux of CO2. These variables are computed on a grid with ~3km x 59-levels resolution, and are provided as daily and monthly means.

**DOI (product):**  
https://doi.org/10.48670/mds-00354

**References:**

* Grégoire, M., Vandenbulcke, L. and Capet, A. (2020) “Black Sea Biogeochemical Analysis and Forecast (CMEMS Near-Real Time BLACKSEA Biogeochemistry).” Copernicus Monitoring Environment Marine Service (CMEMS).





<img src="https://mdl-metadata.s3.waw3-1.cloudferro.com/metadata/thumbnails/BLKSEA_ANALYSISFORECAST_PHY_007_001.jpg" width="300" height="200"/>



# Black Sea Physics Analysis and Forecast


**Product ID:** BLKSEA_ANALYSISFORECAST_PHY_007_001

**Description:** The BLKSEA_ANALYSISFORECAST_PHY_007_001 is produced with a hydrodynamic model implemented over the whole Black Sea basin, including the Azov Sea, the Bosporus Strait and a portion of the Marmara Sea for the optimal interface with the Mediterranean Sea through lateral open boundary conditions. The model horizontal grid resolution is 1/40° in zonal and 1/40° in meridional direction (ca. 3 km) and has 121 unevenly spaced vertical levels. The product provides analysis and forecast for 3D potential temperature, salinity, horizontal and vertical currents. Together with the 2D variables sea surface height, bottom potential temperature and mixed layer thickness.

**DOI (Product)**: 
https://doi.org/10.48670/mds-00355

**References:**

* Jansen, E., Martins, D., Causio, S., Maslo, A., Ilicak, M., Cretì, S., Lecci, R., Lima, L., Sözer, A., Trotta, F., Ciliberti, S. A. (2024). Black Sea Physical Analysis and Forecast (Copernicus Marine Service BLK-PHY, EAS7 system) (Version 1) [Data set]. Copernicus Marine Service.





<img src="https://mdl-metadata.s3.waw3-1.cloudferro.com/metadata/thumbnails/BLKSEA_ANALYSISFORECAST_WAV_007_003.jpg" width="300" height="200"/>



# Black Sea Waves Analysis and Forecast


**Product ID:** BLKSEA_ANALYSISFORECAST_WAV_007_003

**Description:** The wave analysis and forecasts for the Black Sea are produced with the third generation spectral wave model WAM Cycle 6. The hindcast and ten days forecast are produced twice a day on the HPC at Helmholtz-Zentrum Hereon. The shallow water Black Sea version is implemented on a spherical grid with a spatial resolution of about 2.5 km (1/40° x 1/40°) with 24 directional and 30 frequency bins. The number of active wave model grid points is 81,531. The model takes into account depth refraction, wave breaking, and assimilation of satellite wave and wind data. The system provides a hindcast and ten days forecast with one-hourly output twice a day. The atmospheric forcing is taken from ECMWF analyses and forecast data. Additionally, WAM is forced by surface currents and sea surface height from BLKSEA_ANALYSISFORECAST_PHY_007_001. Monthly statistics are provided operationally on the Product Quality Dashboard following the CMEMS metrics definitions.

**Citation**: 
Staneva, J., Ricker, M., & Behrens, A. (2022). Black Sea Waves Analysis and Forecast (CMEMS BS-Waves, EAS5 system) (Version 1) [Data set]. Copernicus Monitoring Environment Marine Service (CMEMS).

**DOI (Product)**: 
https://doi.org/10.48670/mds-00360

**References:**

* Staneva, J., Ricker, M., & Behrens, A. (2022). Black Sea Waves Analysis and Forecast (CMEMS BS-Waves, EAS5 system) (Version 1) [Data set]. Copernicus Monitoring Environment Marine Service (CMEMS). https://doi.org/10.25423/CMCC/BLKSEA_ANALYSISFORECAST_WAV_007_003_EAS5
* Ricker, M., Behrens, A., & Staneva, J. (2024). The operational CMEMS wind wave forecasting system of the Black Sea. Journal of Operational Oceanography, 1–22. https://doi.org/10.1080/1755876X.2024.2364974





<img src="https://mdl-metadata.s3.waw3-1.cloudferro.com/metadata/thumbnails/BLKSEA_MULTIYEAR_BGC_007_005.jpg" width="300" height="200"/>



# Black Sea Biogeochemistry Reanalysis


**Product ID:** BLKSEA_MULTIYEAR_BGC_007_005

**Description:** The biogeochemical reanalysis for the Black Sea is produced by the MAST/ULiege Production Unit by means of the BAMHBI biogeochemical model. The workflow runs on the CECI hpc infrastructure (Wallonia, Belgium).

**DOI (product)**:
https://doi.org/10.25423/CMCC/BLKSEA_MULTIYEAR_BGC_007_005_BAMHBI

**References:**

* Grégoire, M., Vandenbulcke, L., & Capet, A. (2020). Black Sea Biogeochemical Reanalysis (CMEMS BS-Biogeochemistry) (Version 1) [Data set]. Copernicus Monitoring Environment Marine Service (CMEMS). https://doi.org/10.25423/CMCC/BLKSEA_REANALYSIS_BIO_007_005_BAMHBI





<img src="https://mdl-metadata.s3.waw3-1.cloudferro.com/metadata/thumbnails/BLKSEA_MULTIYEAR_PHY_007_004.jpg" width="300" height="200"/>



# Black Sea Physics Reanalysis


**Product ID:** BLKSEA_MULTIYEAR_PHY_007_004

**Description:** The BLKSEA_MULTIYEAR_PHY_007_004 product provides ocean fields for the Black Sea basin starting from 01/01/1993. The hydrodynamic core is based on the NEMOv4.0 general circulation ocean model, implemented in the BS domain with horizontal resolution of 1/40º and 121 vertical levels. NEMO is forced by atmospheric fluxes computed from a bulk formulation applied to ECMWF ERA5 atmospheric fields at the resolution of 1/4º in space and 1-h in time. A heat flux correction through sea surface temperature (SST) relaxation is employed using the ESA-CCI SST-L4 product. This version has an open lateral boundary, a new model characteristic that allows a better inflow/outflow representation across the Bosphorus Strait. The model is online coupled to OceanVar assimilation scheme to assimilate sea level anomaly (SLA) along-track observations from Copernicus and available in situ vertical profiles of temperature and salinity from both SeaDataNet and Copernicus datasets. Upgrades on data assimilation include an improved background error covariance matrix and an observation-based mean dynamic topography for the SLA assimilation.

**DOI (Product)**: 
https://doi.org/10.48670/mds-00356




<img src="https://mdl-metadata.s3.waw3-1.cloudferro.com/metadata/thumbnails/BLKSEA_MULTIYEAR_WAV_007_006.jpg" width="300" height="200"/>



# Black Sea Waves Reanalysis


**Product ID:** BLKSEA_MULTIYEAR_WAV_007_006

**Description:** The wave reanalysis for the Black Sea is produced with the third generation spectral wave model WAM Cycle 6. The reanalysis is produced on the HPC at Helmholtz-Zentrum Hereon. The shallow water Black Sea version is implemented on a spherical grid with a spatial resolution of about 2.5 km (1/40° x 1/40°) with 24 directional and 30 frequency bins. The number of active wave model grid points is 74,518. The model takes into account wave breaking and assimilation of Jason satellite wave and wind data. The system provides one-hourly output and the atmospheric forcing is taken from ECMWF ERA5 data. In addition, the product comprises a monthly climatology dataset based on significant wave height and Tm02 wave period as well as an air-sea-flux dataset.

**Citation**: 
Staneva, J., Ricker, M., & Behrens, A. (2022). Black Sea Waves Reanalysis (CMEMS BS-Waves, EAS4 system) (Version 1) [Data set]. Copernicus Monitoring Environment Marine Service.

**DOI (Product)**: 
https://doi.org/10.48670/mds-00357




<img src="https://documentation.marine.copernicus.eu/IMG/BLKSEA_OMI_HEALTH_oxygen_trend.png" width="300" height="200"/>



# Black Sea Oxygen Trend from Observations Reprocessing


**Product ID:** BLKSEA_OMI_HEALTH_oxygen_trend

**Description:** **DEFINITION**

The oxygenation status of the Black Sea open basin is described by three complementary indicators, derived from vertical profiles and spatially averaged over the Black Sea open basin (depth > 50m). (1) The oxygen penetration depth is the depth at which  [O2] < 20µM, expressed in [m]. (2) The oxygen penetration density is the potential density anomaly at the oxygen penetration depth [kg/m³]. (3) The oxygen inventory is the vertically integrated oxygen content [mol O2/m²]. The 20µM threshold was chosen to minimize the indicator sensitivity to sensor’s precision. Those three metrics are complementary: Oxygen penetration depth is more easily understood, but present more spatial variability. Oxygen penetration density helps in dissociating biogeochemical processes from shifts in the physical structure. Although less intuitive, the oxygen inventory is a more integrative diagnostic and its definition is more easily transposed to other areas.

**CONTEXT**

The Black Sea is permanently stratified, due to the contrast in density between large riverine and Mediterranean inflows. This stratification restrains the ventilation of intermediate and deep waters and confines, within a restricted surface layer, the waters that are oxygenated by photosynthesis and exchanges with the atmosphere. The vertical extent of the oxic layer determines the volume of habitat available for pelagic populations (Ostrovskii and Zatsepin 2011, Sakınan and Gücü 2017) and present spatial and temporal variations (Murray et al. 1989; Tugrul et al. 1992; Konovalov and Murray 2001). At long and mid-term, these variations can be monitored with three metrics (Capet et al. 2016), derived from the vertical profiles that can obtained from traditional ship casts or autonomous Argo profilers (Stanev et al., 2013). A large source of uncertainty associated with the spatial and temporal average of those metrics stems from the small number of Argo floats, scarcely adequate to sample the known spatial variability of those metrics.

**CMEMS KEY FINDINGS**

During the past 60 years, the vertical extent of the Black Sea oxygenated layer has narrowed from 140m to 90m (Capet et al. 2016). The Argo profilers active for 2016 suggested an ongoing deoxygenation trend and indicated an average oxygen penetration depth of 72m at the end of 2016, the lowest value recorded during the past 60 years. The oxygenation of subsurface water is closely related to the intensity of cold water formation, an annual ventilation processes which has been recently limited by warmer-than-usual winter air temperature (Capet et al. 2020). In 2017, 2018 and 2020, cold waters formation resulted in a partial reoxygenation of the intermediate layer. Yet, such ventilation has been lacking in winter 2020-2021, and the updated 2021 indicators reveals the lowest oxygen inventory ever reported in this OMI time series. This results in significant detrimental trends now depicted also over the Argo period (2012-2021).

**DOI (product):** 
https://doi.org/10.48670/moi-00213

**References:**

* Capet, A., Vandenbulcke, L., & Grégoire, M. (2020). A new intermittent regime of convective ventilation threatens the Black Sea oxygenation status. Biogeosciences , 17(24), 6507–6525.
* Capet A, Stanev E, Beckers JM, Murray J, Grégoire M. (2016). Decline of the Black Sea oxygen inventory. Biogeosciences. 13:1287-1297.
* Capet Arthur, Vandenbulcke Luc, Veselka Marinova, Grégoire Marilaure. (2018). Decline of the Black Sea oxygen inventory. In: Copernicus Marine Service Ocean State Report, Issue 2, Journal of Operational Oceanography, 11:sup1, s13–s16, DOI: 10.1080/1755876X.2018.1489208
* Konovalov S, Murray JW. (2001). Variations in the chemistry of the Black Sea on a time scale of decades (1960–1995). J Marine Syst. 31: 217–243.
* Murray J, Jannasch H, Honjo S, Anderson R, Reeburgh W, Top Z, Friederich G, Codispoti L, Izdar E. (1989). Unexpected changes in the oxic/anoxic interface in the Black Sea. Nature. 338: 411–413.
* Ostrovskii A and Zatsepin A. (2011). Short-term hydrophysical and biological variability over the northeastern Black Sea continental slope as inferred from multiparametric tethered profiler surveys, Ocean Dynam., 61, 797–806, 2011.
* Özsoy E and Ünlüata Ü. (1997). Oceanography of the Black Sea: a review of some recent results. Earth-Science Reviews. 42(4):231-72.
* Sakınan S, Gücü AC. (2017). Spatial distribution of the Black Sea copepod, Calanus euxinus, estimated using multi-frequency acoustic backscatter. ICES J Mar Sci. 74(3):832-846. doi:10.1093/icesjms/fsw183
* Stanev E, He Y, Grayek S, Boetius A. (2013). Oxygen dynamics in the Black Sea as seen by Argo profiling floats. Geophys Res Lett. 40(12), 3085-3090.
* Tugrul S, Basturk O, Saydam C, Yilmaz A. (1992). Changes in the hydrochemistry of the Black Sea inferred from water density profiles. Nature. 359: 137-139.
* von Schuckmann, K. et al. Copernicus Marine Service Ocean State Report. Journal of Operational Oceanography 11, S1–S142 (2018).





<img src="https://documentation.marine.copernicus.eu/IMG/BLKSEA_OMI_SEASTATE_extreme_var_swh_mean_and_anomaly.png" width="300" height="200"/>



# Black Sea Significant Wave Height extreme from Reanalysis


**Product ID:** BLKSEA_OMI_SEASTATE_extreme_var_swh_mean_and_anomaly

**Description:** **DEFINITION**

The CMEMS BLKSEA_OMI_seastate_extreme_var_swh_mean_and_anomaly OMI indicator is based on the computation of the annual 99th percentile of Significant Wave Height (SWH) from model data. Two different CMEMS products are used to compute the indicator: The Iberia-Biscay-Ireland Multi Year Product (BLKSEA_MULTIYEAR_WAV_007_006) and the Analysis product (BLKSEA_ANALYSISFORECAST_WAV_007_003).
Two parameters have been considered for this OMI:
* Map of the 99th mean percentile: It is obtained from the Multy Year Product, the annual 99th percentile is computed for each year of the product. The percentiles are temporally averaged in the whole period (1979-2019).
* Anomaly of the 99th percentile in 2020: The 99th percentile of the year 2020 is computed from the Analysis product. The anomaly is obtained by subtracting the mean percentile to the percentile in 2020.
This indicator is aimed at monitoring the extremes of annual significant wave height and evaluate the spatio-temporal variability. The use of percentiles instead of annual maxima, makes this extremes study less affected by individual data. This approach was first successfully applied to sea level variable (Pérez Gómez et al., 2016) and then extended to other essential variables, such as sea surface temperature and significant wave height (Pérez Gómez et al 2018 and Álvarez-Fanjul et al., 2019). Further details and in-depth scientific evaluation can be found in the CMEMS Ocean State report (Álvarez- Fanjul et al., 2019).

**CONTEXT**

The sea state and its related spatio-temporal variability affect maritime activities and the physical connectivity between offshore waters and coastal ecosystems, including biodiversity of marine protected areas (González-Marco et al., 2008; Savina et al., 2003; Hewitt, 2003). Over the last decades, significant attention has been devoted to extreme wave height events since their destructive effects in both the shoreline environment and human infrastructures have prompted a wide range of adaptation strategies to deal with natural hazards in coastal areas (Hansom et al., 2015, IPCC, 2019). Complementarily, there is also an emerging question about the role of anthropogenic global climate change on present and future extreme wave conditions (IPCC, 2021).
Significant Wave Height mean 99th percentile in the Black Sea region shows west-eastern  dependence demonstrating that the highest values of the average annual 99th percentiles are in the areas where high winds and long fetch are simultaneously present. The largest values of the mean 99th percentile in the Black Sea in the southewestern Black Sea are around 3.5 m, while in the eastern part of the basin are around 2.5 m (Staneva et al., 2019a and 2019b).

**CMEMS KEY FINDINGS**

Significant Wave Height mean 99th percentile in the Black Sea region shows west-eastern  dependence with largest values in the southwestern Black Sea, with values as high as 3.5 m, while the 99th percentile values in the eastern part of the basin are around 2.5 m.  The Black Sea, the 99th mean percentile for 2002-2019 shows a similar pattern demonstrating that the highest values of the mean annual 99th percentile are in the western Black Sea. This pattern is consistent with the previous studies, e.g. of (Akpınar and Kömürcü, 2012; and Akpinar et al., 2016).
The anomaly of the 99th percentile in 2020 is mostly negative with values down to ~-45 cm. The highest negative anomalies for 2020 are observed in the southeastern area where the multi-year mean 99th percentile is the lowest. The highest positive anomalies of the 99th percentile in 2020 are located in the southwestern Black Sea and along the eastern coast. The map of anomalies for 2020, presenting alternate bands of positive and negative values depending on latitude, is consistent with the yearly west-east displacement of the tracks of the largest storms. 

Note: The key findings will be updated annually in November, in line with OMI evolutions.

**DOI (product):**
https://doi.org/10.48670/moi-00214

**References:**

* Akpınar, A.; Kömürcü, M.˙I. Wave energy potential along the south-east coasts of the Black Sea. Energy 2012, 42, 289–302.
* Akpınar, A., Bingölbali, B., Van Vledder, G., 2016. Wind and wave characteristics in the Black Sea based on the SWAN wave model forced with the CFSR winds. Ocean Eng. 126, 276—298, http://dx. doi.org/10.1016/j.oceaneng.2016.09.026.
* Álvarez Fanjul E, Pascual Collar A, Pérez Gómez B, De Alfonso M, García Sotillo M, Staneva J, Clementi E, Grandi A, Zacharioudaki A, Korres G, Ravdas M, Renshaw R, Tinker J, Raudsepp U, Lagemaa P, Maljutenko I, Geyer G, Müller M, Çağlar Yumruktepe V. Sea level, sea surface temperature and SWH extreme percentiles: combined analysis from model results and in situ observations, Section 2.7, p:31. In: Schuckmann K, Le Traon P-Y, Smith N, Pascual A, Djavidnia S, Gattuso J-P, Grégoire M, Nolan G, et al. 2019. Copernicus Marine Service Ocean State Report, Issue 3, Journal of Operational Oceanography, 12:sup1, S1-S123, DOI: 10.1080/1755876X.2019.1633075
* Bauer E. 2001. Interannual changes of the ocean wave variability in the North Atlantic and in the North Sea, Climate Research, 18, 63–69.
* González-Marco D, Sierra J P, Ybarra O F, Sánchez-Arcilla A. 2008. Implications of long waves in harbor management: The Gijón port case study. Ocean & Coastal Management, 51, 180-201. doi:10.1016/j.ocecoaman.2007.04.001.
* Hanson et al., 2015. Extreme Waves: Causes, Characteristics and Impact on Coastal Environments and Society January 2015 In book: Coastal and Marine Hazards, Risks, and Disasters Edition: Hazards and Disasters Series, Elsevier Major Reference Works Chapter: Chapter 11: Extreme Waves: Causes, Characteristics and Impact on Coastal Environments and Society. Publisher: Elsevier Editors: Ellis, J and Sherman, D. J.
* Hewit J E, Cummings V J, Elis J I, Funnell G, Norkko A, Talley T S, Thrush S.F. 2003. The role of waves in the colonisation of terrestrial sediments deposited in the marine environment. Journal of Experimental marine Biology and Ecology, 290, 19-47, doi:10.1016/S0022-0981(03)00051-0.
* IPCC, 2019: IPCC Special Report on the Ocean and Cryosphere in a Changing Climate [H.-O. Pörtner, D.C. Roberts, V. Masson-Delmotte, P. Zhai, M. Tignor, E. Poloczanska, K. Mintenbeck, A. Alegría, M. Nicolai, A. Okem, J. Petzold, B. Rama, N.M. Weyer (eds.)]. In press.
* IPCC, 2021: Climate Change 2021: The Physical Science Basis. Contribution of Working Group I to the Sixth Assessment Report of the Intergovernmental Panel on Climate Change [Masson-Delmotte, V., P. Zhai, A. Pirani, S.L. Connors, C. Péan, S. Berger, N. Caud, Y. Chen, L. Goldfarb, M.I. Gomis, M. Huang, K. Leitzell, E. Lonnoy, J.B.R. Matthews, T.K. Maycock, T. Waterfield, O. Yelekçi, R. Yu, and B. Zhou (eds.)]. Cambridge University Press. In Press.
* Pérez-Gómez B, Álvarez-Fanjul E, She J, Pérez-González I, Manzano F. 2016. Extreme sea level events, Section 4.4, p:300. In: Von Schuckmann K, Le Traon PY, Alvarez-Fanjul E, Axell L, Balmaseda M, Breivik LA, Brewin RJW, Bricaud C, Drevillon M, Drillet Y, Dubois C , Embury O, Etienne H, García-Sotillo M, Garric G, Gasparin F, Gutknecht E, Guinehut S, Hernandez F, Juza M, Karlson B, Korres G, Legeais JF, Levier B, Lien VS, Morrow R, Notarstefano G, Parent L, Pascual A, Pérez-Gómez B, Perruche C, Pinardi N, Pisano A, Poulain PM , Pujol IM, Raj RP, Raudsepp U, Roquet H, Samuelsen A, Sathyendranath S, She J, Simoncelli S, Solidoro C, Tinker J, Tintoré J, Viktorsson L, Ablain M, Almroth-Rosell E, Bonaduce A, Clementi E, Cossarini G, Dagneaux Q, Desportes C, Dye S, Fratianni C, Good S, Greiner E, Gourrion J, Hamon M, Holt J, Hyder P, Kennedy J, Manzano-Muñoz F, Melet A, Meyssignac B, Mulet S, Nardelli BB, O’Dea E, Olason E, Paulmier A, Pérez-González I, Reid R, Racault MF, Raitsos DE, Ramos A, Sykes P, Szekely T, Verbrugge N. 2016. The Copernicus Marine Environment Monitoring Service Ocean State Report, Journal of Operational Oceanography. 9 (sup2): 235-320. http://dx.doi.org/10.1080/1755876X.2016.1273446
* Pérez Gómez B., De Alfonso M., Zacharioudaki A., Pérez González I., Álvarez Fanjul E., Müller M., Marcos M., Manzano F., Korres G., Ravdas M., Tamm S. 2018. Sea level, SST and waves: extremes variability. In: Copernicus Marine Service Ocean State Report, Issue 2, Journal of Operational Oceanography, 11:sup1, Chap. 3.1, s79–s88, DOI: https://doi.org/10.1080/1755876X.2018.1489208
* Savina H, Lefevre J-M, Josse P, Dandin P. 2003. Definition of warning criteria. Proceedings of MAXWAVE Final Meeting, October 8-11, Geneva, Switzerland.
* Staneva, J. Behrens, A., Gayer G, Ricker M. (2019a) Black sea CMEMS MYP QUID Report
* Staneva J, Behrens A., Gayer G, Aouf A., (2019b). Synergy between CMEMS products and newly available data from SENTINEL, Section 3.3, In: Schuckmann K,et al. 2019. Copernicus Marine Service Ocean State Report, Issue 3, Journal of Operational Oceanography, doi: 10.1080/1755876X.2019.1633075.





<img src="https://documentation.marine.copernicus.eu/IMG/BLKSEA_OMI_TEMPSAL_extreme_var_temp_mean_and_anomaly.png" width="300" height="200"/>



# Black Sea Surface Temperature extreme from Reanalysis


**Product ID:** BLKSEA_OMI_TEMPSAL_extreme_var_temp_mean_and_anomaly

**Description:** **DEFINITION**

The CMEMS BLKSEA_OMI_tempsal_extreme_var_temp_mean_and_anomaly OMI indicator is based on the computation of the annual 99th percentile of Sea Surface Temperature (SST) from model data. Two different CMEMS products are used to compute the indicator: The Iberia-Biscay-Ireland Multi Year Product (BLKSEA_MULTIYEAR_PHY_007_004) and the Analysis product (BLKSEA_ANALYSIS_FORECAST_PHYS_007_001).
Two parameters have been considered for this OMI:
* Map of the 99th mean percentile: It is obtained from the Multi Year Product, the annual 99th percentile is computed for each year of the product. The percentiles are temporally averaged over the whole period (1993-2019).
* Anomaly of the 99th percentile in 2020: The 99th percentile of the year 2020 is computed from the Analysis product. The anomaly is obtained by subtracting the mean percentile from the 2020 percentile.
This indicator is aimed at monitoring the extremes of sea surface temperature every year and at checking their variations in space. The use of percentiles instead of annual maxima, makes this extremes study less affected by individual data. This study of extreme variability was first applied to the sea level variable (Pérez Gómez et al 2016) and then extended to other essential variables, such as sea surface temperature and significant wave height (Pérez Gómez et al 2018 and Alvarez Fanjul et al., 2019). More details and a full scientific evaluation can be found in the CMEMS Ocean State report (Alvarez Fanjul et al., 2019).

**CONTEXT**

The Sea Surface Temperature is one of the Essential Ocean Variables, hence the monitoring of this variable is of key importance, since its variations can affect the ocean circulation, marine ecosystems, and ocean-atmosphere exchange processes. Particularly in the Black Sea, ocean-atmospheric processes together with its general cyclonic circulation (Rim Current) play an important role on the sea surface temperature variability (Capet et al. 2012). As the oceans continuously interact with the atmosphere, trends of sea surface temperature can also have an effect on the global climate. The 99th mean percentile of sea surface temperature provides a worth information about the variability of the sea surface temperature and warming trends but has not been investigated with details in the Black Sea.
While the global-averaged sea surface temperatures have increased since the beginning of the 20th century (Hartmann et al., 2013). Recent studies indicated a warming trend of the sea surface temperature in the Black Sea in the latest years (Mulet et al., 2018; Sakali and Başusta, 2018). A specific analysis on the interannual variability of the basin-averaged sea surface temperature revealed a higher positive trend in its eastern region (Ginzburg et al., 2004). For the past three decades, Sakali and Başusta (2018) presented an increase in sea surface temperature that varied along both east–west and south–north directions in the Black Sea. 

**CMEMS KEY FINDINGS**

The mean annual 99th percentile in the period 1993–2019 exhibits values ranging from 25.50 to 26.50 oC in the western and central regions of the Black Sea. The values increase towards the east, exceeding 27.5 oC. This contrasting west-east pattern may be linked to the basin wide cyclonic circulation. There are regions showing lower values, below 25.75 oC, such as a small area west of Crimean Peninsula in the vicinity of the Sevastopol anticyclone, the Northern Ukraine region, in particular close to the Odessa and the Karkinytska Gulf due to the freshwaters from the land and a narrow area along the Turkish coastline in the south. Results for 2020 show negative anomalies in the area of influence of the Bosporus and the Bulgarian offshore region up to the Crimean peninsula, while the North West shelf exhibits a positive anomaly as in the Eastern basin. The highest positive value is occurring in the Eastern Tukish coastline nearest the Batumi gyre area. This may be related to the variously increase of sea surface temperature in such a way the southern regions have experienced a higher warming.

Note: The key findings will be updated annually in November, in line with OMI evolutions.

**DOI (product):** 
https://doi.org/10.48670/moi-00216

**References:**

* Álvarez Fanjul E, Pascual Collar A, Pérez Gómez B, De Alfonso M, García Sotillo M, Staneva J, Clementi E, Grandi A, Zacharioudaki A, Korres G, Ravdas M, Renshaw R, Tinker J, Raudsepp U, Lagemaa P, Maljutenko I, Geyer G, Müller M, Çağlar Yumruktepe V. Sea level, sea surface temperature and SWH extreme percentiles: combined analysis from model results and in situ observations, Section 2.7, p:31. In: Schuckmann K, Le Traon P-Y, Smith N, Pascual A, Djavidnia S, Gattuso J-P, Grégoire M, Nolan G, et al. 2019. Copernicus Marine Service Ocean State Report, Issue 3, Journal of Operational Oceanography, 12:sup1, S1-S123, DOI: 10.1080/1755876X.2019.1633075
* Capet, A., Barth, A., Beckers, J. M., & Marilaure, G. (2012). Interannual variability of Black Sea's hydrodynamics and connection to atmospheric patterns. Deep Sea Research Part II: Topical Studies in Oceanography, 77, 128-142. https://doi.org/10.1016/j.dsr2.2012.04.010
* Ginzburg, A. I.; Kostianoy, A. G.; Sheremet, N. A. (2004). Seasonal and interannual variability of the Black Sea surface temperature as revealed from satellite data (1982–2000), Journal of Marine Systems, 52, 33-50. https://doi.org/10.1016/j.jmarsys.2004.05.002.
* Hartmann DL, Klein Tank AMG, Rusticucci M, Alexander LV, Brönnimann S, Charabi Y, Dentener FJ, Dlugokencky EJ, Easterling DR, Kaplan A, Soden BJ, Thorne PW, Wild M, Zhai PM. 2013. Observations: Atmosphere and Surface. In: Climate Change 2013: The Physical Science Basis. Contribution of Working Group I to the Fifth Assessment Report of the Intergovernmental Panel on Climate Change. Cambridge University Press, Cambridge, United Kingdom and New York, NY, USA.
* Mulet S, Nardelli BB, Good S, Pisano A, Greiner E, Monier M. 2018. Ocean temperature and salinity. In: Copernicus Marine Service Ocean State Report, Issue 2, Journal of Operational Oceanography, 11:sup1, Chap. 1.1, s5–s13, DOI: https://doi.org/10.1080/1755876X.2018.1489208
* Pérez-Gómez B, Álvarez-Fanjul E, She J, Pérez-González I, Manzano F. 2016. Extreme sea level events, Section 4.4, p:300. In: Von Schuckmann K, Le Traon PY, Alvarez-Fanjul E, Axell L, Balmaseda M, Breivik LA, Brewin RJW, Bricaud C, Drevillon M, Drillet Y, Dubois C , Embury O, Etienne H, García-Sotillo M, Garric G, Gasparin F, Gutknecht E, Guinehut S, Hernandez F, Juza M, Karlson B, Korres G, Legeais JF, Levier B, Lien VS, Morrow R, Notarstefano G, Parent L, Pascual A, Pérez-Gómez B, Perruche C, Pinardi N, Pisano A, Poulain PM , Pujol IM, Raj RP, Raudsepp U, Roquet H, Samuelsen A, Sathyendranath S, She J, Simoncelli S, Solidoro C, Tinker J, Tintoré J, Viktorsson L, Ablain M, Almroth-Rosell E, Bonaduce A, Clementi E, Cossarini G, Dagneaux Q, Desportes C, Dye S, Fratianni C, Good S, Greiner E, Gourrion J, Hamon M, Holt J, Hyder P, Kennedy J, Manzano-Muñoz F, Melet A, Meyssignac B, Mulet S, Nardelli BB, O’Dea E, Olason E, Paulmier A, Pérez-González I, Reid R, Racault MF, Raitsos DE, Ramos A, Sykes P, Szekely T, Verbrugge N. 2016. The Copernicus Marine Environment Monitoring Service Ocean State Report, Journal of Operational Oceanography. 9 (sup2): 235-320. http://dx.doi.org/10.1080/1755876X.2016.1273446
* Pérez Gómez B., De Alfonso M., Zacharioudaki A., Pérez González I., Álvarez Fanjul E., Müller M., Marcos M., Manzano F., Korres G., Ravdas M., Tamm S. 2018. Sea level, SST and waves: extremes variability. In: Copernicus Marine Service Ocean State Report, Issue 2, Journal of Operational Oceanography, 11:sup1, Chap. 3.1, s79–s88, DOI: https://doi.org/10.1080/1755876X.2018.1489208
* Sakalli A, Başusta N. 2018. Sea surface temperature change in the Black Sea under climate change: A simulation of the sea surface temperature up to 2100. International Journal of Climatology, 38(13), 4687-4698. https://doi.org/10.1002/joc.5688





<img src="https://documentation.marine.copernicus.eu/IMG/BLKSEA_OMI_TEMPSAL_sst_area_averaged_anomalies.png" width="300" height="200"/>



# Black Sea Surface Temperature time series and trend from Observations Reprocessing


**Product ID:** BLKSEA_OMI_TEMPSAL_sst_area_averaged_anomalies

**Description:** "_DEFINITION_'

The blksea_omi_tempsal_sst_area_averaged_anomalies product for 2023 includes unfiltered Sea Surface Temperature (SST) anomalies, given as monthly mean time series starting on 1982 and averaged over the Black Sea, and 24-month filtered SST anomalies, obtained by using the X11-seasonal adjustment procedure. This OMI is derived from the CMEMS Reprocessed Black Sea L4 SST satellite product (SST_BS_SST_L4_REP_OBSERVATIONS_010_022, see e.g. the OMI QUID, http://marine.copernicus.eu/documents/QUID/CMEMS-OMI-QUID-BLKSEA-SST.pdf), which provided the SSTs used to compute the evolution of SST anomalies (unfiltered and filtered) over the Black Sea. This reprocessed product consists of daily (nighttime) optimally interpolated 0.05° grid resolution SST maps over the Black Sea built from the ESA Climate Change Initiative (CCI) (Merchant et al., 2019) and Copernicus Climate Change Service (C3S) initiatives, including also an adjusted version of the AVHRR Pathfinder dataset version 5.3 (Saha et al., 2018) to increase the input observation coverage. Anomalies are computed against the 1991-2020 reference period. The 30-year climatology 1991-2020 is defined according to the WMO recommendation (WMO, 2017) and recent U.S. National Oceanic and Atmospheric Administration practice (https://wmo.int/media/news/updated-30-year-reference-period-reflects-changing-climate). The reference for this OMI can be found in the first and second issue of the Copernicus Marine Service Ocean State Report (OSR), Section 1.1 (Roquet et al., 2016; Mulet et al., 2018).

**CONTEXT**

Sea surface temperature (SST) is a key climate variable since it deeply contributes in regulating climate and its variability (Deser et al., 2010). SST is then essential to monitor and characterise the state of the global climate system (GCOS 2010). Long-term SST variability, from interannual to (multi-)decadal timescales, provides insight into the slow variations/changes in SST, i.e. the temperature trend (e.g., Pezzulli et al., 2005). In addition, on shorter timescales, SST anomalies become an essential indicator for extreme events, as e.g. marine heatwaves (Hobday et al., 2018). In the last decades, since the availability of satellite data (beginning of 1980s), the Black Sea has experienced a warming trend in SST (see e.g. Buongiorno Nardelli et al., 2010; Mulet et al., 2018).

**KEY FINDINGS**

During 2023, the Black Sea basin average SST anomaly was ~1.1 °C above the 1991-2020 climatology, doubling that of previous year (~0.5 °C). The Black Sea SST monthly anomalies ranged between -1.0/+1.0 °C. The highest temperature anomaly (~1.8 °C) was reached in January 2023, while the lowest (~-0.28 °C) in May. This year, along with 2022, was characterized by milder temperature anomalies with respect to the previous three consecutive years (2018-2020) marked by peaks of ~3 °C occurred in May 2018, June 2019, and October 2020.
Over the period 1982-2023, the Black Sea SST has warmed at a rate of 0.065 ± 0.002 °C/year, which corresponds to an average increase of about 2.7 °C during these last 42 years.

**DOI (product):** 
https://doi.org/10.48670/moi-00217

**References:**

* Buongiorno Nardelli, B., Colella, S. Santoleri, R., Guarracino, M., Kholod, A., 2010. A re-analysis of Black Sea surface temperature. Journal of Marine Systems, 79, Issues 1–2, 50-64, ISSN 0924-7963, https://doi.org/10.1016/j.jmarsys.2009.07.001.
* Deser, C., Alexander, M. A., Xie, S.-P., Phillips, A. S., 2010. Sea Surface Temperature Variability: Patterns and Mechanisms. Annual Review of Marine Science 2010 2:1, 115-143. https://doi.org/10.1146/annurev-marine-120408-151453
* GCOS. Global Climate Observing System. 2010. Update of the Implementation Plan for the Global Observing System for Climate in Support of the UNFCCC (GCO-138).
* Hobday, A. J., Oliver, E. C., Gupta, A. S., Benthuysen, J. A., Burrows, M. T., Donat, M. G., ... & Smale, D. A. (2018). Categorizing and naming marine heatwaves. Oceanography, 31(2), 162-173.
* Merchant, C. J., Embury, O., Bulgin, C. E., Block, T., Corlett, G. K., Fiedler, E., ... & Eastwood, S. (2019). Satellite-based time-series of sea-surface temperature since 1981 for climate applications. Scientific data, 6(1), 1-18.
* Mulet, S., Buongiorno Nardelli, B., Good, S., Pisano, A., Greiner, E., Monier, M., Autret, E., Axell, L., Boberg, F., Ciliberti, S., Drévillon, M., Droghei, R., Embury, O., Gourrion, J., Høyer, J., Juza, M., Kennedy, J., Lemieux-Dudon, B., Peneva, E., Reid, R., Simoncelli, S., Storto, A., Tinker, J., Von Schuckmann, K., Wakelin, S. L., 2018. Ocean temperature and salinity. In: Copernicus Marine Service Ocean State Report, Issue 2, Journal of Operational Oceanography, 11:sup1, s5–s13, DOI: 10.1080/1755876X.2018.1489208
* Pezzulli, S., Stephenson, D. B., Hannachi, A., 2005. The Variability of Seasonality. J. Climate. 18:71–88. doi:10.1175/JCLI-3256.1.





<img src="https://documentation.marine.copernicus.eu/IMG/BLKSEA_OMI_TEMPSAL_sst_trend.png" width="300" height="200"/>



# Black Sea Surface Temperature cumulative trend map from Observations Reprocessing


**Product ID:** BLKSEA_OMI_TEMPSAL_sst_trend

**Description:** **DEFINITION**

The blksea_omi_tempsal_sst_trend product includes the cumulative/net Sea Surface Temperature (SST) trend for the Black Sea over the period 1982-2023, i.e. the rate of change (°C/year) multiplied by the number years in the timeseries (42). This OMI is derived from the CMEMS Reprocessed Black Sea L4 SST satellite product (SST_BS_SST_L4_REP_OBSERVATIONS_010_022, see e.g. the OMI QUID, http://marine.copernicus.eu/documents/QUID/CMEMS-OMI-QUID-BLKSEA-SST.pdf), which provided the SSTs used to compute the SST trend over the Black Sea. This reprocessed product consists of daily (nighttime) optimally interpolated 0.05° grid resolution SST maps over the Black Sea built from the ESA Climate Change Initiative (CCI) (Merchant et al., 2019) and Copernicus Climate Change Service (C3S) initiatives, including also an adjusted version of the AVHRR Pathfinder dataset version 5.3 (Saha et al., 2018) to increase the input observation coverage. Trend analysis has been performed by using the X-11 seasonal adjustment procedure (see e.g. Pezzulli et al., 2005), which has the effect of filtering the input SST time series acting as a low bandpass filter for interannual variations. Mann-Kendall test and Sens’s method (Sen 1968) were applied to assess whether there was a monotonic upward or downward trend and to estimate the slope of the trend and its 95% confidence interval. The reference for this OMI can be found in the first and second issue of the Copernicus Marine Service Ocean State Report (OSR), Section 1.1 (Roquet et al., 2016; Mulet et al., 2018).

**CONTEXT**

Sea surface temperature (SST) is a key climate variable since it deeply contributes in regulating climate and its variability (Deser et al., 2010). SST is then essential to monitor and characterise the state of the global climate system (GCOS 2010). Long-term SST variability, from interannual to (multi-)decadal timescales, provides insight into the slow variations/changes in SST, i.e. the temperature trend (e.g., Pezzulli et al., 2005). In addition, on shorter timescales, SST anomalies become an essential indicator for extreme events, as e.g. marine heatwaves (Hobday et al., 2018). In the last decades, since the availability of satellite data (beginning of 1980s), the Black Sea has experienced a warming trend in SST (see e.g. Buongiorno Nardelli et al., 2010; Mulet et al., 2018).
**KEY FINDINGS**

Over the past four decades (1982-2023), the Black Sea surface temperature (SST) warmed at a rate of 0.065 ± 0.002 °C per year, corresponding to a mean surface temperature warming of about 2.7 °C. The spatial pattern of the Black Sea SST trend reveals a general warming tendency, ranging from 0.053 °C/year to 0.080 °C/year. The spatial pattern of SST trend is rather homogeneous over the whole basin. Highest values characterize the eastern basin, where the trend reaches the extreme value, while lower values are found close to the western coasts, in correspondence of main rivers inflow. The Black Sea SST trend continues to show the highest intensity among all the other European Seas.
**DOI (product):** 
https://doi.org/10.48670/moi-00218

**References:**

* Buongiorno Nardelli, B., Colella, S. Santoleri, R., Guarracino, M., Kholod, A., 2010. A re-analysis of Black Sea surface temperature. Journal of Marine Systems, 79, Issues 1–2, 50-64, ISSN 0924-7963, https://doi.org/10.1016/j.jmarsys.2009.07.001.
* Deser, C., Alexander, M. A., Xie, S.-P., Phillips, A. S., 2010. Sea Surface Temperature Variability: Patterns and Mechanisms. Annual Review of Marine Science 2010 2:1, 115-143. https://doi.org/10.1146/annurev-marine-120408-151453
* GCOS. Global Climate Observing System. 2010. Update of the Implementation Plan for the Global Observing System for Climate in Support of the UNFCCC (GCO-138).
* Hobday, A. J., Oliver, E. C., Gupta, A. S., Benthuysen, J. A., Burrows, M. T., Donat, M. G., ... & Smale, D. A. (2018). Categorizing and naming marine heatwaves. Oceanography, 31(2), 162-173.
* Merchant, C. J., Embury, O., Bulgin, C. E., Block, T., Corlett, G. K., Fiedler, E., ... & Eastwood, S. (2019). Satellite-based time-series of sea-surface temperature since 1981 for climate applications. Scientific data, 6(1), 1-18.
* Mulet, S., Buongiorno Nardelli, B., Good, S., Pisano, A., Greiner, E., Monier, M., Autret, E., Axell, L., Boberg, F., Ciliberti, S., Drévillon, M., Droghei, R., Embury, O., Gourrion, J., Høyer, J., Juza, M., Kennedy, J., Lemieux-Dudon, B., Peneva, E., Reid, R., Simoncelli, S., Storto, A., Tinker, J., Von Schuckmann, K., Wakelin, S. L., 2018. Ocean temperature and salinity. In: Copernicus Marine Service Ocean State Report, Issue 2, Journal of Operational Oceanography, 11:sup1, s5–s13, DOI: 10.1080/1755876X.2018.1489208
* Pezzulli, S., Stephenson, D. B., Hannachi, A., 2005. The Variability of Seasonality. J. Climate. 18:71–88. doi:10.1175/JCLI-3256.1.
* Saha, Korak; Zhao, Xuepeng; Zhang, Huai-min; Casey, Kenneth S.; Zhang, Dexin; Baker-Yeboah, Sheekela; Kilpatrick, Katherine A.; Evans, Robert H.; Ryan, Thomas; Relph, John M. (2018). AVHRR Pathfinder version 5.3 level 3 collated (L3C) global 4km sea surface temperature for 1981-Present. NOAA National Centers for Environmental Information. Dataset. https://doi.org/10.7289/v52j68xx Sen, P. K., 1968. Estimates of the regression coefficient based on Kendall’s tau. J Am Statist Assoc. 63:1379–1389.





<img src="https://mdl-metadata.s3.waw3-1.cloudferro.com/metadata/thumbnails/GLOBAL_ANALYSISFORECAST_BGC_001_028.jpg" width="300" height="200"/>



# Global Ocean Biogeochemistry Analysis and Forecast


**Product ID:** GLOBAL_ANALYSISFORECAST_BGC_001_028

**Description:** The Operational Mercator Ocean biogeochemical global ocean analysis and forecast system  at 1/4 degree is providing 10 days of 3D global ocean forecasts updated weekly. The time series is aggregated in time, in order to reach a two full year’s time series sliding window. This product includes daily and monthly mean files of biogeochemical parameters (chlorophyll, nitrate, phosphate, silicate, dissolved oxygen, dissolved iron, primary production, phytoplankton, zooplankton, PH, and surface partial pressure of carbon dioxyde) over the global ocean. The global ocean output files are displayed with a 1/4 degree horizontal resolution with regular longitude/latitude equirectangular projection. 50 vertical levels are ranging from 0 to 5700 meters.

* NEMO version (v3.6_STABLE)
* Forcings: GLOBAL_ANALYSIS_FORECAST_PHYS_001_024 at daily frequency.                                                                           
* Outputs mean fields are interpolated on a standard regular grid in NetCDF format.
* Initial conditions: World Ocean Atlas 2013 for nitrate, phosphate, silicate and dissolved oxygen, GLODAPv2 for DIC and Alkalinity, and climatological model outputs for Iron and DOC 
* Quality/Accuracy/Calibration information: See the related [QuID](https://documentation.marine.copernicus.eu/QUID/CMEMS-GLO-QUID-001-028.pdf)

**DOI (product):**   
https://doi.org/10.48670/moi-00015




<img src="https://mdl-metadata.s3.waw3-1.cloudferro.com/metadata/thumbnails/GLOBAL_ANALYSISFORECAST_PHY_001_024.jpg" width="300" height="200"/>



# Global Ocean Physics Analysis and Forecast


**Product ID:** GLOBAL_ANALYSISFORECAST_PHY_001_024

**Description:** The Operational Mercator global ocean analysis and forecast system at 1/12 degree is providing 10 days of 3D global ocean forecasts updated daily. The time series is aggregated in time in order to reach a two full year’s time series sliding window.

This product includes daily and monthly mean files of temperature, salinity, currents, sea level, mixed layer depth and ice parameters from the top to the bottom over the global ocean. It also includes hourly mean surface fields for sea level height, temperature and currents. The global ocean output files are displayed with a 1/12 degree horizontal resolution with regular longitude/latitude equirectangular projection.

50 vertical levels are ranging from 0 to 5500 meters.

This product also delivers a special dataset for surface current which also includes wave and tidal drift called SMOC (Surface merged Ocean Current).

**DOI (product):**   
https://doi.org/10.48670/moi-00016




<img src="https://mdl-metadata.s3.waw3-1.cloudferro.com/metadata/thumbnails/GLOBAL_ANALYSISFORECAST_WAV_001_027.jpg" width="300" height="200"/>



# Global Ocean Waves Analysis and Forecast


**Product ID:** GLOBAL_ANALYSISFORECAST_WAV_001_027

**Description:** The operational global ocean analysis and forecast system of Météo-France with a resolution of 1/12 degree is providing daily analyses and 10 days forecasts for the global ocean sea surface waves. This product includes 3-hourly instantaneous fields of integrated wave parameters from the total spectrum (significant height, period, direction, Stokes drift,...etc), as well as the following partitions: the wind wave, the primary and secondary swell waves.
 
The global wave system of Météo-France is based on the wave model MFWAM which is a third generation wave model. MFWAM uses the computing code ECWAM-IFS-38R2 with a dissipation terms developed by Ardhuin et al. (2010). The model MFWAM was upgraded on november 2014 thanks to improvements obtained from the european research project « my wave » (Janssen et al. 2014). The model mean bathymetry is generated by using 2-minute gridded global topography data ETOPO2/NOAA. Native model grid is irregular with decreasing distance in the latitudinal direction close to the poles. At the equator the distance in the latitudinal direction is more or less fixed with grid size 1/10°. The operational model MFWAM is driven by 6-hourly analysis and 3-hourly forecasted winds from the IFS-ECMWF atmospheric system. The wave spectrum is discretized in 24 directions and 30 frequencies starting from 0.035 Hz to 0.58 Hz. The model MFWAM uses the assimilation of altimeters with a time step of 6 hours. The global wave system provides analysis 4 times a day, and a forecast of 10 days at 0:00 UTC. The wave model MFWAM uses the partitioning to split the swell spectrum in primary and secondary swells.

**DOI (product):**   
https://doi.org/10.48670/moi-00017

**References:**

* F. Ardhuin, R. Magne, J-F. Filipot, A. Van der Westhyusen, A. Roland, P. Quefeulou, J. M. Lefèvre, L. Aouf, A. Babanin and F. Collard : Semi empirical dissipation source functions for wind-wave models : Part I, definition and calibration and validation at global scales. Journal of Physical Oceanography, March 2010.
* P. Janssen, L. Aouf, A. Behrens, G. Korres, L. Cavalieri, K. Christiensen, O. Breivik : Final report of work-package I in my wave project. December 2014.





<img src="https://mdl-metadata.s3.waw3-1.cloudferro.com/metadata/thumbnails/GLOBAL_MULTIYEAR_BGC_001_029.jpg" width="300" height="200"/>



# Global Ocean Biogeochemistry Hindcast


**Product ID:** GLOBAL_MULTIYEAR_BGC_001_029

**Description:** The biogeochemical hindcast for global ocean is produced at Mercator-Ocean (Toulouse. France). It provides 3D biogeochemical fields since year 1993 at 1/4 degree and on 75 vertical levels. It uses PISCES biogeochemical model (available on the NEMO modelling platform). No data assimilation in this product.

* Latest NEMO version (v3.6_STABLE)
* Forcings: FREEGLORYS2V4 ocean physics produced at Mercator-Ocean and ERA-Interim atmosphere produced at ECMWF at a daily frequency                                                                           
* Outputs: Daily (chlorophyll. nitrate. phosphate. silicate. dissolved oxygen. primary production) and monthly (chlorophyll. nitrate. phosphate. silicate. dissolved oxygen. primary production. iron. phytoplankton in carbon) 3D mean fields interpolated on a standard regular grid in NetCDF format. The simulation is performed once and for all.
* Initial conditions: World Ocean Atlas 2013 for nitrate. phosphate. silicate and dissolved oxygen. GLODAPv2 for DIC and Alkalinity. and climatological model outputs for Iron and DOC 
* Quality/Accuracy/Calibration information: See the related QuID

**DOI (product):**
https://doi.org/10.48670/moi-00019




<img src="https://mdl-metadata.s3.waw3-1.cloudferro.com/metadata/thumbnails/GLOBAL_MULTIYEAR_BGC_001_033.jpg" width="300" height="200"/>



# Global ocean low and mid trophic levels biomass content hindcast


**Product ID:** GLOBAL_MULTIYEAR_BGC_001_033

**Description:** The Low and Mid-Trophic Levels (LMTL) reanalysis for global ocean is produced at [CLS](https://www.cls.fr) on behalf of Global Ocean Marine Forecasting Center. It provides 2D fields of biomass content of zooplankton and six functional groups of micronekton. It uses the LMTL component of SEAPODYM dynamical population model (http://www.seapodym.eu). No data assimilation has been done. This product also contains forcing data: net primary production, euphotic depth, depth of each pelagic layers zooplankton and micronekton inhabit, average temperature and currents over pelagic layers.

**Forcings sources:**
* Ocean currents and temperature (CMEMS multiyear product)
* Net Primary Production computed from chlorophyll a, Sea Surface Temperature and Photosynthetically Active Radiation observations (chlorophyll from CMEMS multiyear product, SST from NOAA NCEI AVHRR-only Reynolds, PAR from INTERIM) and relaxed by model outputs at high latitudes (CMEMS biogeochemistry multiyear product)

**Vertical coverage:**
* Epipelagic layer 
* Upper mesopelagic layer
* Lower mesopelagic layer (max. 1000m)

**DOI (product):**   
https://doi.org/10.48670/moi-00020

**References:**

* Lehodey P., Murtugudde R., Senina I. (2010). Bridging the gap from ocean models to population dynamics of large marine predators: a model of mid-trophic functional groups. Progress in Oceanography, 84, p. 69-84.
* Lehodey, P., Conchon, A., Senina, I., Domokos, R., Calmettes, B., Jouanno, J., Hernandez, O., Kloser, R. (2015) Optimization of a micronekton model with acoustic data. ICES Journal of Marine Science, 72(5), p. 1399-1412.
* Conchon A. (2016). Modélisation du zooplancton et du micronecton marins. Thèse de Doctorat, Université de La Rochelle, 136 p.





<img src="https://mdl-metadata.s3.waw3-1.cloudferro.com/metadata/thumbnails/GLOBAL_MULTIYEAR_PHY_001_030.jpg" width="300" height="200"/>



# Global Ocean Physics Reanalysis


**Product ID:** GLOBAL_MULTIYEAR_PHY_001_030

**Description:** The GLORYS12V1 product is the CMEMS global ocean eddy-resolving (1/12° horizontal resolution, 50 vertical levels) reanalysis covering the altimetry (1993 onward).

It is based largely on the current real-time global forecasting CMEMS system. The model component is the NEMO platform driven at surface by ECMWF ERA-Interim then ERA5 reanalyses for recent years. Observations are assimilated by means of a reduced-order Kalman filter. Along track altimeter data (Sea Level Anomaly), Satellite Sea Surface Temperature, Sea Ice Concentration and In situ Temperature and Salinity vertical Profiles are jointly assimilated. Moreover, a 3D-VAR scheme provides a correction for the slowly-evolving large-scale biases in temperature and salinity.

This product includes daily and monthly mean files for  temperature, salinity, currents, sea level, mixed layer depth and ice parameters from the top to the bottom. The global ocean output files are displayed on a standard regular grid at 1/12° (approximatively 8 km) and on 50 standard levels.

**DOI (product):**
https://doi.org/10.48670/moi-00021




<img src="https://mdl-metadata.s3.waw3-1.cloudferro.com/metadata/thumbnails/GLOBAL_MULTIYEAR_PHY_ENS_001_031.jpg" width="300" height="200"/>



# Global Ocean Ensemble Physics Reanalysis


**Product ID:** GLOBAL_MULTIYEAR_PHY_ENS_001_031

**Description:** You can find here the CMEMS Global Ocean Ensemble Reanalysis product at ¼ degree resolution: monthly means of Temperature, Salinity, Currents and Ice variables for 75 vertical levels, starting from 1993 onward.
 
Global ocean reanalyses are homogeneous 3D gridded descriptions of the physical state of the ocean covering several decades, produced with a numerical ocean model constrained with data assimilation of satellite and in situ observations. These reanalyses are built to be as close as possible to the observations (i.e. realistic) and in agreement with the model physics The multi-model ensemble approach allows uncertainties or error bars in the ocean state to be estimated.

The ensemble mean may even provide for certain regions and/or periods a more reliable estimate than any individual reanalysis product.

The three reanalyses, used to create the ensemble, covering “altimetric era” period (starting from 1st of January 1993) during which altimeter altimetry data observations are available:
 * GLORYS2V4 from Mercator Ocean (Fr);
 * ORAS5 from ECMWF;
 * and C-GLORSv7 from CMCC (It);
 
These three products provided three different time series of global ocean simulations 3D monthly estimates. All numerical products available for users are monthly or daily mean averages describing the ocean.

**DOI (product):**   
https://doi.org/10.48670/moi-00024




<img src="https://mdl-metadata.s3.waw3-1.cloudferro.com/metadata/thumbnails/GLOBAL_MULTIYEAR_WAV_001_032.jpg" width="300" height="200"/>



# Global Ocean Waves Reanalysis


**Product ID:** GLOBAL_MULTIYEAR_WAV_001_032

**Description:** GLOBAL_REANALYSIS_WAV_001_032 for the global wave reanalysis describing past sea states since years 1980. This product also bears the name of WAVERYS within the GLO-HR MFC for correspondence to other global multi-year products like GLORYS. BIORYS. etc. The core of WAVERYS is based on the MFWAM model. a third generation wave model that calculates the wave spectrum. i.e. the distribution of sea state energy in frequency and direction on a 1/5° irregular grid. Average wave quantities derived from this wave spectrum such as the SWH (significant wave height) or the average wave period are delivered on a regular 1/5° grid with a 3h time step. The wave spectrum is discretized into 30 frequencies obtained from a geometric sequence of first member 0.035 Hz and a reason 7.5. WAVERYS takes into account oceanic currents from the GLORYS12 physical ocean reanalysis and assimilates SWH observed from historical altimetry missions and directional wave spectra from Sentinel 1 SAR from 2017 onwards.   

**DOI (product):**
https://doi.org/10.48670/moi-00022




<img src="https://documentation.marine.copernicus.eu/IMG/GLOBAL_OMI_CLIMVAR_enso_Tzt_anomaly.png" width="300" height="200"/>



# Nino 3.4 Temporal Evolution of Vertical Profile of Temperature from Reanalysis


**Product ID:** GLOBAL_OMI_CLIMVAR_enso_Tzt_anomaly

**Description:** "_DEFINITION_'

NINO34 sub surface temperature anomaly (°C) is defined as the difference between the subsurface temperature  averaged over the 170°W-120°W 5°S,-5°N area  and the climatological reference value over same area  (GLOBAL_MULTIYEAR_PHY_ENS_001_031). Spatial averaging was weighted by surface area. Monthly mean values are given here. The reference period is 1993-2014.  

**CONTEXT**

El Nino Southern Oscillation (ENSO) is one of the most important sources of climatic variability resulting from a strong coupling between ocean and atmosphere in the central tropical Pacific and affecting surrounding populations. Globally, it impacts ecosystems, precipitation, and freshwater resources (Glantz, 2001). ENSO is mainly characterized by two anomalous states that last from several months to more than a year and recur irregularly on a typical time scale of 2-7 years. The warm phase El Niño is broadly characterized by a weakening of the easterly trade winds at interannual timescales associated with surface and subsurface processes leading to a surface warming in the eastern Pacific. Opposite changes are observed during the cold phase La Niña (review in Wang et al., 2017). Nino 3.4 sub-surface Temperature Anomaly is a good indicator of the state of the Central tropical Pacific el Nino conditions and enable to monitor the evolution the ENSO phase.

**CMEMS KEY FINDINGS **

Over the 1993-2023 period, there were several episodes of strong positive ENSO (el nino) phases in particular during the 1997/1998 winter and the 2015/2016 winter, where NINO3.4 indicator reached positive values larger than 2°C (and remained above 0.5°C during more than 6 months). Several La Nina events were also observed like during the 1998/1999 winter and during the 2010/2011 winter.  
The NINO34 subsurface indicator is a good index to monitor the state of ENSO phase and a useful tool to help seasonal forecasting of atmospheric conditions. 

Note: The key findings will be updated annually in November, in line with OMI evolutions.

**DOI (product):** 
https://doi.org/10.48670/moi-00220

**References:**

* Copernicus Marine Service Ocean State Report. (2018). Journal of Operational Oceanography, 11(sup1), S1–S142. https://doi.org/10.1080/1755876X.2018.1489208





<img src="https://documentation.marine.copernicus.eu/IMG/GLOBAL_OMI_CLIMVAR_enso_sst_area_averaged_anomaly.png" width="300" height="200"/>



# Nino 3.4 Sea Surface Temperature time series from Reanalysis


**Product ID:** GLOBAL_OMI_CLIMVAR_enso_sst_area_averaged_anomalies

**Description:** **DEFINITION**

NINO34 sea surface temperature anomaly (°C) is defined as the difference between the sea surface temperature averaged over the 170°W-120°W 5°S,-5°N area  and the climatological reference value over same area  (GLOBAL_MULTIYEAR_PHY_ENS_001_031) . Spatial averaging was weighted by surface area. Monthly mean values are given here. The reference period is 1993-2014. El Nino or La Nina events are defined when the NINO3.4 SST anomalies exceed +/- 0.4°C during a period of six month.

**CONTEXT**

El Nino Southern Oscillation (ENSO) is one of the most  important source of climatic  variability resulting  from a strong coupling between ocean and atmosphere in the central tropical Pacific and affecting surrounding populations.  Globally, it impacts ecosystems, precipitation, and freshwater resources (Glantz, 2001). ENSO is mainly characterized by two anomalous states that last from several months to more than a year and recur irregularly on a typical time scale of 2-7 years. The warm phase El Niño is broadly characterized by a weakening of the easterly trade winds at interannual timescales associated with surface and subsurface processes leading to a surface warming in the eastern Pacific. Opposite changes are observed during the cold phase La Niña (review in Wang et al., 2017). Nino 3.4 Sea surface Temperature Anomaly is a good indicator of the state of the Central tropical Pacific El Nino conditions and enable to monitor the evolution the ENSO phase.

**CMEMS KEY FINDINGS**

Over the 1993-2023 period, there were several episodes of strong positive ENSO phases in particular in 1998 and 2016, where NINO3.4 indicator reached positive values larger than 2°C (and remained above 0.5°C during more than 6 months). Several La Nina events were also observed like in 2000 and 2008.  
The NINO34 indicator is a good index to monitor the state of ENSO phase and a useful tool to help seasonal forecasting of meteorological conditions. 

Note: The key findings will be updated annually in November, in line with OMI evolutions.

**DOI (product):** 
https://doi.org/10.48670/moi-00219

**References:**

* Copernicus Marine Service Ocean State Report. (2018). Journal of Operational Oceanography, 11(sup1), S1–S142. https://doi.org/10.1080/1755876X.2018.1489208





<img src="https://documentation.marine.copernicus.eu/IMG/GLOBAL_OMI_HEALTH_carbon_co2_flux_integrated.png" width="300" height="200"/>



# Global Ocean Yearly CO2 Sink from Multi-Observations Reprocessing


**Product ID:** GLOBAL_OMI_HEALTH_carbon_co2_flux_integrated

**Description:** **DEFINITION**

The global yearly ocean CO2 sink represents the ocean uptake of CO2 from the atmosphere computed over the whole ocean. It is expressed in PgC per year. The ocean monitoring index is presented for the period 1985 to year-1. The yearly estimate of the ocean CO2 sink corresponds to the mean of a 100-member ensemble of CO2 flux estimates (Chau et al. 2022). The range of an estimate with the associated uncertainty is then defined by the empirical 68% interval computed from the ensemble.

**CONTEXT**

Since the onset of the industrial era in 1750, the atmospheric CO2 concentration has increased from about 277±3 ppm (Joos and Spahni, 2008) to 412.44±0.1 ppm in 2020 (Dlugokencky and Tans, 2020). By 2011, the ocean had absorbed approximately 28 ± 5% of all anthropogenic CO2 emissions, thus providing negative feedback to global warming and climate change (Ciais et al., 2013). The ocean CO2 sink is evaluated every year as part of the Global Carbon Budget (Friedlingstein et al. 2022). The uptake of CO2 occurs primarily in response to increasing atmospheric levels. The global flux is characterized by a significant variability on interannual to decadal time scales largely in response to natural climate variability (e.g., ENSO) (Friedlingstein et al. 2022, Chau et al. 2022). 

**CMEMS KEY FINDINGS**

The rate of change of the integrated yearly surface downward flux has increased by 0.04±0.03e-1 PgC/yr2 over the period 1985 to year-1. The yearly flux time series shows a plateau in the 90s followed by an increase since 2000 with a growth rate of 0.06±0.04e-1 PgC/yr2. In  2021 (resp. 2020), the global ocean CO2 sink was    2.41±0.13 (resp.  2.50±0.12) PgC/yr. The average over the full period is 1.61±0.10 PgC/yr with an interannual variability (temporal standard deviation) of 0.46 PgC/yr.  In order to compare these fluxes to Friedlingstein et al. (2022), the estimate of preindustrial outgassing of riverine carbon of  0.61 PgC/yr, which is in between the estimate by Jacobson et al. (2007) (0.45±0.18 PgC/yr) and the one by Resplandy et al. (2018) (0.78±0.41 PgC/yr) needs to be added. A full discussion regarding this OMI can be found in section 2.10 of the Ocean State Report 4 (Gehlen et al., 2020) and in Chau et al. (2022).

**DOI (product):** 
https://doi.org/10.48670/moi-00223

**References:**

* Chau, T. T. T., Gehlen, M., and Chevallier, F.: A seamless ensemble-based reconstruction of surface ocean pCO2 and air–sea CO2 fluxes over the global coastal and open oceans, Biogeosciences, 19, 1087–1109, https://doi.org/10.5194/bg-19-1087-2022, 2022.
* Ciais, P., Sabine, C., Govindasamy, B., Bopp, L., Brovkin, V., Canadell, J., Chhabra, A., DeFries, R., Galloway, J., Heimann, M., Jones, C., Le Quéré, C., Myneni, R., Piao, S., and Thorn- ton, P.: Chapter 6: Carbon and Other Biogeochemical Cycles, in: Climate Change 2013 The Physical Science Basis, edited by: Stocker, T., Qin, D., and Platner, G.-K., Cambridge University Press, Cambridge, 2013.
* Dlugokencky, E. and Tans, P.: Trends in atmospheric carbon dioxide, National Oceanic and Atmospheric Administration, Earth System Research Laboratory (NOAA/ESRL), http://www.esrl. noaa.gov/gmd/ccgg/trends/global.html, last access: 11 March 2022.
* Joos, F. and Spahni, R.: Rates of change in natural and anthropogenic radiative forcing over the past 20,000 years, P. Natl. Acad. Sci. USA, 105, 1425–1430, https://doi.org/10.1073/pnas.0707386105, 2008.
* Friedlingstein, P., Jones, M. W., O'Sullivan, M., Andrew, R. M., Bakker, D. C. E., Hauck, J., Le Quéré, C., Peters, G. P., Peters, W., Pongratz, J., Sitch, S., Canadell, J. G., Ciais, P., Jackson, R. B., Alin, S. R., Anthoni, P., Bates, N. R., Becker, M., Bellouin, N., Bopp, L., Chau, T. T. T., Chevallier, F., Chini, L. P., Cronin, M., Currie, K. I., Decharme, B., Djeutchouang, L. M., Dou, X., Evans, W., Feely, R. A., Feng, L., Gasser, T., Gilfillan, D., Gkritzalis, T., Grassi, G., Gregor, L., Gruber, N., Gürses, Ö., Harris, I., Houghton, R. A., Hurtt, G. C., Iida, Y., Ilyina, T., Luijkx, I. T., Jain, A., Jones, S. D., Kato, E., Kennedy, D., Klein Goldewijk, K., Knauer, J., Korsbakken, J. I., Körtzinger, A., Landschützer, P., Lauvset, S. K., Lefèvre, N., Lienert, S., Liu, J., Marland, G., McGuire, P. C., Melton, J. R., Munro, D. R., Nabel, J. E. M. S., Nakaoka, S.-I., Niwa, Y., Ono, T., Pierrot, D., Poulter, B., Rehder, G., Resplandy, L., Robertson, E., Rödenbeck, C., Rosan, T. M., Schwinger, J., Schwingshackl, C., Séférian, R., Sutton, A. J., Sweeney, C., Tanhua, T., Tans, P. P., Tian, H., Tilbrook, B., Tubiello, F., van der Werf, G. R., Vuichard, N., Wada, C., Wanninkhof, R., Watson, A. J., Willis, D., Wiltshire, A. J., Yuan, W., Yue, C., Yue, X., Zaehle, S., and Zeng, J.: Global Carbon Budget 2021, Earth Syst. Sci. Data, 14, 1917–2005, https://doi.org/10.5194/essd-14-1917-2022, 2022.
* Jacobson, A. R., Mikaloff Fletcher, S. E., Gruber, N., Sarmiento, J. L., and Gloor, M. (2007), A joint atmosphere-ocean inversion for surface fluxes of carbon dioxide: 1. Methods and global-scale fluxes, Global Biogeochem. Cycles, 21, GB1019, doi:10.1029/2005GB002556.
* Gehlen M., Thi Tuyet Trang Chau, Anna Conchon, Anna Denvil-Sommer, Frédéric Chevallier, Mathieu Vrac, Carlos Mejia (2020). Ocean acidification. In: Copernicus Marine Service Ocean State Report, Issue 4, Journal of Operational Oceanography, 13:sup1, s88–s91; DOI: 10.1080/1755876X.2020.1785097
* Resplandy, L., Keeling, R. F., Rödenbeck, C., Stephens, B. B., Khatiwala, S., Rodgers, K. B., Long, M. C., Bopp, L. and Tans, P. P.: Revision of global carbon fluxes based on a reassessment of oceanic and riverine carbon transport. Nature Geoscience, 11(7), p.504, 2018.





<img src="https://documentation.marine.copernicus.eu/IMG/GLOBAL_OMI_HEALTH_carbon_ph_area_averaged.png" width="300" height="200"/>



# Global Ocean acidification - mean sea water pH time series and trend from Multi-Observations Reprocessing


**Product ID:** GLOBAL_OMI_HEALTH_carbon_ph_area_averaged

**Description:** **DEFINITION**

Ocean acidification is quantified by decreases in pH, which is a measure of acidity: a decrease in pH value means an increase in acidity, that is, acidification. The observed decrease in ocean pH resulting from increasing concentrations of CO2 is an important indicator of global change. The estimate of global mean pH builds on a reconstruction methodology, 
* Obtain values for alkalinity based on the so called “locally interpolated alkalinity regression (LIAR)” method after Carter et al., 2016; 2018. 
* Build on surface ocean partial pressure of carbon dioxide (CMEMS product: MULTIOBS_GLO_BIO_CARBON_SURFACE_REP_015_008) obtained from an ensemble of Feed-Forward Neural Networks (Chau et al. 2022) which exploit sampling data gathered in the Surface Ocean CO2 Atlas (SOCAT) (https://www.socat.info/)
* Derive a gridded field of ocean surface pH based on the van Heuven et al., (2011) CO2 system calculations using reconstructed pCO2 (MULTIOBS_GLO_BIO_CARBON_SURFACE_REP_015_008) and alkalinity.
The global mean average of pH at yearly time steps is then calculated from the gridded ocean surface pH field. It is expressed in pH unit on total hydrogen ion scale. In the figure, the amplitude of the uncertainty (1σ  ) of yearly mean surface sea water pH varies at a range of (0.0023, 0.0029) pH unit (see Quality Information Document for more details). The trend and uncertainty estimates amount to -0.0017±0.0004e-1 pH units per year.
The indicator is derived from in situ observations of CO2 fugacity (SOCAT data base, www.socat.info, Bakker et al., 2016). These observations are still sparse in space and time.  Monitoring pH at higher space and time resolutions, as well as in coastal regions will require a denser network of observations and preferably direct pH measurements.  
A full discussion regarding this OMI can be found in section 2.10 of the Ocean State Report 4 (Gehlen et al., 2020).

**CONTEXT**

The decrease in surface ocean pH is a direct consequence of the uptake by the ocean of carbon dioxide. It is referred to as ocean acidification. The International Panel on Climate Change (IPCC) Workshop on Impacts of Ocean Acidification on Marine Biology and Ecosystems (2011) defined Ocean Acidification as “a reduction in the pH of the ocean over an extended period, typically decades or longer, which is caused primarily by uptake of carbon dioxide from the atmosphere, but can also be caused by other chemical additions or subtractions from the ocean”. The pH of contemporary surface ocean waters is already 0.1 lower than at pre-industrial times and an additional decrease by 0.33 pH units is projected over the 21st century in response to the high concentration pathway RCP8.5 (Bopp et al., 2013). Ocean acidification will put marine ecosystems at risk (e.g. Orr et al., 2005; Gehlen et al., 2011; Kroeker et al., 2013). The monitoring of surface ocean pH has become a focus of many international scientific initiatives (http://goa-on.org/) and constitutes one target for SDG14 (https://sustainabledevelopment.un.org/sdg14).  

**CMEMS KEY FINDINGS**

Since the year 1985, global ocean surface pH is decreasing at a rate of  -0.0017±0.0004e-1 per year. 

**DOI (product):** 
https://doi.org/10.48670/moi-00224

**References:**

* Bakker, D. et al.: A multi-decade record of high-quality fCO2 data in version 3 of the Surface Ocean CO2 Atlas (SOCAT), Earth Syst. Sci. Data, 8, 383-413, https://doi.org/10.5194/essd-8-383-2016, 2016.
* Bopp, L. et al.: Multiple stressors of ocean ecosystems in the 21st century: projections with CMIP5 models, Biogeosciences, 10, 6225–6245, doi: 10.5194/bg-10-6225-2013, 2013.
* Carter, B.R., et al.: Updated methods for global locally interpolated estimation of alkalinity, pH, and nitrate, Limnol. Oceanogr.: Methods 16, 119–131, 2018.
* Carter, B. R., et al.: Locally interpolated alkalinity regression for global alkalinity estimation. Limnol. Oceanogr.: Methods 14: 268–277. doi:10.1002/lom3.10087, 2016.
* Chau, T. T. T., Gehlen, M., and Chevallier, F.: A seamless ensemble-based reconstruction of surface ocean pCO2 and air–sea CO2 fluxes over the global coastal and open oceans, Biogeosciences, 19, 1087–1109, https://doi.org/10.5194/bg-19-1087-2022, 2022. Gehlen, M. et al.: Biogeochemical consequences of ocean acidification and feedback to the Earth system. p. 230, in: Gattuso J.-P. & Hansson L. (Eds.), Ocean acidification. Oxford: Oxford University Press., 2011.
* Gehlen M., Thi Tuyet Trang Chau, Anna Conchon, Anna Denvil-Sommer, Frédéric Chevallier, Mathieu Vrac, Carlos Mejia (2020). Ocean acidification. In: Copernicus Marine Service Ocean State Report, Issue 4, Journal of Operational Oceanography, 13:sup1, s88–s91; DOI: 10.1080/1755876X.2020.1785097
* IPCC, 2011: Workshop Report of the Intergovernmental Panel on Climate Change Workshop on Impacts of Ocean Acidification on Marine Biology and Ecosystems. [Field, C.B., V. Barros, T.F. Stocker, D. Qin, K.J. Mach, G.-K. Plattner, M.D. Mastrandrea, M. Tignor and K.L. Ebi (eds.)]. IPCC Working Group II Technical Support Unit, Carnegie Institution, Stanford, California, United States of America, pp.164.
* Kroeker, K. J. et al.: Meta- analysis reveals negative yet variable effects of ocean acidifica- tion on marine organisms, Ecol. Lett., 13, 1419–1434, 2010.
* Orr, J. C. et al.: Anthropogenic ocean acidification over the twenty-first century and its impact on cal- cifying organisms, Nature, 437, 681–686, 2005.
* van Heuven, S., et al.: MATLAB program developed for CO2 system calculations, ORNL/CDIAC-105b, Carbon Dioxide Inf. Anal. Cent., Oak Ridge Natl. Lab., US DOE, Oak Ridge, Tenn., 2011.





<img src="https://documentation.marine.copernicus.eu/IMG/GLOBAL_OMI_HEALTH_carbon_ph_trend.png" width="300" height="200"/>



# Global ocean acidification - mean sea water pH trend map from Multi-Observations Reprocessing


**Product ID:** GLOBAL_OMI_HEALTH_carbon_ph_trend

**Description:** **DEFINITION**

This ocean monitoring indicator (OMI) consists of annual mean rates of changes in surface ocean pH (yr-1) computed at 0.25°×0.25° resolution from 1985 until the last year. This indicator is derived from monthly pH time series distributed with the Copernicus Marine product MULTIOBS_GLO_BIO_CARBON_SURFACE_REP_015_008 (Chau et al., 2022a). For each grid cell, a linear least-squares regression was used to fit a linear function of pH versus time, where the slope (μ) and residual standard deviation (σ) are defined as estimates of the long-term trend and associated uncertainty. Finally, the estimates of pH associated with the highest uncertainty, i.e.,  σ-to-µ ratio over a threshold of 1  0%, are excluded from the global trend map (see QUID document for detailed description and method illustrations). This threshold is chosen at the 90th confidence level of all ratio values computed across the global ocean.

**CONTEXT**

A decrease in surface ocean pH (i.e., ocean acidification) is primarily a consequence of an increase in ocean uptake of atmospheric carbon dioxide (CO2) concentrations that have been augmented by anthropogenic emissions (Bates et al, 2014; Gattuso et al, 2015; Pérez et al, 2021).      As projected in Gattuso et al (2015), “under our current rate of emissions, most marine organisms evaluated will have very high risk of impacts by 2100 and many by 2050”. Ocean acidification is thus an ongoing source of concern due to its strong influence on marine ecosystems (e.g., Doney et al., 2009; Gehlen et al., 2011; Pörtner et al. 2019). Tracking changes in yearly mean values of surface ocean pH at the global scale has become an important indicator of both ocean acidification and global change (Gehlen et al., 2020; Chau et al., 2022b). In line with a sustained establishment of ocean measuring stations and thus a rapid increase in observations of ocean pH and other carbonate variables (e.g. dissolved inorganic carbon, total alkalinity, and CO2 fugacity) since the last decades (Bakker et al., 2016; Lauvset et al., 2021), recent studies including Bates et al (2014), Lauvset et al (2015), and Pérez et al (2021) put attention on analyzing secular trends of pH and their drivers from time-series stations to ocean basins. This OMI consists of the global maps of long-term pH trends and associated 1σ-uncertainty derived from the Copernicus Marine data-based product of monthly surface water pH (Chau et al., 2022a) at 0.25°×0.25° grid cells over the global ocean.

**CMEMS KEY FINDINGS**

Since 1985, pH has been decreasing at a rate between -0.0008 yr-1 and -0.0022 yr-1 over most of the global ocean basins. Tropical and subtropical regions, the eastern equatorial Pacific excepted, show pH trends falling in the interquartile range of all the trend estimates (between -0.0012 yr-1 and -0.0018 yr-1). pH over the eastern equatorial Pacific decreases much faster, reaching a growth rate larger than -0.0024 yr-1. Such a high rate of change in pH is also observed over a sector south of the Indian Ocean. Part of the polar and subpolar North Atlantic and the Southern Ocean has no significant trend.  

**DOI (product):** 
https://doi.org/10.48670/moi-00277

**References:**

* Bakker, D. C. E., Pfeil, B., Landa, C. S., Metzl, N., O'Brien, K. M., Olsen, A., Smith, K., Cosca, C., Harasawa, S., Jones, S. D., Nakaoka, S.-I. et al.: A multi-decade record of high-quality fCO2 data in version 3 of the Surface Ocean CO2 Atlas (SOCAT), Earth Syst. Sci. Data, 8, 383–413, DOI:10.5194/essd-8-383- 2016, 2016.
* Bates, N. R., Astor, Y. M., Church, M. J., Currie, K., Dore, J. E., Gonzalez-Davila, M., Lorenzoni, L., Muller-Karger, F., Olafsson, J., and Magdalena Santana-Casiano, J.: A Time-Series View of Changing Surface Ocean Chemistry Due to Ocean Uptake of Anthropogenic CO2 and Ocean Acidification, Oceanography, 27, 126–141, 2014.
* Chau, T. T. T., Gehlen, M., Chevallier, F. : Global Ocean Surface Carbon: MULTIOBS_GLO_BIO_CARBON_SURFACE_REP_015_008, E.U. Copernicus Marine Service Information, DOI:10.48670/moi-00047, 2022a.
* Chau, T. T. T., Gehlen, M., Chevallier, F.: Global mean seawater pH (GLOBAL_OMI_HEALTH_carbon_ph_area_averaged), E.U. Copernicus Marine Service Information, DOI: 10.48670/moi-00224, 2022b.
* Doney, S. C., Balch, W. M., Fabry, V. J., and Feely, R. A.: Ocean Acidification: A critical emerging problem for the ocean sciences, Oceanography, 22, 16–25, 2009.
* Gattuso, J-P., Alexandre Magnan, Raphaël Billé, William WL Cheung, Ella L. Howes, Fortunat Joos, Denis Allemand et al. ""Contrasting futures for ocean and society from different anthropogenic CO2 emissions scenarios."" Science 349, no. 6243 (2015).
* Gehlen, M. et al.: Biogeochemical consequences of ocean acidification and feedback to the Earth system. p. 230, in: Gattuso J.-P. & Hansson L. (Eds.), Ocean acidification. Oxford: Oxford University Press., 2011.
* Gehlen M., Chau T T T., Conchon A., Denvil-Sommer A., Chevallier F., Vrac M., Mejia C. : Ocean acidification. In: Copernicus Marine Service Ocean State Report, Issue 4, Journal of Operational Oceanography, 13:sup1, s88–s91; DOI:10.1080/1755876X.2020.1785097, 2020.
* Lauvset, S. K., Gruber, N., Landschützer, P., Olsen, A., and Tjiputra, J.: Trends and drivers in global surface ocean pH over the past 3 decades, Biogeosciences, 12, 1285–1298, DOI:10.5194/bg-12-1285-2015, 2015.
* Lauvset, S. K., Lange, N., Tanhua, T., Bittig, H. C., Olsen, A., Kozyr, A., Álvarez, M., Becker, S., Brown, P. J., Carter, B. R., Cotrim da Cunha, L., Feely, R. A., van Heuven, S., Hoppema, M., Ishii, M., Jeansson, E., Jutterström, S., Jones, S. D., Karlsen, M. K., Lo Monaco, C., Michaelis, P., Murata, A., Pérez, F. F., Pfeil, B., Schirnick, C., Steinfeldt, R., Suzuki, T., Tilbrook, B., Velo, A., Wanninkhof, R., Woosley, R. J., and Key, R. M.: An updated version of the global interior ocean biogeochemical data product, GLODAPv2.2021, Earth Syst. Sci. Data, 13, 5565–5589, DOI:10.5194/essd-13-5565-2021, 2021.
* Pörtner, H. O. et al. IPCC Special Report on the Ocean and Cryosphere in a Changing Climate (Wiley IPCC Intergovernmental Panel on Climate Change, Geneva, 2019).
* Pérez FF, Olafsson J, Ólafsdóttir SR, Fontela M, Takahashi T. Contrasting drivers and trends of ocean acidification in the subarctic Atlantic. Sci Rep 11, 13991, DOI:10.1038/s41598-021-93324-3, 2021.





<img src="https://documentation.marine.copernicus.eu/IMG/GLOBAL_OMI_NATLANTIC_amoc_26N_profile.png" width="300" height="200"/>



# Atlantic Meridional Overturning Circulation AMOC profile at 26N from Reanalysis


**Product ID:** GLOBAL_OMI_NATLANTIC_amoc_26N_profile

**Description:** **DEFINITION**

The Atlantic Meridional Overturning profile at 26.5N is obtained by integrating the meridional transport at 26.5 N across the Atlantic basin (zonally) and then doing a cumulative integral in depth. A climatological mean is then taken over time. This is done by using GLOBAL_MULTIYEAR_PHY_ENS_001_031 over the whole time period (1993-2023) and over the period for which there are comparable observations (Apr 2004-Mar 2023). The observations come from the RAPID array (Smeed et al, 2017).  

**CONTEXT**

The Atlantic Meridional Overturning Circulation (AMOC) transports heat northwards in the Atlantic and plays a key role in regional and global climate (Srokosz et al, 2012). There is a northwards transport in the upper kilometer resulting from northwards flow in the Gulf Stream and wind-driven Ekman transport, and southwards flow in the ocean interior and in deep western boundary currents (Srokosz et al, 2012). There are uncertainties in the deep profile associated with how much transport is returned in the upper (1-3km) or lower (3-5km) North Atlantic deep waters (Roberts et al 2013, Sinha et al 2018).

**CMEMS KEY FINDINGS** 

The AMOC strength at 1000m is found to be 17.0 ± 3.2 Sv (1 Sverdrup=106m3/s; range is 2 x standard deviation of multi-product). See also Jackson et al (2018).

Note: The key findings will be updated annually in November, in line with OMI evolutions.

**DOI (product):** 
https://doi.org/10.48670/moi-00231

**References:**

* Jackson, L., C. Dubois, S. Masina, A Storto and H Zuo, 2018: Atlantic Meridional Overturning Circulation. In Copernicus Marine Service Ocean State Report, Issue 2. Journal of Operational Oceanography , 11:sup1, S65-S66, 10.1080/1755876X.2018.1489208
* Roberts, C. D., J. Waters, K. A. Peterson, M. D. Palmer, G. D. McCarthy, E. Frajka‐Williams, K. Haines, D. J. Lea, M. J. Martin, D. Storkey, E. W. Blockley and H. Zuo (2013), Atmosphere drives recent interannual variability of the Atlantic meridional overturning circulation at 26.5°N, Geophys. Res. Lett., 40, 5164–5170 doi: 10.1002/grl.50930.
* Sinha, B., Smeed, D.A., McCarthy, G., Moat, B.I., Josey, S.A., Hirschi, J.J.-M., Frajka-Williams, E., Blaker, A.T., Rayner, D. and Madec, G. (2018), The accuracy of estimates of the overturning circulation from basin-wide mooring arrays. Progress in Oceanography, 160. 101-123
* Smeed D., McCarthy G., Rayner D., Moat B.I., Johns W.E., Baringer M.O. and Meinen C.S. (2017). Atlantic meridional overturning circulation observed by the RAPID-MOCHA-WBTS (RAPID-Meridional Overturning Circulation and Heatflux Array-Western Boundary Time Series) array at 26N from 2004 to 2017. British Oceanographic Data Centre - Natural Environment Research Council, UK. doi: 10.5285/5acfd143-1104-7b58-e053-6c86abc0d94b
* Srokosz, M., M. Baringer, H. Bryden, S. Cunningham, T. Delworth, S. Lozier, J. Marotzke, and R. Sutton, 2012: Past, Present, and Future Changes in the Atlantic Meridional Overturning Circulation. Bull. Amer. Meteor. Soc., 93, 1663–1676, https://doi.org/10.1175/BAMS-D-11-00151.1





<img src="https://documentation.marine.copernicus.eu/IMG/GLOBAL_OMI_NATLANTIC_amoc_max26N_timeseries.png" width="300" height="200"/>



# Atlantic Meridional Overturning Circulation AMOC timeseries at 26N from Reanalysis


**Product ID:** GLOBAL_OMI_NATLANTIC_amoc_max26N_timeseries

**Description:** **DEFINITION**

The Atlantic Meridional Overturning strength at 26.5N is obtained by integrating the meridional transport at 26.5 N across the Atlantic basin (zonally) and then doing a cumulative integral in depth by using GLOBAL_MULTIYEAR_PHY_ENS_001_031 . The maximum value in depth is then taken as the strength in Sverdrups (Sv=1x106m3/s). The observations come from the RAPID array (Smeed et al, 2017).

**CONTEXT**

The Atlantic Meridional Overturning Circulation (AMOC) transports heat northwards in the Atlantic and plays a key role in regional and global climate (Srokosz et al, 2012). There is a northwards transport in the upper kilometer resulting from northwards flow in the Gulf Stream and wind-driven Ekman transport, and southwards flow in the ocean interior and in deep western boundary currents (Srokosz et al, 2012). The observations have revealed variability at monthly to decadal timescales including a temporary weakening in 2009/10 (McCarthy et al, 2012) and a decrease from 2005-2012 (Smeed et al, 2014; Smeed et al, 2018). Other studies have suggested that this weakening may be a result of variability (Smeed et al, 2014; Jackson et al 2017).

**CMEMS KEY FINDINGS **

The AMOC strength exhibits significant variability on many timescales with a temporary weakening in 2009/10. There has been a weakening from 2005-2012 (-0.67 Sv/year, (p=0.03) in the observations and -0.53 Sv/year (p=0.04) in the multi-product mean). The multi-product suggests an earlier increase from 2001-2006 (0.48 Sv/yr, p=0.04), and a weakening in 1998-99, however before this period there is significant uncertainty. This indicates that the changes observed are likely to be variability rather than an ongoing trend (see also Jackson et al, 2018).

Note: The key findings will be updated annually in November, in line with OMI evolutions.

**DOI (product):** 
https://doi.org/10.48670/moi-00232

**References:**

* Jackson, L. C., Peterson, K. A., Roberts, C. D. & Wood, R. A. (2016). Recent slowing of Atlantic overturning circulation as a recovery from earlier strengthening. Nature Geosci, 9, 518—522
* Jackson, L., C. Dubois, S. Masina, A Storto and H Zuo, 2018: Atlantic Meridional Overturning Circulation. In Copernicus Marine Service Ocean State Report, Issue 2. Journal of Operational Oceanography , 11:sup1, S65-S66, 10.1080/1755876X.2018.1489208
* McCarthy, G., Frajka-Williams, E., Johns, W. E., Baringer, M. O., Meinen, C. S., Bryden, H. L., Rayner, D., Duchez, A., Roberts, C. & Cunningham, S. A. (2012). Observed interannual variability of the Atlantic meridional overturning circulation at 26.5°N. Geophys. Res. Lett., 39, L19609+
* Smeed, D. A., McCarthy, G. D., Cunningham, S. A., Frajka-Williams, E., Rayner, D., Johns, W. E., Meinen, C. S., Baringer, M. O., Moat, B. I., Duchez, A. & Bryden, H. L. (2014). Observed decline of the Atlantic meridional overturning circulation 2004&2012. Ocean Science, 10, 29--38.
* Smeed D., McCarthy G., Rayner D., Moat B.I., Johns W.E., Baringer M.O. and Meinen C.S. (2017). Atlantic meridional overturning circulation observed by the RAPID-MOCHA-WBTS (RAPID-Meridional Overturning Circulation and Heatflux Array-Western Boundary Time Series) array at 26N from 2004 to 2017. British Oceanographic Data Centre - Natural Environment Research Council, UK. doi: 10.5285/5acfd143-1104-7b58-e053-6c86abc0d94b
* Smeed, D. A., Josey, S. A., Beaulieu, C., Johns, W. E., Moat, B. I., Frajka-Williams, E., Rayner, D., Meinen, C. S., Baringer, M. O., Bryden, H. L. & McCarthy, G. D. (2018). The North Atlantic Ocean Is in a State of Reduced Overturning. Geophys. Res. Lett., 45, 2017GL076350+. doi: 10.1002/2017gl076350
* Srokosz, M., M. Baringer, H. Bryden, S. Cunningham, T. Delworth, S. Lozier, J. Marotzke, and R. Sutton, 2012: Past, Present, and Future Changes in the Atlantic Meridional Overturning Circulation. Bull. Amer. Meteor. Soc., 93, 1663–1676, https://doi.org/10.1175/BAMS-D-11-00151.1





<img src="https://documentation.marine.copernicus.eu/IMG/GLOBAL_OMI_OHC_area_averaged_anomalies_0_2000.png" width="300" height="200"/>



# Global Ocean Heat Content (0-2000m) time series and trend from Reanalysis & Multi-Observations Reprocessing


**Product ID:** GLOBAL_OMI_OHC_area_averaged_anomalies_0_2000

**Description:** **DEFINITION**

Estimates of Ocean Heat Content (OHC) are obtained from integrated differences of the measured temperature and a climatology along a vertical profile in the ocean (von Schuckmann et al., 2018). The regional OHC values are then averaged from 60°S-60°N aiming 
i)	to obtain the mean OHC as expressed in Joules per meter square (J/m2) to monitor the large-scale variability and change.
ii)	to monitor the amount of energy in the form of heat stored in the ocean (i.e. the change of OHC in time), expressed in Watt per square meter (W/m2). 
Ocean heat content is one of the six Global Climate Indicators recommended by the World Meterological Organisation for Sustainable Development Goal 13 implementation (WMO, 2017).

**CONTEXT**

Knowing how much and where heat energy is stored and released in the ocean is essential for understanding the contemporary Earth system state, variability and change, as the ocean shapes our perspectives for the future (von Schuckmann et al., 2020). Variations in OHC can induce changes in ocean stratification, currents, sea ice and ice shelfs (IPCC, 2019; 2021); they set time scales and dominate Earth system adjustments to climate variability and change (Hansen et al., 2011); they are a key player in ocean-atmosphere interactions and sea level change (WCRP, 2018) and they can impact marine ecosystems and human livelihoods (IPCC, 2019).

**CMEMS KEY FINDINGS**

Since the year 2005, the upper (0-2000m) near-global (60°S-60°N) ocean warms at a rate of 1.0 ± 0.1 W/m2. 

Note: The key findings will be updated annually in November, in line with OMI evolutions.

**DOI (product):** 
https://doi.org/10.48670/moi-00235

**References:**

* Hansen, J., Sato, M., Kharecha, P., & von Schuckmann, K. (2011). Earth’s energy imbalance and implications. Atmos. Chem. Phys., 11(24), 13421–13449. https://doi.org/10.5194/acp-11-13421-2011
* IPCC Special Report on the Ocean and Cryosphere in a Changing Climate. (2019). In H. O. Pörtner, D. C. Roberts, V. Masson-Delmotte, P. Zhai, M. Tignor, E. Poloczanska, K. Mintenbeck, A. Alegría, M. Nicolai, A. Okem, J. Petzold, B. Rama, & N. M. Weyer (Eds.), IPCC Intergovernmental Panel on Climate Change: Geneva, Switzerland. https://www.ipcc.ch/srocc/
* IPCC, 2021: Climate Change 2021: The Physical Science Basis. Contribution of Working Group I to the Sixth Assessment Report of the Intergovernmental Panel on Climate Change [Masson-Delmotte, V., P. Zhai, A. Pirani, S.L. Connors, C. Péan, S. Berger, N. Caud, Y. Chen, L. Goldfarb, M.I. Gomis, M. Huang, K. Leitzell, E. Lonnoy, J.B.R. Matthews, T.K. Maycock, T. Waterfield, O. Yelekçi, R. Yu, and B. Zhou (eds.)]. Cambridge University Press. In Press.
* von Schuckmann, K., A. Storto, S. Simoncelli, R. Raj, A. Samuelsen, A. de Pascual Collar, M. Garcia Sotillo, T. Szerkely, M. Mayer, D. Peterson, H. Zuo, G. Garric, M. Monier, 2018: Ocean heat content. In: Copernicus Marine Service Ocean State Report, Issue 2, Journal of Operational Oceanography, 11:sup1, s13–s16, DOI: 10.1080/1755876X.2018.1489208
* von Schuckmann, K., Cheng, L., Palmer, M. D., Tassone, C., Aich, V., Adusumilli, S., Beltrami, H., Boyer, T., Cuesta-Valero, F. J., Desbruyères, D., Domingues, C., García-García, A., Gentine, P., Gilson, J., Gorfer, M., Haimberger, L., Ishii, M., Johnson, G. C., Killik, R., … Wijffels, S. E. (2020). Heat stored in the Earth system: Where does the energy go? The GCOS Earth heat inventory team. Earth Syst. Sci. Data Discuss., 2020, 1–45. https://doi.org/10.5194/essd-2019-255
* von Schuckmann, K., & Le Traon, P.-Y. (2011). How well can we derive Global Ocean Indicators from Argo data? Ocean Sci., 7(6), 783–791. https://doi.org/10.5194/os-7-783-2011
* WCRP (2018). Global sea-level budget 1993–present. Earth Syst. Sci. Data, 10(3), 1551–1590. https://doi.org/10.5194/essd-10-1551-2018
* WMO, 2017: World Meterological Organisation Bulletin, 66(2), https://public.wmo.int/en/resources/bulletin.





<img src="https://documentation.marine.copernicus.eu/IMG/GLOBAL_OMI_OHC_area_averaged_anomalies_0_300.png" width="300" height="200"/>



# Global Ocean Heat Content (0-300m) from Reanalysis & Multi-Observations Reprocessing


**Product ID:** GLOBAL_OMI_OHC_area_averaged_anomalies_0_300

**Description:** **DEFINITION**

Estimates of Ocean Heat Content (OHC) are obtained from integrated differences of the measured temperature and a climatology along a vertical profile in the ocean (von Schuckmann et al., 2018). The regional OHC values are then averaged from 60°S-60°N aiming 
i)	to obtain the mean OHC as expressed in Joules per meter square (J/m2) to monitor the large-scale variability and change.
ii)	to monitor the amount of energy in the form of heat stored in the ocean (i.e. the change of OHC in time), expressed in Watt per square meter (W/m2). 
Ocean heat content is one of the six Global Climate Indicators recommended by the World Meterological Organisation for Sustainable Development Goal 13 implementation (WMO, 2017).

**CONTEXT**

Knowing how much and where heat energy is stored and released in the ocean is essential for understanding the contemporary Earth system state, variability and change, as the ocean shapes our perspectives for the future (von Schuckmann et al., 2020). Variations in OHC can induce changes in ocean stratification, currents, sea ice and ice shelfs (IPCC, 2019; 2021); they set time scales and dominate Earth system adjustments to climate variability and change (Hansen et al., 2011); they are a key player in ocean-atmosphere interactions and sea level change (WCRP, 2018) and they can impact marine ecosystems and human livelihoods (IPCC, 2019).

**CMEMS KEY FINDINGS**

Since the year 2005, the near-surface (0-300m) near-global (60°S-60°N) ocean warms at a rate of 0.4 ± 0.1 W/m2. 

Note: The key findings will be updated annually in November, in line with OMI evolutions.

**DOI (product):** 
https://doi.org/10.48670/moi-00233

**References:**

* Hansen, J., Sato, M., Kharecha, P., & von Schuckmann, K. (2011). Earth’s energy imbalance and implications. Atmos. Chem. Phys., 11(24), 13421–13449. https://doi.org/10.5194/acp-11-13421-2011
* IPCC Special Report on the Ocean and Cryosphere in a Changing Climate. (2019). In H. O. Pörtner, D. C. Roberts, V. Masson-Delmotte, P. Zhai, M. Tignor, E. Poloczanska, K. Mintenbeck, A. Alegría, M. Nicolai, A. Okem, J. Petzold, B. Rama, & N. M. Weyer (Eds.), IPCC Intergovernmental Panel on Climate Change: Geneva, Switzerland. https://www.ipcc.ch/srocc/
* IPCC, 2021: Climate Change 2021: The Physical Science Basis. Contribution of Working Group I to the Sixth Assessment Report of the Intergovernmental Panel on Climate Change [Masson-Delmotte, V., P. Zhai, A. Pirani, S.L. Connors, C. Péan, S. Berger, N. Caud, Y. Chen, L. Goldfarb, M.I. Gomis, M. Huang, K. Leitzell, E. Lonnoy, J.B.R. Matthews, T.K. Maycock, T. Waterfield, O. Yelekçi, R. Yu, and B. Zhou (eds.)]. Cambridge University Press. In Press.
* von Schuckmann, K., A. Storto, S. Simoncelli, R. Raj, A. Samuelsen, A. de Pascual Collar, M. Garcia Sotillo, T. Szerkely, M. Mayer, D. Peterson, H. Zuo, G. Garric, M. Monier, 2018: Ocean heat content. In: Copernicus Marine Service Ocean State Report, Issue 2, Journal of Operational Oceanography, 11:sup1, s13–s16, DOI: 10.1080/1755876X.2018.1489208
* von Schuckmann, K., Cheng, L., Palmer, M. D., Tassone, C., Aich, V., Adusumilli, S., Beltrami, H., Boyer, T., Cuesta-Valero, F. J., Desbruyères, D., Domingues, C., García-García, A., Gentine, P., Gilson, J., Gorfer, M., Haimberger, L., Ishii, M., Johnson, G. C., Killik, R., … Wijffels, S. E. (2020). Heat stored in the Earth system: Where does the energy go? The GCOS Earth heat inventory team. Earth Syst. Sci. Data Discuss., 2020, 1–45. https://doi.org/10.5194/essd-2019-255
* von Schuckmann, K., & Le Traon, P.-Y. (2011). How well can we derive Global Ocean Indicators from Argo data? Ocean Sci., 7(6), 783–791. https://doi.org/10.5194/os-7-783-2011
* WCRP (2018). Global sea-level budget 1993–present. Earth Syst. Sci. Data, 10(3), 1551–1590. https://doi.org/10.5194/essd-10-1551-2018
* WMO, 2017: World Meterological Organisation Bulletin, 66(2), https://public.wmo.int/en/resources/bulletin.





<img src="https://documentation.marine.copernicus.eu/IMG/GLOBAL_OMI_OHC_area_averaged_anomalies_0_700.png" width="300" height="200"/>



# Global Ocean Heat Content (0-700m) from Reanalysis & Multi-Observations Reprocessing


**Product ID:** GLOBAL_OMI_OHC_area_averaged_anomalies_0_700

**Description:** **DEFINITION**

Estimates of Ocean Heat Content (OHC) are obtained from integrated differences of the measured temperature and a climatology along a vertical profile in the ocean (von Schuckmann et al., 2018). The regional OHC values are then averaged from 60°S-60°N aiming 
i)	to obtain the mean OHC as expressed in Joules per meter square (J/m2) to monitor the large-scale variability and change.
ii)	to monitor the amount of energy in the form of heat stored in the ocean (i.e. the change of OHC in time), expressed in Watt per square meter (W/m2). 
Ocean heat content is one of the six Global Climate Indicators recommended by the World Meterological Organisation for Sustainable Development Goal 13 implementation (WMO, 2017).

**CONTEXT**

Knowing how much and where heat energy is stored and released in the ocean is essential for understanding the contemporary Earth system state, variability and change, as the ocean shapes our perspectives for the future (von Schuckmann et al., 2020). Variations in OHC can induce changes in ocean stratification, currents, sea ice and ice shelfs (IPCC, 2019; 2021); they set time scales and dominate Earth system adjustments to climate variability and change (Hansen et al., 2011); they are a key player in ocean-atmosphere interactions and sea level change (WCRP, 2018) and they can impact marine ecosystems and human livelihoods (IPCC, 2019).

**CMEMS KEY FINDINGS**

Since the year 2005, the upper (0-700m) near-global (60°S-60°N) ocean warms at a rate of 0.6 ± 0.1 W/m2. 

Note: The key findings will be updated annually in November, in line with OMI evolutions.

**DOI (product):** 
https://doi.org/10.48670/moi-00234

**References:**

* Hansen, J., Sato, M., Kharecha, P., & von Schuckmann, K. (2011). Earth’s energy imbalance and implications. Atmos. Chem. Phys., 11(24), 13421–13449. https://doi.org/10.5194/acp-11-13421-2011
* IPCC Special Report on the Ocean and Cryosphere in a Changing Climate. (2019). In H. O. Pörtner, D. C. Roberts, V. Masson-Delmotte, P. Zhai, M. Tignor, E. Poloczanska, K. Mintenbeck, A. Alegría, M. Nicolai, A. Okem, J. Petzold, B. Rama, & N. M. Weyer (Eds.), IPCC Intergovernmental Panel on Climate Change: Geneva, Switzerland. https://www.ipcc.ch/srocc/
* IPCC, 2021: Climate Change 2021: The Physical Science Basis. Contribution of Working Group I to the Sixth Assessment Report of the Intergovernmental Panel on Climate Change [Masson-Delmotte, V., P. Zhai, A. Pirani, S.L. Connors, C. Péan, S. Berger, N. Caud, Y. Chen, L. Goldfarb, M.I. Gomis, M. Huang, K. Leitzell, E. Lonnoy, J.B.R. Matthews, T.K. Maycock, T. Waterfield, O. Yelekçi, R. Yu, and B. Zhou (eds.)]. Cambridge University Press. In Press.
* von Schuckmann, K., A. Storto, S. Simoncelli, R. Raj, A. Samuelsen, A. de Pascual Collar, M. Garcia Sotillo, T. Szerkely, M. Mayer, D. Peterson, H. Zuo, G. Garric, M. Monier, 2018: Ocean heat content. In: Copernicus Marine Service Ocean State Report, Issue 2, Journal of Operational Oceanography, 11:sup1, s13–s16, DOI: 10.1080/1755876X.2018.1489208
* von Schuckmann, K., Cheng, L., Palmer, M. D., Tassone, C., Aich, V., Adusumilli, S., Beltrami, H., Boyer, T., Cuesta-Valero, F. J., Desbruyères, D., Domingues, C., García-García, A., Gentine, P., Gilson, J., Gorfer, M., Haimberger, L., Ishii, M., Johnson, G. C., Killik, R., … Wijffels, S. E. (2020). Heat stored in the Earth system: Where does the energy go? The GCOS Earth heat inventory team. Earth Syst. Sci. Data Discuss., 2020, 1–45. https://doi.org/10.5194/essd-2019-255
* von Schuckmann, K., & Le Traon, P.-Y. (2011). How well can we derive Global Ocean Indicators from Argo data? Ocean Sci., 7(6), 783–791. https://doi.org/10.5194/os-7-783-2011
* WCRP (2018). Global sea-level budget 1993–present. Earth Syst. Sci. Data, 10(3), 1551–1590. https://doi.org/10.5194/essd-10-1551-2018
* WMO, 2017: World Meterological Organisation Bulletin, 66(2), https://public.wmo.int/en/resources/bulletin.





<img src="https://documentation.marine.copernicus.eu/IMG/GLOBAL_OMI_OHC_trend.png" width="300" height="200"/>



# Global Ocean Heat Content trend map from Reanalysis & Multi-Observations Reprocessing


**Product ID:** GLOBAL_OMI_OHC_trend

**Description:** **DEFINITION**

Estimates of Ocean Heat Content (OHC) are obtained from integrated differences of the measured temperature and a climatology along a vertical profile in the ocean (von Schuckmann et al., 2018). The regional OHC values are then averaged from 60°S-60°N aiming 
i)	to obtain the mean OHC as expressed in Joules per meter square (J/m2) to monitor the large-scale variability and change.
ii)	to monitor the amount of energy in the form of heat stored in the ocean (i.e. the change of OHC in time), expressed in Watt per square meter (W/m2). 
Ocean heat content is one of the six Global Climate Indicators recommended by the World Meterological Organisation for Sustainable Development Goal 13 implementation (WMO, 2017).

**CONTEXT**

Knowing how much and where heat energy is stored and released in the ocean is essential for understanding the contemporary Earth system state, variability and change, as the ocean shapes our perspectives for the future (von Schuckmann et al., 2020). Variations in OHC can induce changes in ocean stratification, currents, sea ice and ice shelfs (IPCC, 2019; 2021); they set time scales and dominate Earth system adjustments to climate variability and change (Hansen et al., 2011); they are a key player in ocean-atmosphere interactions and sea level change (WCRP, 2018) and they can impact marine ecosystems and human livelihoods (IPCC, 2019).

**CMEMS KEY FINDINGS**

Regional trends for the period 2005-2019 from the Copernicus Marine Service multi-ensemble approach show warming at rates ranging from the global mean average up to more than 8 W/m2 in some specific regions (e.g. northern hemisphere western boundary current regimes). There are specific regions where a negative trend is observed above noise at rates up to about -5 W/m2 such as in the subpolar North Atlantic, or the western tropical Pacific. These areas are characterized by strong year-to-year variability (Dubois et al., 2018; Capotondi et al., 2020). 

Note: The key findings will be updated annually in November, in line with OMI evolutions.

**DOI (product):** 
https://doi.org/10.48670/moi-00236

**References:**

* Capotondi, A., Wittenberg, A.T., Kug, J.-S., Takahashi, K. and McPhaden, M.J. (2020). ENSO Diversity. In El Niño Southern Oscillation in a Changing Climate (eds M.J. McPhaden, A. Santoso and W. Cai). https://doi.org/10.1002/9781119548164.ch4
* Dubois et al., 2018 : Changes in the North Atlantic. Copernicus Marine Service Ocean State Report, Issue 2, Journal of Operational Oceanography, 11:sup1, s1–s142, DOI: 10.1080/1755876X.2018.1489208
* Hansen, J., Sato, M., Kharecha, P., & von Schuckmann, K. (2011). Earth’s energy imbalance and implications. Atmos. Chem. Phys., 11(24), 13421–13449. https://doi.org/10.5194/acp-11-13421-2011
* IPCC Special Report on the Ocean and Cryosphere in a Changing Climate. (2019). In H. O. Pörtner, D. C. Roberts, V. Masson-Delmotte, P. Zhai, M. Tignor, E. Poloczanska, K. Mintenbeck, A. Alegría, M. Nicolai, A. Okem, J. Petzold, B. Rama, & N. M. Weyer (Eds.), IPCC Intergovernmental Panel on Climate Change: Geneva, Switzerland. https://www.ipcc.ch/srocc/
* IPCC, 2021: Climate Change 2021: The Physical Science Basis. Contribution of Working Group I to the Sixth Assessment Report of the Intergovernmental Panel on Climate Change [Masson-Delmotte, V., P. Zhai, A. Pirani, S.L. Connors, C. Péan, S. Berger, N. Caud, Y. Chen, L. Goldfarb, M.I. Gomis, M. Huang, K. Leitzell, E. Lonnoy, J.B.R. Matthews, T.K. Maycock, T. Waterfield, O. Yelekçi, R. Yu, and B. Zhou (eds.)]. Cambridge University Press. In Press.
* von Schuckmann, K., A. Storto, S. Simoncelli, R. Raj, A. Samuelsen, A. de Pascual Collar, M. Garcia Sotillo, T. Szerkely, M. Mayer, D. Peterson, H. Zuo, G. Garric, M. Monier, 2018: Ocean heat content. In: Copernicus Marine Service Ocean State Report, Issue 2, Journal of Operational Oceanography, 11:sup1, s13–s16, DOI: 10.1080/1755876X.2018.1489208
* von Schuckmann, K., Cheng, L., Palmer, M. D., Tassone, C., Aich, V., Adusumilli, S., Beltrami, H., Boyer, T., Cuesta-Valero, F. J., Desbruyères, D., Domingues, C., García-García, A., Gentine, P., Gilson, J., Gorfer, M., Haimberger, L., Ishii, M., Johnson, G. C., Killik, R., … Wijffels, S. E. (2020). Heat stored in the Earth system: Where does the energy go? The GCOS Earth heat inventory team. Earth Syst. Sci. Data Discuss., 2020, 1–45. https://doi.org/10.5194/essd-2019-255
* WCRP (2018). Global sea-level budget 1993–present. Earth Syst. Sci. Data, 10(3), 1551–1590. https://doi.org/10.5194/essd-10-1551-2018
* WMO, 2017: World Meterological Organisation Bulletin, 66(2), https://public.wmo.int/en/resources/bulletin.





<img src="https://documentation.marine.copernicus.eu/IMG/GLOBAL_OMI_SL_thsl_area_averaged_anomalies_0_2000.png" width="300" height="200"/>



# Global Ocean Thermosteric Sea Level anomaly (0-2000m) time series and trend from Reanalysis & Multi-Observations Reprocessing


**Product ID:** GLOBAL_OMI_SL_thsl_area_averaged_anomalies_0_2000

**Description:** **DEFINITION**

The temporal evolution of thermosteric sea level in an ocean layer is obtained from an integration of temperature driven ocean density variations, which are subtracted from a reference climatology to obtain the fluctuations from an average field. The regional thermosteric sea level values are then averaged from 60°S-60°N aiming to monitor interannual to long term global sea level variations caused by temperature driven ocean volume changes through thermal expansion as expressed in meters (m). 

**CONTEXT**

The global mean sea level is reflecting changes in the Earth’s climate system in response to natural and anthropogenic forcing factors such as ocean warming, land ice mass loss and changes in water storage in continental river basins. Thermosteric sea-level variations result from temperature related density changes in sea water associated with volume expansion and contraction. Global thermosteric sea level rise caused by ocean warming is known as one of the major drivers of contemporary global mean sea level rise (Cazenave et al., 2018; Oppenheimer et al., 2019).

**CMEMS KEY FINDINGS**

Since the year 2005 the upper (0-2000m) near-global (60°S-60°N) thermosteric sea level rises at a rate of 1.3±0.2 mm/year. 

Note: The key findings will be updated annually in November, in line with OMI evolutions.

**DOI (product):**
https://doi.org/10.48670/moi-00240

**References:**

* Oppenheimer, M., B.C. Glavovic , J. Hinkel, R. van de Wal, A.K. Magnan, A. Abd-Elgawad, R. Cai, M. CifuentesJara, R.M. DeConto, T. Ghosh, J. Hay, F. Isla, B. Marzeion, B. Meyssignac, and Z. Sebesvari, 2019: Sea Level Rise and Implications for Low-Lying Islands, Coasts and Communities. In: IPCC Special Report on the Ocean and Cryosphere in a Changing Climate [H.-O. Pörtner, D.C. Roberts, V. Masson-Delmotte, P. Zhai, M. Tignor, E. Poloczanska, K. Mintenbeck, A. Alegría, M. Nicolai, A. Okem, J. Petzold, B. Rama, N.M. Weyer (eds.)]. In press.
* WCRP Global Sea Level Group, 2018: Global sea-level budget: 1993-present. Earth Syst. Sci. Data, 10, 1551-1590, https://doi.org/10.5194/essd-10-1551-2018.
* von Storto et al., 2018: Thermosteric Sea Level. In: Copernicus Marine Service Ocean State Report, Issue 2, Journal of Operational Oceanography, 11:sup1, s13–s16, DOI: 10.1080/1755876X.2018.1489208
* von Schuckmann, K., & Le Traon, P.-Y. (2011). How well can we derive Global Ocean Indicators from Argo data? Ocean Sci., 7(6), 783–791. https://doi.org/10.5194/os-7-783-2011





<img src="https://documentation.marine.copernicus.eu/IMG/GLOBAL_OMI_SL_thsl_area_averaged_anomalies_0_700.png" width="300" height="200"/>



# Global Ocean Thermosteric Sea Level anomaly (0-700m) time series and trend from Reanalysis & Multi-Observations Reprocessing


**Product ID:** GLOBAL_OMI_SL_thsl_area_averaged_anomalies_0_700

**Description:** **DEFINITION**

The temporal evolution of thermosteric sea level in an ocean layer is obtained from an integration of temperature driven ocean density variations, which are subtracted from a reference climatology to obtain the fluctuations from an average field. The regional thermosteric sea level values are then averaged from 60°S-60°N aiming to monitor interannual to long term global sea level variations caused by temperature driven ocean volume changes through thermal expansion as expressed in meters (m). 

**CONTEXT**

The global mean sea level is reflecting changes in the Earth’s climate system in response to natural and anthropogenic forcing factors such as ocean warming, land ice mass loss and changes in water storage in continental river basins. Thermosteric sea-level variations result from temperature related density changes in sea water associated with volume expansion and contraction. Global thermosteric sea level rise caused by ocean warming is known as one of the major drivers of contemporary global mean sea level rise (Cazenave et al., 2018; Oppenheimer et al., 2019).

**CMEMS KEY FINDINGS**

Since the year 2005 the upper (0-700m) near-global (60°S-60°N) thermosteric sea level rises at a rate of 0.9±0.1 mm/year. 

Note: The key findings will be updated annually in November, in line with OMI evolutions.

**DOI (product):**
https://doi.org/10.48670/moi-00239

**References:**

* Oppenheimer, M., B.C. Glavovic , J. Hinkel, R. van de Wal, A.K. Magnan, A. Abd-Elgawad, R. Cai, M. CifuentesJara, R.M. DeConto, T. Ghosh, J. Hay, F. Isla, B. Marzeion, B. Meyssignac, and Z. Sebesvari, 2019: Sea Level Rise and Implications for Low-Lying Islands, Coasts and Communities. In: IPCC Special Report on the Ocean and Cryosphere in a Changing Climate [H.-O. Pörtner, D.C. Roberts, V. Masson-Delmotte, P. Zhai, M. Tignor, E. Poloczanska, K. Mintenbeck, A. Alegría, M. Nicolai, A. Okem, J. Petzold, B. Rama, N.M. Weyer (eds.)]. In press.
* WCRP Global Sea Level Group, 2018: Global sea-level budget: 1993-present. Earth Syst. Sci. Data, 10, 1551-1590, https://doi.org/10.5194/essd-10-1551-2018.
* von Storto et al., 2018: Thermosteric Sea Level. In: Copernicus Marine Service Ocean State Report, Issue 2, Journal of Operational Oceanography, 11:sup1, s13–s16, DOI: 10.1080/1755876X.2018.1489208
* von Schuckmann, K., & Le Traon, P.-Y. (2011). How well can we derive Global Ocean Indicators from Argo data? Ocean Sci., 7(6), 783–791. https://doi.org/10.5194/os-7-783-2011





<img src="https://documentation.marine.copernicus.eu/IMG/GLOBAL_OMI_SL_thsl_trend.png" width="300" height="200"/>



# Global Ocean Thermosteric Sea Level trend map from Reanalysis & Multi-Observations Reprocessing


**Product ID:** GLOBAL_OMI_SL_thsl_trend

**Description:** **DEFINITION**

The temporal evolution of thermosteric sea level in an ocean layer is obtained from an integration of temperature driven ocean density variations, which are subtracted from a reference climatology to obtain the fluctuations from an average field. The regional thermosteric sea level values are then averaged from 60°S-60°N aiming to monitor interannual to long term global sea level variations caused by temperature driven ocean volume changes through thermal expansion as expressed in meters (m).

**CONTEXT**

Most of the interannual variability and trends in regional sea level is caused by changes in steric sea level. At mid and low latitudes, the steric sea level signal is essentially due to temperature changes, i.e. the thermosteric effect (Stammer et al., 2013, Meyssignac et al., 2016). Salinity changes play only a local role. Regional trends of thermosteric sea level can be significantly larger compared to their globally averaged versions (Storto et al., 2018). Except for shallow shelf sea and high latitudes (> 60° latitude), regional thermosteric sea level variations are mostly related to ocean circulation changes, in particular in the tropics where the sea level variations and trends are the most intense over the last two decades. 

**CMEMS KEY FINDINGS**

Significant (i.e. when the signal exceeds the noise) regional trends for the period 2005-2019 from the Copernicus Marine Service multi-ensemble approach show a thermosteric sea level rise at rates ranging from the global mean average up to more than 8 mm/year. There are specific regions where a negative trend is observed above noise at rates up to about -8 mm/year such as in the subpolar North Atlantic, or the western tropical Pacific. These areas are characterized by strong year-to-year variability (Dubois et al., 2018; Capotondi et al., 2020). 

Note: The key findings will be updated annually in November, in line with OMI evolutions.

**DOI (product):** 
https://doi.org/10.48670/moi-00241

**References:**

* Capotondi, A., Wittenberg, A.T., Kug, J.-S., Takahashi, K. and McPhaden, M.J. (2020). ENSO Diversity. In El Niño Southern Oscillation in a Changing Climate (eds M.J. McPhaden, A. Santoso and W. Cai). https://doi.org/10.1002/9781119548164.ch4
* Dubois et al., 2018 : Changes in the North Atlantic. Copernicus Marine Service Ocean State Report, Issue 2, Journal of Operational Oceanography, 11:sup1, s1–s142, DOI: 10.1080/1755876X.2018.1489208
* Storto et al., 2018: Thermosteric Sea Level. In: Copernicus Marine Service Ocean State Report, Issue 2, Journal of Operational Oceanography, 11:sup1, s13–s16, DOI: 10.1080/1755876X.2018.1489208
* Stammer D, Cazenave A, Ponte RM, Tamisiea ME (2013) Causes for contemporary regional sea level changes. Ann Rev Mar Sci 5:21–46. doi:10.1146/annurev-marine-121211-172406
* Meyssignac, B., C. G. Piecuch, C. J. Merchant, M.-F. Racault, H. Palanisamy, C. MacIntosh, S. Sathyendranath, R. Brewin, 2016: Causes of the Regional Variability in Observed Sea Level, Sea Surface Temperature and Ocean Colour Over the Period 1993–2011





<img src="https://documentation.marine.copernicus.eu/IMG/GLOBAL_OMI_TEMPSAL_Tyz_trend.png" width="300" height="200"/>



# Global Ocean Zonal Mean Subsurface Temperature cumulative trend from Multi-Observations Reprocessing


**Product ID:** GLOBAL_OMI_TEMPSAL_Tyz_trend

**Description:** **DEFINITION**

The linear change of zonal mean subsurface temperature over the period 1993-2019 at each grid point (in depth and latitude) is evaluated to obtain a global mean depth-latitude plot of subsurface temperature trend, expressed in °C.
The linear change is computed using the slope of the linear regression at each grid point scaled by the number of time steps (27 years, 1993-2019). A multi-product approach is used, meaning that the linear change is first computed for 5 different zonal mean temperature estimates. The average linear change is then computed, as well as the standard deviation between the five linear change computations. The evaluation method relies in the study of the consistency in between the 5 different estimates, which provides a qualitative estimate of the robustness of the indicator. See Mulet et al. (2018) for more details.

**CONTEXT**

Large-scale temperature variations in the upper layers are mainly related to the heat exchange with the atmosphere and surrounding oceanic regions, while the deeper ocean temperature in the main thermocline and below varies due to many dynamical forcing mechanisms (Bindoff et al., 2019). Together with ocean acidification and deoxygenation (IPCC, 2019), ocean warming can lead to dramatic changes in ecosystem assemblages, biodiversity, population extinctions, coral bleaching and infectious disease, change in behavior (including reproduction), as well as redistribution of habitat (e.g. Gattuso et al., 2015, Molinos et al., 2016, Ramirez et al., 2017). Ocean warming also intensifies tropical cyclones (Hoegh-Guldberg et al., 2018; Trenberth et al., 2018; Sun et al., 2017).

**CMEMS KEY FINDINGS**

The results show an overall ocean warming of the upper global ocean over the period 1993-2019, particularly in the upper 300m depth. In some areas, this warming signal reaches down to about 800m depth such as for example in the Southern Ocean south of 40°S. In other areas, the signal-to-noise ratio in the deeper ocean layers is less than two, i.e. the different products used for the ensemble mean show weak agreement. However, interannual-to-decadal fluctuations are superposed on the warming signal, and can interfere with the warming trend. For example, in the subpolar North Atlantic decadal variations such as the so called ‘cold event’ prevail (Dubois et al., 2018; Gourrion et al., 2018), and the cumulative trend over a quarter of a decade does not exceed twice the noise level below about 100m depth.

Note: The key findings will be updated annually in November, in line with OMI evolutions.

**DOI (product):** 
https://doi.org/10.48670/moi-00244

**References:**

* Dubois, C., K. von Schuckmann, S. Josey, A. Ceschin, 2018: Changes in the North Atlantic. In: Copernicus Marine Service Ocean State Report, Journal of Operational Oceanography, 11:sup1, S1-S142, DOI: 10.1080/1755876X.2018.1489208.
* Gattuso, J-P., et al. (2015): Contrasting futures for ocean and society from different anthropogenic CO2 emissions scenarios. Science 349, no. 6243.
* Gourrion, J., J. Deshayes, M. Juza, T. Szekely, J. Tontore, 2018: A persisting regional cold and fresh water anomaly in the Northern Atlantic. In: Copernicus Marine Service Ocean State Report, Journal of Operational Oceanography, 11:sup1, S1-S142, DOI: 10.1080/1755876X.2018.1489208.
* Hoegh-Guldberg, O., et al., 2018: Impacts of 1.5ºC Global Warming on Natural and Human Systems. In: Global Warming of 1.5°C. An IPCC Special Report on the impacts of global warming of 1.5°C above preindustrial levels and related global greenhouse gas emission pathways, in the context of strengthening the global response to the threat of climate change, sustainable development, and efforts to eradicate poverty [Masson-Delmotte, V., P. Zhai, H.-O. Pörtner, D. Roberts, J. Skea, P.R. Shukla, A. Pirani, W. Moufouma- Okia, C. Péan, R. Pidcock, S. Connors, J.B.R. Matthews, Y. Chen, X. Zhou, M.I. Gomis, E. Lonnoy, T. Maycock, M. Tignor, and T. Waterfield (eds.)]. In Press.
* IPCC, 2019: Summary for Policymakers. In: IPCC Special Report on the Ocean and Cryosphere in a Changing Climate [H.-O. Pörtner, D.C. Roberts, V. Masson-Delmotte, P. Zhai, M. Tignor, E. Poloczanska, K. Mintenbeck, A. Alegría, M. Nicolai, A. Okem, J. Petzold, B. Rama, N.M. Weyer (eds.)]. In press.
* Molinos, J.G., et al. (2016): Climate velocity and the future global redistribution of marine biodiversity, NATURE Climate Change 6 doi:10.10383/NCLIMATE2769.
* Mulet S, Buongiorno Nardelli B, Good S, A. Pisano A, E. Greiner, Monier M, 2018. Ocean temperature and salinity. In: Copernicus Marine Service Ocean State Report, Journal of Operational Oceanography, 11:sup1, S1-S142, DOI: 10.1080/1755876X.2018.1489208.
* Ramírez, F., I. Afán, L.S. Davis, and A. Chiaradia (2017): Climate impacts on global hot spots of marine biodiversity. Science Advances 3, no. 2 : e1601198.
* Sun, Y., Z. Zhong, T. Li, L. Yi, Y. Hu, H. Wan, H. Chen, Q. Liao, C. Ma and Q. Li, 2017: Impact of Ocean Warming on Tropical Cyclone Size and Its Destructiveness, Nature Scientific Reports, Volume 7 (8154), https://doi.org/10.1038/s41598-017-08533-6.
* Trenberth, K. E., L. J. Cheng, P. Jacobs, Y. X. Zhang, and J. Fasullo (2018): Hurricane Harvey links to ocean heat content and climate change adaptation. Earth's Future, 6, 730--744, https://doi.org/10.1029/2018EF000825.





<img src="https://documentation.marine.copernicus.eu/IMG/GLOBAL_OMI_TEMPSAL_sst_area_averaged_anomalies.png" width="300" height="200"/>



# Global Ocean Sea Surface Temperature time series and trend from Observations Reprocessing


**Product ID:** GLOBAL_OMI_TEMPSAL_sst_area_averaged_anomalies

**Description:** **DEFINITION**

Based on daily, global climate sea surface temperature (SST) analyses generated by the European Space Agency (ESA) SST Climate Change Initiative (CCI) and the Copernicus Climate Change Service (C3S) (Merchant et al., 2019; product SST-GLO-SST-L4-REP-OBSERVATIONS-010-024).  
Analysis of the data was based on the approach described in Mulet et al. (2018) and is described and discussed in Good et al. (2020). The processing steps applied were: 
1.	The daily analyses were averaged to create monthly means.  
2.	A climatology was calculated by averaging the monthly means over the period 1993 - 2014.  
3.	Monthly anomalies were calculated by differencing the monthly means and the climatology.  
4.	An area averaged time series was calculated by averaging the monthly fields over the globe, with each grid cell weighted according to its area.  
5.	The time series was passed through the X11 seasonal adjustment procedure, which decomposes the time series into a residual seasonal component, a trend component and errors (e.g., Pezzulli et al., 2005). The trend component is a filtered version of the monthly time series. 
6.	The slope of the trend component was calculated using a robust method (Sen 1968). The method also calculates the 95% confidence range in the slope.  

**CONTEXT**

Sea surface temperature (SST) is one of the Essential Climate Variables (ECVs) defined by the Global Climate Observing System (GCOS) as being needed for monitoring and characterising the state of the global climate system (GCOS 2010). It provides insight into the flow of heat into and out of the ocean, into modes of variability in the ocean and atmosphere, can be used to identify features in the ocean such as fronts and upwelling, and knowledge of SST is also required for applications such as ocean and weather prediction (Roquet et al., 2016).

**CMEMS KEY FINDINGS**

Over the period 1993 to 2021, the global average linear trend was 0.015 ± 0.001°C / year (95% confidence interval). 2021 is nominally the sixth warmest year in the time series. Aside from this trend, variations in the time series can be seen which are associated with changes between El Niño and La Niña conditions. For example, peaks in the time series coincide with the strong El Niño events that occurred in 1997/1998 and 2015/2016 (Gasparin et al., 2018).

**DOI (product):** 
https://doi.org/10.48670/moi-00242

**References:**

* GCOS. Global Climate Observing System. 2010. Update of the Implementation Plan for the Global Observing System for Climate in Support of the UNFCCC (GCO-138).
* Gasparin, F., von Schuckmann, K., Desportes, C., Sathyendranath, S. and Pardo, S. 2018. El Niño southern oscillation. In: Copernicus marine service ocean state report, issue 2. J Operat Oceanogr. 11(Sup1):s11–ss4. doi:10.1080/1755876X.2018.1489208.
* Good, S.A., Kennedy, J.J, and Embury, O. Global sea surface temperature anomalies in 2018 and historical changes since 1993. In: von Schuckmann et al. 2020, Copernicus Marine Service Ocean State Report, Issue 4, Journal of Operational Oceanography, 13:sup1, S1-S172, doi: 10.1080/1755876X.2020.1785097.
* Merchant, C.J., Embury, O., Bulgin, C.E. et al. Satellite-based time-series of sea-surface temperature since 1981 for climate applications. Sci Data 6, 223 (2019) doi:10.1038/s41597-019-0236-x. 
* Mulet S., Nardelli B.B., Good S., Pisano A., Greiner E., Monier M., Autret E., Axell L., Boberg F., Ciliberti S. 2018. Ocean temperature and salinity. In: Copernicus marine service ocean state report, issue 2. J Operat Oceanogr. 11(Sup1):s11–ss4. doi:10.1080/1755876X.2018.1489208.
* Pezzulli, S., Stephenson, D.B. and Hannachi A. 2005. The variability of seasonality. J Clim. 18: 71– 88, doi: 10.1175/JCLI-3256.1.
* Roquet H , Pisano A., Embury O. 2016. Sea surface temperature. In: von Schuckmann et al. 2016, The Copernicus marine environment monitoring service ocean state report. J Oper Ocean. 9(suppl. 2). doi:10.1080/1755876X.2016.1273446.
* Sen, P.K. 1968. Estimates of the regression coefficient based on Kendall’s tau. J Am Statist Assoc. 63: 1379– 1389, doi: 10.1080/01621459.1968.10480934.





<img src="https://documentation.marine.copernicus.eu/IMG/GLOBAL_OMI_TEMPSAL_sst_trend.png" width="300" height="200"/>



# Global Ocean Sea Surface Temperature trend map from Observations Reprocessing


**Product ID:** GLOBAL_OMI_TEMPSAL_sst_trend

**Description:** **DEFINITION**

Based on daily, global climate sea surface temperature (SST) analyses generated by the European Space Agency (ESA) SST Climate Change Initiative (CCI) and the Copernicus Climate Change Service (C3S) (Merchant et al., 2019; product SST-GLO-SST-L4-REP-OBSERVATIONS-010-024).  
Analysis of the data was based on the approach described in Mulet et al. (2018) and is described and discussed in Good et al. (2020). The processing steps applied were: 
1.	The daily analyses were averaged to create monthly means.  
2.	A climatology was calculated by averaging the monthly means over the period 1993 - 2014.  
3.	Monthly anomalies were calculated by differencing the monthly means and the climatology.  
4.	The time series for each grid cell was passed through the X11 seasonal adjustment procedure, which decomposes a time series into a residual seasonal component, a trend component and errors (e.g., Pezzulli et al., 2005). The trend component is a filtered version of the monthly time series. 
5.	The slope of the trend component was calculated using a robust method (Sen 1968). The method also calculates the 95% confidence range in the slope.  

**CONTEXT**

Sea surface temperature (SST) is one of the Essential Climate Variables (ECVs) defined by the Global Climate Observing System (GCOS) as being needed for monitoring and characterising the state of the global climate system (GCOS 2010). It provides insight into the flow of heat into and out of the ocean, into modes of variability in the ocean and atmosphere, can be used to identify features in the ocean such as fronts and upwelling, and knowledge of SST is also required for applications such as ocean and weather prediction (Roquet et al., 2016).

**CMEMS KEY FINDINGS**

Warming trends occurred over most of the globe between 1993 and 2021. One of the exceptions is the North Atlantic, which has a region south of Greenland where a cooling trend is found. The cooling in this area has been previously noted as occurring on centennial time scales (IPCC, 2013; Caesar et al., 2018; Sevellee et al., 2017).

**DOI (product):** 
https://doi.org/10.48670/moi-00243

**References:**

* Caesar, L., Rahmstorf, S., Robinson, A., Feulner, G. and Saba, V., 2018. Observed fingerprint of a weakening Atlantic Ocean overturning circulation. Nature, 556(7700), p.191. DOI: 10.1038/s41586-018-0006-5.
* GCOS. Global Climate Observing System. 2010. Update of the Implementation Plan for the Global Observing System for Climate in Support of the UNFCCC (GCO-138).
* Good, S.A., Kennedy, J.J, and Embury, O. Global sea surface temperature anomalies in 2018 and historical changes since 1993. In: von Schuckmann et al. 2020, Copernicus Marine Service Ocean State Report, Issue 4, Journal of Operational Oceanography, 13:sup1, S1-S172, doi: 10.1080/1755876X.2020.1785097.
* Merchant, C.J., Embury, O., Bulgin, C.E. et al. Satellite-based time-series of sea-surface temperature since 1981 for climate applications. Sci Data 6, 223 (2019) doi:10.1038/s41597-019-0236-x. 
* Mulet S., Nardelli B.B., Good S., Pisano A., Greiner E., Monier M., Autret E., Axell L., Boberg F., Ciliberti S. 2018. Ocean temperature and salinity. In: Copernicus marine service ocean state report, issue 2. J Operat Oceanogr. 11(Sup1):s11–ss4. doi:10.1080/1755876X.2018.1489208.
* IPCC, 2013: Climate Change 2013: The Physical Science Basis. Contribution of Working Group I to the Fifth Assessment Report of the Intergovernmental Panel on Climate Change [Stocker, T.F., D. Qin, G.-K. Plattner, M. Tignor, S.K. Allen, J. Boschung, A. Nauels, Y. Xia, V. Bex and P.M. Midgley (eds.)]. Cambridge University Press, Cambridge, United Kingdom and New York, NY, USA, 1535 pp.
* Pezzulli, S., Stephenson, D.B. and Hannachi A. 2005. The variability of seasonality. J Clim. 18: 71– 88, doi: 10.1175/JCLI-3256.1.
* Roquet H , Pisano A., Embury O. 2016. Sea surface temperature. In: von Schuckmann et al. 2016, The Copernicus marine environment monitoring service ocean state report. J Oper Ocean. 9(suppl. 2). doi:10.1080/1755876X.2016.1273446.
* Sen, P.K. 1968. Estimates of the regression coefficient based on Kendall’s tau. J Am Statist Assoc. 63: 1379– 1389, doi: 10.1080/01621459.1968.10480934.
* Sévellec, F., Fedorov, A.V. and Liu, W., 2017. Arctic sea-ice decline weakens the Atlantic meridional overturning circulation. Nature Climate Change, 7(8), p.604, doi: 10.1038/nclimate3353.





<img src="https://documentation.marine.copernicus.eu/IMG/GLOBAL_OMI_WMHE_heattrp.png" width="300" height="200"/>



# Mean Heat Transport across sections from Reanalysis


**Product ID:** GLOBAL_OMI_WMHE_heattrp

**Description:** **DEFINITION**

Heat transport across lines are obtained by integrating the heat fluxes along some selected sections and from top to bottom of the ocean. The values are computed from models’ daily output.
The mean value over a reference period (1993-2014) and over the last full year are provided for the ensemble product and the individual reanalysis, as well as the standard deviation for the ensemble product over the reference period (1993-2014). The values are given in PetaWatt (PW).

**CONTEXT**

The ocean transports heat and mass by vertical overturning and horizontal circulation, and is one of the fundamental dynamic components of the Earth’s energy budget (IPCC, 2013). There are spatial asymmetries in the energy budget resulting from the Earth’s orientation to the sun and the meridional variation in absorbed radiation which support a transfer of energy from the tropics towards the poles. However, there are spatial variations in the loss of heat by the ocean through sensible and latent heat fluxes, as well as differences in ocean basin geometry and current systems. These complexities support a pattern of oceanic heat transport that is not strictly from lower to high latitudes. Moreover, it is not stationary and we are only beginning to unravel its variability. 

**CMEMS KEY FINDINGS**

The mean transports estimated by the ensemble global reanalysis are comparable to estimates based on observations; the uncertainties on these integrated quantities are still large in all the available products. 

Note: The key findings will be updated annually in November, in line with OMI evolutions.

**DOI (product):** 
https://doi.org/10.48670/moi-00245

**References:**

* Lumpkin R, Speer K. 2007. Global ocean meridional overturning. J. Phys. Oceanogr., 37, 2550–2562, doi:10.1175/JPO3130.1.
* Madec G : NEMO ocean engine, Note du Pôle de modélisation, Institut Pierre-Simon Laplace (IPSL), France, No 27, ISSN No 1288-1619, 2008
* Bricaud C, Drillet Y, Garric G. 2016. Ocean mass and heat transport. In CMEMS Ocean State Report, Journal of Operational Oceanography, 9, http://dx.doi.org/10.1080/1755876X.2016.1273446





<img src="https://documentation.marine.copernicus.eu/IMG/GLOBAL_OMI_WMHE_northward_mht.png" width="300" height="200"/>



# Northward Heat Transport for Global Ocean, Atlantic and Indian+Pacific basins from Reanalysis


**Product ID:** GLOBAL_OMI_WMHE_northward_mht

**Description:** **DEFINITION**

Meridional Heat Transport is computed by integrating the heat fluxes along the zonal direction and from top to bottom of the ocean. 
They are given over 3 basins (Global Ocean, Atlantic Ocean and Indian+Pacific Ocean) and for all the grid points in the meridional grid of each basin. The mean value over a reference period (1993-2014) and over the last full year are provided for the ensemble product and the individual reanalysis, as well as the standard deviation for the ensemble product over the reference period (1993-2014). The values are given in PetaWatt (PW).

**CONTEXT**

The ocean transports heat and mass by vertical overturning and horizontal circulation, and is one of the fundamental dynamic components of the Earth’s energy budget (IPCC, 2013). There are spatial asymmetries in the energy budget resulting from the Earth’s orientation to the sun and the meridional variation in absorbed radiation which support a transfer of energy from the tropics towards the poles. However, there are spatial variations in the loss of heat by the ocean through sensible and latent heat fluxes, as well as differences in ocean basin geometry and current systems. These complexities support a pattern of oceanic heat transport that is not strictly from lower to high latitudes. Moreover, it is not stationary and we are only beginning to unravel its variability. 

**CMEMS KEY FINDINGS**

After an anusual 2016 year (Bricaud 2016), with a higher global meridional heat transport in the tropical band explained by, the increase of northward heat transport at 5-10 ° N in the Pacific Ocean during the El Niño event, 2017 northward heat transport is lower than the 1993-2014 reference value in the tropical band, for both Atlantic and Indian + Pacific Oceans. At the higher latitudes, 2017 northward heat transport is closed to 1993-2014 values.

Note: The key findings will be updated annually in November, in line with OMI evolutions.

**DOI (product):** 
https://doi.org/10.48670/moi-00246

**References:**

* Crosnier L, Barnier B, Treguier AM, 2001. Aliasing inertial oscillations in a 1/6° Atlantic circulation model: impact on the mean meridional heat transport. Ocean Modelling, vol 3, issues 1-2, pp21-31. https://doi.org/10.1016/S1463-5003(00)00015-9
* Ganachaud, A. , Wunsch C. 2003. Large-Scale Ocean Heat and Freshwater Transports during the World Ocean Circulation Experiment. J. Climate, 16, 696–705, https://doi.org/10.1175/1520-0442(2003)016<0696:LSOHAF>2.0.CO;2
* Lumpkin R, Speer K. 2007. Global ocean meridional overturning. J. Phys. Oceanogr., 37, 2550–2562, doi:10.1175/JPO3130.1.
* Madec G : NEMO ocean engine, Note du Pôle de modélisation, Institut Pierre-Simon Laplace (IPSL), France, No 27, ISSN No 1288-1619, 2008





<img src="https://documentation.marine.copernicus.eu/IMG/GLOBAL_OMI_WMHE_voltrp.png" width="300" height="200"/>



# Mean Volume Transport across sections from Reanalysis


**Product ID:** GLOBAL_OMI_WMHE_voltrp

**Description:** **DEFINITION**

Volume transport across lines are obtained by integrating the volume fluxes along some selected sections and from top to bottom of the ocean. The values are computed from models’ daily output.
The mean value over a reference period (1993-2014) and over the last full year are provided for the ensemble product and the individual reanalysis, as well as the standard deviation for the ensemble product over the reference period (1993-2014). The values are given in Sverdrup (Sv).

**CONTEXT**

The ocean transports heat and mass by vertical overturning and horizontal circulation, and is one of the fundamental dynamic components of the Earth’s energy budget (IPCC, 2013). There are spatial asymmetries in the energy budget resulting from the Earth’s orientation to the sun and the meridional variation in absorbed radiation which support a transfer of energy from the tropics towards the poles. However, there are spatial variations in the loss of heat by the ocean through sensible and latent heat fluxes, as well as differences in ocean basin geometry and current systems. These complexities support a pattern of oceanic heat transport that is not strictly from lower to high latitudes. Moreover, it is not stationary and we are only beginning to unravel its variability. 

**CMEMS KEY FINDINGS**

The mean transports estimated by the ensemble global reanalysis are comparable to estimates based on observations; the uncertainties on these integrated quantities are still large in all the available products. At Drake Passage, the multi-product approach (product no. 2.4.1) is larger than the value (130 Sv) of Lumpkin and Speer (2007), but smaller than the new observational based results of Colin de Verdière and Ollitrault, (2016) (175 Sv) and Donohue (2017) (173.3 Sv).

Note: The key findings will be updated annually in November, in line with OMI evolutions.

**DOI (product):** 
https://doi.org/10.48670/moi-00247

**References:**

* Lumpkin R, Speer K. 2007. Global ocean meridional overturning. J. Phys. Oceanogr., 37, 2550–2562, doi:10.1175/JPO3130.1.
* Madec G : NEMO ocean engine, Note du Pôle de modélisation, Institut Pierre-Simon Laplace (IPSL), France, No 27, ISSN No 1288-1619, 2008





<img src="https://mdl-metadata.s3.waw3-1.cloudferro.com/metadata/thumbnails/IBI_ANALYSISFORECAST_BGC_005_004.jpg" width="300" height="200"/>



# Atlantic-Iberian Biscay Irish- Ocean Biogeochemical Analysis and Forecast


**Product ID:** IBI_ANALYSISFORECAST_BGC_005_004

**Description:** The IBI-MFC provides a high-resolution biogeochemical analysis and forecast product covering the European waters, and more specifically the Iberia–Biscay–Ireland (IBI) area. The last 2 years before now (historic best estimates) as well as daily averaged forecasts with a horizon of 10 days (updated on a weekly basis) are available on the catalogue.
To this aim, an online coupled physical-biogeochemical operational system is based on NEMO-PISCES at 1/36° and adapted to the IBI area, being Mercator-Ocean in charge of the model code development. PISCES is a model of intermediate complexity, with 24 prognostic variables. It simulates marine biological productivity of the lower trophic levels and describes the biogeochemical cycles of carbon and of the main nutrients (P, N, Si, Fe).
The product provides daily and monthly averages of the main biogeochemical variables:  chlorophyll, oxygen, nitrate, phosphate, silicate, iron, ammonium, net primary production, euphotic zone depth, phytoplankton carbon, pH, dissolved inorganic carbon, surface partial pressure of carbon dioxide, zooplankton and light attenuation.

**DOI (Product)**: 
https://doi.org/10.48670/moi-00026

**References:**

* Gutknecht, E. and Reffray, G. and Mignot, A. and Dabrowski, T. and Sotillo, M. G. Modelling the marine ecosystem of Iberia-Biscay-Ireland (IBI) European waters for CMEMS operational applications. Ocean Sci., 15, 1489–1516, 2019. https://doi.org/10.5194/os-15-1489-2019





<img src="https://mdl-metadata.s3.waw3-1.cloudferro.com/metadata/thumbnails/IBI_ANALYSISFORECAST_PHY_005_001.jpg" width="300" height="200"/>



# Atlantic-Iberian Biscay Irish- Ocean Physics Analysis and Forecast


**Product ID:** IBI_ANALYSISFORECAST_PHY_005_001

**Description:** The IBI-MFC provides a high-resolution ocean analysis and forecast product (daily run  by Nologin with the support of CESGA in terms of supercomputing resources), covering the European waters, and more specifically the Iberia–Biscay–Ireland (IBI) area. The last 2 years before now (historic best estimates) as well as forecasts of different temporal resolutions with a horizon of 10 days (updated on a daily basis) are available on the catalogue.
The system is based on a eddy-resolving NEMO model application at 1/36º horizontal resolution, being Mercator-Ocean in charge of the model code development. The hydrodynamic forecast includes high frequency processes of paramount importance to characterize regional scale marine processes: tidal forcing, surges and high frequency atmospheric forcing, fresh water river discharge, wave forcing in forecast, etc. A weekly update of IBI downscaled analysis is also delivered as historic IBI best estimates.
The product offers 3D daily and monthly ocean fields, as well as hourly mean and 15-minute instantaneous values for some surface variables. Daily and monthly averages of 3D Temperature, 3D Salinity, 3D Zonal, Meridional and vertical Velocity components, Mix Layer Depth, Sea Bottom Temperature and Sea Surface Height are provided. Additionally, hourly means of surface fields for variables such as Sea Surface Height, Mix Layer Depth, Surface Temperature and Currents, together with Barotropic Velocities are delivered. Doodson-filtered detided mean sea level and horizontal surface currents are also provided. Finally, 15-minute instantaneous values of Sea Surface Height and Currents are also given.

**DOI (Product)**: 
https://doi.org/10.48670/moi-00027

**References:**

* Sotillo, M.G.; Campuzano, F.; Guihou, K.; Lorente, P.; Olmedo, E.; Matulka, A.; Santos, F.; Amo-Baladrón, M.A.; Novellino, A. River Freshwater Contribution in Operational Ocean Models along the European Atlantic Façade: Impact of a New River Discharge Forcing Data on the CMEMS IBI Regional Model Solution. J. Mar. Sci. Eng. 2021, 9, 401. https://doi.org/10.3390/jmse9040401
* Mason, E. and Ruiz, S. and Bourdalle-Badie, R. and Reffray, G. and García-Sotillo, M. and Pascual, A. New insight into 3-D mesoscale eddy properties from CMEMS operational models in the western Mediterranean. Ocean Sci., 15, 1111–1131, 2019. https://doi.org/10.5194/os-15-1111-2019
* Lorente, P. and García-Sotillo, M. and Amo-Baladrón, A. and Aznar, R. and Levier, B. and Sánchez-Garrido, J. C. and Sammartino, S. and de Pascual-Collar, Á. and Reffray, G. and Toledano, C. and Álvarez-Fanjul, E. Skill assessment of global, regional, and coastal circulation forecast models: evaluating the benefits of dynamical downscaling in IBI (Iberia-Biscay-Ireland) surface waters. Ocean Sci., 15, 967–996, 2019. https://doi.org/10.5194/os-15-967-2019
* Aznar, R., Sotillo, M. G., Cailleau, S., Lorente, P., Levier, B., Amo-Baladrón, A., Reffray, G., and Alvarez Fanjul, E. Strengths and weaknesses of the CMEMS forecasted and reanalyzed solutions for the Iberia-Biscay-Ireland (IBI) waters. J. Mar. Syst., 159, 1–14, https://doi.org/10.1016/j.jmarsys.2016.02.007, 2016
* Sotillo, M. G., Cailleau, S., Lorente, P., Levier, B., Reffray, G., Amo-Baladrón, A., Benkiran, M., and Alvarez Fanjul, E.: The MyOcean IBI Ocean Forecast and Reanalysis Systems: operational products and roadmap to the future Copernicus Service, J. Oper. Oceanogr., 8, 63–79, https://doi.org/10.1080/1755876X.2015.1014663, 2015.





<img src="https://mdl-metadata.s3.waw3-1.cloudferro.com/metadata/thumbnails/IBI_ANALYSISFORECAST_WAV_005_005.jpg" width="300" height="200"/>



# Atlantic-Iberian Biscay Irish- Ocean Wave Analysis and Forecast


**Product ID:** IBI_ANALYSISFORECAST_WAV_005_005

**Description:** The IBI-MFC provides a high-resolution wave analysis and forecast product (run twice a day by Nologin with the support of CESGA in terms of supercomputing resources), covering the European waters, and more specifically the Iberia–Biscay–Ireland (IBI) area. The last 2 years before now (historic best estimates), as well as hourly instantaneous forecasts with a horizon of up to 10 days (updated on a daily basis) are available on the catalogue.
The IBI wave model system is based on the MFWAM model and runs on a grid of 1/36º of horizontal resolution forced with the ECMWF hourly wind data. The system assimilates significant wave height (SWH) altimeter data and CFOSAT wave spectral data (supplied by Météo-France), and it is forced by currents provided by the IBI ocean circulation system. 
The product offers hourly instantaneous fields of different wave parameters, including Wave Height, Period and Direction for total spectrum; fields of Wind Wave (or wind sea), Primary Swell Wave and Secondary Swell for partitioned wave spectra; and the highest wave variables, such as maximum crest height and maximum crest-to-trough height. Additionally, the IBI wave system is set up to provide internally some key parameters adequate to be used as forcing in the IBI NEMO ocean model forecast run.

**DOI (Product)**: 
https://doi.org/10.48670/moi-00025

**References:**

* Toledano, C.; Ghantous, M.; Lorente, P.; Dalphinet, A.; Aouf, L.; Sotillo, M.G. Impacts of an Altimetric Wave Data Assimilation Scheme and Currents-Wave Coupling in an Operational Wave System: The New Copernicus Marine IBI Wave Forecast Service. J. Mar. Sci. Eng. 2022, 10, 457. https://doi.org/10.3390/jmse10040457





<img src="https://mdl-metadata.s3.waw3-1.cloudferro.com/metadata/thumbnails/IBI_MULTIYEAR_BGC_005_003.jpg" width="300" height="200"/>



# Atlantic-Iberian Biscay Irish- Ocean BioGeoChemistry NON ASSIMILATIVE Hindcast


**Product ID:** IBI_MULTIYEAR_BGC_005_003

**Description:** The IBI-MFC provides a biogeochemical reanalysis product for the Iberia-Biscay-Ireland (IBI) area starting in 01/01/1993 and being regularly updated on a yearly basis. The model system is run by Mercator-Ocean, being the product post-processed to the user’s format by Nologin with the support of CESGA in terms of supercomputing resources.
To this aim, an application of the biogeochemical model PISCES is run simultaneously with the ocean physical IBI reanalysis, generating both products at the same 1/12° horizontal resolution. The PISCES model is able to simulate the first levels of the marine food web, from nutrients up to mesozooplankton and it has 24 state variables.
The product provides daily, monthly and yearly averages of the main biogeochemical variables:  chlorophyll, oxygen, nitrate, phosphate, silicate, iron, ammonium, net primary production, euphotic zone depth, phytoplankton carbon, pH, dissolved inorganic carbon, zooplankton and surface partial pressure of carbon dioxide. Additionally, climatological parameters (monthly mean and standard deviation) of these variables for the period 1993-2016 are delivered.
For all the abovementioned variables new interim datasets will be provided to cover period till month - 4.

**DOI (Product)**: 
https://doi.org/10.48670/moi-00028

**References:**

* Aznar, R., Sotillo, M. G., Cailleau, S., Lorente, P., Levier, B., Amo-Baladrón, A., Reffray, G., and Alvarez Fanjul, E. Strengths and weaknesses of the CMEMS forecasted and reanalyzed solutions for the Iberia-Biscay-Ireland (IBI) waters. J. Mar. Syst., 159, 1–14, https://doi.org/10.1016/j.jmarsys.2016.02.007, 2016





<img src="https://mdl-metadata.s3.waw3-1.cloudferro.com/metadata/thumbnails/IBI_MULTIYEAR_PHY_005_002.jpg" width="300" height="200"/>



# Atlantic-Iberian Biscay Irish- Ocean Physics Reanalysis


**Product ID:** IBI_MULTIYEAR_PHY_005_002

**Description:** The IBI-MFC provides a ocean physical reanalysis product for the Iberia-Biscay-Ireland (IBI) area starting in 01/01/1993 and being regularly updated on a yearly basis. The model system is run by Mercator-Ocean, being the product post-processed to the user’s format by Nologin with the support of CESGA in terms of supercomputing resources. 
The IBI model numerical core is based on the NEMO v3.6 ocean general circulation model run at 1/12° horizontal resolution. Altimeter data, in situ temperature and salinity vertical profiles and satellite sea surface temperature are assimilated.
The product offers 3D daily, monthly and yearly ocean fields, as well as hourly mean fields for surface variables. Daily, monthly and yearly averages of 3D Temperature, 3D Salinity, 3D Zonal, Meridional and vertical Velocity components, Mix Layer Depth, Sea Bottom Temperature and Sea Surface Height are provided. Additionally, hourly means of surface fields for variables such as Sea Surface Height, Mix Layer Depth, Surface Temperature and Currents, together with Barotropic Velocities are distributed. Besides, daily means of air-sea fluxes are provided. Additionally, climatological parameters (monthly mean and standard deviation) of these variables for the period 1993-2016 are delivered. For all the abovementioned variables new interim datasets will be provided to cover period till month - 4.

**DOI (Product)**: 
https://doi.org/10.48670/moi-00029




<img src="https://mdl-metadata.s3.waw3-1.cloudferro.com/metadata/thumbnails/IBI_MULTIYEAR_WAV_005_006.jpg" width="300" height="200"/>



# Atlantic -Iberian Biscay Irish- Ocean Wave Reanalysis


**Product ID:** IBI_MULTIYEAR_WAV_005_006

**Description:** The IBI-MFC provides a high-resolution wave reanalysis product for the Iberia-Biscay-Ireland (IBI) area starting in 01/01/1980 and being regularly extended on a yearly basis. The model system is run by Nologin with the support of CESGA in terms of supercomputing resources. 
The Multi-Year model configuration is based on the MFWAM model developed by Météo-France (MF), covering the same region as the IBI-MFC Near Real Time (NRT) analysis and forecasting product and with the same horizontal resolution (1/36º). The system assimilates significant wave height (SWH) altimeter data and wave spectral data (Envisat and CFOSAT), supplied by MF. Both, the MY and the NRT products, are fed by ECMWF hourly winds. Specifically, the MY system is forced by the ERA5 reanalysis wind data. As boundary conditions, the NRT system uses the 2D wave spectra from the Copernicus Marine GLOBAL forecast system, whereas the MY system is nested to the GLOBAL reanalysis.
The product offers hourly instantaneous fields of different wave parameters, including Wave Height, Period and Direction for total spectrum; fields of Wind Wave (or wind sea), Primary Swell Wave and Secondary Swell for partitioned wave spectra; and the highest wave variables, such as maximum crest height and maximum crest-to-trough height. Besides, air-sea fluxes are provided. Additionally, climatological parameters of significant wave height (VHM0) and zero -crossing wave period (VTM02) are delivered for the time interval 1993-2016.

**DOI (Product)**: 
https://doi.org/10.48670/moi-00030




<img src="https://documentation.marine.copernicus.eu/IMG/IBI_OMI_CURRENTS_cui.png" width="300" height="200"/>



# Iberia Biscay Ireland Coastal Upwelling Index from Reanalysis


**Product ID:** IBI_OMI_CURRENTS_cui

**Description:** **DEFINITION**

The Coastal Upwelling Index (CUI) is computed along the African and the Iberian Peninsula coasts. For each latitudinal point from 27°N to 42°N the Coastal Upwelling Index is defined as the temperature difference between the maximum and minimum temperature in a range of distance from the coast up to 3.5º westwards.
〖CUI〗_lat=max⁡(T_lat )-min⁡(T_lat)
A high Coastal Upwelling Index indicates intense upwelling conditions.
The index is computed from the following Copernicus Marine products:
	IBI-MYP: IBI_MULTIYEAR_PHY_005_002 (1993-2019)
	IBI-NRT: IBI_ANALYSISFORECAST_PHYS_005_001 (2020 onwards)

**CONTEXT**

Coastal upwelling process occurs along coastlines as the result of deflection of the oceanic water away from the shore. Such deflection is produced by Ekman transport induced by persistent winds parallel to the coastline (Sverdrup, 1938). When this transported water is forced, the mass balance is maintained by pumping of ascending intermediate water. This water is typically denser, cooler and richer in nutrients. The Iberia-Biscay-Ireland domain contains two well-documented Eastern Boundary Upwelling Ecosystems, they are hosted under the same system known as Canary Current Upwelling System (Fraga, 1981; Hempel, 1982). This system is one of the major coastal upwelling regions of the world and it is produced by the eastern closure of the Subtropical Gyre. The North West African (NWA) coast presents an intense upwelling region that extends from Morocco to south of Senegal, likewise the western coast of the Iberian Peninsula (IP) shows a seasonal upwelling behavior. These two upwelling domains are separated by the presence of the Gulf of Cadiz, where the coastline does not allow the formation of upwelling conditions from 34ºN up to 37ºN.
The Copernicus Marine Service Coastal Upwelling Index is defined following the steps of several previous upwelling indices described in literature. More details and full scientific evaluation can be found in the dedicated section in the first issue of the Copernicus Marine Service Ocean State report (Sotillo et al., 2016).

**CMEMS KEY FINDINGS**

The NWA coast (latitudes below 34ºN) shows  a quite constantlow variability of the periodicity and the intensity of the upwelling, few periods of upwelling intensifications are found in years 1993-1995, and 2003-2004.
In the IP coast (latitudes higher than 37ºN) the interannual variability is more remarkable showing years with high upwelling activity (1994, 2004, and 2015-2017) followed by periods of lower activity (1996-1998, 2003, 2011, and 2013).
According to the results of the IBI-NRT system, 2020 was a year with weak upwelling in the IP latitudes. 
While in the NWA coast the upwelling activity was more intense than the average.
The analysis of trends in the period 1993-2019 shows significant positive trend of the upwelling index in the IP latitudes. This trend implies an increase of temperature differences between the coastal and offshore waters of approximately 0.02 °C/Year.

Note: The key findings will be updated annually in November, in line with OMI evolutions.

**DOI (product):** 
https://doi.org/10.48670/moi-00248

**References:**

* Fraga F. 1981. Upwelling off the Galician Coast, Northwest Spain. In: Richardson FA, editor. Coastal Upwelling. Washington (DC): Am Geoph Union; p. 176–182.
* Hempel G. 1982. The Canary current: studies of an upwelling system. Introduction. Rapp. Proc. Reun. Cons. Int. Expl. Mer., 180, 7–8.
* Sotillo MG, Levier B, Pascual A, Gonzalez A. 2016 Iberian-Biscay-Irish Sea. In von Scuckmann et al. (2016) The Copernicus Marine Environment Monitoring Service Ocean State Report, Journal of Operational Oceanography, 9:sup2, s235-s320, DOI: 10.1080/1755876X.2016.1273446
* Sverdrup HV. 1938. On the process of upwelling. J Mar Res. 1:155–164.





<img src="https://documentation.marine.copernicus.eu/IMG/IBI_OMI_SEASTATE_extreme_var_swh_mean_and_anomaly.png" width="300" height="200"/>



# Iberia Biscay Ireland Significant Wave Height extreme from Reanalysis


**Product ID:** IBI_OMI_SEASTATE_extreme_var_swh_mean_and_anomaly

**Description:** **DEFINITION**

The CMEMS IBI_OMI_seastate_extreme_var_swh_mean_and_anomaly OMI indicator is based on the computation of the annual 99th percentile of Significant Wave Height (SWH) from model data. Two different CMEMS products are used to compute the indicator: The Iberia-Biscay-Ireland Multi Year Product (IBI_MULTIYEAR_WAV_005_006) and the Analysis product (IBI_ANALYSIS_FORECAST_WAV_005_005).
Two parameters have been considered for this OMI:
•	Map of the 99th mean percentile: It is obtained from the Multi-Year Product, the annual 99th percentile is computed for each year of the product. The percentiles are temporally averaged in the whole period (1993-2021).
•	Anomaly of the 99th percentile in 2022: The 99th percentile of the year 2022 is computed from the Analysis product. The anomaly is obtained by subtracting the mean percentile to the percentile in 2022.
This indicator is aimed at monitoring the extremes of annual significant wave height and evaluate the spatio-temporal variability. The use of percentiles instead of annual maxima, makes this extremes study less affected by individual data. This approach was first successfully applied to sea level variable (Pérez Gómez et al., 2016) and then extended to other essential variables, such as sea surface temperature and significant wave height (Pérez Gómez et al 2018 and Álvarez-Fanjul et al., 2019). Further details and in-depth scientific evaluation can be found in the CMEMS Ocean State report (Álvarez- Fanjul et al., 2019).

**CONTEXT**

The sea state and its related spatio-temporal variability affect dramatically maritime activities and the physical connectivity between offshore waters and coastal ecosystems, impacting therefore on the biodiversity of marine protected areas (González-Marco et al., 2008; Savina et al., 2003; Hewitt, 2003). Over the last decades, significant attention has been devoted to extreme wave height events since their destructive effects in both the shoreline environment and human infrastructures have prompted a wide range of adaptation strategies to deal with natural hazards in coastal areas (Hansom et al., 2019). Complementarily, there is also an emerging question about the role of anthropogenic global climate change on present and future extreme wave conditions.
The Iberia-Biscay-Ireland region, which covers the North-East Atlantic Ocean from Canary Islands to Ireland, is characterized by two different sea state wave climate regions: whereas the northern half, impacted by the North Atlantic subpolar front, is of one of the world’s greatest wave generating regions (Mørk et al., 2010; Folley, 2017), the southern half,  located at subtropical latitudes, is by contrast influenced by persistent trade winds and thus by constant and moderate wave regimes.
The North Atlantic Oscillation (NAO), which refers to changes in the atmospheric sea level pressure difference between the Azores and Iceland, is a significant driver of wave climate variability in the Northern Hemisphere. The influence of North Atlantic Oscillation on waves along the Atlantic coast of Europe is particularly strong in and has a major impact on northern latitudes wintertime (Martínez-Asensio et al. 2016; Bacon and Carter, 1991; Bouws et al., 1996; Bauer, 2001; Wolf et al., 2002; Gleeson et al., 2017). Swings in the North Atlantic Oscillation index produce changes in the storms track and subsequently in the wind speed and direction over the Atlantic that alter the wave regime. When North Atlantic Oscillation index is in its positive phase, storms usually track northeast of Europe and enhanced westerly winds induce higher than average waves in the northernmost Atlantic Ocean. Conversely, in the negative North Atlantic Oscillation phase, the track of the storms is more zonal and south than usual, with trade winds (mid latitude westerlies) being slower and producing higher than average waves in southern latitudes (Marshall et al., 2001; Wolf et al., 2002; Wolf and Woolf, 2006).   
Additionally a variety of previous studies have uniquevocally determined the relationship between the sea state variability in the IBI region and other atmospheric climate modes such as the East Atlantic pattern, the Arctic Oscillation, the East Atlantic Western Russian pattern and the Scandinavian pattern (Izaguirre et al., 2011, Martínez-Asensio et al., 2016). 
In this context, long‐term statistical analysis of reanalyzed model data is mandatory not only to disentangle other driving agents of wave climate but also to attempt inferring any potential trend in the number and/or intensity of extreme wave events in coastal areas with subsequent socio-economic and environmental consequences.

**CMEMS KEY FINDINGS**

The climatic mean of 99th percentile (1993-2021) reveals a north-south gradient of Significant Wave Height with the highest values in northern latitudes (above 8m) and lowest values (2-3 m) detected southeastward of Canary Islands, in the seas between Canary Islands and the African Continental Shelf. This north-south pattern is the result of the two climatic conditions prevailing in the region and previously described.
The 99th percentile anomalies in 2023 show that during this period, the central latitudes of the domain (between 37 ºN and 50 ºN) were affected by extreme wave events that exceeded up to twice the standard deviation of the anomalies. These events impacted not only the open waters of the Northeastern Atlantic but also European coastal areas such as the west coast of Portugal, the Spanish Atlantic coast, and the French coast, including the English Channel.
Additionally, the impact of significant wave extremes exceeding twice the standard deviation of anomalies was detected in the Mediterranean region of the Balearic Sea and the Algerian Basin. This pattern is commonly associated with the impact of intense Tramontana winds originating from storms that cross the Iberian Peninsula from the Gulf of Biscay.

**Figure caption**

Iberia-Biscay-Ireland Significant Wave Height extreme variability: Map of the 99th mean percentile computed from the Multi Year Product (left panel) and anomaly of the 99th percentile in 2022 computed from the Analysis product (right panel). Transparent grey areas (if any) represent regions where anomaly exceeds the climatic standard deviation (light grey) and twice the climatic standard deviation (dark grey).

**DOI (product):** 
https://doi.org/10.48670/moi-00249

**References:**

* Álvarez Fanjul E, Pascual Collar A, Pérez Gómez B, De Alfonso M, García Sotillo M, Staneva J, Clementi E, Grandi A, Zacharioudaki A, Korres G, Ravdas M, Renshaw R, Tinker J, Raudsepp U, Lagemaa P, Maljutenko I, Geyer G, Müller M, Çağlar Yumruktepe V. Sea level, sea surface temperature and SWH extreme percentiles: combined analysis from model results and in situ observations, Section 2.7, p:31. In: Schuckmann K, Le Traon P-Y, Smith N, Pascual A, Djavidnia S, Gattuso J-P, Grégoire M, Nolan G, et al. 2019. Copernicus Marine Service Ocean State Report, Issue 3, Journal of Operational Oceanography, 12:sup1, S1-S123, DOI: 10.1080/1755876X.2019.1633075
* Bacon S, Carter D J T. 1991. Wave climate changes in the north Atlantic and North Sea, International Journal of Climatology, 11, 545–558.
* Bauer E. 2001. Interannual changes of the ocean wave variability in the North Atlantic and in the North Sea, Climate Research, 18, 63–69.
* Bouws E, Jannink D, Komen GJ. 1996. The increasing wave height in the North Atlantic Ocean, Bull. Am. Met. Soc., 77, 2275–2277.
* Folley M. 2017. The wave energy resource. In Pecher A, Kofoed JP (ed.), Handbook of Ocean Wave Energy, Ocean Engineering & Oceanography 7, doi:10.1007/978-3-319-39889-1_3.
* Gleeson E, Gallagher S, Clancy C, Dias F. 2017. NAO and extreme ocean states in the Northeast Atlantic Ocean, Adv. Sci. Res., 14, 23–33, doi:10.5194/asr-14-23-2017.
* González-Marco D, Sierra J P, Ybarra O F, Sánchez-Arcilla A. 2008. Implications of long waves in harbor management: The Gijón port case study. Ocean & Coastal Management, 51, 180-201. doi:10.1016/j.ocecoaman.2007.04.001.
* Hanson et al., 2015. Extreme Waves: Causes, Characteristics and Impact on Coastal Environments and Society January 2015 In book: .Coastal and Marine Hazards, Risks, and Disasters Edition: Hazards and Disasters Series, Elsevier Major Reference Works Chapter: Chapter 11: Extreme Waves: Causes, Characteristics and Impact on Coastal Environments and Society. Publisher: Elsevier Editors: Ellis, J and Sherman, D. J.
* Hewit J E, Cummings V J, Elis J I, Funnell G, Norkko A, Talley T S, Thrush S.F. 2003. The role of waves in the colonisation of terrestrial sediments deposited in the marine environment. Journal of Experimental marine Biology and Ecology, 290, 19-47, doi:10.1016/S0022-0981(03)00051-0.
* Izaguirre C, Méndez F J, Menéndez M, Losada I J. 2011. Global extreme wave height variability based on satellite data Cristina. Geoph. Res. Letters, Vol. 38, L10607, doi: 10.1029/2011GL047302.
* Martínez-Asensio A, Tsimplis M N, Marcos M, Feng F, Gomis D, Jordàa G, Josey S A. 2016. Response of the North Atlantic wave climate to atmospheric modes of variability. International Journal of Climatology, 36, 1210–1225, doi: 10.1002/joc.4415.
* Mørk G, Barstow S, Kabush A, Pontes MT. 2010. Assessing the global wave energy potential. Proceedings of OMAE2010 29th International Conference on Ocean, Offshore Mechanics and Arctic Engineering June 6-11, 2010, Shanghai, China.
* Pérez-Gómez B, Álvarez-Fanjul E, She J, Pérez-González I, Manzano F. 2016. Extreme sea level events, Section 4.4, p:300. In: Von Schuckmann K, Le Traon PY, Alvarez-Fanjul E, Axell L, Balmaseda M, Breivik LA, Brewin RJW, Bricaud C, Drevillon M, Drillet Y, Dubois C , Embury O, Etienne H, García-Sotillo M, Garric G, Gasparin F, Gutknecht E, Guinehut S, Hernandez F, Juza M, Karlson B, Korres G, Legeais JF, Levier B, Lien VS, Morrow R, Notarstefano G, Parent L, Pascual A, Pérez-Gómez B, Perruche C, Pinardi N, Pisano A, Poulain PM , Pujol IM, Raj RP, Raudsepp U, Roquet H, Samuelsen A, Sathyendranath S, She J, Simoncelli S, Solidoro C, Tinker J, Tintoré J, Viktorsson L, Ablain M, Almroth-Rosell E, Bonaduce A, Clementi E, Cossarini G, Dagneaux Q, Desportes C, Dye S, Fratianni C, Good S, Greiner E, Gourrion J, Hamon M, Holt J, Hyder P, Kennedy J, Manzano-Muñoz F, Melet A, Meyssignac B, Mulet S, Nardelli BB, O’Dea E, Olason E, Paulmier A, Pérez-González I, Reid R, Racault MF, Raitsos DE, Ramos A, Sykes P, Szekely T, Verbrugge N. 2016. The Copernicus Marine Environment Monitoring Service Ocean State Report, Journal of Operational Oceanography. 9 (sup2): 235-320. http://dx.doi.org/10.1080/1755876X.2016.1273446
* Pérez Gómez B., De Alfonso M., Zacharioudaki A., Pérez González I., Álvarez Fanjul E., Müller M., Marcos M., Manzano F., Korres G., Ravdas M., Tamm S. 2018. Sea level, SST and waves: extremes variability. In: Copernicus Marine Service Ocean State Report, Issue 2, Journal of Operational Oceanography, 11:sup1, Chap. 3.1, s79–s88, DOI: https://doi.org/10.1080/1755876X.2018.1489208
* Savina H, Lefevre J-M, Josse P, Dandin P. 2003. Definition of warning criteria. Proceedings of MAXWAVE Final Meeting, October 8-11, Geneva, Switzerland.
* Woolf D K, Challenor P G, Cotton P D. 2002. Variability and predictability of the North Atlantic wave climate, J. Geophys. Res., 107(C10), 3145, doi:10.1029/2001JC001124.
* Wolf J, Woolf D K. 2006. Waves and climate change in the north-east Atlantic. Geophysical Research Letters, Vol. 33, L06604, doi: 10.1029/2005GL025113.
* Young I R, Ribal A. 2019. Multiplatform evaluation of global trends in wind speed and wave height. Science, 364, 548-552, doi: 10.1126/science.aav9527.
* Kushnir Y, Cardone VJ, Greenwood JG, Cane MA. 1997. The recent increase in North Atlantic wave heights. Journal of Climate 10:2107–2113.
* Marshall, J., Kushnir, Y., Battisti, D., Chang, P., Czaja, A., Dickson, R., ... & Visbeck, M. (2001). North Atlantic climate variability: phenomena, impacts and mechanisms. International Journal of Climatology: A Journal of the Royal Meteorological Society, 21(15), 1863-1898.





<img src="https://documentation.marine.copernicus.eu/IMG/IBI_OMI_SEASTATE_swi.png" width="300" height="200"/>



# Iberia Biscay Ireland Strong Wave Incidence index from Reanalysis


**Product ID:** IBI_OMI_SEASTATE_swi

**Description:** **DEFINITION**

The Strong Wave Incidence index is proposed to quantify the variability of strong wave conditions in the Iberia-Biscay-Ireland regional seas. The anomaly of exceeding a threshold of Significant Wave Height is used to characterize the wave behavior. A sensitivity test of the threshold has been performed evaluating the differences using several ones (percentiles 75, 80, 85, 90, and 95). From this indicator, it has been chosen the 90th percentile as the most representative, coinciding with the state-of-the-art.
Two CMEMS products are used to compute the Strong Wave Incidence index:
•	IBI-WAV-MYP: IBI_REANALYSIS_WAV_005_006
•	IBI-WAV-NRT: IBI_ANALYSIS_FORECAST_WAV_005_005
The Strong Wave Incidence index (SWI) is defined as the difference between the climatic frequency of exceedance (Fclim) and the observational frequency of exceedance (Fobs) of the threshold defined by the 90th percentile (ThP90) of Significant Wave Height (SWH) computed on a monthly basis from hourly data of IBI-WAV-MYP product:
SWI = Fobs(SWH > ThP90) – Fclim(SWH > ThP90)
Since the Strong Wave Incidence index is defined as a difference of a climatic mean and an observed value, it can be considered an anomaly. Such index represents the percentage that the stormy conditions have occurred above/below the climatic average. Thus, positive/negative values indicate the percentage of hourly data that exceed the threshold above/below the climatic average, respectively.

**CONTEXT**

Ocean waves have a high relevance over the coastal ecosystems and human activities. Extreme wave events can entail severe impacts over human infrastructures and coastal dynamics. However, the incidence of severe (90th percentile) wave events also have valuable relevance affecting the development of human activities and coastal environments. The Strong Wave Incidence index based on the CMEMS regional analysis and reanalysis product provides information on the frequency of severe wave events.
The IBI-MFC covers the Europe’s Atlantic coast in a region bounded by the 26ºN and 56ºN parallels, and the 19ºW and 5ºE meridians. The western European coast is located at the end of the long fetch of the subpolar North Atlantic (Mørk et al., 2010), one of the world’s greatest wave generating regions (Folley, 2017). Several studies have analyzed changes of the ocean wave variability in the North Atlantic Ocean (Bacon and Carter, 1991; Kursnir et al., 1997; WASA Group, 1998; Bauer, 2001; Wang and Swail, 2004; Dupuis et al., 2006; Wolf and Woolf, 2006; Dodet et al., 2010; Young et al., 2011; Young and Ribal, 2019). The observed variability is composed of fluctuations ranging from the weather scale to the seasonal scale, together with long-term fluctuations on interannual to decadal scales associated with large-scale climate oscillations. Since the ocean surface state is mainly driven by wind stresses, part of this variability in Iberia-Biscay-Ireland region is connected to the North Atlantic Oscillation (NAO) index (Bacon and Carter, 1991; Hurrell, 1995;  Bouws et al., 1996, Bauer, 2001; Woolf et al., 2002; Tsimplis et al., 2005; Gleeson et al., 2017). However, later studies have quantified the relationships between the wave climate and other atmospheric climate modes such as the East Atlantic pattern, the Arctic Oscillation pattern, the East Atlantic Western Russian pattern and the Scandinavian pattern (Izaguirre et al., 2011, Matínez-Asensio et al., 2016).
The Strong Wave Incidence index provides information on incidence of stormy events in four monitoring regions in the IBI domain. The selected monitoring regions (Figure 1.A) are aimed to provide a summarized view of the diverse climatic conditions in the IBI regional domain: Wav1 region monitors the influence of stormy conditions in the West coast of Iberian Peninsula, Wav2 region is devoted to monitor the variability of stormy conditions in the Bay of Biscay, Wav3 region is focused in the northern half of IBI domain, this region is strongly affected by the storms transported by the subpolar front, and Wav4 is focused in the influence of marine storms in the North-East African Coast, the Gulf of Cadiz and Canary Islands.
More details and a full scientific evaluation can be found in the CMEMS Ocean State report (Pascual et al., 2020).

**CMEMS KEY FINDINGS**

The analysis of the index in the last decades do not show significant trends of the strong wave conditions over the period 1992-2021 with 99% confidence. The maximum wave event reported in region WAV1 (B) occurred in February 2014, producing an increment of 25% of strong wave conditions in the region. Two maximum wave events are found in WAV2 (C) with an increment of 15% of high wave conditions in November 2009 and February 2014. As in regions WAV1 and WAV2, in the region WAV3 (D), a strong wave event took place in February 2014, this event is one of the maximum events reported in the region with an increment of strong wave conditions of 20%, two months before (December 2013) there was a storm of similar characteristics affecting this region, other events of similar magnitude are detected on October 2000 and November 2009. The region WAV4 (E) present its maximum wave event in January 1996, such event produced a 25% of increment of strong wave conditions in the region. Despite of each monitoring region is affected by independent wave events; the analysis shows several past higher-than-average wave events that were propagated though several monitoring regions: November-December 2010 (WAV3 and WAV2); February 2014 (WAV1, WAV2, and WAV3); and February-March 2018 (WAV1 and WAV4).
The analysis of the NRT period (January 2022 onwards) depicts a significant event that occurred in November 2022, which affected the WAV2 and WAV3 regions, resulting in a 15% and 25% increase in maximum wave conditions, respectively. In the case of the WAV3 region, this event was the strongest event recorded in this region.
In the WAV4 region, an event that occurred in February 2024 was the second most intense on record, showing an 18% increase in strong wave conditions in the region.
In the WAV1 region, the NRT period includes two high-intensity events that occurred in February 2024 (21% increase in strong wave conditions) and April 2022 (18% increase in maximum wave conditions).

**Figure caption**

(A) Mean 90th percentile of Sea Wave Height computed from IBI_REANALYSIS_WAV_005_006 product at an hourly basis. Gray dotted lines denote the four monitoring areas where the Strong Wave Incidence index is computed. (B, C, D, and E) Strong Wave Incidence index averaged in monitoring regions WAV1 (A), WAV2 (B), WAV3 (C), and WAV4 (D). Panels show merged results of two CMEMS products: IBI_REANALYSIS_WAV_005_006 (blue), IBI_ANALYSIS_FORECAST_WAV_005_005 (orange). The trend and 99% confidence interval of IBI-MYP product is included (bottom right).

**DOI (product):** 
https://doi.org/10.48670/moi-00251

**References:**

* Bacon S, Carter D J T. 1991. Wave climate changes in the north Atlantic and North Sea, International Journal of Climatology, 11, 545–558.
* Bauer E. 2001. Interannual changes of the ocean wave variability in the North Atlantic and in the North Sea, Climate Research, 18, 63–69.
* Bouws E, Jannink D, Komen GJ. 1996. The increasing wave height in the North Atlantic Ocean, Bull. Am. Met. Soc., 77, 2275–2277.
* Dodet G, Bertin X, Taborda R. 2010. Wave climate variability in the North-East Atlantic Ocean over the last six decades, Ocean Modelling, 31, 120–131.
* Dupuis H, Michel D, Sottolichio A. 2006. Wave climate evolution in the Bay of Biscay over two decades. Journal of Marine Systems, 63, 105–114.
* Folley M. 2017. The wave energy resource. In Pecher A, Kofoed JP (ed.), Handbook of Ocean Wave Energy, Ocean Engineering & Oceanography 7, doi:10.1007/978-3-319-39889-1_3.
* Gleeson E, Gallagher S, Clancy C, Dias F. 2017. NAO and extreme ocean states in the Northeast Atlantic Ocean, Adv. Sci. Res., 14, 23–33, doi:10.5194/asr-14-23-2017.
* González-Marco D, Sierra J P, Ybarra O F, Sánchez-Arcilla A. 2008. Implications of long waves in harbor management: The Gijón port case study. Ocean & Coastal Management, 51, 180-201. doi:10.1016/j.ocecoaman.2007.04.001.
* Hurrell JW. 1995. Decadal trends in the North Atlantic Oscillation: regional temperatures and precipitation, Science, 269:676–679.
* Izaguirre C, Méndez F J, Menéndez M, Losada I J. 2011. Global extreme wave height variability based on satellite data Cristina. Geoph. Res. Letters, Vol. 38, L10607, doi: 10.1029/2011GL047302.
* Kushnir Y, Cardone VJ, Greenwood JG, Cane MA. 1997. The recent increase in North Atlantic wave heights. Journal of Climate 10:2107–2113.
* Martínez-Asensio A, Tsimplis M N, Marcos M, Feng F, Gomis D, Jordàa G, Josey S A. 2016. Response of the North Atlantic wave climate to atmospheric modes of variability. International Journal of Climatology, 36, 1210–1225, doi: 10.1002/joc.4415.
* Mørk G, Barstow S, Kabush A, Pontes MT. 2010. Assessing the global wave energy potential. Proceedings of OMAE2010 29th International Conference on Ocean, Offshore Mechanics and Arctic Engineering June 6-11, 2010, Shanghai, China.
* Pascual A., Levier B., Aznar R., Toledano C., García-Valdecasas JM., García M., Sotillo M., Aouf L., Álvarez E. (2020) Monitoring of wave sea state in the Iberia-Biscay-Ireland regional seas. In von Scuckmann et al. (2020) Copernicus Marine Service Ocean State Report, Issue 4, Journal of Operational Oceanography, 13:sup1, S1-S172, DOI: 10.1080/1755876X.2020.1785097
* Tsimplis M N, Woolf D K, Osborn T J, Wakelin S, Wolf J, Flather R, Shaw A G P, Woodworth P, Challenor P, Blackman D, Pert F, Yan Z, Jevrejeva S. 2005. Towards a vulnerability assessment of the UK and northern European coasts: the role of regional climate variability. Phil. Trans. R. Soc. A, Vol. 363, 1329–1358 doi:10.1098/rsta.2005.1571.
* Wang X, Swail V. 2004. Historical and possible future changes of wave heights in northern hemisphere oceans. In: Perrie W (ed), Atmosphere ocean interactions, vol 2. Wessex Institute of Technology Press, Ashurst.
* WASA-Group. 1998. Changing waves and storms in the Northeast Atlantic?, Bull. Am. Meteorol. Soc., 79:741–760.
* Wolf J, Woolf D K. 2006. Waves and climate change in the north-east Atlantic. Geophysical Research Letters, Vol. 33, L06604, doi: 10.1029/2005GL025113.
* Woolf D K, Challenor P G, Cotton P D. 2002. Variability and predictability of the North Atlantic wave climate, J. Geophys. Res., 107(C10), 3145, doi:10.1029/2001JC001124.
* Young I R, Zieger S, Babanin A V. 2011. Global Trends in Wind Speed and Wave Height. Science, Vol. 332, Issue 6028, 451-455, doi: 10.1126/science.1197219.
* Young I R, Ribal A. 2019. Multiplatform evaluation of global trends in wind speed and wave height. Science, 364, 548-552, doi: 10.1126/science.aav9527.





<img src="https://documentation.marine.copernicus.eu/IMG/IBI_OMI_TEMPSAL_extreme_var_temp_mean_and_anomaly.png" width="300" height="200"/>



# Iberia Biscay Ireland Sea Surface Temperature extreme from Reanalysis


**Product ID:** IBI_OMI_TEMPSAL_extreme_var_temp_mean_and_anomaly

**Description:** **DEFINITION**

The CMEMS IBI_OMI_tempsal_extreme_var_temp_mean_and_anomaly OMI indicator is based on the computation of the annual 99th percentile of Sea Surface Temperature (SST) from model data. Two different CMEMS products are used to compute the indicator: The Iberia-Biscay-Ireland Multi Year Product (IBI_MULTIYEAR_PHY_005_002) and the Analysis product (IBI_ANALYSISFORECAST_PHY_005_001).
Two parameters have been considered for this OMI:
•	Map of the 99th mean percentile: It is obtained from the Multi Year Product, the annual 99th percentile is computed for each year of the product. The percentiles are temporally averaged over the whole period (1993-2021).
•	Anomaly of the 99th percentile in 2022: The 99th percentile of the year 2022 is computed from the Analysis product. The anomaly is obtained by subtracting the mean percentile from the 2022 percentile.
This indicator is aimed at monitoring the extremes of sea surface temperature every year and at checking their variations in space. The use of percentiles instead of annual maxima, makes this extremes study less affected by individual data. This study of extreme variability was first applied to the sea level variable (Pérez Gómez et al 2016) and then extended to other essential variables, such as sea surface temperature and significant wave height (Pérez Gómez et al 2018 and Alvarez Fanjul et al., 2019). More details and a full scientific evaluation can be found in the CMEMS Ocean State report (Alvarez Fanjul et al., 2019).

**CONTEXT**

The Sea Surface Temperature is one of the essential ocean variables, hence the monitoring of this variable is of key importance, since its variations can affect the ocean circulation, marine ecosystems, and ocean-atmosphere exchange processes. As the oceans continuously interact with the atmosphere, trends of sea surface temperature can also have an effect on the global climate. While the global-averaged sea surface temperatures have increased since the beginning of the 20th century (Hartmann et al., 2013) in the North Atlantic, anomalous cold conditions have also been reported since 2014 (Mulet et al., 2018; Dubois et al., 2018).

The IBI area is a complex dynamic region with a remarkable variety of ocean physical processes and scales involved. The Sea Surface Temperature field in the region is strongly dependent on latitude, with higher values towards the South (Locarnini et al. 2013). This latitudinal gradient is supported by the presence of the eastern part of the North Atlantic subtropical gyre that transports cool water from the northern latitudes towards the equator. Additionally, the Iberia-Biscay-Ireland region is under the influence of the Sea Level Pressure dipole established between the Icelandic low and the Bermuda high. Therefore, the interannual and interdecadal variability of the surface temperature field may be influenced by the North Atlantic Oscillation pattern (Czaja and Frankignoul, 2002; Flatau et al., 2003).
Also relevant in the region are the upwelling processes taking place in the coastal margins. The most referenced one is the eastern boundary coastal upwelling system off the African and western Iberian coast (Sotillo et al., 2016), although other smaller upwelling systems have also been described in the northern coast of the Iberian Peninsula (Alvarez et al., 2011), the south-western Irish coast (Edwars et al., 1996) and the European Continental Slope (Dickson, 1980).

**CMEMS KEY FINDINGS**

In the IBI region, the 99th mean percentile for 1993-2021 shows a north-south pattern driven by the climatological distribution of temperatures in the North Atlantic. In the coastal regions of Africa and the Iberian Peninsula, the mean values are influenced by the upwelling processes (Sotillo et al., 2016). These results are consistent with the ones presented in Álvarez Fanjul (2019) for the period 1993-2016.
The analysis of the 99th percentile anomaly in the year 2023 shows that this period has been affected by a severe impact of maximum SST values. Anomalies exceeding the standard deviation affect almost the entire IBI domain, and regions impacted by thermal anomalies surpassing twice the standard deviation are also widespread below the 43ºN parallel.
Extreme SST values exceeding twice the standard deviation affect not only the open ocean waters but also the easter boundary upwelling areas such as the northern half of Portugal, the Spanish Atlantic coast up to Cape Ortegal, and the African coast south of Cape Aguer.
It is worth noting the impact of anomalies that exceed twice the standard deviation is widespread throughout the entire Mediterranean region included in this analysis.

**Figure caption**

Iberia-Biscay-Ireland Surface Temperature extreme variability: Map of the 99th mean percentile computed from the Multi Year Product (left panel) and anomaly of the 99th percentile in 2022 computed from the Analysis product (right panel). Transparent grey areas (if any) represent regions where anomaly exceeds the climatic standard deviation (light grey) and twice the climatic standard deviation (dark grey).

**DOI (product):** 
https://doi.org/10.48670/moi-00254

**References:**

* Alvarez I, Gomez-Gesteira M, DeCastro M, Lorenzo MN, Crespo AJC, Dias JM. 2011. Comparative analysis of upwelling influence between the western and northern coast of the Iberian Peninsula. Continental Shelf Research, 31(5), 388-399.
* Álvarez Fanjul E, Pascual Collar A, Pérez Gómez B, De Alfonso M, García Sotillo M, Staneva J, Clementi E, Grandi A, Zacharioudaki A, Korres G, Ravdas M, Renshaw R, Tinker J, Raudsepp U, Lagemaa P, Maljutenko I, Geyer G, Müller M, Çağlar Yumruktepe V. Sea level, sea surface temperature and SWH extreme percentiles: combined analysis from model results and in situ observations, Section 2.7, p:31. In: Schuckmann K, Le Traon P-Y, Smith N, Pascual A, Djavidnia S, Gattuso J-P, Grégoire M, Nolan G, et al. 2019. Copernicus Marine Service Ocean State Report, Issue 3, Journal of Operational Oceanography, 12:sup1, S1-S123, DOI: 10.1080/1755876X.2019.1633075
* Czaja A, Frankignoul C. 2002. Observed impact of Atlantic SST anomalies on the North Atlantic Oscillation. Journal of Climate, 15(6), 606-623.
* Dickson RR, Gurbutt PA, Pillai VN. 1980. Satellite evidence of enhanced upwelling along the European continental slope. Journal of Physical Oceanography, 10(5), 813-819.
* Dubois C, von Schuckmann K, Josey S. 2018. Changes in the North Atlantic. In: Copernicus Marine Service Ocean State Report, Issue 2, Journal of Operational Oceanography, 11:sup1, Chap. 2.9, s66–s70, DOI: https://doi.org/10.1080/1755876X.2018.1489208
* Edwards A, Jones K, Graham JM, Griffiths CR, MacDougall N, Patching J, Raine R. 1996. Transient coastal upwelling and water circulation in Bantry Bay, a ria on the south-west coast of Ireland. Estuarine, Coastal and Shelf Science, 42(2), 213-230.
* Flatau MK, Talley L, Niiler PP. 2003. The North Atlantic Oscillation, surface current velocities, and SST changes in the subpolar North Atlantic. Journal of Climate, 16(14), 2355-2369.
* Hartmann DL, Klein Tank AMG, Rusticucci M, Alexander LV, Brönnimann S, Charabi Y, Dentener FJ, Dlugokencky EJ, Easterling DR, Kaplan A, Soden BJ, Thorne PW, Wild M, Zhai PM. 2013. Observations: Atmosphere and Surface. In: Climate Change 2013: The Physical Science Basis. Contribution of Working Group I to the Fifth Assessment Report of the Intergovernmental Panel on Climate Change. Cambridge University Press, Cambridge, United Kingdom and New York, NY, USA.
* Mulet S, Nardelli BB, Good S, Pisano A, Greiner E, Monier M. 2018. Ocean temperature and salinity. In: Copernicus Marine Service Ocean State Report, Issue 2, Journal of Operational Oceanography, 11:sup1, Chap. 1.1, s5–s13, DOI: https://doi.org/10.1080/1755876X.2018.1489208
* Pérez-Gómez B, Álvarez-Fanjul E, She J, Pérez-González I, Manzano F. 2016. Extreme sea level events, Section 4.4, p:300. In: Von Schuckmann K, Le Traon PY, Alvarez-Fanjul E, Axell L, Balmaseda M, Breivik LA, Brewin RJW, Bricaud C, Drevillon M, Drillet Y, Dubois C , Embury O, Etienne H, García-Sotillo M, Garric G, Gasparin F, Gutknecht E, Guinehut S, Hernandez F, Juza M, Karlson B, Korres G, Legeais JF, Levier B, Lien VS, Morrow R, Notarstefano G, Parent L, Pascual A, Pérez-Gómez B, Perruche C, Pinardi N, Pisano A, Poulain PM , Pujol IM, Raj RP, Raudsepp U, Roquet H, Samuelsen A, Sathyendranath S, She J, Simoncelli S, Solidoro C, Tinker J, Tintoré J, Viktorsson L, Ablain M, Almroth-Rosell E, Bonaduce A, Clementi E, Cossarini G, Dagneaux Q, Desportes C, Dye S, Fratianni C, Good S, Greiner E, Gourrion J, Hamon M, Holt J, Hyder P, Kennedy J, Manzano-Muñoz F, Melet A, Meyssignac B, Mulet S, Nardelli BB, O’Dea E, Olason E, Paulmier A, Pérez-González I, Reid R, Racault MF, Raitsos DE, Ramos A, Sykes P, Szekely T, Verbrugge N. 2016. The Copernicus Marine Environment Monitoring Service Ocean State Report, Journal of Operational Oceanography. 9 (sup2): 235-320. http://dx.doi.org/10.1080/1755876X.2016.1273446
* Pérez Gómez B., De Alfonso M., Zacharioudaki A., Pérez González I., Álvarez Fanjul E., Müller M., Marcos M., Manzano F., Korres G., Ravdas M., Tamm S. 2018. Sea level, SST and waves: extremes variability. In: Copernicus Marine Service Ocean State Report, Issue 2, Journal of Operational Oceanography, 11:sup1, Chap. 3.1, s79–s88, DOI: https://doi.org/10.1080/1755876X.2018.1489208
* Sotillo MG, Levier B, Pascual A, Gonzalez A. 2016. Iberian-Biscay-Irish Sea. In von Schuckmann et al. 2016. The Copernicus Marine Environment Monitoring Service Ocean State Report No.1, Journal of Operational Oceanography, 9:sup2, s235-s320, DOI: 10.1080/1755876X.2016.1273446





<img src="https://documentation.marine.copernicus.eu/IMG/IBI_OMI_WMHE_mow.png" width="300" height="200"/>



# Mediterrranean Outflow Water Index from Reanalysis & Multi-Observations Reprocessing


**Product ID:** IBI_OMI_WMHE_mow

**Description:** **DEFINITION**

Variations of the Mediterranean Outflow Water at 1000 m depth are monitored through area-averaged salinity anomalies in specifically defined boxes. The salinity data are extracted from several CMEMS products and averaged in the corresponding monitoring domain: 
* IBI-MYP: IBI_MULTIYEAR_PHY_005_002
* IBI-NRT: IBI_ANALYSISFORECAST_PHYS_005_001
* GLO-MYP: GLOBAL_REANALYSIS_PHY_001_030
* CORA: INSITU_GLO_TS_REP_OBSERVATIONS_013_002_b
* ARMOR: MULTIOBS_GLO_PHY_TSUV_3D_MYNRT_015_012

The anomalies of salinity have been computed relative to the monthly climatology obtained from IBI-MYP. Outcomes from diverse products are combined to deliver a unique multi-product result. Multi-year products (IBI-MYP, GLO,MYP, CORA, and ARMOR) are used to show an ensemble mean and the standard deviation of members in the covered period. The IBI-NRT short-range product is not included in the ensemble, but used to provide the deterministic analysis of salinity anomalies in the most recent year.

**CONTEXT**

The Mediterranean Outflow Water is a saline and warm water mass generated from the mixing processes of the North Atlantic Central Water and the Mediterranean waters overflowing the Gibraltar sill (Daniault et al., 1994). The resulting water mass is accumulated in an area west of the Iberian Peninsula (Daniault et al., 1994) and spreads into the North Atlantic following advective pathways (Holliday et al. 2003; Lozier and Stewart 2008, de Pascual-Collar et al., 2019).
The importance of the heat and salt transport promoted by the Mediterranean Outflow Water flow has implications beyond the boundaries of the Iberia-Biscay-Ireland domain (Reid 1979, Paillet et al. 1998, van Aken 2000). For example, (i) it contributes substantially to the salinity of the Norwegian Current (Reid 1979), (ii) the mixing processes with the Labrador Sea Water promotes a salt transport into the inner North Atlantic (Talley and MacCartney, 1982; van Aken, 2000), and (iii) the deep anti-cyclonic Meddies developed in the African slope is a cause of the large-scale westward penetration of Mediterranean salt (Iorga and Lozier, 1999).
Several studies have demonstrated that the core of Mediterranean Outflow Water is affected by inter-annual variability. This variability is mainly caused by a shift of the MOW dominant northward-westward pathways (Bozec et al. 2011), it is correlated with the North Atlantic Oscillation (Bozec et al. 2011) and leads to the displacement of the boundaries of the water core (de Pascual-Collar et al., 2019). The variability of the advective pathways of MOW is an oceanographic process that conditions the destination of the Mediterranean salt transport in the North Atlantic. Therefore, monitoring the Mediterranean Outflow Water variability becomes decisive to have a proper understanding of the climate system and its evolution (e.g. Bozec et al. 2011, Pascual-Collar et al. 2019).
The CMEMS IBI-OMI_WMHE_mow product is aimed to monitor the inter-annual variability of the Mediterranean Outflow Water in the North Atlantic. The objective is the establishment of a long-term monitoring program to observe the variability and trends of the Mediterranean water mass in the IBI regional seas. To do that, the salinity anomaly is monitored in key areas selected to represent the main reservoir and the three main advective spreading pathways. More details and a full scientific evaluation can be found in the CMEMS Ocean State report Pascual et al., 2018 and de Pascual-Collar et al. 2019.

**CMEMS KEY FINDINGS**

The absence of long-term trends in the monitoring domain Reservoir (b) suggests the steadiness of water mass properties involved on the formation of Mediterranean Outflow Water.
Results obtained in monitoring box North (c) present an alternance of periods with positive and negative anomalies. The last negative period started in 2016 reaching up to the present. Such negative events are linked to the decrease of the northward pathway of Mediterranean Outflow Water (Bozec et al., 2011), which appears to return to steady conditions in 2020 and 2021. 
Results for box West (d) reveal a cycle of negative (2015-2017) and positive (2017 up to the present) anomalies. The positive anomalies of salinity in this region are correlated with an increase of the westward transport of salinity into the inner North Atlantic (de Pascual-Collar et al., 2019), which appear to be maintained for years 2020-2021.
Results in monitoring boxes North and West are consistent with independent studies (Bozec et al., 2011; and de Pascual-Collar et al., 2019), suggesting a westward displacement of Mediterranean Outflow Water and the consequent contraction of the northern boundary.

Note: The key findings will be updated annually in November, in line with OMI evolutions.

**DOI (product):** 
https://doi.org/10.48670/moi-00258

**References:**

* Bozec A, Lozier MS, Chassignet EP, Halliwell GR. 2011. On the variability of the Mediterranean outflow water in the North Atlantic from 1948 to 2006. J Geophys Res 116:C09033. doi:10.1029/2011JC007191.
* Daniault N, Maze JP, Arhan M. 1994. Circulation and mixing of MediterraneanWater west of the Iberian Peninsula. Deep Sea Res. Part I. 41:1685–1714.
* de Pascual-Collar A, Sotillo MG, Levier B, Aznar R, Lorente P, Amo-Baladrón A, Álvarez-Fanjul E. 2019. Regional circulation patterns of Mediterranean Outflow Water near the Iberian and African continental slopes. Ocean Sci., 15, 565–582. https://doi.org/10.5194/os-15-565-2019.
* Holliday NP. 2003. Air-sea interaction and circulation changes in the northeast Atlantic. J Geophys Res. 108(C8):3259. doi:10.1029/2002JC001344.
* Iorga MC, Lozier MS. 1999. Signatures of the Mediterranean outflow from a North Atlantic climatology: 1. Salinity and density fields. Journal of Geophysical Research: Oceans, 104(C11), 25985-26009.
* Lozier MS, Stewart NM. 2008. On the temporally varying northward penetration of Mediterranean overflow water and eastward penetration of Labrador Sea water. J Phys Oceanogr. 38(9):2097–2103. doi:10.1175/2008JPO3908.1.
* Paillet J, Arhan M, McCartney M. 1998. Spreading of labrador Sea water in the eastern North Atlantic. J Geophys Res. 103 (C5):10223–10239.
* Pascual A, Levier B, Sotillo M, Verbrugge N, Aznar R, Le Cann B. 2018. Characterisation of Mediterranean outflow wáter in the Iberia-Gulf of Biscay-Ireland region. In: von Schuckmann, K., Le Traon, P.-Y., Smith, N., Pascual, A., Braseur, P., Fennel, K., Djavidnia, S.: Copernicus Marine Service Ocean State Report, Issue 2, Journal of Operational Oceanography, 11, sup1, s1-s142, doi:10.1080/1755876X.2018.1489208, 2018.
* Reid JL. 1979. On the contribution of the Mediterranean Sea outflow to the Norwegian‐Greenland Sea, Deep Sea Res., Part A, 26, 1199–1223, doi:10.1016/0198-0149(79)90064-5.
* Talley LD, McCartney MS. 1982. Distribution and circulation of Labrador Sea water. Journal of Physical Oceanography, 12(11), 1189-1205.
* van Aken HM. 2000. The hydrography of the mid-latitude northeast Atlantic Ocean I: the deep water masses. Deep Sea Res. Part I. 47:757–788.





<img src="https://mdl-metadata.s3.waw3-1.cloudferro.com/metadata/thumbnails/INSITU_ARC_PHYBGCWAV_DISCRETE_MYNRT_013_031.jpg" width="300" height="200"/>



# Arctic Ocean- In Situ Near Real Time Observations


**Product ID:** INSITU_ARC_PHYBGCWAV_DISCRETE_MYNRT_013_031

**Description:** Arctic Oceans  - near real-time (NRT) in situ quality controlled observations, hourly updated and distributed by INSTAC  within 24-48 hours from acquisition in average

**DOI (product):**   
https://doi.org/10.48670/moi-00031




<img src="https://mdl-metadata.s3.waw3-1.cloudferro.com/metadata/thumbnails/INSITU_BAL_PHYBGCWAV_DISCRETE_MYNRT_013_032.jpg" width="300" height="200"/>



# Baltic Sea- In Situ Near Real Time Observations


**Product ID:** INSITU_BAL_PHYBGCWAV_DISCRETE_MYNRT_013_032

**Description:** Baltic Sea  - near real-time (NRT) in situ quality controlled observations, hourly updated and distributed by INSTAC  within 24-48 hours from acquisition in average

**DOI (product):**   
https://doi.org/10.48670/moi-00032




<img src="https://mdl-metadata.s3.waw3-1.cloudferro.com/metadata/thumbnails/INSITU_BLK_PHYBGCWAV_DISCRETE_MYNRT_013_034.jpg" width="300" height="200"/>



# Black Sea- In-Situ Near Real Time Observations


**Product ID:** INSITU_BLK_PHYBGCWAV_DISCRETE_MYNRT_013_034

**Description:** Black Sea  - near real-time (NRT) in situ quality controlled observations, hourly updated and distributed by INSTAC  within 24-48 hours from acquisition in average

**DOI (product):**   
https://doi.org/10.48670/moi-00033




<img src="https://mdl-metadata.s3.waw3-1.cloudferro.com/metadata/thumbnails/INSITU_GLO_BGC_CARBON_DISCRETE_MY_013_050.jpg" width="300" height="200"/>



# Global Ocean - In Situ reprocessed carbon observations - SOCATv2024 / GLODAPv2.2023


**Product ID:** INSITU_GLO_BGC_CARBON_DISCRETE_MY_013_050

**Description:** Global Ocean- in-situ reprocessed Carbon observations. This product contains observations and gridded files from two up-to-date carbon and biogeochemistry community data products: Surface Ocean Carbon ATlas SOCATv2024 and GLobal Ocean Data Analysis Project GLODAPv2.2023. 
The SOCATv2024-OBS dataset contains >38 million observations of fugacity of CO2 of the surface global ocean from 1957 to early 2024. The quality control procedures are described in Bakker et al. (2016). These observations form the basis of the gridded products included in SOCATv2024-GRIDDED: monthly, yearly and decadal averages of fCO2 over a 1x1 degree grid over the global ocean, and a 0.25x0.25 degree, monthly average for the coastal ocean.
GLODAPv2.2023-OBS contains >1 million observations from individual seawater samples of temperature, salinity, oxygen, nutrients, dissolved inorganic carbon, total alkalinity and pH from 1972 to 2021. These data were subjected to an extensive quality control and bias correction described in Olsen et al. (2020). GLODAPv2-GRIDDED contains global climatologies for temperature, salinity, oxygen, nitrate, phosphate, silicate, dissolved inorganic carbon, total alkalinity and pH over a 1x1 degree horizontal grid and 33 standard depths using the observations from the previous major iteration of GLODAP, GLODAPv2. 
SOCAT and GLODAP are based on community, largely volunteer efforts, and the data providers will appreciate that those who use the data cite the corresponding articles (see References below) in order to support future sustainability of the data products.

**DOI (product):**   
https://doi.org/10.17882/99089

**References:**

* Bakker et al., 2016. A multi-decade record of high-quality fCO2 data in version 3 of the Surface Ocean CO2 Atlas (SOCAT). Earth Syst. Sci. Data, 8, 383–413, https://doi.org/10.5194/essd-8-383-2016.
* Lauvset et al. 2024. The annual update GLODAPv2.2023: the global interior ocean biogeochemical data product. Earth Syst. Sci. Data Discuss. [preprint], https://doi.org/10.5194/essd-2023-468.
* Lauvset et al., 2016. A new global interior ocean mapped climatology: t ×  1° GLODAP version 2. Earth Syst. Sci. Data, 8, 325–340, https://doi.org/10.5194/essd-8-325-2016.





<img src="https://mdl-metadata.s3.waw3-1.cloudferro.com/metadata/thumbnails/INSITU_GLO_BGC_DISCRETE_MY_013_046.jpg" width="300" height="200"/>



# Global Ocean - Delayed Mode Biogeochemical product


**Product ID:** INSITU_GLO_BGC_DISCRETE_MY_013_046

**Description:** For the Global Ocean- In-situ observation delivered in delayed mode. This In Situ delayed mode product integrates the best available version of in situ oxygen,  chlorophyll / fluorescence and nutrients data.

**DOI (product):**   
https://doi.org/10.17882/86207




<img src="https://mdl-metadata.s3.waw3-1.cloudferro.com/metadata/thumbnails/INSITU_GLO_PHYBGCWAV_DISCRETE_MYNRT_013_030.jpg" width="300" height="200"/>



# Global Ocean- In-Situ Near-Real-Time Observations


**Product ID:** INSITU_GLO_PHYBGCWAV_DISCRETE_MYNRT_013_030

**Description:** Global Ocean   - near real-time (NRT) in situ quality controlled observations, hourly updated and distributed by INSTAC  within 24-48 hours from acquisition in average. Data are collected mainly through global networks (Argo, OceanSites, GOSUD, EGO) and through the GTS

**DOI (product):**   
https://doi.org/10.48670/moi-00036




<img src="https://mdl-metadata.s3.waw3-1.cloudferro.com/metadata/thumbnails/INSITU_GLO_PHY_SSH_DISCRETE_MY_013_053.jpg" width="300" height="200"/>



# Global Ocean - Delayed Mode Sea level product


**Product ID:** INSITU_GLO_PHY_SSH_DISCRETE_MY_013_053

**Description:** This product integrates sea level observations aggregated and validated from the Regional EuroGOOS consortium (Arctic-ROOS, BOOS, NOOS, IBI-ROOS, MONGOOS) and Black Sea GOOS as well as from the Global telecommunication system (GTS) used by the Met Offices.

**DOI (product):**   
https://doi.org/10.17882/93670




<img src="https://mdl-metadata.s3.waw3-1.cloudferro.com/metadata/thumbnails/INSITU_GLO_PHY_TS_DISCRETE_MY_013_001.jpg" width="300" height="200"/>



# Global Ocean- CORA- In-situ Observations Yearly Delivery in Delayed Mode


**Product ID:** INSITU_GLO_PHY_TS_DISCRETE_MY_013_001

**Description:** For the Global Ocean- In-situ observation yearly delivery in delayed mode. The In Situ delayed mode product designed for reanalysis purposes integrates the best available version of in situ data for temperature and salinity measurements. These data are collected from main global networks (Argo, GOSUD, OceanSITES, World Ocean Database) completed by European data provided by EUROGOOS regional systems and national system by the regional INS TAC components. It is updated on a yearly basis. This version is a merged product between the previous verion of CORA and EN4 distributed by the Met Office  for the period 1950-1990.

**DOI (product):**  
https://doi.org/10.17882/46219




<img src="https://mdl-metadata.s3.waw3-1.cloudferro.com/metadata/thumbnails/INSITU_GLO_PHY_TS_OA_MY_013_052.jpg" width="300" height="200"/>



# Global Ocean- Delayed Mode gridded CORA- In-situ Observations objective analysis in Delayed Mode


**Product ID:** INSITU_GLO_PHY_TS_OA_MY_013_052

**Description:** Global Ocean- Gridded objective analysis fields of temperature and salinity using profiles from the reprocessed in-situ global product CORA (INSITU_GLO_TS_REP_OBSERVATIONS_013_001) using the ISAS software. Objective analysis is based on a statistical estimation method that allows presenting a synthesis and a validation of the dataset, providing a validation source for operational models, observing seasonal cycle and inter-annual variability.

**DOI (product):**  
https://doi.org/10.17882/46219




<img src="https://mdl-metadata.s3.waw3-1.cloudferro.com/metadata/thumbnails/INSITU_GLO_PHY_TS_OA_NRT_013_002.jpg" width="300" height="200"/>



# Global Ocean- Real time in-situ observations objective analysis


**Product ID:** INSITU_GLO_PHY_TS_OA_NRT_013_002

**Description:** For the Global Ocean- Gridded objective analysis fields of temperature and salinity using profiles from the in-situ near real time database are produced monthly. Objective analysis is based on a statistical estimation method that allows presenting a synthesis and a validation of the dataset, providing a support for localized experience (cruises), providing a validation source for operational models, observing seasonal cycle and inter-annual variability.

**DOI (product):**   
https://doi.org/10.48670/moi-00037




<img src="https://mdl-metadata.s3.waw3-1.cloudferro.com/metadata/thumbnails/INSITU_GLO_PHY_UV_DISCRETE_MY_013_044.jpg" width="300" height="200"/>



# Global Ocean-Delayed Mode in-situ Observations of surface and sub-surface ocean currents


**Product ID:** INSITU_GLO_PHY_UV_DISCRETE_MY_013_044

**Description:** Global Ocean - This delayed mode product designed for reanalysis purposes integrates the best available version of in situ data for ocean surface and subsurface currents. Current data from 5 different types of instruments are distributed:
* The drifter's near-surface velocities computed from their position measurements. In addition, a wind slippage correction is provided from 1993. Information on the presence of the drogue of the drifters is also provided.
* The near-surface zonal and meridional total velocities, and near-surface radial velocities, measured by High Frequency (HF) radars that are part of the European HF radar Network. These data are delivered together with standard deviation of near-surface zonal and meridional raw velocities, Geometrical Dilution of Precision (GDOP), quality flags and metadata.
* The zonal and meridional velocities, at parking depth (mostly around 1000m) and at the surface, calculated along the trajectories of the floats which are part of the Argo Program.
* The velocity profiles within the water column coming from Acoustic Doppler Current Profiler (vessel mounted ADCP, Moored ADCP, saildrones) platforms
* The near-surface and subsurface velocities calculated from gliders (autonomous underwater vehicle) trajectories

**DOI (product):**
https://doi.org/10.17882/86236




<img src="https://mdl-metadata.s3.waw3-1.cloudferro.com/metadata/thumbnails/INSITU_GLO_PHY_UV_DISCRETE_NRT_013_048.jpg" width="300" height="200"/>



# Global Ocean- in-situ Near real time observations of ocean currents


**Product ID:** INSITU_GLO_PHY_UV_DISCRETE_NRT_013_048

**Description:** This product is entirely dedicated to ocean current data observed in near-real time. Current data from 3 different types of instruments are distributed:
* The near-surface zonal and meridional velocities calculated along the trajectories of the drifting buoys which are part of the DBCP’s Global Drifter Program. These data are delivered together with wind stress components, surface temperature and a wind-slippage correction for drogue-off and drogue-on drifters trajectories. 
* The near-surface zonal and meridional total velocities, and near-surface radial velocities, measured by High Frequency radars that are part of the European High Frequency radar Network. These data are delivered together with standard deviation of near-surface zonal and meridional raw velocities, Geometrical Dilution of Precision (GDOP), quality flags and metadata.
* The zonal and meridional velocities, at parking depth and in surface, calculated along the trajectories of the floats which are part of the Argo Program.

**DOI (product):**
https://doi.org/10.48670/moi-00041




<img src="https://mdl-metadata.s3.waw3-1.cloudferro.com/metadata/thumbnails/INSITU_GLO_WAV_DISCRETE_MY_013_045.jpg" width="300" height="200"/>



# Global Ocean - Delayed Mode Wave product


**Product ID:** INSITU_GLO_WAV_DISCRETE_MY_013_045

**Description:** These products integrate wave observations aggregated and validated from the Regional EuroGOOS consortium (Arctic-ROOS, BOOS, NOOS, IBI-ROOS, MONGOOS) and Black Sea GOOS as well as from National Data Centers (NODCs) and JCOMM global systems (OceanSITES, DBCP) and the Global telecommunication system (GTS) used by the Met Offices.

**DOI (product):**  
https://doi.org/10.17882/70345




<img src="https://mdl-metadata.s3.waw3-1.cloudferro.com/metadata/thumbnails/INSITU_IBI_PHYBGCWAV_DISCRETE_MYNRT_013_033.jpg" width="300" height="200"/>



# Atlantic Iberian Biscay Irish Ocean- In-Situ Near Real Time Observations


**Product ID:** INSITU_IBI_PHYBGCWAV_DISCRETE_MYNRT_013_033

**Description:** IBI Seas  - near real-time (NRT) in situ quality controlled observations, hourly updated and distributed by INSTAC  within 24-48 hours from acquisition in average

**DOI (product):**   
https://doi.org/10.48670/moi-00043




<img src="https://mdl-metadata.s3.waw3-1.cloudferro.com/metadata/thumbnails/INSITU_MED_PHYBGCWAV_DISCRETE_MYNRT_013_035.jpg" width="300" height="200"/>



# Mediterranean Sea- In-Situ Near Real Time Observations


**Product ID:** INSITU_MED_PHYBGCWAV_DISCRETE_MYNRT_013_035

**Description:** Mediterranean Sea  - near real-time (NRT) in situ quality controlled observations, hourly updated and distributed by INSTAC  within 24-48 hours from acquisition in average

**DOI (product):**   
https://doi.org/10.48670/moi-00044




<img src="https://mdl-metadata.s3.waw3-1.cloudferro.com/metadata/thumbnails/INSITU_NWS_PHYBGCWAV_DISCRETE_MYNRT_013_036.jpg" width="300" height="200"/>



# Atlantic- European North West Shelf- Ocean In-Situ Near Real Time observations


**Product ID:** INSITU_NWS_PHYBGCWAV_DISCRETE_MYNRT_013_036

**Description:** NorthWest Shelf area  - near real-time (NRT) in situ quality controlled observations, hourly updated and distributed by INSTAC  within 24-48 hours from acquisition in average

**DOI (product):**  
https://doi.org/10.48670/moi-00045




<img src="https://mdl-metadata.s3.waw3-1.cloudferro.com/metadata/thumbnails/MEDSEA_ANALYSISFORECAST_BGC_006_014.jpg" width="300" height="200"/>



# Mediterranean Sea Biogeochemistry Analysis and Forecast


**Product ID:** MEDSEA_ANALYSISFORECAST_BGC_006_014

**Description:** The biogeochemical analysis and forecasts for the Mediterranean Sea at 1/24° of horizontal resolution (ca. 4 km) are produced by means of the MedBFM4 model system. MedBFM4, which is run by OGS (IT), consists of the coupling of the multi-stream atmosphere radiative model OASIM, the multi-stream in-water radiative and tracer transport model OGSTM_BIOPTIMOD v4.6, and the biogeochemical flux model BFM v5.3. Additionally, MedBFM4 features the 3D variational data assimilation scheme 3DVAR-BIO v4.1 with the assimilation of surface chlorophyll (CMEMS-OCTAC NRT product) and of vertical profiles of chlorophyll, nitrate and oxygen (BGC-Argo floats provided by CORIOLIS DAC). The biogeochemical MedBFM system, which is forced by the NEMO-OceanVar model (MEDSEA_ANALYSIS_FORECAST_PHY_006_013), produces one day of hindcast and ten days of forecast (every day) and seven days of analysis (weekly on Tuesday).

Salon, S.; Cossarini, G.; Bolzon, G.; Feudale, L.; Lazzari, P.; Teruzzi, A.; Solidoro, C., and Crise, A. (2019) Novel metrics based on Biogeochemical Argo data to improve the model uncertainty evaluation of the CMEMS Mediterranean marine ecosystem forecasts. Ocean Science, 15, pp.997–1022. DOI: https://doi.org/10.5194/os-15-997-2019

_DOI (Product)_: 
https://doi.org/10.48670/mds-00358

**References:**

* Feudale, L., Bolzon, G., Lazzari, P., Salon, S., Teruzzi, A., Di Biagio, V., Coidessa, G., Alvarez, E., Amadio, C., & Cossarini, G. (2022). Mediterranean Sea Biogeochemical Analysis and Forecast (CMEMS MED-Biogeochemistry, MedBFM4 system) (Version 1) [Data set]. Copernicus Marine Service. https://doi.org/10.25423/CMCC/MEDSEA_ANALYSISFORECAST_BGC_006_014_MEDBFM4





<img src="https://mdl-metadata.s3.waw3-1.cloudferro.com/metadata/thumbnails/MEDSEA_ANALYSISFORECAST_PHY_006_013.jpg" width="300" height="200"/>



# Mediterranean Sea Physics Analysis and Forecast


**Product ID:** MEDSEA_ANALYSISFORECAST_PHY_006_013

**Description:** The physical component of the Mediterranean Forecasting System (Med-Physics) is a coupled hydrodynamic-wave model implemented over the whole Mediterranean Basin including tides. The model horizontal grid resolution is 1/24˚ (ca. 4 km) and has 141 unevenly spaced vertical levels.
The hydrodynamics are supplied by the Nucleous for European Modelling of the Ocean NEMO (v4.2) and include the representation of tides, while the wave component is provided by Wave Watch-III (v6.07) coupled through OASIS; the model solutions are corrected by a 3DVAR assimilation scheme (OceanVar) for temperature and salinity vertical profiles and along track satellite Sea Level Anomaly observations.

_DOI (Product)_:
https://doi.org/10.48670/mds-00359




<img src="https://mdl-metadata.s3.waw3-1.cloudferro.com/metadata/thumbnails/MEDSEA_ANALYSISFORECAST_WAV_006_017.jpg" width="300" height="200"/>



# Mediterranean Sea Waves Analysis and Forecast


**Product ID:** MEDSEA_ANALYSISFORECAST_WAV_006_017

**Description:** MEDSEA_ANALYSISFORECAST_WAV_006_017 is the nominal wave product of the Mediterranean Sea Forecasting system, composed by hourly wave parameters at 1/24º horizontal resolution covering the Mediterranean Sea and extending up to 18.125W into the Atlantic Ocean. The waves forecast component (Med-WAV system) is a wave model based on the WAM Cycle 6. The Med-WAV modelling system resolves the prognostic part of the wave spectrum with 24 directional and 32 logarithmically distributed frequency bins and the model solutions are corrected by an optimal interpolation data assimilation scheme of all available along track satellite significant wave height observations. The atmospheric forcing is provided by the operational ECMWF Numerical Weather Prediction model and the wave model is forced with hourly averaged surface currents and sea level obtained from MEDSEA_ANALYSISFORECAST_PHY_006_013 at 1/24° resolution. The model uses wave spectra for Open Boundary Conditions from GLOBAL_ANALYSIS_FORECAST_WAV_001_027 product. The wave system includes 2 forecast cycles providing twice per day a Mediterranean wave analysis and 10 days of wave forecasts.

**DOI (product)**: https://doi.org/10.25423/cmcc/medsea_analysisforecast_wav_006_017_medwam4

**References:**

* Korres, G., Oikonomou, C., Denaxa, D., & Sotiropoulou, M. (2023). Mediterranean Sea Waves Analysis and Forecast (Copernicus Marine Service MED-Waves, MEDWAΜ4 system) (Version 1) [Data set]. Copernicus Marine Service (CMS). https://doi.org/10.25423/CMCC/MEDSEA_ANALYSISFORECAST_WAV_006_017_MEDWAM4





<img src="https://mdl-metadata.s3.waw3-1.cloudferro.com/metadata/thumbnails/MEDSEA_MULTIYEAR_BGC_006_008.jpg" width="300" height="200"/>



# Mediterranean Sea Biogeochemistry Reanalysis


**Product ID:** MEDSEA_MULTIYEAR_BGC_006_008

**Description:** The Mediterranean Sea biogeochemical reanalysis at 1/24° of horizontal resolution (ca. 4 km) covers the period from Jan 1999 to 1 month to the present and is produced by means of the MedBFM3 model system. MedBFM3, which is run by OGS (IT), includes the transport model OGSTM v4.0 coupled with the biogeochemical flux model BFM v5 and the variational data assimilation module 3DVAR-BIO v2.1 for surface chlorophyll. MedBFM3 is forced by the physical reanalysis (MEDSEA_MULTIYEAR_PHY_006_004 product run by CMCC) that provides daily forcing fields (i.e., currents, temperature, salinity, diffusivities, wind and solar radiation). The ESA-CCI database of surface chlorophyll concentration (CMEMS-OCTAC REP product) is assimilated with a weekly frequency. 

Cossarini, G., Feudale, L., Teruzzi, A., Bolzon, G., Coidessa, G., Solidoro C., Amadio, C., Lazzari, P., Brosich, A., Di Biagio, V., and Salon, S., 2021. High-resolution reanalysis of the Mediterranean Sea biogeochemistry (1999-2019). Frontiers in Marine Science. Front. Mar. Sci. 8:741486.doi: 10.3389/fmars.2021.741486

_DOI (Product)_: https://doi.org/10.25423/cmcc/medsea_multiyear_bgc_006_008_medbfm3

_DOI (Interim dataset)_:
https://doi.org/10.25423/CMCC/MEDSEA_MULTIYEAR_BGC_006_008_MEDBFM3I

**References:**

* Teruzzi, A., Di Biagio, V., Feudale, L., Bolzon, G., Lazzari, P., Salon, S., Coidessa, G., & Cossarini, G. (2021). Mediterranean Sea Biogeochemical Reanalysis (CMEMS MED-Biogeochemistry, MedBFM3 system) (Version 1) [Data set]. Copernicus Monitoring Environment Marine Service (CMEMS).  https://doi.org/10.25423/CMCC/MEDSEA_MULTIYEAR_BGC_006_008_MEDBFM3
* Teruzzi, A., Feudale, L., Bolzon, G., Lazzari, P., Salon, S., Di Biagio, V., Coidessa, G., & Cossarini, G. (2021). Mediterranean Sea Biogeochemical Reanalysis INTERIM (CMEMS MED-Biogeochemistry, MedBFM3i system) (Version 1) [Data set]. Copernicus Monitoring Environment Marine Service (CMEMS) https://doi.org/10.25423/CMCC/MEDSEA_MULTIYEAR_BGC_006_008_MEDBFM3I





<img src="https://mdl-metadata.s3.waw3-1.cloudferro.com/metadata/thumbnails/MEDSEA_MULTIYEAR_PHY_006_004.jpg" width="300" height="200"/>



# Mediterranean Sea Physics Reanalysis


**Product ID:** MEDSEA_MULTIYEAR_PHY_006_004

**Description:** The Med MFC physical multiyear product is generated by a numerical system composed of an hydrodynamic model, supplied by the Nucleous for European Modelling of the Ocean (NEMO) and a variational data assimilation scheme (OceanVAR) for temperature and salinity vertical profiles and satellite Sea Level Anomaly along track data. It contains a reanalysis dataset and an interim dataset which covers the period after the reanalysis until 1 month before present. The model horizontal grid resolution is 1/24˚ (ca. 4-5 km) and the unevenly spaced vertical levels are 141. 

_DOI (Product)_: https://doi.org/10.25423/CMCC/MEDSEA_MULTIYEAR_PHY_006_004_E3R1

_DOI (Interim dataset)_:
https://doi.org/10.25423/CMCC/MEDSEA_MULTIYEAR_PHY_006_004_E3R1I

**References:**

* Escudier, R., Clementi, E., Omar, M., Cipollone, A., Pistoia, J., Aydogdu, A., Drudi, M., Grandi, A., Lyubartsev, V., Lecci, R., Cretí, S., Masina, S., Coppini, G., & Pinardi, N. (2020). Mediterranean Sea Physical Reanalysis (CMEMS MED-Currents) (Version 1) [Data set]. Copernicus Monitoring Environment Marine Service (CMEMS). https://doi.org/10.25423/CMCC/MEDSEA_MULTIYEAR_PHY_006_004_E3R1
* Escudier, R., Clementi, E., Cipollone, A., Pistoia, J., Drudi, M., Grandi, A., Lyubartsev, V., Lecci, R., Aydogdu, A., Delrosso, D., Omar, M., Masina, S., Coppini G., Pinardi, N. (2021). A High Resolution Reanalysis for the Mediterranean Sea. Frontiers in Earth Science, 9, 1060, https://www.frontiersin.org/article/10.3389/feart.2021.702285, DOI=10.3389/feart.2021.702285
* Nigam, T., Escudier, R., Pistoia, J., Aydogdu, A., Omar, M., Clementi, E., Cipollone, A., Drudi, M., Grandi, A., Mariani, A., Lyubartsev, V., Lecci, R., Cretí, S., Masina, S., Coppini, G., & Pinardi, N. (2021). Mediterranean Sea Physical Reanalysis INTERIM (CMEMS MED-Currents, E3R1i system) (Version 1) [Data set]. Copernicus Monitoring Environment Marine Service (CMEMS). https://doi.org/10.25423/CMCC/MEDSEA_MULTIYEAR_PHY_006_004_E3R1I





<img src="https://mdl-metadata.s3.waw3-1.cloudferro.com/metadata/thumbnails/MEDSEA_MULTIYEAR_WAV_006_012.jpg" width="300" height="200"/>



# Mediterranean Sea Waves Reanalysis


**Product ID:** MEDSEA_MULTIYEAR_WAV_006_012

**Description:** MEDSEA_MULTIYEAR_WAV_006_012  is the multi-year wave product of the Mediterranean Sea Waves forecasting system (Med-WAV). It contains a Reanalysis dataset, an Interim dataset covering the period after the reanalysis until 1 month before present and a monthly climatological dataset (reference period 1993-2016). The Reanalysis dataset is a multi-year wave reanalysis starting from January 1985, composed by hourly wave parameters at 1/24° horizontal resolution, covering the Mediterranean Sea and extending up to 18.125W into the Atlantic Ocean. The Med-WAV modelling system is based on wave model WAM 4.6.2 and has been developed as a nested sequence of two computational grids (coarse and fine) to ensure that swell propagating from the North Atlantic (NA) towards the strait of Gibraltar is correctly entering the Mediterranean Sea. The coarse grid covers the North Atlantic Ocean from 75°W to 10°E and from 70° N to 10° S in 1/6° resolution while the nested fine grid covers the Mediterranean Sea from 18.125° W to 36.2917° E and from 30.1875° N to 45.9792° N with a 1/24° resolution. The modelling system resolves the prognostic part of the wave spectrum with 24 directional and 32 logarithmically distributed frequency bins.  The wave system also includes an optimal interpolation assimilation scheme assimilating significant wave height along track satellite observations available through CMEMS and it is forced with daily averaged currents from Med-Physics and with 1-h, 0.25° horizontal-resolution ERA5 reanalysis 10m-above-sea-surface winds from ECMWF.

_DOI (Product)_: https://doi.org/10.25423/cmcc/medsea_multiyear_wav_006_012

_DOI (Interim dataset)_:    
https://doi.org/10.25423/ CMCC/MEDSEA_MULTIYEAR_WAV_006_012_MEDWAM3I  
                                                                                                                                                      
_DOI (climatological dataset)_:    
https://doi.org/10.25423/ CMCC/MEDSEA_MULTIYEAR_WAV_006_012_CLIM

**References:**

* Korres, G., Ravdas, M., Zacharioudaki, A., Denaxa, D., & Sotiropoulou, M. (2021). Mediterranean Sea Waves Reanalysis (CMEMS Med-Waves, MedWAM3 system) (Version 1) [Data set]. Copernicus Monitoring Environment Marine Service (CMEMS).
* Korres, G., Ravdas, M., Denaxa, D., & Sotiropoulou, M. (2021). Mediterranean Sea Waves Reanalysis INTERIM (CMEMS Med-Waves, MedWAM3I system) (Version 1) [Data set]. Copernicus Monitoring Environment Marine Service (CMEMS).
* Korres, G., Oikonomou, C., Denaxa, D., & Sotiropoulou, M. (2023). Mediterranean Sea Waves Monthly Climatology (CMS Med-Waves, MedWAM3 system) (Version 1) [Data set]. Copernicus Marine Service (CMS).





<img src="https://documentation.marine.copernicus.eu/IMG/MEDSEA_OMI_OHC_area_averaged_anomalies.png" width="300" height="200"/>



# Mediterranean Ocean Heat Content Anomaly (0-700m) time series and trend from Reanalysis & Multi-Observations Reprocessing


**Product ID:** MEDSEA_OMI_OHC_area_averaged_anomalies

**Description:** **DEFINITION**

Ocean heat content (OHC) is defined here as the deviation from a reference period (1993-2014) and is closely proportional to the average temperature change from z1 = 0 m to z2 = 700 m depth:
OHC=∫_(z_1)^(z_2)ρ_0  c_p (T_yr-T_clim )dz 								[1]
with a reference density of = 1030 kgm-3 and a specific heat capacity of cp = 3980 J kg-1 °C-1 (e.g. von Schuckmann et al., 2009).
Time series of annual mean values area averaged ocean heat content is provided for the Mediterranean Sea (30°N, 46°N; 6°W, 36°E) and is evaluated for topography deeper than 300m.

**CONTEXT**

Knowing how much and where heat energy is stored and released in the ocean is essential for understanding the contemporary Earth system state, variability and change, as the oceans shape our perspectives for the future.
The quality evaluation of MEDSEA_OMI_OHC_area_averaged_anomalies is based on the “multi-product” approach as introduced in the second issue of the Ocean State Report (von Schuckmann et al., 2018), and following the MyOcean’s experience (Masina et al., 2017). 
Six global products and a regional (Mediterranean Sea) product have been used to build an ensemble mean, and its associated ensemble spread. The reference products are:
	The Mediterranean Sea Reanalysis at 1/24 degree horizontal resolution (MEDSEA_MULTIYEAR_PHY_006_004, DOI: https://doi.org/10.25423/CMCC/MEDSEA_MULTIYEAR_PHY_006_004_E3R1, Escudier et al., 2020)
	Four global reanalyses at 1/4 degree horizontal resolution (GLOBAL_REANALYSIS_PHY_001_031): 
GLORYS, C-GLORS, ORAS5, FOAM
	Two observation based products: 
CORA (INSITU_GLO_TS_REP_OBSERVATIONS_013_001_b) and 
ARMOR3D (MULTIOBS_GLO_PHY_TSUV_3D_MYNRT_015_012). 
Details on the products are delivered in the PUM and QUID of this OMI. 

**CMEMS KEY FINDINGS**

The ensemble mean ocean heat content anomaly time series over the Mediterranean Sea shows a continuous increase in the period 1993-2019 at rate of 1.4±0.3 W/m2 in the upper 700m. After 2005 the rate has clearly increased with respect the previous decade, in agreement with Iona et al. (2018).

**DOI (product):** 
https://doi.org/10.48670/moi-00261

**References:**

* Escudier, R., Clementi, E., Omar, M., Cipollone, A., Pistoia, J., Aydogdu, A., Drudi, M., Grandi, A., Lyubartsev, V., Lecci, R., Cretí, S., Masina, S., Coppini, G., & Pinardi, N. (2020). Mediterranean Sea Physical Reanalysis (CMEMS MED-Currents) (Version 1) [Data set]. Copernicus Monitoring Environment Marine Service (CMEMS). https://doi.org/10.25423/CMCC/MEDSEA_MULTIYEAR_PHY_006_004_E3R1
* Iona, A., A. Theodorou, S. Sofianos, S. Watelet, C. Troupin, J.-M. Beckers, 2018: Mediterranean Sea climatic indices: monitoring long term variability and climate changes, Earth Syst. Sci. Data Discuss., https://doi.org/10.5194/essd-2018-51, in review.
* Masina S., A. Storto, N. Ferry, M. Valdivieso, K. Haines, M. Balmaseda, H. Zuo, M. Drevillon, L. Parent, 2017: An ensemble of eddy-permitting global ocean reanalyses from the MyOcean project. Climate Dynamics, 49 (3): 813-841. DOI: 10.1007/s00382-015-2728-5
* von Schuckmann, K., F. Gaillard and P.-Y. Le Traon, 2009: Global hydrographic variability patterns during 2003-2008, Journal of Geophysical Research, 114, C09007, doi:10.1029/2008JC005237.
* von Schuckmann et al., 2016: Ocean heat content. In: The Copernicus Marine Environment Monitoring Service Ocean State Report, issue 1, Journal of Operational Oceanography, Volume 9, 2016 - Issue sup2: The Copernicus Marine Environment Monitoring Service Ocean, http://dx.doi.org/10.1080/1755876X.2016.1273446.
* von Schuckmann et al., 2018: Ocean heat content. In: The Copernicus Marine Environment Monitoring Service Ocean State Report, issue 2, Journal of Operational Oceanography, 11:sup1, s1-s142, DOI: 10.1080/1755876X.2018.1489208





<img src="https://documentation.marine.copernicus.eu/IMG/MEDSEA_OMI_SEASTATE_extreme_var_swh_mean_and_anomaly.png" width="300" height="200"/>



# Mediterranean Sea Significant Wave Height extreme from Reanalysis


**Product ID:** MEDSEA_OMI_SEASTATE_extreme_var_swh_mean_and_anomaly

**Description:** **DEFINITION**

The CMEMS MEDSEA_OMI_seastate_extreme_var_swh_mean_and_anomaly OMI indicator is based on the computation of the annual 99th percentile of Significant Wave Height (SWH) from model data. Two different CMEMS products are used to compute the indicator: The Iberia-Biscay-Ireland Multi Year Product (MEDSEA_MULTIYEAR_WAV_006_012) and the Analysis product (MEDSEA_ANALYSIS_FORECAST_WAV_006_017).
Two parameters have been considered for this OMI:
* Map of the 99th mean percentile: It is obtained from the Multy Year Product, the annual 99th percentile is computed for each year of the product. The percentiles are temporally averaged in the whole period (1993-2019).
* Anomaly of the 99th percentile in 2020: The 99th percentile of the year 2020 is computed from the Analysis product. The anomaly is obtained by subtracting the mean percentile to the percentile in 2020.
This indicator is aimed at monitoring the extremes of annual significant wave height and evaluate the spatio-temporal variability. The use of percentiles instead of annual maxima, makes this extremes study less affected by individual data. This approach was first successfully applied to sea level variable (Pérez Gómez et al., 2016) and then extended to other essential variables, such as sea surface temperature and significant wave height (Pérez Gómez et al 2018 and Álvarez-Fanjul et al., 2019). Further details and in-depth scientific evaluation can be found in the CMEMS Ocean State report (Álvarez- Fanjul et al., 2019).

**CONTEXT**

The sea state and its related spatio-temporal variability affect maritime activities and the physical connectivity between offshore waters and coastal ecosystems, impacting therefore on the biodiversity of marine protected areas (González-Marco et al., 2008; Savina et al., 2003; Hewitt, 2003). Over the last decades, significant attention has been devoted to extreme wave height events since their destructive effects in both the shoreline environment and human infrastructures have prompted a wide range of adaptation strategies to deal with natural hazards in coastal areas (Hansom et al., 2014). Complementarily, there is also an emerging question about the role of anthropogenic global climate change on present and future extreme wave conditions.
The Mediterranean Sea is an almost enclosed basin where the complexity of its orographic characteristics deeply influences the atmospheric circulation at local scale, giving rise to strong regional wind regimes (Drobinski et al. 2018). Therefore, since waves are primarily driven by winds, high waves are present over most of the Mediterranean Sea and tend to reach the highest values where strong wind and long fetch (i.e. the horizontal distance over which wave-generating winds blow) are simultaneously present (Lionello et al. 2006). Specifically, as seen in figure and in agreement with other studies (e.g. Sartini et al. 2017), the highest values (5 – 6 m in figure, top) extend from the Gulf of Lion to the southwestern Sardinia through the Balearic Sea and are sustained southwards approaching the Algerian coast. They result from northerly winds dominant in the western Mediterranean Sea (Mistral or Tramontana), that become stronger due to orographic effects (Menendez et al. 2014), and act over a large area. In the Ionian Sea, the northerly Mistral wind is still the main cause of high waves (4-5 m in figure, top). In the Aegean and Levantine Seas, high waves (4-5 m in figure, top) are caused by the northerly Bora winds, prevalent in winter, and the northerly Etesian winds, prevalent in summer (Lionello et al. 2006; Chronis et al. 2011; Menendez et al. 2014). In general, northerly winds are responsible for most high waves in the Mediterranean (e.g. Chronis et al. 2011; Menendez et al. 2014). In agreement with figure (top), studies on the eastern Mediterranean and the Hellenic Seas have found that the typical wave height range in the Aegean Sea is similar to the one observed in the Ionian Sea despite the shorter fetches characterizing the former basin (Zacharioudaki et al. 2015). This is because of the numerous islands in the Aegean Sea which cause wind funneling and enhance the occurrence of extreme winds and thus of extreme waves (Kotroni et al. 2001). Special mention should be made of the high waves, sustained throughout the year, observed east and west of the island of Crete, i.e. around the exiting points of the northerly airflow in the Aegean Sea (Zacharioudaki et al. 2015). This airflow is characterized by consistently high magnitudes that are sustained during all seasons in contrast to other airflows in the Mediterranean Sea that exhibit a more pronounced seasonality (Chronis et al. 2011). 

**CMEMS KEY FINDINGS**

In 2020 (bottom panel), higher-than-average values of the 99th percentile of Significant Wave Height are seen over most of the northern Mediterranean Sea, in the eastern Alboran Sea, and along stretches of the African coast (Tunisia, Libya and Egypt). In many cases they exceed the climatic standard deviation. Regions where the climatic standard deviation is exceeded twice are the European and African coast of the eastern Alboran Sea, a considerable part of the eastern Spanish coast, the Ligurian Sea and part of the east coast of France as well as areas of the southern Adriatic. These anomalies correspond to the maximum positive anomalies computed in the Mediterranean Sea for year 2020 with values that reach up to 1.1 m. Spatially constrained maxima are also found at other coastal stretches (e.g. Algeri, southeast Sardinia).  Part of the positive anomalies found along the French and Spanish coast, including the coast of the Balearic Islands, can be associated with the wind storm “Gloria” (19/1 – 24/1) during which exceptional eastern winds originated in the Ligurian Sea and propagated westwards. The storm, which was of a particularly high intensity and long duration, caused record breaking wave heights in the region, and, in return, great damage to the coast (Amores et al., 2020; de Alfonso et al., 2021). Other storms that could have contributed to the positive anomalies observed in the western Mediterranean Sea include: storm Karine (25/2 – 5/4), which caused high waves from the eastern coast of Spain to the Balearic Islands (Copernicus, Climate Change Service, 2020); storm Bernardo (7/11 – 18/11) which also affected the Balearic islands and the Algerian coast and; storm Hervé (2/2 – 8/2) during which the highest wind gust was recorded at north Corsica (Wikiwand, 2021). In the eastern Mediterranean Sea, the medicane Ianos (14/9 – 21/9) may have contributed to the positive anomalies shown in the central Ionian Sea since this area coincides with the area of peak wave height values during the medicane (Copernicus, 2020a and Copernicus, 2020b). Otherwise, higher-than-average values in the figure are the result of severe, yet not unusual, wind events, which occurred during the year. Negative anomalies occur over most of the southern Mediterranean Sea, east of the Alboran Sea. The maximum negative anomalies reach about -1 m and are located in the southeastern Ionian Sea and west of the south part of mainland Greece as well as in coastal locations of the north and east Aegean  They appear to be quite unusual since they are greater than two times the climatic standard deviation in the region. They could imply less severe southerly wind activity during 2020 (Drobinski et al., 2018). 

Note: The key findings will be updated annually in November, in line with OMI evolutions.

**DOI (product):** 
https://doi.org/10.48670/moi-00262

**References:**

* Álvarez Fanjul E, Pascual Collar A, Pérez Gómez B, De Alfonso M, García Sotillo M, Staneva J, Clementi E, Grandi A, Zacharioudaki A, Korres G, Ravdas M, Renshaw R, Tinker J, Raudsepp U, Lagemaa P, Maljutenko I, Geyer G, Müller M, Çağlar Yumruktepe V. Sea level, sea surface temperature and SWH extreme percentiles: combined analysis from model results and in situ observations, Section 2.7, p:31. In: Schuckmann K, Le Traon P-Y, Smith N, Pascual A, Djavidnia S, Gattuso J-P, Grégoire M, Nolan G, et al. 2019. Copernicus Marine Service Ocean State Report, Issue 3, Journal of Operational Oceanography, 12:sup1, S1-S123, DOI: 10.1080/1755876X.2019.1633075
* Amores, A., Marcos, M., Carrió, Di.S., Gomez-Pujol, L., 2020. Coastal impacts of Storm Gloria (January 2020) over the north-western Mediterranean. Nat. Hazards Earth Syst. Sci. 20, 1955–1968. doi:10.5194/nhess-20-1955-2020
* Chronis T, Papadopoulos V, Nikolopoulos EI. 2011. QuickSCAT observations of extreme wind events over the Mediterranean and Black Seas during 2000-2008. Int J Climatol. 31: 2068–2077.
* Copernicus: Climate Change Service. 2020a (Last accessed July 2021): URL: https://surfobs.climate.copernicus.eu/stateoftheclimate/march2020.php
* Copernicus, Copernicus Marine Service. 2020b (Last accessed July 2021): URL: https://marine.copernicus.eu/news/following-cyclone-ianos-across-mediterranean-sea
* de Alfonso, M., Lin-Ye, J., García-Valdecasas, J.M., Pérez-Rubio, S., Luna, M.Y., Santos-Muñoz, D., Ruiz, M.I., Pérez-Gómez, B., Álvarez-Fanjul, E., 2021. Storm Gloria: Sea State Evolution Based on in situ Measurements and Modeled Data and Its Impact on Extreme Values. Front. Mar. Sci. 8, 1–17. doi:10.3389/fmars.2021.646873
* Drobinski P, Alpert P, Cavicchia L, Flaoumas E, Hochman A, Kotroni V. 2018. Strong winds Observed trends, future projections, Sub-chapter 1.3.2, p. 115-122. In: Moatti JP, Thiébault S (dir.). The Mediterranean region under climate change: A scientific update. Marseille: IRD Éditions.
* González-Marco D, Sierra J P, Ybarra O F, Sánchez-Arcilla A. 2008. Implications of long waves in harbor management: The Gijón port case study. Ocean & Coastal Management, 51, 180-201. doi:10.1016/j.ocecoaman.2007.04.001.
* Hanson et al., 2014. Extreme Waves: Causes, Characteristics and Impact on Coastal Environments and Society January 2014 In book: Coastal and Marine Hazards, Risks, and Disasters Edition: Hazards and Disasters Series, Elsevier Major Reference Works Chapter: Chapter 11: Extreme Waves: Causes, Characteristics and Impact on Coastal Environments and Society. Publisher: Elsevier Editors: Ellis, J and Sherman, D. J.
* Hewit J E, Cummings V J, Elis J I, Funnell G, Norkko A, Talley T S, Thrush S.F. 2003. The role of waves in the colonisation of terrestrial sediments deposited in the marine environment. Journal of Experimental marine Biology and Ecology, 290, 19-47, doi:10.1016/S0022-0981(03)00051-0.
* Kotroni V, Lagouvardos K, Lalas D. 2001. The effect of the island of Crete on the Etesian winds over the Aegean Sea. Q J R Meteorol Soc. 127: 1917–1937. doi:10.1002/qj.49712757604
* Lionello P, Rizzoli PM, Boscolo R. 2006. Mediterranean climate variability, developments in earth and environmental sciences. Elsevier.
* Menendez M, García-Díez M, Fita L, Fernández J, Méndez FJ, Gutiérrez JM. 2014. High-resolution sea wind hindcasts over the Mediterranean area. Clim Dyn. 42:1857–1872.
* Pérez-Gómez B, Álvarez-Fanjul E, She J, Pérez-González I, Manzano F. 2016. Extreme sea level events, Section 4.4, p:300. In: Von Schuckmann K, Le Traon PY, Alvarez-Fanjul E, Axell L, Balmaseda M, Breivik LA, Brewin RJW, Bricaud C, Drevillon M, Drillet Y, Dubois C , Embury O, Etienne H, García-Sotillo M, Garric G, Gasparin F, Gutknecht E, Guinehut S, Hernandez F, Juza M, Karlson B, Korres G, Legeais JF, Levier B, Lien VS, Morrow R, Notarstefano G, Parent L, Pascual A, Pérez-Gómez B, Perruche C, Pinardi N, Pisano A, Poulain PM , Pujol IM, Raj RP, Raudsepp U, Roquet H, Samuelsen A, Sathyendranath S, She J, Simoncelli S, Solidoro C, Tinker J, Tintoré J, Viktorsson L, Ablain M, Almroth-Rosell E, Bonaduce A, Clementi E, Cossarini G, Dagneaux Q, Desportes C, Dye S, Fratianni C, Good S, Greiner E, Gourrion J, Hamon M, Holt J, Hyder P, Kennedy J, Manzano-Muñoz F, Melet A, Meyssignac B, Mulet S, Nardelli BB, O’Dea E, Olason E, Paulmier A, Pérez-González I, Reid R, Racault MF, Raitsos DE, Ramos A, Sykes P, Szekely T, Verbrugge N. 2016. The Copernicus Marine Environment Monitoring Service Ocean State Report, Journal of Operational Oceanography. 9 (sup2): 235-320. http://dx.doi.org/10.1080/1755876X.2016.1273446
* Pérez Gómez B., De Alfonso M., Zacharioudaki A., Pérez González I., Álvarez Fanjul E., Müller M., Marcos M., Manzano F., Korres G., Ravdas M., Tamm S. 2018. Sea level, SST and waves: extremes variability. In: Copernicus Marine Service Ocean State Report, Issue 2, Journal of Operational Oceanography, 11:sup1, Chap. 3.1, s79–s88, DOI: https://doi.org/10.1080/1755876X.2018.1489208
* Sartini L, Besio G, Cassola F. 2017. Spatio-temporal modelling of extreme wave heights in the Mediterranean Sea. Ocean Modelling, 117, 52-69.
* Savina H, Lefevre J-M, Josse P, Dandin P. 2003. Definition of warning criteria. Proceedings of MAXWAVE Final Meeting, October 8-11, Geneva, Switzerland.
* Wikiwand: 2019 - 20 European windstorm season. URL: https://www.wikiwand.com/en/2019%E2%80%9320_European_windstorm_season
* Zacharioudaki A, Korres G, Perivoliotis L, 2015. Wave climate of the Hellenic Seas obtained from a wave hindcast for the period 1960–2001. Ocean Dynamics. 65: 795–816. https://doi.org/10.1007/s10236-015-0840-z





<img src="https://documentation.marine.copernicus.eu/IMG/MEDSEA_OMI_TEMPSAL_extreme_var_temp_mean_and_anomaly.png" width="300" height="200"/>



# Mediterranean Sea Surface Temperature extreme from Reanalysis


**Product ID:** MEDSEA_OMI_TEMPSAL_extreme_var_temp_mean_and_anomaly

**Description:** **DEFINITION**

The CMEMS MEDSEA_OMI_tempsal_extreme_var_temp_mean_and_anomaly OMI indicator is based on the computation of the annual 99th percentile of Sea Surface Temperature (SST) from model data. Two different CMEMS products are used to compute the indicator: The Iberia-Biscay-Ireland Multi Year Product (MEDSEA_MULTIYEAR_PHY_006_004) and the Analysis product (MEDSEA_ANALYSISFORECAST_PHY_006_013).
Two parameters have been considered for this OMI:
* Map of the 99th mean percentile: It is obtained from the Multi Year Product, the annual 99th percentile is computed for each year of the product. The percentiles are temporally averaged over the whole period (1987-2019).
* Anomaly of the 99th percentile in 2020: The 99th percentile of the year 2020 is computed from the Near Real Time product. The anomaly is obtained by subtracting the mean percentile from the 2020 percentile.
This indicator is aimed at monitoring the extremes of sea surface temperature every year and at checking their variations in space. The use of percentiles instead of annual maxima, makes this extremes study less affected by individual data. This study of extreme variability was first applied to the sea level variable (Pérez Gómez et al 2016) and then extended to other essential variables, such as sea surface temperature and significant wave height (Pérez Gómez et al 2018 and Alvarez Fanjul et al., 2019). More details and a full scientific evaluation can be found in the CMEMS Ocean State report (Alvarez Fanjul et al., 2019).

**CONTEXT**

The Sea Surface Temperature is one of the Essential Ocean Variables, hence the monitoring of this variable is of key importance, since its variations can affect the ocean circulation, marine ecosystems, and ocean-atmosphere exchange processes. As the oceans continuously interact with the atmosphere, trends of sea surface temperature can also have an effect on the global climate. In recent decades (from mid ‘80s) the Mediterranean Sea showed a trend of increasing temperatures (Ducrocq et al., 2016), which has been observed also by means of the CMEMS SST_MED_SST_L4_REP_OBSERVATIONS_010_021 satellite product and reported in the following CMEMS OMI: MEDSEA_OMI_TEMPSAL_sst_area_averaged_anomalies and MEDSEA_OMI_TEMPSAL_sst_trend.
The Mediterranean Sea is a semi-enclosed sea characterized by an annual average surface temperature which varies horizontally from ~14°C in the Northwestern part of the basin to ~23°C in the Southeastern areas. Large-scale temperature variations in the upper layers are mainly related to the heat exchange with the atmosphere and surrounding oceanic regions. The Mediterranean Sea annual 99th percentile presents a significant interannual and multidecadal variability with a significant increase starting from the 80’s as shown in Marbà et al. (2015) which is also in good agreement with the multidecadal change of the mean SST reported in Mariotti et al. (2012). Moreover the spatial variability of the SST 99th percentile shows large differences at regional scale (Darmariaki et al., 2019; Pastor et al. 2018).

**CMEMS KEY FINDINGS**

The Mediterranean mean Sea Surface Temperature 99th percentile evaluated in the period 1987-2019 (upper panel) presents highest values (~ 28-30 °C) in the eastern Mediterranean-Levantine basin and along the Tunisian coasts especially in the area of the Gulf of Gabes, while the lowest (~ 23–25 °C) are found in the Gulf of Lyon (a deep water formation area), in the Alboran Sea (affected by incoming Atlantic waters) and the eastern part of the Aegean Sea (an upwelling region). These results are in agreement with previous findings in Darmariaki et al. (2019) and Pastor et al. (2018) and are consistent with the ones presented in CMEMS OSR3 (Alvarez Fanjul et al., 2019) for the period 1993-2016.
The 2020      Sea Surface Temperature 99th percentile anomaly map (bottom panel) shows a general positive pattern up to +3°C in the North-West Mediterranean area while colder anomalies are visible in the Gulf of Lion      and North Aegean Sea     . This Ocean Monitoring Indicator confirms the continuous warming of the SST and in particular it shows that the year 2020      is characterized by an overall increase of the extreme Sea Surface Temperature values in almost the whole domain with respect to the reference period. This finding can be probably affected      by the different dataset used to evaluate this anomaly map: the 2020      Sea Surface Temperature 99th percentile derived from the Near Real Time Analysis product compared to the mean (1987-2019) Sea Surface Temperature 99th percentile evaluated from the      Reanalysis product which, among the others, is characterized by different atmospheric forcing).

Note: The key findings will be updated annually in November, in line with OMI evolutions.

**DOI (product):** 
https://doi.org/10.48670/moi-00266

**References:**

* Álvarez Fanjul E, Pascual Collar A, Pérez Gómez B, De Alfonso M, García Sotillo M, Staneva J, Clementi E, Grandi A, Zacharioudaki A, Korres G, Ravdas M, Renshaw R, Tinker J, Raudsepp U, Lagemaa P, Maljutenko I, Geyer G, Müller M, Çağlar Yumruktepe V. Sea level, sea surface temperature and SWH extreme percentiles: combined analysis from model results and in situ observations, Section 2.7, p:31. In: Schuckmann K, Le Traon P-Y, Smith N, Pascual A, Djavidnia S, Gattuso J-P, Grégoire M, Nolan G, et al. 2019. Copernicus Marine Service Ocean State Report, Issue 3, Journal of Operational Oceanography, 12:sup1, S1-S123, DOI: 10.1080/1755876X.2019.1633075
* Darmaraki S, Somot S, Sevault F, Nabat P, Cabos W, Cavicchia L, et al. 2019. Future evolution of marine heatwaves in the Mediterranean Sea. Clim. Dyn. 53, 1371–1392. doi: 10.1007/s00382-019-04661-z
* Ducrocq V., Drobinski P., Gualdi S., Raimbault P. 2016. The water cycle in the Mediterranean. Chapter 1.2.1 in The Mediterranean region under climate change. IRD Éditions. DOI : 10.4000/books.irdeditions.22908.
* Marbà N, Jordà G, Agustí S, Girard C, Duarte CM. 2015. Footprints of climate change on Mediterranean Sea biota. Front.Mar.Sci.2:56. doi: 10.3389/fmars.2015.00056
* Mariotti A and Dell’Aquila A. 2012. Decadal climate variability in the Mediterranean region: roles of large-scale forcings and regional processes. Clim Dyn. 38,1129–1145. doi:10.1007/s00382-011-1056-7
* Pastor F, Valiente JA, Palau JL. 2018. Sea Surface Temperature in the Mediterranean: Trends and Spatial Patterns (1982–2016). Pure Appl. Geophys, 175: 4017. https://doi.org/10.1007/s00024-017-1739-zPérez-Gómez B, Álvarez-Fanjul E, She J, Pérez-González I, Manzano F. 2016. Extreme sea level events, Section 4.4, p:300. In: Von Schuckmann K, Le Traon PY, Alvarez-Fanjul E, Axell L, Balmaseda M, Breivik LA, Brewin RJW, Bricaud C, Drevillon M, Drillet Y, Dubois C , Embury O, Etienne H, García-Sotillo M, Garric G, Gasparin F, Gutknecht E, Guinehut S, Hernandez F, Juza M, Karlson B, Korres G, Legeais JF, Levier B, Lien VS, Morrow R, Notarstefano G, Parent L, Pascual A, Pérez-Gómez B, Perruche C, Pinardi N, Pisano A, Poulain PM , Pujol IM, Raj RP, Raudsepp U, Roquet H, Samuelsen A, Sathyendranath S, She J, Simoncelli S, Solidoro C, Tinker J, Tintoré J, Viktorsson L, Ablain M, Almroth-Rosell E, Bonaduce A, Clementi E, Cossarini G, Dagneaux Q, Desportes C, Dye S, Fratianni C, Good S, Greiner E, Gourrion J, Hamon M, Holt J, Hyder P, Kennedy J, Manzano-Muñoz F, Melet A, Meyssignac B, Mulet S, Nardelli BB, O’Dea E, Olason E, Paulmier A, Pérez-González I, Reid R, Racault MF, Raitsos DE, Ramos A, Sykes P, Szekely T, Verbrugge N. 2016. The Copernicus Marine Environment Monitoring Service Ocean State Report, Journal of Operational Oceanography. 9 (sup2): 235-320. http://dx.doi.org/10.1080/1755876X.2016.1273446
* Pérez Gómez B., De Alfonso M., Zacharioudaki A., Pérez González I., Álvarez Fanjul E., Müller M., Marcos M., Manzano F., Korres G., Ravdas M., Tamm S. 2018. Sea level, SST and waves: extremes variability. In: Copernicus Marine Service Ocean State Report, Issue 2, Journal of Operational Oceanography, 11:sup1, Chap. 3.1, s79–s88, DOI: https://doi.org/10.1080/1755876X.2018.1489208
* Pisano A, Marullo S, Artale V, Falcini F, Yang C, Leonelli FE, Santoleri R, Buongiorno Nardelli B. 2020. New Evidence of Mediterranean Climate Change and Variability from Sea Surface Temperature Observations. Remote Sens. 2020, 12, 132.





<img src="https://documentation.marine.copernicus.eu/IMG/MEDSEA_OMI_TEMPSAL_sst_area_averaged_anomalies.png" width="300" height="200"/>



# Mediterranean Sea Surface Temperature time series and trend from Observations Reprocessing


**Product ID:** MEDSEA_OMI_TEMPSAL_sst_area_averaged_anomalies

**Description:** **DEFINITION**

The medsea_omi_tempsal_sst_area_averaged_anomalies product for 2023 includes unfiltered Sea Surface Temperature (SST) anomalies, given as monthly mean time series starting on 1982 and averaged over the Mediterranean Sea, and 24-month filtered SST anomalies, obtained by using the X11-seasonal adjustment procedure (see e.g. Pezzulli et al., 2005; Pisano et al., 2020). This OMI is derived from the CMEMS Reprocessed Mediterranean L4 SST satellite product (SST_MED_SST_L4_REP_OBSERVATIONS_010_021, see also the OMI QUID, http://marine.copernicus.eu/documents/QUID/CMEMS-OMI-QUID-MEDSEA-SST.pdf), which provides the SSTs used to compute the evolution of SST anomalies (unfiltered and filtered) over the Mediterranean Sea. This reprocessed product consists of daily (nighttime) optimally interpolated 0.05° grid resolution SST maps over the Mediterranean Sea built from the ESA Climate Change Initiative (CCI) (Merchant et al., 2019) and Copernicus Climate Change Service (C3S) initiatives, including also an adjusted version of the AVHRR Pathfinder dataset version 5.3 (Saha et al., 2018) to increase the input observation coverage. Anomalies are computed against the 1991-2020 reference period. The 30-year climatology 1991-2020 is defined according to the WMO recommendation (WMO, 2017) and recent U.S. National Oceanic and Atmospheric Administration practice (https://wmo.int/media/news/updated-30-year-reference-period-reflects-changing-climate). The reference for this OMI can be found in the first and second issue of the Copernicus Marine Service Ocean State Report (OSR), Section 1.1 (Roquet et al., 2016; Mulet et al., 2018).

**CONTEXT**

Sea surface temperature (SST) is a key climate variable since it deeply contributes in regulating climate and its variability (Deser et al., 2010). SST is then essential to monitor and characterise the state of the global climate system (GCOS 2010). Long-term SST variability, from interannual to (multi-)decadal timescales, provides insight into the slow variations/changes in SST, i.e. the temperature trend (e.g., Pezzulli et al., 2005). In addition, on shorter timescales, SST anomalies become an essential indicator for extreme events, as e.g. marine heatwaves (Hobday et al., 2018). The Mediterranean Sea is a climate change hotspot (Giorgi F., 2006). Indeed, Mediterranean SST has experienced a continuous warming trend since the beginning of 1980s (e.g., Pisano et al., 2020; Pastor et al., 2020). Specifically, since the beginning of the 21st century (from 2000 onward), the Mediterranean Sea featured the highest SSTs and this warming trend is expected to continue throughout the 21st century (Kirtman et al., 2013). 

**KEY FINDINGS**

During 2023, the Mediterranean Sea continued experiencing the intense sea surface temperatures’ warming (marine heatwave event) that started in May 2022 (Marullo et al., 2023). The basin average SST anomaly was 0.9 ± 0.1 °C in 2023, the highest in this record. The Mediterranean SST warming started in May 2022, when the mean anomaly increased abruptly from 0.01 °C (April) to 0.76 °C (May), reaching the highest values during June (1.66 °C) and July (1.52 °C), and persisting until summer 2023 with anomalies around 1 °C above the 1991-2020 climatology. The peak of July 2023 (1.76 °C) set the record of highest SST anomaly ever recorded since 1982. The 2022/2023 Mediterranean marine heatwave is comparable to that occurred in 2003 (see e.g. Olita et al., 2007) in terms of anomaly magnitude but longer in duration.
Over the period 1982-2023, the Mediterranean SST has warmed at a rate of 0.041 ± 0.001 °C/year, which corresponds to an average increase of about 1.7 °C during these last 42 years. Within its error (namely, the 95% confidence interval), this warming trend is consistent with recent trend estimates in the Mediterranean Sea (Pisano et al., 2020; Pastor et al., 2020). However, though the linear trend being constantly increasing during the whole period, the picture of the Mediterranean SST trend in 2022 seems to reveal a restarting after the pause occurred in the last years (since 2015-2021).

**DOI (product):** 
https://doi.org/10.48670/moi-00268

**References:**

* Giorgi, F., 2006. Climate change hot-spots. Geophys. Res. Lett., 33:L08707, https://doi.org/10.1029/2006GL025734
* Deser, C., Alexander, M. A., Xie, S.-P., Phillips, A. S., 2010. Sea Surface Temperature Variability: Patterns and Mechanisms. Annual Review of Marine Science 2010 2:1, 115-143. https://doi.org/10.1146/annurev-marine-120408-151453
* GCOS. Global Climate Observing System. 2010. Update of the Implementation Plan for the Global Observing System for Climate in Support of the UNFCCC (GCO-138).
* Hobday, A. J., Oliver, E. C., Gupta, A. S., Benthuysen, J. A., Burrows, M. T., Donat, M. G., ... & Smale, D. A. (2018). Categorizing and naming marine heatwaves. Oceanography, 31(2), 162-173.
* Merchant, C. J., Embury, O., Bulgin, C. E., Block, T., Corlett, G. K., Fiedler, E., ... & Eastwood, S. (2019). Satellite-based time-series of sea-surface temperature since 1981 for climate applications. Scientific data, 6(1), 1-18.
* Mulet, S., Buongiorno Nardelli, B., Good, S., Pisano, A., Greiner, E., Monier, M., Autret, E., Axell, L., Boberg, F., Ciliberti, S., Drévillon, M., Droghei, R., Embury, O., Gourrion, J., Høyer, J., Juza, M., Kennedy, J., Lemieux-Dudon, B., Peneva, E., Reid, R., Simoncelli, S., Storto, A., Tinker, J., Von Schuckmann, K., Wakelin, S. L., 2018. Ocean temperature and salinity. In: Copernicus Marine Service Ocean State Report, Issue 2, Journal of Operational Oceanography, 11:sup1, s5–s13, DOI: 10.1080/1755876X.2018.1489208
* Pezzulli, S., Stephenson, D. B., Hannachi, A., 2005. The Variability of Seasonality. J. Climate. 18:71–88. doi:10.1175/JCLI-3256.1.
* Roquet, H., Pisano, A., Embury, O., 2016. Sea surface temperature. In: von Schuckmann et al. 2016, The Copernicus Marine Environment Monitoring Service Ocean State Report, Jour. Operational Ocean., vol. 9, suppl. 2. doi:10.1080/1755876X.2016.1273446.
* Saha, Korak; Zhao, Xuepeng; Zhang, Huai-min; Casey, Kenneth S.; Zhang, Dexin; Baker-Yeboah, Sheekela; Kilpatrick, Katherine A.; Evans, Robert H.; Ryan, Thomas; Relph, John M. (2018). AVHRR Pathfinder version 5.3 level 3 collated (L3C) global 4km sea surface temperature for 1981-Present. NOAA National Centers for Environmental Information. Dataset. https://doi.org/10.7289/v52j68xx
* Sen, P. K., 1968. Estimates of the regression coefficient based on Kendall’s tau. J Am Statist Assoc. 63:1379–1389.
* Pisano, A., Marullo, S., Artale, V., Falcini, F., Yang, C., Leonelli, F. E., Santoleri, R. and Buongiorno Nardelli, B.: New Evidence of Mediterranean Climate Change and Variability from Sea Surface Temperature Observations, Remote Sens., 12(1), 132, doi:10.3390/rs12010132, 2020.
* Pastor, F., Valiente, J. A., & Khodayar, S. (2020). A Warming Mediterranean: 38 Years of Increasing Sea Surface Temperature. Remote Sensing, 12(17), 2687.
* Olita, A., Sorgente, R., Natale, S., Gaberšek, S., Ribotti, A., Bonanno, A., & Patti, B. (2007). Effects of the 2003 European heatwave on the Central Mediterranean Sea: surface fluxes and the dynamical response. Ocean Science, 3(2), 273-289.
* Sen, P. K., 1968. Estimates of the regression coefficient based on Kendall’s tau. J Am Statist Assoc. 63:1379–1389.





<img src="https://documentation.marine.copernicus.eu/IMG/MEDSEA_OMI_TEMPSAL_sst_trend.png" width="300" height="200"/>



# Mediterranean Sea Surface Temperature cumulative trend map from Observations Reprocessing


**Product ID:** MEDSEA_OMI_TEMPSAL_sst_trend

**Description:** **DEFINITION**

The medsea_omi_tempsal_sst_trend product includes the cumulative/net Sea Surface Temperature (SST) trend for the Mediterranean Sea over the period 1982-2023, i.e. the rate of change (°C/year) multiplied by the number years in the time series (42 years). This OMI is derived from the CMEMS Reprocessed Mediterranean L4 SST product (SST_MED_SST_L4_REP_OBSERVATIONS_010_021, see also the OMI QUID, http://marine.copernicus.eu/documents/QUID/CMEMS-OMI-QUID-MEDSEA-SST.pdf), which provides the SSTs used to compute the SST trend over the Mediterranean Sea. This reprocessed product consists of daily (nighttime) optimally interpolated 0.05° grid resolution SST maps over the Mediterranean Sea built from the ESA Climate Change Initiative (CCI) (Merchant et al., 2019) and Copernicus Climate Change Service (C3S) initiatives, including also an adjusted version of the AVHRR Pathfinder dataset version 5.3 (Saha et al., 2018) to increase the input observation coverage. Trend analysis has been performed by using the X-11 seasonal adjustment procedure (see e.g. Pezzulli et al., 2005; Pisano et al., 2020), which has the effect of filtering the input SST time series acting as a low bandpass filter for interannual variations. Mann-Kendall test and Sens’s method (Sen 1968) were applied to assess whether there was a monotonic upward or downward trend and to estimate the slope of the trend and its 95% confidence interval. The reference for this OMI can be found in the first and second issue of the Copernicus Marine Service Ocean State Report (OSR), Section 1.1 (Roquet et al., 2016; Mulet et al., 2018).

**CONTEXT**

Sea surface temperature (SST) is a key climate variable since it deeply contributes in regulating climate and its variability (Deser et al., 2010). SST is then essential to monitor and characterize the state of the global climate system (GCOS 2010). Long-term SST variability, from interannual to (multi-)decadal timescales, provides insight into the slow variations/changes in SST, i.e. the temperature trend (e.g., Pezzulli et al., 2005). In addition, on shorter timescales, SST anomalies become an essential indicator for extreme events, as e.g. marine heatwaves (Hobday et al., 2018). The Mediterranean Sea is a climate change hotspot (Giorgi F., 2006). Indeed, Mediterranean SST has experienced a continuous warming trend since the beginning of 1980s (e.g., Pisano et al., 2020; Pastor et al., 2020). Specifically, since the beginning of the 21st century (from 2000 onward), the Mediterranean Sea featured the highest SSTs and this warming trend is expected to continue throughout the 21st century (Kirtman et al., 2013).  

**KEY FINDINGS**

Over the past four decades (1982-2023), the Mediterranean Sea surface temperature (SST) warmed at a rate of 0.041 ± 0.001 °C per year, corresponding to a mean surface temperature warming of about 1.7 °C. The spatial pattern of the Mediterranean SST trend shows a general warming tendency, ranging from 0.002 °C/year to 0.063 °C/year. Overall, a higher SST trend intensity characterizes the Eastern and Central Mediterranean basin with respect to the Western basin. In particular, the Balearic Sea, Tyrrhenian and Adriatic Seas, as well as the northern Ionian and Aegean-Levantine Seas show the highest SST trends (from 0.04 °C/year to 0.05 °C/year on average). Trend patterns of warmer intensity characterize some of main sub-basin Mediterranean features, such as the Pelops Anticyclone, the Cretan gyre and the Rhodes Gyre. On the contrary, less intense values characterize the southern Mediterranean Sea (toward the African coast), where the trend attains around 0.025 °C/year. The SST warming rate spatial change, mostly showing an eastward increase pattern (see, e.g., Pisano et al., 2020, and references therein), i.e. the Levantine basin getting warm faster than the Western, appears now to have tilted more along a North-South direction.

**DOI (product):** 
https://doi.org/10.48670/moi-00269

**References:**

* Deser, C., Alexander, M. A., Xie, S.-P., Phillips, A. S., 2010. Sea Surface Temperature Variability: Patterns and Mechanisms. Annual Review of Marine Science 2010 2:1, 115-143. https://doi.org/10.1146/annurev-marine-120408-151453
* GCOS. Global Climate Observing System. 2010. Update of the Implementation Plan for the Global Observing System for Climate in Support of the UNFCCC (GCO-138).
* Giorgi, F., 2006. Climate change hot-spots. Geophys. Res. Lett., 33:L08707, https://doi.org/10.1029/2006GL025734 Hobday, A. J., Oliver, E. C., Gupta, A. S., Benthuysen, J. A., Burrows, M. T., Donat, M. G., ... & Smale, D. A. (2018). Categorizing and naming marine heatwaves. Oceanography, 31(2), 162-173.
* Kirtman, B., Power, S. B, Adedoyin, J. A., Boer, G. J., Bojariu, R. et al., 2013. Near-term climate change: Projections and Predictability. In: Stocker, T.F., et al. (Eds.), Climate change 2013: The physical science basis. Contribution of Working Group I to the Fifth Assessment Report of the Intergovernmental Panel on Climate Change. Cambridge University Press, Cambridge and New York.
* Merchant, C. J., Embury, O., Bulgin, C. E., Block, T., Corlett, G. K., Fiedler, E., ... & Eastwood, S. (2019). Satellite-based time-series of sea-surface temperature since 1981 for climate applications. Scientific data, 6(1), 1-18.
* Mulet, S., Buongiorno Nardelli, B., Good, S., Pisano, A., Greiner, E., Monier, M., Autret, E., Axell, L., Boberg, F., Ciliberti, S., Drévillon, M., Droghei, R., Embury, O., Gourrion, J., Høyer, J., Juza, M., Kennedy, J., Lemieux-Dudon, B., Peneva, E., Reid, R., Simoncelli, S., Storto, A., Tinker, J., Von Schuckmann, K., Wakelin, S. L., 2018. Ocean temperature and salinity. In: Copernicus Marine Service Ocean State Report, Issue 2, Journal of Operational Oceanography, 11:sup1, s5–s13, DOI: 10.1080/1755876X.2018.1489208
* Pastor, F., Valiente, J. A., & Khodayar, S. (2020). A Warming Mediterranean: 38 Years of Increasing Sea Surface Temperature. Remote Sensing, 12(17), 2687.
* Pezzulli, S., Stephenson, D. B., Hannachi, A., 2005. The Variability of Seasonality. J. Climate. 18:71–88. doi:10.1175/JCLI-3256.1.
* Pisano, A., Marullo, S., Artale, V., Falcini, F., Yang, C., Leonelli, F. E., Santoleri, R. and Buongiorno Nardelli, B.: New Evidence of Mediterranean Climate Change and Variability from Sea Surface Temperature Observations, Remote Sens., 12(1), 132, doi:10.3390/rs12010132, 2020.
* Roquet, H., Pisano, A., Embury, O., 2016. Sea surface temperature. In: von Schuckmann et al. 2016, The Copernicus Marine Environment Monitoring Service Ocean State Report, Jour. Operational Ocean., vol. 9, suppl. 2. doi:10.1080/1755876X.2016.1273446.
* Saha, Korak; Zhao, Xuepeng; Zhang, Huai-min; Casey, Kenneth S.; Zhang, Dexin; Baker-Yeboah, Sheekela; Kilpatrick, Katherine A.; Evans, Robert H.; Ryan, Thomas; Relph, John M. (2018). AVHRR Pathfinder version 5.3 level 3 collated (L3C) global 4km sea surface temperature for 1981-Present. NOAA National Centers for Environmental Information. Dataset. https://doi.org/10.7289/v52j68xx
* Sen, P. K., 1968. Estimates of the regression coefficient based on Kendall’s tau. J Am Statist Assoc. 63:1379–1389.
* Hobday, A. J., Oliver, E. C., Gupta, A. S., Benthuysen, J. A., Burrows, M. T., Donat, M. G., ... & Smale, D. A. (2018). Categorizing and naming marine heatwaves. Oceanography, 31(2), 162-173.
* Sen, P. K., 1968. Estimates of the regression coefficient based on Kendall’s tau. J Am Statist Assoc. 63:1379–1389.





<img src="https://mdl-metadata.s3.waw3-1.cloudferro.com/metadata/thumbnails/MULTIOBS_GLO_BGC_NUTRIENTS_CARBON_PROFILES_MYNRT_015_009.jpg" width="300" height="200"/>



# Nutrient and carbon profiles vertical distribution


**Product ID:** MULTIOBS_GLO_BGC_NUTRIENTS_CARBON_PROFILES_MYNRT_015_009

**Description:** This product consists of vertical profiles of the concentration of nutrients (nitrates, phosphates, and silicates) and carbonate system variables (total alkalinity, dissolved inorganic carbon, pH, and partial pressure of carbon dioxide), computed for each Argo float equipped with an oxygen sensor.
The method called CANYON (Carbonate system and Nutrients concentration from hYdrological properties and Oxygen using a Neural-network) is based on a neural network trained using high-quality nutrient data collected over the last 30 years (GLODAPv2 database, https://www.glodap.info/). The method is applied to each Argo float equipped with an oxygen sensor using as input the properties measured by the float (pressure, temperature, salinity, oxygen), and its date and position.

**DOI (product):**
https://doi.org/10.48670/moi-00048

**References:**

* Sauzede R., H. C. Bittig, H. Claustre, O. Pasqueron de Fommervault, J.-P. Gattuso, L. Legendre and K. S. Johnson, 2017: Estimates of Water-Column Nutrient Concentrations and Carbonate System Parameters in the Global Ocean: A novel Approach Based on Neural Networks. Front. Mar. Sci. 4:128. doi: 10.3389/fmars.2017.00128.
* Bittig H. C., T. Steinhoff, H. Claustre, B. Fiedler, N. L. Williams, R. Sauzède, A. Körtzinger and J.-P. Gattuso,2018: An Alternative to Static Climatologies: Robust Estimation of Open Ocean CO2 Variables and Nutrient Concentrations From T, S, and O2 Data Using Bayesian Neural Networks. Front. Mar. Sci. 5:328. doi: 10.3389/fmars.2018.00328.





<img src="https://mdl-metadata.s3.waw3-1.cloudferro.com/metadata/thumbnails/MULTIOBS_GLO_BIO_BGC_3D_REP_015_010.jpg" width="300" height="200"/>



# Global Ocean 3D Chlorophyll-a concentration, Particulate Backscattering coefficient and Particulate Organic Carbon


**Product ID:** MULTIOBS_GLO_BIO_BGC_3D_REP_015_010

**Description:** This product consists of 3D fields of Particulate Organic Carbon (POC), Particulate Backscattering coefficient (bbp) and Chlorophyll-a concentration (Chla) at depth. The reprocessed product is provided at 0.25°x0.25° horizontal resolution, over 36 levels from the surface to 1000 m depth. 
A neural network method estimates both the vertical distribution of Chla concentration and of particulate backscattering coefficient (bbp), a bio-optical proxy for POC, from merged surface ocean color satellite measurements with hydrological properties and additional relevant drivers. 

**DOI (product):**
https://doi.org/10.48670/moi-00046

**References:**

* Sauzede R., H. Claustre, J. Uitz, C. Jamet, G. Dall’Olmo, F. D’Ortenzio, B. Gentili, A. Poteau, and C. Schmechtig, 2016: A neural network-based method for merging ocean color and Argo data to extend surface bio-optical properties to depth: Retrieval of the particulate backscattering coefficient, J. Geophys. Res. Oceans, 121, doi:10.1002/2015JC011408.





<img src="https://mdl-metadata.s3.waw3-1.cloudferro.com/metadata/thumbnails/MULTIOBS_GLO_BIO_CARBON_SURFACE_MYNRT_015_008.jpg" width="300" height="200"/>



# Surface ocean carbon fields


**Product ID:** MULTIOBS_GLO_BIO_CARBON_SURFACE_MYNRT_015_008

**Description:** This product corresponds to a REP L4 time series of monthly global reconstructed surface ocean pCO2, air-sea fluxes of CO2, pH, total alkalinity, dissolved inorganic carbon, saturation state with respect to calcite and aragonite, and associated uncertainties on a 0.25° x 0.25° regular grid. The product is obtained from an ensemble-based forward feed neural network approach mapping situ data for surface ocean fugacity (SOCAT data base, Bakker et al.  2016, https://www.socat.info/) and sea surface salinity, temperature, sea surface height, chlorophyll a, mixed layer depth and atmospheric CO2 mole fraction. Sea-air flux fields are computed from the air-sea gradient of pCO2 and the dependence on wind speed of Wanninkhof (2014). Surface ocean pH on total scale, dissolved inorganic carbon, and saturation states are then computed from surface ocean pCO2 and reconstructed surface ocean alkalinity using the CO2sys speciation software.

**DOI (product):**
https://doi.org/10.48670/moi-00047

**References:**

* Chau, T. T. T., Gehlen, M., and Chevallier, F.: A seamless ensemble-based reconstruction of surface ocean pCO2 and air–sea CO2 fluxes over the global coastal and open oceans, Biogeosciences, 19, 1087–1109, https://doi.org/10.5194/bg-19-1087-2022, 2022.
* Chau, T.-T.-T., Chevallier, F., & Gehlen, M. (2024). Global analysis of surface ocean CO2 fugacity and air-sea fluxes with low latency. Geophysical Research Letters, 51, e2023GL106670. https://doi.org/10.1029/2023GL106670
* Chau, T.-T.-T., Gehlen, M., Metzl, N., and Chevallier, F.: CMEMS-LSCE: a global, 0.25°, monthly reconstruction of the surface ocean carbonate system, Earth Syst. Sci. Data, 16, 121–160, https://doi.org/10.5194/essd-16-121-2024, 2024.





<img src="https://mdl-metadata.s3.waw3-1.cloudferro.com/metadata/thumbnails/MULTIOBS_GLO_PHY_MYNRT_015_003.jpg" width="300" height="200"/>



# Global Total (COPERNICUS-GLOBCURRENT), Ekman and Geostrophic currents at the Surface and 15m


**Product ID:** MULTIOBS_GLO_PHY_MYNRT_015_003

**Description:** This product is a  L4 REP and NRT global total velocity field at 0m and 15m together wiht its individual components (geostrophy and Ekman) and related uncertainties. It consists of the zonal and meridional velocity at a 1h frequency and at 1/4 degree regular grid. The total velocity fields are obtained by combining CMEMS  satellite Geostrophic surface currents and modelled Ekman currents at the surface and 15m depth (using ERA5 wind stress in REP and ERA5* in NRT). 1 hourly product, daily and monthly means are available. This product has been initiated in the frame of CNES/CLS projects. Then it has been consolidated during the Globcurrent project (funded by the ESA User Element Program).

**DOI (product):** 
https://doi.org/10.48670/mds-00327

**References:**

* Rio, M.-H., S. Mulet, and N. Picot: Beyond GOCE for the ocean circulation estimate: Synergetic use of altimetry, gravimetry, and in situ data provides new insight into geostrophic and Ekman currents, Geophys. Res. Lett., 41, doi:10.1002/2014GL061773, 2014.





<img src="https://mdl-metadata.s3.waw3-1.cloudferro.com/metadata/thumbnails/MULTIOBS_GLO_PHY_SSS_L3_MYNRT_015_014.jpg" width="300" height="200"/>



# SMOS CATDS Qualified (L2Q) Sea Surface Salinity product


**Product ID:** MULTIOBS_GLO_PHY_SSS_L3_MYNRT_015_014

**Description:** The product MULTIOBS_GLO_PHY_SSS_L3_MYNRT_015_014 is a reformatting and a simplified version of the CATDS L3 product called “2Q” or “L2Q”. it is an intermediate product, that provides, in daily files, SSS corrected from land-sea contamination and latitudinal bias, with/without rain freshening correction.

**DOI (product):** 
https://doi.org/10.1016/j.rse.2016.02.061

**References:**

* Boutin, J., J. L. Vergely, S. Marchand, F. D'Amico, A. Hasson, N. Kolodziejczyk, N. Reul, G. Reverdin, and J. Vialard (2018), New SMOS Sea Surface Salinity with reduced systematic errors and improved variability, Remote Sensing of Environment, 214, 115-134. doi:https://doi.org/10.1016/j.rse.2018.05.022
* Kolodziejczyk, N., J. Boutin, J.-L. Vergely, S. Marchand, N. Martin, and G. Reverdin (2016), Mitigation of systematic errors in SMOS sea surface salinity, Remote Sensing of Environment, 180, 164-177. doi:https://doi.org/10.1016/j.rse.2016.02.061





<img src="https://mdl-metadata.s3.waw3-1.cloudferro.com/metadata/thumbnails/MULTIOBS_GLO_PHY_SSS_L4_MY_015_015.jpg" width="300" height="200"/>



# SSS SMOS/SMAP L4 OI - LOPS-v2023


**Product ID:** MULTIOBS_GLO_PHY_SSS_L4_MY_015_015

**Description:** The product MULTIOBS_GLO_PHY_SSS_L4_MY_015_015 is a reformatting and a simplified version of the CATDS L4 product called “SMOS-OI”. This product is obtained using optimal interpolation (OI) algorithm, that combine, ISAS in situ SSS OI analyses to reduce large scale and temporal variable bias, SMOS satellite image, SMAP satellite image, and satellite SST information.

Kolodziejczyk Nicolas, Hamon Michel, Boutin Jacqueline, Vergely Jean-Luc, Reverdin Gilles, Supply Alexandre, Reul Nicolas (2021). Objective analysis of SMOS and SMAP Sea Surface Salinity to reduce large scale and time dependent biases from low to high latitudes. Journal Of Atmospheric And Oceanic Technology, 38(3), 405-421. Publisher's official version: https://doi.org/10.1175/JTECH-D-20-0093.1, Open Access version: https://archimer.ifremer.fr/doc/00665/77702/

**DOI (product):** 
https://doi.org/10.1175/JTECH-D-20-0093.1

**References:**

* Kolodziejczyk Nicolas, Hamon Michel, Boutin Jacqueline, Vergely Jean-Luc, Reverdin Gilles, Supply Alexandre, Reul Nicolas (2021). Objective analysis of SMOS and SMAP Sea Surface Salinity to reduce large scale and time dependent biases from low to high latitudes. Journal Of Atmospheric And Oceanic Technology, 38(3), 405-421. Publisher's official version : https://doi.org/10.1175/JTECH-D-20-0093.1, Open Access version : https://archimer.ifremer.fr/doc/00665/77702/





<img src="https://mdl-metadata.s3.waw3-1.cloudferro.com/metadata/thumbnails/MULTIOBS_GLO_PHY_S_SURFACE_MYNRT_015_013.jpg" width="300" height="200"/>



# Multi Observation Global Ocean Sea Surface Salinity and Sea Surface Density


**Product ID:** MULTIOBS_GLO_PHY_S_SURFACE_MYNRT_015_013

**Description:** This product consits of daily global gap-free Level-4 (L4) analyses of the Sea Surface Salinity (SSS) and Sea Surface Density (SSD) at 1/8° of resolution, obtained through a multivariate optimal interpolation algorithm that combines sea surface salinity images from multiple satellite sources as NASA’s Soil Moisture Active Passive (SMAP) and ESA’s Soil Moisture Ocean Salinity (SMOS) satellites with in situ salinity measurements and satellite SST information. The product was developed by the Consiglio Nazionale delle Ricerche (CNR) and includes 4 datasets:
* cmems_obs-mob_glo_phy-sss_nrt_multi_P1D, which provides near-real-time (NRT) daily data
* cmems_obs-mob_glo_phy-sss_nrt_multi_P1M, which provides near-real-time (NRT) monthly data
* cmems_obs-mob_glo_phy-sss_my_multi_P1D, which provides multi-year reprocessed (REP) daily data 
* cmems_obs-mob_glo_phy-sss_my_multi_P1M, which provides multi-year reprocessed (REP) monthly data  

**DOI (product):** 
https://doi.org/10.48670/moi-00051

**References:**

* Droghei, R., B. Buongiorno Nardelli, and R. Santoleri, 2016: Combining in-situ and satellite observations to retrieve salinity and density at the ocean surface. J. Atmos. Oceanic Technol. doi:10.1175/JTECH-D-15-0194.1.
* Buongiorno Nardelli, B., R. Droghei, and R. Santoleri, 2016: Multi-dimensional interpolation of SMOS sea surface salinity with surface temperature and in situ salinity data. Rem. Sens. Environ., doi:10.1016/j.rse.2015.12.052.
* Droghei, R., B. Buongiorno Nardelli, and R. Santoleri, 2018: A New Global Sea Surface Salinity and Density Dataset From Multivariate Observations (1993–2016), Front. Mar. Sci., 5(March), 1–13, doi:10.3389/fmars.2018.00084.
* Sammartino, Michela, Salvatore Aronica, Rosalia Santoleri, and Bruno Buongiorno Nardelli. (2022). Retrieving Mediterranean Sea Surface Salinity Distribution and Interannual Trends from Multi-Sensor Satellite and In Situ Data, Remote Sensing 14, 2502: https://doi.org/10.3390/rs14102502.





<img src="https://mdl-metadata.s3.waw3-1.cloudferro.com/metadata/thumbnails/MULTIOBS_GLO_PHY_TSUV_3D_MYNRT_015_012.jpg" width="300" height="200"/>



# Multi Observation Global Ocean 3D Temperature Salinity Height Geostrophic Current and MLD


**Product ID:** MULTIOBS_GLO_PHY_TSUV_3D_MYNRT_015_012

**Description:** You can find here the Multi Observation Global Ocean ARMOR3D L4 analysis and multi-year reprocessing. It consists of 3D Temperature, Salinity, Heights, Geostrophic Currents and Mixed Layer Depth, available on a 1/4 degree regular grid and on 50 depth levels from the surface down to the bottom. The product includes 4 datasets: 

dataset-armor-3d-nrt-weekly, which delivers near-real-time (NRT) weekly data

dataset-armor-3d-nrt-monthly, which delivers near-real-time (NRT) monthly data

dataset-armor-3d-rep-weekly, which delivers multi-year reprocessed (REP) weekly data 

dataset-armor-3d-rep-monthly, which delivers multi-year reprocessed (REP) monthly data

**DOI (product):** 
https://doi.org/10.48670/moi-00052

**References:**

* Guinehut S., A.-L. Dhomps, G. Larnicol and P.-Y. Le Traon, 2012: High resolution 3D temperature and salinity fields derived from in situ and satellite observations. Ocean Sci., 8(5):845–857.
* Mulet, S., M.-H. Rio, A. Mignot, S. Guinehut and R. Morrow, 2012: A new estimate of the global 3D geostrophic ocean circulation based on satellite data and in-situ measurements. Deep Sea Research Part II : Topical Studies in Oceanography, 77–80(0):70–81.





<img src="https://mdl-metadata.s3.waw3-1.cloudferro.com/metadata/thumbnails/MULTIOBS_GLO_PHY_W_3D_REP_015_007.jpg" width="300" height="200"/>



# Global Observed Ocean Physics 3D Quasi-Geostrophic Currents (OMEGA3D)


**Product ID:** MULTIOBS_GLO_PHY_W_3D_REP_015_007

**Description:** You can find here the OMEGA3D observation-based  quasi-geostrophic vertical and horizontal ocean currents developed by the Consiglio Nazionale delle RIcerche. The data are provided weekly over a regular grid at 1/4° horizontal resolution, from the surface to 1500 m depth (representative of each Wednesday). The velocities are obtained by solving a diabatic formulation of the Omega equation, starting from ARMOR3D data (MULTIOBS_GLO_PHY_REP_015_002 which corresponds to former version of MULTIOBS_GLO_PHY_TSUV_3D_MYNRT_015_012) and ERA-Interim surface fluxes. 

**DOI (product):** 
https://doi.org/10.25423/cmcc/multiobs_glo_phy_w_rep_015_007

**References:**

* Buongiorno Nardelli, B. A Multi-Year Timeseries of Observation-Based 3D Horizontal and Vertical Quasi-Geostrophic Global Ocean Currents. Earth Syst. Sci. Data 2020, No. 12, 1711–1723. https://doi.org/10.5194/essd-12-1711-2020.





<img src="https://documentation.marine.copernicus.eu/IMG/NORTHWESTSHELF_OMI_TEMPSAL_extreme_var_temp_mean_and_anomaly.png" width="300" height="200"/>



# North West Shelf Sea Surface Temperature extreme from Reanalysis


**Product ID:** NORTHWESTSHELF_OMI_TEMPSAL_extreme_var_temp_mean_and_anomaly

**Description:** **DEFINITION**

The CMEMS NORTHWESTSHELF_OMI_tempsal_extreme_var_temp_mean_and_anomaly OMI indicator is based on the computation of the annual 99th percentile of Sea Surface Temperature (SST) from model data. Two different CMEMS products are used to compute the indicator: The North-West Shelf Multi Year Product (NWSHELF_MULTIYEAR_PHY_004_009) and the Analysis product (NORTHWESTSHELF_ANALYSIS_FORECAST_PHY_004_013).
Two parameters are included on this OMI:
* Map of the 99th mean percentile: It is obtained from the Multi Year Product, the annual 99th percentile is computed for each year of the product. The percentiles are temporally averaged over the whole period (1993-2019).
* Anomaly of the 99th percentile in 2020: The 99th percentile of the year 2020 is computed from the Analysis product. The anomaly is obtained by subtracting the mean percentile from the 2020 percentile.
This indicator is aimed at monitoring the extremes of sea surface temperature every year and at checking their variations in space. The use of percentiles instead of annual maxima, makes this extremes study less affected by individual data. This study of extreme variability was first applied to the sea level variable (Pérez Gómez et al 2016) and then extended to other essential variables, such as sea surface temperature and significant wave height (Pérez Gómez et al 2018 and Alvarez Fanjul et al., 2019). More details and a full scientific evaluation can be found in the CMEMS Ocean State report (Alvarez Fanjul et al., 2019).

**CONTEXT**

This domain comprises the North West European continental shelf where depths do not exceed 200m and deeper Atlantic waters to the North and West. For these deeper waters, the North-South temperature gradient dominates (Liu and Tanhua, 2021). Temperature over the continental shelf is affected also by the various local currents in this region and by the shallow depth of the water (Elliott et al., 1990). Atmospheric heat waves can warm the whole water column, especially in the southern North Sea, much of which is no more than 30m deep (Holt et al., 2012). Warm summertime water observed in the Norwegian trench is outflow heading North from the Baltic Sea and from the North Sea itself.

**CMEMS KEY FINDINGS**

The 99th percentile SST product can be considered to represent approximately the warmest 4 days for the sea surface in Summer. Maximum anomalies for 2020 are up to 4oC warmer than the 1993-2019 average in the western approaches, Celtic and Irish Seas, English Channel and the southern North Sea. For the atmosphere, Summer 2020 was exceptionally warm and sunny in southern UK (Kendon et al., 2021), with heatwaves in June and August. Further north in the UK, the atmosphere was closer to long-term average temperatures. Overall, the 99th percentile SST anomalies show a similar pattern, with the exceptional warm anomalies in the south of the domain.

Note: The key findings will be updated annually in November, in line with OMI evolutions.

**DOI (product)**
https://doi.org/10.48670/moi-00273

**References:**

* Álvarez Fanjul E, Pascual Collar A, Pérez Gómez B, De Alfonso M, García Sotillo M, Staneva J, Clementi E, Grandi A, Zacharioudaki A, Korres G, Ravdas M, Renshaw R, Tinker J, Raudsepp U, Lagemaa P, Maljutenko I, Geyer G, Müller M, Çağlar Yumruktepe V. Sea level, sea surface temperature and SWH extreme percentiles: combined analysis from model results and in situ observations, Section 2.7, p:31. In: Schuckmann K, Le Traon P-Y, Smith N, Pascual A, Djavidnia S, Gattuso J-P, Grégoire M, Nolan G, et al. 2019. Copernicus Marine Service Ocean State Report, Issue 3, Journal of Operational Oceanography, 12:sup1, S1-S123, DOI: 10.1080/1755876X.2019.1633075
* Elliott, A.J., Clarke, T., Li, ., 1990: Monthly distributions of surface and bottom temperatures in the northwest European shelf seas. Continental Shelf Research, Vol 11, no 5, pp 453-466, http://doi.org/10.1016/0278-4343(91)90053-9
* Holt, J., Hughes, S., Hopkins, J., Wakelin, S., Holliday, P.N., Dye, S., González-Pola, C., Hjøllo, S., Mork, K., Nolan, G., Proctor, R., Read, J., Shammon, T., Sherwin, T., Smyth, T., Tattersall, G., Ward, B., Wiltshire, K., 2012: Multi-decadal variability and trends in the temperature of the northwest European continental shelf: A model-data synthesis. Progress in Oceanography, 96-117, 106, http://doi.org/10.1016/j.pocean.2012.08.001
* Kendon, M., McCarthy, M., Jevrejeva, S., Matthews, A., Sparks, T. and Garforth, J. (2021), State of the UK Climate 2020. Int J Climatol, 41 (Suppl 2): 1-76. https://doi.org/10.1002/joc.7285
* Liu, M., Tanhua, T., 2021: Water masses in the Atlantic Ocean: characteristics and distributions. Ocean Sci, 17, 463-486, http://doi.org/10.5194/os-17-463-2021
* Pérez-Gómez B, Álvarez-Fanjul E, She J, Pérez-González I, Manzano F. 2016. Extreme sea level events, Section 4.4, p:300. In: Von Schuckmann K, Le Traon PY, Alvarez-Fanjul E, Axell L, Balmaseda M, Breivik LA, Brewin RJW, Bricaud C, Drevillon M, Drillet Y, Dubois C , Embury O, Etienne H, García-Sotillo M, Garric G, Gasparin F, Gutknecht E, Guinehut S, Hernandez F, Juza M, Karlson B, Korres G, Legeais JF, Levier B, Lien VS, Morrow R, Notarstefano G, Parent L, Pascual A, Pérez-Gómez B, Perruche C, Pinardi N, Pisano A, Poulain PM , Pujol IM, Raj RP, Raudsepp U, Roquet H, Samuelsen A, Sathyendranath S, She J, Simoncelli S, Solidoro C, Tinker J, Tintoré J, Viktorsson L, Ablain M, Almroth-Rosell E, Bonaduce A, Clementi E, Cossarini G, Dagneaux Q, Desportes C, Dye S, Fratianni C, Good S, Greiner E, Gourrion J, Hamon M, Holt J, Hyder P, Kennedy J, Manzano-Muñoz F, Melet A, Meyssignac B, Mulet S, Nardelli BB, O’Dea E, Olason E, Paulmier A, Pérez-González I, Reid R, Racault MF, Raitsos DE, Ramos A, Sykes P, Szekely T, Verbrugge N. 2016. The Copernicus Marine Environment Monitoring Service Ocean State Report, Journal of Operational Oceanography. 9 (sup2): 235-320. http://dx.doi.org/10.1080/1755876X.2016.1273446
* Pérez Gómez B., De Alfonso M., Zacharioudaki A., Pérez González I., Álvarez Fanjul E., Müller M., Marcos M., Manzano F., Korres G., Ravdas M., Tamm S. 2018. Sea level, SST and waves: extremes variability. In: Copernicus Marine Service Ocean State Report, Issue 2, Journal of Operational Oceanography, 11:sup1, Chap. 3.1, s79–s88, DOI: https://doi.org/10.1080/1755876X.2018.1489208





<img src="https://mdl-metadata.s3.waw3-1.cloudferro.com/metadata/thumbnails/NWSHELF_ANALYSISFORECAST_BGC_004_002.jpg" width="300" height="200"/>



# Atlantic - European North West Shelf - Ocean Biogeochemistry Analysis and Forecast


**Product ID:** NWSHELF_ANALYSISFORECAST_BGC_004_002

**Description:** The NWSHELF_ANALYSISFORECAST_BGC_004_002 is produced by a coupled physical-biogeochemical model, implemented over the North East Atlantic and Shelf Seas at 1/36 degrees of horizontal resolution and 50 vertical levels.
The product is updated weekly, providing 10-day forecast of the main biogeochemical variables.
Products are provided as daily and monthly means.

**DOI (product):**   
https://doi.org/10.48670/moi-00056




<img src="https://mdl-metadata.s3.waw3-1.cloudferro.com/metadata/thumbnails/NWSHELF_ANALYSISFORECAST_PHY_004_013.jpg" width="300" height="200"/>



# Atlantic - European North West Shelf - Ocean Physics Analysis and Forecast


**Product ID:** NWSHELF_ANALYSISFORECAST_PHY_004_013

**Description:** The NWSHELF_ANALYSISFORECAST_PHY_004_013 is produced by a hydrodynamic model with tides, implemented over the North East Atlantic and Shelf Seas at 1/36 degrees of horizontal resolution and 50 vertical levels.
The product is updated daily, providing 10-day forecast for temperature, salinity, currents, sea level and mixed layer depth.
Products are provided at quarter-hourly, hourly, daily de-tided (with Doodson filter), and monthly frequency.

**DOI (product):**   
https://doi.org/10.48670/moi-00054




<img src="https://mdl-metadata.s3.waw3-1.cloudferro.com/metadata/thumbnails/NWSHELF_ANALYSISFORECAST_WAV_004_014.jpg" width="300" height="200"/>



# Atlantic - European North West Shelf - Ocean Wave Analysis and Forecast


**Product ID:** NWSHELF_ANALYSISFORECAST_WAV_004_014

**Description:** The NWSHELF_ANALYSISFORECAST_WAV_004_014 is produced by a wave model system based on MFWAV, implemented over the North East Atlantic and Shelf Seas at 1/36 degrees of horizontal resolution forced by ECMWF wind data. The system assimilates significant wave height altimeter data and spectral data, and it is forced by currents provided by the [ ref t the physical system] ocean circulation system.
The product is updated twice a day, providing 10-day forecast of wave parameters integrated from the two-dimensional (frequency, direction) wave spectrum and describe wave height, period and directional characteristics for both the overall sea-state, and wind-state, and swell components. 
Products are provided at hourly frequency

**DOI (product):**   
https://doi.org/10.48670/moi-00055

**References:**

* The impact of ocean-wave coupling on the upper ocean circulation during storm events (Bruciaferri, D., Tonani, M., Lewis, H., Siddorn, J., Saulter, A., Castillo, J.M., Garcia Valiente, N., Conley, D., Sykes, P., Ascione, I., McConnell, N.) in Journal of Geophysical Research, Oceans, 2021, 126, 6. https://doi.org/10.1029/2021JC017343





<img src="https://mdl-metadata.s3.waw3-1.cloudferro.com/metadata/thumbnails/NWSHELF_MULTIYEAR_BGC_004_011.jpg" width="300" height="200"/>



# Atlantic- European North West Shelf- Ocean Biogeochemistry Reanalysis


**Product ID:** NWSHELF_MULTIYEAR_BGC_004_011

**Description:** **Short  Description:**

The ocean biogeochemistry reanalysis for the North-West European Shelf is produced using the European Regional Seas Ecosystem Model (ERSEM), coupled online to the forecasting ocean assimilation model at 7 km horizontal resolution, NEMO-NEMOVAR. ERSEM (Butensch&ouml;n et al. 2016) is developed and maintained at Plymouth Marine Laboratory. NEMOVAR system was used to assimilate observations of sea surface chlorophyll concentration from ocean colour satellite data and all the physical variables described in [NWSHELF_MULTIYEAR_PHY_004_009](https://resources.marine.copernicus.eu/?option=com_csw&view=details&product_id=NWSHELF_MULTIYEAR_PHY_004_009). Biogeochemical boundary conditions and river inputs used climatologies; nitrogen deposition at the surface used time-varying data.

The description of the model and its configuration, including the products validation is provided in the [CMEMS-NWS-QUID-004-011](https://documentation.marine.copernicus.eu/QUID/CMEMS-NWS-QUID-004-011.pdf). 

Products are provided as monthly and daily 25-hour, de-tided, averages. The datasets available are concentration of chlorophyll, nitrate, phosphate, oxygen, phytoplankton biomass, net primary production, light attenuation coefficient, pH, surface partial pressure of CO2, concentration of diatoms expressed as chlorophyll, concentration of dinoflagellates expressed as chlorophyll, concentration of nanophytoplankton expressed as chlorophyll, concentration of picophytoplankton expressed as chlorophyll in sea water. All, as multi-level variables, are interpolated from the model 51 hybrid s-sigma terrain-following system to 24 standard geopotential depths (z-levels). Grid-points near to the model boundaries are masked. The product is updated biannually,  providing a six-month extension of the time series. See [CMEMS-NWS-PUM-004-009_011](https://documentation.marine.copernicus.eu/PUM/CMEMS-NWS-PUM-004-009-011.pdf) for details.

**Associated products:**

This model is coupled with a hydrodynamic model (NEMO) available as CMEMS product [NWSHELF_MULTIYEAR_PHY_004_009](https://resources.marine.copernicus.eu/?option=com_csw&view=details&product_id=NWSHELF_MULTIYEAR_PHY_004_009).
An analysis-forecast product is available from: [NWSHELF_MULTIYEAR_BGC_004_011](https://resources.marine.copernicus.eu/?option=com_csw&view=details&product_id=NWSHELF_MULTIYEAR_BGC_004_011).

**DOI (product):**   
https://doi.org/10.48670/moi-00058

**References:**

* Ciavatta, S., Brewin, R. J. W., Skákala, J., Polimene, L., de Mora, L., Artioli, Y., & Allen, J. I. (2018). [https://doi.org/10.1002/2017JC013490 Assimilation of ocean‐color plankton functional types to improve marine ecosystem simulations]. Journal of Geophysical Research: Oceans, 123, 834–854. https://doi.org/10.1002/2017JC013490





<img src="https://mdl-metadata.s3.waw3-1.cloudferro.com/metadata/thumbnails/NWSHELF_MULTIYEAR_PHY_004_009.jpg" width="300" height="200"/>



# Atlantic- European North West Shelf- Ocean Physics Reanalysis


**Product ID:** NWSHELF_MULTIYEAR_PHY_004_009

**Description:** **Short  Description:**

The ocean physics reanalysis for the North-West European Shelf is produced using an ocean assimilation model, with tides, at 7 km horizontal resolution.  
The ocean model is NEMO (Nucleus for European Modelling of the Ocean), using the 3DVar NEMOVAR system to assimilate observations. These are surface temperature and vertical profiles of temperature and salinity. The model is forced by lateral boundary conditions from the GloSea5, one of the multi-models used by [GLOBAL_REANALYSIS_PHY_001_026](https://resources.marine.copernicus.eu/?option=com_csw&view=details&product_id=GLOBAL_REANALYSIS_PHY_001_026) and at the Baltic boundary by the [BALTICSEA_REANALYSIS_PHY_003_011](https://resources.marine.copernicus.eu/?option=com_csw&view=details&product_id=BALTICSEA_REANALYSIS_PHY_003_011). The atmospheric forcing is given by the ECMWF ERA5 atmospheric reanalysis. The river discharge is from a daily climatology. 

Further details of the model, including the product validation are provided in the [CMEMS-NWS-QUID-004-009](https://documentation.marine.copernicus.eu/QUID/CMEMS-NWS-QUID-004-009.pdf). 

Products are provided as monthly and daily 25-hour, de-tided, averages. The datasets available are temperature, salinity, horizontal currents, sea level, mixed layer depth, and bottom temperature. Temperature, salinity and currents, as multi-level variables, are interpolated from the model 51 hybrid s-sigma terrain-following system to 24 standard geopotential depths (z-levels). Grid-points near to the model boundaries are masked. The product is updated biannually provinding six-month extension of the time series.

See [CMEMS-NWS-PUM-004-009_011](https://documentation.marine.copernicus.eu/PUM/CMEMS-NWS-PUM-004-009-011.pdf) for further details.

**Associated products:**

This model is coupled with a biogeochemistry model (ERSEM) available as CMEMS product [](https://resources.marine.copernicus.eu/?option=com_csw&view=details&product_id=NWSHELF_MULTIYEAR_BGC_004_011). An analysis-forecast product is available from [NWSHELF_ANALYSISFORECAST_PHY_LR_004_011](https://resources.marine.copernicus.eu/?option=com_csw&view=details&product_id=NWSHELF_ANALYSISFORECAST_PHY_LR_004_001).
The product is updated biannually provinding six-month extension of the time series.

**DOI (product):**   
https://doi.org/10.48670/moi-00059




<img src="https://mdl-metadata.s3.waw3-1.cloudferro.com/metadata/thumbnails/NWSHELF_REANALYSIS_WAV_004_015.jpg" width="300" height="200"/>



# Atlantic- European North West Shelf- Wave Physics Reanalysis


**Product ID:** NWSHELF_REANALYSIS_WAV_004_015

**Description:** **Short  description:**

This product provides long term hindcast outputs from a wave model for the North-West European Shelf. The wave model is WAVEWATCH III and the North-West Shelf configuration is based on a two-tier Spherical Multiple Cell grid mesh (3 and 1.5 km cells) derived from with the 1.5km grid used for [NORTHWESTSHELF_ANALYSIS_FORECAST_PHY_004_013](https://resources.marine.copernicus.eu/?option=com_csw&view=details&product_id=NORTHWESTSHELF_ANALYSIS_FORECAST_PHY_004_013). The model is forced by lateral boundary conditions from a Met Office Global wave hindcast. The atmospheric forcing is given by the [ECMWF ERA-5](https://www.ecmwf.int/en/forecasts/datasets/reanalysis-datasets/era5) Numerical Weather Prediction reanalysis. Model outputs comprise wave parameters integrated from the two-dimensional (frequency, direction) wave spectrum and describe wave height, period and directional characteristics for both the overall sea-state and wind-sea and swell components. The data are delivered on a regular grid at approximately 1.5km resolution, consistent with physical ocean and wave analysis-forecast products. See [CMEMS-NWS-PUM-004-015](https://documentation.marine.copernicus.eu/PUM/CMEMS-NWS-PUM-004-015.pdf) for more information. Further details of the model, including source term physics, propagation schemes, forcing and boundary conditions, and validation, are provided in the [CMEMS-NWS-QUID-004-015](https://documentation.marine.copernicus.eu/QUID/CMEMS-NWS-QUID-004-015.pdf).
The product is updated biannually provinding six-month extension of the time series.

**Associated products:**

[NORTHWESTSHELF_ANALYSIS_FORECAST_WAV_004_014](https://resources.marine.copernicus.eu/?option=com_csw&view=details&product_id=NORTHWESTSHELF_ANALYSIS_FORECAST_WAV_004_014).

**DOI (product):**   
https://doi.org/10.48670/moi-00060




<img src="https://catalogue.marine.copernicus.eu/documents/IMG/OCEANCOLOUR_ARC_BGC_HR_L3_NRT_009_201.png" width="300" height="200"/>



# Arctic Region, Bio-Geo-Chemical, L3, daily observation


**Product ID:** OCEANCOLOUR_ARC_BGC_HR_L3_NRT_009_201

**Description:** The High-Resolution Ocean Colour (HR-OC) Consortium (Brockmann Consult, Royal Belgian Institute of Natural Sciences, Flemish Institute for Technological Research) distributes Remote Sensing Reflectances (RRS, expressed in sr-1), Turbidity (TUR, expressed in FNU), Solid Particulate Matter Concentration (SPM, expressed in mg/l), spectral particulate backscattering (BBP, expressed in m-1) and chlorophyll-a concentration (CHL, expressed in µg/l) for the Sentinel 2/MSI sensor at 100m resolution for a 20km coastal zone. The products of region ARC are delivered in polar Lambertian Azimuthal Equal Area (LAEA) projection (EPSG:6931, EASE2). To limit file size the products are provided in tiles of 600x800 km². RRS and BBP are delivered at nominal central bands of 443, 492, 560, 665, 704, 740, 783, 865 nm. The primary variable from which it is virtually possible to derive all the geophysical and transparency products is the spectral RRS. This, together with the spectral BBP, constitute the category of the 'optics' products. The spectral BBP product is generated from the RRS products using a quasi-analytical algorithm (Lee et al. 2002). The 'transparency' products include TUR and SPM). They are retrieved through the application of automated switching algorithms to the RRS spectra adapted to varying water conditions (Novoa et al. 2017). The GEOPHYSICAL product consists of the Chlorophyll-a concentration (CHL) retrieved via a multi-algorithm approach with optimized quality flagging (O'Reilly et al. 2019, Gons et al. 2005, Lavigne et al. 2021). The NRT products are generally provided withing 24 hours up to 3 days after end of the day.The RRS product is accompanied by a relative uncertainty estimate (unitless) derived by direct comparison of the products to corresponding fiducial reference measurements provided through the AERONET-OC network. The current day data temporal consistency is evaluated as Quality Index (QI) for TUR, SPM and CHL: QI=(CurrentDataPixel-ClimatologyDataPixel)/STDDataPixel where QI is the difference between current data and the relevant climatological field as a signed multiple of climatological standard deviations (STDDataPixel).

**Processing information:**

The HR-OC processing system is deployed on Creodias where Sentinel 2/MSI L1C data are available. The production control element is being hosted within the infrastructure of Brockmann Consult. The processing chain consists of:
* Resampling to 60m and mosaic generation of the set of Sentinel-2 MSI L1C granules of a single overpass that cover a single UTM zone.
* Application of  a glint correction taking into account the detector viewing angles
* Application of a coastal mask with 20km water + 20km land. The result is a L1C mosaic tile with data just in the coastal area optimized for compression.
* Level 2 processing with pixel identification (IdePix), atmospheric correction (C2RCC and ACOLITE or iCOR), in-water processing and merging (HR-OC L2W processor). The result is a 60m product with the same extent as the L1C mosaic, with variables for optics, transparency, and geophysics, and with data filled in the water part of the coastal area.
* invalid pixel identification takes into account  corrupted (L1) pixels, clouds, cloud shadow, glint, dry-fallen intertidal flats, coastal mixed-pixels, sea ice, melting ice, floating vegetation, non-water objects, and bottom reflection.    
* Daily L3 aggregation merges all Level 2 mosaics of a day intersecting with a target tile. All valid water pixels are included in the 20km coastal stripes; all other values are set to NaN. There may be more than a single overpass a day, in particular in the northern regions. The main contribution usually is the mosaic of the zone, but also adjacent mosaics may overlap. This step comprises resampling to the 100m target grid. 
* Monthly L4 aggregation combines all Level 3 products of a month and a single tile. The output is a set of 3 NetCDF datasets for optics, transparency, and geophysics respectively, for the tile and month.
* Gap filling combines all daily products of a period and generates (partially) gap-filled daily products again. The output of gap filling are 3 datasets for optics (BBP443 only), transparency, and geophysics per day.


**Description of observation methods/instruments:**

Ocean colour technique exploits the emerging electromagnetic radiation from the sea surface in different wavelengths. The spectral variability of this signal defines the so-called ocean colour which is affected by the presence of phytoplankton.


**Quality / Accuracy / Calibration information:**

A detailed description of the calibration and validation activities performed over this product can be found on the CMEMS web portal and in CMEMS-BGP_HR-QUID-009-201to212.


**Suitability, Expected type of users / uses:**

This product is meant for use for educational purposes and for the managing of the marine safety, marine resources, marine and coastal environment and for climate and seasonal studies.

**Dataset names: **

*cmems_obs_oc_arc_bgc_geophy_nrt_l3-hr_P1D-m
*cmems_obs_oc_arc_bgc_transp_nrt_l3-hr_P1D-m
*cmems_obs_oc_arc_bgc_optics_nrt_l3-hr_P1D-m

**Files format:**
*netCDF-4, CF-1.7
*INSPIRE compliant.

**DOI (product):**   
https://doi.org/10.48670/moi-00061

**References:**

* Lavigne, H., et al. (2021), Quality-control tests for OC4, OC5 and NIR-red satellite chlorophyll-a algorithms applied to coastal waters, Remote Sensing of Environment, in press.
* Lee, Z. P., et al. (2002), Deriving inherent optical properties from water color: A multi- band quasi-analytical algorithm for optically deep waters, Applied Optics, 41, 5755-5772.
* Novoa, S., et al. (2017), Atmospheric corrections and multi-conditional algorithm for multi-sensor remote sensing of suspended particulate matter in low-to-high turbidity levels coastal waters. Remote Sens., v. 9, 61.
* Gons, et al. (2005), Effect of a waveband shift on chlorophyll retrieval from MERIS imagery of inland and coastal waters, J. Plankton Res., v. 27, n. 1, p. 125-127.
* O'Reilly, et al. (2019), Chlorophyll algorithms for ocean color sensors-OC4, OC5 & OC6. Remote Sensing of Environment. 229, 32–47.





<img src="https://catalogue.marine.copernicus.eu/documents/IMG/OCEANCOLOUR_ARC_BGC_HR_L4_NRT_009_207.png" width="300" height="200"/>



# Arctic Region, Bio-Geo-Chemical, L4, monthly means and interpolated daily observation


**Product ID:** OCEANCOLOUR_ARC_BGC_HR_L4_NRT_009_207

**Description:** The High-Resolution Ocean Colour (HR-OC) Consortium (Brockmann Consult, Royal Belgian Institute of Natural Sciences, Flemish Institute for Technological Research) distributes Level 4 (L4) Turbidity (TUR, expressed in FNU), Solid Particulate Matter Concentration (SPM, expressed in mg/l), particulate backscattering at 443nm (BBP443, expressed in m-1) and chlorophyll-a concentration (CHL, expressed in µg/l) for the Sentinel 2/MSI sensor at 100m resolution for a 20km coastal zone. The products of region ARC are delivered in polar Lambertian Azimuthal Equal Area (LAEA) projection (EPSG:6931, EASE2). To limit file size the products are provided in tiles of 600x800 km². BBP443, constitute the category of the 'optics' products. The  BBP443 product is generated from the L3 RRS products using a quasi-analytical algorithm (Lee et al. 2002). The 'transparency' products include TUR and SPM). They are retrieved through the application of automated switching algorithms to the RRS spectra adapted to varying water conditions (Novoa et al. 2017). The GEOPHYSICAL product consists of the Chlorophyll-a concentration (CHL) retrieved via a multi-algorithm approach with optimized quality flagging (O'Reilly et al. 2019, Gons et al. 2005, Lavigne et al. 2021). Monthly products (P1M) are temporal aggregates of the daily L3 products. Daily products contain gaps in cloudy areas and where there is no overpass at the respective day. Aggregation collects the non-cloudy (and non-frozen) contributions to each pixel. Contributions are averaged per variable. While this does not guarantee data availability in all pixels in case of persistent clouds, it provides a more complete product compared to the sparsely filled daily products. The Monthly L4 products (P1M) are generally provided withing 4 days after the last acquisition date of the month. Daily gap filled L4 products (P1D) are generated using the DINEOF (Data Interpolating Empirical Orthogonal Functions) approach which reconstructs missing data in geophysical datasets by using a truncated Empirical Orthogonal Functions (EOF) basis in an iterative approach. DINEOF reconstructs missing data in a geophysical dataset by extracting the main patterns of temporal and spatial variability from the data. While originally designed for low resolution data products, recent research has resulted in the optimization of DINEOF to handle high resolution data provided by Sentinel-2 MSI, including cloud shadow detection (Alvera-Azcárate et al., 2021). These types of L4 products are generated and delivered one month after the respective period.


**Processing information:**

The HR-OC processing system is deployed on Creodias where Sentinel 2/MSI L1C data are available. The production control element is being hosted within the infrastructure of Brockmann Consult. The processing chain consists of:
* Resampling to 60m and mosaic generation of the set of Sentinel-2 MSI L1C granules of a single overpass that cover a single UTM zone.
* Application of  a glint correction taking into account the detector viewing angles
* Application of a coastal mask with 20km water + 20km land. The result is a L1C mosaic tile with data just in the coastal area optimized for compression.
* Level 2 processing with pixel identification (IdePix), atmospheric correction (C2RCC and ACOLITE or iCOR), in-water processing and merging (HR-OC L2W processor). The result is a 60m product with the same extent as the L1C mosaic, with variables for optics, transparency, and geophysics, and with data filled in the water part of the coastal area.
* invalid pixel identification takes into account  corrupted (L1) pixels, clouds, cloud shadow, glint, dry-fallen intertidal flats, coastal mixed-pixels, sea ice, melting ice, floating vegetation, non-water objects, and bottom reflection.
* Daily L3 aggregation merges all Level 2 mosaics of a day intersecting with a target tile. All valid water pixels are included in the 20km coastal stripes; all other values are set to NaN. There may be more than a single overpass a day, in particular in the northern regions. The main contribution usually is the mosaic of the zone, but also adjacent mosaics may overlap. This step comprises resampling to the 100m target grid. 
* Monthly L4 aggregation combines all Level 3 products of a month and a single tile. The output is a set of 3 NetCDF datasets for optics, transparency, and geophysics respectively, for the tile and month.
* Gap filling combines all daily products of a period and generates (partially) gap-filled daily products again. The output of gap filling are 3 datasets for optics (BBP443 only), transparency, and geophysics per day.

**Description of observation methods/instruments:**

Ocean colour technique exploits the emerging electromagnetic radiation from the sea surface in different wavelengths. The spectral variability of this signal defines the so-called ocean colour which is affected by the presence of phytoplankton.

**Quality / Accuracy / Calibration information:**

A detailed description of the calibration and validation activities performed over this product can be found on the CMEMS web portal and in CMEMS-BGP_HR-QUID-009-201_to_212.

**Suitability, Expected type of users / uses:**

This product is meant for use for educational purposes and for the managing of the marine safety, marine resources, marine and coastal environment and for climate and seasonal studies.


**Dataset names: **
*cmems_obs_oc_arc_bgc_geophy_nrt_l4-hr_P1M-v01
*cmems_obs_oc_arc_bgc_transp_nrt_l4-hr_P1M-v01
*cmems_obs_oc_arc_bgc_optics_nrt_l4-hr_P1M-v01
*cmems_obs_oc_arc_bgc_geophy_nrt_l4-hr_P1D-v01
*cmems_obs_oc_arc_bgc_transp_nrt_l4-hr_P1D-v01
*cmems_obs_oc_arc_bgc_optics_nrt_l4-hr_P1D-v01

**Files format:**
*netCDF-4, CF-1.7
*INSPIRE compliant.

**DOI (product):**  
https://doi.org/10.48670/moi-00062

**References:**

* Alvera-Azcárate, Aida, et al. (2021), Detection of shadows in high spatial resolution ocean satellite data using DINEOF. Remote Sensing of Environment 253: 112229.
* Lavigne, H., et al. (2021), Quality-control tests for OC4, OC5 and NIR-red satellite chlorophyll-a algorithms applied to coastal waters, Remote Sensing of Environment, in press.
* Lee, Z. P., et al. (2002), Deriving inherent optical properties from water color: A multi- band quasi-analytical algorithm for optically deep waters, Applied Optics, 41, 5755-5772.
* Novoa, S., et al. (2017), Atmospheric corrections and multi-conditional algorithm for multi-sensor remote sensing of suspended particulate matter in low-to-high turbidity levels coastal waters. Remote Sens., v. 9, 61.
* Gons, et al. (2005), Effect of a waveband shift on chlorophyll retrieval from MERIS imagery of inland and coastal waters, J. Plankton Res., v. 27, n. 1, p. 125-127.
* O'Reilly, et al. (2019), Chlorophyll algorithms for ocean color sensors-OC4, OC5 & OC6. Remote Sensing of Environment. 229, 32–47.





<img src="https://mdl-metadata.s3.waw3-1.cloudferro.com/metadata/thumbnails/OCEANCOLOUR_ARC_BGC_L3_MY_009_123.jpg" width="300" height="200"/>



# Arctic Ocean Colour Plankton, Reflectance, Transparency and Optics MY L3 daily observations


**Product ID:** OCEANCOLOUR_ARC_BGC_L3_MY_009_123

**Description:** For the **Arctic** Ocean **Satellite Observations**, Italian National Research Council (CNR – Rome, Italy) is providing **Bio-Geo_Chemical (BGC)** products.
* Upstreams:  OCEANCOLOUR_GLO_BGC_L3_MY_009_107 for the **"multi"** products and S3A & S3B only for the **"OLCI"** products.
* Variables: Chlorophyll-a (**CHL**), Diffuse Attenuation (**KD490**) and  Reflectance (**RRS**).

* Temporal resolutions: **daily**.
* Spatial resolutions: **4 km** (multi) or **300 m** (OLCI).
* Recent products are organized in datasets called Near Real Time (**NRT**) and long time-series (from 1997) in datasets called Multi-Years (**MY**).

**DOI (product):**   
https://doi.org/10.48670/moi-00292




<img src="https://mdl-metadata.s3.waw3-1.cloudferro.com/metadata/thumbnails/OCEANCOLOUR_ARC_BGC_L3_NRT_009_121.jpg" width="300" height="200"/>



# Arctic Ocean Colour Plankton, Reflectance, Transparency and Optics L3 NRT daily observations


**Product ID:** OCEANCOLOUR_ARC_BGC_L3_NRT_009_121

**Description:** For the **Arctic** Ocean **Satellite Observations**, Italian National Research Council (CNR – Rome, Italy) is providing **Bio-Geo_Chemical (BGC)** products.
* Upstreams: OLCI-S3A & OLCI-S3B for the **""olci""** products.
* Variables: Chlorophyll-a (**CHL**), Suspended Matter (**SPM**), Diffuse Attenuation (**KD490**), Detrital and Dissolved Material Absorption Coef. (**ADG443_), Phytoplankton Absorption Coef. (**APH443_), Total Absorption Coef. (**ATOT443_) and  Reflectance (**RRS_').

* Temporal resolutions: **daily**, **monthly**.
* Spatial resolutions:  **300 meters** (olci).
* Recent products are organized in datasets called Near Real Time (**NRT**).

**DOI (product):**   
https://doi.org/10.48670/moi-00290




<img src="https://mdl-metadata.s3.waw3-1.cloudferro.com/metadata/thumbnails/OCEANCOLOUR_ARC_BGC_L4_MY_009_124.jpg" width="300" height="200"/>



# Arctic Ocean Colour Plankton MY L4 daily climatology and monthly observations


**Product ID:** OCEANCOLOUR_ARC_BGC_L4_MY_009_124

**Description:** For the **Arctic** Ocean **Satellite Observations**, Italian National Research Council (CNR – Rome, Italy) is providing **Bio-Geo_Chemical (BGC)** products.
* Upstreams: OCEANCOLOUR_GLO_BGC_L3_MY_009_107 for the **"multi"** products , and S3A & S3B only for the **"OLCI"** products.
* Variables: Chlorophyll-a (**CHL**), Diffuse Attenuation (**KD490**)


* Temporal resolutions: **monthly**.
* Spatial resolutions: **4 km** (multi) or **300 meters** (OLCI).
* Recent products are organized in datasets called Near Real Time (**NRT**) and long time-series (from 1997) in datasets called Multi-Years (**MY**).

**DOI (product):**   
https://doi.org/10.48670/moi-00293




<img src="https://mdl-metadata.s3.waw3-1.cloudferro.com/metadata/thumbnails/OCEANCOLOUR_ARC_BGC_L4_NRT_009_122.jpg" width="300" height="200"/>



# Arctic Ocean Colour Plankton and Transparency L4 NRT monthly observations


**Product ID:** OCEANCOLOUR_ARC_BGC_L4_NRT_009_122

**Description:** For the **Arctic** Ocean **Satellite Observations**, Italian National Research Council (CNR – Rome, Italy) is providing **Bio-Geo_Chemical (BGC)** products.
* Upstreams: OLCI-S3A & OLCI-S3B for the **""olci""** products.
* Variables: Chlorophyll-a (**CHL**) and Diffuse Attenuation (**KD490**).

* Temporal resolutions:**monthly**.
* Spatial resolutions:  **300 meters** (olci).
* Recent products are organized in datasets called Near Real Time (**NRT**).

**DOI (product):**   
https://doi.org/10.48670/moi-00291




<img src="https://mdl-metadata.s3.waw3-1.cloudferro.com/metadata/thumbnails/OCEANCOLOUR_ATL_BGC_L3_MY_009_113.jpg" width="300" height="200"/>



# North Atlantic Ocean Colour Plankton, Reflectance, Transparency and Optics MY L3 daily observations


**Product ID:** OCEANCOLOUR_ATL_BGC_L3_MY_009_113

**Description:** For the **Global** Ocean **Satellite Observations**, ACRI-ST company (Sophia Antipolis, France) is providing **Bio-Geo-Chemical (BGC)** products based on the **Copernicus-GlobColour** processor.
* Upstreams: SeaWiFS, MODIS, MERIS, VIIRS-SNPP & JPSS1, OLCI-S3A & S3B for the **""multi""** products, and S3A & S3B only for the **""olci""** products.
* Variables: Chlorophyll-a (**CHL**), Gradient of Chlorophyll-a (**CHL_gradient**), Phytoplankton Functional types and sizes (**PFT**), Suspended Matter (**SPM**), Secchi Transparency Depth (**ZSD**), Diffuse Attenuation (**KD490**), Particulate Backscattering (**BBP**), Absorption Coef. (**CDM**) and  Reflectance (**RRS**).

* Temporal resolutions: **daily**.
* Spatial resolutions: **1 km** and a finer resolution based on olci **300 meters** inputs.
* Recent products are organized in datasets called Near Real Time (**NRT**) and long time-series (from 1997) in datasets called Multi-Years (**MY**).

To find the **Copernicus-GlobColour** products in the catalogue, use the search keyword **""GlobColour""**.

**DOI (product):**   
https://doi.org/10.48670/moi-00286




<img src="https://mdl-metadata.s3.waw3-1.cloudferro.com/metadata/thumbnails/OCEANCOLOUR_ATL_BGC_L3_NRT_009_111.jpg" width="300" height="200"/>



# North Atlantic Ocean Colour Plankton, Reflectance, Transparency and Optics L3 NRT daily observations


**Product ID:** OCEANCOLOUR_ATL_BGC_L3_NRT_009_111

**Description:** For the **Global** Ocean **Satellite Observations**, ACRI-ST company (Sophia Antipolis, France) is providing **Bio-Geo-Chemical (BGC)** products based on the **Copernicus-GlobColour** processor.
* Upstreams: SeaWiFS, MODIS, MERIS, VIIRS-SNPP & JPSS1, OLCI-S3A & S3B for the **""multi""** products, and S3A & S3B only for the **""olci""** products.
* Variables: Chlorophyll-a (**CHL**), Gradient of Chlorophyll-a (**CHL_gradient**), Phytoplankton Functional types and sizes (**PFT**), Suspended Matter (**SPM**), Secchi Transparency Depth (**ZSD**), Diffuse Attenuation (**KD490**), Particulate Backscattering (**BBP**), Absorption Coef. (**CDM**) and  Reflectance (**RRS**).

* Temporal resolutions: **daily**.
* Spatial resolutions: **1 km** and a finer resolution based on olci **300 meters** inputs.
* Recent products are organized in datasets called Near Real Time (**NRT**) and long time-series (from 1997) in datasets called Multi-Years (**MY**).

To find the **Copernicus-GlobColour** products in the catalogue, use the search keyword **""GlobColour""**.

**DOI (product):**   
https://doi.org/10.48670/moi-00284




<img src="https://mdl-metadata.s3.waw3-1.cloudferro.com/metadata/thumbnails/OCEANCOLOUR_ATL_BGC_L4_MY_009_118.jpg" width="300" height="200"/>



# Atlantic Ocean Colour (Copernicus-GlobColour), Bio-Geo-Chemical, L4 (daily interpolated) from Satellite Observations (1997-ongoing)


**Product ID:** OCEANCOLOUR_ATL_BGC_L4_MY_009_118

**Description:** For the **Global** Ocean **Satellite Observations**, ACRI-ST company (Sophia Antipolis, France) is providing **Bio-Geo-Chemical (BGC)** products based on the **Copernicus-GlobColour** processor.
* Upstreams: SeaWiFS, MODIS, MERIS, VIIRS-SNPP & JPSS1, OLCI-S3A & S3B for the **"multi"** products, and S3A & S3B only for the **"olci"** products.
* Variables: Chlorophyll-a (**CHL**), Phytoplankton Functional types and sizes (**PFT**), Primary Production (**PP**).

* Temporal resolutions: **monthly** plus, for some variables, **daily gap-free** based on a space-time interpolation to provide a "cloud free" product.
* Spatial resolutions: **1 km**.
* Recent products are organized in datasets called Near Real Time (**NRT**) and long time-series (from 1997) in datasets called Multi-Years (**MY**).

To find the **Copernicus-GlobColour** products in the catalogue, use the search keyword **"GlobColour"**.

**DOI (product):**   
https://doi.org/10.48670/moi-00289




<img src="https://mdl-metadata.s3.waw3-1.cloudferro.com/metadata/thumbnails/OCEANCOLOUR_ATL_BGC_L4_NRT_009_116.jpg" width="300" height="200"/>



# Atlantic Ocean Colour (Copernicus-GlobColour), Bio-Geo-Chemical, L4 (daily interpolated) from Satellite Observations (Near Real Time)


**Product ID:** OCEANCOLOUR_ATL_BGC_L4_NRT_009_116

**Description:** For the **Global** Ocean **Satellite Observations**, ACRI-ST company (Sophia Antipolis, France) is providing **Bio-Geo-Chemical (BGC)** products based on the **Copernicus-GlobColour** processor.
* Upstreams: SeaWiFS, MODIS, MERIS, VIIRS-SNPP & JPSS1, OLCI-S3A & S3B for the **"multi"** products, and S3A & S3B only for the **"olci"** products.
* Variables: Chlorophyll-a (**CHL**), Phytoplankton Functional types and sizes (**PFT**), Primary Production (**PP**).

* Temporal resolutions: **monthly** plus, for some variables, **daily gap-free** based on a space-time interpolation to provide a "cloud free" product.
* Spatial resolutions: **1 km**.
* Recent products are organized in datasets called Near Real Time (**NRT**) and long time-series (from 1997) in datasets called Multi-Years (**MY**).

To find the **Copernicus-GlobColour** products in the catalogue, use the search keyword **"GlobColour"**.

**DOI (product):**   
https://doi.org/10.48670/moi-00288




<img src="https://mdl-metadata.s3.waw3-1.cloudferro.com/metadata/thumbnails/OCEANCOLOUR_BAL_BGC_HR_L3_NRT_009_202.jpg" width="300" height="200"/>



# Baltic Sea, Bio-Geo-Chemical, L3, daily observation


**Product ID:** OCEANCOLOUR_BAL_BGC_HR_L3_NRT_009_202

**Description:** The High-Resolution Ocean Colour (HR-OC) Consortium (Brockmann Consult, Royal Belgian Institute of Natural Sciences, Flemish Institute for Technological Research) distributes Remote Sensing Reflectances (RRS, expressed in sr-1), Turbidity (TUR, expressed in FNU), Solid Particulate Matter Concentration (SPM, expressed in mg/l), spectral particulate backscattering (BBP, expressed in m-1) and chlorophyll-a concentration (CHL, expressed in µg/l) for the Sentinel 2/MSI sensor at 100m resolution for a 20km coastal zone. The products are delivered on a geographic lat-lon grid (EPSG:4326). To limit file size the products are provided in tiles of 600x800 km². RRS and BBP are delivered at nominal central bands of 443, 492, 560, 665, 704, 740, 783, 865 nm. The primary variable from which it is virtually possible to derive all the geophysical and transparency products is the spectral RRS. This, together with the spectral BBP, constitute the category of the 'optics' products. The spectral BBP product is generated from the RRS products using a quasi-analytical algorithm (Lee et al. 2002). The 'transparency' products include TUR and SPM). They are retrieved through the application of automated switching algorithms to the RRS spectra adapted to varying water conditions (Novoa et al. 2017). The GEOPHYSICAL product consists of the Chlorophyll-a concentration (CHL) retrieved via a multi-algorithm approach with optimized quality flagging (O'Reilly et al. 2019, Gons et al. 2005, Lavigne et al. 2021). The NRT products are generally provided withing 24 hours up to 3 days after end of the day.The RRS product is accompanied by a relative uncertainty estimate (unitless) derived by direct comparison of the products to corresponding fiducial reference measurements provided through the AERONET-OC network. The current day data temporal consistency is evaluated as Quality Index (QI) for TUR, SPM and CHL: QI=(CurrentDataPixel-ClimatologyDataPixel)/STDDataPixel where QI is the difference between current data and the relevant climatological field as a signed multiple of climatological standard deviations (STDDataPixel).

**Processing information:**

The HR-OC processing system is deployed on Creodias where Sentinel 2/MSI L1C data are available. The production control element is being hosted within the infrastructure of Brockmann Consult. The processing chain consists of:
* Resampling to 60m and mosaic generation of the set of Sentinel-2 MSI L1C granules of a single overpass that cover a single UTM zone.
* Application of  a glint correction taking into account the detector viewing angles
* Application of a coastal mask with 20km water + 20km land. The result is a L1C mosaic tile with data just in the coastal area optimized for compression.
* Level 2 processing with pixel identification (IdePix), atmospheric correction (C2RCC and ACOLITE or iCOR), in-water processing and merging (HR-OC L2W processor). The result is a 60m product with the same extent as the L1C mosaic, with variables for optics, transparency, and geophysics, and with data filled in the water part of the coastal area.
* invalid pixel identification takes into account  corrupted (L1) pixels, clouds, cloud shadow, glint, dry-fallen intertidal flats, coastal mixed-pixels, sea ice, melting ice, floating vegetation, non-water objects, and bottom reflection.
* Daily L3 aggregation merges all Level 2 mosaics of a day intersecting with a target tile. All valid water pixels are included in the 20km coastal stripes; all other values are set to NaN. There may be more than a single overpass a day, in particular in the northern regions. The main contribution usually is the mosaic of the zone, but also adjacent mosaics may overlap. This step comprises resampling to the 100m target grid. 
* Monthly L4 aggregation combines all Level 3 products of a month and a single tile. The output is a set of 3 NetCDF datasets for optics, transparency, and geophysics respectively, for the tile and month.
* Gap filling combines all daily products of a period and generates (partially) gap-filled daily products again. The output of gap filling are 3 datasets for optics (BBP443 only), transparency, and geophysics per day.

**Description of observation methods/instruments:**

Ocean colour technique exploits the emerging electromagnetic radiation from the sea surface in different wavelengths. The spectral variability of this signal defines the so-called ocean colour which is affected by the presence of phytoplankton.

**Quality / Accuracy / Calibration information:**

A detailed description of the calibration and validation activities performed over this product can be found on the CMEMS web portal and in CMEMS-BGP_HR-QUID-009-201to212.

**Suitability, Expected type of users / uses:**

This product is meant for use for educational purposes and for the managing of the marine safety, marine resources, marine and coastal environment and for climate and seasonal studies.

**Dataset names: **

*cmems_obs_oc_bal_bgc_geophy_nrt_l3-hr_P1D-v01
*cmems_obs_oc_bal_bgc_transp_nrt_l3-hr_P1D-v01
*cmems_obs_oc_bal_bgc_optics_nrt_l3-hr_P1D-v01

**Files format:**
*netCDF-4, CF-1.7
*INSPIRE compliant.

**DOI (product):**   
https://doi.org/10.48670/moi-00079

**References:**

* Lavigne, H., et al. (2021), Quality-control tests for OC4, OC5 and NIR-red satellite chlorophyll-a algorithms applied to coastal waters, Remote Sensing of Environment, in press.
* Lee, Z. P., et al. (2002), Deriving inherent optical properties from water color: A multi- band quasi-analytical algorithm for optically deep waters, Applied Optics, 41, 5755-5772.
* Novoa, S., et al. (2017), Atmospheric corrections and multi-conditional algorithm for multi-sensor remote sensing of suspended particulate matter in low-to-high turbidity levels coastal waters. Remote Sens., v. 9, 61.
* Gons, et al. (2005), Effect of a waveband shift on chlorophyll retrieval from MERIS imagery of inland and coastal waters, J. Plankton Res., v. 27, n. 1, p. 125-127.
* O'Reilly, et al. (2019), Chlorophyll algorithms for ocean color sensors-OC4, OC5 & OC6. Remote Sensing of Environment. 229, 32–47.





<img src="https://mdl-metadata.s3.waw3-1.cloudferro.com/metadata/thumbnails/OCEANCOLOUR_BAL_BGC_HR_L4_NRT_009_208.jpg" width="300" height="200"/>



# Baltic Sea, Bio-Geo-Chemical, L4, monthly means and interpolated daily observation


**Product ID:** OCEANCOLOUR_BAL_BGC_HR_L4_NRT_009_208

**Description:** The High-Resolution Ocean Colour (HR-OC) Consortium (Brockmann Consult, Royal Belgian Institute of Natural Sciences, Flemish Institute for Technological Research) distributes Level 4 (L4) Turbidity (TUR, expressed in FNU), Solid Particulate Matter Concentration (SPM, expressed in mg/l), particulate backscattering at 443nm (BBP443, expressed in m-1) and chlorophyll-a concentration (CHL, expressed in µg/l) for the Sentinel 2/MSI sensor at 100m resolution for a 20km coastal zone. The products are delivered on a geographic lat-lon grid (EPSG:4326). To limit file size the products are provided in tiles of 600x800 km². BBP443, constitute the category of the 'optics' products. The  BBP443 product is generated from the L3 RRS products using a quasi-analytical algorithm (Lee et al. 2002). The 'transparency' products include TUR and SPM). They are retrieved through the application of automated switching algorithms to the RRS spectra adapted to varying water conditions (Novoa et al. 2017). The GEOPHYSICAL product consists of the Chlorophyll-a concentration (CHL) retrieved via a multi-algorithm approach with optimized quality flagging (O'Reilly et al. 2019, Gons et al. 2005, Lavigne et al. 2021). Monthly products (P1M) are temporal aggregates of the daily L3 products. Daily products contain gaps in cloudy areas and where there is no overpass at the respective day. Aggregation collects the non-cloudy (and non-frozen) contributions to each pixel. Contributions are averaged per variable. While this does not guarantee data availability in all pixels in case of persistent clouds, it provides a more complete product compared to the sparsely filled daily products. The Monthly L4 products (P1M) are generally provided withing 4 days after the last acquisition date of the month. Daily gap filled L4 products (P1D) are generated using the DINEOF (Data Interpolating Empirical Orthogonal Functions) approach which reconstructs missing data in geophysical datasets by using a truncated Empirical Orthogonal Functions (EOF) basis in an iterative approach. DINEOF reconstructs missing data in a geophysical dataset by extracting the main patterns of temporal and spatial variability from the data. While originally designed for low resolution data products, recent research has resulted in the optimization of DINEOF to handle high resolution data provided by Sentinel-2 MSI, including cloud shadow detection (Alvera-Azcárate et al., 2021). These types of L4 products are generated and delivered one month after the respective period.


**Processing information:**

The HR-OC processing system is deployed on Creodias where Sentinel 2/MSI L1C data are available. The production control element is being hosted within the infrastructure of Brockmann Consult. The processing chain consists of:
* Resampling to 60m and mosaic generation of the set of Sentinel-2 MSI L1C granules of a single overpass that cover a single UTM zone.
* Application of  a glint correction taking into account the detector viewing angles
* Application of a coastal mask with 20km water + 20km land. The result is a L1C mosaic tile with data just in the coastal area optimized for compression.
* Level 2 processing with pixel identification (IdePix), atmospheric correction (C2RCC and ACOLITE or iCOR), in-water processing and merging (HR-OC L2W processor). The result is a 60m product with the same extent as the L1C mosaic, with variables for optics, transparency, and geophysics, and with data filled in the water part of the coastal area.
* invalid pixel identification takes into account  corrupted (L1) pixels, clouds, cloud shadow, glint, dry-fallen intertidal flats, coastal mixed-pixels, sea ice, melting ice, floating vegetation, non-water objects, and bottom reflection.
* Daily L3 aggregation merges all Level 2 mosaics of a day intersecting with a target tile. All valid water pixels are included in the 20km coastal stripes; all other values are set to NaN. There may be more than a single overpass a day, in particular in the northern regions. The main contribution usually is the mosaic of the zone, but also adjacent mosaics may overlap. This step comprises resampling to the 100m target grid. 
* Monthly L4 aggregation combines all Level 3 products of a month and a single tile. The output is a set of 3 NetCDF datasets for optics, transparency, and geophysics respectively, for the tile and month.
* Gap filling combines all daily products of a period and generates (partially) gap-filled daily products again. The output of gap filling are 3 datasets for optics (BBP443 only), transparency, and geophysics per day.

**Description of observation methods/instruments:**

Ocean colour technique exploits the emerging electromagnetic radiation from the sea surface in different wavelengths. The spectral variability of this signal defines the so-called ocean colour which is affected by the presence of phytoplankton.

**Quality / Accuracy / Calibration information:**

A detailed description of the calibration and validation activities performed over this product can be found on the CMEMS web portal and in CMEMS-BGP_HR-QUID-009-201_to_212.

**Suitability, Expected type of users / uses:**

This product is meant for use for educational purposes and for the managing of the marine safety, marine resources, marine and coastal environment and for climate and seasonal studies.

**Dataset names: **
*cmems_obs_oc_bal_bgc_geophy_nrt_l4-hr_P1M-v01
*cmems_obs_oc_bal_bgc_transp_nrt_l4-hr_P1M-v01
*cmems_obs_oc_bal_bgc_optics_nrt_l4-hr_P1M-v01
*cmems_obs_oc_bal_bgc_geophy_nrt_l4-hr_P1D-v01
*cmems_obs_oc_bal_bgc_transp_nrt_l4-hr_P1D-v01
*cmems_obs_oc_bal_bgc_optics_nrt_l4-hr_P1D-v01

**Files format:**
*netCDF-4, CF-1.7
*INSPIRE compliant.

**DOI (product):**   
https://doi.org/10.48670/moi-00080

**References:**

* Alvera-Azcárate, Aida, et al. (2021), Detection of shadows in high spatial resolution ocean satellite data using DINEOF. Remote Sensing of Environment 253: 112229.
* Lavigne, H., et al. (2021), Quality-control tests for OC4, OC5 and NIR-red satellite chlorophyll-a algorithms applied to coastal waters, Remote Sensing of Environment, in press.
* Lee, Z. P., et al. (2002), Deriving inherent optical properties from water color: A multi- band quasi-analytical algorithm for optically deep waters, Applied Optics, 41, 5755-5772.
* Novoa, S., et al. (2017), Atmospheric corrections and multi-conditional algorithm for multi-sensor remote sensing of suspended particulate matter in low-to-high turbidity levels coastal waters. Remote Sens., v. 9, 61.
* Gons, et al. (2005), Effect of a waveband shift on chlorophyll retrieval from MERIS imagery of inland and coastal waters, J. Plankton Res., v. 27, n. 1, p. 125-127.
* O'Reilly, et al. (2019), Chlorophyll algorithms for ocean color sensors-OC4, OC5 & OC6. Remote Sensing of Environment. 229, 32–47.





<img src="https://mdl-metadata.s3.waw3-1.cloudferro.com/metadata/thumbnails/OCEANCOLOUR_BAL_BGC_L3_MY_009_133.jpg" width="300" height="200"/>



# Baltic Sea Multiyear Ocean Colour Plankton, Reflectances and Transparency L3 daily observations


**Product ID:** OCEANCOLOUR_BAL_BGC_L3_MY_009_133

**Description:** For the **Baltic Sea** Ocean **Satellite Observations**, the Italian National Research Council (CNR – Rome, Italy), is providing multi-years **Bio-Geo_Chemical (BGC)** regional datasets:
* **_plankton**_ with the phytoplankton chlorophyll concentration (CHL) evaluated via region-specific neural network (Brando et al. 2021) and Phytoplankton Functional Types (PFT) evaluated via region-specific algorithm 
* **_reflectance**_ with the spectral Remote Sensing Reflectance (RRS)
* **_transparency**_ with the diffuse attenuation coefficient of light at 490 nm (KD490) 
* **_pp**_ with the Integrated Primary Production (PP).

**Upstreams**: SeaWiFS, MODIS, MERIS, VIIRS, OLCI-S3A (ESA OC-CCIv6) for the **""multi""** products, and OLCI-S3A & S3B for the **""olci""** products

**Temporal resolution**: daily

**Spatial resolution**: 1 km for **""multi""** (4 km for **""pp""**) and 300 meters for **""olci""**

To find this product in the catalogue, use the search keyword **""OCEANCOLOUR_BAL_BGC_L3_MY""**.

**DOI (product):**   
https://doi.org/10.48670/moi-00296

**References:**

* Brando, V. E., Sammartino, M., Colella, S., Bracaglia, M., Di Cicco, A., D’Alimonte, D., ... & Attila, J. (2021). Phytoplankton bloom dynamics in the Baltic sea using a consistently reprocessed time series of multi-sensor reflectance and novel chlorophyll-a retrievals. Remote Sensing, 13(16), 3071





<img src="https://mdl-metadata.s3.waw3-1.cloudferro.com/metadata/thumbnails/OCEANCOLOUR_BAL_BGC_L3_NRT_009_131.jpg" width="300" height="200"/>



# Baltic Sea Ocean Colour Plankton, Reflectances, Transparency and Optics L3 NRT daily observations


**Product ID:** OCEANCOLOUR_BAL_BGC_L3_NRT_009_131

**Description:** For the **Baltic Sea** Ocean **Satellite Observations**, the Italian National Research Council (CNR – Rome, Italy), is providing **Bio-Geo_Chemical (BGC)** regional datasets:
* **_plankton**_ with the phytoplankton chlorophyll concentration (CHL) evaluated via region-specific neural network (Brando et al. 2021) and Phytoplankton Functional Types (PFT) evaluated via region-specific algorithm
* **_reflectance**_ with the spectral Remote Sensing Reflectance (RRS)
* **_transparency**_ with the diffuse attenuation coefficient of light at 490 nm (KD490) 
* **_optics**_ including the IOPs (Inherent Optical Properties) such as absorption and scattering and particulate and dissolved matter (ADG, APH, BBP), via QAAv6 model (Lee et al., 2002 and updates)

**Upstreams**: OLCI-S3A & S3B 

**Temporal resolution**: daily

**Spatial resolution**: 300 meters 

To find this product in the catalogue, use the search keyword **""OCEANCOLOUR_BAL_BGC_L3_NRT""**.

**DOI (product):**   
https://doi.org/10.48670/moi-00294

**References:**

* Brando, V. E., Sammartino, M., Colella, S., Bracaglia, M., Di Cicco, A., D’Alimonte, D., ... & Attila, J. (2021). Phytoplankton bloom dynamics in the Baltic Sea using a consistently reprocessed time series of multi-sensor reflectance and novel chlorophyll-a retrievals. Remote Sensing, 13(16), 3071
* Lee, Z. P., et al. (2002), Deriving inherent optical properties from water color: A multi- band quasi-analytical algorithm for optically deep waters, Applied Optics, 41, 5755-5772





<img src="https://mdl-metadata.s3.waw3-1.cloudferro.com/metadata/thumbnails/OCEANCOLOUR_BAL_BGC_L4_MY_009_134.jpg" width="300" height="200"/>



# Baltic Sea Multiyear Ocean Colour Plankton monthly observations


**Product ID:** OCEANCOLOUR_BAL_BGC_L4_MY_009_134

**Description:** For the **Baltic Sea** Ocean **Satellite Observations**, the Italian National Research Council (CNR – Rome, Italy), is providing multi-years **Bio-Geo_Chemical (BGC)** regional datasets:
* **_plankton**_ with the phytoplankton chlorophyll concentration (CHL) evaluated via region-specific neural network (Brando et al. 2021)

**Upstreams**: SeaWiFS, MODIS, MERIS, VIIRS, OLCI-S3A (ESA OC-CCIv5) for the **""multi""** products, and OLCI-S3A & S3B for the **""olci""** products

**Temporal resolutions**: monthly

**Spatial resolution**: 1 km for **""multi""** and 300 meters for **""olci""**

To find this product in the catalogue, use the search keyword **""OCEANCOLOUR_BAL_BGC_L4_MY""**.

**DOI (product):**   
https://doi.org/10.48670/moi-00308

**References:**

* Brando, V. E., Sammartino, M., Colella, S., Bracaglia, M., Di Cicco, A., D’Alimonte, D., ... & Attila, J. (2021). Phytoplankton bloom dynamics in the Baltic sea using a consistently reprocessed time series of multi-sensor reflectance and novel chlorophyll-a retrievals. Remote Sensing, 13(16), 3071





<img src="https://mdl-metadata.s3.waw3-1.cloudferro.com/metadata/thumbnails/OCEANCOLOUR_BAL_BGC_L4_NRT_009_132.jpg" width="300" height="200"/>



# Baltic Sea Surface Ocean Colour Plankton from Sentinel-3 OLCI L4 monthly observations


**Product ID:** OCEANCOLOUR_BAL_BGC_L4_NRT_009_132

**Description:** For the **Baltic Sea** Ocean **Satellite Observations**, the Italian National Research Council (CNR – Rome, Italy), is providing **Bio-Geo_Chemical (BGC)** regional datasets:
* **_plankton**_ with the phytoplankton chlorophyll concentration (CHL) evaluated via region-specific neural network (Brando et al. 2021)

**Upstreams**: OLCI-S3A & S3B 

**Temporal resolution**: monthly 

**Spatial resolution**: 300 meters 

To find this product in the catalogue, use the search keyword **""OCEANCOLOUR_BAL_BGC_L4_NRT""**.

**DOI (product):**   
https://doi.org/10.48670/moi-00295

**References:**

* Brando, V. E., Sammartino, M., Colella, S., Bracaglia, M., Di Cicco, A., D’Alimonte, D., ... & Attila, J. (2021). Phytoplankton bloom dynamics in the Baltic Sea using a consistently reprocessed time series of multi-sensor reflectance and novel chlorophyll-a retrievals. Remote Sensing, 13(16), 3071





<img src="https://mdl-metadata.s3.waw3-1.cloudferro.com/metadata/thumbnails/OCEANCOLOUR_BLK_BGC_HR_L3_NRT_009_206.jpg" width="300" height="200"/>



# Black Sea, Bio-Geo-Chemical, L3, daily observation


**Product ID:** OCEANCOLOUR_BLK_BGC_HR_L3_NRT_009_206

**Description:** The High-Resolution Ocean Colour (HR-OC) Consortium (Brockmann Consult, Royal Belgian Institute of Natural Sciences, Flemish Institute for Technological Research) distributes Remote Sensing Reflectances (RRS, expressed in sr-1), Turbidity (TUR, expressed in FNU), Solid Particulate Matter Concentration (SPM, expressed in mg/l), spectral particulate backscattering (BBP, expressed in m-1) and chlorophyll-a concentration (CHL, expressed in µg/l) for the Sentinel 2/MSI sensor at 100m resolution for a 20km coastal zone. The products are delivered on a geographic lat-lon grid (EPSG:4326). To limit file size the products are provided in tiles of 600x800 km². RRS and BBP are delivered at nominal central bands of 443, 492, 560, 665, 704, 740, 783, 865 nm. The primary variable from which it is virtually possible to derive all the geophysical and transparency products is the spectral RRS. This, together with the spectral BBP, constitute the category of the 'optics' products. The spectral BBP product is generated from the RRS products using a quasi-analytical algorithm (Lee et al. 2002). The 'transparency' products include TUR and SPM). They are retrieved through the application of automated switching algorithms to the RRS spectra adapted to varying water conditions (Novoa et al. 2017). The GEOPHYSICAL product consists of the Chlorophyll-a concentration (CHL) retrieved via a multi-algorithm approach with optimized quality flagging (O'Reilly et al. 2019, Gons et al. 2005, Lavigne et al. 2021). The NRT products are generally provided within 24 hours up to 3 days after end of the day.The RRS product is accompanied by a relative uncertainty estimate (unitless) derived by direct comparison of the products to corresponding fiducial reference measurements provided through the AERONET-OC network. The current day data temporal consistency is evaluated as Quality Index (QI) for TUR, SPM and CHL: QI=(CurrentDataPixel-ClimatologyDataPixel)/STDDataPixel where QI is the difference between current data and the relevant climatological field as a signed multiple of climatological standard deviations (STDDataPixel).

**Processing information:**

The HR-OC processing system is deployed on Creodias where Sentinel 2/MSI L1C data are available. The production control element is being hosted within the infrastructure of Brockmann Consult. The processing chain consists of:
* Resampling to 60m and mosaic generation of the set of Sentinel-2 MSI L1C granules of a single overpass that cover a single UTM zone.
* Application of  a glint correction taking into account the detector viewing angles
* Application of a coastal mask with 20km water + 20km land. The result is a L1C mosaic tile with data just in the coastal area optimized for compression.
* Level 2 processing with pixel identification (IdePix), atmospheric correction (C2RCC and ACOLITE or iCOR), in-water processing and merging (HR-OC L2W processor). The result is a 60m product with the same extent as the L1C mosaic, with variables for optics, transparency, and geophysics, and with data filled in the water part of the coastal area.
* invalid pixel identification takes into account  corrupted (L1) pixels, clouds, cloud shadow, glint, dry-fallen intertidal flats, coastal mixed-pixels, sea ice, melting ice, floating vegetation, non-water objects, and bottom reflection.
* Daily L3 aggregation merges all Level 2 mosaics of a day intersecting with a target tile. All valid water pixels are included in the 20km coastal stripes; all other values are set to NaN. There may be more than a single overpass a day, in particular in the northern regions. The main contribution usually is the mosaic of the zone, but also adjacent mosaics may overlap. This step comprises resampling to the 100m target grid. 
* Monthly L4 aggregation combines all Level 3 products of a month and a single tile. The output is a set of 3 NetCDF datasets for optics, transparency, and geophysics respectively, for the tile and month.
* Gap filling combines all daily products of a period and generates (partially) gap-filled daily products again. The output of gap filling are 3 datasets for optics (BBP443 only), transparency, and geophysics per day.

**Description of observation methods/instruments:**

Ocean colour technique exploits the emerging electromagnetic radiation from the sea surface in different wavelengths. The spectral variability of this signal defines the so-called ocean colour which is affected by the presence of phytoplankton.

**Quality / Accuracy / Calibration information:**

A detailed description of the calibration and validation activities performed over this product can be found on the CMEMS web portal and in CMEMS-BGP_HR-QUID-009-201to212.

**Suitability, Expected type of users / uses:**

This product is meant for use for educational purposes and for the managing of the marine safety, marine resources, marine and coastal environment and for climate and seasonal studies.

**Dataset names: **

*cmems_obs_oc_blk_bgc_geophy_nrt_l3-hr_P1D-v01
*cmems_obs_oc_blk_bgc_transp_nrt_l3-hr_P1D-v01
*cmems_obs_oc_blk_bgc_optics_nrt_l3-hr_P1D-v01

**Files format:**
*netCDF-4, CF-1.7
*INSPIRE compliant.

**DOI (product):**   
https://doi.org/10.48670/moi-00086

**References:**

* Lavigne, H., et al. (2021), Quality-control tests for OC4, OC5 and NIR-red satellite chlorophyll-a algorithms applied to coastal waters, Remote Sensing of Environment, in press.
* Lee, Z. P., et al. (2002), Deriving inherent optical properties from water color: A multi- band quasi-analytical algorithm for optically deep waters, Applied Optics, 41, 5755-5772.
* Novoa, S., et al. (2017), Atmospheric corrections and multi-conditional algorithm for multi-sensor remote sensing of suspended particulate matter in low-to-high turbidity levels coastal waters. Remote Sens., v. 9, 61.
* Gons, et al. (2005), Effect of a waveband shift on chlorophyll retrieval from MERIS imagery of inland and coastal waters, J. Plankton Res., v. 27, n. 1, p. 125-127.
* O'Reilly, et al. (2019), Chlorophyll algorithms for ocean color sensors-OC4, OC5 & OC6. Remote Sensing of Environment. 229, 32–47.





<img src="https://mdl-metadata.s3.waw3-1.cloudferro.com/metadata/thumbnails/OCEANCOLOUR_BLK_BGC_HR_L4_NRT_009_212.jpg" width="300" height="200"/>



# Black Sea, Bio-Geo-Chemical, L4, monthly means and interpolated daily observation


**Product ID:** OCEANCOLOUR_BLK_BGC_HR_L4_NRT_009_212

**Description:** The High-Resolution Ocean Colour (HR-OC) Consortium (Brockmann Consult, Royal Belgian Institute of Natural Sciences, Flemish Institute for Technological Research) distributes Level 4 (L4) Turbidity (TUR, expressed in FNU), Solid Particulate Matter Concentration (SPM, expressed in mg/l), particulate backscattering at 443nm (BBP443, expressed in m-1) and chlorophyll-a concentration (CHL, expressed in µg/l) for the Sentinel 2/MSI sensor at 100m resolution for a 20km coastal zone. The products are delivered on a geographic lat-lon grid (EPSG:4326). To limit file size the products are provided in tiles of 600x800 km². BBP443, constitute the category of the 'optics' products. The  BBP443 product is generated from the L3 RRS products using a quasi-analytical algorithm (Lee et al. 2002). The 'transparency' products include TUR and SPM). They are retrieved through the application of automated switching algorithms to the RRS spectra adapted to varying water conditions (Novoa et al. 2017). The GEOPHYSICAL product consists of the Chlorophyll-a concentration (CHL) retrieved via a multi-algorithm approach with optimized quality flagging (O'Reilly et al. 2019, Gons et al. 2005, Lavigne et al. 2021). Monthly products (P1M) are temporal aggregates of the daily L3 products. Daily products contain gaps in cloudy areas and where there is no overpass at the respective day. Aggregation collects the non-cloudy (and non-frozen) contributions to each pixel. Contributions are averaged per variable. While this does not guarantee data availability in all pixels in case of persistent clouds, it provides a more complete product compared to the sparsely filled daily products. The Monthly L4 products (P1M) are generally provided withing 4 days after the last acquisition date of the month. Daily gap filled L4 products (P1D) are generated using the DINEOF (Data Interpolating Empirical Orthogonal Functions) approach which reconstructs missing data in geophysical datasets by using a truncated Empirical Orthogonal Functions (EOF) basis in an iterative approach. DINEOF reconstructs missing data in a geophysical dataset by extracting the main patterns of temporal and spatial variability from the data. While originally designed for low resolution data products, recent research has resulted in the optimization of DINEOF to handle high resolution data provided by Sentinel-2 MSI, including cloud shadow detection (Alvera-Azcárate et al., 2021). These types of L4 products are generated and delivered one month after the respective period.

**Processing information:**

The HR-OC processing system is deployed on Creodias where Sentinel 2/MSI L1C data are available. The production control element is being hosted within the infrastructure of Brockmann Consult. The processing chain consists of:
* Resampling to 60m and mosaic generation of the set of Sentinel-2 MSI L1C granules of a single overpass that cover a single UTM zone.
* Application of  a glint correction taking into account the detector viewing angles
* Application of a coastal mask with 20km water + 20km land. The result is a L1C mosaic tile with data just in the coastal area optimized for compression.
* Level 2 processing with pixel identification (IdePix), atmospheric correction (C2RCC and ACOLITE or iCOR), in-water processing and merging (HR-OC L2W processor). The result is a 60m product with the same extent as the L1C mosaic, with variables for optics, transparency, and geophysics, and with data filled in the water part of the coastal area.
* invalid pixel identification takes into account  corrupted (L1) pixels, clouds, cloud shadow, glint, dry-fallen intertidal flats, coastal mixed-pixels, sea ice, melting ice, floating vegetation, non-water objects, and bottom reflection.
* invalid pixel identification takes into account  corrupted (L1) pixels, clouds, cloud shadow, glint, dry-fallen intertidal flats, coastal mixed-pixels, sea ice, melting ice, floating vegetation, non-water objects, and bottom reflection.    
* Daily L3 aggregation merges all Level 2 mosaics of a day intersecting with a target tile. All valid water pixels are included in the 20km coastal stripes; all other values are set to NaN. There may be more than a single overpass a day, in particular in the northern regions. The main contribution usually is the mosaic of the zone, but also adjacent mosaics may overlap. This step comprises resampling to the 100m target grid. 
* Monthly L4 aggregation combines all Level 3 products of a month and a single tile. The output is a set of 3 NetCDF datasets for optics, transparency, and geophysics respectively, for the tile and month.
* Gap filling combines all daily products of a period and generates (partially) gap-filled daily products again. The output of gap filling are 3 datasets for optics (BBP443 only), transparency, and geophysics per day.

**Description of observation methods/instruments:**

Ocean colour technique exploits the emerging electromagnetic radiation from the sea surface in different wavelengths. The spectral variability of this signal defines the so-called ocean colour which is affected by the presence of phytoplankton.

**Quality / Accuracy / Calibration information:**

A detailed description of the calibration and validation activities performed over this product can be found on the CMEMS web portal and in CMEMS-BGP_HR-QUID-009-201_to_212.

**Suitability, Expected type of users / uses:**

This product is meant for use for educational purposes and for the managing of the marine safety, marine resources, marine and coastal environment and for climate and seasonal studies.

**Dataset names: **
*cmems_obs_oc_blk_bgc_geophy_nrt_l4-hr_P1M-v01
*cmems_obs_oc_blk_bgc_transp_nrt_l4-hr_P1M-v01
*cmems_obs_oc_blk_bgc_optics_nrt_l4-hr_P1M-v01
*cmems_obs_oc_blk_bgc_geophy_nrt_l4-hr_P1D-v01
*cmems_obs_oc_blk_bgc_transp_nrt_l4-hr_P1D-v01
*cmems_obs_oc_blk_bgc_optics_nrt_l4-hr_P1D-v01

**Files format:**
*netCDF-4, CF-1.7
*INSPIRE compliant.
**DOI (product):**   
https://doi.org/10.48670/moi-00087

**References:**

* Alvera-Azcárate, Aida, et al. (2021), Detection of shadows in high spatial resolution ocean satellite data using DINEOF. Remote Sensing of Environment 253: 112229.
* Lavigne, H., et al. (2021), Quality-control tests for OC4, OC5 and NIR-red satellite chlorophyll-a algorithms applied to coastal waters, Remote Sensing of Environment, in press.
* Lee, Z. P., et al. (2002), Deriving inherent optical properties from water color: A multi- band quasi-analytical algorithm for optically deep waters, Applied Optics, 41, 5755-5772.
* Novoa, S., et al. (2017), Atmospheric corrections and multi-conditional algorithm for multi-sensor remote sensing of suspended particulate matter in low-to-high turbidity levels coastal waters. Remote Sens., v. 9, 61.
* Gons, et al. (2005), Effect of a waveband shift on chlorophyll retrieval from MERIS imagery of inland and coastal waters, J. Plankton Res., v. 27, n. 1, p. 125-127.
* O'Reilly, et al. (2019), Chlorophyll algorithms for ocean color sensors-OC4, OC5 & OC6. Remote Sensing of Environment. 229, 32–47.





<img src="https://mdl-metadata.s3.waw3-1.cloudferro.com/metadata/thumbnails/OCEANCOLOUR_BLK_BGC_L3_MY_009_153.jpg" width="300" height="200"/>



# Black Sea, Bio-Geo-Chemical, L3, daily Satellite Observations (1997-ongoing)


**Product ID:** OCEANCOLOUR_BLK_BGC_L3_MY_009_153

**Description:** For the **Black Sea** Ocean **Satellite Observations**, the Italian National Research Council (CNR – Rome, Italy), is providing multi-years **Bio-Geo_Chemical (BGC)** regional datasets:
* **_plankton**_ with the phytoplankton chlorophyll concentration (CHL) evaluated via region-specific algorithms (Zibordi et al., 2015; Kajiyama et al., 2018) and Phytoplankton Functional Types (PFT) evaluated via region-specific algorithm 
* **_reflectance**_ with the spectral Remote Sensing Reflectance (RRS)
* **_transparency**_ with the diffuse attenuation coefficient of light at 490 nm (KD490) 
* **_optics**_ including the IOPs (Inherent Optical Properties) such as absorption and scattering and particulate and dissolved matter (ADG, APH, BBP), via QAAv6 model (Lee et al., 2002 and updates)

**Upstreams**: SeaWiFS, MODIS, MERIS, VIIRS-SNPP & JPSS1, OLCI-S3A & S3B for the **"multi"** products, and OLCI-S3A & S3B for the **"olci"** products

**Temporal resolution**: daily

**Spatial resolution**: 1 km for **"multi"** and 300 meters for **"olci"**

To find this product in the catalogue, use the search keyword **"OCEANCOLOUR_BLK_BGC_L3_MY"**.

**DOI (product):**   
https://doi.org/10.48670/moi-00303

**References:**

* Kajiyama T., D. D’Alimonte, and G. Zibordi, “Algorithms merging for the determination of Chlorophyll-a concentration in the Black Sea,” IEEE Geoscience and Remote Sensing Letters, 2018. [Online]. Available: https://-www.doi.org/¬10.1+D7109/¬LGRS.2018.2883539
* Lee, Z. P., et al. (2002), Deriving inherent optical properties from water color: A multi- band quasi-analytical algorithm for optically deep waters, Applied Optics, 41, 5755-5772
* Zibordi, G., F. Mélin, J.-F. Berthon, and M. Talone (2015). In situ autonomous optical radiometry measurements for satellite ocean color validation in the Western Black Sea. Ocean Sci., 11, 275–286.





<img src="https://mdl-metadata.s3.waw3-1.cloudferro.com/metadata/thumbnails/OCEANCOLOUR_BLK_BGC_L3_NRT_009_151.jpg" width="300" height="200"/>



# Black Sea, Bio-Geo-Chemical, L3, daily Satellite Observations (Near Real Time)


**Product ID:** OCEANCOLOUR_BLK_BGC_L3_NRT_009_151

**Description:** For the **Black Sea** Ocean **Satellite Observations**, the Italian National Research Council (CNR – Rome, Italy), is providing **Bio-Geo_Chemical (BGC)** regional datasets:
* **_plankton**_ with the phytoplankton chlorophyll concentration (CHL) evaluated via region-specific algorithms (Zibordi et al., 2015; Kajiyama et al., 2018) and Phytoplankton Functional Types (PFT) evaluated via region-specific algorithm
* **_reflectance**_ with the spectral Remote Sensing Reflectance (RRS)
* **_transparency**_ with the diffuse attenuation coefficient of light at 490 nm (KD490) 
* **_optics**_ including the IOPs (Inherent Optical Properties) such as absorption and scattering and particulate and dissolved matter (ADG, APH, BBP), via QAAv6 model (Lee et al., 2002 and updates)

**Upstreams**: SeaWiFS, MODIS, MERIS, VIIRS-SNPP & JPSS1, OLCI-S3A & S3B for the **""multi""** products, and OLCI-S3A & S3B for the **""olci""** products

**Temporal resolution**: daily

**Spatial resolutions**: 1 km for **""multi""** and 300 meters for **""olci""**

To find this product in the catalogue, use the search keyword **""OCEANCOLOUR_BLK_BGC_L3_NRT""**.

**DOI (product):**   
https://doi.org/10.48670/moi-00301

**References:**

* Kajiyama T., D. D’Alimonte, and G. Zibordi, “Algorithms merging for the determination of Chlorophyll-a concentration in the Black Sea,” IEEE Geoscience and Remote Sensing Letters, 2018. [Online]. Available: https://-www.doi.org/¬10.1+D7109/¬LGRS.2018.2883539
* Lee, Z. P., et al. (2002), Deriving inherent optical properties from water color: A multi- band quasi-analytical algorithm for optically deep waters, Applied Optics, 41, 5755-5772
* Zibordi, G., F. Mélin, J.-F. Berthon, and M. Talone (2015). In situ autonomous optical radiometry measurements for satellite ocean color validation in the Western Black Sea. Ocean Sci., 11, 275–286.





<img src="https://mdl-metadata.s3.waw3-1.cloudferro.com/metadata/thumbnails/OCEANCOLOUR_BLK_BGC_L4_MY_009_154.jpg" width="300" height="200"/>



# Black Sea, Bio-Geo-Chemical, L4, monthly means, daily gapfree and climatology Satellite Observations (1997-ongoing)


**Product ID:** OCEANCOLOUR_BLK_BGC_L4_MY_009_154

**Description:** For the **Black Sea** Ocean **Satellite Observations**, the Italian National Research Council (CNR – Rome, Italy), is providing multi-years **Bio-Geo_Chemical (BGC)** regional datasets:
* **_plankton**_ with the phytoplankton chlorophyll concentration (CHL) evaluated via region-specific algorithms (Zibordi et al., 2015; Kajiyama et al., 2018), and the interpolated **gap-free** Chl concentration (to provide a ""cloud free"" product) estimated by means of a modified version of the DINEOF algorithm (Volpe et al., 2018); moreover, daily climatology for chlorophyll concentration is provided.
* **_pp**_ with the Integrated Primary Production (PP).

**Upstreams**: SeaWiFS, MODIS, MERIS, VIIRS-SNPP & JPSS1, OLCI-S3A & S3B for the **""multi""** products, and OLCI-S3A & S3B for the **""olci""** products

**Temporal resolutions**: monthly and daily (for **""gap-free""**, **""pp""** and climatology data)

**Spatial resolution**: 1 km for **""multi""** (4 km for **""pp""**) and 300 meters for **""olci""**

To find this product in the catalogue, use the search keyword **""OCEANCOLOUR_BLK_BGC_L4_MY""**.

**DOI (product):**   
https://doi.org/10.48670/moi-00304

**References:**

* Kajiyama T., D. D’Alimonte, and G. Zibordi, “Algorithms merging for the determination of Chlorophyll-a concentration in the Black Sea,” IEEE Geoscience and Remote Sensing Letters, 2018. [Online]. Available: https://-www.doi.org/¬10.1+D7109/¬LGRS.2018.2883539
* Volpe, G., Buongiorno Nardelli, B., Colella, S., Pisano, A. and Santoleri, R. (2018). An Operational Interpolated Ocean Colour Product in the Mediterranean Sea, in New Frontiers in Operational Oceanography, edited by E. P. Chassignet, A. Pascual, J. Tintorè, and J. Verron, pp. 227–244
* Zibordi, G., F. Mélin, J.-F. Berthon, and M. Talone (2015). In situ autonomous optical radiometry measurements for satellite ocean color validation in the Western Black Sea. Ocean Sci., 11, 275–286.





<img src="https://mdl-metadata.s3.waw3-1.cloudferro.com/metadata/thumbnails/OCEANCOLOUR_BLK_BGC_L4_NRT_009_152.jpg" width="300" height="200"/>



# Black Sea, Bio-Geo-Chemical, L4, monthly means, daily gapfree and climatology Satellite Observations (Near Real Time)


**Product ID:** OCEANCOLOUR_BLK_BGC_L4_NRT_009_152

**Description:** For the **Black Sea** Ocean **Satellite Observations**, the Italian National Research Council (CNR – Rome, Italy), is providing **Bio-Geo_Chemical (BGC)** regional datasets:
* **_plankton**_ with the phytoplankton chlorophyll concentration (CHL) evaluated via region-specific algorithms (Zibordi et al., 2015; Kajiyama et al., 2018), and the interpolated **gap-free** Chl concentration (to provide a ""cloud free"" product) estimated by means of a modified version of the DINEOF algorithm (Volpe et al., 2018)
* **_transparency**_ with the diffuse attenuation coefficient of light at 490 nm (KD490) (for **""multi**"" observations achieved via region-specific algorithm, Volpe et al., 2019)
* **_pp**_ with the Integrated Primary Production (PP).

**Upstreams**: SeaWiFS, MODIS, MERIS, VIIRS-SNPP & JPSS1, OLCI-S3A & S3B for the **""multi""** products, and OLCI-S3A & S3B for the **""olci""** products

**Temporal resolutions**: monthly and daily (for **""gap-free""** and **""pp""** data)

**Spatial resolutions**: 1 km for **""multi""** (4 km for **""pp""**) and 300 meters for **""olci""**

To find this product in the catalogue, use the search keyword **""OCEANCOLOUR_BLK_BGC_L4_NRT""**.

**DOI (product):**   
https://doi.org/10.48670/moi-00302

**References:**

* Kajiyama T., D. D’Alimonte, and G. Zibordi, “Algorithms merging for the determination of Chlorophyll-a concentration in the Black Sea,” IEEE Geoscience and Remote Sensing Letters, 2018. [Online]. Available: https://-www.doi.org/¬10.1+D7109/¬LGRS.2018.2883539
* Volpe, G., Buongiorno Nardelli, B., Colella, S., Pisano, A. and Santoleri, R. (2018). An Operational Interpolated Ocean Colour Product in the Mediterranean Sea, in New Frontiers in Operational Oceanography, edited by E. P. Chassignet, A. Pascual, J. Tintorè, and J. Verron, pp. 227–244
* Zibordi, G., F. Mélin, J.-F. Berthon, and M. Talone (2015). In situ autonomous optical radiometry measurements for satellite ocean color validation in the Western Black Sea. Ocean Sci., 11, 275–286.





<img src="https://mdl-metadata.s3.waw3-1.cloudferro.com/metadata/thumbnails/OCEANCOLOUR_GLO_BGC_L3_MY_009_103.jpg" width="300" height="200"/>



# Global Ocean Colour (Copernicus-GlobColour), Bio-Geo-Chemical, L3 (daily) from Satellite Observations (1997-ongoing)


**Product ID:** OCEANCOLOUR_GLO_BGC_L3_MY_009_103

**Description:** For the **Global** Ocean **Satellite Observations**, ACRI-ST company (Sophia Antipolis, France) is providing **Bio-Geo-Chemical (BGC)** products based on the **Copernicus-GlobColour** processor.
* Upstreams: SeaWiFS, MODIS, MERIS, VIIRS-SNPP & JPSS1, OLCI-S3A & S3B for the **""multi""** products, and S3A & S3B only for the **""olci""** products.
* Variables: Chlorophyll-a (**CHL**), Gradient of Chlorophyll-a (**CHL_gradient**), Phytoplankton Functional types and sizes (**PFT**), Suspended Matter (**SPM**), Secchi Transparency Depth (**ZSD**), Diffuse Attenuation (**KD490**), Particulate Backscattering (**BBP**), Absorption Coef. (**CDM**) and  Reflectance (**RRS**).

* Temporal resolutions: **daily**.
* Spatial resolutions: **4 km** and a finer resolution based on olci **300 meters** inputs.
* Recent products are organized in datasets called Near Real Time (**NRT**) and long time-series (from 1997) in datasets called Multi-Years (**MY**).

To find the **Copernicus-GlobColour** products in the catalogue, use the search keyword **""GlobColour""**."

**DOI (product):**   
https://doi.org/10.48670/moi-00280




<img src="https://mdl-metadata.s3.waw3-1.cloudferro.com/metadata/thumbnails/OCEANCOLOUR_GLO_BGC_L3_MY_009_107.jpg" width="300" height="200"/>



# Global Ocean Colour Plankton and Reflectances MY L3 daily observations


**Product ID:** OCEANCOLOUR_GLO_BGC_L3_MY_009_107

**Description:** For the **Global** Ocean **Satellite Observations**, Brockmann Consult (BC) is providing **Bio-Geo_Chemical (BGC)** products based on the ESA-CCI inputs.
* Upstreams: SeaWiFS, MODIS, MERIS, VIIRS-SNPP, OLCI-S3A & OLCI-S3B for the **""multi""** products.
* Variables: Chlorophyll-a (**CHL**), Phytoplankton Functional types and sizes (**PFT**) and  Reflectance (**RRS**).

* Temporal resolutions: **daily**, **monthly**.
* Spatial resolutions: **4 km** (multi).
* Recent products are organized in datasets called Near Real Time (**NRT**) and long time-series (from 1997) in datasets called Multi-Years (**MY**).

To find these products in the catalogue, use the search keyword **""ESA-CCI""**.

**DOI (product):**   
https://doi.org/10.48670/moi-00282




<img src="https://mdl-metadata.s3.waw3-1.cloudferro.com/metadata/thumbnails/OCEANCOLOUR_GLO_BGC_L3_NRT_009_101.jpg" width="300" height="200"/>



# Global Ocean Colour (Copernicus-GlobColour), Bio-Geo-Chemical, L3 (daily) from Satellite Observations (Near Real Time)


**Product ID:** OCEANCOLOUR_GLO_BGC_L3_NRT_009_101

**Description:** For the **Global** Ocean **Satellite Observations**, ACRI-ST company (Sophia Antipolis, France) is providing **Bio-Geo-Chemical (BGC)** products based on the **Copernicus-GlobColour** processor.
* Upstreams: SeaWiFS, MODIS, MERIS, VIIRS-SNPP & JPSS1, OLCI-S3A & S3B for the **""multi""** products, and S3A & S3B only for the **""olci""** products.
* Variables: Chlorophyll-a (**CHL**), Gradient of Chlorophyll-a (**CHL_gradient**), Phytoplankton Functional types and sizes (**PFT**), Suspended Matter (**SPM**), Secchi Transparency Depth (**ZSD**), Diffuse Attenuation (**KD490**), Particulate Backscattering (**BBP**), Absorption Coef. (**CDM**) and  Reflectance (**RRS**).

* Temporal resolutions: **daily**
* Spatial resolutions: **4 km** and a finer resolution based on olci **300 meters** inputs.
* Recent products are organized in datasets called Near Real Time (**NRT**) and long time-series (from 1997) in datasets called Multi-Years (**MY**).

To find the **Copernicus-GlobColour** products in the catalogue, use the search keyword **""GlobColour""**.

**DOI (product):**   
https://doi.org/10.48670/moi-00278




<img src="https://mdl-metadata.s3.waw3-1.cloudferro.com/metadata/thumbnails/OCEANCOLOUR_GLO_BGC_L4_MY_009_104.jpg" width="300" height="200"/>



# Global Ocean Colour (Copernicus-GlobColour), Bio-Geo-Chemical, L4 (monthly and interpolated) from Satellite Observations (1997-ongoing)


**Product ID:** OCEANCOLOUR_GLO_BGC_L4_MY_009_104

**Description:** For the **Global** Ocean **Satellite Observations**, ACRI-ST company (Sophia Antipolis, France) is providing **Bio-Geo-Chemical (BGC)** products based on the **Copernicus-GlobColour** processor.
* Upstreams: SeaWiFS, MODIS, MERIS, VIIRS-SNPP & JPSS1, OLCI-S3A & S3B for the **""multi""** products, and S3A & S3B only for the **""olci""** products.
* Variables: Chlorophyll-a (**CHL**), Phytoplankton Functional types and sizes (**PFT**), Primary Production (**PP**), Suspended Matter (**SPM**), Secchi Transparency Depth (**ZSD**), Diffuse Attenuation (**KD490**), Particulate Backscattering (**BBP**), Absorption Coef. (**CDM**) and  Reflectance (**RRS**).

* Temporal resolutions: **monthly** plus, for some variables, **daily gap-free** based on a space-time interpolation to provide a ""cloud free"" product.
* Spatial resolutions: **4 km** and a finer resolution based on olci **300 meters** inputs.
* Recent products are organized in datasets called Near Real Time (**NRT**) and long time-series (from 1997) in datasets called Multi-Years (**MY**).

To find the **Copernicus-GlobColour** products in the catalogue, use the search keyword **""GlobColour""**."

**DOI (product):**   
https://doi.org/10.48670/moi-00281




<img src="https://mdl-metadata.s3.waw3-1.cloudferro.com/metadata/thumbnails/OCEANCOLOUR_GLO_BGC_L4_MY_009_108.jpg" width="300" height="200"/>



# Global Ocean Colour Plankton MY L4 monthly observations


**Product ID:** OCEANCOLOUR_GLO_BGC_L4_MY_009_108

**Description:** For the **Global** Ocean **Satellite Observations**, Brockmann Consult (BC) is providing **Bio-Geo_Chemical (BGC)** products based on the ESA-CCI inputs.
* Upstreams: SeaWiFS, MODIS, MERIS, VIIRS-SNPP,  OLCI-S3A & OLCI-S3B for the **""multi""** products.
* Variables: Chlorophyll-a (**CHL**).

* Temporal resolutions: **monthly**.
* Spatial resolutions: **4 km** (multi).
* Recent products are organized in datasets called Near Real Time (**NRT**) and long time-series (from 1997) in datasets called Multi-Years (**MY**).

To find these products in the catalogue, use the search keyword **""ESA-CCI""**.

**DOI (product):**   
https://doi.org/10.48670/moi-00283




<img src="https://mdl-metadata.s3.waw3-1.cloudferro.com/metadata/thumbnails/OCEANCOLOUR_GLO_BGC_L4_NRT_009_102.jpg" width="300" height="200"/>



# Global Ocean Colour (Copernicus-GlobColour), Bio-Geo-Chemical, L4 (monthly and interpolated) from Satellite Observations (Near Real Time)


**Product ID:** OCEANCOLOUR_GLO_BGC_L4_NRT_009_102

**Description:** For the **Global** Ocean **Satellite Observations**, ACRI-ST company (Sophia Antipolis, France) is providing **Bio-Geo-Chemical (BGC)** products based on the **Copernicus-GlobColour** processor.
* Upstreams: SeaWiFS, MODIS, MERIS, VIIRS-SNPP & JPSS1, OLCI-S3A & S3B for the **"multi"** products, and S3A & S3B only for the **"olci"** products.
* Variables: Chlorophyll-a (**CHL**), Phytoplankton Functional types and sizes (**PFT**), Primary Production (**PP**), Suspended Matter (**SPM**), Secchi Transparency Depth (**ZSD**), Diffuse Attenuation (**KD490**), Particulate Backscattering (**BBP**), Absorption Coef. (**CDM**) and  Reflectance (**RRS**).

* Temporal resolutions: **monthly** plus, for some variables, **daily gap-free** based on a space-time interpolation to provide a "cloud free" product.
* Spatial resolutions: **4 km** and a finer resolution based on olci **300 meters** inputs.
* Recent products are organized in datasets called Near Real Time (**NRT**) and long time-series (from 1997) in datasets called Multi-Years (**MY**).

To find the **Copernicus-GlobColour** products in the catalogue, use the search keyword **"GlobColour"**.

**DOI (product):**   
https://doi.org/10.48670/moi-00279




<img src="https://mdl-metadata.s3.waw3-1.cloudferro.com/metadata/thumbnails/OCEANCOLOUR_IBI_BGC_HR_L3_NRT_009_204.jpg" width="300" height="200"/>



# Iberic Sea, Bio-Geo-Chemical, L3, daily observation


**Product ID:** OCEANCOLOUR_IBI_BGC_HR_L3_NRT_009_204

**Description:** The High-Resolution Ocean Colour (HR-OC) Consortium (Brockmann Consult, Royal Belgian Institute of Natural Sciences, Flemish Institute for Technological Research) distributes Remote Sensing Reflectances (RRS, expressed in sr-1), Turbidity (TUR, expressed in FNU), Solid Particulate Matter Concentration (SPM, expressed in mg/l), spectral particulate backscattering (BBP, expressed in m-1) and chlorophyll-a concentration (CHL, expressed in µg/l) for the Sentinel 2/MSI sensor at 100m resolution for a 20km coastal zone. The products are delivered on a geographic lat-lon grid (EPSG:4326). To limit file size the products are provided in tiles of 600x800 km². RRS and BBP are delivered at nominal central bands of 443, 492, 560, 665, 704, 740, 783, 865 nm. The primary variable from which it is virtually possible to derive all the geophysical and transparency products is the spectral RRS. This, together with the spectral BBP, constitute the category of the 'optics' products. The spectral BBP product is generated from the RRS products using a quasi-analytical algorithm (Lee et al. 2002). The 'transparency' products include TUR and SPM). They are retrieved through the application of automated switching algorithms to the RRS spectra adapted to varying water conditions (Novoa et al. 2017). The GEOPHYSICAL product consists of the Chlorophyll-a concentration (CHL) retrieved via a multi-algorithm approach with optimized quality flagging (O'Reilly et al. 2019, Gons et al. 2005, Lavigne et al. 2021). The NRT products are generally provided withing 24 hours after end of the day.The RRS product is accompanied by a relative uncertainty estimate (unitless) derived by direct comparison of the products to corresponding fiducial reference measurements provided through the AERONET-OC network. The current day data temporal consistency is evaluated as Quality Index (QI) for TUR, SPM and CHL: QI=(CurrentDataPixel-ClimatologyDataPixel)/STDDataPixel where QI is the difference between current data and the relevant climatological field as a signed multiple of climatological standard deviations (STDDataPixel).

**Processing information:**

The HR-OC processing system is deployed on Creodias where Sentinel 2/MSI L1C data are available. The production control element is being hosted within the infrastructure of Brockmann Consult. The processing chain consists of:
* Resampling to 60m and mosaic generation of the set of Sentinel-2 MSI L1C granules of a single overpass that cover a single UTM zone.
* Application of  a glint correction taking into account the detector viewing angles
* Application of a coastal mask with 20km water + 20km land. The result is a L1C mosaic tile with data just in the coastal area optimized for compression.
* Level 2 processing with pixel identification (IdePix), atmospheric correction (C2RCC and ACOLITE or iCOR), in-water processing and merging (HR-OC L2W processor). The result is a 60m product with the same extent as the L1C mosaic, with variables for optics, transparency, and geophysics, and with data filled in the water part of the coastal area.
* invalid pixel identification takes into account  corrupted (L1) pixels, clouds, cloud shadow, glint, dry-fallen intertidal flats, coastal mixed-pixels, sea ice, melting ice, floating vegetation, non-water objects, and bottom reflection.
* Daily L3 aggregation merges all Level 2 mosaics of a day intersecting with a target tile. All valid water pixels are included in the 20km coastal stripes; all other values are set to NaN. There may be more than a single overpass a day, in particular in the northern regions. The main contribution usually is the mosaic of the zone, but also adjacent mosaics may overlap. This step comprises resampling to the 100m target grid. 
* Monthly L4 aggregation combines all Level 3 products of a month and a single tile. The output is a set of 3 NetCDF datasets for optics, transparency, and geophysics respectively, for the tile and month.
* Gap filling combines all daily products of a period and generates (partially) gap-filled daily products again. The output of gap filling are 3 datasets for optics (BBP443 only), transparency, and geophysics per day.

**Description of observation methods/instruments:**

Ocean colour technique exploits the emerging electromagnetic radiation from the sea surface in different wavelengths. The spectral variability of this signal defines the so-called ocean colour which is affected by the presence of phytoplankton.

**Quality / Accuracy / Calibration information:**

A detailed description of the calibration and validation activities performed over this product can be found on the CMEMS web portal and in CMEMS-BGP_HR-QUID-009-201to212.

**Suitability, Expected type of users / uses:**

This product is meant for use for educational purposes and for the managing of the marine safety, marine resources, marine and coastal environment and for climate and seasonal studies.

**Dataset names: **

*cmems_obs_oc_nws_bgc_geophy_nrt_l3-hr_P1D-v01
*cmems_obs_oc_nws_bgc_transp_nrt_l3-hr_P1D-v01
*cmems_obs_oc_nws_bgc_optics_nrt_l3-hr_P1D-v01

**Files format:**
*netCDF-4, CF-1.7
*INSPIRE compliant

**DOI (product):**
https://doi.org/10.48670/moi-00107

**References:**

* Lavigne, H., et al. (2021), Quality-control tests for OC4, OC5 and NIR-red satellite chlorophyll-a algorithms applied to coastal waters, Remote Sensing of Environment, in press.
* Lee, Z. P., et al. (2002), Deriving inherent optical properties from water color: A multi- band quasi-analytical algorithm for optically deep waters, Applied Optics, 41, 5755-5772.
* Novoa, S., et al. (2017), Atmospheric corrections and multi-conditional algorithm for multi-sensor remote sensing of suspended particulate matter in low-to-high turbidity levels coastal waters. Remote Sens., v. 9, 61.
* Gons, et al. (2005), Effect of a waveband shift on chlorophyll retrieval from MERIS imagery of inland and coastal waters, J. Plankton Res., v. 27, n. 1, p. 125-127.
* O'Reilly, et al. (2019), Chlorophyll algorithms for ocean color sensors-OC4, OC5 & OC6. Remote Sensing of Environment. 229, 32–47.





<img src="https://mdl-metadata.s3.waw3-1.cloudferro.com/metadata/thumbnails/OCEANCOLOUR_IBI_BGC_HR_L4_NRT_009_210.jpg" width="300" height="200"/>



# Iberic Sea, Bio-Geo-Chemical, L4, monthly means and interpolated daily observation


**Product ID:** OCEANCOLOUR_IBI_BGC_HR_L4_NRT_009_210

**Description:** The High-Resolution Ocean Colour (HR-OC) Consortium (Brockmann Consult, Royal Belgian Institute of Natural Sciences, Flemish Institute for Technological Research) distributes Level 4 (L4) Turbidity (TUR, expressed in FNU), Solid Particulate Matter Concentration (SPM, expressed in mg/l), particulate backscattering at 443nm (BBP443, expressed in m-1) and chlorophyll-a concentration (CHL, expressed in µg/l) for the Sentinel 2/MSI sensor at 100m resolution for a 20km coastal zone. The products are delivered on a geographic lat-lon grid (EPSG:4326). To limit file size the products are provided in tiles of 600x800 km². BBP443, constitute the category of the 'optics' products. The  BBP443 product is generated from the L3 RRS products using a quasi-analytical algorithm (Lee et al. 2002). The 'transparency' products include TUR and SPM). They are retrieved through the application of automated switching algorithms to the RRS spectra adapted to varying water conditions (Novoa et al. 2017). The GEOPHYSICAL product consists of the Chlorophyll-a concentration (CHL) retrieved via a multi-algorithm approach with optimized quality flagging (O'Reilly et al. 2019, Gons et al. 2005, Lavigne et al. 2021). Monthly products (P1M) are temporal aggregates of the daily L3 products. Daily products contain gaps in cloudy areas and where there is no overpass at the respective day. Aggregation collects the non-cloudy (and non-frozen) contributions to each pixel. Contributions are averaged per variable. While this does not guarantee data availability in all pixels in case of persistent clouds, it provides a more complete product compared to the sparsely filled daily products. The Monthly L4 products (P1M) are generally provided withing 4 days after the last acquisition date of the month. Daily gap filled L4 products (P1D) are generated using the DINEOF (Data Interpolating Empirical Orthogonal Functions) approach which reconstructs missing data in geophysical datasets by using a truncated Empirical Orthogonal Functions (EOF) basis in an iterative approach. DINEOF reconstructs missing data in a geophysical dataset by extracting the main patterns of temporal and spatial variability from the data. While originally designed for low resolution data products, recent research has resulted in the optimization of DINEOF to handle high resolution data provided by Sentinel-2 MSI, including cloud shadow detection (Alvera-Azcárate et al., 2021). These types of L4 products are generated and delivered one month after the respective period.

**Processing information:**

The HR-OC processing system is deployed on Creodias where Sentinel 2/MSI L1C data are available. The production control element is being hosted within the infrastructure of Brockmann Consult. The processing chain consists of:
* Resampling to 60m and mosaic generation of the set of Sentinel-2 MSI L1C granules of a single overpass that cover a single UTM zone.
* Application of  a glint correction taking into account the detector viewing angles
* Application of a coastal mask with 20km water + 20km land. The result is a L1C mosaic tile with data just in the coastal area optimized for compression.
* Level 2 processing with pixel identification (IdePix), atmospheric correction (C2RCC and ACOLITE or iCOR), in-water processing and merging (HR-OC L2W processor). The result is a 60m product with the same extent as the L1C mosaic, with variables for optics, transparency, and geophysics, and with data filled in the water part of the coastal area.
* invalid pixel identification takes into account  corrupted (L1) pixels, clouds, cloud shadow, glint, dry-fallen intertidal flats, coastal mixed-pixels, sea ice, melting ice, floating vegetation, non-water objects, and bottom reflection.    
* Daily L3 aggregation merges all Level 2 mosaics of a day intersecting with a target tile. All valid water pixels are included in the 20km coastal stripes; all other values are set to NaN. There may be more than a single overpass a day, in particular in the northern regions. The main contribution usually is the mosaic of the zone, but also adjacent mosaics may overlap. This step comprises resampling to the 100m target grid. 
* Monthly L4 aggregation combines all Level 3 products of a month and a single tile. The output is a set of 3 NetCDF datasets for optics, transparency, and geophysics respectively, for the tile and month.
* Gap filling combines all daily products of a period and generates (partially) gap-filled daily products again. The output of gap filling are 3 datasets for optics (BBP443 only), transparency, and geophysics per day.

**Description of observation methods/instruments:**

Ocean colour technique exploits the emerging electromagnetic radiation from the sea surface in different wavelengths. The spectral variability of this signal defines the so-called ocean colour which is affected by the presence of phytoplankton.

**Quality / Accuracy / Calibration information:**

A detailed description of the calibration and validation activities performed over this product can be found on the CMEMS web portal and in CMEMS-BGP_HR-QUID-009-201_to_212.

**Suitability, Expected type of users / uses:**

This product is meant for use for educational purposes and for the managing of the marine safety, marine resources, marine and coastal environment and for climate and seasonal studies.

**Dataset names: **
*cmems_obs_oc_ibi_bgc_geophy_nrt_l4-hr_P1M-v01
*cmems_obs_oc_ibi_bgc_transp_nrt_l4-hr_P1M-v01
*cmems_obs_oc_ibi_bgc_optics_nrt_l4-hr_P1M-v01
*cmems_obs_oc_ibi_bgc_geophy_nrt_l4-hr_P1D-v01
*cmems_obs_oc_ibi_bgc_transp_nrt_l4-hr_P1D-v01
*cmems_obs_oc_ibi_bgc_optics_nrt_l4-hr_P1D-v01

**Files format:**
*netCDF-4, CF-1.7
*INSPIRE compliant.

**DOI (product):**   
https://doi.org/10.48670/moi-00108

**References:**

* Alvera-Azcárate, Aida, et al. (2021), Detection of shadows in high spatial resolution ocean satellite data using DINEOF. Remote Sensing of Environment 253: 112229.
* Lavigne, H., et al. (2021), Quality-control tests for OC4, OC5 and NIR-red satellite chlorophyll-a algorithms applied to coastal waters, Remote Sensing of Environment, in press.
* Lee, Z. P., et al. (2002), Deriving inherent optical properties from water color: A multi- band quasi-analytical algorithm for optically deep waters, Applied Optics, 41, 5755-5772.
* Novoa, S., et al. (2017), Atmospheric corrections and multi-conditional algorithm for multi-sensor remote sensing of suspended particulate matter in low-to-high turbidity levels coastal waters. Remote Sens., v. 9, 61.
* Gons, et al. (2005), Effect of a waveband shift on chlorophyll retrieval from MERIS imagery of inland and coastal waters, J. Plankton Res., v. 27, n. 1, p. 125-127.
* O'Reilly, et al. (2019), Chlorophyll algorithms for ocean color sensors-OC4, OC5 & OC6. Remote Sensing of Environment. 229, 32–47.





<img src="https://mdl-metadata.s3.waw3-1.cloudferro.com/metadata/thumbnails/OCEANCOLOUR_MED_BGC_HR_L3_NRT_009_205.jpg" width="300" height="200"/>



# Mediterranean Sea, Bio-Geo-Chemical, L3, daily observation


**Product ID:** OCEANCOLOUR_MED_BGC_HR_L3_NRT_009_205

**Description:** The High-Resolution Ocean Colour (HR-OC) Consortium (Brockmann Consult, Royal Belgian Institute of Natural Sciences, Flemish Institute for Technological Research) distributes Remote Sensing Reflectances (RRS, expressed in sr-1), Turbidity (TUR, expressed in FNU), Solid Particulate Matter Concentration (SPM, expressed in mg/l), spectral particulate backscattering (BBP, expressed in m-1) and chlorophyll-a concentration (CHL, expressed in µg/l) for the Sentinel 2/MSI sensor at 100m resolution for a 20km coastal zone. The products are delivered on a geographic lat-lon grid (EPSG:4326). To limit file size the products are provided in tiles of 600x800 km². RRS and BBP are delivered at nominal central bands of 443, 492, 560, 665, 704, 740, 783, 865 nm. The primary variable from which it is virtually possible to derive all the geophysical and transparency products is the spectral RRS. This, together with the spectral BBP, constitute the category of the 'optics' products. The spectral BBP product is generated from the RRS products using a quasi-analytical algorithm (Lee et al. 2002). The 'transparency' products include TUR and SPM). They are retrieved through the application of automated switching algorithms to the RRS spectra adapted to varying water conditions (Novoa et al. 2017). The GEOPHYSICAL product consists of the Chlorophyll-a concentration (CHL) retrieved via a multi-algorithm approach with optimized quality flagging (O'Reilly et al. 2019, Gons et al. 2005, Lavigne et al. 2021). The NRT products are generally provided withing 24 hours up to 3 days after end of the day.The RRS product is accompanied by a relative uncertainty estimate (unitless) derived by direct comparison of the products to corresponding fiducial reference measurements provided through the AERONET-OC network. The current day data temporal consistency is evaluated as Quality Index (QI) for TUR, SPM and CHL: QI=(CurrentDataPixel-ClimatologyDataPixel)/STDDataPixel where QI is the difference between current data and the relevant climatological field as a signed multiple of climatological standard deviations (STDDataPixel).

**Processing information:**

The HR-OC processing system is deployed on Creodias where Sentinel 2/MSI L1C data are available. The production control element is being hosted within the infrastructure of Brockmann Consult. The processing chain consists of:
* Resampling to 60m and mosaic generation of the set of Sentinel-2 MSI L1C granules of a single overpass that cover a single UTM zone.
* Application of  a glint correction taking into account the detector viewing angles
* Application of a coastal mask with 20km water + 20km land. The result is a L1C mosaic tile with data just in the coastal area optimized for compression.
* Level 2 processing with pixel identification (IdePix), atmospheric correction (C2RCC and ACOLITE or iCOR), in-water processing and merging (HR-OC L2W processor). The result is a 60m product with the same extent as the L1C mosaic, with variables for optics, transparency, and geophysics, and with data filled in the water part of the coastal area.
* invalid pixel identification takes into account  corrupted (L1) pixels, clouds, cloud shadow, glint, dry-fallen intertidal flats, coastal mixed-pixels, sea ice, melting ice, floating vegetation, non-water objects, and bottom reflection.
* Daily L3 aggregation merges all Level 2 mosaics of a day intersecting with a target tile. All valid water pixels are included in the 20km coastal stripes; all other values are set to NaN. There may be more than a single overpass a day, in particular in the northern regions. The main contribution usually is the mosaic of the zone, but also adjacent mosaics may overlap. This step comprises resampling to the 100m target grid. 
* Monthly L4 aggregation combines all Level 3 products of a month and a single tile. The output is a set of 3 NetCDF datasets for optics, transparency, and geophysics respectively, for the tile and month.
* Gap filling combines all daily products of a period and generates (partially) gap-filled daily products again. The output of gap filling are 3 datasets for optics (BBP443 only), transparency, and geophysics per day.

**Description of observation methods/instruments:**

Ocean colour technique exploits the emerging electromagnetic radiation from the sea surface in different wavelengths. The spectral variability of this signal defines the so-called ocean colour which is affected by the presence of phytoplankton.

**Quality / Accuracy / Calibration information:**

A detailed description of the calibration and validation activities performed over this product can be found on the CMEMS web portal and in CMEMS-BGP_HR-QUID-009-201to212.

**Suitability, Expected type of users / uses:**

This product is meant for use for educational purposes and for the managing of the marine safety, marine resources, marine and coastal environment and for climate and seasonal studies.

**Dataset names: **

*cmems_obs_oc_ibi_bgc_geophy_nrt_l3-hr_P1D-v01
*cmems_obs_oc_ibi_bgc_transp_nrt_l3-hr_P1D-v01
*cmems_obs_oc_ibi_bgc_optics_nrt_l3-hr_P1D-v01

**Files format:**
*netCDF-4, CF-1.7
*INSPIRE compliant.

**DOI (product):**   
https://doi.org/10.48670/moi-00109

**References:**

* Lavigne, H., et al. (2021), Quality-control tests for OC4, OC5 and NIR-red satellite chlorophyll-a algorithms applied to coastal waters, Remote Sensing of Environment, in press.
* Lee, Z. P., et al. (2002), Deriving inherent optical properties from water color: A multi- band quasi-analytical algorithm for optically deep waters, Applied Optics, 41, 5755-5772.
* Novoa, S., et al. (2017), Atmospheric corrections and multi-conditional algorithm for multi-sensor remote sensing of suspended particulate matter in low-to-high turbidity levels coastal waters. Remote Sens., v. 9, 61.
* Gons, et al. (2005), Effect of a waveband shift on chlorophyll retrieval from MERIS imagery of inland and coastal waters, J. Plankton Res., v. 27, n. 1, p. 125-127.
* O'Reilly, et al. (2019), Chlorophyll algorithms for ocean color sensors-OC4, OC5 & OC6. Remote Sensing of Environment. 229, 32–47.





<img src="https://mdl-metadata.s3.waw3-1.cloudferro.com/metadata/thumbnails/OCEANCOLOUR_MED_BGC_HR_L4_NRT_009_211.jpg" width="300" height="200"/>



# Mediterranean Sea, Bio-Geo-Chemical, L4, monthly means and interpolated daily observation


**Product ID:** OCEANCOLOUR_MED_BGC_HR_L4_NRT_009_211

**Description:** The High-Resolution Ocean Colour (HR-OC) Consortium (Brockmann Consult, Royal Belgian Institute of Natural Sciences, Flemish Institute for Technological Research) distributes Level 4 (L4) Turbidity (TUR, expressed in FNU), Solid Particulate Matter Concentration (SPM, expressed in mg/l), particulate backscattering at 443nm (BBP443, expressed in m-1) and chlorophyll-a concentration (CHL, expressed in µg/l) for the Sentinel 2/MSI sensor at 100m resolution for a 20km coastal zone. The products are delivered on a geographic lat-lon grid (EPSG:4326). To limit file size the products are provided in tiles of 600x800 km². BBP443, constitute the category of the 'optics' products. The  BBP443 product is generated from the L3 RRS products using a quasi-analytical algorithm (Lee et al. 2002). The 'transparency' products include TUR and SPM). They are retrieved through the application of automated switching algorithms to the RRS spectra adapted to varying water conditions (Novoa et al. 2017). The GEOPHYSICAL product consists of the Chlorophyll-a concentration (CHL) retrieved via a multi-algorithm approach with optimized quality flagging (O'Reilly et al. 2019, Gons et al. 2005, Lavigne et al. 2021). Monthly products (P1M) are temporal aggregates of the daily L3 products. Daily products contain gaps in cloudy areas and where there is no overpass at the respective day. Aggregation collects the non-cloudy (and non-frozen) contributions to each pixel. Contributions are averaged per variable. While this does not guarantee data availability in all pixels in case of persistent clouds, it provides a more complete product compared to the sparsely filled daily products. The Monthly L4 products (P1M) are generally provided withing 4 days after the last acquisition date of the month. Daily gap filled L4 products (P1D) are generated using the DINEOF (Data Interpolating Empirical Orthogonal Functions) approach which reconstructs missing data in geophysical datasets by using a truncated Empirical Orthogonal Functions (EOF) basis in an iterative approach. DINEOF reconstructs missing data in a geophysical dataset by extracting the main patterns of temporal and spatial variability from the data. While originally designed for low resolution data products, recent research has resulted in the optimization of DINEOF to handle high resolution data provided by Sentinel-2 MSI, including cloud shadow detection (Alvera-Azcárate et al., 2021). These types of L4 products are generated and delivered one month after the respective period.

**Processing information:**

The HR-OC processing system is deployed on Creodias where Sentinel 2/MSI L1C data are available. The production control element is being hosted within the infrastructure of Brockmann Consult. The processing chain consists of:
* Resampling to 60m and mosaic generation of the set of Sentinel-2 MSI L1C granules of a single overpass that cover a single UTM zone.
* Application of  a glint correction taking into account the detector viewing angles
* Application of a coastal mask with 20km water + 20km land. The result is a L1C mosaic tile with data just in the coastal area optimized for compression.
* Level 2 processing with pixel identification (IdePix), atmospheric correction (C2RCC and ACOLITE or iCOR), in-water processing and merging (HR-OC L2W processor). The result is a 60m product with the same extent as the L1C mosaic, with variables for optics, transparency, and geophysics, and with data filled in the water part of the coastal area.
* invalid pixel identification takes into account  corrupted (L1-) pixels, clouds, cloud shadow, glint, dry-fallen intertidal flats, coastal mixed-pixels, sea ice, melting ice, floating vegetation, non-water objects, and bottom reflection.
* Daily L3 aggregation merges all Level 2 mosaics of a day intersecting with a target tile. All valid water pixels are included in the 20km coastal stripes; all other values are set to NaN. There may be more than a single overpass a day, in particular in the northern regions. The main contribution usually is the mosaic of the zone, but also adjacent mosaics may overlap. This step comprises resampling to the 100m target grid. 
* Monthly L4 aggregation combines all Level 3 products of a month and a single tile. The output is a set of 3 NetCDF datasets for optics, transparency, and geophysics respectively, for the tile and month.
* Gap filling combines all daily products of a period and generates (partially) gap-filled daily products again. The output of gap filling are 3 datasets for optics (BBP443 only), transparency, and geophysics per day.

**Description of observation methods/instruments:**

Ocean colour technique exploits the emerging electromagnetic radiation from the sea surface in different wavelengths. The spectral variability of this signal defines the so-called ocean colour which is affected by the presence of phytoplankton.

**Quality / Accuracy / Calibration information:**

A detailed description of the calibration and validation activities performed over this product can be found on the CMEMS web portal and in CMEMS-BGP_HR-QUID-009-201_to_212.

**Suitability, Expected type of users / uses:**

This product is meant for use for educational purposes and for the managing of the marine safety, marine resources, marine and coastal environment and for climate and seasonal studies.

**Dataset names: **
*cmems_obs_oc_med_bgc_geophy_nrt_l4-hr_P1M-v01+D19
*cmems_obs_oc_med_bgc_transp_nrt_l4-hr_P1M-v01
*cmems_obs_oc_med_bgc_optics_nrt_l4-hr_P1M-v01
*cmems_obs_oc_med_bgc_geophy_nrt_l4-hr_P1D-v01
*cmems_obs_oc_med_bgc_transp_nrt_l4-hr_P1D-v01
*cmems_obs_oc_med_bgc_optics_nrt_l4-hr_P1D-v01

**Files format:**
*netCDF-4, CF-1.7
*INSPIRE compliant.

**DOI (product):**   
https://doi.org/10.48670/moi-00110

**References:**

* Alvera-Azcárate, Aida, et al. (2021), Detection of shadows in high spatial resolution ocean satellite data using DINEOF. Remote Sensing of Environment 253: 112229.
* Lavigne, H., et al. (2021), Quality-control tests for OC4, OC5 and NIR-red satellite chlorophyll-a algorithms applied to coastal waters, Remote Sensing of Environment, in press.
* Lee, Z. P., et al. (2002), Deriving inherent optical properties from water color: A multi- band quasi-analytical algorithm for optically deep waters, Applied Optics, 41, 5755-5772.
* Novoa, S., et al. (2017), Atmospheric corrections and multi-conditional algorithm for multi-sensor remote sensing of suspended particulate matter in low-to-high turbidity levels coastal waters. Remote Sens., v. 9, 61.
* Gons, et al. (2005), Effect of a waveband shift on chlorophyll retrieval from MERIS imagery of inland and coastal waters, J. Plankton Res., v. 27, n. 1, p. 125-127.
* O'Reilly, et al. (2019), Chlorophyll algorithms for ocean color sensors-OC4, OC5 & OC6. Remote Sensing of Environment. 229, 32–47.





<img src="https://mdl-metadata.s3.waw3-1.cloudferro.com/metadata/thumbnails/OCEANCOLOUR_MED_BGC_L3_MY_009_143.jpg" width="300" height="200"/>



# Mediterranean Sea, Bio-Geo-Chemical, L3, daily Satellite Observations (1997-ongoing)


**Product ID:** OCEANCOLOUR_MED_BGC_L3_MY_009_143

**Description:** For the **Mediterranean Sea** Ocean **Satellite Observations**, the Italian National Research Council (CNR – Rome, Italy), is providing multi-years **Bio-Geo_Chemical (BGC)** regional datasets:
* **_plankton**_ with the phytoplankton chlorophyll concentration (CHL) evaluated via region-specific algorithms (Case 1 waters: Volpe et al., 2019, with new coefficients; Case 2 waters, Berthon and Zibordi, 2004) and Phytoplankton Functional Types (PFT) evaluated via region-specific algorithm (Di Cicco et al. 2017)
* **_reflectance**_ with the spectral Remote Sensing Reflectance (RRS)
* **_transparency**_ with the diffuse attenuation coefficient of light at 490 nm (KD490) (for **"multi**" observations achieved via region-specific algorithm, Volpe et al., 2019)
* **_optics**_ including the IOPs (Inherent Optical Properties) such as absorption and scattering and particulate and dissolved matter (ADG, APH, BBP), via QAAv6 model (Lee et al., 2002 and updates)
* **_pp**_ with the Integrated Primary Production (PP)

**Upstreams**: SeaWiFS, MODIS, MERIS, VIIRS-SNPP & JPSS1, OLCI-S3A & S3B for the **"multi"** products, and OLCI-S3A & S3B for the **"olci"** products

**Temporal resolution**: daily

**Spatial resolution**: 1 km for **"multi"** and 300 meters for **"olci"**

To find this product in the catalogue, use the search keyword **"OCEANCOLOUR_MED_BGC_L3_MY"**.

**DOI (product):**   
https://doi.org/10.48670/moi-00299

**References:**

* Berthon, J.-F., Zibordi, G.: Bio-optical relationships for the northern Adriatic Sea. Int. J. Remote Sens., 25, 1527-1532, 2004
* Di Cicco A, Sammartino M, Marullo S and Santoleri R (2017) Regional Empirical Algorithms for an Improved Identification of Phytoplankton Functional Types and Size Classes in the Mediterranean Sea Using Satellite Data. Front. Mar. Sci. 4:126. doi: 10.3389/fmars.2017.00126
* Lee, Z. P., et al. (2002), Deriving inherent optical properties from water color: A multi- band quasi-analytical algorithm for optically deep waters, Applied Optics, 41, 5755-5772
* Volpe, G., Colella, S., Brando, V. E., Forneris, V., Padula, F. L., Cicco, A. D., ... & Santoleri, R. (2019). Mediterranean ocean colour Level 3 operational multi-sensor processing. Ocean Science, 15(1), 127-146.





<img src="https://mdl-metadata.s3.waw3-1.cloudferro.com/metadata/thumbnails/OCEANCOLOUR_MED_BGC_L3_NRT_009_141.jpg" width="300" height="200"/>



# Mediterranean Sea, Bio-Geo-Chemical, L3, daily Satellite Observations (Near Real Time)


**Product ID:** OCEANCOLOUR_MED_BGC_L3_NRT_009_141

**Description:** For the **Mediterranean Sea** Ocean **Satellite Observations**, the Italian National Research Council (CNR – Rome, Italy), is providing **Bio-Geo_Chemical (BGC)** regional datasets:
* **_plankton**_ with the phytoplankton chlorophyll concentration (CHL) evaluated via region-specific algorithms (Case 1 waters: Volpe et al., 2019, with new coefficients; Case 2 waters, Berthon and Zibordi, 2004) and Phytoplankton Functional Types (PFT) evaluated via region-specific algorithm (Di Cicco et al. 2017)
* **_reflectance**_ with the spectral Remote Sensing Reflectance (RRS)
* **_transparency**_ with the diffuse attenuation coefficient of light at 490 nm (KD490) (for **""multi**"" observations achieved via region-specific algorithm, Volpe et al., 2019)
* **_optics**_ including the IOPs (Inherent Optical Properties) such as absorption and scattering and particulate and dissolved matter (ADG, APH, BBP), via QAAv6 model (Lee et al., 2002 and updates)

**Upstreams**: SeaWiFS, MODIS, MERIS, VIIRS-SNPP & JPSS1, OLCI-S3A & S3B for the **""multi""** products, and OLCI-S3A & S3B for the **""olci""** products

**Temporal resolution**: daily

**Spatial resolutions**: 1 km for **""multi""** and 300 meters for **""olci""**

To find this product in the catalogue, use the search keyword **""OCEANCOLOUR_MED_BGC_L3_NRT""**.

**DOI (product):**   
https://doi.org/10.48670/moi-00297

**References:**

* Berthon, J.-F., Zibordi, G.: Bio-optical relationships for the northern Adriatic Sea. Int. J. Remote Sens., 25, 1527-1532, 2004
* Di Cicco A, Sammartino M, Marullo S and Santoleri R (2017) Regional Empirical Algorithms for an Improved Identification of Phytoplankton Functional Types and Size Classes in the Mediterranean Sea Using Satellite Data. Front. Mar. Sci. 4:126. doi: 10.3389/fmars.2017.00126
* Lee, Z. P., et al. (2002), Deriving inherent optical properties from water color: A multi- band quasi-analytical algorithm for optically deep waters, Applied Optics, 41, 5755-5772.
* Volpe, G., Colella, S., Brando, V. E., Forneris, V., Padula, F. L., Cicco, A. D., ... & Santoleri, R. (2019). Mediterranean ocean colour Level 3 operational multi-sensor processing. Ocean Science, 15(1), 127-146.





<img src="https://mdl-metadata.s3.waw3-1.cloudferro.com/metadata/thumbnails/OCEANCOLOUR_MED_BGC_L4_MY_009_144.jpg" width="300" height="200"/>



# Mediterranean Sea, Bio-Geo-Chemical, L4, monthly means, daily gapfree and climatology Satellite Observations (1997-ongoing)


**Product ID:** OCEANCOLOUR_MED_BGC_L4_MY_009_144

**Description:** For the **Mediterranean Sea** Ocean **Satellite Observations**, the Italian National Research Council (CNR – Rome, Italy), is providing multi-years **Bio-Geo_Chemical (BGC)** regional datasets:
* **_plankton**_ with the phytoplankton chlorophyll concentration (CHL) evaluated via region-specific algorithms (Case 1 waters: Volpe et al., 2019, with new coefficients; Case 2 waters, Berthon and Zibordi, 2004), and the interpolated **gap-free** Chl concentration (to provide a "cloud free" product) estimated by means of a modified version of the DINEOF algorithm (Volpe et al., 2018); moreover, daily climatology for chlorophyll concentration is provided.
* **_pp**_ with the Integrated Primary Production (PP).

**Upstreams**: SeaWiFS, MODIS, MERIS, VIIRS-SNPP & JPSS1, OLCI-S3A & S3B for the **"multi"** products, and OLCI-S3A & S3B for the **"olci"** products

**Temporal resolutions**: monthly and daily (for **"gap-free"** and climatology data)

**Spatial resolution**: 1 km for **"multi"** and 300 meters for **"olci"**

To find this product in the catalogue, use the search keyword **"OCEANCOLOUR_MED_BGC_L4_MY"**.

**DOI (product):**   
https://doi.org/10.48670/moi-00300

**References:**

* Berthon, J.-F., Zibordi, G.: Bio-optical relationships for the northern Adriatic Sea. Int. J. Remote Sens., 25, 1527-1532, 2004
* Volpe, G., Buongiorno Nardelli, B., Colella, S., Pisano, A. and Santoleri, R. (2018). An Operational Interpolated Ocean Colour Product in the Mediterranean Sea, in New Frontiers in Operational Oceanography, edited by E. P. Chassignet, A. Pascual, J. Tintorè, and J. Verron, pp. 227–244
* Volpe, G., Colella, S., Brando, V. E., Forneris, V., Padula, F. L., Cicco, A. D., ... & Santoleri, R. (2019). Mediterranean ocean colour Level 3 operational multi-sensor processing. Ocean Science, 15(1), 127-146.





<img src="https://mdl-metadata.s3.waw3-1.cloudferro.com/metadata/thumbnails/OCEANCOLOUR_MED_BGC_L4_NRT_009_142.jpg" width="300" height="200"/>



# Mediterranean Sea, Bio-Geo-Chemical, L4, monthly means, daily gapfree and climatology Satellite Observations (Near Real Time)


**Product ID:** OCEANCOLOUR_MED_BGC_L4_NRT_009_142

**Description:** For the **Mediterranean Sea** Ocean **Satellite Observations**, the Italian National Research Council (CNR – Rome, Italy), is providing **Bio-Geo_Chemical (BGC)** regional datasets:
* **_plankton**_ with the phytoplankton chlorophyll concentration (CHL) evaluated via region-specific algorithms (Case 1 waters: Volpe et al., 2019, with new coefficients; Case 2 waters, Berthon and Zibordi, 2004), and the interpolated **gap-free** Chl concentration (to provide a ""cloud free"" product) estimated by means of a modified version of the DINEOF algorithm (Volpe et al., 2018)
* **_transparency**_ with the diffuse attenuation coefficient of light at 490 nm (KD490) (for **""multi**"" observations achieved via region-specific algorithm, Volpe et al., 2019)
* **_pp**_ with the Integrated Primary Production (PP).

**Upstreams**: SeaWiFS, MODIS, MERIS, VIIRS-SNPP & JPSS1, OLCI-S3A & S3B for the **""multi""** products, and OLCI-S3A & S3B for the **""olci""** products

**Temporal resolutions**: monthly and daily (for **""gap-free""** and **""pp""** data)

**Spatial resolutions**: 1 km for **""multi""** (4 km for **""pp""**) and 300 meters for **""olci""**

To find this product in the catalogue, use the search keyword **""OCEANCOLOUR_MED_BGC_L4_NRT""**.

**DOI (product):**   
https://doi.org/10.48670/moi-00298

**References:**

* Berthon, J.-F., Zibordi, G.: Bio-optical relationships for the northern Adriatic Sea. Int. J. Remote Sens., 25, 1527-1532, 2004
* Volpe, G., Buongiorno Nardelli, B., Colella, S., Pisano, A. and Santoleri, R. (2018). An Operational Interpolated Ocean Colour Product in the Mediterranean Sea, in New Frontiers in Operational Oceanography, edited by E. P. Chassignet, A. Pascual, J. Tintorè, and J. Verron, pp. 227–244
* Volpe, G., Colella, S., Brando, V. E., Forneris, V., Padula, F. L., Cicco, A. D., ... & Santoleri, R. (2019). Mediterranean ocean colour Level 3 operational multi-sensor processing. Ocean Science, 15(1), 127-146.





<img src="https://mdl-metadata.s3.waw3-1.cloudferro.com/metadata/thumbnails/OCEANCOLOUR_NWS_BGC_HR_L3_NRT_009_203.jpg" width="300" height="200"/>



# North West Shelf Region, Bio-Geo-Chemical, L3, daily observation


**Product ID:** OCEANCOLOUR_NWS_BGC_HR_L3_NRT_009_203

**Description:** The High-Resolution Ocean Colour (HR-OC) Consortium (Brockmann Consult, Royal Belgian Institute of Natural Sciences, Flemish Institute for Technological Research) distributes Remote Sensing Reflectances (RRS, expressed in sr-1), Turbidity (TUR, expressed in FNU), Solid Particulate Matter Concentration (SPM, expressed in mg/l), spectral particulate backscattering (BBP, expressed in m-1) and chlorophyll-a concentration (CHL, expressed in µg/l) for the Sentinel 2/MSI sensor at 100m resolution for a 20km coastal zone. The products are delivered on a geographic lat-lon grid (EPSG:4326). To limit file size the products are provided in tiles of 600x800 km². RRS and BBP are delivered at nominal central bands of 443, 492, 560, 665, 704, 740, 783, 865 nm. The primary variable from which it is virtually possible to derive all the geophysical and transparency products is the spectral RRS. This, together with the spectral BBP, constitute the category of the 'optics' products. The spectral BBP product is generated from the RRS products using a quasi-analytical algorithm (Lee et al. 2002). The 'transparency' products include TUR and SPM). They are retrieved through the application of automated switching algorithms to the RRS spectra adapted to varying water conditions (Novoa et al. 2017). The GEOPHYSICAL product consists of the Chlorophyll-a concentration (CHL) retrieved via a multi-algorithm approach with optimized quality flagging (O'Reilly et al. 2019, Gons et al. 2005, Lavigne et al. 2021). The NRT products are generally provided withing 24 hours up to 3 days after end of the day.The RRS product is accompanied by a relative uncertainty estimate (unitless) derived by direct comparison of the products to corresponding fiducial reference measurements provided through the AERONET-OC network. The current day data temporal consistency is evaluated as Quality Index (QI) for TUR, SPM and CHL: QI=(CurrentDataPixel-ClimatologyDataPixel)/STDDataPixel where QI is the difference between current data and the relevant climatological field as a signed multiple of climatological standard deviations (STDDataPixel).

**Processing information:**

The HR-OC processing system is deployed on Creodias where Sentinel 2/MSI L1C data are available. The production control element is being hosted within the infrastructure of Brockmann Consult. The processing chain consists of:
* Resampling to 60m and mosaic generation of the set of Sentinel-2 MSI L1C granules of a single overpass that cover a single UTM zone.
* Application of  a glint correction taking into account the detector viewing angles
* Application of a coastal mask with 20km water + 20km land. The result is a L1C mosaic tile with data just in the coastal area optimized for compression.
* Level 2 processing with pixel identification (IdePix), atmospheric correction (C2RCC and ACOLITE or iCOR), in-water processing and merging (HR-OC L2W processor). The result is a 60m product with the same extent as the L1C mosaic, with variables for optics, transparency, and geophysics, and with data filled in the water part of the coastal area.
* invalid pixel identification takes into account  corrupted (L1) pixels, clouds, cloud shadow, glint, dry-fallen intertidal flats, coastal mixed-pixels, sea ice, melting ice, floating vegetation, non-water objects, and bottom reflection.
* Daily L3 aggregation merges all Level 2 mosaics of a day intersecting with a target tile. All valid water pixels are included in the 20km coastal stripes; all other values are set to NaN. There may be more than a single overpass a day, in particular in the northern regions. The main contribution usually is the mosaic of the zone, but also adjacent mosaics may overlap. This step comprises resampling to the 100m target grid. 
* Monthly L4 aggregation combines all Level 3 products of a month and a single tile. The output is a set of 3 NetCDF datasets for optics, transparency, and geophysics respectively, for the tile and month.
* Gap filling combines all daily products of a period and generates (partially) gap-filled daily products again. The output of gap filling are 3 datasets for optics (BBP443 only), transparency, and geophysics per day.

**Description of observation methods/instruments:**

Ocean colour technique exploits the emerging electromagnetic radiation from the sea surface in different wavelengths. The spectral variability of this signal defines the so-called ocean colour which is affected by the presence of phytoplankton.

**Quality / Accuracy / Calibration information:**

A detailed description of the calibration and validation activities performed over this product can be found on the CMEMS web portal and in CMEMS-BGP_HR-QUID-009-201to212.

**Suitability, Expected type of users / uses:**

This product is meant for use for educational purposes and for the managing of the marine safety, marine resources, marine and coastal environment and for climate and seasonal studies.

**Dataset names: **

*cmems_obs_oc_arc_bgc_geophy_nrt_l3-hr_P1D-v01
*cmems_obs_oc_arc_bgc_transp_nrt_l3-hr_P1D-v01
*cmems_obs_oc_arc_bgc_optics_nrt_l3-hr_P1D-v01

**Files format:**
*netCDF-4, CF-1.7
*INSPIRE compliant.

**DOI (product):**
https://doi.org/10.48670/moi-00118

**References:**

* Lavigne, H., et al. (2021), Quality-control tests for OC4, OC5 and NIR-red satellite chlorophyll-a algorithms applied to coastal waters, Remote Sensing of Environment, in press.
* Lee, Z. P., et al. (2002), Deriving inherent optical properties from water color: A multi- band quasi-analytical algorithm for optically deep waters, Applied Optics, 41, 5755-5772.
* Novoa, S., et al. (2017), Atmospheric corrections and multi-conditional algorithm for multi-sensor remote sensing of suspended particulate matter in low-to-high turbidity levels coastal waters. Remote Sens., v. 9, 61.
* Gons, et al. (2005), Effect of a waveband shift on chlorophyll retrieval from MERIS imagery of inland and coastal waters, J. Plankton Res., v. 27, n. 1, p. 125-127.
* O'Reilly, et al. (2019), Chlorophyll algorithms for ocean color sensors-OC4, OC5 & OC6. Remote Sensing of Environment. 229, 32–47.





<img src="https://mdl-metadata.s3.waw3-1.cloudferro.com/metadata/thumbnails/OCEANCOLOUR_NWS_BGC_HR_L4_NRT_009_209.jpg" width="300" height="200"/>



# North West Shelf Region, Bio-Geo-Chemical, L4, monthly means and interpolated daily observation


**Product ID:** OCEANCOLOUR_NWS_BGC_HR_L4_NRT_009_209

**Description:** The High-Resolution Ocean Colour (HR-OC) Consortium (Brockmann Consult, Royal Belgian Institute of Natural Sciences, Flemish Institute for Technological Research) distributes Level 4 (L4) Turbidity (TUR, expressed in FNU), Solid Particulate Matter Concentration (SPM, expressed in mg/l), particulate backscattering at 443nm (BBP443, expressed in m-1) and chlorophyll-a concentration (CHL, expressed in µg/l) for the Sentinel 2/MSI sensor at 100m resolution for a 20km coastal zone. The products are delivered on a geographic lat-lon grid (EPSG:4326). To limit file size the products are provided in tiles of 600x800 km². BBP443, constitute the category of the 'optics' products. The  BBP443 product is generated from the L3 RRS products using a quasi-analytical algorithm (Lee et al. 2002). The 'transparency' products include TUR and SPM). They are retrieved through the application of automated switching algorithms to the RRS spectra adapted to varying water conditions (Novoa et al. 2017). The GEOPHYSICAL product consists of the Chlorophyll-a concentration (CHL) retrieved via a multi-algorithm approach with optimized quality flagging (O'Reilly et al. 2019, Gons et al. 2005, Lavigne et al. 2021). Monthly products (P1M) are temporal aggregates of the daily L3 products. Daily products contain gaps in cloudy areas and where there is no overpass at the respective day. Aggregation collects the non-cloudy (and non-frozen) contributions to each pixel. Contributions are averaged per variable. While this does not guarantee data availability in all pixels in case of persistent clouds, it provides a more complete product compared to the sparsely filled daily products. The Monthly L4 products (P1M) are generally provided withing 4 days after the last acquisition date of the month. Daily gap filled L4 products (P1D) are generated using the DINEOF (Data Interpolating Empirical Orthogonal Functions) approach which reconstructs missing data in geophysical datasets by using a truncated Empirical Orthogonal Functions (EOF) basis in an iterative approach. DINEOF reconstructs missing data in a geophysical dataset by extracting the main patterns of temporal and spatial variability from the data. While originally designed for low resolution data products, recent research has resulted in the optimization of DINEOF to handle high resolution data provided by Sentinel-2 MSI, including cloud shadow detection (Alvera-Azcárate et al., 2021). These types of L4 products are generated and delivered one month after the respective period.

**Processing information:**

The HR-OC processing system is deployed on Creodias where Sentinel 2/MSI L1C data are available. The production control element is being hosted within the infrastructure of Brockmann Consult. The processing chain consists of:
* Resampling to 60m and mosaic generation of the set of Sentinel-2 MSI L1C granules of a single overpass that cover a single UTM zone.
* Application of  a glint correction taking into account the detector viewing angles
* Application of a coastal mask with 20km water + 20km land. The result is a L1C mosaic tile with data just in the coastal area optimized for compression.
* Level 2 processing with pixel identification (IdePix), atmospheric correction (C2RCC and ACOLITE or iCOR), in-water processing and merging (HR-OC L2W processor). The result is a 60m product with the same extent as the L1C mosaic, with variables for optics, transparency, and geophysics, and with data filled in the water part of the coastal area.
* invalid pixel identification takes into account  corrupted (L1) pixels, clouds, cloud shadow, glint, dry-fallen intertidal flats, coastal mixed-pixels, sea ice, melting ice, floating vegetation, non-water objects, and bottom reflection.
* Daily L3 aggregation merges all Level 2 mosaics of a day intersecting with a target tile. All valid water pixels are included in the 20km coastal stripes; all other values are set to NaN. There may be more than a single overpass a day, in particular in the northern regions. The main contribution usually is the mosaic of the zone, but also adjacent mosaics may overlap. This step comprises resampling to the 100m target grid. 
* Monthly L4 aggregation combines all Level 3 products of a month and a single tile. The output is a set of 3 NetCDF datasets for optics, transparency, and geophysics respectively, for the tile and month.
* Gap filling combines all daily products of a period and generates (partially) gap-filled daily products again. The output of gap filling are 3 datasets for optics (BBP443 only), transparency, and geophysics per day.

**Description of observation methods/instruments:**

Ocean colour technique exploits the emerging electromagnetic radiation from the sea surface in different wavelengths. The spectral variability of this signal defines the so-called ocean colour which is affected by the presence of phytoplankton.

**Quality / Accuracy / Calibration information:**

A detailed description of the calibration and validation activities performed over this product can be found on the CMEMS web portal and in CMEMS-BGP_HR-QUID-009-201_to_212.

**Suitability, Expected type of users / uses:**

This product is meant for use for educational purposes and for the managing of the marine safety, marine resources, marine and coastal environment and for climate and seasonal studies.

**Dataset names: **
*cmems_obs_oc_nws_bgc_geophy_nrt_l4-hr_P1M-v01
*cmems_obs_oc_nws_bgc_transp_nrt_l4-hr_P1M-v01
*cmems_obs_oc_nws_bgc_optics_nrt_l4-hr_P1M-v01
*cmems_obs_oc_nws_bgc_geophy_nrt_l4-hr_P1D-v01
*cmems_obs_oc_nws_bgc_transp_nrt_l4-hr_P1D-v01
*cmems_obs_oc_nws_bgc_optics_nrt_l4-hr_P1D-v01

**Files format:**
*netCDF-4, CF-1.7
*INSPIRE compliant.

**DOI (product):**  
https://doi.org/10.48670/moi-00119

**References:**

* Alvera-Azcárate, Aida, et al. (2021), Detection of shadows in high spatial resolution ocean satellite data using DINEOF. Remote Sensing of Environment 253: 112229.
* Lavigne, H., et al. (2021), Quality-control tests for OC4, OC5 and NIR-red satellite chlorophyll-a algorithms applied to coastal waters, Remote Sensing of Environment, in press.
* Lee, Z. P., et al. (2002), Deriving inherent optical properties from water color: A multi- band quasi-analytical algorithm for optically deep waters, Applied Optics, 41, 5755-5772.
* Novoa, S., et al. (2017), Atmospheric corrections and multi-conditional algorithm for multi-sensor remote sensing of suspended particulate matter in low-to-high turbidity levels coastal waters. Remote Sens., v. 9, 61.
* Gons, et al. (2005), Effect of a waveband shift on chlorophyll retrieval from MERIS imagery of inland and coastal waters, J. Plankton Res., v. 27, n. 1, p. 125-127.
* O'Reilly, et al. (2019), Chlorophyll algorithms for ocean color sensors-OC4, OC5 & OC6. Remote Sensing of Environment. 229, 32–47.





<img src="https://documentation.marine.copernicus.eu/IMG/OMI_CIRCULATION_BOUNDARY_BLKSEA_rim_current_index.png" width="300" height="200"/>



# Black Sea Rim Current Intensity Index


**Product ID:** OMI_CIRCULATION_BOUNDARY_BLKSEA_rim_current_index

**Description:** **DEFINITION**

The Black Sea Rim Current index (BSRCI) reflects the intensity of the Rim current, which is a main feature of the Black Sea circulation, a basin scale cyclonic current. The index was computed using sea surface current speed averaged over two areas of intense currents based on reanalysis data. The areas are confined between the 200 and 1800 m isobaths in the northern section 33-39E (from the Caucasus coast to the Crimea Peninsula), and in the southern section 31.5-35E (from Sakarya region to near Sinop Peninsula). Thus, three indices were defined: one for the northern section (BSRCIn), for the southern section (BSRCIs) and an average for the entire basin (BSRCI).
BSRCI=(V ̅_ann-V ̅_cl)/V ̅_cl 
where V ̅ denotes the representative area average, the “ann” denotes the annual mean for each individual year in the analysis, and “cl” indicates the long-term mean over the whole period 1993-2020. In general, BSRCI is defined as the relative annual anomaly from the long-term mean speed.  An index close to zero means close to the average conditions a positive index indicates that the Rim current is more intense than average, or negative - if it is less intense than average. In other words, positive BSRCI would mean higher circumpolar speed, enhanced baroclinicity, enhanced dispersion of pollutants, less degree of exchange between open sea and coastal areas, intensification of the heat redistribution, etc.
The BSRCI is introduced in the fifth issue of the Ocean State Report (von Schuckmann et al., 2021). The Black Sea Physics Reanalysis (BLKSEA_REANALYSIS_PHYS_007_004) has been used as a data base to build the index. Details on the products are delivered in the PUM and QUID of this OMI.

**CONTEXT**

The Black Sea circulation is driven by the regional winds and large freshwater river inflow in the north-western part (including the main European rivers Danube, Dnepr and Dnestr). The major cyclonic gyre encompasses the sea, referred to as Rim current. It is quasi-geostrophic and the Sverdrup balance approximately applies to it. 
The Rim current position and speed experiences significant interannual variability (Stanev and Peneva, 2002), intensifying in winter due to the dominating severe northeastern winds in the region (Stanev et al., 2000). Consequently, this impacts the vertical stratification, Cold Intermediate Water formation, the biological activity distribution and the coastal mesoscale eddies’ propagation along the current and their evolution. The higher circumpolar speed leads to enhanced dispersion of pollutants, less degree of exchange between open sea and coastal areas, enhanced baroclinicity, intensification of the heat redistribution which is important for the winter freezing in the northern zones (Simonov and Altman, 1991). Fach (2015) finds that the anchovy larval dispersal in the Black Sea is strongly controlled at the basin scale by the Rim Current and locally - by mesoscale eddies. 
Several recent studies of the Black Sea pollution claim that the understanding of the Rim Current behavior and how the mesoscale eddies evolve would help to predict the transport of various pollution such as oil spills (Korotenko, 2018) and floating marine litter (Stanev and Ricker, 2019) including microplastic debris (Miladinova et al., 2020) raising a serious environmental concern today. 
To summarize, the intensity of the Black Sea Rim Current could give valuable integral measure for a great deal of physical and biogeochemical processes manifestation. Thus our objective is to develop a comprehensive index reflecting the annual mean state of the Black Sea general circulation to be used by policy makers and various end users. 

**CMEMS KEY FINDINGS**

The Black Sea Rim Current Index is defined as the relative annual anomaly of the long-term mean speed. The BSRCI value characterizes the annual circulation state: a value close to zero would mean close to average conditions, positive value indicates enhanced circulation, and negative value – weaker circulation than usual. The time-series of the BSRCI suggest that the Black Sea Rim current speed varies within ~30% in the period 1993-2020 with a positive trend of ~0.1 m/s/decade. In the years 2005 and 2014 there is evidently higher mean velocity, and on the opposite end are the years –2004, 2013 and 2016. The time series of the BSRCI gives possibility to check the relationship with the wind vorticity and validate the Sverdrup balance hypothesis. 

**Figure caption**

Time series of the Black Sea Rim Current Index (BSRCI) at the north section (BSRCIn), south section (BSRCIs), the average (BSRCI) and its tendency for the period 1993-2020.

**DOI (product):** 
https://doi.org/10.48670/mds-00326

**References:**

* Capet, A., A. Barth, J.-M. Beckers, and M. Grégoire (2012), Interannual variability of Black Sea’s hydrodynamics and connection to atmospheric patterns, Deep-Sea Res. Pt. II, 77 – 80, 128–142, doi:10.1016/j.dsr2.2012.04.010
* Fach, B., (2015), Modeling the Influence of Hydrodynamic Processes on Anchovy Distribution and Connectivity in the Black Sea, Turkish Journal of Fisheries and Aquatic Sciences 14: 1-2, doi: 10.4194/1303-2712-v14_2_06
* Ivanov V.A., Belokopytov V.N. (2013) Oceanography of the Black Sea. Editorial publishing board of Marine Hydrophysical Institute, 210 p, Printed by ECOSY-Gidrofizika, Sevastopol Korotaev, G., T. Oguz, A. Nikiforov, and C. Koblinsky. Seasonal, interannual, and mesoscale variability of the Black Sea upper layer circulation derived from altimeter data. Journal of Geophysical Research (Oceans). 108. C4. doi: 10. 1029/2002JC001508, 2003
* Korotenko KA. Effects of mesoscale eddies on behavior of an oil spill resulting from an accidental deepwater blowout in the Black Sea: an assessment of the environmental impacts. PeerJ. 2018 Aug 29;6:e5448. doi: 10.7717/peerj.5448. PMID: 30186680; PMCID: PMC6119461.
* Kubryakov, A. A., and S.V. Stanichny (2015), Seasonal and interannual variability of the Black Sea eddies and its dependence on characteristics of the large-scale circulation, Deep-Sea Res. Pt. I, 97, 80-91, https://doi.org/10.1016/j.dsr.2014.12.002
* Miladinova S., A. Stips, D. Macias Moy, E. Garcia-Gorriz, (2020a) Pathways and mixing of the north western river waters in the Black Sea Estuarine, Coastal and Shelf Science, Volume 236, 5 May 2020, https://doi





<img src="https://documentation.marine.copernicus.eu/IMG/OMI_CIRCULATION_BOUNDARY_PACIFIC_kuroshio_phase_area_averaged.png" width="300" height="200"/>



# Kuroshio Phase from Observations Reprocessing


**Product ID:** OMI_CIRCULATION_BOUNDARY_PACIFIC_kuroshio_phase_area_averaged

**Description:** **DEFINITION**

The indicator of the Kuroshio extension phase variations is based on the standardized high frequency altimeter Eddy Kinetic Energy (EKE) averaged in the area 142-149°E and 32-37°N and computed from the DUACS (https://duacs.cls.fr) delayed-time (reprocessed version DT-2021, CMEMS SEALEVEL_GLO_PHY_L4_MY_008_047, including “my” (multi-year)  & “myint” (multi-year interim) datasets) and near real-time (CMEMS SEALEVEL_GLO_PHY_L4_NRT _008_046) altimeter sea level gridded products. The change in the reprocessed version (previously DT-2018) and the extension of the mean value of the EKE (now 27 years, previously 20 years) induce some slight changes not impacting the general variability of the Kuroshio extension (correlation coefficient of 0.988 for the total period, 0.994 for the delayed time period only).   

""CONTEXT"" 

The Kuroshio Extension is an eastward-flowing current in the subtropical western North Pacific after the Kuroshio separates from the coast of Japan at 35°N, 140°E. Being the extension of a wind-driven western boundary current, the Kuroshio Extension is characterized by a strong variability and is rich in large-amplitude meanders and energetic eddies (Niiler et al., 2003; Qiu, 2003, 2002). The Kuroshio Extension region has the largest sea surface height variability on sub-annual and decadal time scales in the extratropical North Pacific Ocean (Jayne et al., 2009; Qiu and Chen, 2010, 2005). Prediction and monitoring of the path of the Kuroshio are of huge importance for local economies as the position of the Kuroshio extension strongly determines the regions where phytoplankton and hence fish are located. Unstable (contracted) phase of the Kuroshio enhance the production of Chlorophyll (Lin et al., 2014). 

 

""CMEMS KEY FINDINGS"" 

The different states of the Kuroshio extension phase have been presented and validated by (Bessières et al., 2013) and further reported by Drévillon et al. (2018) in the Copernicus Ocean State Report #2. Two rather different states of the Kuroshio extension are observed: an ‘elongated state’ (also called ‘strong state’) corresponding to a narrow strong steady jet, and a ‘contracted state’ (also called ‘weak state’) in which the jet is weaker and more unsteady, spreading on a wider latitudinal band. When the Kuroshio Extension jet is in a contracted (elongated) state, the upstream Kuroshio Extension path tends to become more (less) variable and regional eddy kinetic energy level tends to be higher (lower). In between these two opposite phases, the Kuroshio extension jet has many intermediate states of transition and presents either progressively weakening or strengthening trends. In 2018, the indicator reveals an elongated state followed by a weakening neutral phase since then. 

**DOI (product):** 
https://doi.org/10.48670/moi-00222

**References:**

* Bessières, L., Rio, M.H., Dufau, C., Boone, C., Pujol, M.I., 2013. Ocean state indicators from MyOcean altimeter products. Ocean Sci. 9, 545–560. https://doi.org/10.5194/os-9-545-2013
* Drévillon, M., Legeais, J.-F., Peterson, A., Zuo, H., Rio, M.-H., Drillet, Y., Greiner, E., 2018. Western boundary currents. J. Oper. Oceanogr., Copernicus Marine Service Ocean State Report Issue 2, s60–s65. https://doi.org/10.1080/1755876X.2018.1489208
* Jayne, S.R., Hogg, N.G., Waterman, S.N., Rainville, L., Donohue, K.A., Randolph Watts, D., Tracey, K.L., McClean, J.L., Maltrud, M.E., Qiu, B., Chen, S., Hacker, P., 2009. The Kuroshio Extension and its recirculation gyres. Deep Sea Res. Part Oceanogr. Res. Pap. 56, 2088–2099. https://doi.org/10.1016/j.dsr.2009.08.006
* Niiler, P.P., Maximenko, N.A., Panteleev, G.G., Yamagata, T., Olson, D.B., 2003. Near-surface dynamical structure of the Kuroshio Extension. J. Geophys. Res. Oceans 108. https://doi.org/10.1029/2002JC001461
* Qiu, B., 2003. Kuroshio Extension Variability and Forcing of the Pacific Decadal Oscillations: Responses and Potential Feedback. J. Phys. Oceanogr. 33, 2465–2482. https://doi.org/10.1175/2459.1
* Qiu, B., 2002. The Kuroshio Extension System: Its Large-Scale Variability and Role in the Midlatitude Ocean-Atmosphere Interaction. J. Oceanogr. 58, 57–75. https://doi.org/10.1023/A:1015824717293
* Qiu, B., Chen, S., 2010. Eddy-mean flow interaction in the decadally modulating Kuroshio Extension system. Deep Sea Res. Part II Top. Stud. Oceanogr., North Pacific Oceanography after WOCE: A Commemoration to Nobuo Suginohara 57, 1098–1110. https://doi.org/10.1016/j.dsr2.2008.11.036
* Qiu, B., Chen, S., 2005. Variability of the Kuroshio Extension Jet, Recirculation Gyre, and Mesoscale Eddies on Decadal Time Scales. J. Phys. Oceanogr. 35, 2090–2103. https://doi.org/10.1175/JPO2807.1
* Lin, P., Chai, F., Xue, H., Xiu, P., 2014. Modulation of decadal oscillation on surface chlorophyll in the Kuroshio Extension. J. Geophys. Res. Oceans 119, 187–199. https://doi.org/10.1002/2013JC009359





<img src="https://documentation.marine.copernicus.eu/IMG/OMI_CIRCULATION_MOC_BLKSEA_area_averaged_mean.png" width="300" height="200"/>



# Black Sea Overturning Circulation Index from Reanalysis


**Product ID:** OMI_CIRCULATION_MOC_BLKSEA_area_averaged_mean

**Description:** **DEFINITION**

This ocean monitoring indicator (OMI) provides a time series of Meridional Overturning Circulation (MOC) Strength in density coordinates, area-averaged and calculated for the period from 1993 to the most recent year with the availability of reanalysis data in the Black Sea (BS). It contains 1D (time dimension) maximum MOC data computed from the Black Sea Reanalysis (BLK-REA; BLKSEA_MULTIYEAR_PHY_007_004) (Ilicak et al., 2022). The MOC is calculated by summing the meridional transport provided by the Copernicus Marine BLK-REA within density bins. The Black Sea MOC indicator represents the maximum MOC value across the basin for a density range between 22.45 and 23.85 kg/m³, which corresponds approximately to a depth interval of 25 to 80 m. To understand the overturning circulation of the Black Sea, we compute the residual meridional overturning circulation in density space. Residual overturning as a function of latitude (y) and density (σ ̅) bins can be computed as follows:
ψ^* (y,σ ̅ )=-1/T ∫_(t_0)^(t_1)▒∫_(x_B1)^(x_B2)▒〖∫_(-H)^0▒H[σ ̅-σ(x,y,z,t)] ×ν(x,y,z,t)dzdxdt,〗
where H is the Heaviside function and ν is the meridional velocity. We used 100 σ_2  (potential density anomaly with reference pressure of 2000 dbar) density bins to remap the mass flux fields.

**CONTEXT**

The BS meridional overturning circulation (BS-MOC) is a clockwise circulation in the northern part up to 150 m connected to cold intermediate layer (CIL) and an anticlockwise circulation in the southern part that could be connected to the influence of the Mediterranean Water inflow into the BS. In contrast to counterparts observed in the deep Atlantic and Mediterranean overturning circulations, the BS-MOC is characterized by shallowness and relatively low strength. However, its significance lies in its capacity to monitor the dynamics and evolution of the CIL which is crucial for the ventilation of the subsurface BS waters. The monitoring of the BS-MOC evolution from the BLK-REA can support the understanding how the CIL formation is affected due to climate change.  The study of Black Sea MOC is relatively new. For more details, see Ilicak et al., (2022).

**KEY FINDINGS**

The MOC values show a significant decline from 1994 to 2009, corresponding to the reduction in the CIL during that period. However, after 2010, the MOC in the Black Sea increased from 0.07 Sv (1 Sv = 106 m3/s) to 0.10 Sv. The CIL has nearly disappeared in recent years, as discussed by Stanev et al. (2019) and Lima et al. (2021) based on observational data and reanalysis results. The opposite pattern observed since 2010 suggests that mechanisms other than the CIL may be influencing the Black Sea MOC.
For the OMI we have used an updated version of the reanalysis (version E4R1) which has a different spinup compared to the OSR6 (version E3R1).

**DOI (product):** 
https://doi.org/10.48670/mds-00349

**References:**

* Ilicak, M., Causio, S., Ciliberti, S., Coppini, G., Lima, L., Aydogdu, A., Azevedo, D., Lecci, R., Cetin, D. U., Masina, S., Peneva, E., Gunduz, M., Pinardi, N. (2022). The Black Sea overturning circulation and its indicator of change. In: Copernicus Ocean State Report, issue 6, Journal of Operational Oceanography, 15:sup1, s64:s71; DOI: doi.org/10.1080/1755876X.2022.2095169
* Lima, L., Ciliberti, S.A., Aydoğdu, A., Masina, S., Escudier, R., Cipollone, A., Azevedo, D., Causio, S., Peneva, E., Lecci, R., Clementi, E., Jansen, E., Ilicak, M., Cretì, S., Stefanizzi, L., Palermo, F., Coppini, G. (2021). Climate Signals in the Black Sea From a Multidecadal Eddy-Resolving Reanalysis. Front. Mar. Sci. 8:710973. doi: 10.3389/fmars.2021.710973
* Stanev, E. V., Peneva, E., Chtirkova, B. (2019). Climate change and regional ocean water mass disappearance: case of the Black Sea. J. Geophys. Res. Oceans 124, 4803–4819. doi: 10.1029/2019JC015076





<img src="https://documentation.marine.copernicus.eu/IMG/OMI_CIRCULATION_MOC_MEDSEA_area_averaged_mean.png" width="300" height="200"/>



# Mediterranean Meridional Overturning Circulation Index from Reanalysis


**Product ID:** OMI_CIRCULATION_MOC_MEDSEA_area_averaged_mean

**Description:** **DEFINITION**

Time mean meridional Eulerian streamfunctions are computed using the velocity field estimate provided by the Copernicus Marine Mediterranean Sea reanalysis over the last 35 years (1987–2021). The Eulerian meridional streamfunction is evaluated by integrating meridional velocity daily data first in a vertical direction, then in a meridional direction, and finally averaging over the reanalysis period.
The Mediterranean overturning indices are derived for the eastern and western Mediterranean Sea by computing the annual streamfunction in the two areas separated by the Strait of Sicily around 36.5°N, and then considering the associated maxima. 
In each case a geographical constraint focused the computation on the main region of interest. For the western index, we focused on deep-water formation regions, thus excluding both the effect of shallow physical processes and the Gibraltar net inflow. For the eastern index, we investigate the Levantine and Cretan areas corresponding to the strongest meridional overturning cell locations, thus only a zonal constraint is defined.
Time series of annual mean values is provided for the Mediterranean Sea using the Mediterranean 1/24o eddy resolving reanalysis (Escudier et al., 2020, 2021).
More details can be found in the Copernicus Marine Ocean State Report issue 4 (OSR4, von Schuckmann et al., 2020) Section 2.4 (Lyubartsev et al., 2020).

**CONTEXT**

The western and eastern Mediterranean clockwise meridional overturning circulation is connected to deep-water formation processes. The Mediterranean Sea 1/24o eddy resolving reanalysis (Escudier et al., 2020, 2021) is used to show the interannual variability of the Meridional Overturning Index. Details on the product are delivered in the PUM and QUID of this OMI. 
The Mediterranean Meridional Overturning Index is defined here as the maxima of the clockwise cells in the eastern and western Mediterranean Sea and is associated with deep and intermediate water mass formation processes that occur in specific areas of the basin: Gulf of Lion, Southern Adriatic Sea, Cretan Sea and Rhodes Gyre (Pinardi et al., 2015).
As in the global ocean, the overturning circulation of the western and eastern Mediterranean are paramount to determine the stratification of the basins (Cessi, 2019). In turn, the stratification and deep water formation mediate the exchange of oxygen and other tracers between the surface and the deep ocean (e.g., Johnson et al., 2009; Yoon et al., 2018). In this sense, the overturning indices are potential gauges of the ecosystem health of the Mediterranean Sea, and in particular they could instruct early warning indices for the Mediterranean Sea to support the Sustainable Development Goal (SDG) 13 Target 13.3.

**CMEMS KEY FINDINGS**

The western and eastern Mediterranean overturning indices (WMOI and EMOI) are synthetic indices of changes in the thermohaline properties of the Mediterranean basin related to changes in the main drivers of the basin scale circulation. The western sub-basin clockwise overturning circulation is associated with the deep-water formation area of the Gulf of Lion, while the eastern clockwise meridional overturning circulation is composed of multiple cells associated with different intermediate and deep-water sources in the Levantine, Aegean, and Adriatic Seas. 
On average, the EMOI shows higher values than the WMOI indicating a more vigorous overturning circulation in eastern Mediterranean. The difference is mostly related to the occurrence of the eastern Mediterranean transient (EMT) climatic event, and linked to a peak of the EMOI in 1992. In 1999, the difference between the two indices started to decrease because EMT water masses reached the Sicily Strait flowing into the western Mediterranean Sea (Schroeder et al., 2016). The western peak in 2006 is discussed to be linked to anomalous deep-water formation during the Western Mediterranean Transition (Smith, 2008; Schroeder et al., 2016). Thus, the WMOI and EMOI indices are a useful tool for long-term climate monitoring of overturning changes in the Mediterranean Sea. 

**Figure caption**

Time series of Mediterranean overturning indices [Sverdrup] calculated from the annual average of the meridional streamfunction over the period 1987 to 2021. Blue: Eastern Mediterranean Overturning Index (lat<36.5°N); Red: Western Mediterranean Overturning Index (lat≥40°N, z>300m). Product used: MEDSEA_MULTIYEAR_PHY_006_004.

**DOI (product):** 
https://doi.org/10.48670/mds-00317

**References:**

* Cessi, P. 2019. The global overturning circulation. Ann Rev Mar Sci. 11:249–270. DOI:10.1146/annurev-marine- 010318-095241. Escudier, R., Clementi, E., Cipollone, A., Pistoia, J., Drudi, M., Grandi, A., Lyubartsev, V., Lecci, R., Aydogdu, A., Delrosso, D., Omar, M., Masina, S., Coppini, G., Pinardi, N. 2021. A High Resolution Reanalysis for the Mediterranean Sea. Frontiers in Earth Science, Vol.9, pp.1060, DOI:10.3389/feart.2021.702285.
* Escudier, R., Clementi, E., Omar, M., Cipollone, A., Pistoia, J., Aydogdu, A., Drudi, M., Grandi, A., Lyubartsev, V., Lecci, R., Cretí, S., Masina, S., Coppini, G., & Pinardi, N. (2020). Mediterranean Sea Physical Reanalysis (CMEMS MED-Currents) (Version 1) set. Copernicus Monitoring Environment Marine Service (CMEMS). https://doi.org/10.25423/CMCC/MEDSEA_MULTIYEAR_PHY_006_004_E3R1
* Gertman, I., Pinardi, N., Popov, Y., Hecht, A. 2006. Aegean Sea water masses during the early stages of the eastern Mediterranean climatic Transient (1988–1990). J Phys Oceanogr. 36(9):1841–1859. DOI:10.1175/JPO2940.1.
* Johnson, K.S., Berelson, W.M., Boss, E.S., Chase, Z., Claustre, H., Emerson, S.R., Gruber, N., Körtzinger, A., Perry, M.J., Riser, S.C. 2009. Observing biogeochemical cycles at global scales with profiling floats and gliders: prospects for a global array. Oceanography. 22:216–225. DOI:10.5670/oceanog. 2009.81.
* Lyubartsev, V., Borile, F., Clementi, E., Masina, S., Drudi, M/. Coppini, G., Cessi, P., Pinardi, N. 2020. Interannual variability in the Eastern and Western Mediterranean Overturning Index. In: Copernicus Marine Service Ocean State Report, Issue 4, Journal of Operational Oceanography, 13:sup1, s88–s91; DOI: 10.1080/1755876X.2020.1785097.
* Pinardi, N., Cessi, P., Borile, F., Wolfe, C.L.P. 2019. The Mediterranean Sea overturning circulation. J Phys Oceanogr. 49:1699–1721. DOI:10.1175/JPO-D-18-0254.1.
* Pinardi, N., Zavatarelli, M., Adani, M., Coppini, G., Fratianni, C., Oddo, P., Tonani, M., Lyubartsev, V., Dobricic, S., Bonaduce, A. 2015. Mediterranean Sea large-scale, low-frequency ocean variability and water mass formation rates from 1987 to 2007: a retrospective analysis. Prog Oceanogr. 132:318–332. DOI:10.1016/j.pocean.2013.11.003.
* Roether, W., Klein, B., Hainbucher, D. 2014. Chap 6. The eastern Mediterranean transient. In: GL Eusebi Borzelli, M Gacic, P Lionello, P Malanotte-Rizzoli, editors. The Mediterranean Sea. American Geophysical Union (AGU); p. 75–83. DOI:10.1002/9781118847572.ch6.
* Roether, W., Manca, B.B., Klein, B., Bregant, D., Georgopoulos, D., Beitzel, V., Kovačević, V., Luchetta, A. 1996. Recent changes in the eastern Mediterranean deep waters. Science. 271:333–335. DOI:10.1126/science.271.5247.333.
* Schroeder, K., Chiggiato, J., Bryden, H., Borghini, M., Ismail, S.B. 2016. Abrupt climate shift in the western Mediterranean Sea. Sci Rep. 6:23009. DOI:10.1038/srep23009.
* Smith, R.O., Bryden, H.L., Stansfield, K. 2008. Observations of new western Mediterranean deep water formation using Argo floats 2004-2006. Ocean Science, 4 (2), 133-149.
* Von Schuckmann, K. et al. 2020. Copernicus Marine Service Ocean State Report, Issue 4, Journal of Operational Oceanography, 13:sup1, S1-S172, DOI: 10.1080/1755876X.2020.1785097.
* Yoon, S., Chang, K., Nam, S., Rho, T.K., Kang, D.J., Lee, T., Park, K.A., Lobanov, V., Kaplunenko, D., Tishchenko, P., Kim, K.R. 2018. Re-initiation of bottom water formation in the East Sea (Japan Sea) in a warming world. Sci Rep. 8:1576. DOI:10. 1038/s41598-018-19952-4.





<img src="https://documentation.marine.copernicus.eu/IMG/OMI_CIRCULATION_VOLTRANS_ARCTIC_averaged.png" width="300" height="200"/>



# Nordic Seas Volume Transports from Reanalysis


**Product ID:** OMI_CIRCULATION_VOLTRANS_ARCTIC_averaged

**Description:** **DEFINITION**

Net (positive minus negative) volume transport of Atlantic Water through the sections (see Figure 1):  Faroe Shetland Channel (Water mass criteria, T > 5 °C); Barents Sea Opening (T > 3 °C) and the Fram Strait (T > 2 °C). Net volume transport of Overflow Waters (σθ >27.8 kg/m3) exiting from the Nordic Seas to the North Atlantic via the Denmark Strait and Faroe Shetland Channel. For further details, see Ch. 3.2 in von Schuckmann et al. (2018).

**CONTEXT**

The poleward flow of relatively warm and saline Atlantic Water through the Nordic Seas to the Arctic Basin, balanced by the overflow waters exiting the Nordic Seas, governs the exchanges between the North Atlantic and the Arctic as well as the distribution of oceanic heat within the Arctic (e.g., Mauritzen et al., 2011; Rudels, 2012). Atlantic Water transported poleward has been found to significantly influence the sea-ice cover in the Barents Sea (Sandø et al., 2010; Årthun et al., 2012; Onarheim et al., 2015) and near Svalbard (Piechura and Walczowski, 2009). Furthermore, Atlantic Water flow through the eastern Nordic seas and its associated heat loss and densification are important factors for the formation of overflow waters in the region (Mauritzen, 1996; Eldevik et al., 2009). These overflow waters together with those generated in the Arctic, exit the Greenland Scotland Ridge, which further contribute to the North Atlantic Deep Water (Dickson and Brown, 1994) and thus play an important role in the Atlantic Meridional Overturning Circulation (Eldevik et al., 2009; Ch. 2.3 in von Schuckmann et al., 2016). In addition to the transport of heat, the Atlantic Water also transports nutrients and zooplankton (e.g., Sundby, 2000), and it carries large amounts of ichthyoplankton of commercially important species, such as Arcto-Norwegian cod (Gadus morhua) and Norwegian spring-spawning herring (Clupea harengus) along the Norwegian coast. The Atlantic Water flow thus plays an integral part in defining both the physical and biological border between the boreal and Arctic realm. Variability of Atlantic Water flow to the Barents Sea has been found to move the position of the ice edge (Onarheim et al., 2015) as well as habitats of various species in the Barents Sea ecosystem (Fossheim et al., 2015).

**CMEMS KEY FINDINGS**

The flow of Atlantic Water through the Færøy-Shetland Channel amounts to 2.7 Sv (Berx et al., 2013). The corresponding model-based estimate was 2.5 Sv for the period 1993-2021. 
In the Barents Sea Opening, the model indicates a long-term average net Atlantic Water inflow of 2.2 Sv, as compared with the long-term estimate from observations of 1.8 Sv (Smedsrud et al., 2013).
In the Fram Strait, the model data indicates a positive trend in the Atlantic Water transport to the Arctic. This trend may be explained by increased temperature in the West Spitsbergen Current during the period 2005-2010 (e.g., Walczowski et al., 2012), which caused a larger fraction of the water mass to be characterized as Atlantic Water (T > 2 °C).

**DOI (product):** 
https://doi.org/10.48670/moi-00189

**References:**

* Berx B. Hansen B, Østerhus S, Larsen KM, Sherwin T, Jochumsen K. 2013. Combining in situ measurements and altimetry to estimate volume, heat and salt transport variability through the Færøy-Shetland Channel. Ocean Sci. 9, 639-654
* Dickson RR, Brown J. 1994. The production of North-Atlantic deep-water – sources, rates, and pathways. J Geophys Res Oceans. 99(C6), 12319-12341
* Eldevik T, Nilsen JEØ, Iovino D, Olsson KA, Sandø AB, Drange H. 2009. Observed sources and variability of Nordic seas overflow. Nature Geosci. 2(6), 405-409
* Fossheim M, Primicerio R, Johannesen E, Ingvaldsen RB, Aschan M.M, Dolgov AV. 2015. Recent warming leads to a rapid borealization of fish communities in the Arctic. Nat Climate Change. 5, 673-678.
* Mauritzen C. 1996. Production of dense overflow waters feeding the North Atlantic across the Greenland-Scotland Ridge. 1. Evidence for a revised circulation scheme. Deep-Sea Res Part I. 43(6), 769-806
* Mauritzen C, Hansen E, Andersson M, Berx B, Beszczynzka-Möller A, Burud I, Christensen KH, Debernard J, de Steur L, Dodd P, et al. 2011. Closing the loop – Approaches to monitoring the state of the Arctic Mediterranean during the International Polar Year 2007-2008. Prog Oceanogr. 90, 62-89
* Onarheim IH, Eldevik T, Årthun M, Ingvaldsen RB, Smedsrud LH. 2015. Skillful prediction of Barents Sea ice cover. Geophys Res Lett. 42(13), 5364-5371
* Raj RP, Johannessen JA, Eldevik T, Nilsen JEØ, Halo I. 2016. Quantifying mesoscale eddies in the Lofoten basin. J Geophys Res Oceans. 121. doi:10.1002/2016JC011637
* Rudels B. 2012. Arctic Ocean circulation and variability – advection and external forcing encounter constraints and local processes. Ocean Sci. 8(2), 261-286
* Sandø, A.B., J.E.Ø. Nilsen, Y. Gao and K. Lohmann, 2010: Importance of heat transport and local air-sea heat fluxes for Barents Sea climate variability. J Geophys Res. 115, C07013
* von Schuckmann K, et al. 2016. The Copernicus Marine Environment Monitoring Service Ocean State Report. J Oper Oceanogr. 9, 235-320
* von Schuckmann K. 2018. Copernicus Marine Service Ocean State Report, J Oper Oceanogr. 11, sup1, S1-S142. Smedsrud LH, Esau I, Ingvaldsen RB, Eldevik T, Haugan PM, Li C, Lien VS, Olsen A, Omar AM, Otterå OH, Risebrobakken B, Sandø AB, Semenov VA, Sorokina SA. 2013. The role of the Barents Sea in the climate system. Rev Geophys. 51, 415-449
* Sundby, S., 2000. Recruitment of Atlantic cod stocks in relation to temperature and advection of copepod populations. Sarsia. 85, 277-298.
* Walczowski W, Piechura J, Goszczko I, Wieczorek P. 2012. Changes in Atlantic water properties: an important factor in the European Arctic marine climate. ICES J Mar Sys. 69(5), 864-869.
* Piechura J, Walczowski W. 2009. Warming of the West Spitsbergen Current and sea ice north of Svalbard. Oceanol. 51(2), 147-164
* Årthun, M., Eldevik, T., Smedsrud, L.H., Skagseth, Ø., Ingvaldsen, R.B., 2012. Quantifying the Influence of Atlantic Heat on Barents Sea Ice Variability and Retreat. J. Climate. 25, 4736-4743.





<img src="https://documentation.marine.copernicus.eu/IMG/OMI_CIRCULATION_VOLTRANS_IBI_section_integrated_anomalies.png" width="300" height="200"/>



# Volume Transport Anomaly in Selected Vertical Sections


**Product ID:** OMI_CIRCULATION_VOLTRANS_IBI_section_integrated_anomalies

**Description:** **DEFINITION**

The product OMI_IBI_CURRENTS_VOLTRANS_section_integrated_anomalies is defined as the time series of annual mean volume transport calculated across a set of vertical ocean sections. These sections have been chosen to be representative of the temporal variability of various ocean currents within the IBI domain.
The currents that are monitored include: transport towards the North Sea through Rockall Trough (RTE) (Holliday et al., 2008; Lozier and Stewart, 2008), Canary Current (CC) (Knoll et al. 2002, Mason et al. 2011), Azores Current (AC) (Mason et al., 2011), Algerian Current (ALG) (Tintoré et al, 1988; Benzohra and Millot, 1995; Font et al., 1998), and net transport along the 48ºN latitude parallel (N48) (see OMI Figure).
To provide ensemble-based results, four Copernicus products have been used. Among these products are three reanalysis products (GLO-REA, IBI-REA and MED-REA) and one product obtained from reprocessed observations (GLO-ARM).
•	GLO-REA: GLOBAL_MULTIYEAR_PHY_001_030 (Reanalysis)
•	IBI-REA: IBI_MULTIYEAR_PHY_005_002 (Reanalysis)
•	MED-REA: MULTIOBS_GLO_PHY_TSUV_3D_MYNRT_015_012 (Reprocessed observations)
•	MED-REA: MEDSEA_MULTIYEAR_PHY_006_004MEDSEA_MULTIYEAR_PHY_006_004 (Reanalysis)
The time series comprises the ensemble mean (blue line), the ensemble spread (grey shaded area), and the mean transport with the sign reversed (red dashed line) to indicate the threshold of anomaly values that would entail a reversal of the current transport. Additionally, the analysis of trends in the time series at the 95% confidence interval is included in the bottom right corner of each diagram.
Details on the product are given in the corresponding Product User Manual (de Pascual-Collar et al., 2024a) and QUality Information Document (de Pascual-Collar et al., 2024b) as well as the CMEMS Ocean State Report: de Pascual-Collar et al., 2024c.

**CONTEXT**

The IBI area is a very complex region characterized by a remarkable variety of ocean currents. Among them, Podemos destacar las que se originan como resultado del closure of the North Atlantic Drift (Mason et al., 2011; Holliday et al., 2008; Peliz et al., 2007; Bower et al., 2002; Knoll et al., 2002; Pérez et al., 2001; Jia, 2000), las corrientes subsuperficiales que fluyen hacia el norte a lo largo del talud continental (de Pascual-Collar et al., 2019; Pascual et al., 2018; Machin et al., 2010; Fricourt et al., 2007; Knoll et al., 2002; Mazé et al., 1997; White & Bowyer, 1997). Y las corrientes de intercambio que se producen en el Estrecho de Gibraltar y el Mar de Alboran (Sotillo et al., 2016; Font et al., 1998; Benzohra and Millot, 1995; Tintoré et al., 1988).
The variability of ocean currents in the IBI domain is relevant to the global thermohaline circulation and other climatic and environmental issues. For example, as discussed by Fasullo and Trenberth (2008), subtropical gyres play a crucial role in the meridional energy balance. The poleward salt transport of Mediterranean water, driven by subsurface slope currents, has significant implications for salinity anomalies in the Rockall Trough and the Nordic Seas, as studied by Holliday (2003), Holliday et al. (2008), and Bozec et al. (2011). The Algerian current serves as the sole pathway for Atlantic Water to reach the Western Mediterranean.

**CMEMS KEY FINDINGS**

The volume transport time series show periods in which the different monitored currents exhibited significantly high or low variability. In this regard, we can mention the periods 1997-1998 and 2014-2015 for the RTE current, the period 2012-2014 in the N48 section, the years 2006 and 2017 for the ALG current, the year 2021 for the AC current, and the period 2009-2012 for the CC current.
Additionally, periods are detected where the anomalies are large enough (in absolute value) to indicate a reversal of the net transport of the current. This is the case for the years 1999, 2003, and 2012-2014 in the N48 section (with a net transport towards the north), the year 2017 in the ALC current (with net transport towards the west), and the year 2010 in the CC current (with net transport towards the north).
The trend analysis of the monitored currents does not detect any significant trends over the analyzed period (1993-2022). However, the confidence interval for the trend in the RTE section is on the verge of rejecting the hypothesis of no trend.

**Figure caption**

Annual anomalies of cross-section volume transport in monitoring sections RTE, N48, AC, ALC, and CC. Time series computed and averaged from different Copernicus Marine products for each window (see section Definition) providing a multi-product result. The blue line represents the ensemble mean, and shaded grey areas represent the standard deviation of the ensemble. Red dashed lines depict the velocity value at which the direction of the current reverses. This aligns with the average transport value (with sign reversed) and the point where absolute transport becomes zero. The analysis of trends (at 95% confidence interval) computed in the period 1993–2021 is included (bottom right box). Trend lines (gray dashed line) are only included in the figures when a significant trend is obtained.

**DOI (product):**
https://doi.org/10.48670/mds-00351

**References:**

* Benzohra, M., Millot, C.: Characteristics and circulation of the surface and intermediate water masses off Algeria. Deep Sea Research Part I: Oceanographic Research Papers, 42(10), 1803-1830, https://doi.org/10.1016/0967-0637(95)00043-6, 1995.
* Bower, A. S., Le Cann, B., Rossby, T., Zenk, T., Gould, J., Speer, K., Richardson, P. L., Prater, M. D., Zhang, H.-M.: Directly measured mid-depth circulation in the northeastern North Atlantic Ocean: Nature, 419, 6907, 603–607, https://doi.org/10.1038/nature01078, 2002.
* Bozec, A., Lozier, M. S., Chasignet, E. P., Halliwel, G. R.: On the variability of the Mediterranean Outflow Water in the North Atlantic from 1948 to 2006, J. Geophys. Res.-Oceans, 116, C09033, https://doi.org/10.1029/2011JC007191, 2011.
* Fasullo, J. T., Trenberth, K. E.: The annual cycle of the energy budget. Part II: Meridional structures and poleward transports. Journal of Climate, 21(10), 2313-2325, https://doi.org/10.1175/2007JCLI1936.1, 2008.
* Font, J., Millot, C., Salas, J., Juliá, A., Chic, O.: The drift of Modified Atlantic Water from the Alboran Sea to the eastern Mediterranean, Scientia Marina, 62-3, https://doi.org/10.3989/scimar.1998.62n3211, 1998.
* Friocourt Y, Levier B, Speich S, Blanke B, Drijfhout SS. A regional numerical ocean model of the circulation in the Bay of Biscay, J. Geophys. Res.,112:C09008, https://doi.org/10.1029/2006JC003935, 2007.
* Font, J., Millot, C., Salas, J., Juliá, A., Chic, O.: The drift of Modified Atlantic Water from the Alboran Sea to the eastern Mediterranean, Scientia Marina, 62-3, https://doi.org/10.3989/scimar.1998.62n3211, 1998.
* Holliday, N. P., Hughes, S. L., Bacon, S., Beszczynska-Möller, A., Hansen, B., Lavín, A., Loeng, H., Mork, K. A., Østerhus, S., Sherwin, T., Walczowski, W.: Reversal of the 1960s to 1990s freshening trend in the northeast North Atlantic and Nordic Seas, Geophys. Res. Lett., 35, L03614, https://doi.org/10.1029/2007GL032675, 2008.
* Holliday, N. P.: Air‐sea interactionand circulation changes in the north- east Atlantic, J. Geophys. Res., 108(C8), 3259, https://doi.org/10.1029/2002JC001344, 2003.
* Jia, Y.: Formation of an Azores Current Due to Mediterranean Overflow in a Modeling Study of the North Atlantic. J. Phys. Oceanogr., 30, 9, 2342–2358, https://doi.org/10.1175/1520-0485(2000)030<2342:FOAACD>2.0.CO;2, 2000.
* Knoll, M., Hernández-Guerra, A., Lenz, B., López Laatzen, F., Machı́n, F., Müller, T. J., Siedler, G.: The Eastern Boundary Current system between the Canary Islands and the African Coast, Deep-Sea Research. 49-17, 3427-3440, https://doi.org/10.1016/S0967-0645(02)00105-4, 2002.
* Lozier, M. S., Stewart, N. M.: On the temporally varying penetration ofMediterranean overflowwaters and eastward penetration ofLabrador Sea Water, J. Phys. Oceanogr., 38,2097–2103, https://doi.org/10.1175/2008JPO3908.1, 2008.
* Machín, F., Pelegrí, J. L., Fraile-Nuez, E., Vélez-Belchí, P., López-Laatzen, F., Hernández-Guerra, A., Seasonal Flow Reversals of Intermediate Waters in the Canary Current System East of the Canary Islands. J. Phys. Oceanogr, 40, 1902–1909, https://doi.org/10.1175/2010JPO4320.1, 2010.
* Mason, E., Colas, F., Molemaker, J., Shchepetkin, A. F., Troupin, C., McWilliams, J. C., Sangra, P.: Seasonal variability of the Canary Current: A numerical study. Journal of Geophysical Research: Oceans, 116(C6), https://doi.org/10.1029/2010JC006665, 2011.
* Mazé, J. P., Arhan, M., Mercier, H., Volume budget of the eastern boundary layer off the Iberian Peninsula, Deep-Sea Research. 1997, 44(9-10), 1543-1574, https://doi.org/10.1016/S0967-0637(97)00038-1, 1997. Mazé, J. P., Arhan, M., Mercier, H., Volume budget of the eastern boundary layer off the Iberian Peninsula, Deep-Sea Research. 1997, 44(9-10), 1543-1574, https://doi.org/10.1016/S0967-0637(97)00038-1, 1997.
* Pascual, A., Levier ,B., Sotillo, M., Verbrugge, N., Aznar, R., Le Cann, B.: Characterization of Mediterranean Outflow Water in the Iberia-Gulf of Biscay-Ireland region. In: von Schuckmann et al. (2018) The Copernicus Marine Environment Monitoring Service Ocean State Report, Journal of Operational Oceanography, 11:sup1, S1-S142, https://doi.org/10.1080/1755876X.2018.1489208, 2018.
* de Pascual-Collar, A., Aznar, R., Levirer, B., Sotillo, M.: EU Copernicus Marine Service Product Quality Information Document for Global Reanalysis Products, OMI_CURRENTS_VOLTRANS_section_integrated_anomalies, Issue 1.0, Mercator Ocean International, https://catalogue.marine.copernicus.eu/documents/QUID/CMEMS-IBI-OMI-QUID-CIRCULATION-VOLTRANS_section_integrated_anomalies.pdf, 2024a.
* de Pascual-Collar, A., Aznar, R., Levirer, B., Sotillo, M.: EU Copernicus Marine Service Product User Manual for OMI_CURRENTS_VOLTRANS_section_integrated_anomalies. Issue 1.0, Mercator Ocean International, https://catalogue.marine.copernicus.eu/documents/PUM/CMEMS-IBI-OMI-PUM-CIRCULATION-VOLTRANS_section_integrated_anomalies.pdf, 2024b.
* de Pascual-Collar, A., Aznar, R., Levier, B., García-Sotillo, M.: Monitoring Main Ocean Currents of the IBI Region, in: 8th edition of the Copernicus Ocean State Report (OSR8), accepted pending of publication, 2004c.
* de Pascual-Collar, A., Sotillo, M. G., Levier, B., Aznar, R., Lorente, P., Amo-Baladrón, A., Álvarez-Fanjul E.: Regional circulation patterns of Mediterranean Outflow Water near the Iberian and African continental slopes. Ocean Sci., 15, 565–582. https://doi.org/10.5194/os-15-565-2019, 2019.
* Peliz, A., Dubert, J., Marchesiello, P., Teles‐Machado, A.: Surface circulation in the Gulf of Cadiz: Model and mean flow structure. Journal of Geophysical Research: Oceans, 112, C11, https://doi.org/10.1029/2007JC004159, 2007.
* Perez, F. F., Castro, C. G., Álvarez-Salgado, X. A., Ríos, A. F.: Coupling between the Iberian basin-scale circulation and the Portugal boundary current system: a chemical study, Deep-Sea Research. I 48,1519 -1533, https://doi.org/10.1016/S0967-0637(00)00101-1, 2001.
* Sotillo, M. G., Amo-Baladrón, A., Padorno, E., Garcia-Ladona, E., Orfila, A., Rodríguez-Rubio, P., Conti, D., Jiménez Madrid, J. A., de los Santos, F. J., Alvarez Fanjul E.: How is the surface Atlantic water inflow through the Gibraltar Strait forecasted? A lagrangian validation of operational oceanographic services in the Alboran Sea and the Western Mediterranean, Deep-Sea Research. 133, 100-117, https://doi.org/10.1016/j.dsr2.2016.05.020, 2016.
* Tintore, J., La Violette, P. E., Blade, I., Cruzado, A.: A study of an intense density front in the eastern Alboran Sea: the Almeria–Oran front. Journal of Physical Oceanography, 18, 10, 1384-1397, https://doi.org/10.1175/1520-0485(1988)018%3C1384:ASOAID%3E2.0.CO;2, 1988.
* White, M., Bowyer, P.: The shelf-edge current north-west of Ireland. Annales Geophysicae 15, 1076–1083. https://doi.org/10.1007/s00585-997-1076-0, 1997.





<img src="https://documentation.marine.copernicus.eu/IMG/OMI_CLIMATE_OFC_BALTIC_area_averaged_anomalies.png" width="300" height="200"/>



# Baltic Sea Ocean Freshwater Content Anomaly (0-300m) from Reanalysis


**Product ID:** OMI_CLIMATE_OFC_BALTIC_area_averaged_anomalies

**Description:** **DEFINITION**

The Ocean Freshwater Content (OFC) is calculated according to Boyer et al. (2007)
OFC =    ρ(Sref, Tref, p) / ρ(0, Tref, p ) · ( Sref - S) / Sref
where S(x, y, z, t) and Sref (x, y, z) are actual salinity and reference salinity, respectively, and x,y,z,t are zonal, meridional, vertical and temporal coordinates, respectively. The density, ρ, is calculated according to the TEOS10 (IOC et al., 2010). The key issue of OFC calculations lies in how the reference salinity is defined. The climatological range of salinity in the Baltic Sea varies from the freshwater conditions in the northern and eastern parts to the oceanic water conditions in the Kattegat. We follow the Boyer et al. (2007) formulation and calculate the climatological OFC from the three-dimensional temperature (Tref) and salinity (Sref) fields averaged over the period of 1993–2014.
The method for calculating the ocean freshwater content anomaly is based on the daily mean sea water salinity fields (S) derived from the Baltic Sea reanalysis product BALTICSEA_MULTIYEAR_PHY_003_011. The total freshwater content anomaly is determined using the following formula:
OFC(t) = ∭V OFC(x, y, z, t) dx dy dz
The vertical integral is computed using the static cell vertical thicknesses (dz) sourced from the reanalysis product BALTICSEA_MULTIYEAR_PHY_003_011 dataset cmems_mod_bal_phy_my_static, spanning from the sea surface to the 300 m depth. Spatial integration is performed over the Baltic Sea spatial domain, defined as the region between 9° - 31° E and 53° - 66° N using product grid definition in cmems_mod_bal_phy_my_static. 
We evaluate the uncertainty from the mean standard deviation of monthly mean OFC. The shaded area in the figure corresponds to the annual standard deviation of monthly mean OFC. 
Linear trend (km3y-1) has been estimated from the annual anomalies with the uncertainty of 1.96-times standard error.

**CONTEXT**
Climate warming has resulted in the intensification of the global hydrological cycle but not necessarily on the regional scale (Pratap and Markonis, 2022). The increase of net precipitation over land and sea areas, decrease of ice cover, and increase of river runoff are the main components of the global hydrological cycle that increase freshwater content in the ocean (Boyer et al., 2007) and decrease ocean salinity.
The Baltic Sea is one of the marginal seas where water salinity and OFC are strongly influenced by the water exchange with the North Sea. The Major Baltic Inflows (MBIs) are the most voluminous event-type sources of saline water to the Baltic Sea (Mohrholz, 2018). The frequency and intensity of the MBIs and other large volume inflows have no long-term trends but do have a multidecadal variability of about 30 years (Mohrholz, 2018; Lehmann and Post, 2015; Lehmann et al., 2017; Radtke et al., 2020). Smaller barotropic and baroclinically driven inflows transport saline water into the halocline or below it, depending on the density of the inflow water (Reissmann et al., 2009). 

**KEY FINDINGS**

The Baltic Sea's ocean freshwater content is exhibiting a declining trend of -37±8.8 km³/year, along with decadal fluctuations as also noted by Lehmann et al. (2022). Elevated freshwater levels were recorded prior to the Major Baltic Inflows of 1993, 2002, and 2013, which subsequently led to a swift decrease in freshwater content. The lowest ocean freshwater content was recorded in 2019. Over the past four years, the freshwater content anomaly has remained comparatively stable. 

**DOI (product):** 
https://doi.org/10.48670/mds-00347

**References:**

* Boyer, T., Levitus, S., Antonov, J., Locarnini, R., Mishonov, A., Garcia, H., Josey, S.A., 2007. Changes in freshwater content in the North Atlantic Ocean 1955–2006. Geophysical Research Letters, 34(16), L16603. Doi: 10.1029/2007GL030126
* IOC, SCOR and IAPSO, 2010: The international thermodynamic equation of seawater - 2010: Calculation and use of thermodynamic properties. Intergovernmental Oceanographic Commission, Manuals and Guides No. 56, UNESCO (English), 196 pp. Available from http://www.TEOS-10.org (11.10.2021).
* Lehmann, A., Post, P., 2015. Variability of atmospheric circulation patterns associated with large volume changes of the Baltic Sea. Advances in Science and Research, 12, 219–225, doi:10.5194/asr-12-219-2015
* Lehmann, A., Höflich, K., Post, P., Myrberg, K., 2017. Pathways of deep cyclones associated with large volume changes (LVCs) and major Baltic inflows (MBIs). Journal of Marine Systems, 167, pp.11-18. doi:10.1016/j.jmarsys.2016.10.014
* Lehmann, A., Myrberg, K., Post, P., Chubarenko, I., Dailidiene, I., Hinrichsen, H.-H., Hüssy, K., Liblik, T., Meier, H. E. M., Lips, U., Bukanova, T., 2022. Salinity dynamics of the Baltic Sea. Earth System Dynamics, 13(1), pp 373 - 392. doi:10.5194/esd-13-373-2022
* Mohrholz, V., 2018. Major Baltic inflow statistics–revised. Frontiers in Marine Science, 5, p.384. doi:10.3389/fmars.2018.00384
* Pratap, S., Markonis, Y., 2022. The response of the hydrological cycle to temperature changes in recent and distant climatic history, Progress in Earth and Planetary Science 9(1),30. doi:10.1186/s40645-022-00489-0
* Radtke, H., Brunnabend, S.-E., Gräwe, U., Meier, H. E. M., 2020. Investigating interdecadal salinity changes in the Baltic Sea in a 1850–2008 hindcast simulation, Climate of the Past, 16, 1617–1642, doi:10.5194/cp-16-1617-2020
* Reissmann, J. H., Burchard, H., Feistel,R., Hagen, E., Lass, H. U., Mohrholz, V., Nausch, G., Umlauf, L., Wiecczorek, G., 2009. Vertical mixing in the Baltic Sea and consequences for eutrophication a review, Progress in Oceanography, 82, 47–80. doi:10.1016/j.pocean.2007.10.004





<img src="https://documentation.marine.copernicus.eu/IMG/OMI_CLIMATE_OHC_BLKSEA_area_averaged_anomalies.png" width="300" height="200"/>



# Black Sea Ocean Heat Content Anomaly (0-300m) time series and trend from Reanalysis & Multi-Observations Reprocessing


**Product ID:** OMI_CLIMATE_OHC_BLKSEA_area_averaged_anomalies

**Description:** **DEFINITION**

Ocean heat content (OHC) is defined here as the deviation from a reference period (1993-2014) and is closely proportional to the average temperature change from z1 = 0 m to z2 = 300 m depth:
OHC=∫_(z_1)^(z_2)ρ_0  c_p (T_m-T_clim )dz [1]
with a reference density = 1020 kg m-3 and a specific heat capacity of cp = 3980 J kg-1 °C-1 (e.g. von Schuckmann et al., 2009; Lima et al., 2020); T_m corresponds to the monthly average temperature and T_clim is the climatological temperature of the corresponding month that varies according to each individual product.
Time series of monthly mean values area averaged ocean heat content is provided for the Black Sea (40.86°N, 46.8°N; 27.32°E, 41.96°E) and is evaluated in areas where the topography is deeper than 300m. The Azov and Marmara Seas are not considered.
The quality evaluation of OMI_CLIMATE_OHC_BLKSEA_area_averaged_anomalies is based on the “multi-product” approach as introduced in the second issue of the Ocean State Report (von Schuckmann et al., 2018), and following the MyOcean’s experience (Masina et al., 2017). Three global products and one regional (Black Sea) product have been used to build an ensemble mean, and its associated ensemble spread. Details on the products are delivered in the PUM and QUID of this OMI.

**CONTEXT**

Knowing how much and where heat energy is stored and released in the ocean is essential for understanding the contemporary Earth system state, variability and change, as the oceans shape our perspectives for the future.
Several studies discuss a warming in the Black Sea using either observations or model results (Akpinar et al., 2017; Stanev et al. 2019; Lima et al. 2020). Using satellite sea surface temperature observations (SST), Degtyarev (2000) detected a positive temperature trend of 0.016 ºC years-1 in the 50-100 m layer from 1985 to 1997. From Argo floats Stanev et al. (2019) found a warming trend in the cold intermediate layer (CIL; at approximately 25 – 70 m) of about 0.05 oC year-1 in recent years. The warming signal was also present in ocean heat content analyses conducted by Lima et al. (2020). Their results from the Black Sea regional reanalysis showed an increase rate of 0.880±0.181 W m-2 in the upper layers (0 – 200 m), which has been reflected in the disappearance of Black Sea cold intermediate layer in recent years. The newest version of reanalysis also presents a warming of 0.814±0.045 W m-2 in 0 – 200 m (Lima et al. (2021). This warming has been reflected in a more incidence of marine heat waves in the Black Sea over the past few years (Mohammed et al. 2022).

**CMEMS KEY FINDINGS**

Time series of ocean heat content anomalies present a significant interannual variability, altering between cool and warm events. This important characteristic becomes evident over the years 2012 to 2015: a minimum of ocean heat content anomaly is registered close to – 2.00 x 108 J m-2 in 2012, followed by positive values around 2.00 x 108 J m-2 in 2013 and above 2.0 x 108 J m-2 most of time in 2014 and 2015. Since 2005 the Black Sea experienced an increase in ocean heat content (0-300 m), and record OHC values are noticed in 2020. The Black Sea is warming at a rate of 0.995±0.084 W m-2, which is higher than the global average warming rate.
The increase in ocean heat content weakens the CIL, whereas its decreasing favours the CIL restoration (Akpinar et al., 2017). The years 2012 and 2017 exhibited a more evident warming interruption that induced a replenishment of the CIL (Lima et al. 2021).

**Figure caption**

Time series of the ensemble mean and ensemble spread (shaded area) of the monthly Black Sea averaged ocean heat content anomalies integrated over the 0-300m depth layer (J m–2) during Jan 2005 – December 2020. The monthly ocean heat content anomalies are defined as the deviation from the climatological ocean heat content mean (1993–2014) of each corresponding month. Mean trend values are also reported at the bottom right corner. The ensemble is based on different data products, i.e. Black Sea Reanalysis, global ocean reanalysis GLORYS12V1; global observational based products CORA5.2, ARMOR3D. Details on the products are given in the corresponding PUM and QUID for this OMI.

**DOI (product):** 
 https://doi.org/10.48670/moi-00306

**References:**

* Akpinar, A., Fach, B. A., Oguz, T., 2017: Observing the subsurface thermal signature of the Black Sea cold intermediate layer with Argo profiling floats. Deep Sea Res. I Oceanogr. Res. Papers 124, 140–152. doi: 10.1016/j.dsr.2017.04.002.
* Lima, L., Peneva, E., Ciliberti, S., Masina, S., Lemieux, B., Storto, A., Chtirkova, B., 2020: Ocean heat content in the Black Sea. In: Copernicus marine service Ocean State Report, issue 4, Journal of Operational Oceanography, 13:Sup1, s41–s47, doi: 10.1080/1755876X.2020.1785097.
* Lima L., Ciliberti S. A., Aydoğdu A., Masina S., Escudier R., Cipollone A., Azevedo D., Causio S., Peneva E., Lecci R., Clementi E., Jansen E., Ilicak M., Cretì S., Stefanizzi L., Palermo F., Coppini G., 2021: Climate Signals in the Black Sea From a Multidecadal Eddy-Resolving Reanalysis, Frontier in Marine Science, 8:710973, doi: 10.3389/fmars.2021.710973.
* Masina S., A. Storto, N. Ferry, M. Valdivieso, K. Haines, M. Balmaseda, H. Zuo, M. Drevillon, L. Parent, 2017: An ensemble of eddy-permitting global ocean reanalyses from the MyOcean project. Climate Dynamics, 49 (3): 813-841, DOI: 10.1007/s00382-015-2728-5.
* Stanev, E. V., Peneva, E., and Chtirkova, B. 2019: Climate change and regional ocean water mass disappearance: case of the Black Sea. J. Geophys. Res. Oceans, 124, 4803–4819, doi: 10.1029/2019JC015076.
* von Schuckmann, K., F. Gaillard and P.-Y. Le Traon, 2009: Global hydrographic variability patterns during 2003-2008, Journal of Geophysical Research, 114, C09007, doi:10.1029/2008JC005237.
* von Schuckmann et al., 2016: Ocean heat content. In: The Copernicus Marine Environment Monitoring Service Ocean State Report, issue 1, Journal of Operational Oceanography, Volume 9, 2016 - Issue sup2: The Copernicus Marine Environment Monitoring Service Ocean, http://dx.doi.org/10.1080/1755876X.2016.1273446.
* von Schuckmann et al., 2018: Ocean heat content. In: The Copernicus Marine Environment Monitoring Service Ocean State Report, issue 2, Journal of Operational Oceanography, 11:Sup1, s1-s142, doi: 10.1080/1755876X.2018.1489208.
* Degtyarev, A. K., 2000: Estimation of temperature increase of the Black Sea active layer during the period 1985– 1997, Meteorilogiya i Gidrologiya, 6, 72– 76 (in Russian).





<img src="https://documentation.marine.copernicus.eu/IMG/OMI_CLIMATE_OHC_IBI_area_averaged_anomalies.png" width="300" height="200"/>



# Iberia Biscay Ireland Ocean Heat Content Anomaly (0-2000m) time series and trend from Reanalysis & Multi-Observations Reprocessing


**Product ID:** OMI_CLIMATE_OHC_IBI_area_averaged_anomalies

**Description:** **DEFINITION**

Ocean heat content (OHC) is defined here as the deviation from a reference period (1993-20210) and is closely proportional to the average temperature change from z1 = 0 m to z2 = 2000 m depth:
 
 With a reference density of ρ0 = 1030 kgm-3 and a specific heat capacity of cp = 3980 J/kg°C (e.g. von Schuckmann et al., 2009)
Averaged time series for ocean heat content and their error bars are calculated for the Iberia-Biscay-Ireland region (26°N, 56°N; 19°W, 5°E).
This OMI is computed using IBI-MYP, GLO-MYP reanalysis and CORA, ARMOR data from observations which provide temperatures. Where the CMEMS product for each acronym is:
•	IBI-MYP: IBI_MULTIYEAR_PHY_005_002 (Reanalysis)
•	GLO-MYP: GLOBAL_REANALYSIS_PHY_001_031 (Reanalysis)
•	CORA: INSITU_GLO_TS_OA_REP_OBSERVATIONS_013_002_b (Observations)
•	ARMOR: MULTIOBS_GLO_PHY_TSUV_3D_MYNRT_015_012 (Reprocessed observations)
The figure comprises ensemble mean (blue line) and the ensemble spread (grey shaded). Details on the product are given in the corresponding PUM for this OMI as well as the CMEMS Ocean State Report: von Schuckmann et al., 2016; von Schuckmann et al., 2018.

**CONTEXT**

Change in OHC is a key player in ocean-atmosphere interactions and sea level change (WCRP, 2018) and can impact marine ecosystems and human livelihoods (IPCC, 2019). Additionally, OHC is one of the six Global Climate Indicators recommended by the World Meterological Organisation (WMO, 2017). 
In the last decades, the upper North Atlantic Ocean experienced a reversal of climatic trends for temperature and salinity. While the period 1990-2004 is characterized by decadal-scale ocean warming, the period 2005-2014 shows a substantial cooling and freshening. Such variations are discussed to be linked to ocean internal dynamics, and air-sea interactions (Fox-Kemper et al., 2021; Collins et al., 2019; Robson et al 2016). Together with changes linked to the connectivity between the North Atlantic Ocean and the Mediterranean Sea (Masina et al., 2022), these variations affect the temporal evolution of regional ocean heat content in the IBI region.
Recent studies (de Pascual-Collar et al., 2023) highlight the key role that subsurface water masses play in the OHC trends in the IBI region. These studies conclude that the vertically integrated trend is the result of different trends (both positive and negative) contributing at different layers. Therefore, the lack of representativeness of the OHC trends in the surface-intermediate waters (from 0 to 1000 m) causes the trends in intermediate and deep waters (from 1000 m to 2000 m) to be masked when they are calculated by integrating the upper layers of the ocean (from surface down to 2000 m).

**CMEMS KEY FINDINGS**

The ensemble mean OHC anomaly time series over the Iberia-Biscay-Ireland region are dominated by strong year-to-year variations, and an ocean warming trend of 0.41±0.4 W/m2 is barely significant.

**Figure caption**

Time series of annual mean area averaged ocean heat content in the Iberia-Biscay-Ireland region (basin wide) and integrated over the 0-2000m depth layer during 1993-2022: ensemble mean (blue line) and ensemble spread (shaded area). The ensemble mean is based on different data products i.e., the IBI Reanalysis, global ocean reanalysis, and the global observational based products CORA, and ARMOR3D. Trend of ensemble mean (dashed line and bottom-right box) with 95% confidence interval computed in the period 1993-2022. Details on the products are given in the corresponding PUM and QUID for this OMI.

**DOI (product):** 
https://doi.org/10.48670/mds-00316

**References:**

* Collins M., M. Sutherland, L. Bouwer, S.-M. Cheong, T. Frölicher, H. Jacot Des Combes, M. Koll Roxy, I. Losada, K. McInnes, B. Ratter, E. Rivera-Arriaga, R.D. Susanto, D. Swingedouw, and L. Tibig, 2019: Extremes, Abrupt Changes and Managing Risk. In: IPCC Special Report on the Ocean and Cryosphere in a Changing Climate [H.-O. Pörtner, D.C. Roberts, V. Masson-Delmotte, P. Zhai, M. Tignor, E. Poloczanska, K. Mintenbeck, A. Alegría, M. Nicolai, A. Okem, J. Petzold, B. Rama, N.M. Weyer (eds.)]. Cambridge University Press, Cambridge, UK and New York, NY, USA, pp. 589–655. https://doi.org/10.1017/9781009157964.008.
* Fox-Kemper, B., H.T. Hewitt, C. Xiao, G. Aðalgeirsdóttir, S.S. Drijfhout, T.L. Edwards, N.R. Golledge, M. Hemer, R.E. Kopp, G. Krinner, A. Mix, D. Notz, S. Nowicki, I.S. Nurhati, L. Ruiz, J.-B. Sallée, A.B.A. Slangen, and Y. Yu, 2021: Ocean, Cryosphere and Sea Level Change. In Climate Change 2021: The Physical Science Basis. Contribution of Working Group I to the Sixth Assessment Report of the Intergovernmental Panel on Climate Change [Masson-Delmotte, V., P. Zhai, A. Pirani, S.L. Connors, C. Péan, S. Berger, N. Caud, Y. Chen, L. Goldfarb, M.I. Gomis, M. Huang, K. Leitzell, E. Lonnoy, J.B.R. Matthews, T.K. Maycock, T. Waterfield, O. Yelekçi, R. Yu, and B. Zhou (eds.)]. Cambridge University Press, Cambridge, United Kingdom and New York, NY, USA, pp. 1211–1362, doi: 10.1017/9781009157896.011.
* IPCC Special Report on the Ocean and Cryosphere in a Changing Climate. (2019). In H. O. Pörtner, D. C. Roberts, V. Masson-Delmotte, P. Zhai, M. Tignor, E. Poloczanska, K. Mintenbeck, A. Alegría, M. Nicolai, A. Okem, J. Petzold, B. Rama, & N. M. Weyer (Eds.), IPCC Intergovernmental Panel on Climate Change: Geneva, Switzerland. https://www.ipcc.ch/srocc/
* Masina, S., Pinardi, N., Cipollone, A., Banerjee, D. S., Lyubartsev, V., von Schuckmann, K., Jackson, L., Escudier, R., Clementi, E., Aydogdu, A. and Iovino D., (2022). The Atlantic Meridional Overturning Circulation forcing the mean se level in the Mediterranean Sea through the Gibraltar transport. In: Copernicus Ocean State Report, Issue 6, Journal of Operational Oceanography,15:sup1, s119–s126; DOI: 10.1080/1755876X.2022.2095169
* Potter, R. A., and Lozier, M. S. 2004: On the warming and salinification of the Mediterranean outflow waters in the North Atlantic, Geophys. Res. Lett., 31, 1–4, doi:10.1029/2003GL018161.
* Robson, J., Ortega, P., Sutton, R., 2016: A reversal of climatic trends in the North Atlantic since 2005. Nature Geosci 9, 513–517. https://doi.org/10.1038/ngeo2727.
* von Schuckmann, K., F. Gaillard and P.-Y. Le Traon, 2009: Global hydrographic variability patterns during 2003-2008, Journal of Geophysical Research, 114, C09007, doi:10.1029/2008JC005237.
* von Schuckmann et al., 2016: The Copernicus Marine Environment Monitoring Service Ocean State Report, Journal of Operational Oceanography, Volume 9, 2016 - Issue sup2: The Copernicus Marine Environment Monitoring Service Ocean, http://dx.doi.org/10.1080/1755876X.2016.1273446.
* von Schuckmann, K., Le Traon, P.-Y., Smith, N., Pascual, A., Brasseur, P., Fennel, K., Djavidnia, S., Aaboe, S., Fanjul, E. A., Autret, E., Axell, L., Aznar, R., Benincasa, M., Bentamy, A., Boberg, F., Bourdallé-Badie, R., Nardelli, B. B., Brando, V. E., Bricaud, C., … Zuo, H. (2018). Copernicus Marine Service Ocean State Report. Journal of Operational Oceanography, 11(sup1), S1–S142. https://doi.org/10.1080/1755876X.2018.1489208
* WCRP (2018). Global sea-level budget 1993–present. Earth Syst. Sci. Data, 10(3), 1551–1590. https://doi.org/10.5194/essd-10-1551-2018
* WMO, 2017: World Meterological Organisation Bulletin, 66(2), https://public.wmo.int/en/resources/bulletin.





<img src="https://documentation.marine.copernicus.eu/IMG/OMI_CLIMATE_OSC_MEDSEA_volume_mean.png" width="300" height="200"/>



# Mediterranean Ocean Salt Content (0-300m)


**Product ID:** OMI_CLIMATE_OSC_MEDSEA_volume_mean

**Description:** **DEFINITION**

Ocean salt content (OSC) is defined and represented here as the volume average of the integral of salinity in the Mediterranean Sea from z1 = 0 m to z2 = 300 m depth:
¯S=1/V ∫V S dV
Time series of annual mean values area averaged ocean salt content are provided for the Mediterranean Sea (30°N, 46°N; 6°W, 36°E) and are evaluated in the upper 300m excluding the shelf areas close to the coast with a depth less than 300 m. The total estimated volume is approximately 5.7e+5 km3.

**CONTEXT**

The freshwater input from the land (river runoff) and atmosphere (precipitation) and inflow from the Black Sea and the Atlantic Ocean are balanced by the evaporation in the Mediterranean Sea. Evolution of the salt content may have an impact in the ocean circulation and dynamics which possibly will have implication on the entire Earth climate system. Thus monitoring changes in the salinity content is essential considering its link  to changes in: the hydrological cycle, the water masses formation, the regional halosteric sea level and salt/freshwater transport, as well as for their impact on marine biodiversity.
The OMI_CLIMATE_OSC_MEDSEA_volume_mean is based on the “multi-product” approach introduced in the seventh issue of the Ocean State Report (contribution by Aydogdu et al., 2023). Note that the estimates in Aydogdu et al. (2023) are provided monthly while here we evaluate the results per year.
Six global products and a regional (Mediterranean Sea) product have been used to build an ensemble mean, and its associated ensemble spread. The reference products are:
	The Mediterranean Sea Reanalysis at 1/24°horizontal resolution (MEDSEA_MULTIYEAR_PHY_006_004, DOI: https://doi.org/10.25423/CMCC/MEDSEA_MULTIYEAR_PHY_006_004_E3R1, Escudier et al., 2020)
	Four global reanalyses at 1/4°horizontal resolution (GLOBAL_REANALYSIS_PHY_001_031, 
GLORYS, C-GLORS, ORAS5, FOAM, DOI: https://doi.org/10.48670/moi-00024, Desportes et al., 2022)
	Two observation-based products: 
CORA (INSITU_GLO_TS_REP_OBSERVATIONS_013_001_b, DOI:  https://doi.org/10.17882/46219, Szekely et al., 2022) and 
ARMOR3D (MULTIOBS_GLO_PHY_TSUV_3D_MYNRT_015_012, DOI: https://doi.org/10.48670/moi-00052, Grenier et al., 2021). 
Details on the products are delivered in the PUM and QUID of this OMI. 

**CMEMS KEY FINDINGS**

The Mediterranean Sea salt content shows a positive trend in the upper 300 m with a continuous increase over the period 1993-2019 at rate of 5.6*10-3 ±3.5*10-4 psu yr-1. 
The overall ensemble mean of different products is 38.57 psu. During the early 1990s in the entire Mediterranean Sea there is a large spread in salinity with the observational based datasets showing a higher salinity, while the reanalysis products present relatively lower salinity. The maximum spread between the period 1993–2019 occurs in the 1990s with a value of 0.12 psu, and it decreases to as low as 0.02 psu by the end of the 2010s.

**Figure caption**

Time series of annual mean volume ocean salt content in the Mediterranean Sea (basin wide), integrated over the 0-300m depth layer during 1993-2019 (or longer according to data availability) including ensemble mean and ensemble spread (shaded area). The ensemble mean and associated ensemble spread are based on different data products, i.e., Mediterranean Sea Reanalysis (MED-REA), global ocean reanalysis (GLORYS, C-GLORS, ORAS5, and FOAM) and global observational based products (CORA and ARMOR3D). Details on the products are given in the corresponding PUM and QUID for this OMI.

**DOI (product):** 
https://doi.org/10.48670/mds-00325

**References:**

* Aydogdu, A., Miraglio, P., Escudier, R., Clementi, E., Masina, S.: The dynamical role of upper layer salinity in the Mediterranean Sea, State of the Planet, accepted, 2023.
* Desportes, C., Garric, G., Régnier, C., Drévillon, M., Parent, L., Drillet, Y., Masina, S., Storto, A., Mirouze, I., Cipollone, A., Zuo, H., Balmaseda, M., Peterson, D., Wood, R., Jackson, L., Mulet, S., Grenier, E., and Gounou, A.: EU Copernicus Marine Service Quality Information Document for the Global Ocean Ensemble Physics Reanalysis, GLOBAL_REANALYSIS_PHY_001_031, Issue 1.1, Mercator Ocean International, https://documentation.marine.copernicus.eu/QUID/CMEMS-GLO-QUID-001-031.pdf (last access: 3 May 2023), 2022.
* Escudier, R., Clementi, E., Omar, M., Cipollone, A., Pistoia, J., Aydogdu, A., Drudi, M., Grandi, A., Lyubartsev, V., Lecci, R., Cretí, S., Masina, S., Coppini, G., & Pinardi, N. (2020).
* Mediterranean Sea Physical Reanalysis (CMEMS MED-Currents) (Version 1) [Data set]. Copernicus Monitoring Environment Marine Service (CMEMS). https://doi.org/10.25423/CMCC/MEDSEA_MULTIYEAR_PHY_006_004_E3R1
* Grenier, E., Verbrugge, N., Mulet, S., and Guinehut, S.: EU Copernicus Marine Service Quality Information Document for the Multi Observation Global Ocean 3D Temperature Salinity Height Geostrophic Current and MLD, MULTIOBS_GLO_PHY_TSUV_3D_MYNRT_015_012, Issue 1.1, Mercator Ocean International, https://documentation.marine.copernicus.eu/QUID/CMEMS-MOB-QUID-015-012.pdf (last access: 3 May 2023), 2021.
* Szekely, T.: EU Copernicus Marine Service Quality Information Document for the Global Ocean-Delayed Mode gridded CORA – In-situ Observations objective analysis in Delayed Mode, INSITU_GLO_PHY_TS_OA_MY_013_052, issue 1.2, Mercator Ocean International, https://documentation.marine.copernicus.eu/QUID/CMEMS-INS-QUID-013-052.pdf (last access: 4 April 2023), 2022.





<img src="https://documentation.marine.copernicus.eu/IMG/OMI_CLIMATE_SL_BALTIC_area_averaged_anomalies.png" width="300" height="200"/>



# Baltic Sea Mean Sea Level time series and trend from Observations Reprocessing


**Product ID:** OMI_CLIMATE_SL_BALTIC_area_averaged_anomalies

**Description:** **DEFINITION**

The sea level ocean monitoring indicator has been presented in the Copernicus Ocean State Report #8.  The sea level ocean monitoring indicator is derived from the DUACS delayed-time (DT-2024 version, “my” (multi-year) dataset used when available, “myint” (multi-year interim) used after) sea level anomaly maps from satellite altimetry based on a stable number of altimeters (two) in the satellite constellation. These products are distributed by the Copernicus Climate Change Service and the Copernicus Marine Service (SEALEVEL_GLO_PHY_CLIMATE_L4_MY_008_057). 

The time series of area averaged anomalies correspond to the area average of the maps in the Baltic Sea weighted by the cosine of the latitude (to consider the changing area in each grid with latitude) and by the proportion of ocean in each grid (to consider the coastal areas). The time series are corrected from regional mean GIA correction (weighted GIA mean of a 27 ensembles model following Spada et Melini, 2019). The time series are adjusted for seasonal annual and semi-annual signals and low-pass filtered at 6 months. Then, the trends/accelerations are estimated on the time series using ordinary least square fit. 

The trend uncertainty is provided in a 90% confidence interval. It is calculated as the weighted mean uncertainties in the region from Prandi et al., 2021. This estimate only considers errors related to the altimeter observation system (i.e., orbit determination errors, geophysical correction errors and inter-mission bias correction errors). The presence of the interannual signal can strongly influence the trend estimation considering to the altimeter period considered (Wang et al., 2021; Cazenave et al., 2014). The uncertainty linked to this effect is not considered. 

**CONTEXT**

Change in mean sea level is an essential indicator of our evolving climate, as it reflects both the thermal expansion of the ocean in response to its warming and the increase in ocean mass due to the melting of ice sheets and glaciers (WCRP Global Sea Level Budget Group, 2018). At regional scale, sea level does not change homogenously. It is influenced by various other processes, with different spatial and temporal scales, such as local ocean dynamic, atmospheric forcing, Earth gravity and vertical land motion changes (IPCC WGI, 2021). The adverse effects of floods, storms and tropical cyclones, and the resulting losses and damage, have increased as a result of rising sea levels, increasing people and infrastructure vulnerability and food security risks, particularly in low-lying areas and island states (IPCC, 2022a). Adaptation and mitigation measures such as the restoration of mangroves and coastal wetlands, reduce the risks from sea level rise (IPCC, 2022b).  

The Baltic Sea is a relatively small semi-enclosed basin with shallow bathymetry. Different forcings have been discussed to trigger sea level variations in the Baltic Sea at different time scales. In addition to steric effects, decadal and longer sea level variability in the basin can be induced by sea water exchange with the North Sea, and in response to atmospheric forcing and climate variability (e.g., the North Atlantic Oscillation; Gräwe et al., 2019). 

**KEY FINDINGS**

Over the [1999/02/21 to 2023/12/31] period, the area-averaged sea level in the Baltic Sea rises at a rate of 4.5 ± 0.8 mm/year  with an acceleration of 0.10 ± 0.07 mm/year2. This trend estimation is based on the altimeter measurements corrected from regional GIA correction (Spada et Melini, 2019) to consider the ongoing movement of land. The TOPEX-A is no longer included in the computation of regional mean sea level parameters (trend and acceleration) with version 2024 products due to potential drifts, and ongoing work aims to develop a new empirical correction. Calculation begins in February 1999 (the start of the TOPEX-B period). 

**DOI (product):** 
https://doi.org/10.48670/moi-00202

**References:**

* Cazenave, A., Dieng, H.-B., Meyssignac, B., von Schuckmann, K., Decharme, B., and Berthier, E.: The rate of sea-level rise, Nat. Clim. Change, 4, 358–361, https://doi.org/10.1038/nclimate2159, 2014.
* Gräwe, U., Klingbeil, K., Kelln, J., and Dangendorf, S.: Decomposing Mean Sea Level Rise in a Semi-Enclosed Basin, the Baltic Sea, J. Clim., 32, 3089–3108, https://doi.org/10.1175/JCLI-D-18-0174.1, 2019.
* IPCC: Summary for Policymakers [H.-O. Pörtner, D.C. Roberts, E.S. Poloczanska, K. Mintenbeck, M. Tignor, A. Alegría, M. Craig, S. Langsdorf, S. Löschke, V. Möller, A. Okem (eds.)]. In: Climate Change 2022: Impacts, Adaptation, and Vulnerability. Contribution of Working Group II to the Sixth Assessment Report of the Intergovernmental Panel on Climate Change [H.-O. Pörtner, D.C. Roberts, M. Tignor, E.S. Poloczanska, K. Mintenbeck, A. Alegría, M. Craig, S. Langsdorf, S. Löschke, V. Möller, A. Okem, B. Rama (eds.)], 2022a.
* IPCC: Summary for Policymakers. In: Climate Change 2022: Mitigation of Climate Change. Contribution of Working Group III to the Sixth Assessment Report of the Intergovernmental Panel on Climate Change [P.R. Shukla, J. Skea, R. Slade, A. Al Khourdajie, R. van Diemen, D. McCollum, M. Pathak, S. Some, P. Vyas, R. Fradera, M. Belkacemi, A. Hasija, G. Lisboa, S. Luz, J. Malley, (eds.)], , https://doi.org/10.1017/9781009157926.001, 2022b.
* IPCC WGI: Climate Change 2021: The Physical Science Basis. Contribution of Working Group I to the Sixth Assessment Report of the Intergovernmental Panel on Climate Change, 2021.
* Prandi, P., Meyssignac, B., Ablain, M., Spada, G., Ribes, A., and Benveniste, J.: Local sea level trends, accelerations and uncertainties over 1993–2019, Sci. Data, 8, 1, https://doi.org/10.1038/s41597-020-00786-7, 2021.
* Spada, G. and Melini, D.: SELEN4 (SELEN version 4.0): a Fortran program for solving the gravitationally and topographically self-consistent sea-level equation in glacial isostatic adjustment modeling, Geosci. Model Dev., 12, 5055–5075, https://doi.org/10.5194/gmd-12-5055-2019, 2019.
* Von Schuckmann et al., “The State of the Global Ocean, Issue 8.”
* Wang, J., Church, J. A., Zhang, X., and Chen, X.: Reconciling global mean and regional sea level change in projections and observations, Nat. Commun., 12, 990, https://doi.org/10.1038/s41467-021-21265-6, 2021.
* WCRP Global Sea Level Budget Group: Global sea-level budget 1993–present, Earth Syst. Sci. Data, 10, 1551–1590, https://doi.org/10.5194/essd-10-1551-2018, 2018.





<img src="https://documentation.marine.copernicus.eu/IMG/OMI_CLIMATE_SL_BLKSEA_area_averaged_anomalies.png" width="300" height="200"/>



# Black Sea Mean Sea Level time series and trend from Observations Reprocessing


**Product ID:** OMI_CLIMATE_SL_BLKSEA_area_averaged_anomalies

**Description:** **DEFINITION**

The sea level ocean monitoring indicator has been presented in the Copernicus Ocean State Report #8. The ocean monitoring indicator on mean sea level is derived from the DUACS delayed-time (DT-2024 version, “my” (multi-year) dataset used when available, “myint” (multi-year interim) used after) sea level anomaly maps from satellite altimetry based on a stable number of altimeters (two) in the satellite constellation. These products are distributed by the Copernicus Climate Change Service and the Copernicus Marine Service (SEALEVEL_GLO_PHY_CLIMATE_L4_MY_008_057). 

The time series of area averaged anomalies correspond to the area average of the maps in the Black Sea weighted by the cosine of the latitude (to consider the changing area in each grid with latitude) and by the proportion of ocean in each grid (to consider the coastal areas). The time series are corrected from regional mean GIA correction (weighted GIA mean of a 27 ensemble model following Spada et Melini, 2019). The time series are adjusted for seasonal annual and semi-annual signals and low-pass filtered at 6 months. Then, the trends/accelerations are estimated on the time series using ordinary least square fit.The trend uncertainty is provided in a 90% confidence interval. It is calculated as the weighted mean uncertainties in the region from Prandi et al., 2021. This estimate only considers errors related to the altimeter observation system (i.e., orbit determination errors, geophysical correction errors and inter-mission bias correction errors). The presence of the interannual signal can strongly influence the trend estimation considering to the altimeter period considered (Wang et al., 2021; Cazenave et al., 2014). The uncertainty linked to this effect is not considered. 

**CONTEXT**

Change in mean sea level is an essential indicator of our evolving climate, as it reflects both the thermal expansion of the ocean in response to its warming and the increase in ocean mass due to the melting of ice sheets and glaciers (WCRP Global Sea Level Budget Group, 2018). At regional scale, sea level does not change homogenously. It is influenced by various other processes, with different spatial and temporal scales, such as local ocean dynamic, atmospheric forcing, Earth gravity and vertical land motion changes (IPCC WGI, 2021). The adverse effects of floods, storms and tropical cyclones, and the resulting losses and damage, have increased as a result of rising sea levels, increasing people and infrastructure vulnerability and food security risks, particularly in low-lying areas and island states (IPCC, 2022b). Adaptation and mitigation measures such as the restoration of mangroves and coastal wetlands, reduce the risks from sea level rise (IPCC, 2022c).  

In the Black Sea, major drivers of change have been attributed to  anthropogenic climate change (steric expansion), and mass changes induced by various water exchanges with the Mediterranean Sea, river discharge, and precipitation/evaporation changes (e.g. Volkov and Landerer, 2015). The sea level variation in the basin also shows an important interannual variability, with an increase observed before 1999 predominantly linked to steric effects, and comparable lower values afterward (Vigo et al., 2005). 

**KEY FINDINGS**

Over the [1999/02/21 to 2023/12/31] period, the area-averaged sea level in the Black Sea rises at a rate of 0.5 ± 0.8 mm/yr with an acceleration of 0.12 ± 0.06 mm/yr². Nevertheless, the area-averaged sea level anomaly is dominated by interannual variability and the trend estimate is not statistically significant at 95% confidence level. This trend estimation is based on the altimeter measurements corrected from the regional GIA correction (Spada et Melini, 2019) to consider the ongoing movement of land. The TOPEX-A is no longer included in the computation of regional mean sea level parameters (trend and acceleration) with version 2024 products due to potential drifts, and ongoing work aims to develop a new empirical correction. Calculation begins in February 1999 (the start of the TOPEX-B period). 

**DOI (product):** 
https://doi.org/10.48670/moi-00215

**References:**

* Cazenave, A., Dieng, H.-B., Meyssignac, B., von Schuckmann, K., Decharme, B., and Berthier, E.: The rate of sea-level rise, Nat. Clim. Change, 4, 358–361, https://doi.org/10.1038/nclimate2159, 2014.
* IPCC: Summary for Policymakers [H.-O. Pörtner, D.C. Roberts, E.S. Poloczanska, K. Mintenbeck, M. Tignor, A. Alegría, M. Craig, S. Langsdorf, S. Löschke, V. Möller, A. Okem (eds.)]. In: Climate Change 2022: Impacts, Adaptation, and Vulnerability. Contribution of Working Group II to the Sixth Assessment Report of the Intergovernmental Panel on Climate Change [H.-O. Pörtner, D.C. Roberts, M. Tignor, E.S. Poloczanska, K. Mintenbeck, A. Alegría, M. Craig, S. Langsdorf, S. Löschke, V. Möller, A. Okem, B. Rama (eds.)], 2022b.
* IPCC: Summary for Policymakers. In: Climate Change 2022: Mitigation of Climate Change. Contribution of Working Group III to the Sixth Assessment Report of the Intergovernmental Panel on Climate Change [P.R. Shukla, J. Skea, R. Slade, A. Al Khourdajie, R. van Diemen, D. McCollum, M. Pathak, S. Some, P. Vyas, R. Fradera, M. Belkacemi, A. Hasija, G. Lisboa, S. Luz, J. Malley, (eds.)], , https://doi.org/10.1017/9781009157926.001, 2022c.
* IPCC WGI: Climate Change 2021: The Physical Science Basis. Contribution of Working Group I to the Sixth Assessment Report of the Intergovernmental Panel on Climate Change, 2021.
* Prandi, P., Meyssignac, B., Ablain, M., Spada, G., Ribes, A., and Benveniste, J.: Local sea level trends, accelerations and uncertainties over 1993–2019, Sci. Data, 8, 1, https://doi.org/10.1038/s41597-020-00786-7, 2021.
* Spada, G. and Melini, D.: SELEN4 (SELEN version 4.0): a Fortran program for solving the gravitationally and topographically self-consistent sea-level equation in glacial isostatic adjustment modeling, Geosci. Model Dev., 12, 5055–5075, https://doi.org/10.5194/gmd-12-5055-2019, 2019.
* Vigo, I., Garcia, D., and Chao, B. F.: Change of sea level trend in the Mediterranean and Black seas, J. Mar. Res., 63, 1085–1100, https://doi.org/10.1357/002224005775247607, 2005.
* Volkov, D. L. and Landerer, F. W.: Internal and external forcing of sea level variability in the Black Sea, Clim. Dyn., 45, 2633–2646, https://doi.org/10.1007/s00382-015-2498-0, 2015.
* Von Schuckmann et al., “The State of the Global Ocean, Issue 8.”
* Wang, J., Church, J. A., Zhang, X., and Chen, X.: Reconciling global mean and regional sea level change in projections and observations, Nat. Commun., 12, 990, https://doi.org/10.1038/s41467-021-21265-6, 2021.
* WCRP Global Sea Level Budget Group: Global sea-level budget 1993–present, Earth Syst. Sci. Data, 10, 1551–1590, https://doi.org/10.5194/essd-10-1551-2018, 2018.





<img src="https://documentation.marine.copernicus.eu/IMG/OMI_CLIMATE_SL_EUROPE_area_averaged_anomalies.png" width="300" height="200"/>



# European Seas Mean Sea Level time series and trend from Observations Reprocessing


**Product ID:** OMI_CLIMATE_SL_EUROPE_area_averaged_anomalies

**Description:** **DEFINITION**

The sea level ocean monitoring indicator has been presented in the Copernicus Ocean State Report #8. The ocean monitoring indicator on mean sea level is derived from the DUACS delayed-time (DT-2024 version, “my” (multi-year) dataset used when available, “myint” (multi-year interim) used after) sea level anomaly maps from satellite altimetry based on a stable number of altimeters (two) in the satellite constellation. These products are distributed by the Copernicus Climate Change Service and by the Copernicus Marine Service (SEALEVEL_GLO_PHY_CLIMATE_L4_MY_008_057). 

The time series of area averaged anomalies correspond to the area average of the maps in the Northeast Atlantic Ocean and adjacent  seas Sea weighted by the cosine of the latitude (to consider the changing area in each grid with latitude) and by the proportion of ocean in each grid (to consider the coastal areas). The time series are corrected from regional mean GIA correction (weighted GIA mean of a 27 ensemble model following Spada et Melini, 2019). The time series are adjusted for seasonal annual and semi-annual signals and low-pass filtered at 6 months. Then, the trends/accelerations are estimated on the time series using ordinary least square fit. 

Uncertainty is provided in a 90% confidence interval. It is calculated as the weighted mean uncertainties in the region from Prandi et al., 2021. This estimate only considers errors related to the altimeter observation system (i.e., orbit determination errors, geophysical correction errors and inter-mission bias correction errors). The presence of the interannual signal can strongly influence the trend estimation depending on the period considered (Wang et al., 2021; Cazenave et al., 2014). The uncertainty linked to this effect is not considered. 

""CONTEXT ""

Change in mean sea level is an essential indicator of our evolving climate, as it reflects both the thermal expansion of the ocean in response to its warming and the increase in ocean mass due to the melting of ice sheets and glaciers (WCRP Global Sea Level Budget Group, 2018). At regional scale, sea level does not change homogenously. It is influenced by various other processes, with different spatial and temporal scales, such as local ocean dynamic, atmospheric forcing, Earth gravity and vertical land motion changes (IPCC WGI, 2021). The adverse effects of floods, storms and tropical cyclones, and the resulting losses and damage, have increased as a result of rising sea levels, increasing people and infrastructure vulnerability and food security risks, particularly in low-lying areas and island states (IPCC, 2022a). Adaptation and mitigation measures such as the restoration of mangroves and coastal wetlands, reduce the risks from sea level rise (IPCC, 2022b).  

In this region, sea level variations are influenced by the North Atlantic Oscillation (NAO) (e.g. Delworth and Zeng, 2016) and the Atlantic Meridional Overturning Circulation (AMOC) (e.g. Chafik et al., 2019). Hermans et al., 2020 also reported the dominant influence of wind on interannual sea level variability in a large part of this area. This region encompasses the Mediterranean, IBI, North-West shelf, Black Sea and Baltic regions with different sea level dynamics detailed in the regional indicators. 

""KEY FINDINGS"" 

Over the [1999/02/21 to 2023/12/31] period, the area-averaged sea level in the Northeast Atlantic Ocean and adjacent seas area rises at a rate of 3.6 ± 0.8 mm/yr with an acceleration of 0.24 ± 0.06 mm/yr². This trend estimation is based on the altimeter measurements corrected from regional GIA correction (Spada et Melini, 2019) to consider the ongoing movement of land. The TOPEX-A is no longer included in the computation of regional mean sea level parameters (trend and acceleration) with version 2024 products due to potential drifts, and ongoing work aims to develop a new empirical correction. Calculation begins in February 1999 (the start of the TOPEX-B period). 

**DOI (product):** 
https://doi.org/10.48670/mds-00335

**References:**

* Cazenave, A., Dieng, H.-B., Meyssignac, B., von Schuckmann, K., Decharme, B., and Berthier, E.: The rate of sea-level rise, Nat. Clim. Change, 4, 358–361, https://doi.org/10.1038/nclimate2159, 2014.
* Chafik, L., Nilsen, J. E. Ø., Dangendorf, S., Reverdin, G., and Frederikse, T.: North Atlantic Ocean Circulation and Decadal Sea Level Change During the Altimetry Era, Sci. Rep., 9, 1041, https://doi.org/10.1038/s41598-018-37603-6, 2019.
* Delworth, T. L. and Zeng, F.: The Impact of the North Atlantic Oscillation on Climate through Its Influence on the Atlantic Meridional Overturning Circulation, J. Clim., 29, 941–962, https://doi.org/10.1175/JCLI-D-15-0396.1, 2016.
* Hermans, T. H. J., Le Bars, D., Katsman, C. A., Camargo, C. M. L., Gerkema, T., Calafat, F. M., Tinker, J., and Slangen, A. B. A.: Drivers of Interannual Sea Level Variability on the Northwestern European Shelf, J. Geophys. Res. Oceans, 125, e2020JC016325, https://doi.org/10.1029/2020JC016325, 2020.
* IPCC: Summary for Policymakers [H.-O. Pörtner, D.C. Roberts, E.S. Poloczanska, K. Mintenbeck, M. Tignor, A. Alegría, M. Craig, S. Langsdorf, S. Löschke, V. Möller, A. Okem (eds.)]. In: Climate Change 2022: Impacts, Adaptation, and Vulnerability. Contribution of Working Group II to the Sixth Assessment Report of the Intergovernmental Panel on Climate Change [H.-O. Pörtner, D.C. Roberts, M. Tignor, E.S. Poloczanska, K. Mintenbeck, A. Alegría, M. Craig, S. Langsdorf, S. Löschke, V. Möller, A. Okem, B. Rama (eds.)], 2022a.
* IPCC: Summary for Policymakers. In: Climate Change 2022: Mitigation of Climate Change. Contribution of Working Group III to the Sixth Assessment Report of the Intergovernmental Panel on Climate Change [P.R. Shukla, J. Skea, R. Slade, A. Al Khourdajie, R. van Diemen, D. McCollum, M. Pathak, S. Some, P. Vyas, R. Fradera, M. Belkacemi, A. Hasija, G. Lisboa, S. Luz, J. Malley, (eds.)], , https://doi.org/10.1017/9781009157926.001, 2022b.
* IPCC WGI: Climate Change 2021: The Physical Science Basis. Contribution of Working Group I to the Sixth Assessment Report of the Intergovernmental Panel on Climate Change, 2021.
* Prandi, P., Meyssignac, B., Ablain, M., Spada, G., Ribes, A., and Benveniste, J.: Local sea level trends, accelerations and uncertainties over 1993–2019, Sci. Data, 8, 1, https://doi.org/10.1038/s41597-020-00786-7, 2021.
* Spada, G. and Melini, D.: SELEN4 (SELEN version 4.0): a Fortran program for solving the gravitationally and topographically self-consistent sea-level equation in glacial isostatic adjustment modeling, Geosci. Model Dev., 12, 5055–5075, https://doi.org/10.5194/gmd-12-5055-2019, 2019.
* Von Schuckmann et al., “The State of the Global Ocean, Issue 8.
* Wang, J., Church, J. A., Zhang, X., and Chen, X.: Reconciling global mean and regional sea level change in projections and observations, Nat. Commun., 12, 990, https://doi.org/10.1038/s41467-021-21265-6, 2021.





<img src="https://documentation.marine.copernicus.eu/IMG/OMI_CLIMATE_SL_GLOBAL_area_averaged_anomalies.png" width="300" height="200"/>



# Global Ocean Mean Sea Level time series and trend from Observations Reprocessing


**Product ID:** OMI_CLIMATE_SL_GLOBAL_area_averaged_anomalies

**Description:** **DEFINITION**

The ocean monitoring indicator on mean sea level has been presented in the Copernicus Ocean State Report #8. The ocean monitoring indicator on mean sea level is derived from the DUACS delayed-time (DT-2024 version, “my” (multi-year) dataset used when available, “myint” (multi-year interim) used after) sea level anomaly maps from satellite altimetry based on a stable number of altimeters (two) in the satellite constellation. These products are distributed by the Copernicus Climate Change Service and by the Copernicus Marine Service (SEALEVEL_GLO_PHY_CLIMATE_L4_MY_008_057). 

The time series of area averaged anomalies correspond to the area average of the maps in the Global Ocean weighted by the cosine of the latitude (to consider the changing area in each grid with latitude) and by the proportion of ocean in each grid (to consider the coastal areas). The time series are corrected from global GIA correction of -0.3mm/yr (common global GIA correction, see Spada, 2017). The time series are adjusted for seasonal annual and semi-annual signals and low-pass filtered at 6 months. Then, the trends/accelerations are estimated on the time series using ordinary least square fit. 

The trend uncertainty of 0.3 mm/yr is provided at 90% confidence level using altimeter error budget (Quet et al 2024 [in prep.]). This estimate only considers errors related to the altimeter observation system (i.e., orbit determination errors, geophysical correction errors and inter-mission bias correction errors). The presence of the interannual signal can strongly influence the trend estimation depending on the period considered (Wang et al., 2021; Cazenave et al., 2014). The uncertainty linked to this effect is not considered.  

""CONTEXT"" 

Change in mean sea level is an essential indicator of our evolving climate, as it reflects both the thermal expansion of the ocean in response to its warming and the increase in ocean mass due to the melting of ice sheets and glaciers(WCRP Global Sea Level Budget Group, 2018). According to the recent IPCC 6th assessment report (IPCC WGI, 2021), global mean sea level (GMSL) increased by 0.20 [0.15 to 0.25] m over the period 1901 to 2018 with a rate of rise that has accelerated since the 1960s to 3.7 [3.2 to 4.2] mm/yr for the period 2006–2018. Human activity was very likely the main driver of observed GMSL rise since 1970 (IPCC WGII, 2021). The weight of the different contributions evolves with time and in the recent decades the mass change has increased, contributing to the on-going acceleration of the GMSL trend (IPCC, 2022a; Legeais et al., 2020; Horwath et al., 2022). The adverse effects of floods, storms and tropical cyclones, and the resulting losses and damage, have increased as a result of rising sea levels, increasing people and infrastructure vulnerability and food security risks, particularly in low-lying areas and island states (IPCC, 2022b). Adaptation and mitigation measures such as the restoration of mangroves and coastal wetlands, reduce the risks from sea level rise (IPCC, 2022c). 

""KEY FINDINGS ""

Over the [1999/02/21, 2023/12/31] period, global mean sea level  rises at an average rate of 3.6  0.3 mm/year. This trend estimation is based on the altimeter measurements corrected from the global GIA correction (Spada, 2017) to consider the ongoing movement of land. The TOPEX-A is no longer included in the computation of regional mean sea level parameters (trend and acceleration) with version 2024 products due to potential drifts, and ongoing work aims to develop a new empirical correction. Calculation begins in February 1999 (the start of the TOPEX-B period).  

The observed global trend agrees with other recent estimates (Oppenheimer et al., 2019; IPCC WGI, 2021). About 30% of this rise can be attributed to ocean thermal expansion (WCRP Global Sea Level Budget Group, 2018; von Schuckmann et al., 2018), 60% is due to land ice melt from glaciers and from the Antarctic and Greenland ice sheets. The remaining 10% is attributed to changes in land water storage, such as soil moisture, surface water and groundwater. From year to year, the global mean sea level record shows significant variations related mainly to the El Niño Southern Oscillation (Cazenave and Cozannet, 2014). 
**DOI (product):** 
https://doi.org/10.48670/moi-00237

**References:**

* Cazenave, A. and Cozannet, G. L.: Sea level rise and its coastal impacts, Earths Future, 2, 15–34, https://doi.org/10.1002/2013EF000188, 2014.
* Cazenave, A., Dieng, H.-B., Meyssignac, B., von Schuckmann, K., Decharme, B., and Berthier, E.: The rate of sea-level rise, Nat. Clim. Change, 4, 358–361, https://doi.org/10.1038/nclimate2159, 2014.
* Horwath, M., Gutknecht, B. D., Cazenave, A., Palanisamy, H. K., Marti, F., Marzeion, B., Paul, F., Le Bris, R., Hogg, A. E., Otosaka, I., Shepherd, A., Döll, P., Cáceres, D., Müller Schmied, H., Johannessen, J. A., Nilsen, J. E. Ø., Raj, R. P., Forsberg, R., Sandberg Sørensen, L., Barletta, V. R., Simonsen, S. B., Knudsen, P., Andersen, O. B., Ranndal, H., Rose, S. K., Merchant, C. J., Macintosh, C. R., von Schuckmann, K., Novotny, K., Groh, A., Restano, M., and Benveniste, J.: Global sea-level budget and ocean-mass budget, with a focus on advanced data products and uncertainty characterisation, Earth Syst. Sci. Data, 14, 411–447, https://doi.org/10.5194/essd-14-411-2022, 2022.
* IPCC: AR6 Synthesis Report: Climate Change 2022, 2022a.
* IPCC: Summary for Policymakers [H.-O. Pörtner, D.C. Roberts, E.S. Poloczanska, K. Mintenbeck, M. Tignor, A. Alegría, M. Craig, S. Langsdorf, S. Löschke, V. Möller, A. Okem (eds.)]. In: Climate Change 2022: Impacts, Adaptation, and Vulnerability. Contribution of Working Group II to the Sixth Assessment Report of the Intergovernmental Panel on Climate Change [H.-O. Pörtner, D.C. Roberts, M. Tignor, E.S. Poloczanska, K. Mintenbeck, A. Alegría, M. Craig, S. Langsdorf, S. Löschke, V. Möller, A. Okem, B. Rama (eds.)], 2022b.
* IPCC: Summary for Policymakers. In: Climate Change 2022: Mitigation of Climate Change. Contribution of Working Group III to the Sixth Assessment Report of the Intergovernmental Panel on Climate Change [P.R. Shukla, J. Skea, R. Slade, A. Al Khourdajie, R. van Diemen, D. McCollum, M. Pathak, S. Some, P. Vyas, R. Fradera, M. Belkacemi, A. Hasija, G. Lisboa, S. Luz, J. Malley, (eds.)], , https://doi.org/10.1017/9781009157926.001, 2022c.
* IPCC WGII: Climate Change 2021: Impacts, Adaptation and Vulnerability; Summary for Policemakers. Contribution of Working Group II to the Sixth Assessment Report of the Intergovernmental Panel on Climate Change, 2021.
* Legeais, J. F., Llowel, W., Melet, A., and Meyssignac, B.: Evidence of the TOPEX-A altimeter instrumental anomaly and acceleration of the global mean sea level, Copernic. Mar. Serv. Ocean State Rep. Issue 4, 13, s77–s82, https://doi.org/10.1080/1755876X.2021.1946240, 2020.
* Oppenheimer, M., Glavovic, B. C., Hinkel, J., Van de Wal, R., Magnan, A. K., Abd-Elgaward, A., Cai, R., Cifuentes Jara, M., DeConto, R. M., Ghosh, T., Hay, J., Isla, F., Marzeion, B., Meyssignac, B., and Sebesvari, Z.: Sea Level Rise and Implications for Low-Lying Islands, Coasts and Communities — Special Report on the Ocean and Cryosphere in a Changing Climate: Chapter 4, 2019.
* Quet V, Prandi P, Meyssignac B, Octau F, Jussiau E, Mangilli A, Dibarboure G, Bignalet-Cazalet F, Ablain M, Long term assessment of the Global Mean Sea Level record and associated uncertainties based on new L2P DT 2024 products (in prep)
* Spada, G. (2017). Glacial Isostatic Adjustment and Contemporary Sea Level Rise: An Overview. In: Cazenave, A., Champollion, N., Paul, F., Benveniste, J. (eds) Integrative Study of the Mean Sea Level and Its Components. Space Sciences Series of ISSI, vol 58. Springer, Cham. https://doi.org/10.1007/978-3-319-56490-6_8
* Von Schuckmann et al., “The State of the Global Ocean, Issue 8.”
* Wang, J., Church, J. A., Zhang, X., and Chen, X.: Reconciling global mean and regional sea level change in projections and observations, Nat. Commun., 12, 990, https://doi.org/10.1038/s41467-021-21265-6, 2021.
* WCRP Global Sea Level Budget Group: Global sea-level budget 1993–present, Earth Syst. Sci. Data, 10, 1551–1590, https://doi.org/10.5194/essd-10-1551-2018, 2018."





<img src="https://documentation.marine.copernicus.eu/IMG/OMI_CLIMATE_SL_GLOBAL_regional_trends.png" width="300" height="200"/>



# Global Ocean Mean Sea Level trend map from Observations Reprocessing


**Product ID:** OMI_CLIMATE_SL_GLOBAL_regional_trends

**Description:** **DEFINITION**

The sea level ocean monitoring indicator has been presented in the Copernicus Ocean State Report #8. The sea level ocean monitoring indicator is derived from the DUACS delayed-time (DT-2024 version, “my” (multi-year) dataset used when available, “myint” (multi-year interim) used after) sea level anomaly maps from satellite altimetry based on a stable number of altimeters (two) in the satellite constellation. The product is distributed by the Copernicus Climate Change Service and the Copernicus Marine Service (SEALEVEL_GLO_PHY_CLIMATE_L4_MY_008_057). At each grid point, the trends/accelerations are estimated on the time series corrected from regional GIA correction (GIA map of a 27 ensemble model following Spada et Melini, 2019) and adjusted from annual and semi-annual signals. Regional uncertainties on the trends estimates can be found in Prandi et al., 2021. 

**CONTEXT**

Change in mean sea level is an essential indicator of our evolving climate, as it reflects both the thermal expansion of the ocean in response to its warming and the increase in ocean mass due to the melting of ice sheets and glaciers(WCRP Global Sea Level Budget Group, 2018). According to the IPCC 6th assessment report (IPCC WGI, 2021), global mean sea level (GMSL) increased by 0.20 [0.15 to 0.25] m over the period 1901 to 2018 with a rate of rise that has accelerated since the 1960s to 3.7 [3.2 to 4.2] mm/yr for the period 2006–2018. Human activity was very likely the main driver of observed GMSL rise since 1970 (IPCC WGII, 2021). The weight of the different contributions evolves with time and in the recent decades the mass change has increased, contributing to the on-going acceleration of the GMSL trend (IPCC, 2022a; Legeais et al., 2020; Horwath et al., 2022). At regional scale, sea level does not change homogenously, and regional sea level change is also influenced by various other processes, with different spatial and temporal scales, such as local ocean dynamic, atmospheric forcing, Earth gravity and vertical land motion changes (IPCC WGI, 2021). The adverse effects of floods, storms and tropical cyclones, and the resulting losses and damage, have increased as a result of rising sea levels, increasing people and infrastructure vulnerability and food security risks, particularly in low-lying areas and island states (IPCC, 2019, 2022b). Adaptation and mitigation measures such as the restoration of mangroves and coastal wetlands, reduce the risks from sea level rise (IPCC, 2022c).  

**KEY FINDINGS**

The altimeter sea level trends over the [1999/02/21 to 2023/12/31] period exhibit large-scale variations with trends up to +10 mm/yr in regions such as the western tropical Pacific Ocean. In this area, trends are mainly of thermosteric origin (Legeais et al., 2018; Meyssignac et al., 2017) in response to increased easterly winds during the last two decades associated with the decreasing Interdecadal Pacific Oscillation (IPO)/Pacific Decadal Oscillation (e.g., McGregor et al., 2012; Merrifield et al., 2012; Palanisamy et al., 2015; Rietbroek et al., 2016). 

Prandi et al. (2021) have estimated a regional altimeter sea level error budget from which they determine a regional error variance-covariance matrix and they provide uncertainties of the regional sea level trends. Over 1993-2019, the averaged local sea level trend uncertainty is around 0.83 mm/yr with local values ranging from 0.78 to 1.22 mm/yr.  

**DOI (product):** 
https://doi.org/10.48670/moi-00238

**References:**

* Horwath, M., Gutknecht, B. D., Cazenave, A., Palanisamy, H. K., Marti, F., Marzeion, B., Paul, F., Le Bris, R., Hogg, A. E., Otosaka, I., Shepherd, A., Döll, P., Cáceres, D., Müller Schmied, H., Johannessen, J. A., Nilsen, J. E. Ø., Raj, R. P., Forsberg, R., Sandberg Sørensen, L., Barletta, V. R., Simonsen, S. B., Knudsen, P., Andersen, O. B., Ranndal, H., Rose, S. K., Merchant, C. J., Macintosh, C. R., von Schuckmann, K., Novotny, K., Groh, A., Restano, M., and Benveniste, J.: Global sea-level budget and ocean-mass budget, with a focus on advanced data products and uncertainty characterisation, Earth Syst. Sci. Data, 14, 411–447, https://doi.org/10.5194/essd-14-411-2022, 2022.
* IPCC: Summary for Policymakers. In: IPCC Special Report on the Ocean and Cryosphere in a Changing Climate [H.-O. Pörtner, D.C. Roberts, V. Masson-Delmotte, P. Zhai, M. Tignor, E. Poloczanska, K. Mintenbeck, A. Alegría, M. Nicolai, A. Okem, J. Petzold, B. Rama, N.M. Weyer (eds.)]. In press., 2019.
* IPCC: AR6 Synthesis Report: Climate Change 2022, 2022a.
* IPCC: Summary for Policymakers [H.-O. Pörtner, D.C. Roberts, E.S. Poloczanska, K. Mintenbeck, M. Tignor, A. Alegría, M. Craig, S.
* IPCC: Summary for Policymakers. In: Climate Change 2022: Mitigation of Climate Change. Contribution of Working Group III to the Sixth Assessment Report of the Intergovernmental Panel on Climate Change [P.R. Shukla, J. Skea, R. Slade, A. Al Khourdajie, R. van Diemen, D. McCollum, M. Pathak, S. Some, P. Vyas, R. Fradera, M. Belkacemi, A. Hasija, G. Lisboa, S. Luz, J. Malley, (eds.)], , https://doi.org/10.1017/9781009157926.001, 2022c.
* IPCC WGI: Climate Change 2021: The Physical Science Basis. Contribution of Working Group I to the Sixth Assessment Report of the Intergovernmental Panel on Climate Change, 2021.
* IPCC WGII: Climate Change 2021: Impacts, Adaptation and Vulnerability; Summary for Policemakers. Contribution of Working Group II to the Sixth Assessment Report of the Intergovernmental Panel on Climate Change, 2021.
* Legeais, J. F., von Schuckmann, K., Melet, A., Storto, A., and Meyssignac, B.: Sea Level, Journal of Operational Oceanography, 11, s13–s16, https://doi.org/10.1080/1755876X.2018.1489208, 2018.
* McGregor, S., Gupta, A. S., and England, M. H.: Constraining Wind Stress Products with Sea Surface Height Observations and Implications for Pacific Ocean Sea Level Trend Attribution, Journal of Climate, 25, 8164–8176, https://doi.org/10.1175/JCLI-D-12-00105.1, 2012.
* Merrifield, M. A., Thompson, P. R., and Lander, M.: Multidecadal sea level anomalies and trends in the western tropical Pacific, Geophysical Research Letters, 39, https://doi.org/10.1029/2012GL052032, 2012.
* Meyssignac, B., Piecuch, C. G., Merchant, C. J., Racault, M.-F., Palanisamy, H., MacIntosh, C., Sathyendranath, S., and Brewin, R.: Causes of the Regional Variability in Observed Sea Level, Sea Surface Temperature and Ocean Colour Over the Period 1993–2011, in: Integrative Study of the Mean Sea Level and Its Components, edited by: Cazenave, A., Champollion, N., Paul, F., and Benveniste, J., Springer International Publishing, Cham, 191–219, https://doi.org/10.1007/978-3-319-56490-6_9, 2017.
* Palanisamy, H., Cazenave, A., Delcroix, T., and Meyssignac, B.: Spatial trend patterns in the Pacific Ocean sea level during the altimetry era: the contribution of thermocline depth change and internal climate variability, Ocean Dynamics, 65, 341–356, https://doi.org/10.1007/s10236-014-0805-7, 2015.
* Prandi, P., Meyssignac, B., Ablain, M., Spada, G., Ribes, A., and Benveniste, J.: Local sea level trends, accelerations and uncertainties over 1993–2019, Sci Data, 8, 1, https://doi.org/10.1038/s41597-020-00786-7, 2021.
* Rietbroek, R., Brunnabend, S.-E., Kusche, J., Schröter, J., and Dahle, C.: Revisiting the contemporary sea-level budget on global and regional scales, Proceedings of the National Academy of Sciences, 113, 1504–1509, https://doi.org/10.1073/pnas.1519132113, 2016.
* Spada, Giorgio, and Daniele Melini. “SELEN&lt;Sup&gt;4&lt;/Sup&gt; (SELEN Version 4.0): A Fortran Program for Solving the Gravitationally and Topographically Self-Consistent Sea-Level Equation in Glacial Isostatic Adjustment Modeling.” Geoscientific Model Development 12, no. 12 (December 4, 2019): 5055–75. https://doi.org/10.5194/gmd-12-5055-2019.





<img src="https://documentation.marine.copernicus.eu/IMG/OMI_CLIMATE_SL_IBI_area_averaged_anomalies.png" width="300" height="200"/>



# Atlantic Iberian Biscay Mean Sea Level time series and trend from Observations Reprocessing


**Product ID:** OMI_CLIMATE_SL_IBI_area_averaged_anomalies

**Description:** **DEFINITION**

The sea level ocean monitoring indicator has been presented in the Copernicus Ocean State Report #8. The ocean monitoring indicator on regional mean sea level is derived from the DUACS delayed-time (DT-2024 version, “my” (multi-year) dataset used when available, “myint” (multi-year interim) used after) sea level anomaly maps from satellite altimetry based on a stable number of altimeters (two) in the satellite constellation. These products are distributed by the Copernicus Climate Change Service and the Copernicus Marine Service (SEALEVEL_GLO_PHY_CLIMATE_L4_MY_008_057). 

The time series of area averaged anomalies correspond to the area average of the maps in the Irish-Biscay-Iberian (IBI) Sea weighted by the cosine of the latitude (to consider the changing area in each grid with latitude) and by the proportion of ocean in each grid (to consider the coastal areas). The time series are corrected from regional mean GIA correction (weighted GIA mean of a 27 ensemble model following Spada et Melini, 2019). The time series are adjusted for seasonal annual and semi-annual signals and low-pass filtered at 6 months. Then, the trends/accelerations are estimated on the time series using ordinary least square fit.The trend uncertainty is provided in a 90% confidence interval. It is calculated as the weighted mean uncertainties in the region from Prandi et al., 2021. This estimate only considers errors related to the altimeter observation system (i.e., orbit determination errors, geophysical correction errors and inter-mission bias correction errors). The presence of the interannual signal can strongly influence the trend estimation considering to the altimeter period considered (Wang et al., 2021; Cazenave et al., 2014). The uncertainty linked to this effect is not considered. 

""CONTEXT ""

Change in mean sea level is an essential indicator of our evolving climate, as it reflects both the thermal expansion of the ocean in response to its warming and the increase in ocean mass due to the melting of ice sheets and glaciers (WCRP Global Sea Level Budget Group, 2018). At regional scale, sea level does not change homogenously. It is influenced by various other processes, with different spatial and temporal scales, such as local ocean dynamic, atmospheric forcing, Earth gravity and vertical land motion changes (IPCC WGI, 2021). The adverse effects of floods, storms and tropical cyclones, and the resulting losses and damage, have increased as a result of rising sea levels, increasing people and infrastructure vulnerability and food security risks, particularly in low-lying areas and island states (IPCC, 2022a). Adaptation and mitigation measures such as the restoration of mangroves and coastal wetlands, reduce the risks from sea level rise (IPCC, 2022b).  

In IBI region, the RMSL trend is modulated by decadal variations. As observed over the global ocean, the main actors of the long-term RMSL trend are associated with anthropogenic global/regional warming. Decadal variability is mainly linked to the strengthening or weakening of the Atlantic Meridional Overturning Circulation (AMOC) (e.g. Chafik et al., 2019). The latest is driven by the North Atlantic Oscillation (NAO)  for decadal (20-30y) timescales (e.g. Delworth and Zeng, 2016). Along the European coast, the NAO also influences the along-slope winds dynamic which in return significantly contributes to the local sea level variability observed (Chafik et al., 2019). 

""KEY FINDINGS ""

Over the [1999/02/21 to 2023/12/31] period, the area-averaged sea level in the IBI area rises at a rate of 4.7 ± 0.8 mm/yr with an acceleration of 0.30 ± 0.06 mm/yr². This trend estimation is based on the altimeter measurements corrected from global GIA correction (Spada et Melini, 2019) to consider the ongoing movement of land. T The TOPEX-A is no longer included in the computation of regional mean sea level parameters (trend and acceleration) with version 2024 products due to potential drifts, and ongoing work aims to develop a new empirical correction. Calculation begins in February 1999 (the start of the TOPEX-B period). 

**DOI (product):** 
https://doi.org/10.48670/moi-00252

**References:**

* Cazenave, A., Dieng, H.-B., Meyssignac, B., von Schuckmann, K., Decharme, B., and Berthier, E.: The rate of sea-level rise, Nat. Clim. Change, 4, 358–361, https://doi.org/10.1038/nclimate2159, 2014.
* Chafik, L., Nilsen, J. E. Ø., Dangendorf, S., Reverdin, G., and Frederikse, T.: North Atlantic Ocean Circulation and Decadal Sea Level Change During the Altimetry Era, Sci. Rep., 9, 1041, https://doi.org/10.1038/s41598-018-37603-6, 2019.
* Delworth, T. L. and Zeng, F.: The Impact of the North Atlantic Oscillation on Climate through Its Influence on the Atlantic Meridional Overturning Circulation, J. Clim., 29, 941–962, https://doi.org/10.1175/JCLI-D-15-0396.1, 2016.
* IPCC: Summary for Policymakers [H.-O. Pörtner, D.C. Roberts, E.S. Poloczanska, K. Mintenbeck, M. Tignor, A. Alegría, M. Craig, S. Langsdorf, S. Löschke, V. Möller, A. Okem (eds.)]. In: Climate Change 2022: Impacts, Adaptation, and Vulnerability. Contribution of Working Group II to the Sixth Assessment Report of the Intergovernmental Panel on Climate Change [H.-O. Pörtner, D.C. Roberts, M. Tignor, E.S. Poloczanska, K. Mintenbeck, A. Alegría, M. Craig, S. Langsdorf, S. Löschke, V. Möller, A. Okem, B. Rama (eds.)], 2022a.
* IPCC: Summary for Policymakers. In: Climate Change 2022: Mitigation of Climate Change. Contribution of Working Group III to the Sixth Assessment Report of the Intergovernmental Panel on Climate Change [P.R. Shukla, J. Skea, R. Slade, A. Al Khourdajie, R. van Diemen, D. McCollum, M. Pathak, S. Some, P. Vyas, R. Fradera, M. Belkacemi, A. Hasija, G. Lisboa, S. Luz, J. Malley, (eds.)], , https://doi.org/10.1017/9781009157926.001, 2022b.
* IPCC WGI: Climate Change 2021: The Physical Science Basis. Contribution of Working Group I to the Sixth Assessment Report of the Intergovernmental Panel on Climate Change, 2021.
* Prandi, P., Meyssignac, B., Ablain, M., Spada, G., Ribes, A., and Benveniste, J.: Local sea level trends, accelerations and uncertainties over 1993–2019, Sci. Data, 8, 1, https://doi.org/10.1038/s41597-020-00786-7, 2021.
* Spada, G. and Melini, D.: SELEN4 (SELEN version 4.0): a Fortran program for solving the gravitationally and topographically self-consistent sea-level equation in glacial isostatic adjustment modeling, Geosci. Model Dev., 12, 5055–5075, https://doi.org/10.5194/gmd-12-5055-2019, 2019.
* Von Schuckmann et al., “The State of the Global Ocean, Issue 8.”
* Wang, J., Church, J. A., Zhang, X., and Chen, X.: Reconciling global mean and regional sea level change in projections and observations, Nat. Commun., 12, 990, https://doi.org/10.1038/s41467-021-21265-6, 2021.
* WCRP Global Sea Level Budget Group: Global sea-level budget 1993–present, Earth Syst. Sci. Data, 10, 1551–1590, https://doi.org/10.5194/essd-10-1551-2018, 2018.





<img src="https://documentation.marine.copernicus.eu/IMG/OMI_CLIMATE_SL_MEDSEA_area_averaged_anomalies.png" width="300" height="200"/>



# Mediterranean Sea Mean Sea Level time series and trend from Observations Reprocessing


**Product ID:** OMI_CLIMATE_SL_MEDSEA_area_averaged_anomalies

**Description:** **DEFINITION**

The sea level ocean monitoring indicator has been presented in the Copernicus Ocean State Report #8. The ocean monitoring indicator of regional mean sea level is derived from the DUACS delayed-time (DT-2024 version, “my” (multi-year) dataset used when available, “myint” (multi-year interim) used after) sea level anomaly maps from satellite altimetry based on a stable number of altimeters (two) in the satellite constellation. These products are distributed by the Copernicus Climate Change Service and the Copernicus Marine Service (SEALEVEL_GLO_PHY_CLIMATE_L4_MY_008_057). 

The time series of area averaged anomalies correspond to the area average of the maps in the Mediterranean Sea weighted by the cosine of the latitude (to consider the changing area in each grid with latitude) and by the proportion of ocean in each grid (to consider the coastal areas). The time series are corrected from regional mean GIA correction (weighted GIA mean of a 27 ensemble model following Spada et Melini, 2019). The time series are adjusted for seasonal annual and semi-annual signals and low-pass filtered at 6 months. Then, the trends/accelerations are estimated on the time series using ordinary least square fit.The trend uncertainty is provided in a 90% confidence interval. It is calculated as the weighted mean uncertainties in the region from Prandi et al., 2021. This estimate only considers errors related to the altimeter observation system (i.e., orbit determination errors, geophysical correction errors and inter-mission bias correction errors). The presence of the interannual signal can strongly influence the trend estimation considering to the period considered (Wang et al., 2021; Cazenave et al., 2014). The uncertainty linked to this effect is not considered. 

""CONTEXT"" 

Change in mean sea level is an essential indicator of our evolving climate, as it reflects both the thermal expansion of the ocean in response to its warming and the increase in ocean mass due to the melting of ice sheets and glaciers (WCRP Global Sea Level Budget Group, 2018). At regional scale, sea level does not change homogenously. It is influenced by various other processes, with different spatial and temporal scales, such as local ocean dynamic, atmospheric forcing, Earth gravity and vertical land motion changes (IPCC WGI, 2021). The adverse effects of floods, storms and tropical cyclones, and the resulting losses and damage, have increased as a result of rising sea levels, increasing people and infrastructure vulnerability and food security risks, particularly in low-lying areas and island states (IPCC, 2022a). Adaptation and mitigation measures such as the restoration of mangroves and coastal wetlands, reduce the risks from sea level rise (IPCC, 2022b).  

Beside a clear long-term trend, the regional mean sea level variation in the Mediterranean Sea shows an important interannual variability, with a high trend observed between 1993 and 1999 (nearly 8.4 mm/y) and relatively lower values afterward (nearly 2.4 mm/y between 2000 and 2022). This variability is associated with a variation of the different forcing. Steric effect has been the most important forcing before 1999 (Fenoglio-Marc, 2002; Vigo et al., 2005). Important change of the deep-water formation site also occurred in the 90’s. Their influence contributed to change the temperature and salinity property of the intermediate and deep water masses. These changes in the water masses and distribution is also associated with sea surface circulation changes, as the one observed in the Ionian Sea in 1997-1998 (e.g. Gačić et al., 2011), under the influence of the North Atlantic Oscillation (NAO) and negative Atlantic Multidecadal Oscillation (AMO) phases (Incarbona et al., 2016). These circulation changes may also impact the sea level trend in the basin (Vigo et al., 2005). In 2010-2011, high regional mean sea level has been related to enhanced water mass exchange at Gibraltar, under the influence of wind forcing during the negative phase of NAO (Landerer and Volkov, 2013).The relatively high contribution of both sterodynamic (due to steric and circulation changes) and gravitational, rotational, and deformation (due to mass and water storage changes) after 2000 compared to the [1960, 1989] period is also underlined by (Calafat et al., 2022). 

""KEY FINDINGS"" 

Over the [1999/02/21 to 2023/12/31] period, the area-averaged sea level in the Mediterranean Sea rises at a rate of 3.4 ± 0.8 mm/yr with an acceleration of 0.09 ± 0.06 mm/yr². This trend estimation is based on the altimeter measurements corrected from regional GIA correction (Spada et Melini, 2019) to consider the ongoing movement of land. The TOPEX-A is no longer included in the computation of regional mean sea level parameters (trend and acceleration) with version 2024 products due to potential drifts, and ongoing work aims to develop a new empirical correction. Calculation begins in February 1999 (the start of the TOPEX-B period). 

**DOI (product):** 
https://doi.org/10.48670/moi-00264

**References:**

* Calafat, F. M., Frederikse, T., and Horsburgh, K.: The Sources of Sea-Level Changes in the Mediterranean Sea Since 1960, J. Geophys. Res. Oceans, 127, e2022JC019061, https://doi.org/10.1029/2022JC019061, 2022.
* Cazenave, A., Dieng, H.-B., Meyssignac, B., von Schuckmann, K., Decharme, B., and Berthier, E.: The rate of sea-level rise, Nat. Clim. Change, 4, 358–361, https://doi.org/10.1038/nclimate2159, 2014.
* Fenoglio-Marc, L.: Long-term sea level change in the Mediterranean Sea from multi-satellite altimetry and tide gauges, Phys. Chem. Earth Parts ABC, 27, 1419–1431, https://doi.org/10.1016/S1474-7065(02)00084-0, 2002.
* Gačić, M., Civitarese, G., Eusebi Borzelli, G. L., Kovačević, V., Poulain, P.-M., Theocharis, A., Menna, M., Catucci, A., and Zarokanellos, N.: On the relationship between the decadal oscillations of the northern Ionian Sea and the salinity distributions in the eastern Mediterranean, J. Geophys. Res. Oceans, 116, https://doi.org/10.1029/2011JC007280, 2011.
* Incarbona, A., Martrat, B., Mortyn, P. G., Sprovieri, M., Ziveri, P., Gogou, A., Jordà, G., Xoplaki, E., Luterbacher, J., Langone, L., Marino, G., Rodríguez-Sanz, L., Triantaphyllou, M., Di Stefano, E., Grimalt, J. O., Tranchida, G., Sprovieri, R., and Mazzola, S.: Mediterranean circulation perturbations over the last five centuries: Relevance to past Eastern Mediterranean Transient-type events, Sci. Rep., 6, 29623, https://doi.org/10.1038/srep29623, 2016.
* IPCC: Summary for Policymakers [H.-O. Pörtner, D.C. Roberts, E.S. Poloczanska, K. Mintenbeck, M. Tignor, A. Alegría, M. Craig, S. Langsdorf, S. Löschke, V. Möller, A. Okem (eds.)]. In: Climate Change 2022: Impacts, Adaptation, and Vulnerability. Contribution of Working Group II to the Sixth Assessment Report of the Intergovernmental Panel on Climate Change [H.-O. Pörtner, D.C. Roberts, M. Tignor, E.S. Poloczanska, K. Mintenbeck, A. Alegría, M. Craig, S. Langsdorf, S. Löschke, V. Möller, A. Okem, B. Rama (eds.)], 2022a.
* IPCC: Summary for Policymakers. In: Climate Change 2022: Mitigation of Climate Change. Contribution of Working Group III to the Sixth Assessment Report of the Intergovernmental Panel on Climate Change [P.R. Shukla, J. Skea, R. Slade, A. Al Khourdajie, R. van Diemen, D. McCollum, M. Pathak, S. Some, P. Vyas, R. Fradera, M. Belkacemi, A. Hasija, G. Lisboa, S. Luz, J. Malley, (eds.)], , https://doi.org/10.1017/9781009157926.001, 2022b.
* IPCC WGI: Climate Change 2021: The Physical Science Basis. Contribution of Working Group I to the Sixth Assessment Report of the Intergovernmental Panel on Climate Change, 2021.
* Landerer, F. W. and Volkov, D. L.: The anatomy of recent large sea level fluctuations in the Mediterranean Sea, Geophys. Res. Lett., 40, 553–557, https://doi.org/10.1002/grl.50140, 2013.
* Prandi, P., Meyssignac, B., Ablain, M., Spada, G., Ribes, A., and Benveniste, J.: Local sea level trends, accelerations and uncertainties over 1993–2019, Sci. Data, 8, 1, https://doi.org/10.1038/s41597-020-00786-7, 2021.
* Spada, G. and Melini, D.: SELEN4 (SELEN version 4.0): a Fortran program for solving the gravitationally and topographically self-consistent sea-level equation in glacial isostatic adjustment modeling, Geosci. Model Dev., 12, 5055–5075, https://doi.org/10.5194/gmd-12-5055-2019, 2019.
* Vigo, I., Garcia, D., and Chao, B. F.: Change of sea level trend in the Mediterranean and Black seas, J. Mar. Res., 63, 1085–1100, https://doi.org/10.1357/002224005775247607, 2005.
* Von Schuckmann et al., “The State of the Global Ocean, Issue 8.”
* Wang, J., Church, J. A., Zhang, X., and Chen, X.: Reconciling global mean and regional sea level change in projections and observations, Nat. Commun., 12, 990, https://doi.org/10.1038/s41467-021-21265-6, 2021.
* WCRP Global Sea Level Budget Group: Global sea-level budget 1993–present, Earth Syst. Sci. Data, 10, 1551–1590, https://doi.org/10.5194/essd-10-1551-2018, 2018.





<img src="https://documentation.marine.copernicus.eu/IMG/OMI_CLIMATE_SL_NORTHWESTSHELF_area_averaged_anomalies.png" width="300" height="200"/>



# North West Atlantic Shelf Mean Sea Level time series and trend from Observations Reprocessing


**Product ID:** OMI_CLIMATE_SL_NORTHWESTSHELF_area_averaged_anomalies

**Description:** **DEFINITION**

The sea level ocean monitoring indicator has been presented in the Copernicus Ocean State Report #8. The ocean monitoring indicator on mean sea level is derived from the DUACS delayed-time (DT-2024 version, “my” (multi-year) dataset used when available, “myint” (multi-year interim) used after) sea level anomaly maps from satellite altimetry based on a stable number of altimeters (two) in the satellite constellation. These products are distributed by the Copernicus Climate Change Service and by the Copernicus Marine Service (SEALEVEL_GLO_PHY_CLIMATE_L4_MY_008_057). 

The time series of area averaged anomalies correspond to the area average of the maps in the North-West Shelf Sea weighted by the cosine of the latitude (to consider the changing area in each grid with latitude) and by the proportion of ocean in each grid (to consider the coastal areas). The time series are corrected from regional mean GIA correction (weighted GIA mean of a 27 ensemble model following Spada et Melini, 2019). The time series are adjusted for seasonal annual and semi-annual signals and low-pass filtered at 6 months. Then, the trends/accelerations are estimated on the time series using ordinary least square fit. The trend uncertainty is provided in a 90% confidence interval. It is calculated as the weighted mean uncertainties in the region from Prandi et al., 2021. This estimate only considers errors related to the altimeter observation system (i.e., orbit determination errors, geophysical correction errors and inter-mission bias correction errors). The presence of the interannual signal can strongly influence the trend estimation depending on the period considered (Wang et al., 2021; Cazenave et al., 2014). The uncertainty linked to this effect is not considered. 

""CONTEXT"" 

Change in mean sea level is an essential indicator of our evolving climate, as it reflects both the thermal expansion of the ocean in response to its warming and the increase in ocean mass due to the melting of ice sheets and glaciers (WCRP Global Sea Level Budget Group, 2018). At regional scale, sea level does not change homogenously. It is influenced by various other processes, with different spatial and temporal scales, such as local ocean dynamic, atmospheric forcing, Earth gravity and vertical land motion changes (IPCC WGI, 2021). The adverse effects of floods, storms and tropical cyclones, and the resulting losses and damage, have increased as a result of rising sea levels, increasing people and infrastructure vulnerability and food security risks, particularly in low-lying areas and island states (IPCC, 2022a). Adaptation and mitigation measures such as the restoration of mangroves and coastal wetlands, reduce the risks from sea level rise (IPCC, 2022b).  

In this region, the time series shows decadal variations. As observed over the global ocean, the main actors of the long-term sea level trend are associated with anthropogenic global/regional warming (IPCC WGII, 2021). Decadal variability is mainly linked to the Strengthening or weakening of the Atlantic Meridional Overturning Circulation (AMOC) (e.g. Chafik et al., 2019). The latest is driven by the North Atlantic Oscillation (NAO) for decadal (20-30y) timescales (e.g. Delworth and Zeng, 2016). Along the European coast, the NAO also influences the along-slope winds dynamic which in return significantly contributes to the local sea level variability observed (Chafik et al., 2019). Hermans et al., 2020 also reported the dominant influence of wind on interannual sea level variability in a large part of this area. They also underscored the influence of the inverse barometer forcing in some coastal regions. 

""KEY FINDINGS"" 

Over the [1999/02/21 to 2023/12/31] period, the area-averaged sea level in the NWS area rises at a rate of 3.8 ± 0.8 mm/yr  with an acceleration of 0.25 ± 0.06 mm/yr². This trend estimation is based on the altimeter measurements corrected from regional GIA correction (Spada et Melini, 2019) to consider the ongoing movement of land. The TOPEX-A is no longer included in the computation of regional mean sea level parameters (trend and acceleration) with version 2024 products due to potential drifts, and ongoing work aims to develop a new empirical correction. Calculation begins in February 1999 (the start of the TOPEX-B period). 

**DOI (product):** 
https://doi.org/10.48670/moi-00271

**References:**

* Cazenave, A., Dieng, H.-B., Meyssignac, B., von Schuckmann, K., Decharme, B., and Berthier, E.: The rate of sea-level rise, Nat. Clim. Change, 4, 358–361, https://doi.org/10.1038/nclimate2159, 2014.
* Chafik, L., Nilsen, J. E. Ø., Dangendorf, S., Reverdin, G., and Frederikse, T.: North Atlantic Ocean Circulation and Decadal Sea Level Change During the Altimetry Era, Sci. Rep., 9, 1041, https://doi.org/10.1038/s41598-018-37603-6, 2019.
* Delworth, T. L. and Zeng, F.: The Impact of the North Atlantic Oscillation on Climate through Its Influence on the Atlantic Meridional Overturning Circulation, J. Clim., 29, 941–962, https://doi.org/10.1175/JCLI-D-15-0396.1, 2016.
* Hermans, T. H. J., Le Bars, D., Katsman, C. A., Camargo, C. M. L., Gerkema, T., Calafat, F. M., Tinker, J., and Slangen, A. B. A.: Drivers of Interannual Sea Level Variability on the Northwestern European Shelf, J. Geophys. Res. Oceans, 125, e2020JC016325, https://doi.org/10.1029/2020JC016325, 2020.
* IPCC: Summary for Policymakers [H.-O. Pörtner, D.C. Roberts, E.S. Poloczanska, K. Mintenbeck, M. Tignor, A. Alegría, M. Craig, S. Langsdorf, S. Löschke, V. Möller, A. Okem (eds.)]. In: Climate Change 2022: Impacts, Adaptation, and Vulnerability. Contribution of Working Group II to the Sixth Assessment Report of the Intergovernmental Panel on Climate Change [H.-O. Pörtner, D.C. Roberts, M. Tignor, E.S. Poloczanska, K. Mintenbeck, A. Alegría, M. Craig, S. Langsdorf, S. Löschke, V. Möller, A. Okem, B. Rama (eds.)], 2022b.
* IPCC: Summary for Policymakers. In: Climate Change 2022: Mitigation of Climate Change. Contribution of Working Group III to the Sixth Assessment Report of the Intergovernmental Panel on Climate Change [P.R. Shukla, J. Skea, R. Slade, A. Al Khourdajie, R. van Diemen, D. McCollum, M. Pathak, S. Some, P. Vyas, R. Fradera, M. Belkacemi, A. Hasija, G. Lisboa, S. Luz, J. Malley, (eds.)], , https://doi.org/10.1017/9781009157926.001, 2022c.
* IPCC WGI: Climate Change 2021: The Physical Science Basis. Contribution of Working Group I to the Sixth Assessment Report of the Intergovernmental Panel on Climate Change, 2021.
* IPCC WGII: Climate Change 2021: Impacts, Adaptation and Vulnerability; Summary for Policemakers. Contribution of Working Group II to the Sixth Assessment Report of the Intergovernmental Panel on Climate Change, 2021.
* Prandi, P., Meyssignac, B., Ablain, M., Spada, G., Ribes, A., and Benveniste, J.: Local sea level trends, accelerations and uncertainties over 1993–2019, Sci. Data, 8, 1, https://doi.org/10.1038/s41597-020-00786-7, 2021.
* Spada, G. and Melini, D.: SELEN4 (SELEN version 4.0): a Fortran program for solving the gravitationally and topographically self-consistent sea-level equation in glacial isostatic adjustment modeling, Geosci. Model Dev., 12, 5055–5075, https://doi.org/10.5194/gmd-12-5055-2019, 2019.
* Von Schuckmann et al., “The State of the Global Ocean, Issue 8.”
* Wang, J., Church, J. A., Zhang, X., and Chen, X.: Reconciling global mean and regional sea level change in projections and observations, Nat. Commun., 12, 990, https://doi.org/10.1038/s41467-021-21265-6, 2021.
* WCRP Global Sea Level Budget Group: Global sea-level budget 1993–present, Earth Syst. Sci. Data, 10, 1551–1590, https://doi.org/10.5194/essd-10-1551-2018, 2018.





<img src="https://documentation.marine.copernicus.eu/IMG/OMI_CLIMATE_SST_BAL_area_averaged_anomalies.png" width="300" height="200"/>



# Baltic Sea Surface Temperature anomaly time series and trend from Observations Reprocessing


**Product ID:** OMI_CLIMATE_SST_BAL_area_averaged_anomalies

**Description:** **DEFINITION**

The OMI_CLIMATE_SST_BAL_area_averaged_anomalies product includes time series of monthly mean SST anomalies over the period 1982-2023, relative to the 1991-2020 climatology, averaged for the Baltic Sea. The SST Level 4 analysis products that provide the input to the monthly averages are taken from the reprocessed product SST_BAL_SST_L4_REP_OBSERVATIONS_010_016 with a recent update to include 2023. The product has a spatial resolution of 0.02 in latitude and longitude.
The OMI time series runs from Jan 1, 1982 to December 31, 2023 and is constructed by calculating monthly averages from the daily level 4 SST analysis fields of the SST_BAL_SST_L4_REP_OBSERVATIONS_010_016. See the Copernicus Marine Service Ocean State Reports (section 1.1 in Von Schuckmann et al., 2016; section 3 in Von Schuckmann et al., 2018) for more information on the OMI product. 

**CONTEXT**

Sea Surface Temperature (SST) is an Essential Climate Variable (GCOS) that is an important input for initialising numerical weather prediction models and fundamental for understanding air-sea interactions and monitoring climate change (GCOS 2010). The Baltic Sea is a region that requires special attention regarding the use of satellite SST records and the assessment of climatic variability (Høyer and She 2007; Høyer and Karagali 2016). The Baltic Sea is a semi-enclosed basin with natural variability and it is influenced by large-scale atmospheric processes and by the vicinity of land. In addition, the Baltic Sea is one of the largest brackish seas in the world. When analysing regional-scale climate variability, all these effects have to be considered, which requires dedicated regional and validated SST products. Satellite observations have previously been used to analyse the climatic SST signals in the North Sea and Baltic Sea (BACC II Author Team 2015; Lehmann et al. 2011). Recently, Høyer and Karagali (2016) demonstrated that the Baltic Sea had warmed 1-2 oC from 1982 to 2012 considering all months of the year and 3-5 oC when only July-September months were considered. This was corroborated in the Ocean State Reports (section 1.1 in Von Schuckmann et al., 2016; section 3 in Von Schuckmann et al., 2018). 

**CMEMS KEY FINDINGS**

The basin-average trend of SST anomalies for Baltic Sea region amounts to 0.038±0.004°C/year over the period 1982-2023 which corresponds to an average warming of 1.60°C. Adding the North Sea area, the average trend amounts to 0.029±0.002°C/year over the same period, which corresponds to an average warming of 1.22°C for the entire region since 1982. 

**Figure caption**

Time series of monthly mean (turquoise line) and annual mean (blue line) of sea surface temperature anomalies for January 1982 to December 2023, relative to the 1991-2020 mean, combined for the Baltic Sea and North Sea SST  (OMI_CLIMATE_SST_BAL_area_averaged_anomalies). The data are based on the multi-year Baltic Sea L4 satellite SST reprocessed product SST_BAL_SST_L4_REP_OBSERVATIONS_010_016.

**DOI (product):** 
https://doi.org/10.48670/moi-00205

**References:**

* BACC II Author Team 2015. Second Assessment of Climate Change for the Baltic Sea Basin. Springer Science & Business Media, 501 pp., doi:10.1007/978-3-319-16006-1.
* GCOS. Global Climate Observing System. 2010. Update of the Implementation Plan for the Global Observing System for Climate in Support of the UNFCCC (GCO-138).
* Høyer JL, She J. 2007. Optimal interpolation of sea surface temperature for the North Sea and Baltic Sea. J. Mar. Syst., 65, 176–189, doi:10.1016/j.jmarsys.2005.03.008.
* Lehmann A, Getzlaff K, Harlaß J. 2011. Detailed assessment of climate variability of the Baltic Sea area for the period 1958–2009. Climate Res., 46, 185–196, doi:10.3354/cr00876.
* Karina von Schuckmann ((Editor)), Pierre-Yves Le Traon ((Editor)), Neville Smith ((Editor)), Ananda Pascual ((Editor)), Pierre Brasseur ((Editor)), Katja Fennel ((Editor)), Samy Djavidnia ((Editor)), Signe Aaboe, Enrique Alvarez Fanjul, Emmanuelle Autret, Lars Axell, Roland Aznar, Mario Benincasa, Abderahim Bentamy, Fredrik Boberg, Romain Bourdallé-Badie, Bruno Buongiorno Nardelli, Vittorio E. Brando, Clément Bricaud, Lars-Anders Breivik, Robert J.W. Brewin, Arthur Capet, Adrien Ceschin, Stefania Ciliberti, Gianpiero Cossarini, Mar-ta de Alfonso, Alvaro de Pascual Collar, Jos de Kloe, Julie Deshayes, Charles Desportes, Marie Drévillon, Yann Drillet, Riccardo Droghei, Clotilde Dubois, Owen Embury, Hélène Etienne, Claudia Fratianni, Jesús García La-fuente, Marcos Garcia Sotillo, Gilles Garric, Florent Gasparin, Riccardo Gerin, Simon Good, Jérome Gourrion, Marilaure Grégoire, Eric Greiner, Stéphanie Guinehut, Elodie Gutknecht, Fabrice Hernandez, Olga Hernandez, Jacob Høyer, Laura Jackson, Simon Jandt, Simon Josey, Mélanie Juza, John Kennedy, Zoi Kokkini, Gerasimos Korres, Mariliis Kõuts, Priidik Lagemaa, Thomas Lavergne, Bernard le Cann, Jean-François Legeais, Benedicte Lemieux-Dudon, Bruno Levier, Vidar Lien, Ilja Maljutenko, Fernando Manzano, Marta Marcos, Veselka Mari-nova, Simona Masina, Elena Mauri, Michael Mayer, Angelique Melet, Frédéric Mélin, Benoit Meyssignac, Maeva Monier, Malte Müller, Sandrine Mulet, Cristina Naranjo, Giulio Notarstefano, Aurélien Paulmier, Begoña Pérez Gomez, Irene Pérez Gonzalez, Elisaveta Peneva, Coralie Perruche, K. Andrew Peterson, Nadia Pinardi, Andrea Pisano, Silvia Pardo, Pierre-Marie Poulain, Roshin P. Raj, Urmas Raudsepp, Michaelis Ravdas, Rebecca Reid, Marie-Hélène Rio, Stefano Salon, Annette Samuelsen, Michela Sammartino, Simone Sammartino, Anne Britt Sandø, Rosalia Santoleri, Shubha Sathyendranath, Jun She, Simona Simoncelli, Cosimo Solidoro, Ad Stoffelen, Andrea Storto, Tanguy Szerkely, Susanne Tamm, Steffen Tietsche, Jonathan Tinker, Joaquín Tintore, Ana Trindade, Daphne van Zanten, Luc Vandenbulcke, Anton Verhoef, Nathalie Verbrugge, Lena Viktorsson, Karina von Schuckmann, Sarah L. Wakelin, Anna Zacharioudaki & Hao Zuo (2018) Copernicus Marine Service Ocean State Report, Journal of Operational Oceanography, 11:sup1, S1-S142, DOI: 10.1080/1755876X.2018.1489208
* Karina von Schuckmann, Pierre-Yves Le Traon, Enrique Alvarez-Fanjul, Lars Axell, Magdalena Balmaseda, Lars-Anders Breivik, Robert J. W. Brewin, Clement Bricaud, Marie Drevillon, Yann Drillet, Clotilde Dubois, Owen Embury, Hélène Etienne, Marcos García Sotillo, Gilles Garric, Florent Gasparin, Elodie Gutknecht, Stéphanie Guinehut, Fabrice Hernandez, Melanie Juza, Bengt Karlson, Gerasimos Korres, Jean-François Legeais, Bruno Levier, Vidar S. Lien, Rosemary Morrow, Giulio Notarstefano, Laurent Parent, Álvaro Pascual, Begoña Pérez-Gómez, Coralie Perruche, Nadia Pinardi, Andrea Pisano, Pierre-Marie Poulain, Isabelle M. Pujol, Roshin P. Raj, Urmas Raudsepp, Hervé Roquet, Annette Samuelsen, Shubha Sathyendranath, Jun She, Simona Simoncelli, Cosimo Solidoro, Jonathan Tinker, Joaquín Tintoré, Lena Viktorsson, Michael Ablain, Elin Almroth-Rosell, Antonio Bonaduce, Emanuela Clementi, Gianpiero Cossarini, Quentin Dagneaux, Charles Desportes, Stephen Dye, Claudia Fratianni, Simon Good, Eric Greiner, Jerome Gourrion, Mathieu Hamon, Jason Holt, Pat Hyder, John Kennedy, Fernando Manzano-Muñoz, Angélique Melet, Benoit Meyssignac, Sandrine Mulet, Bruno Buongiorno Nardelli, Enda O’Dea, Einar Olason, Aurélien Paulmier, Irene Pérez-González, Rebecca Reid, Ma-rie-Fanny Racault, Dionysios E. Raitsos, Antonio Ramos, Peter Sykes, Tanguy Szekely & Nathalie Verbrugge (2016) The Copernicus Marine Environment Monitoring Service Ocean State Report, Journal of Operational Oceanography, 9:sup2, s235-s320, DOI: 10.1080/1755876X.2016.1273446
* Høyer, JL, Karagali, I. 2016. Sea surface temperature climate data record for the North Sea and Baltic Sea. Journal of Climate, 29(7), 2529-2541.





<img src="https://documentation.marine.copernicus.eu/IMG/OMI_CLIMATE_SST_BAL_trend.png" width="300" height="200"/>



# Baltic Sea Surface Temperature cumulative trend map from Observations Reprocessing


**Product ID:** OMI_CLIMATE_SST_BAL_trend

**Description:** **DEFINITION**

The  OMI_CLIMATE_SST_BAL_trend product includes the cumulative/net trend in sea surface temperature anomalies for the Baltic Sea from 1982-2023. The cumulative trend is the rate of change (°C/year) scaled by the number of years (42 years). The SST Level 4 analysis products that provide the input to the trend calculations are taken from the reprocessed product SST_BAL_SST_L4_REP_OBSERVATIONS_010_016 with a recent update to include 2023. The product has a spatial resolution of 0.02 in latitude and longitude.
The OMI time series runs from Jan 1, 1982 to December 31, 2023 and is constructed by calculating monthly averages from the daily level 4 SST analysis fields of the SST_BAL_SST_L4_REP_OBSERVATIONS_010_016. See the Copernicus Marine Service Ocean State Reports for more information on the OMI product (section 1.1 in Von Schuckmann et al., 2016; section 3 in Von Schuckmann et al., 2018). The times series of monthly anomalies have been used to calculate the trend in SST using Sen’s method with confidence intervals from the Mann-Kendall test (section 3 in Von Schuckmann et al., 2018).

**CONTEXT**

SST is an essential climate variable that is an important input for initialising numerical weather prediction models and fundamental for understanding air-sea interactions and monitoring climate change. The Baltic Sea is a region that requires special attention regarding the use of satellite SST records and the assessment of climatic variability (Høyer and She 2007; Høyer and Karagali 2016). The Baltic Sea is a semi-enclosed basin with natural variability and it is influenced by large-scale atmospheric processes and by the vicinity of land. In addition, the Baltic Sea is one of the largest brackish seas in the world. When analysing regional-scale climate variability, all these effects have to be considered, which requires dedicated regional and validated SST products. Satellite observations have previously been used to analyse the climatic SST signals in the North Sea and Baltic Sea (BACC II Author Team 2015; Lehmann et al. 2011). Recently, Høyer and Karagali (2016) demonstrated that the Baltic Sea had warmed 1-2oC from 1982 to 2012 considering all months of the year and 3-5oC when only July- September months were considered. This was corroborated in the Ocean State Reports (section 1.1 in Von Schuckmann et al., 2016; section 3 in Von Schuckmann et al., 2018). 

**CMEMS KEY FINDINGS**

SST trends were calculated for the Baltic Sea area and the whole region including the North Sea, over the period January 1982 to December 2023. The average trend for the Baltic Sea domain (east of 9°E longitude) is 0.038°C/year, which represents an average warming of 1.60°C for the 1982-2023 period considered here. When the North Sea domain is included, the trend decreases to 0.029°C/year corresponding to an average warming of 1.22°C for the 1982-2023 period. 
**Figure caption**

**Figure caption**

Cumulative trends in sea surface temperature anomalies calculated from 1982 to 2023 for the Baltic Sea (OMI_CLIMATE_SST_BAL_trend). Trend calculations are based on the multi-year Baltic Sea L4 SST satellite product SST_BAL_SST_L4_REP_OBSERVATIONS_010_016.

**DOI (product):** 
https://doi.org/10.48670/moi-00206

**References:**

* BACC II Author Team 2015. Second Assessment of Climate Change for the Baltic Sea Basin. Springer Science & Business Media, 501 pp., doi:10.1007/978-3-319-16006-1.
* Høyer, JL, Karagali, I. 2016. Sea surface temperature climate data record for the North Sea and Baltic Sea. Journal of Climate, 29(7), 2529-2541.
* Høyer JL, She J. 2007. Optimal interpolation of sea surface temperature for the North Sea and Baltic Sea. J. Mar. Syst., 65, 176–189, doi:10.1016/j.jmarsys.2005.03.008.
* Lehmann A, Getzlaff K, Harlaß J. 2011. Detailed assessment of climate variability of the Baltic Sea area for the period 1958–2009. Climate Res., 46, 185–196, doi:10.3354/cr00876.
* Karina von Schuckmann ((Editor)), Pierre-Yves Le Traon ((Editor)), Neville Smith ((Editor)), Ananda Pascual ((Editor)), Pierre Brasseur ((Editor)), Katja Fennel ((Editor)), Samy Djavidnia ((Editor)), Signe Aaboe, Enrique Alvarez Fanjul, Emmanuelle Autret, Lars Axell, Roland Aznar, Mario Benincasa, Abderahim Bentamy, Fredrik Boberg, Romain Bourdallé-Badie, Bruno Buongiorno Nardelli, Vittorio E. Brando, Clément Bricaud, Lars-Anders Breivik, Robert J.W. Brewin, Arthur Capet, Adrien Ceschin, Stefania Ciliberti, Gianpiero Cossarini, Marta de Alfonso, Alvaro de Pascual Collar, Jos de Kloe, Julie Deshayes, Charles Desportes, Marie Drévillon, Yann Drillet, Riccardo Droghei, Clotilde Dubois, Owen Embury, Hélène Etienne, Claudia Fratianni, Jesús García Lafuente, Marcos Garcia Sotillo, Gilles Garric, Florent Gasparin, Riccardo Gerin, Simon Good, Jérome Gourrion, Marilaure Grégoire, Eric Greiner, Stéphanie Guinehut, Elodie Gutknecht, Fabrice Hernandez, Olga Hernandez, Jacob Høyer, Laura Jackson, Simon Jandt, Simon Josey, Mélanie Juza, John Kennedy, Zoi Kokkini, Gerasimos Korres, Mariliis Kõuts, Priidik Lagemaa, Thomas Lavergne, Bernard le Cann, Jean-François Legeais, Benedicte Lemieux-Dudon, Bruno Levier, Vidar Lien, Ilja Maljutenko, Fernando Manzano, Marta Marcos, Veselka Marinova, Simona Masina, Elena Mauri, Michael Mayer, Angelique Melet, Frédéric Mélin, Benoit Meyssignac, Maeva Monier, Malte Müller, Sandrine Mulet, Cristina Naranjo, Giulio Notarstefano, Aurélien Paulmier, Begoña Pérez Gomez, Irene Pérez Gonzalez, Elisaveta Peneva, Coralie Perruche, K. Andrew Peterson, Nadia Pinardi, Andrea Pisano, Silvia Pardo, Pierre-Marie Poulain, Roshin P. Raj, Urmas Raudsepp, Michaelis Ravdas, Rebecca Reid, Marie-Hélène Rio, Stefano Salon, Annette Samuelsen, Michela Sammartino, Simone Sammartino, Anne Britt Sandø, Rosalia Santoleri, Shubha Sathyendranath, Jun She, Simona Simoncelli, Cosimo Solidoro, Ad Stoffelen, Andrea Storto, Tanguy Szerkely, Susanne Tamm, Steffen Tietsche, Jonathan Tinker, Joaquín Tintore, Ana Trindade, Daphne van Zanten, Luc Vandenbulcke, Anton Verhoef, Nathalie Verbrugge, Lena Viktorsson, Karina von Schuckmann, Sarah L. Wakelin, Anna Zacharioudaki & Hao Zuo (2018) Copernicus Marine Service Ocean State Report, Journal of Operational Oceanography, 11:sup1, S1-S142, DOI: 10.1080/1755876X.2018.1489208
* Karina von Schuckmann, Pierre-Yves Le Traon, Enrique Alvarez-Fanjul, Lars Axell, Magdalena Balmaseda, Lars-Anders Breivik, Robert J. W. Brewin, Clement Bricaud, Marie Drevillon, Yann Drillet, Clotilde Dubois, Owen Embury, Hélène Etienne, Marcos García Sotillo, Gilles Garric, Florent Gasparin, Elodie Gutknecht, Stéphanie Guinehut, Fabrice Hernandez, Melanie Juza, Bengt Karlson, Gerasimos Korres, Jean-François Legeais, Bruno Levier, Vidar S. Lien, Rosemary Morrow, Giulio Notarstefano, Laurent Parent, Álvaro Pascual, Begoña Pérez-Gómez, Coralie Perruche, Nadia Pinardi, Andrea Pisano, Pierre-Marie Poulain, Isabelle M. Pujol, Roshin P. Raj, Urmas Raudsepp, Hervé Roquet, Annette Samuelsen, Shubha Sathyendranath, Jun She, Simona Simoncelli, Cosimo Solidoro, Jonathan Tinker, Joaquín Tintoré, Lena Viktorsson, Michael Ablain, Elin Almroth-Rosell, Antonio Bonaduce, Emanuela Clementi, Gianpiero Cossarini, Quentin Dagneaux, Charles Desportes, Stephen Dye, Claudia Fratianni, Simon Good, Eric Greiner, Jerome Gourrion, Mathieu Hamon, Jason Holt, Pat Hyder, John Kennedy, Fernando Manzano-Muñoz, Angélique Melet, Benoit Meyssignac, Sandrine Mulet, Bruno Buongiorno Nardelli, Enda O’Dea, Einar Olason, Aurélien Paulmier, Irene Pérez-González, Rebecca Reid, Ma-rie-Fanny Racault, Dionysios E. Raitsos, Antonio Ramos, Peter Sykes, Tanguy Szekely & Nathalie Verbrugge (2016) The Copernicus Marine Environment Monitoring Service Ocean State Report, Journal of Operational Oceanography, 9:sup2, s235-s320, DOI: 10.1080/1755876X.2016.1273446





<img src="https://documentation.marine.copernicus.eu/IMG/OMI_CLIMATE_SST_IBI_area_averaged_anomalies.png" width="300" height="200"/>



# Iberia Biscay Ireland Sea Surface Temperature time series and trend from Observations Reprocessing


**Product ID:** OMI_CLIMATE_SST_IBI_area_averaged_anomalies

**Description:** **DEFINITION**

The omi_climate_sst_ibi_area_averaged_anomalies product for 2022 includes Sea Surface Temperature (SST) anomalies, given as monthly mean time series starting on 1993 and averaged over the Iberia-Biscay-Irish Seas. The IBI SST OMI is built from the CMEMS Reprocessed European North West Shelf Iberai-Biscay-Irish Seas  (SST_MED_SST_L4_REP_OBSERVATIONS_010_026, see e.g. the OMI QUID, http://marine.copernicus.eu/documents/QUID/CMEMS-OMI-QUID-CLIMATE-SST-IBI_v2.1.pdf), which provided the SSTs used to compute the evolution of SST anomalies over the European North West Shelf Seas. This reprocessed product consists of daily (nighttime) interpolated 0.05° grid resolution SST maps over the European North West Shelf Iberia-Biscay-Irish Seas built from the ESA Climate Change Initiative (CCI) (Merchant et al., 2019), Copernicus Climate Change Service (C3S) initiatives and Eumetsat data. Anomalies are computed against the 1993-2014 reference period.

**CONTEXT**

Sea surface temperature (SST) is a key climate variable since it deeply contributes in regulating climate and its variability (Deser et al., 2010). SST is then essential to monitor and characterise the state of the global climate system (GCOS 2010). Long-term SST variability, from interannual to (multi-)decadal timescales, provides insight into the slow variations/changes in SST, i.e. the temperature trend (e.g., Pezzulli et al., 2005). In addition, on shorter timescales, SST anomalies become an essential indicator for extreme events, as e.g. marine heatwaves (Hobday et al., 2018).

**CMEMS KEY FINDINGS**

The overall trend in the SST anomalies in this region is 0.013 ±0.001 °C/year over the period 1993-2022.    

**Figure caption**

Time series of monthly mean and 12-month filtered sea surface temperature anomalies in the Iberia-Biscay-Irish Seas during the period 1993-2022. Anomalies are relative to the climatological period 1993-2014 and built from the CMEMS SST_ATL_SST_L4_REP_OBSERVATIONS_010_026 satellite product (see e.g. the OMI QUID, http://marine.copernicus.eu/documents/QUID/CMEMS-OMI-QUID-IBI-SST.pdf). The sea surface temperature trend with its 95% confidence interval (shown in the box) is estimated by using the X-11 seasonal adjustment procedure (e.g. Pezzulli et al., 2005) and Sen’s method (Sen 1968).

**DOI (product):** 
https://doi.org/10.48670/moi-00256

**References:**

* Deser, C., Alexander, M. A., Xie, S.-P., Phillips, A. S., 2010. Sea Surface Temperature Variability: Patterns and Mechanisms. Annual Review of Marine Science 2010 2:1, 115-143. https://doi.org/10.1146/annurev-marine-120408-151453
* GCOS. Global Climate Observing System. 2010. Update of the Implementation Plan for the Global Observing System for Climate in Support of the UNFCCC (GCO-138).
* Hobday, A. J., Oliver, E. C., Gupta, A. S., Benthuysen, J. A., Burrows, M. T., Donat, M. G., ... & Smale, D. A. (2018). Categorizing and naming marine heatwaves. Oceanography, 31(2), 162-173.
* Merchant, C. J., Embury, O., Bulgin, C. E., Block, T., Corlett, G. K., Fiedler, E., ... & Eastwood, S. (2019). Satellite-based time-series of sea-surface temperature since 1981 for climate applications. Scientific data, 6(1), 1-18.
* Pezzulli, S., Stephenson, D. B., Hannachi, A., 2005. The Variability of Seasonality. J. Climate. 18:71–88. doi:10.1175/JCLI-3256.1.
* Sen, P. K., 1968. Estimates of the regression coefficient based on Kendall’s tau. J Am Statist Assoc. 63:1379–1389.





<img src="https://documentation.marine.copernicus.eu/IMG/OMI_CLIMATE_SST_IBI_trend.png" width="300" height="200"/>



# Iberia Biscay Ireland Sea Surface Temperature trend map from Observations Reprocessing


**Product ID:** OMI_CLIMATE_SST_IBI_trend

**Description:** **DEFINITION**

The omi_climate_sst_ibi_trend product includes the Sea Surface Temperature (SST) trend for the Iberia-Biscay-Irish areas over the period 1982-2023, i.e. the rate of change (°C/year). This OMI is derived from the CMEMS REP ATL L4 SST product (SST_ATL_SST_L4_REP_OBSERVATIONS_010_026), see e.g. the OMI QUID, http://marine.copernicus.eu/documents/QUID/CMEMS-OMI-QUID-CLIMATE-SST-IBI_v3.pdf), which provided the SSTs used to compute the SST trend over the Iberia-Biscay-Irish areas. This reprocessed product consists of daily (nighttime) interpolated 0.05° grid resolution SST maps built from the ESA Climate Change Initiative (CCI) (Merchant et al., 2019) and Copernicus Climate Change Service (C3S) initiatives. Trend analysis has been performed by using the X-11 seasonal adjustment procedure (see e.g. Pezzulli et al., 2005), which has the effect of filtering the input SST time series acting as a low bandpass filter for interannual variations. Mann-Kendall test and Sens’s method (Sen 1968) were applied to assess whether there was a monotonic upward or downward trend and to estimate the slope of the trend and its 95% confidence interval.    

**CONTEXT**

Sea surface temperature (SST) is a key climate variable since it deeply contributes in regulating climate and its variability (Deser et al., 2010). SST is then essential to monitor and characterise the state of the global climate system (GCOS 2010). Long-term SST variability, from interannual to (multi-)decadal timescales, provides insight into the slow variations/changes in SST, i.e. the temperature trend (e.g., Pezzulli et al., 2005). In addition, on shorter timescales, SST anomalies become an essential indicator for extreme events, as e.g. marine heatwaves (Hobday et al., 2018). 

**CMEMS KEY FINDINGS**

The overall trend in the SST anomalies in this region is 0.022 ±0.002 °C/year over the period 1982-2023.    

**Figure caption**
Sea surface temperature trend over the period 1982-2023 in the Iberia-Biscay-Irish areas. The trend is the rate of change (°C/year). The trend map in sea surface temperature is derived from the CMEMS SST_ATL_SST_L4_REP_OBSERVATIONS_010_026 product (see e.g. the OMI QUID, http://marine.copernicus.eu/documents/QUID/CMEMS-OMI-QUID-ATL-SST.pdf). The trend is estimated by using the X-11 seasonal adjustment procedure (e.g. Pezzulli et al., 2005) and Sen’s method (Sen 1968).

**DOI (product):** 
https://doi.org/10.48670/moi-00257

**References:**

* Deser, C., Alexander, M. A., Xie, S.-P., Phillips, A. S., 2010. Sea Surface Temperature Variability: Patterns and Mechanisms. Annual Review of Marine Science 2010 2:1, 115-143. https://doi.org/10.1146/annurev-marine-120408-151453
* GCOS. Global Climate Observing System. 2010. Update of the Implementation Plan for the Global Observing System for Climate in Support of the UNFCCC (GCO-138).
* Hobday, A. J., Oliver, E. C., Gupta, A. S., Benthuysen, J. A., Burrows, M. T., Donat, M. G., ... & Smale, D. A. (2018). Categorizing and naming marine heatwaves. Oceanography, 31(2), 162-173.
* Merchant, C. J., Embury, O., Bulgin, C. E., Block, T., Corlett, G. K., Fiedler, E., ... & Eastwood, S. (2019). Satellite-based time-series of sea-surface temperature since 1981 for climate applications. Scientific data, 6(1), 1-18.
* Pezzulli, S., Stephenson, D. B., Hannachi, A., 2005. The Variability of Seasonality. J. Climate. 18:71–88. doi:10.1175/JCLI-3256.1.
* Sen, P. K., 1968. Estimates of the regression coefficient based on Kendall’s tau. J Am Statist Assoc. 63:1379–1389





<img src="https://documentation.marine.copernicus.eu/IMG/OMI_CLIMATE_SST_IST_ARCTIC_anomaly.png" width="300" height="200"/>



# Arctic Sea and Sea Ice Surface Temperature anomaly based on reprocessed observations


**Product ID:** OMI_CLIMATE_SST_IST_ARCTIC_anomaly

**Description:** **DEFINITION**

The OMI_CLIMATE_SST_IST_ARCTIC_anomaly product includes the 2D annual mean surface temperature anomaly for the Arctic Ocean for 2023. The annual mean surface temperature anomaly is calculated from the climatological mean estimated from 1991 to 2020, defined according to the WMO recommendation (WMO, 2017) and recent U.S. National Oceanic and Atmospheric Administration practice (https://wmo.int/media/news/updated-30-year-reference-period-reflects-changing-climate,). The SST/IST Level 4 analysis that provides the input to the climatology and mean anomaly calculations are taken from the reprocessed product SEAICE_ARC_PHY_CLIMATE_L4_MY_011_016 with a recent update to include 2023. The product has a spatial resolution of 0.05 degrees in latitude and longitude. 
The OMI time series runs from Jan 1, 1982 to December 31, 2023 and is constructed by calculating monthly average anomalies from the reference climatology from 1991 to 2020, using the daily level 4 SST analysis fields of the SEAICE_ARC_PHY_CLIMATE_L4_MY_011_016 product. See the Copernicus Marine Service Ocean State Reports (section 1.1 in Von Schuckmann et al., 2016; section 3 in Von Schuckmann et al., 2018) for more information on the temperature OMI product. The times series of monthly anomalies have been used to calculate the trend in surface temperature (combined SST and IST) using Sen’s method with confidence intervals from the Mann-Kendall test (section 3 in Von Schuckmann et al., 2018).

**CONTEXT**

SST and IST are essential climate variables that act as important input for initializing numerical weather prediction models and fundamental for understanding air-sea interactions and monitoring climate change. Especially in the Arctic, SST/IST feedbacks amplify climate change (AMAP, 2021). In the Arctic Ocean, the surface temperatures play a crucial role for the heat exchange between the ocean and atmosphere, sea ice growth and melt processes (Key et al, 1997) in addition to weather and sea ice forecasts through assimilation into ocean and atmospheric models (Rasmussen et al., 2018). 
The Arctic Ocean is a region that requires special attention regarding the use of satellite SST and IST records and the assessment of climatic variability due to the presence of both seawater and ice, and the large seasonal and inter-annual fluctuations in the sea ice cover which lead to increased complexity in the SST mapping of the Arctic region. Combining SST and ice surface temperature (IST) is identified as the most appropriate method for determining the surface temperature of the Arctic (Minnett et al., 2020).  
Previously, climate trends have been estimated individually for SST and IST records (Bulgin et al., 2020; Comiso and Hall, 2014). However, this is problematic in the Arctic region due to the large temporal variability in the sea ice cover including the overlying northward migration of the ice edge on decadal timescales, and thus, the resulting climate trends are not easy to interpret (Comiso, 2003). A combined surface temperature dataset of the ocean, sea ice and the marginal ice zone (MIZ) provides a consistent climate indicator, which is important for studying climate trends in the Arctic region.

**KEY FINDINGS**

The area average anomaly of 2023 is 1.70±1.08°C (± means 1 standard deviation in this case). The majority of anomalies are positive and exceed 2°C for most areas of the Arctic Ocean, while the largest regional anomalies exceeded 6°C. Near zero and slightly negative anomalies are observed in some areas of the Barents, Norwegian and Greenland Sea and around the Bering Strait. 

**DOI (product):** 
https://doi.org/10.48670/mds-00353

**References:**

* AMAP, 2021. Arctic Climate Change Update 2021: Key Trends and Impacts. Summary for Policy-makers. Arctic Monitoring and Assessment Programme (AMAP), Tromsø, Norway.
* Bulgin, C.E., Merchant, C.J., Ferreira, D., 2020. Tendencies, variability and persistence of sea surface temperature anomalies. Sci Rep 10, 7986. https://doi.org/10.1038/s41598-020-64785-9
* Comiso, J.C., 2003. Warming Trends in the Arctic from Clear Sky Satellite Observations. Journal of Climate. https://doi.org/10.1175/1520-0442(2003)016<3498:WTITAF>2.0.CO;2
* Comiso, J.C., Hall, D.K., 2014. Climate trends in the Arctic as observed from space: Climate trends in the Arctic as observed from space. WIREs Clim Change 5, 389–409. https://doi.org/10.1002/wcc.277
* Kendall MG. 1975. Multivariate analysis. London: CharlesGriffin & Co; p. 210, 4
* Key, J.R., Collins, J.B., Fowler, C., Stone, R.S., 1997. High-latitude surface temperature estimates from thermal satellite data. Remote Sensing of Environment 61, 302–309. https://doi.org/10.1016/S0034-4257(97)89497-7
* Minnett, P.J., Kilpatrick, K.A., Podestá, G.P., Evans, R.H., Szczodrak, M.D., Izaguirre, M.A., Williams, E.J., Walsh, S., Reynolds, R.M., Bailey, S.W., Armstrong, E.M., Vazquez-Cuervo, J., 2020. Skin Sea-Surface Temperature from VIIRS on Suomi-NPP—NASA Continuity Retrievals. Remote Sensing 12, 3369. https://doi.org/10.3390/rs12203369
* Rasmussen, T.A.S., Høyer, J.L., Ghent, D., Bulgin, C.E., Dybkjaer, G., Ribergaard, M.H., Nielsen-Englyst, P., Madsen, K.S., 2018. Impact of Assimilation of Sea-Ice Surface Temperatures on a Coupled Ocean and Sea-Ice Model. Journal of Geophysical Research: Oceans 123, 2440–2460. https://doi.org/10.1002/2017JC013481
* Sen PK. 1968. Estimates of the regression coefficient based on Kendall’s tau. J AmStatist Assoc. 63:1379–1389
* von Schuckmann et al., 2016: The Copernicus Marine Environment Monitoring Service Ocean State Report, Journal of Operational Oceanography, Volume 9, 2016 - Issue sup2: The Copernicus Marine Environment Monitoring Service Ocean, http://dx.doi.org/10.1080/1755876X.2016.1273446.
* von Schuckmann, K., Le Traon, P.-Y., Smith, N., Pascual, A., Brasseur, P., Fennel, K., Djavidnia, S., Aaboe, S., Fanjul, E. A., Autret, E., Axell, L., Aznar, R., Benincasa, M., Bentamy, A., Boberg, F., Bourdallé-Badie, R., Nardelli, B. B., Brando, V. E., Bricaud, C., … Zuo, H. (2018). Copernicus Marine Service Ocean State Report. Journal of Operational Oceanography, 11(sup1), S1–S142. https://doi.org/10.1080/1755876X.2018.1489208
* WMO, Guidelines on the Calculation of Climate Normals, 2017, WMO-No-.1203
* Mann HB. 1945. Nonparametric tests against trend. Econometrica. 13:245–259. p. 42





<img src="https://documentation.marine.copernicus.eu/IMG/OMI_CLIMATE_SST_IST_ARCTIC_area_averaged_anomalies.png" width="300" height="200"/>



# Arctic Sea and Sea Ice Surface Temperature anomaly time series based on reprocessed observations


**Product ID:** OMI_CLIMATE_SST_IST_ARCTIC_area_averaged_anomalies

**Description:** **DEFINITION **

The OMI_CLIMATE_SST_IST_ARCTIC_sst_ist_area_averaged_anomalies product includes time series of monthly mean SST/IST anomalies over the period 1982-2023, relative to the 1991-2020 climatology (30 years), averaged for the Arctic Ocean. The SST/IST Level 4 analysis products that provide the input to the monthly averages are taken from the reprocessed product SEAICE_ARC_PHY_CLIMATE_L4_MY_011_016 with a recent update to include 2023. The product has a spatial resolution of 0.05  degrees in latitude and longitude. 
The OMI time series runs from Jan 1, 1982 to December 31, 2023 and is constructed by calculating monthly average anomalies from the reference climatology from 1991 to 2020, using the daily level 4 SST analysis fields of the SEAICE_ARC_PHY_CLIMATE_L4_MY_011_016 product. The climatological period used is defined according to the WMO recommendation (WMO, 2017) and recent U.S. National Oceanic and Atmospheric Administration practice (https://wmo.int/media/news/updated-30-year-reference-period-reflects-changing-climate,). See the Copernicus Marine Service Ocean State Reports (section 1.1 in Von Schuckmann et al., 2016; section 3 in Von Schuckmann et al., 2018) for more information on the temperature OMI product. The times series of monthly anomalies have been used to calculate the trend in surface temperature (combined SST and IST) using Sen’s method with confidence intervals from the Mann-Kendall test (section 3 in Von Schuckmann et al., 2018).

**CONTEXT**

SST and IST are essential climate variables that act as important input for initializing numerical weather prediction models and fundamental for understanding air-sea interactions and monitoring climate change. Especially in the Arctic, SST/IST feedbacks amplify climate change (AMAP, 2021). In the Arctic Ocean, the surface temperatures play a crucial role for the heat exchange between the ocean and atmosphere, sea ice growth and melt processes (Key et al, 1997) in addition to weather and sea ice forecasts through assimilation into ocean and atmospheric models (Rasmussen et al., 2018). 
The Arctic Ocean is a region that requires special attention regarding the use of satellite SST and IST records and the assessment of climatic variability due to the presence of both seawater and ice, and the large seasonal and inter-annual fluctuations in the sea ice cover which lead to increased complexity in the SST mapping of the Arctic region. Combining SST and ice surface temperature (IST) is identified as the most appropriate method for determining the surface temperature of the Arctic (Minnett et al., 2020).  
Previously, climate trends have been estimated individually for SST and IST records (Bulgin et al., 2020; Comiso and Hall, 2014). However, this is problematic in the Arctic region due to the large temporal variability in the sea ice cover including the overlying northward migration of the ice edge on decadal timescales, and thus, the resulting climate trends are not easy to interpret (Comiso, 2003). A combined surface temperature dataset of the ocean, sea ice and the marginal ice zone (MIZ) provides a consistent climate indicator, which is important for studying climate trends in the Arctic region.

**KEY FINDINGS**

The basin-average trend of SST/IST anomalies for the Arctic Ocean region amounts to 0.104±0.005 °C/year over the period 1982-2023 (42 years) which corresponds to an average warming of 4.37°C. The 2-d map of warming trends indicates these are highest for the Beaufort Sea, Chuckchi Sea, East Siberian Sea, Laptev Sea, Kara Sea and parts of Baffin Bay. The 2d map of Arctic anomalies for 2023 reveals regional peak warming exceeding 6°C.

**DOI (product):** 
https://doi.org/10.48670/mds-00323

**References:**

* AMAP, 2021. Arctic Climate Change Update 2021: Key Trends and Impacts. Summary for Policy-makers. Arctic Monitoring and Assessment Programme (AMAP), Tromsø, Norway.
* Bulgin, C.E., Merchant, C.J., Ferreira, D., 2020. Tendencies, variability and persistence of sea surface temperature anomalies. Sci Rep 10, 7986. https://doi.org/10.1038/s41598-020-64785-9
* Comiso, J.C., 2003. Warming Trends in the Arctic from Clear Sky Satellite Observations. Journal of Climate. https://doi.org/10.1175/1520-0442(2003)016<3498:WTITAF>2.0.CO;2
* Comiso, J.C., Hall, D.K., 2014. Climate trends in the Arctic as observed from space: Climate trends in the Arctic as observed from space. WIREs Clim Change 5, 389–409. https://doi.org/10.1002/wcc.277
* Kendall MG. 1975. Multivariate analysis. London: CharlesGriffin & Co; p. 210, 4
* Key, J.R., Collins, J.B., Fowler, C., Stone, R.S., 1997. High-latitude surface temperature estimates from thermal satellite data. Remote Sensing of Environment 61, 302–309. https://doi.org/10.1016/S0034-4257(97)89497-7
* Minnett, P.J., Kilpatrick, K.A., Podestá, G.P., Evans, R.H., Szczodrak, M.D., Izaguirre, M.A., Williams, E.J., Walsh, S., Reynolds, R.M., Bailey, S.W., Armstrong, E.M., Vazquez-Cuervo, J., 2020. Skin Sea-Surface Temperature from VIIRS on Suomi-NPP—NASA Continuity Retrievals. Remote Sensing 12, 3369. https://doi.org/10.3390/rs12203369
* Rasmussen, T.A.S., Høyer, J.L., Ghent, D., Bulgin, C.E., Dybkjaer, G., Ribergaard, M.H., Nielsen-Englyst, P., Madsen, K.S., 2018. Impact of Assimilation of Sea-Ice Surface Temperatures on a Coupled Ocean and Sea-Ice Model. Journal of Geophysical Research: Oceans 123, 2440–2460. https://doi.org/10.1002/2017JC013481
* Sen PK. 1968. Estimates of the regression coefficient based on Kendall’s tau. J AmStatist Assoc. 63:1379–1389
* von Schuckmann et al., 2016: The Copernicus Marine Environment Monitoring Service Ocean State Report, Journal of Operational Oceanography, Volume 9, 2016 - Issue sup2: The Copernicus Marine Environment Monitoring Service Ocean, http://dx.doi.org/10.1080/1755876X.2016.1273446.
* von Schuckmann, K., Le Traon, P.-Y., Smith, N., Pascual, A., Brasseur, P., Fennel, K., Djavidnia, S., Aaboe, S., Fanjul, E. A., Autret, E., Axell, L., Aznar, R., Benincasa, M., Bentamy, A., Boberg, F., Bourdallé-Badie, R., Nardelli, B. B., Brando, V. E., Bricaud, C., … Zuo, H. (2018). Copernicus Marine Service Ocean State Report. Journal of Operational Oceanography, 11(sup1), S1–S142. https://doi.org/10.1080/1755876X.2018.1489208
* WMO, Guidelines on the Calculation of Climate Normals, 2017, WMO-No-.1203





<img src="https://documentation.marine.copernicus.eu/IMG/OMI_CLIMATE_SST_IST_ARCTIC_trend.png" width="300" height="200"/>



# Arctic Sea and Sea Ice Surface Temperature 2D trend from climatology based on reprocessed observations


**Product ID:** OMI_CLIMATE_SST_IST_ARCTIC_trend

**Description:** **DEFINITION**

The OMI_CLIMATE_sst_ist_ARCTIC_sst_ist_trend product includes the cumulative/net trend in combined sea and ice surface temperature anomalies for the Arctic Ocean from 1982-2023. The cumulative trend is the rate of change (°C/year) scaled by the number of years (42 years). The SST/IST Level 4 analysis that provides the input to the trend calculations are taken from the reprocessed product SEAICE_ARC_PHY_CLIMATE_L4_MY_011_016 with a recent update to include 2023. The product has a spatial resolution of 0.05 degrees in latitude and longitude.

The OMI time series runs from Jan 1, 1982 to December 31, 2023 and is constructed by calculating monthly averages  from the reference climatology defined over the period 1991-2020, according to the WMO recommendation (WMO, 2017) and recent U.S. National Oceanic and Atmospheric Administration practice (https://wmo.int/media/news/updated-30-year-reference-period-reflects-changing-climate), using daily level 4 SST/IST analysis fields of the SEAICE_ARC_PHY_CLIMATE_L4_MY_011_016 product. See the Copernicus Marine Service Ocean State Reports (section 1.1 in Von Schuckmann et al., 2016; section 3 in Von Schuckmann et al., 2018) for more information on the temperature OMI product. The times series of monthly anomalies have been used to calculate the trend in surface temperature (combined SST and IST) using Sen’s method with confidence intervals from the Mann-Kendall test (section 3 in Von Schuckmann et al., 2018).

**CONTEXT**

SST and IST are essential climate variables that act as important input for initializing numerical weather prediction models and fundamental for understanding air-sea interactions and monitoring climate change. Especially in the Arctic, SST/IST feedbacks amplify climate change (AMAP, 2021). In the Arctic Ocean, the surface temperatures play a crucial role for the heat exchange between the ocean and atmosphere, sea ice growth and melt processes (Key et al., 1997) in addition to weather and sea ice forecasts through assimilation into ocean and atmospheric models (Rasmussen et al., 2018). 
The Arctic Ocean is a region that requires special attention regarding the use of satellite SST and IST records and the assessment of climatic variability due to the presence of both seawater and ice, and the large seasonal and inter-annual fluctuations in the sea ice cover which lead to increased complexity in the SST mapping of the Arctic region. Combining SST and ice surface temperature (IST) is identified as the most appropriate method for determining the surface temperature of the Arctic (Minnett et al., 2020).  
Previously, climate trends have been estimated individually for SST and IST records (Bulgin et al., 2020; Comiso and Hall, 2014). However, this is problematic in the Arctic region due to the large temporal variability in the sea ice cover including the overlying northward migration of the ice edge on decadal timescales, and thus, the resulting climate trends are not easy to interpret (Comiso, 2003). A combined surface temperature dataset of the ocean, sea ice and the marginal ice zone (MIZ) provides a consistent climate indicator, which is important for studying climate trends in the Arctic region.

**KEY FINDINGS**

SST/IST trends were calculated for the Arctic Ocean over the period January 1982 to December 2023. The cumulative trends are upwards of 2°C for the greatest part of the Arctic Ocean, with the largest trends occur in the Beaufort Sea, Chuckchi Sea, East Siberian Sea, Laptev Sea, Kara Sea and parts of Baffin Bay. Zero to slightly negative trends are found at the North Atlantic part of the Arctic Ocean. The combined sea and sea ice surface temperature  trend is 0.104+/-0.005°C/yr, i.e. an increase by around 4.37°C between 1982 and 2023. The 2d map of Arctic anomalies reveals regional peak warming exceeding 6°C.

**DOI (product):** 
https://doi.org/10.48670/mds-00324

**References:**

* AMAP, 2021. Arctic Climate Change Update 2021: Key Trends and Impacts. Summary for Policy-makers. Arctic Monitoring and Assessment Programme (AMAP), Tromsø, Norway.
* Bulgin, C.E., Merchant, C.J., Ferreira, D., 2020. Tendencies, variability and persistence of sea surface temperature anomalies. Sci Rep 10, 7986. https://doi.org/10.1038/s41598-020-64785-9
* Comiso, J.C., 2003. Warming Trends in the Arctic from Clear Sky Satellite Observations. Journal of Climate. https://doi.org/10.1175/1520-0442(2003)016<3498:WTITAF>2.0.CO;2
* Comiso, J.C., Hall, D.K., 2014. Climate trends in the Arctic as observed from space: Climate trends in the Arctic as observed from space. WIREs Clim Change 5, 389–409. https://doi.org/10.1002/wcc.277
* Kendall MG. 1975. Multivariate analysis. London: CharlesGriffin & Co; p. 210, 4
* Key, J.R., Collins, J.B., Fowler, C., Stone, R.S., 1997. High-latitude surface temperature estimates from thermal satellite data. Remote Sensing of Environment 61, 302–309. https://doi.org/10.1016/S0034-4257(97)89497-7
* Minnett, P.J., Kilpatrick, K.A., Podestá, G.P., Evans, R.H., Szczodrak, M.D., Izaguirre, M.A., Williams, E.J., Walsh, S., Reynolds, R.M., Bailey, S.W., Armstrong, E.M., Vazquez-Cuervo, J., 2020. Skin Sea-Surface Temperature from VIIRS on Suomi-NPP—NASA Continuity Retrievals. Remote Sensing 12, 3369. https://doi.org/10.3390/rs12203369
* Rasmussen, T.A.S., Høyer, J.L., Ghent, D., Bulgin, C.E., Dybkjaer, G., Ribergaard, M.H., Nielsen-Englyst, P., Madsen, K.S., 2018. Impact of Assimilation of Sea-Ice Surface Temperatures on a Coupled Ocean and Sea-Ice Model. Journal of Geophysical Research: Oceans 123, 2440–2460. https://doi.org/10.1002/2017JC013481
* Sen PK. 1968. Estimates of the regression coefficient based on Kendall’s tau. J AmStatist Assoc. 63:1379–1389
* von Schuckmann et al., 2016: The Copernicus Marine Environment Monitoring Service Ocean State Report, Journal of Operational Oceanography, Volume 9, 2016 - Issue sup2: The Copernicus Marine Environment Monitoring Service Ocean, http://dx.doi.org/10.1080/1755876X.2016.1273446.
* von Schuckmann, K., Le Traon, P.-Y., Smith, N., Pascual, A., Brasseur, P., Fennel, K., Djavidnia, S., Aaboe, S., Fanjul, E. A., Autret, E., Axell, L., Aznar, R., Benincasa, M., Bentamy, A., Boberg, F., Bourdallé-Badie, R., Nardelli, B. B., Brando, V. E., Bricaud, C., … Zuo, H. (2018). Copernicus Marine Service Ocean State Report. Journal of Operational Oceanography, 11(sup1), S1–S142. https://doi.org/10.1080/1755876X.2018.1489208
* WMO, Guidelines on the Calculation of Climate Normals, 2017, WMO-No-.1203





<img src="https://documentation.marine.copernicus.eu/IMG/OMI_CLIMATE_SST_NORTHWESTSHELF_area_averaged_anomalies.png" width="300" height="200"/>



# European North West Shelf Sea Surface Temperature time series and trend from Observations Reprocessing


**Product ID:** OMI_CLIMATE_SST_NORTHWESTSHELF_area_averaged_anomalies

**Description:** **DEFINITION**

The omi_climate_sst_northwestshelf_area_averaged_anomalies product for 2023 includes Sea Surface Temperature (SST) anomalies, given as monthly mean time series starting on 1982 and averaged over the European North West Shelf Seas. The NORTHWESTSHELF SST OMI is built from the CMEMS Reprocessed European North West Shelf Iberai-Biscay-Irish areas(SST_MED_SST_L4_REP_OBSERVATIONS_010_026, see e.g. the OMI QUID, http://marine.copernicus.eu/documents/QUID/CMEMS-OMI-QUID-CLIMATE-SST- NORTHWESTSHELF_v3.pdf), which provided the SSTs used to compute the evolution of SST anomalies over the European North West Shelf Seas. This reprocessed product consists of daily (nighttime) interpolated 0.05° grid resolution SST maps over the European North West Shelf Iberai-Biscay-Irish Seas built from the ESA Climate Change Initiative (CCI) (Merchant et al., 2019) and Copernicus Climate Change Service (C3S) initiatives. Anomalies are computed against the 1991-2020 reference period. 

**CONTEXT**

Sea surface temperature (SST) is a key climate variable since it deeply contributes in regulating climate and its variability (Deser et al., 2010). SST is then essential to monitor and characterise the state of the global climate system (GCOS 2010). Long-term SST variability, from interannual to (multi-)decadal timescales, provides insight into the slow variations/changes in SST, i.e. the temperature trend (e.g., Pezzulli et al., 2005). In addition, on shorter timescales, SST anomalies become an essential indicator for extreme events, as e.g. marine heatwaves (Hobday et al., 2018). 

**CMEMS KEY FINDINGS **

The overall trend in the SST anomalies in this region is 0.024 ±0.002 °C/year over the period 1982-2023.  

**Figure caption**

Time series of monthly mean and 24-month filtered sea surface temperature anomalies in the European North West Shelf Seas during the period 1982-2023. Anomalies are relative to the climatological period 1991-2020 and built from the CMEMS SST_ATL_SST_L4_REP_OBSERVATIONS_010_026 satellite product (see e.g. the OMI QUID, http://marine.copernicus.eu/documents/QUID/CMEMS-OMI-QUID-NORTHWESTSHELF-SST.pdf). The sea surface temperature trend with its 95% confidence interval (shown in the box) is estimated by using the X-11 seasonal adjustment procedure (e.g. Pezzulli et al., 2005) and Sen’s method (Sen 1968).

**DOI (product):** 
https://doi.org/10.48670/moi-00275

**References:**

* Deser, C., Alexander, M. A., Xie, S.-P., Phillips, A. S., 2010. Sea Surface Temperature Variability: Patterns and Mechanisms. Annual Review of Marine Science 2010 2:1, 115-143. https://doi.org/10.1146/annurev-marine-120408-151453
* GCOS. Global Climate Observing System. 2010. Update of the Implementation Plan for the Global Observing System for Climate in Support of the UNFCCC (GCO-138).
* Hobday, A. J., Oliver, E. C., Gupta, A. S., Benthuysen, J. A., Burrows, M. T., Donat, M. G., ... & Smale, D. A. (2018). Categorizing and naming marine heatwaves. Oceanography, 31(2), 162-173.
* Merchant, C. J., Embury, O., Bulgin, C. E., Block, T., Corlett, G. K., Fiedler, E., ... & Eastwood, S. (2019). Satellite-based time-series of sea-surface temperature since 1981 for climate applications. Scientific data, 6(1), 1-18.
* Pezzulli, S., Stephenson, D. B., Hannachi, A., 2005. The Variability of Seasonality. J. Climate. 18:71–88. doi:10.1175/JCLI-3256.1.
* Sen, P. K., 1968. Estimates of the regression coefficient based on Kendall’s tau. J Am Statist Assoc. 63:1379–1389.





<img src="https://documentation.marine.copernicus.eu/IMG/OMI_CLIMATE_SST_NORTHWESTSHELF_trend.png" width="300" height="200"/>



# European North West Shelf Sea Surface Temperature trend map from Observations Reprocessing


**Product ID:** OMI_CLIMATE_SST_NORTHWESTSHELF_trend

**Description:** **DEFINITION**

The omi_climate_sst_northwestshelf_trend product includes the Sea Surface Temperature (SST) trend for the European North West Shelf Seas over the period 1982-2023, i.e. the rate of change (°C/year). This OMI is derived from the CMEMS REP ATL L4 SST product (SST_ATL_SST_L4_REP_OBSERVATIONS_010_026), see e.g. the OMI QUID, http://marine.copernicus.eu/documents/QUID/CMEMS-OMI-QUID-CLIMATE-SST-NORTHWESTSHELF_v3.pdf), which provided the SSTs used to compute the SST trend over the European North West Shelf Seas. This reprocessed product consists of daily (nighttime) interpolated 0.05° grid resolution SST maps built from the ESA Climate Change Initiative (CCI) (Merchant et al., 2019) and Copernicus Climate Change Service (C3S) initiatives. Trend analysis has been performed by using the X-11 seasonal adjustment procedure (see e.g. Pezzulli et al., 2005), which has the effect of filtering the input SST time series acting as a low bandpass filter for interannual variations. Mann-Kendall test and Sens’s method (Sen 1968) were applied to assess whether there was a monotonic upward or downward trend and to estimate the slope of the trend and its 95% confidence interval. 

**CONTEXT **

Sea surface temperature (SST) is a key climate variable since it deeply contributes in regulating climate and its variability (Deser et al., 2010). SST is then essential to monitor and characterise the state of the global climate system (GCOS 2010). Long-term SST variability, from interannual to (multi-)decadal timescales, provides insight into the slow variations/changes in SST, i.e. the temperature trend (e.g., Pezzulli et al., 2005). In addition, on shorter timescales, SST anomalies become an essential indicator for extreme events, as e.g. marine heatwaves (Hobday et al., 2018). 

**CMEMS KEY FINDINGS **

Over the period 1982-2023, the European North West Shelf Seas mean Sea Surface Temperature (SST) increased at a rate of 0.024 ± 0.002 °C/Year.

**Figure caption**

Sea surface temperature trend over the period 1982-2023 in the European North West Shelf Seas. The trend is the rate of change (°C/year). The trend map in sea surface temperature is derived from the CMEMS SST_ATL_SST_L4_REP_OBSERVATIONS_010_026 product (see e.g. the OMI QUID, http://marine.copernicus.eu/documents/QUID/CMEMS-OMI-QUID-ATL-SST.pdf). The trend is estimated by using the X-11 seasonal adjustment procedure (e.g. Pezzulli et al., 2005;) and Sen’s method (Sen 1968).

**DOI (product):** 
https://doi.org/10.48670/moi-00276

**References:**

* Deser, C., Alexander, M. A., Xie, S.-P., Phillips, A. S., 2010. Sea Surface Temperature Variability: Patterns and Mechanisms. Annual Review of Marine Science 2010 2:1, 115-143. https://doi.org/10.1146/annurev-marine-120408-151453
* GCOS. Global Climate Observing System. 2010. Update of the Implementation Plan for the Global Observing System for Climate in Support of the UNFCCC (GCO-138).
* Hobday, A. J., Oliver, E. C., Gupta, A. S., Benthuysen, J. A., Burrows, M. T., Donat, M. G., ... & Smale, D. A. (2018). Categorizing and naming marine heatwaves. Oceanography, 31(2), 162-173.
* Merchant, C. J., Embury, O., Bulgin, C. E., Block, T., Corlett, G. K., Fiedler, E., ... & Eastwood, S. (2019). Satellite-based time-series of sea-surface temperature since 1981 for climate applications. Scientific data, 6(1), 1-18.
* Pezzulli, S., Stephenson, D. B., Hannachi, A., 2005. The Variability of Seasonality. J. Climate. 18:71–88. doi:10.1175/JCLI-3256.1.
* Sen, P. K., 1968. Estimates of the regression coefficient based on Kendall’s tau. J Am Statist Assoc. 63:1379–1389.





<img src="https://documentation.marine.copernicus.eu/IMG/OMI_EXTREME_CLIMVAR_PACIFIC_npgo_sla_eof_mode_projection.png" width="300" height="200"/>



# North Pacific Gyre Oscillation from Observations Reprocessing


**Product ID:** OMI_EXTREME_CLIMVAR_PACIFIC_npgo_sla_eof_mode_projection

**Description:** **DEFINITION**

The North Pacific Gyre Oscillation (NPGO) is a climate pattern introduced by Di Lorenzo et al. (2008) and further reported by Tranchant et al. (2019) in the CMEMS Ocean State Report #3. The NPGO is defined as the second dominant mode of variability of Sea Surface Height (SSH) anomaly and SST anomaly in the Northeast Pacific (25°– 62°N, 180°– 250°E). The spatial and temporal pattern of the NPGO has been deduced over the [1950-2004] period using an empirical orthogonal function (EOF) decomposition on sea level and sea surface temperature fields produced by the Regional Ocean Modeling System (ROMS) (Di Lorenzo et al., 2008; Shchepetkin and McWilliams, 2005). Afterward, the sea level spatial pattern of the NPGO is used/projected with satellite altimeter delayed-time sea level anomalies to calculate and update the NPGO index. 

The NPGO index disseminated on CMEMS was specifically updated from 2004 onward using up-to-date altimeter products (DT2021 version; SEALEVEL_GLO_PHY_L4_MY _008_047 CMEMS product, including “my” & “myint” datasets, and the near-real time SEALEVEL_GLO_PHY_L4_NRT _008_046 CMEMS product). Users that previously used the index disseminated on www.o3d.org/npgo/ web page will find slight differences induced by this update. The change in the reprocessed version (previously DT-2018) and the extension of the mean value of the SSH anomaly (now 27 years, previously 20 years) induce some slight changes not impacting the general variability of the NPGO.   

""CONTEXT ""

NPGO mode emerges as the leading mode of decadal variability for surface salinity and upper ocean nutrients (Di Lorenzo et al., 2009). The North Pacific Gyre Oscillation (NPGO) term is used because its fluctuations reflect changes in the intensity of the central and eastern branches of the North Pacific gyres circulations (Chhak et al., 2009). This index measures change in the North Pacific gyres circulation and explains key physical-biological ocean variables including temperature, salinity, sea level, nutrients, chlorophyll-a. A positive North Pacific Gyre Oscillation phase is a dipole pattern with negative SSH anomaly north of 40°N and the opposite south of 40°N. (Di Lorenzo et al., 2008) suggested that the North Pacific Gyre Oscillation is the oceanic expression of the atmospheric variability of the North Pacific Oscillation (Walker and Bliss, 1932), which has an expression in both the 2nd EOFs of SSH and Sea Surface Temperature (SST) anomalies (Ceballos et al., 2009). This finding is further supported by the recent work of (Yi et al., 2018) showing consistent pattern features between the atmospheric North Pacific Oscillation and the oceanic North Pacific Gyre Oscillation in the Coupled Model Intercomparison Project Phase 5 (CMIP5) database. 

""CMEMS KEY FINDINGS"" 

The NPGO index is presently in a negative phase, associated with a positive SSH anomaly north of 40°N and negative south of 40°N. This reflects a reduced amplitude of the central and eastern branches of the North Pacific gyre, corresponding to a reduced coastal upwelling and thus a lower sea surface salinity and concentration of nutrients.  

**DOI (product):** 
https://doi.org/10.48670/moi-00221

**References:**

* Ceballos, L. I., E. Di Lorenzo, C. D. Hoyos, N. Schneider and B. Taguchi, 2009: North Pacific Gyre Oscillation Synchronizes Climate Fluctuations in the Eastern and Western Boundary Systems. Journal of Climate, 22(19) 5163-5174, doi:10.1175/2009jcli2848.1
* Chhak, K. C., E. Di Lorenzo, N. Schneider and P. F. Cummins, 2009: Forcing of Low-Frequency Ocean Variability in the Northeast Pacific. Journal of Climate, 22(5) 1255-1276, doi:10.1175/2008jcli2639.1.
* Di Lorenzo, E., Fiechter, J., Schneider, N., Bracco, A., Miller, A.J., Franks, P.J.S., Bograd, S.J., Moore, A.M., Thomas, A.C., Crawford, W., Peña, A., Hermann, A.J., 2009. Nutrient and salinity decadal variations in the central and eastern North Pacific. Geophys. Res. Lett. 36. https://doi.org/10.1029/2009GL038261
* Di Lorenzo, E., Schneider, N., Cobb, K.M., Franks, P.J.S., Chhak, K., Miller, A.J., McWilliams, J.C., Bograd, S.J., Arango, H., Curchitser, E., Powell, T.M., Rivière, P., 2008. North Pacific Gyre Oscillation links ocean climate and ecosystem change. Geophys. Res. Lett. 35. https://doi.org/10.1029/2007GL032838
* Shchepetkin, A.F., McWilliams, J.C., 2005. The regional oceanic modeling system (ROMS): a split-explicit, free-surface, topography-following-coordinate oceanic model. Ocean Model. 9, 347–404. https://doi.org/10.1016/j.ocemod.2004.08.002
* Tranchant, B., Pujol, I., Di Lorenzo, E., Legeais, J.-F., 2019. The North Pacific Gyre Oscillation. J. Oper. Oceanogr., Copernicus Marine Service Ocean State Report Issue 3, s29–s31. https://doi.org/10.1080/ 1755876X.2019.1633075
* Yi, D.L., Gan, B., Wu, L., Miller, A.J., 2018. The North Pacific Gyre Oscillation and Mechanisms of Its Decadal Variability in CMIP5 Models. J. Clim. 31, 2487–2509. https://doi.org/10.1175/JCLI-D-17-0344.1





<img src="https://documentation.marine.copernicus.eu/IMG/ARCTIC_OMI_EXTREME_MHW_area_averaged_anomalies.png" width="300" height="200"/>



# Marine Heatwave Index in the Barents Sea from Reanalysis


**Product ID:** OMI_EXTREME_MHW_ARCTIC_area_averaged_anomalies

**Description:** **DEFINITION**

Temperature deviation from the 30-year (1991-2020) daily climatological mean temperature for the Barents Sea region (68°N - 80°N, 18°E - 55°E), relative to the difference between the daily climatological average and the 90th percentile above the climatological mean. Thus, when the index is above 1 the area is in a state of a marine heatwave, and when the index is below -1 the area is in a state of a marine cold spell, following the definition by Hobday et al. (2016). For further details, see Lien et al. (2024).""

**CONTEXT**
Anomalously warm oceanic events, often termed marine heatwaves, can potentially impact the ecosystem in the affected region. The marine heatwave concept and terminology was systemized by Hobday et al. (2016), and a generally adopted definition of a marine heatwave is a period of more than five days where the temperature within a region exceeds the 90th percentile of the seasonally varying climatological average temperature for that region. The Barents Sea region has warmed considerably during the most recent climatological average period (1991-2020) due to a combination of climate warming and positive phase of regional climate variability (e.g., Lind et al., 2018 ; Skagseth et al., 2020 ; Smedsrud et al., 2022), with profound consequences for marine life where boreal species are moving northward at the expense of arctic species (e.g., Fossheim et al., 2015; Oziel et al., 2020;  Husson et al., 2022).

**KEY FINDINGS**

There is a clear tendency of reduced frequency and intensity of marine cold spells, and a tendency towards increased frequency and intensity of marine heat waves in the Barents Sea. 

**DOI (product):** 
https://doi.org/10.48670/mds-00346

**References:**

* Fossheim M, Primicerio R, Johannesen E, Ingvaldsen RB, Aschan MM, Dolgov AV. 2015. Recent warming leads to a rapid borealization of fish communities in the Arctic. Nature Clim Change. doi:10.1038/nclimate2647
* Hobday AJ, Alexander LV, Perkins SE, Smale DA, Straub SC, Oliver ECJ, Benthuysen JA, Burrows MT, Donat MG, Feng M, Holbrook NJ, Moore PJ, Scannell HA, Gupta AS, Wernberg T. 2016. A hierarchical approach to defining marine heatwaves. Progr. Oceanogr., 141, 227-238
* Husson B, Lind S, Fossheim M, Kato-Solvang H, Skern-Mauritzen M, Pécuchet L, Ingvaldsen RB, Dolgov AV, Primicerio R. 2022. Successive extreme climatic events lead to immediate, large-scale, and diverse responses from fish in the Arctic. Global Change Biol, 28, 3728-3744
* Lien VS, Raj RP, Chatterjee S. 2024. Surface and bottom marine heatwave characteristics in the Barents Sea: a model study. State of the Planet (in press)
* Lind S, Ingvaldsen RB, Furevik T. 2018. Arctic warming hotspot in the northern Barents Sea linked to declining sea-ice import. Nat Clim Change, 8, 634-639
* Oziel L, Baudena A, Ardyna M, Massicotte P, Randelhoff A, Sallee J-B, Ingvaldsen RB, Devred E, Babin M. 2020. Faster Atlantic currents drive poleward expansion of temperate phytoplankton in the Arctic Ocean. Nat Commun., 11(1), 1705, doi:10.1038/s41467-020-15485-5
* Skagseth Ø, Eldevik T, Årthun M, Asbjørnsen H, Lien VS, Smedsrud LH. 2020. Reduced efficiency of the Barents Sea cooling machine. Nat Clim Change, doi.org/10.1038/s41558-020-0772-6
* Smedsrud LH, Muilwijk M, Brakstad A, Madonna E, Lauvset SK, Spensberger C, Born A, Eldevik T, Drange H, Jeansson E, Li C, Olsen A, Skagseth Ø, Slater DA, Straneo F, Våge K, Årthun M. 2022.
* Nordic Seas heat loss, Atlantic inflow, and Arctic sea ice cover over the last century. Rev Geophys., 60, e2020RG000725





<img src="https://documentation.marine.copernicus.eu/IMG/OMI_EXTREME_SEASTATE_GLOBAL_swh_mean_and_P95_obs.png" width="300" height="200"/>



# Global Ocean, extreme and mean significant wave height trends from satellite observations - seasonal trends


**Product ID:** OMI_EXTREME_SEASTATE_GLOBAL_swh_mean_and_P95_obs

**Description:** **DEFINITION**

Significant wave height (SWH), expressed in metres, is the average height of the highest one-third of waves. This OMI provides time series of seasonal mean and extreme SWH values in three oceanic regions as well as their trends from 2002 to 2020, computed from the reprocessed global L4 SWH product (WAVE_GLO_PHY_SWH_L4_MY_014_007). The extreme SWH is defined as the 95th percentile of the daily maximum of SWH over the chosen period and region. The 95th percentile represents the value below which 95% of the data points fall, indicating higher wave heights than usual. The mean and the 95th percentile of SWH are calculated for two seasons of the year to take into account the seasonal variability of waves (January, February, and March, and July, August, and September) and are in m while the trends are in cm/yr.

**CONTEXT**

Grasping the nature of global ocean surface waves, their variability, and their long-term interannual shifts is essential for climate research and diverse oceanic and coastal applications. The sixth IPCC Assessment Report underscores the significant role waves play in extreme sea level events (Mentaschi et al., 2017), flooding (Storlazzi et al., 2018), and coastal erosion (Barnard et al., 2017). Additionally, waves impact ocean circulation and mediate interactions between air and sea (Donelan et al., 1997) as well as sea-ice interactions (Thomas et al., 2019). Studying these long-term and interannual changes demands precise time series data spanning several decades. Until now, such records have been available only from global model reanalyses or localised in situ observations. While buoy data are valuable, they offer limited local insights and are especially scarce in the southern hemisphere. In contrast, altimeters deliver global, high-quality measurements of significant wave heights (SWH) (Gommenginger et al., 2002). The growing satellite record of SWH now facilitates more extensive global and long-term analyses. By using SWH data from a multi-mission altimetric product from 2002 to 2020, we can calculate global mean SWH and extreme SWH and evaluate their trends.

**KEY FINDINGS**

Over the period from 2002 to 2020, positive trends in both Significant Wave Height (SWH) and extreme SWH are mostly found in the southern hemisphere. The 95th percentile of wave heights (q95), increases more rapidly than the average values, indicating that extreme waves are growing faster than the average wave height. In the North Atlantic, SWH has increased in summertime (July August September) and decreased during the wintertime: the trend for the 95th percentile SWH is decreasing by 2.1 ± 3.3 cm/year, while the mean SWH shows a decreasing trend of 2.2 ± 1.76 cm/year. In the south of Australia, in boreal winter, the 95th percentile SWH is increasing at a rate of 2.6 ± 1.5 cm/year (a), with the mean SWH increasing by 0.7 ± 0.64 cm/year (b). Finally, in the Antarctic Circumpolar Current, also in boreal winter, the 95th percentile SWH trend is 3.2 ± 2.15 cm/year (a) and the mean SWH trend is 1.4 ± 0.82 cm/year (b). This variation highlights that waves evolve differently across different basins and seasons, illustrating the complex and region-specific nature of wave height trends. A full discussion regarding this OMI can be found in A. Laloue et al. (2024).

**DOI (product):** 
https://doi.org/10.48670/mds-00352

**References:**

* Barnard, P. L., Hoover, D., Hubbard, D. M., Snyder, A., Ludka, B. C., Allan, J., Kaminsky, G. M., Ruggiero, P., Gallien, T. W., Gabel, L., McCandless, D., Weiner, H. M., Cohn, N., Anderson, D. L., and Serafin, K. A.: Extreme oceanographic forcing and coastal response due to the 2015–2016 El Niño, Nature Communications, 8, https://doi.org/10.1038/ncomms14365, 2017.
* Donelan, M. A., Drennan, W. M., and Katsaros, K. B.: The air–sea momentum flux in conditions of wind sea and swell, Journal of Physical Oceanography, 27, 2087–2099, https://doi.org/10.1175/1520-0485(1997)0272.0.co;2, 1997.
* Mentaschi, L., Vousdoukas, M. I., Voukouvalas, E., Dosio, A., and Feyen, L.: Global changes of extreme coastal wave energy fluxes triggered by intensified teleconnection patterns, Geophysical Research Letters, 44, 2416–2426, https://doi.org/10.1002/2016gl072488, 2017
* Thomas, S., Babanin, A. V., Walsh, K. J. E., Stoney, L., and Heil, P.: Effect of wave-induced mixing on Antarctic sea ice in a high-resolution ocean model, Ocean Dynamics, 69, 737–746, https://doi.org/10.1007/s10236-019-01268-0, 2019.
* Gommenginger, C. P., Srokosz, M. A., Challenor, P. G., and Cotton, P. D.: Development and validation of altimeter wind speed algorithms using an extended collocated Buoy/Topex dataset, IEEE Transactions on Geoscience and Remote Sensing, 40, 251–260, https://doi.org/10.1109/36.992782, 2002.
* Storlazzi, C. D., Gingerich, S. B., van Dongeren, A., Cheriton, O. M., Swarzenski, P. W., Quataert, E., Voss, C. I., Field, D. W., Annamalai, H., Piniak, G. A., and McCall, R.: Most atolls will be uninhabitable by the mid-21st century because of sea level rise exacerbating wave-driven flooding, Science Advances, 4, https://doi.org/10.1126/sciadv.aap9741, 2018.
* Husson, R., Charles, E.: EU Copernicus Marine Service Product User Manual for the Global Ocean L 4 Significant Wave Height From Reprocessed Satellite Measurements Product, WAVE_GLO_PHY_SWH_L4_MY_014_007, Issue 2.0, Mercator Ocean International, https://documentation.marine.copernicus.eu/PUM/CMEMS-WAV-PUM-014-005-006-007.pdf, last access: 21 July 2023, 2021 Laloue, A., Ghantous, M., Faugère, Y., Dalphinet. A., Aouf, L.: Statistical analysis of global ocean significant wave heights from satellite altimetry over the past two decades. OSR-8 (under review)





<img src="https://documentation.marine.copernicus.eu/IMG/OMI_EXTREME_SL_BALTIC_slev_mean_and_anomaly_obs.png" width="300" height="200"/>



# Baltic Sea sea level extreme variability mean and anomaly (observations)


**Product ID:** OMI_EXTREME_SL_BALTIC_slev_mean_and_anomaly_obs

**Description:** **DEFINITION**

The OMI_EXTREME_SL_BALTIC_slev_mean_and_anomaly_obs indicator is based on the computation of the 99th and the 1st percentiles from in situ data (observations). It is computed for the variable sea level measured by tide gauges along the coast. The use of percentiles instead of annual maximum and minimum values, makes this extremes study less affected by individual data measurement errors. The annual percentiles referred to annual mean sea level are temporally averaged and their spatial evolution is displayed in the dataset omi_extreme_sl_baltic_slev_mean_and_anomaly_obs, jointly with the anomaly in the target year. This study of extreme variability was first applied to sea level variable (Pérez Gómez et al 2016) and then extended to other essential variables, sea surface temperature and significant wave height (Pérez Gómez et al 2018).

**CONTEXT**
Sea level (SLEV) is one of the Essential Ocean Variables most affected by climate change. Global mean sea level rise has accelerated since the 1990’s (Abram et al., 2019, Legeais et al., 2020), due to the increase of ocean temperature and mass volume caused by land ice melting (WCRP, 2018). Basin scale oceanographic and meteorological features lead to regional variations of this trend that combined with changes in the frequency and intensity of storms could also rise extreme sea levels up to one meter by the end of the century (Vousdoukas et al., 2020, Tebaldi et al., 2021). This will significantly increase coastal vulnerability to storms, with important consequences on the extent of flooding events, coastal erosion and damage to infrastructures caused by waves (Boumis et al., 2023). The increase in extreme sea levels over recent decades is, therefore, primarily due to the rise in mean sea level. Note, however, that the methodology used to compute this OMI removes the annual 50th percentile, thereby discarding the mean sea level trend to isolate changes in storminess.   
The Baltic Sea is affected by vertical land motion due to the Glacial Isostatic Adjustment (Ludwigsen et al., 2020) and consequently relative sea level trends (as measured by tide gauges) have been shown to be strongly negative, especially in the northern part of the basin. On the other hand, Baltic Sea absolute sea level trends (from altimetry-based observations) show statistically significant positive trends (Passaro et al., 2021).

**COPERNICUS MARINE SERVICE KEY FINDINGS**
Up to 45 stations fulfill the completeness index criteria in this region, a few less than in 2020 (51). The spatial variation of the mean 99th percentiles follow the tidal range pattern, reaching its highest values in the northern end of the Gulf of Bothnia (e.g.: 0.81 and 0.78 m above mean sea level at the Finnish stations Kemi and Oulu, respectively) and the inner part of the Gulf of Finland (e.g.: 0.83 m above mean sea level in St. Petersburg,  Russia). Smaller tides and therefore 99th percentiles are found along the southeastern coast of Sweden, between Stockholm and Gotland Island (e.g.: 0.42 m above mean sea level in Visby, Gotland Island-Sweden). Annual 99th percentiles standard deviation ranges between 3-5 cm in the South (e.g.: 3 cm in Korsor, Denmark) to 10-13 cm in the Gulf of Finland (e.g.: 12 cm in Hamina).  Negative anomalies of 2022 99th percentile are observed in the northern part of the basin, in the Gulf of Bothnia, in the inner part of the Gulf of Finland and in Lolland Island stations (Denmark) reaching maximum values of -12 cm in Kemi, -9 cm in St. Petersburg and -8 cm in Rodby, respectively.. Positive anomalies of 2022 99th percentile are however found in the central and southeastern parts of the basin, with maximum values reaching 7 cm in Paldisky (Estonia) and Slipshavn (Denmark).  

**DOI (product):** 
https://doi.org/10.48670/moi-00203

**References:**

* Abram, N., Gattuso, J.-P., Prakash, A., Cheng, L., Chidichimo, M. P., Crate, S., Enomoto, H., Garschagen, M., Gruber, N., Harper, S., Holland, E., Kudela, R. M., Rice, J., Steffen, K., & von Schuckmann, K. (2019). Framing and Context of the Report. In H. O. Pörtner, D. C. Roberts, V. Masson-Delmotte, P. Zhai, M. Tignor, E. Poloczanska, K. Mintenbeck, A. Alegría, M. Nicolai, A. Okem, J. Petzold, B. Rama, & N. M. Weyer (Eds.), IPCC Special Report on the Ocean and Cryosphere in a Changing Climate (pp. 73–129). in press. https://www.ipcc.ch/srocc/
* Legeais J-F, W. Llowel, A. Melet and B. Meyssignac: Evidence of the TOPEX-A Altimeter Instrumental Anomaly and Acceleration of the Global Mean Sea Level, in Copernicus Marine Service Ocean State Report, Issue 4, Journal of Operational Oceanography, 2020, accepted.
* Pérez-Gómez B, Álvarez-Fanjul E, She J, Pérez-González I, Manzano F. 2016. Extreme sea level events, Section 4.4, p:300. In: Von Schuckmann K, Le Traon PY, Alvarez-Fanjul E, Axell L, Balmaseda M, Breivik LA, Brewin RJW, Bricaud C, Drevillon M, Drillet Y, Dubois C , Embury O, Etienne H, García-Sotillo M, Garric G, Gasparin F, Gutknecht E, Guinehut S, Hernandez F, Juza M, Karlson B, Korres G, Legeais JF, Levier B, Lien VS, Morrow R, Notarstefano G, Parent L, Pascual A, Pérez-Gómez B, Perruche C, Pinardi N, Pisano A, Poulain PM , Pujol IM, Raj RP, Raudsepp U, Roquet H, Samuelsen A, Sathyendranath S, She J, Simoncelli S, Solidoro C, Tinker J, Tintoré J, Viktorsson L, Ablain M, Almroth-Rosell E, Bonaduce A, Clementi E, Cossarini G, Dagneaux Q, Desportes C, Dye S, Fratianni C, Good S, Greiner E, Gourrion J, Hamon M, Holt J, Hyder P, Kennedy J, Manzano-Muñoz F, Melet A, Meyssignac B, Mulet S, Nardelli BB, O’Dea E, Olason E, Paulmier A, Pérez-González I, Reid R, Racault MF, Raitsos DE, Ramos A, Sykes P, Szekely T, Verbrugge N. 2016. The Copernicus Marine Environment Monitoring Service Ocean State Report, Journal of Operational Oceanography. 9 (sup2): 235-320. http://dx.doi.org/10.1080/1755876X.2016.1273446
* Pérez Gómez B, De Alfonso M, Zacharioudaki A, Pérez González I, Álvarez Fanjul E, Müller M, Marcos M, Manzano F, Korres G, Ravdas M, Tamm S. 2018. Sea level, SST and waves: extremes variability. In: Copernicus Marine Service Ocean State Report, Issue 2, Journal of Operational Oceanography, 11:sup1, Chap. 3.1, s79–s88, DOI: https://doi.org/10.1080/1755876X.2018.1489208.
* Vousdoukas MI, Mentaschi L, Hinkel J, et al. 2020. Economic motivation for raising coastal flood defenses in Europe. Nat Commun 11, 2119 (2020). https://doi.org/10.1038/s41467-020-15665-3.
* Tebaldi, C., Ranasinghe, R., Vousdoukas, M. et al. 2021. Extreme sea levels at different global warming levels. Nat. Clim. Chang. 11, 746–751. https://doi.org/10.1038/s41558-021-01127-1. Tebaldi, C., Ranasinghe, R., Vousdoukas, M. et al. Author Correction: Extreme sea levels at different global warming levels. Nat. Clim. Chang. 13, 588 (2023). https://doi.org/10.1038/s41558-023-01665-w.
* Boumis, G., Moftakhari, H. R., & Moradkhani, H. 2023. Coevolution of extreme sea levels and sea-level rise under global warming. Earth's Future, 11, e2023EF003649. https://doi. org/10.1029/2023EF003649.
* Passaro M, Müller F L, Oelsmann J, Rautiainen L, Dettmering D, Hart-Davis MG, Abulaitijiang A, Andersen, OB, Høyer JL, Madsen, KS, Ringgaard IM, Särkkä J, Scarrott R, Schwatke C, Seitz F, Tuomi L, Restano M, and Benveniste J. 2021. Absolute Baltic Sea Level Trends in the Satellite Altimetry Era: A Revisit, Front Mar Sci, 8, 647607, https://doi.org/10.3389/FMARS.2021.647607/BIBTEX.





<img src="https://documentation.marine.copernicus.eu/IMG/OMI_EXTREME_SL_IBI_slev_mean_and_anomaly_obs.png" width="300" height="200"/>



# Iberia Biscay Ireland sea level extreme variability mean and anomaly (observations)


**Product ID:** OMI_EXTREME_SL_IBI_slev_mean_and_anomaly_obs

**Description:** **DEFINITION**

The OMI_EXTREME_SL_IBI_slev_mean_and_anomaly_obs indicator is based on the computation of the 99th and the 1st percentiles from in situ data (observations). It is computed for the variable sea level measured by tide gauges along the coast. The use of percentiles instead of annual maximum and minimum values, makes this extremes study less affected by individual data measurement errors. The annual percentiles referred to annual mean sea level are temporally averaged and their spatial evolution is displayed in the dataset omi_extreme_sl_ibi_slev_mean_and_anomaly_obs, jointly with the anomaly in the target year. This study of extreme variability was first applied to sea level variable (Pérez Gómez et al 2016) and then extended to other essential variables, sea surface temperature and significant wave height (Pérez Gómez et al 2018).

**CONTEXT**

Sea level (SLEV) is one of the Essential Ocean Variables most affected by climate change. Global mean sea level rise has accelerated since the 1990’s (Abram et al., 2019, Legeais et al., 2020), due to the increase of ocean temperature and mass volume caused by land ice melting (WCRP, 2018). Basin scale oceanographic and meteorological features lead to regional variations of this trend that combined with changes in the frequency and intensity of storms could also rise extreme sea levels up to one meter by the end of the century (Vousdoukas et al., 2020, Tebaldi et al., 2021). This will significantly increase coastal vulnerability to storms, with important consequences on the extent of flooding events, coastal erosion and damage to infrastructures caused by waves (Boumis et al., 2023). The increase in extreme sea levels over recent decades is, therefore, primarily due to the rise in mean sea level. Note, however, that the methodology used to compute this OMI removes the annual 50th percentile, thereby discarding the mean sea level trend to isolate changes in storminess.   
The Iberian Biscay Ireland region shows positive sea level trend modulated by decadal-to-multidecadal variations driven by ocean dynamics and superposed to the long-term trend (Chafik et al., 2019).

**COPERNICUS MARINE SERVICE KEY FINDINGS**

The completeness index criteria is fulfilled by 57 stations in 2021, two more than those available in 2021 (55), recently added to the multi-year product INSITU_GLO_PHY_SSH_DISCRETE_MY_013_053. The mean 99th percentiles reflect the great tide spatial variability around the UK and the north of France. Minimum values are observed in the Irish eastern coast (e.g.: 0.66 m above mean sea level in Arklow Harbour) and the Canary Islands (e.g.: 0.93 and 0.96 m above mean sea level in Gomera and Hierro, respectively). Maximum values are observed in the Bristol and English Channels (e.g.: 6.26, 5.58 and 5.17 m above mean sea level in Newport, St. Malo and St. Helier, respectively). The annual 99th percentiles standard deviation reflects the south-north increase of storminess, ranging between 1-2 cm in the Canary Islands to 12 cm in Newport (Bristol Channel). Although less pronounced and general than in 2021, negative or close to zero anomalies of 2022 99th percentile still prevail throughout the region this year reaching up to -14 cm in St.Helier (Jersey Island, Channel Islands), or -12 cm in St. Malo. Positive anomalies of 2022 99th percentile are found in the northern part of the region (Irish eastern coast and west Scotland coast) and at a couple of stations in Southern England, with values reaching 9 cm in Bangor (Northern Ireland) and 6 cm in Portsmouth (South England). 

**DOI (product):** 
https://doi.org/10.48670/moi-00253

**References:**

* Abram, N., Gattuso, J.-P., Prakash, A., Cheng, L., Chidichimo, M. P., Crate, S., Enomoto, H., Garschagen, M., Gruber, N., Harper, S., Holland, E., Kudela, R. M., Rice, J., Steffen, K., & von Schuckmann, K. (2019). Framing and Context of the Report. In H. O. Pörtner, D. C. Roberts, V. Masson-Delmotte, P. Zhai, M. Tignor, E. Poloczanska, K. Mintenbeck, A. Alegría, M. Nicolai, A. Okem, J. Petzold, B. Rama, & N. M. Weyer (Eds.), IPCC Special Report on the Ocean and Cryosphere in a Changing Climate (pp. 73–129). in press. https://www.ipcc.ch/srocc/
* Legeais J-F, W. Llowel, A. Melet and B. Meyssignac: Evidence of the TOPEX-A Altimeter Instrumental Anomaly and Acceleration of the Global Mean Sea Level, in Copernicus Marine Service Ocean State Report, Issue 4, Journal of Operational Oceanography, 2020, accepted.
* Pérez-Gómez B, Álvarez-Fanjul E, She J, Pérez-González I, Manzano F. 2016. Extreme sea level events, Section 4.4, p:300. In: Von Schuckmann K, Le Traon PY, Alvarez-Fanjul E, Axell L, Balmaseda M, Breivik LA, Brewin RJW, Bricaud C, Drevillon M, Drillet Y, Dubois C , Embury O, Etienne H, García-Sotillo M, Garric G, Gasparin F, Gutknecht E, Guinehut S, Hernandez F, Juza M, Karlson B, Korres G, Legeais JF, Levier B, Lien VS, Morrow R, Notarstefano G, Parent L, Pascual A, Pérez-Gómez B, Perruche C, Pinardi N, Pisano A, Poulain PM , Pujol IM, Raj RP, Raudsepp U, Roquet H, Samuelsen A, Sathyendranath S, She J, Simoncelli S, Solidoro C, Tinker J, Tintoré J, Viktorsson L, Ablain M, Almroth-Rosell E, Bonaduce A, Clementi E, Cossarini G, Dagneaux Q, Desportes C, Dye S, Fratianni C, Good S, Greiner E, Gourrion J, Hamon M, Holt J, Hyder P, Kennedy J, Manzano-Muñoz F, Melet A, Meyssignac B, Mulet S, Nardelli BB, O’Dea E, Olason E, Paulmier A, Pérez-González I, Reid R, Racault MF, Raitsos DE, Ramos A, Sykes P, Szekely T, Verbrugge N. 2016. The Copernicus Marine Environment Monitoring Service Ocean State Report, Journal of Operational Oceanography. 9 (sup2): 235-320. http://dx.doi.org/10.1080/1755876X.2016.1273446
* Pérez Gómez B, De Alfonso M, Zacharioudaki A, Pérez González I, Álvarez Fanjul E, Müller M, Marcos M, Manzano F, Korres G, Ravdas M, Tamm S. 2018. Sea level, SST and waves: extremes variability. In: Copernicus Marine Service Ocean State Report, Issue 2, Journal of Operational Oceanography, 11:sup1, Chap. 3.1, s79–s88, DOI: https://doi.org/10.1080/1755876X.2018.1489208.
* WCRP Global Sea Level Budget Group: Global sea-level budget 1993–present. 2018. Earth Syst. Sci. Data, 10, 1551-1590, https://doi.org/10.5194/essd-10-1551-2018.
* Vousdoukas MI, Mentaschi L, Hinkel J, et al. 2020. Economic motivation for raising coastal flood defenses in Europe. Nat Commun 11, 2119 (2020). https://doi.org/10.1038/s41467-020-15665-3.
* Tebaldi, C., Ranasinghe, R., Vousdoukas, M. et al. 2021. Extreme sea levels at different global warming levels. Nat. Clim. Chang. 11, 746–751. https://doi.org/10.1038/s41558-021-01127-1. Tebaldi, C., Ranasinghe, R., Vousdoukas, M. et al. Author Correction: Extreme sea levels at different global warming levels. Nat. Clim. Chang. 13, 588 (2023). https://doi.org/10.1038/s41558-023-01665-w.
* Boumis, G., Moftakhari, H. R., & Moradkhani, H. 2023. Coevolution of extreme sea levels and sea-level rise under global warming. Earth's Future, 11, e2023EF003649. https://doi. org/10.1029/2023EF003649.
* Chafik L, Nilsen JEØ, Dangendorf S et al. 2019. North Atlantic Ocean Circulation and Decadal Sea Level Change During the Altimetry Era. Sci Rep 9, 1041. https://doi.org/10.1038/s41598-018-37603-6





<img src="https://documentation.marine.copernicus.eu/IMG/OMI_EXTREME_SL_MEDSEA_slev_mean_and_anomaly_obs.png" width="300" height="200"/>



# Mediterranean Sea sea level extreme variability mean and anomaly (observations)


**Product ID:** OMI_EXTREME_SL_MEDSEA_slev_mean_and_anomaly_obs

**Description:** **DEFINITION**

The OMI_EXTREME_SL_MEDSEA_slev_mean_and_anomaly_obs indicator is based on the computation of the 99th and the 1st percentiles from in situ data (observations). It is computed for the variable sea level measured by tide gauges along the coast. The use of percentiles instead of annual maximum and minimum values, makes this extremes study less affected by individual data measurement errors. The annual percentiles referred to annual mean sea level are temporally averaged and their spatial evolution is displayed in the dataset omi_extreme_sl_medsea_slev_mean_and_anomaly_obs, jointly with the anomaly in the target year. This study of extreme variability was first applied to sea level variable (Pérez Gómez et al 2016) and then extended to other essential variables, sea surface temperature and significant wave height (Pérez Gómez et al 2018). 

**CONTEXT**

Sea level (SLEV) is one of the Essential Ocean Variables most affected by climate change. Global mean sea level rise has accelerated since the 1990’s (Abram et al., 2019, Legeais et al., 2020), due to the increase of ocean temperature and mass volume caused by land ice melting (WCRP, 2018). Basin scale oceanographic and meteorological features lead to regional variations of this trend that combined with changes in the frequency and intensity of storms could also rise extreme sea levels up to one meter by the end of the century (Vousdoukas et al., 2020, Tebaldi et al., 2021). This will significantly increase coastal vulnerability to storms, with important consequences on the extent of flooding events, coastal erosion and damage to infrastructures caused by waves (Boumis et al., 2023). The increase in extreme sea levels over recent decades is, therefore, primarily due to the rise in mean sea level. Note, however, that the methodology used to compute this OMI removes the annual 50th percentile, thereby discarding the mean sea level trend to isolate changes in storminess. 
The Mediterranean Sea shows statistically significant positive sea level trends over the whole basin. However, at sub-basin scale sea level trends show spatial variability arising from local circulation (Calafat et al., 2022; Meli et al., 2023).

**COPERNICUS MARINE SERVICE KEY FINDINGS**

The completeness index criteria is fulfilled in this region by 38 stations, 26 more than in 2021, significantly increasing spatial coverage with new in situ data in the central Mediterranean Sea, primarily from Italian stations. The mean 99th percentiles reflect the spatial variability of the tide, a microtidal regime, along the Spanish,  French and Italian coasts, ranging from around 0.20 m above mean sea level in Sicily and the Balearic Islands (e.g.: 0.22 m in Porto Empedocle, 0.23 m  in Ibiza)) to around 0.60 m above mean sea level in the Northern Adriatic Sea (e.g.: 0.63 m in Trieste, 0.61 m in Venice). . The annual 99th percentiles standard deviation ranges between 2 cm in Málaga and Motril (South of Spain) to 8 cm in Marseille. . The 2022 99th percentile anomalies present negative values mainly along the Spanish coast (as in 2021) and in the islands of Corsica and Sardinia (Western part of the region), while positive values are observed along the Eastern French Mediterranean coast and at most of the Italian stations (closer to the central part of the region), with values ranging from -4 cm in Málaga and Motril (Spain) to +5 cm in Ancona (Italy). 

**DOI (product):** 
https://doi.org/10.48670/moi-00265

**References:**

* Abram, N., Gattuso, J.-P., Prakash, A., Cheng, L., Chidichimo, M. P., Crate, S., Enomoto, H., Garschagen, M., Gruber, N., Harper, S., Holland, E., Kudela, R. M., Rice, J., Steffen, K., & von Schuckmann, K. (2019). Framing and Context of the Report. In H. O. Pörtner, D. C. Roberts, V. Masson-Delmotte, P. Zhai, M. Tignor, E. Poloczanska, K. Mintenbeck, A. Alegría, M. Nicolai, A. Okem, J. Petzold, B. Rama, & N. M. Weyer (Eds.), IPCC Special Report on the Ocean and Cryosphere in a Changing Climate (pp. 73–129). in press. https://www.ipcc.ch/srocc/.
* Boumis, G., Moftakhari, H. R., & Moradkhani, H. 2023. Coevolution of extreme sea levels and sea-level rise under global warming. Earth's Future, 11, e2023EF003649. https://doi. org/10.1029/2023EF003649.
* Calafat, F. M., Frederikse, T., and Horsburgh, K.: The Sources of Sea-Level Changes in the Mediterranean Sea Since 1960, J Geophys Res Oceans, 127, e2022JC019061, https://doi.org/10.1029/2022JC019061, 2022.
* Legeais J-F, Llovel W, Melet A, and Meyssignac B. 2020. Evidence of the TOPEX-A Altimeter Instrumental Anomaly and Acceleration of the Global Mean Sea Level, In: Copernicus Marine Service Ocean State Report, Issue 4, Journal of Operational Oceanography, s77–s82, https://doi.org/10.1080/1755876X.2020.1785097.
* Meli M, Camargo CML, Olivieri M, Slangen ABA, and Romagnoli C. 2023. Sea-level trend variability in the Mediterranean during the 1993–2019 period, Front Mar Sci, 10, 1150488, https://doi.org/10.3389/FMARS.2023.1150488/BIBTEX.
* Pérez-Gómez B, Álvarez-Fanjul E, She J, Pérez-González I, Manzano F. 2016. Extreme sea level events, Section 4.4, p:300. In: Von Schuckmann K, Le Traon PY, Alvarez-Fanjul E, Axell L, Balmaseda M, Breivik LA, Brewin RJW, Bricaud C, Drevillon M, Drillet Y, Dubois C , Embury O, Etienne H, García-Sotillo M, Garric G, Gasparin F, Gutknecht E, Guinehut S, Hernandez F, Juza M, Karlson B, Korres G, Legeais JF, Levier B, Lien VS, Morrow R, Notarstefano G, Parent L, Pascual A, Pérez-Gómez B, Perruche C, Pinardi N, Pisano A, Poulain PM , Pujol IM, Raj RP, Raudsepp U, Roquet H, Samuelsen A, Sathyendranath S, She J, Simoncelli S, Solidoro C, Tinker J, Tintoré J, Viktorsson L, Ablain M, Almroth-Rosell E, Bonaduce A, Clementi E, Cossarini G, Dagneaux Q, Desportes C, Dye S, Fratianni C, Good S, Greiner E, Gourrion J, Hamon M, Holt J, Hyder P, Kennedy J, Manzano-Muñoz F, Melet A, Meyssignac B, Mulet S, Nardelli BB, O’Dea E, Olason E, Paulmier A, Pérez-González I, Reid R, Racault MF, Raitsos DE, Ramos A, Sykes P, Szekely T, Verbrugge N. 2016. The Copernicus Marine Environment Monitoring Service Ocean State Report, Journal of Operational Oceanography. 9 (sup2): 235-320. http://dx.doi.org/10.1080/1755876X.2016.1273446
* Pérez Gómez B, De Alfonso M, Zacharioudaki A, Pérez González I, Álvarez Fanjul E, Müller M, Marcos M, Manzano F, Korres G, Ravdas M, Tamm S. 2018. Sea level, SST and waves: extremes variability. In: Copernicus Marine Service Ocean State Report, Issue 2, Journal of Operational Oceanography, 11:sup1, Chap. 3.1, s79–s88, DOI: https://doi.org/10.1080/1755876X.2018.1489208.
* Tebaldi, C., Ranasinghe, R., Vousdoukas, M. et al. 2021. Extreme sea levels at different global warming levels. Nat. Clim. Chang. 11, 746–751. https://doi.org/10.1038/s41558-021-01127-1.
* Tebaldi, C., Ranasinghe, R., Vousdoukas, M. et al. Author Correction: Extreme sea levels at different global warming levels. Nat. Clim. Chang. 13, 588 (2023). https://doi.org/10.1038/s41558-023-01665-w.
* Vousdoukas MI, Mentaschi L, Hinkel J, et al. 2020. Economic motivation for raising coastal flood defenses in Europe. Nat Commun 11, 2119 (2020). https://doi.org/10.1038/s41467-020-15665-3.
* WCRP Global Sea Level Budget Group: Global sea-level budget 1993–present. 2018. Earth Syst. Sci. Data, 10, 1551-1590, https://doi.org/10.5194/essd-10-1551-2018.





<img src="https://documentation.marine.copernicus.eu/IMG/OMI_EXTREME_SL_NORTHWESTSHELF_slev_mean_and_anomaly_obs.png" width="300" height="200"/>



# North West Shelf sea level extreme variability mean and anomaly (observations)


**Product ID:** OMI_EXTREME_SL_NORTHWESTSHELF_slev_mean_and_anomaly_obs

**Description:** **DEFINITION**

The OMI_EXTREME_SL_NORTHWESTSHELF_slev_mean_and_anomaly_obs indicator is based on the computation of the 99th and the 1st percentiles from in situ data (observations). It is computed for the variable sea level measured by tide gauges along the coast. The use of percentiles instead of annual maximum and minimum values, makes this extremes study less affected by individual data measurement errors. The annual percentiles referred to annual mean sea level are temporally averaged and their spatial evolution is displayed in the dataset omi_extreme_sl_northwestshelf_slev_mean_and_anomaly_obs, jointly with the anomaly in the target year. This study of extreme variability was first applied to sea level variable (Pérez Gómez et al 2016) and then extended to other essential variables, sea surface temperature and significant wave height (Pérez Gómez et al 2018).

**CONTEXT**

Sea level (SLEV) is one of the Essential Ocean Variables most affected by climate change. Global mean sea level rise has accelerated since the 1990’s (Abram et al., 2019, Legeais et al., 2020), due to the increase of ocean temperature and mass volume caused by land ice melting (WCRP, 2018). Basin scale oceanographic and meteorological features lead to regional variations of this trend that combined with changes in the frequency and intensity of storms could also rise extreme sea levels up to one metre by the end of the century (Vousdoukas et al., 2020, Tebaldi et al., 2021). This will significantly increase coastal vulnerability to storms, with important consequences on the extent of flooding events, coastal erosion and damage to infrastructures caused by waves (Boumis et al., 2023). The increase in extreme sea levels over recent decades is, therefore, primarily due to the rise in mean sea level. Note, however, that the methodology used to compute this OMI removes the annual 50th percentile, thereby discarding the mean sea level trend to isolate changes in storminess. 
The North West Shelf area presents positive sea level trends with higher trend estimates in the German Bight and around Denmark, and lower trends around the southern part of Great Britain (Dettmering et al., 2021).

**COPERNICUS MARINE SERVICE KEY FINDINGS**

The completeness index criteria is fulfilled in this region by 34 stations, eight more than in 2021 (26), most of them from Norway. The mean 99th percentiles present a large spatial variability related to the tidal pattern, with largest values found in East England and at the entrance of the English channel, and lowest values along the Danish and Swedish coasts, ranging from the 3.08 m above mean sea level in Immingan (East England)  to 0.57 m above mean sea level in Ringhals (Sweden) and Helgeroa (Norway). The standard deviation of annual 99th percentiles ranges between 2-3 cm in the western part of the region (e.g.: 2 cm in Harwich, 3 cm  in Dunkerke) and 7-8 cm in the eastern part and the Kattegat (e.g. 8 cm in Stenungsund, Sweden).. The 99th percentile anomalies for 2022 show positive values in Southeast England, with a maximum value of +8 cm in Lowestoft, and negative values in the eastern part of the Kattegat, reaching -8 cm in Oslo. The remaining stations exhibit minor positive or negative values.  

**DOI (product):** 
https://doi.org/10.48670/moi-00272

**References:**

* Abram, N., Gattuso, J.-P., Prakash, A., Cheng, L., Chidichimo, M. P., Crate, S., Enomoto, H., Garschagen, M., Gruber, N., Harper, S., Holland, E., Kudela, R. M., Rice, J., Steffen, K., & von Schuckmann, K. (2019). Framing and Context of the Report. In H. O. Pörtner, D. C. Roberts, V. Masson-Delmotte, P. Zhai, M. Tignor, E. Poloczanska, K. Mintenbeck, A. Alegría, M. Nicolai, A. Okem, J. Petzold, B. Rama, & N. M. Weyer (Eds.), IPCC Special Report on the Ocean and Cryosphere in a Changing Climate (pp. 73–129). in press. https://www.ipcc.ch/srocc/
* Legeais J-F, W. Llowel, A. Melet and B. Meyssignac: Evidence of the TOPEX-A Altimeter Instrumental Anomaly and Acceleration of the Global Mean Sea Level, in Copernicus Marine Service Ocean State Report, Issue 4, Journal of Operational Oceanography, 2020, accepted.
* Pérez-Gómez B, Álvarez-Fanjul E, She J, Pérez-González I, Manzano F. 2016. Extreme sea level events, Section 4.4, p:300. In: Von Schuckmann K, Le Traon PY, Alvarez-Fanjul E, Axell L, Balmaseda M, Breivik LA, Brewin RJW, Bricaud C, Drevillon M, Drillet Y, Dubois C , Embury O, Etienne H, García-Sotillo M, Garric G, Gasparin F, Gutknecht E, Guinehut S, Hernandez F, Juza M, Karlson B, Korres G, Legeais JF, Levier B, Lien VS, Morrow R, Notarstefano G, Parent L, Pascual A, Pérez-Gómez B, Perruche C, Pinardi N, Pisano A, Poulain PM , Pujol IM, Raj RP, Raudsepp U, Roquet H, Samuelsen A, Sathyendranath S, She J, Simoncelli S, Solidoro C, Tinker J, Tintoré J, Viktorsson L, Ablain M, Almroth-Rosell E, Bonaduce A, Clementi E, Cossarini G, Dagneaux Q, Desportes C, Dye S, Fratianni C, Good S, Greiner E, Gourrion J, Hamon M, Holt J, Hyder P, Kennedy J, Manzano-Muñoz F, Melet A, Meyssignac B, Mulet S, Nardelli BB, O’Dea E, Olason E, Paulmier A, Pérez-González I, Reid R, Racault MF, Raitsos DE, Ramos A, Sykes P, Szekely T, Verbrugge N. 2016. The Copernicus Marine Environment Monitoring Service Ocean State Report, Journal of Operational Oceanography. 9 (sup2): 235-320. http://dx.doi.org/10.1080/1755876X.2016.1273446
* Pérez Gómez B, De Alfonso M, Zacharioudaki A, Pérez González I, Álvarez Fanjul E, Müller M, Marcos M, Manzano F, Korres G, Ravdas M, Tamm S. 2018. Sea level, SST and waves: extremes variability. In: Copernicus Marine Service Ocean State Report, Issue 2, Journal of Operational Oceanography, 11:sup1, Chap. 3.1, s79–s88, DOI: https://doi.org/10.1080/1755876X.2018.1489208.
* WCRP Global Sea Level Budget Group: Global sea-level budget 1993–present. 2018. Earth Syst. Sci. Data, 10, 1551-1590, https://doi.org/10.5194/essd-10-1551-2018.
* Vousdoukas MI, Mentaschi L, Hinkel J, et al. 2020. Economic motivation for raising coastal flood defenses in Europe. Nat Commun 11, 2119 (2020). https://doi.org/10.1038/s41467-020-15665-3.
* Boumis, G., Moftakhari, H. R., & Moradkhani, H. 2023. Coevolution of extreme sea levels and sea-level rise under global warming. Earth's Future, 11, e2023EF003649. https://doi. org/10.1029/2023EF003649.
* Dettmering D, Müller FL, Oelsmann J, Passaro M, Schwatke C, Restano M, Benveniste J, and Seitz F. 2021. North SEAL: A new dataset of sea level changes in the North Sea from satellite altimetry, Earth Syst Sci Data, 13, 3733–3753, https://doi.org/10.5194/ESSD-13-3733-2021.





<img src="https://documentation.marine.copernicus.eu/IMG/OMI_EXTREME_SST_BALTIC_sst_mean_and_anomaly_obs.png" width="300" height="200"/>



# Baltic Sea sea surface temperature extreme variability mean and anomaly (observations)


**Product ID:** OMI_EXTREME_SST_BALTIC_sst_mean_and_anomaly_obs

**Description:** **DEFINITION**

The OMI_EXTREME_SST_BALTIC_sst_mean_and_anomaly_obs indicator is based on the computation of the 99th and the 1st percentiles from in situ data (observations). It is computed for the variable sea surface temperature measured by in situ buoys at depths between 0 and 5 meters. The use of percentiles instead of annual maximum and minimum values, makes this extremes study less affected by individual data measurement errors. The percentiles are temporally averaged, and the spatial evolution is displayed, jointly with the anomaly in the target year. This study of extreme variability was first applied to sea level variable (Pérez Gómez et al 2016) and then extended to other essential variables, sea surface temperature and significant wave height (Pérez Gómez et al 2018). 

**CONTEXT**

Sea surface temperature (SST) is one of the essential ocean variables affected by climate change (mean SST trends, SST spatial and interannual variability, and extreme events). In Europe, several studies show warming trends in mean SST for the last years (von Schuckmann, 2016; IPCC, 2021, 2022). An exception seems to be the North Atlantic, where, in contrast, anomalous cold conditions have been observed since 2014 (Mulet et al., 2018; Dubois et al. 2018; IPCC 2021, 2022). Extremes may have a stronger direct influence in population dynamics and biodiversity. According to Alexander et al. 2018 the observed warming trend will continue during the 21st Century and this can result in exceptionally large warm extremes. Monitoring the evolution of sea surface temperature extremes is, therefore, crucial.
The Baltic Sea has showed in the last two decades a warming trend across the whole basin with more frequent and severe heat waves (IPCC, 2022). This trend is significantly higher when considering only the summer season, which would affect the high extremes (e.g. Høyer and Karagali, 2016).

**COPERNICUS MARINE SERVICE KEY FINDINGS**

The mean 99th percentiles showed in the area go from 19.6ºC in Tallinn station to 21.4ºC in Rohukula station, and the standard deviation ranges between 1ºC and 5.4ºC reached in the Estonian Coast.
Results for this year show either positive or negative low anomalies in the Coast of Sweeden (-0.7/+0.5ºC) within the standard deviation margin and a general positive anomaly in the rest of the region. This anomaly is noticeable in Rohukula and Virtsu tide gauges (Estonia) with +3.9ºC, but inside the standard deviation in both locations. In the South Baltic two stations, GreifswalderOie and Neustadt, reach an anomaly of +2ºC, but around the standard deviation.

**DOI (product):** 
https://doi.org/10.48670/moi-00204

**References:**

* Alexander MA, Scott JD, Friedland KD, Mills KE, Nye JA, Pershing AJ, Thomas AC. 2018. Projected sea surface temperatures over the 21st century: Changes in the mean, variability and extremes for large marine ecosystem regions of Northern Oceans. Elem Sci Anth, 6(1), p.9. DOI: http://doi.org/10.1525/elementa.191.
* Dubois C, von Schuckmann K, Josey S, Ceschin A. 2018. Changes in the North Atlantic. In: Copernicus Marine Service Ocean State Report, Issue 2, Journal of Operational Oceanography, vol 11, sup1, s66–s70. DOI: 10.1080/1755876X.2018.1489208
* Høyer, JL, Karagali, I. 2016. Sea surface temperature climate data record for the North Sea and Baltic Sea. Journal of Climate, 29(7), 2529-2541. https://doi.org/10.1175/JCLI-D-15-0663.1
* Pérez-Gómez B, Álvarez-Fanjul E, She J, Pérez-González I, Manzano F. 2016. Extreme sea level events, Section 4.4, p:300. In: Von Schuckmann K, Le Traon PY, Alvarez-Fanjul E, Axell L, Balmaseda M, Breivik LA, Brewin RJW, Bricaud C, Drevillon M, Drillet Y, Dubois C , Embury O, Etienne H, García-Sotillo M, Garric G, Gasparin F, Gutknecht E, Guinehut S, Hernandez F, Juza M, Karlson B, Korres G, Legeais JF, Levier B, Lien VS, Morrow R, Notarstefano G, Parent L, Pascual A, Pérez-Gómez B, Perruche C, Pinardi N, Pisano A, Poulain PM , Pujol IM, Raj RP, Raudsepp U, Roquet H, Samuelsen A, Sathyendranath S, She J, Simoncelli S, Solidoro C, Tinker J, Tintoré J, Viktorsson L, Ablain M, Almroth-Rosell E, Bonaduce A, Clementi E, Cossarini G, Dagneaux Q, Desportes C, Dye S, Fratianni C, Good S, Greiner E, Gourrion J, Hamon M, Holt J, Hyder P, Kennedy J, Manzano-Muñoz F, Melet A, Meyssignac B, Mulet S, Nardelli BB, O’Dea E, Olason E, Paulmier A, Pérez-González I, Reid R, Racault MF, Raitsos DE, Ramos A, Sykes P, Szekely T, Verbrugge N. 2016. The Copernicus Marine Environment Monitoring Service Ocean State Report, Journal of Operational Oceanography. 9 (sup2): 235-320. http://dx.doi.org/10.1080/1755876X.2016.1273446
* Pérez Gómez B, De Alfonso M, Zacharioudaki A, Pérez González I, Álvarez Fanjul E, Müller M, Marcos M, Manzano F, Korres G, Ravdas M, Tamm S. 2018. Sea level, SST and waves: extremes variability. In: Copernicus Marine Service Ocean State Report, Issue 2, Journal of Operational Oceanography, 11:sup1, Chap. 3.1, s79–s88, DOI: https://doi.org/10.1080/1755876X.2018.1489208.
* von Schuckmann K, Le Traon PY, Alvarez-Fanjul E, Axell L, Balmaseda M, Breivik LA, … Verbrugge N. 2016. The Copernicus Marine Environment Monitoring Service Ocean State Report. Journal of Operational Oceanography, 9(sup2), s235–s320. https://doi.org/10.1080/1755876X.2016.1273446





<img src="https://documentation.marine.copernicus.eu/IMG/OMI_EXTREME_SST_IBI_sst_mean_and_anomaly_obs.png" width="300" height="200"/>



# Iberia Biscay Ireland sea surface temperature extreme variability mean and anomaly (observations)


**Product ID:** OMI_EXTREME_SST_IBI_sst_mean_and_anomaly_obs

**Description:** **DEFINITION**

The OMI_EXTREME_SST_IBI_sst_mean_and_anomaly_obs indicator is based on the computation of the 99th and the 1st percentiles from in situ data (observations). It is computed for the variable sea surface temperature measured by in situ buoys at depths between 0 and 5 meters. The use of percentiles instead of annual maximum and minimum values, makes this extremes study less affected by individual data measurement errors. The percentiles are temporally averaged, and the spatial evolution is displayed, jointly with the anomaly in the target year. This study of extreme variability was first applied to sea level variable (Pérez Gómez et al 2016) and then extended to other essential variables, sea surface temperature and significant wave height (Pérez Gómez et al 2018).  

**CONTEXT**

Sea surface temperature (SST) is one of the essential ocean variables affected by climate change (mean SST trends, SST spatial and interannual variability, and extreme events). In Europe, several studies show warming trends in mean SST for the last years (von Schuckmann, 2016; IPCC, 2021, 2022). An exception seems to be the North Atlantic, where, in contrast, anomalous cold conditions have been observed since 2014 (Mulet et al., 2018; Dubois et al. 2018; IPCC 2021, 2022). Extremes may have a stronger direct influence in population dynamics and biodiversity. According to Alexander et al. 2018 the observed warming trend will continue during the 21st Century and this can result in exceptionally large warm extremes. Monitoring the evolution of sea surface temperature extremes is, therefore, crucial.
The Iberia Biscay Ireland area is characterized by a great complexity in terms of processes that take place in it. The sea surface temperature varies depending on the latitude with higher values to the South. In this area, the clear warming trend observed in other European Seas is not so evident. The northwest part is influenced by the refreshing trend in the North Atlantic, and a mild warming trend has been observed in the last decade (Pisano et al. 2020).

**COPERNICUS MARINE SERVICE KEY FINDINGS**

The mean 99th percentiles showed in the area present a range from 16-20ºC in the Southwest of the British Isles and the English Channel, 19-21ºC in the West of Galician Coast, 21-23ºC in the south of Bay of Biscay, 23.5ºC in the Gulf of Cadiz to 24.5ºC in the Canary Island. The standard deviations are between 0.5ºC and 1.3ºC in the region except in the English Channel where the standard deviation is higher, reaching 3ºC.
Results for this year show either positive or negative low anomalies below the 45º parallel, with a slight positive anomaly in the Gulf of Cadiz and the Southeast of the Bay of Biscay over 1ºC. In the Southwest of the British Isles and the English Channel, the anomaly is clearly positive, with some stations with an anomaly over 2ºC, but inside the standard deviation in the area. 

**DOI (product):** 
https://doi.org/10.48670/moi-00255

**References:**

* Alexander MA, Scott JD, Friedland KD, Mills KE, Nye JA, Pershing AJ, Thomas AC. 2018. Projected sea surface temperatures over the 21st century: Changes in the mean, variability and extremes for large marine ecosystem regions of Northern Oceans. Elem Sci Anth, 6(1), p.9. DOI: http://doi.org/10.1525/elementa.191.
* Dubois C, von Schuckmann K, Josey S, Ceschin A. 2018. Changes in the North Atlantic. In: Copernicus Marine Service Ocean State Report, Issue 2, Journal of Operational Oceanography, vol 11, sup1, s66–s70. DOI: 10.1080/1755876X.2018.1489208
* Mulet S, Nardelli BB, Good S, Pisano A, Greiner E, Monier M, Autret E, Axell L, Boberg F, Ciliberti S, Drévillon M, Droghei R, Embury O, Gourrion J, Høyer J, Juza M, Kennedy J, Lemieux-Dudon B, Peneva E, Reid R, Simoncelli S, Storto A, Tinker J, von Schuckmann K, Wakelin SL. 2018. Ocean temperature and salinity. In: Copernicus Marine Service Ocean State Report, Issue 2, Journal of Operational Oceanography, vol 11, sup1, s5–s13. DOI: 10.1080/1755876X.2018.1489208
* Pérez-Gómez B, Álvarez-Fanjul E, She J, Pérez-González I, Manzano F. 2016. Extreme sea level events, Section 4.4, p:300. In: Von Schuckmann K, Le Traon PY, Alvarez-Fanjul E, Axell L, Balmaseda M, Breivik LA, Brewin RJW, Bricaud C, Drevillon M, Drillet Y, Dubois C , Embury O, Etienne H, García-Sotillo M, Garric G, Gasparin F, Gutknecht E, Guinehut S, Hernandez F, Juza M, Karlson B, Korres G, Legeais JF, Levier B, Lien VS, Morrow R, Notarstefano G, Parent L, Pascual A, Pérez-Gómez B, Perruche C, Pinardi N, Pisano A, Poulain PM , Pujol IM, Raj RP, Raudsepp U, Roquet H, Samuelsen A, Sathyendranath S, She J, Simoncelli S, Solidoro C, Tinker J, Tintoré J, Viktorsson L, Ablain M, Almroth-Rosell E, Bonaduce A, Clementi E, Cossarini G, Dagneaux Q, Desportes C, Dye S, Fratianni C, Good S, Greiner E, Gourrion J, Hamon M, Holt J, Hyder P, Kennedy J, Manzano-Muñoz F, Melet A, Meyssignac B, Mulet S, Nardelli BB, O’Dea E, Olason E, Paulmier A, Pérez-González I, Reid R, Racault MF, Raitsos DE, Ramos A, Sykes P, Szekely T, Verbrugge N. 2016. The Copernicus Marine Environment Monitoring Service Ocean State Report, Journal of Operational Oceanography. 9 (sup2): 235-320. http://dx.doi.org/10.1080/1755876X.2016.1273446
* Pérez Gómez B, De Alfonso M, Zacharioudaki A, Pérez González I, Álvarez Fanjul E, Müller M, Marcos M, Manzano F, Korres G, Ravdas M, Tamm S. 2018. Sea level, SST and waves: extremes variability. In: Copernicus Marine Service Ocean State Report, Issue 2, Journal of Operational Oceanography, 11:sup1, Chap. 3.1, s79–s88, DOI: https://doi.org/10.1080/1755876X.2018.1489208.
* Pisano A, Marullo S, Artale V, Falcini F, Yang C, Leonelli FE, Santoleri R, Nardelli BB. 2020. New Evidence of Mediterranean Climate Change and Variability from Sea Surface Temperature Observations. Remote Sensing 12(132). DOI: 10.3390/rs12010132.
* von Schuckmann K, Le Traon PY, Alvarez-Fanjul E, Axell L, Balmaseda M, Breivik LA, … Verbrugge N. 2016. The Copernicus Marine Environment Monitoring Service Ocean State Report. Journal of Operational Oceanography, 9(sup2), s235–s320. https://doi.org/10.1080/1755876X.2016.1273446





<img src="https://documentation.marine.copernicus.eu/IMG/OMI_EXTREME_SST_MEDSEA_sst_mean_and_anomaly_obs.png" width="300" height="200"/>



# Mediterranean Sea sea surface temperature extreme variability mean and anomaly (observations)


**Product ID:** OMI_EXTREME_SST_MEDSEA_sst_mean_and_anomaly_obs

**Description:** **DEFINITION**

The OMI_EXTREME_SST_MEDSEA_sst_mean_and_anomaly_obs indicator is based on the computation of the 99th and the 1st percentiles from in situ data (observations). It is computed for the variable sea surface temperature measured by in situ buoys at depths between 0 and 5 meters. The use of percentiles instead of annual maximum and minimum values, makes this extremes study less affected by individual data measurement errors. The percentiles are temporally averaged, and the spatial evolution is displayed, jointly with the anomaly in the target year. This study of extreme variability was first applied to sea level variable (Pérez Gómez et al 2016) and then extended to other essential variables, sea surface temperature and significant wave height (Pérez Gómez et al 2018).  

**CONTEXT**

Sea surface temperature (SST) is one of the essential ocean variables affected by climate change (mean SST trends, SST spatial and interannual variability, and extreme events). In Europe, several studies show warming trends in mean SST for the last years (von Schuckmann et al., 2016; IPCC, 2021, 2022). An exception seems to be the North Atlantic, where, in contrast, anomalous cold conditions have been observed since 2014 (Mulet et al., 2018; Dubois et al. 2018; IPCC 2021, 2022). Extremes may have a stronger direct influence in population dynamics and biodiversity. According to Alexander et al. 2018 the observed warming trend will continue during the 21st Century and this can result in exceptionally large warm extremes. Monitoring the evolution of sea surface temperature extremes is, therefore, crucial.
The Mediterranean Sea has showed a constant increase of the SST in the last three decades across the whole basin with more frequent and severe heat waves (Juza et al., 2022). Deep analyses of the variations have displayed a non-uniform rate in space, being the warming trend more evident in the eastern Mediterranean Sea with respect to the western side. This variation rate is also changing in time over the three decades with differences between the seasons (e.g. Pastor et al. 2018; Pisano et al. 2020), being higher in Spring and Summer, which would affect the extreme values.

**COPERNICUS MARINE SERVICE KEY FINDINGS**

The mean 99th percentiles showed in the area present values from 25ºC in Ionian Sea and 26º in the Alboran sea and Gulf of Lion to 27ºC in the East of Iberian Peninsula. The standard deviation ranges from 0.6ºC to 1.2ºC in the Western Mediterranean and is around 2.2ºC in the Ionian Sea.
Results for this year show a slight negative anomaly in the Ionian Sea (-1ºC) inside the standard deviation and a clear positive anomaly in the Western Mediterranean Sea reaching +2.2ºC, almost two times the standard deviation in the area.

**DOI (product):** 
https://doi.org/10.48670/moi-00267

**References:**

* Alexander MA, Scott JD, Friedland KD, Mills KE, Nye JA, Pershing AJ, Thomas AC. 2018. Projected sea surface temperatures over the 21st century: Changes in the mean, variability and extremes for large marine ecosystem regions of Northern Oceans. Elem Sci Anth, 6(1), p.9. DOI: http://doi.org/10.1525/elementa.191.
* Dubois C, von Schuckmann K, Josey S, Ceschin A. 2018. Changes in the North Atlantic. In: Copernicus Marine Service Ocean State Report, Issue 2, Journal of Operational Oceanography, vol 11, sup1, s66–s70. DOI: 10.1080/1755876X.2018.1489208
* Mulet S, Nardelli BB, Good S, Pisano A, Greiner E, Monier M, Autret E, Axell L, Boberg F, Ciliberti S, Drévillon M, Droghei R, Embury O, Gourrion J, Høyer J, Juza M, Kennedy J, Lemieux-Dudon B, Peneva E, Reid R, Simoncelli S, Storto A, Tinker J, von Schuckmann K, Wakelin SL. 2018. Ocean temperature and salinity. In: Copernicus Marine Service Ocean State Report, Issue 2, Journal of Operational Oceanography, vol 11, sup1, s5–s13. DOI: 10.1080/1755876X.2018.1489208
* Pastor F, Valiente JA, Palau JL. 2018. Sea Surface Temperature in the Mediterranean: Trends and Spatial Patterns (1982–2016). Pure Appl. Geophys, 175: 4017. https://doi.org/10.1007/s00024-017-1739-z.
* Pérez-Gómez B, Álvarez-Fanjul E, She J, Pérez-González I, Manzano F. 2016. Extreme sea level events, Section 4.4, p:300. In: Von Schuckmann K, Le Traon PY, Alvarez-Fanjul E, Axell L, Balmaseda M, Breivik LA, Brewin RJW, Bricaud C, Drevillon M, Drillet Y, Dubois C , Embury O, Etienne H, García-Sotillo M, Garric G, Gasparin F, Gutknecht E, Guinehut S, Hernandez F, Juza M, Karlson B, Korres G, Legeais JF, Levier B, Lien VS, Morrow R, Notarstefano G, Parent L, Pascual A, Pérez-Gómez B, Perruche C, Pinardi N, Pisano A, Poulain PM , Pujol IM, Raj RP, Raudsepp U, Roquet H, Samuelsen A, Sathyendranath S, She J, Simoncelli S, Solidoro C, Tinker J, Tintoré J, Viktorsson L, Ablain M, Almroth-Rosell E, Bonaduce A, Clementi E, Cossarini G, Dagneaux Q, Desportes C, Dye S, Fratianni C, Good S, Greiner E, Gourrion J, Hamon M, Holt J, Hyder P, Kennedy J, Manzano-Muñoz F, Melet A, Meyssignac B, Mulet S, Nardelli BB, O’Dea E, Olason E, Paulmier A, Pérez-González I, Reid R, Racault MF, Raitsos DE, Ramos A, Sykes P, Szekely T, Verbrugge N. 2016. The Copernicus Marine Environment Monitoring Service Ocean State Report, Journal of Operational Oceanography. 9 (sup2): 235-320. http://dx.doi.org/10.1080/1755876X.2016.1273446
* Pérez Gómez B, De Alfonso M, Zacharioudaki A, Pérez González I, Álvarez Fanjul E, Müller M, Marcos M, Manzano F, Korres G, Ravdas M, Tamm S. 2018. Sea level, SST and waves: extremes variability. In: Copernicus Marine Service Ocean State Report, Issue 2, Journal of Operational Oceanography, 11:sup1, Chap. 3.1, s79–s88, DOI: https://doi.org/10.1080/1755876X.2018.1489208.
* Pisano A, Marullo S, Artale V, Falcini F, Yang C, Leonelli FE, Santoleri R, Nardelli BB. 2020. New Evidence of Mediterranean Climate Change and Variability from Sea Surface Temperature Observations. Remote Sensing 12(132). DOI: 10.3390/rs12010132.
* von Schuckmann K, Le Traon PY, Alvarez-Fanjul E, Axell L, Balmaseda M, Breivik LA, … Verbrugge N. 2016. The Copernicus Marine Environment Monitoring Service Ocean State Report. Journal of Operational Oceanography, 9(sup2), s235–s320. https://doi.org/10.1080/1755876X.2016.1273446.





<img src="https://documentation.marine.copernicus.eu/IMG/OMI_EXTREME_SST_NORTHWESTSHELF_sst_mean_and_anomaly_obs.png" width="300" height="200"/>



# North West Shelf sea surface temperature extreme variability mean and anomaly (observations)


**Product ID:** OMI_EXTREME_SST_NORTHWESTSHELF_sst_mean_and_anomaly_obs

**Description:** **DEFINITION**

The OMI_EXTREME_SST_NORTHWESTSHELF_sst_mean_and_anomaly_obs indicator is based on the computation of the 99th and the 1st percentiles from in situ data (observations). It is computed for the variable sea surface temperature measured by in situ buoys at depths between 0 and 5 meters. The use of percentiles instead of annual maximum and minimum values, makes this extremes study less affected by individual data measurement errors. The percentiles are temporally averaged, and the spatial evolution is displayed, jointly with the anomaly in the target year. This study of extreme variability was first applied to sea level variable (Pérez Gómez et al 2016) and then extended to other essential variables, sea surface temperature and significant wave height (Pérez Gómez et al 2018).

**CONTEXT**
Sea surface temperature (SST) is one of the essential ocean variables affected by climate change (mean SST trends, SST spatial and interannual variability, and extreme events). In Europe, several studies show warming trends in mean SST for the last years (von Schuckmann, 2016; IPCC, 2021, 2022). An exception seems to be the North Atlantic, where, in contrast, anomalous cold conditions have been observed since 2014 (Mulet et al., 2018; Dubois et al. 2018; IPCC 2021, 2022). Extremes may have a stronger direct influence in population dynamics and biodiversity. According to Alexander et al. 2018 the observed warming trend will continue during the 21st Century and this can result in exceptionally large warm extremes. Monitoring the evolution of sea surface temperature extremes is, therefore, crucial.
The North-West Self area comprises part of the North Atlantic, where this refreshing trend has been observed, and the North Sea, where a warming trend has been taking place in the last three decades (e.g. Høyer and Karagali, 2016).

**COPERNICUS MARINE SERVICE KEY FINDINGS**
The mean 99th percentiles showed in the area present a range from 14-16ºC in the North of the British Isles, 16-19ºC in the Southwest of the North Sea to 19-21ºC around Denmark (Helgoland Bight, Skagerrak and Kattegat Seas). The standard deviation ranges from 0.5-1ºC in the North of the British Isles, 0.5-2ºC in the Southwest of the North Sea to 1-3ºC in the buoys around Denmark.
Results for this year show either positive or negative low anomalies around their corresponding standard deviation in in the North of the British Isles (-0.5/+0.6ºC) and a clear positive anomaly in the other two areas reaching +2ºC even when they are around the standard deviation margin.

**DOI (product):** 
https://doi.org/10.48670/moi-00274

**References:**

* Alexander MA, Scott JD, Friedland KD, Mills KE, Nye JA, Pershing AJ, Thomas AC. 2018. Projected sea surface temperatures over the 21st century: Changes in the mean, variability and extremes for large marine ecosystem regions of Northern Oceans. Elem Sci Anth, 6(1), p.9. DOI: http://doi.org/10.1525/elementa.191.
* Dubois C, von Schuckmann K, Josey S, Ceschin A. 2018. Changes in the North Atlantic. In: Copernicus Marine Service Ocean State Report, Issue 2, Journal of Operational Oceanography, vol 11, sup1, s66–s70. DOI: 10.1080/1755876X.2018.1489208
* Høyer JL, Karagali I. 2016. Sea surface temperature climate data record for the North Sea and Baltic Sea. Journal of Climate, 29(7), 2529-2541. https://doi.org/10.1175/JCLI-D-15-0663.1
* Mulet S, Nardelli BB, Good S, Pisano A, Greiner E, Monier M, Autret E, Axell L, Boberg F, Ciliberti S, Drévillon M, Droghei R, Embury O, Gourrion J, Høyer J, Juza M, Kennedy J, Lemieux-Dudon B, Peneva E, Reid R, Simoncelli S, Storto A, Tinker J, von Schuckmann K, Wakelin SL. 2018. Ocean temperature and salinity. In: Copernicus Marine Service Ocean State Report, Issue 2, Journal of Operational Oceanography, vol 11, sup1, s5–s13. DOI: 10.1080/1755876X.2018.1489208
* Pérez-Gómez B, Álvarez-Fanjul E, She J, Pérez-González I, Manzano F. 2016. Extreme sea level events, Section 4.4, p:300. In: Von Schuckmann K, Le Traon PY, Alvarez-Fanjul E, Axell L, Balmaseda M, Breivik LA, Brewin RJW, Bricaud C, Drevillon M, Drillet Y, Dubois C , Embury O, Etienne H, García-Sotillo M, Garric G, Gasparin F, Gutknecht E, Guinehut S, Hernandez F, Juza M, Karlson B, Korres G, Legeais JF, Levier B, Lien VS, Morrow R, Notarstefano G, Parent L, Pascual A, Pérez-Gómez B, Perruche C, Pinardi N, Pisano A, Poulain PM , Pujol IM, Raj RP, Raudsepp U, Roquet H, Samuelsen A, Sathyendranath S, She J, Simoncelli S, Solidoro C, Tinker J, Tintoré J, Viktorsson L, Ablain M, Almroth-Rosell E, Bonaduce A, Clementi E, Cossarini G, Dagneaux Q, Desportes C, Dye S, Fratianni C, Good S, Greiner E, Gourrion J, Hamon M, Holt J, Hyder P, Kennedy J, Manzano-Muñoz F, Melet A, Meyssignac B, Mulet S, Nardelli BB, O’Dea E, Olason E, Paulmier A, Pérez-González I, Reid R, Racault MF, Raitsos DE, Ramos A, Sykes P, Szekely T, Verbrugge N. 2016. The Copernicus Marine Environment Monitoring Service Ocean State Report, Journal of Operational Oceanography. 9 (sup2): 235-320. http://dx.doi.org/10.1080/1755876X.2016.1273446
* Pérez Gómez B, De Alfonso M, Zacharioudaki A, Pérez González I, Álvarez Fanjul E, Müller M, Marcos M, Manzano F, Korres G, Ravdas M, Tamm S. 2018. Sea level, SST and waves: extremes variability. In: Copernicus Marine Service Ocean State Report, Issue 2, Journal of Operational Oceanography, 11:sup1, Chap. 3.1, s79–s88, DOI: https://doi.org/10.1080/1755876X.2018.1489208.
* von Schuckmann K, Le Traon PY, Alvarez-Fanjul E, Axell L, Balmaseda M, Breivik LA, … Verbrugge N. 2016. The Copernicus Marine Environment Monitoring Service Ocean State Report. Journal of Operational Oceanography, 9(sup2), s235–s320. https://doi.org/10.1080/1755876X.2016.1273446.





<img src="https://documentation.marine.copernicus.eu/IMG/OMI_EXTREME_WAVE_BALTIC_swh_mean_and_anomaly_obs.png" width="300" height="200"/>



# Baltic Sea significant wave height extreme variability mean and anomaly (observations)


**Product ID:** OMI_EXTREME_WAVE_BALTIC_swh_mean_and_anomaly_obs

**Description:** **DEFINITION**

The OMI_EXTREME_WAVE_BALTIC_swh_mean_and_anomaly_obs indicator is based on the computation of the 99th and the 1st percentiles from in situ data (observations). It is computed for the variable significant wave height (swh) measured by in situ buoys. The use of percentiles instead of annual maximum and minimum values, makes this extremes study less affected by individual data measurement errors. The percentiles are temporally averaged, and the spatial evolution is displayed, jointly with the anomaly in the target year. This study of extreme variability was first applied to sea level variable (Pérez Gómez et al 2016) and then extended to other essential variables, sea surface temperature and significant wave height (Pérez Gómez et al 2018).  

**CONTEXT**

Projections on Climate Change foresee a future with a greater frequency of extreme sea states (Stott, 2016; Mitchell, 2006). The damages caused by severe wave storms can be considerable not only in infrastructure and buildings but also in the natural habitat, crops and ecosystems affected by erosion and flooding aggravated by the extreme wave heights. In addition, wave storms strongly hamper the maritime activities, especially in harbours. These extreme phenomena drive complex hydrodynamic processes, whose understanding is paramount for proper infrastructure management, design and maintenance (Goda, 2010). In recent years, there have been several studies searching possible trends in wave conditions focusing on both mean and extreme values of significant wave height using a multi-source approach with model reanalysis information with high variability in the time coverage, satellite altimeter records covering the last 30 years and in situ buoy measured data since the 1980s decade but with sparse information and gaps in the time series (e.g. Dodet et al., 2020; Timmermans et al., 2020; Young & Ribal, 2019). These studies highlight a remarkable interannual, seasonal and spatial variability of wave conditions and suggest that the possible observed trends are not clearly associated with anthropogenic forcing (Hochet et al. 2021, 2023).
In the Baltic Sea, the particular bathymetry and geography of the basin intensify the seasonal and spatial fluctuations in wave conditions. No clear statistically significant trend in the sea state has been appreciated except a rising trend in significant wave height in winter season, linked with the reduction of sea ice coverage (Soomere, 2023; Tuomi et al., 2019).

**COPERNICUS MARINE SERVICE KEY FINDINGS**

The mean 99th percentiles shown in the area are from 3 to 4 meters and the standard deviation ranges from 0.2 m to 0.4 m. 
Results for this year show a slight positive or negative anomaly in all the stations, from -0.24 m to +0.36 m, inside the margin of the standard deviation.

**DOI (product):** 
https://doi.org/10.48670/moi-00199

**References:**

* Pérez-Gómez B, Álvarez-Fanjul E, She J, Pérez-González I, Manzano F. 2016. Extreme sea level events, Section 4.4, p:300. In: Von Schuckmann K, Le Traon PY, Alvarez-Fanjul E, Axell L, Balmaseda M, Breivik LA, Brewin RJW, Bricaud C, Drevillon M, Drillet Y, Dubois C , Embury O, Etienne H, García-Sotillo M, Garric G, Gasparin F, Gutknecht E, Guinehut S, Hernandez F, Juza M, Karlson B, Korres G, Legeais JF, Levier B, Lien VS, Morrow R, Notarstefano G, Parent L, Pascual A, Pérez-Gómez B, Perruche C, Pinardi N, Pisano A, Poulain PM , Pujol IM, Raj RP, Raudsepp U, Roquet H, Samuelsen A, Sathyendranath S, She J, Simoncelli S, Solidoro C, Tinker J, Tintoré J, Viktorsson L, Ablain M, Almroth-Rosell E, Bonaduce A, Clementi E, Cossarini G, Dagneaux Q, Desportes C, Dye S, Fratianni C, Good S, Greiner E, Gourrion J, Hamon M, Holt J, Hyder P, Kennedy J, Manzano-Muñoz F, Melet A, Meyssignac B, Mulet S, Nardelli BB, O’Dea E, Olason E, Paulmier A, Pérez-González I, Reid R, Racault MF, Raitsos DE, Ramos A, Sykes P, Szekely T, Verbrugge N. 2016. The Copernicus Marine Environment Monitoring Service Ocean State Report, Journal of Operational Oceanography. 9 (sup2): 235-320. http://dx.doi.org/10.1080/1755876X.2016.1273446
* Pérez Gómez B, De Alfonso M, Zacharioudaki A, Pérez González I, Álvarez Fanjul E, Müller M, Marcos M, Manzano F, Korres G, Ravdas M, Tamm S. 2018. Sea level, SST and waves: extremes variability. In: Copernicus Marine Service Ocean State Report, Issue 2, Journal of Operational Oceanography, 11:sup1, Chap. 3.1, s79–s88, DOI: https://doi.org/10.1080/1755876X.2018.1489208.
* Stott P. 2016. How climate change affects extreme weather events. Science, 352(6293), 1517-1518.
* Dodet G, Piolle J-F, Quilfen Y, Abdalla S, Accensi M, Ardhuin F, et al. 2020. The sea state CCI dataset v1: Towards a sea state climate data record based on satellite observations. https://dx.doi.org/10.5194/essd-2019-253
* Hochet A, Dodet G, Sévellec F, Bouin M-N, Patra A, & Ardhuin F. 2023. Time of emergence for altimetry-based significant wave height changes in the North Atlantic. Geophysical Research Letters, 50, e2022GL102348. https://doi.org/10.1029/2022GL102348
* Hochet A, Dodet G, Ardhuin F, Hemer M, Young I. 2021. Sea State Decadal Variability in the North Atlantic: A Review. Climate 2021, 9, 173. https://doi.org/10.3390/cli9120173 Goda Y. 2010. Random seas and design of maritime structures. World scientific. https://doi.org/10.1142/7425.
* Mitchell JF, Lowe J, Wood RA, & Vellinga M. 2006. Extreme events due to human-induced climate change. Philosophical Transactions of the Royal Society A: Mathematical, Physical and Engineering Sciences, 364(1845), 2117-2133.





<img src="https://documentation.marine.copernicus.eu/IMG/OMI_EXTREME_WAVE_BLKSEA_recent_changes.png" width="300" height="200"/>



# Black Sea extreme wave events


**Product ID:** OMI_EXTREME_WAVE_BLKSEA_recent_changes

**Description:** **DEFINITION**

Extreme wave characteristics are computed by analysing single storm events and their long-term means and trends based on the product BLKSEA_MULTIYEAR_WAV_007_006. These storm events were detected using the method proposed by Weisse and Günther (2007). The basis of the method is the definition of a severe event threshold (SET), which we define as the 99th percentile of the significant wave height (SWH). Then, the exceedance and shortfall of the SWH at every grid point was determined and counted as a storm event. The analysis of extreme wave events also comprises the following three parameters but are not part of this OMI. The time period between each exceedance and shortfall of the SET is the lifetime of an event. The difference in the maximum SWH of each event and the SET is defined as the event intensity. The geographic area of storm events and exceedance of the SET are defined as the maximum event area. The number, lifetime, and intensity of events were averaged over each year. Finally, the yearly values were used to compute the long-term means. In addition to these parameters, we estimated the difference (anomaly) of the last available year in the multiyear dataset compared against the long-term average as well as the linear trend. To show multiyear variability, each event, fulfilling the above-described definition, is considered in the statistics. This was done independent of the events’ locations within the domain. To obtain long-term trends, a linear regression was applied to the yearly time series. The statistics are based on the period 1950 to -1Y. This approach has been presented in Staneva et al. (2022) for the area of the Black Sea and was later adapted to the South Atlantic in Gramcianinov et al. (2023a, 2023b).

**CONTEXT**

In the last decade, the European seas have been hit by severe storms, causing serious damage to offshore infrastructure and coastal zones and drawing public attention to the importance of having reliable and comprehensive wave forecasts/hindcasts, especially during extreme events. In addition, human activities such as the offshore wind power industry, the oil industry, and coastal recreation regularly require climate and operational information on maximum wave height at a high resolution in space and time. Thus, there is a broad consensus that a high-quality wave climatology and predictions and a deep understanding of extreme waves caused by storms could substantially contribute to coastal risk management and protection measures, thereby preventing or minimising human and material damage and losses. In this respect and in the frame of climate change, which also affects regional wind patterns and therewith the wave climate, it is important for coastal regions to gain insights into wave extreme characteristics and the related trends. These insights are crucial to initiate necessary abatement strategies especially in combination with extreme wave power statistics (see OMI OMI_EXTREME_WAVE_BLKSEA_wave_power).

**KEY FINDINGS**

The yearly mean number of storm events is rather low in regions where the average annual lifetime and intensity of storms are high. In contrast, the number of events is high where their lifetime and intensity are low. While the southwest Black Sea is exposed to yearly mean storm event numbers of below the long-term spatial averages (7.3 events), it is observed that the yearly mean lifetime of the events in the same region is higher than the long-term averages. The extreme wave statistics based on the 99th percentile threshold of the significant wave height (SWH) are very similar to the wind sea wave parameter, and the swell contribution is much lower. On overall, the yearly trend of the storm events is slightly negative (-0.01 events/year) with two areas showing positive trends located in the very east and west. In terms of the mean number of storm events in 2022, a pronounced area with positive values is located along the eastern coast and another in the western basin. The rest of the Black Sea area mostly experienced less events in 2022.

**DOI (product):** 
https://doi.org/10.48670/mds-00348

**References:**

* Gramcianinov, C.B., Staneva, J., de Camargo, R., & da Silva Dias, P.L. (2023a): Changes in extreme wave events in the southwestern South Atlantic Ocean. Ocean Dynamics, doi:10.1007/s10236-023-01575-7
* Gramcianinov, C.B., Staneva, J., Souza, C.R.G., Linhares, P., de Camargo, R., & da Silva Dias, P.L. (2023b): Recent changes in extreme wave events in the south-western South Atlantic. In: von Schuckmann, K., Moreira, L., Le Traon, P.-Y., Grégoire, M., Marcos, M., Staneva, J., Brasseur, P., Garric, G., Lionello, P., Karstensen, J., & Neukermans, G. (eds.): 7th edition of the Copernicus Ocean State Report (OSR7). Copernicus Publications, State Planet, 1-osr7, 12, doi:10.5194/sp-1-osr7-12-2023
* Staneva, J., Ricker, M., Akpınar, A., Behrens, A., Giesen, R., & von Schuckmann, K. (2022): Long-term interannual changes in extreme winds and waves in the Black Sea. Copernicus Ocean State Report, Issue 6, Journal of Operational Oceanography, 15:suppl, 1-220, S.2.8., 64-72, doi:10.1080/1755876X.2022.2095169
* Weisse, R., & Günther, H. (2007): Wave climate and long-term changes for the Southern North Sea obtained from a high-resolution hindcast 1958–2002. Ocean Dynamics, 57(3), 161–172, doi:10.1007/s10236-006-0094-x





<img src="https://documentation.marine.copernicus.eu/IMG/OMI_EXTREME_WAVE_BLKSEA_wave_power.png" width="300" height="200"/>



# Black Sea wave power


**Product ID:** OMI_EXTREME_WAVE_BLKSEA_wave_power

**Description:** **DEFINITION**

The Wave Power P is defined by:
P=(ρg^2)/64π H_s^2 T_e
Where ρ is the surface water density, g the acceleration due to gravity, Hs the significant wave height (VHM0), and Te the wave energy period (VTM10) also abbreviated with Tm-10. The extreme statistics and related recent changes are defined by (1) the 99th percentile of the Wave Power, (2) the linear trend of 99th percentile of the Wave Power, and (3) the difference (anomaly) of the 99th percentile of the last available year in the multiyear dataset  BLKSEA_MULTIYEAR_WAV_007_006 compared against the long-term average. The statistics are based on the period 1950 to -1Y and are obtained from yearly averages. This approach has been presented in Staneva et al. (2022).

**CONTEXT**

In the last decade, the European seas have been hit by severe storms, causing serious damage to offshore infrastructure and coastal zones and drawing public attention to the importance of having reliable and comprehensive wave forecasts/hindcasts, especially during extreme events. In addition, human activities such as the offshore wind power industry, the oil industry, and coastal recreation regularly require climate and operational information on maximum wave height at a high resolution in space and time. Thus, there is a broad consensus that a high-quality wave climatology and predictions and a deep understanding of extreme waves caused by storms could substantially contribute to coastal risk management and protection measures, thereby preventing or minimising human and material damage and losses. In this respect, the Wave Power is a crucial quantity to plan and operate wave energy converters (WEC) and for coastal and offshore structures. For both reliable estimates of long-term Wave Power extremes are important to secure a high efficiency and to guarantee a robust and secure design, respectively.

**KEY FINDINGS**

The 99th percentile of wave power mean patterns are overall consistent with the respective significant wave height pattern. The maximum 99th percentile of wave power is observed in the southwestern Black Sea. Typical values of in the eastern basin are ~20 kW/m and in the western basin ~45 kW/m. The trend of the 99th percentile of the wave power is decreasing with typical values of 50 W/m/year and a maximum of 120 W/m/year, which is equivalent to a ~25% decrease over whole period with respect to the mean. The pattern of the anomaly of the 99th percentile of wave power in 2022 correlates well with that of the wind speed anomaly in 2022, revealing a negative wave-power anomaly in the western Black Sea (P_2020<P_average) and a mix of positive (P_2020<P_average) and negative anomalies in the eastern basin, where the positive anomalies are mainly present in coastal regions.

**DOI (product):** 
https://doi.org/10.48670/mds-00350

**References:**

* Staneva, J., Ricker, M., Akpınar, A., Behrens, A., Giesen, R., & von Schuckmann, K. (2022): Long-term interannual changes in extreme winds and waves in the Black Sea. Copernicus Ocean State Report, Issue 6, Journal of Operational Oceanography, 15:suppl, 1-220, S.2.8., 64-72, doi:10.1080/1755876X.2022.2095169





<img src="https://documentation.marine.copernicus.eu/IMG/OMI_EXTREME_WAVE_IBI_swh_mean_and_anomaly_obs.png" width="300" height="200"/>



# Iberia Biscay Ireland significant wave height extreme variability mean and anomaly (observations)


**Product ID:** OMI_EXTREME_WAVE_IBI_swh_mean_and_anomaly_obs

**Description:** **DEFINITION**

The OMI_EXTREME_WAVE_IBI_swh_mean_and_anomaly_obs indicator is based on the computation of the 99th and the 1st percentiles from in situ data (observations). It is computed for the variable significant wave height (swh) measured by in situ buoys. The use of percentiles instead of annual maximum and minimum values, makes this extremes study less affected by individual data measurement errors. The percentiles are temporally averaged, and the spatial evolution is displayed, jointly with the anomaly in the target year. This study of extreme variability was first applied to sea level variable (Pérez Gómez et al 2016) and then extended to other essential variables, sea surface temperature and significant wave height (Pérez Gómez et al 2018). 

**CONTEXT**

Projections on Climate Change foresee a future with a greater frequency of extreme sea states (Stott, 2016; Mitchell, 2006). The damages caused by severe wave storms can be considerable not only in infrastructure and buildings but also in the natural habitat, crops and ecosystems affected by erosion and flooding aggravated by the extreme wave heights. In addition, wave storms strongly hamper the maritime activities, especially in harbours. These extreme phenomena drive complex hydrodynamic processes, whose understanding is paramount for proper infrastructure management, design and maintenance (Goda, 2010). In recent years, there have been several studies searching possible trends in wave conditions focusing on both mean and extreme values of significant wave height using a multi-source approach with model reanalysis information with high variability in the time coverage, satellite altimeter records covering the last 30 years and in situ buoy measured data since the 1980s decade but with sparse information and gaps in the time series (e.g. Dodet et al., 2020; Timmermans et al., 2020; Young & Ribal, 2019). These studies highlight a remarkable interannual, seasonal and spatial variability of wave conditions and suggest that the possible observed trends are not clearly associated with anthropogenic forcing (Hochet et al. 2021, 2023).
In the North Atlantic, the mean wave height shows some weak trends not very statistically significant. Young & Ribal (2019) found a mostly positive weak trend in the European Coasts while Timmermans et al.  (2020) showed a weak negative trend in high latitudes, including the North Sea and even more intense in the Norwegian Sea. For extreme values, some authors have found a clearer positive trend in high percentiles (90th-99th) (Young, 2011; Young & Ribal, 2019).

**COPERNICUS MARINE SERVICE KEY FINDINGS**

The mean 99th percentiles showed in the area present a wide range from 2-3.5m in the Canary Island with 0.1-0.3 m of standard deviation (std), 3.5m in the Gulf of Cadiz with 0.5m of std, 3-6m in the English Channel and the Irish Sea with 0.5-0.6m of std, 4-7m in the Bay of Biscay with 0.4-0.9m of std to 8-10m in the West of the British Isles with 0.7-1.4m of std. 
Results for this year show close to zero anomalies in the Canary Island (-0.2/+0.1m), the Gulf of Cadiz (-0.2m) and the English Channel and the Irish Sea (-0.1/+0.1), a general slight negative anomaly in the Bay of Biscay reaching -0.7m but inside the range of the standard deviation, and a positive anomaly (+1.0/+1.55m) in the West of the British Isles, barely out of the standard deviation range in the area. 

**DOI (product):** 
https://doi.org/10.48670/moi-00250

**References:**

* Dodet G, Piolle J-F, Quilfen Y, Abdalla S, Accensi M, Ardhuin F, et al. 2020. The sea state CCI dataset v1: Towards a sea state climate data record based on satellite observations. https://dx.doi.org/10.5194/essd-2019-253
* Mitchell JF, Lowe J, Wood RA, & Vellinga M. 2006. Extreme events due to human-induced climate change. Philosophical Transactions of the Royal Society A: Mathematical, Physical and Engineering Sciences, 364(1845), 2117-2133.
* Pérez-Gómez B, Álvarez-Fanjul E, She J, Pérez-González I, Manzano F. 2016. Extreme sea level events, Section 4.4, p:300. In: Von Schuckmann K, Le Traon PY, Alvarez-Fanjul E, Axell L, Balmaseda M, Breivik LA, Brewin RJW, Bricaud C, Drevillon M, Drillet Y, Dubois C , Embury O, Etienne H, García-Sotillo M, Garric G, Gasparin F, Gutknecht E, Guinehut S, Hernandez F, Juza M, Karlson B, Korres G, Legeais JF, Levier B, Lien VS, Morrow R, Notarstefano G, Parent L, Pascual A, Pérez-Gómez B, Perruche C, Pinardi N, Pisano A, Poulain PM , Pujol IM, Raj RP, Raudsepp U, Roquet H, Samuelsen A, Sathyendranath S, She J, Simoncelli S, Solidoro C, Tinker J, Tintoré J, Viktorsson L, Ablain M, Almroth-Rosell E, Bonaduce A, Clementi E, Cossarini G, Dagneaux Q, Desportes C, Dye S, Fratianni C, Good S, Greiner E, Gourrion J, Hamon M, Holt J, Hyder P, Kennedy J, Manzano-Muñoz F, Melet A, Meyssignac B, Mulet S, Nardelli BB, O’Dea E, Olason E, Paulmier A, Pérez-González I, Reid R, Racault MF, Raitsos DE, Ramos A, Sykes P, Szekely T, Verbrugge N. 2016. The Copernicus Marine Environment Monitoring Service Ocean State Report, Journal of Operational Oceanography. 9 (sup2): 235-320. http://dx.doi.org/10.1080/1755876X.2016.1273446
* Pérez Gómez B, De Alfonso M, Zacharioudaki A, Pérez González I, Álvarez Fanjul E, Müller M, Marcos M, Manzano F, Korres G, Ravdas M, Tamm S. 2018. Sea level, SST and waves: extremes variability. In: Copernicus Marine Service Ocean State Report, Issue 2, Journal of Operational Oceanography, 11:sup1, Chap. 3.1, s79–s88, DOI: https://doi.org/10.1080/1755876X.2018.1489208.
* Stott P. 2016. How climate change affects extreme weather events. Science, 352(6293), 1517-1518.
* Young IR, Zieger S, and Babanin AV. 2011. Global Trends in Wind Speed and Wave Height, Science, 332, 451–455, https://doi.org/10.1126/science.1197219
* Young IR & Ribal A. 2019. Multiplatform evaluation of global trends in wind speed and wave height. Science, 364, 548–552. https://doi.org/10.1126/science.aav9527
* Timmermans BW, Gommenginger CP, Dodet G, Bidlot JR. 2020. Global wave height trends and variability from new multimission satellite altimeter products, reanalyses, and wave buoys, Geophys. Res. Lett., № 47. https://doi.org/10.1029/2019GL086880
* Hochet A, Dodet G, Ardhuin F, Hemer M, Young I. 2021. Sea State Decadal Variability in the North Atlantic: A Review. Climate 2021, 9, 173. https://doi.org/10.3390/cli9120173 Goda Y. 2010. Random seas and design of maritime structures. World scientific. https://doi.org/10.1142/7425.
* Hochet A, Dodet G, Ardhuin F, Hemer M, Young I. 2021. Sea State Decadal Variability in the North Atlantic: A Review. Climate 2021, 9, 173. https://doi.org/10.3390/cli9120173 Goda Y. 2010. Random seas and design of maritime structures. World scientific. https://doi.org/10.1142/7425.





<img src="https://documentation.marine.copernicus.eu/IMG/OMI_EXTREME_WAVE_MEDSEA_swh_mean_and_anomaly_obs.png" width="300" height="200"/>



# Mediterranean Sea significant wave height extreme variability mean and anomaly (observations)


**Product ID:** OMI_EXTREME_WAVE_MEDSEA_swh_mean_and_anomaly_obs

**Description:** **DEFINITION**

The OMI_EXTREME_WAVE_MEDSEA_swh_mean_and_anomaly_obs indicator is based on the computation of the 99th and the 1st percentiles from in situ data (observations). It is computed for the variable significant wave height (swh) measured by in situ buoys. The use of percentiles instead of annual maximum and minimum values, makes this extremes study less affected by individual data measurement errors. The percentiles are temporally averaged, and the spatial evolution is displayed, jointly with the anomaly in the target year. This study of extreme variability was first applied to sea level variable (Pérez Gómez et al 2016) and then extended to other essential variables, sea surface temperature and significant wave height (Pérez Gómez et al 2018). 

**CONTEXT**

Projections on Climate Change foresee a future with a greater frequency of extreme sea states (Stott, 2016; Mitchell, 2006). The damages caused by severe wave storms can be considerable not only in infrastructure and buildings but also in the natural habitat, crops and ecosystems affected by erosion and flooding aggravated by the extreme wave heights. In addition, wave storms strongly hamper the maritime activities, especially in harbours. These extreme phenomena drive complex hydrodynamic processes, whose understanding is paramount for proper infrastructure management, design and maintenance (Goda, 2010). In recent years, there have been several studies searching possible trends in wave conditions focusing on both mean and extreme values of significant wave height using a multi-source approach with model reanalysis information with high variability in the time coverage, satellite altimeter records covering the last 30 years and in situ buoy measured data since the 1980s decade but with sparse information and gaps in the time series (e.g. Dodet et al., 2020; Timmermans et al., 2020; Young & Ribal, 2019). These studies highlight a remarkable interannual, seasonal and spatial variability of wave conditions and suggest that the possible observed trends are not clearly associated with anthropogenic forcing (Hochet et al. 2021, 2023).
For the Mediterranean Sea an interesting publication (De Leo et al., 2024) analyses recent studies in this basin showing the variability in the different results and the difficulties to reach a consensus, especially in the mean wave conditions. The only significant conclusion is the positive trend in extreme values for the western Mediterranean Sea and in particular in the Gulf of Lion and in the Tyrrhenian Sea.

**COPERNICUS MARINE SERVICE KEY FINDINGS**

The mean 99th percentiles showed in the area present a range from 1.5-3.5 in the Gibraltar Strait and Alboran Sea with 0.25-0.55 of standard deviation (std), 2-5m in the East coast of the Iberian Peninsula and Balearic Islands with 0.2-0.4m of std, 3-4m in the Aegean Sea with 0.4-0.6m of std to 3-5m in the Gulf of Lyon with 0.3-0.5m of std. 
Results for this year show a positive anomaly in the Gibraltar Strait (+0.8m), and a negative anomaly in the Aegean Sea (-0.8m), the East Coast of the Iberian Peninsula (-0.7m) and in the Gulf of Lyon (-0.6), all of them slightly over the standard deviation in the respective areas.

**DOI (product):** 
https://doi.org/10.48670/moi-00263

**References:**

* De Leo F, Briganti R & Besio G. 2024. Trends in ocean waves climate within the Mediterranean Sea: a review. Clim Dyn 62, 1555–1566. https://doi.org/10.1007/s00382-023-06984-4
* Mitchell JF, Lowe J, Wood RA, & Vellinga M. 2006. Extreme events due to human-induced climate change. Philosophical Transactions of the Royal Society A: Mathematical, Physical and Engineering Sciences, 364(1845), 2117-2133.
* Pérez-Gómez B, Álvarez-Fanjul E, She J, Pérez-González I, Manzano F. 2016. Extreme sea level events, Section 4.4, p:300. In: Von Schuckmann K, Le Traon PY, Alvarez-Fanjul E, Axell L, Balmaseda M, Breivik LA, Brewin RJW, Bricaud C, Drevillon M, Drillet Y, Dubois C , Embury O, Etienne H, García-Sotillo M, Garric G, Gasparin F, Gutknecht E, Guinehut S, Hernandez F, Juza M, Karlson B, Korres G, Legeais JF, Levier B, Lien VS, Morrow R, Notarstefano G, Parent L, Pascual A, Pérez-Gómez B, Perruche C, Pinardi N, Pisano A, Poulain PM , Pujol IM, Raj RP, Raudsepp U, Roquet H, Samuelsen A, Sathyendranath S, She J, Simoncelli S, Solidoro C, Tinker J, Tintoré J, Viktorsson L, Ablain M, Almroth-Rosell E, Bonaduce A, Clementi E, Cossarini G, Dagneaux Q, Desportes C, Dye S, Fratianni C, Good S, Greiner E, Gourrion J, Hamon M, Holt J, Hyder P, Kennedy J, Manzano-Muñoz F, Melet A, Meyssignac B, Mulet S, Nardelli BB, O’Dea E, Olason E, Paulmier A, Pérez-González I, Reid R, Racault MF, Raitsos DE, Ramos A, Sykes P, Szekely T, Verbrugge N. 2016. The Copernicus Marine Environment Monitoring Service Ocean State Report, Journal of Operational Oceanography. 9 (sup2): 235-320. http://dx.doi.org/10.1080/1755876X.2016.1273446
* Pérez Gómez B, De Alfonso M, Zacharioudaki A, Pérez González I, Álvarez Fanjul E, Müller M, Marcos M, Manzano F, Korres G, Ravdas M, Tamm S. 2018. Sea level, SST and waves: extremes variability. In: Copernicus Marine Service Ocean State Report, Issue 2, Journal of Operational Oceanography, 11:sup1, Chap. 3.1, s79–s88, DOI: https://doi.org/10.1080/1755876X.2018.1489208.
* Stott P. 2016. How climate change affects extreme weather events. Science, 352(6293), 1517-1518.
* Timmermans BW, Gommenginger CP, Dodet G, Bidlot JR. 2020. Global wave height trends and variability from new multimission satellite altimeter products, reanalyses, and wave buoys, Geophys. Res. Lett., № 47. https://doi.org/10.1029/2019GL086880
* Young IR & Ribal A. 2019. Multiplatform evaluation of global trends in wind speed and wave height. Science, 364, 548–552. https://doi.org/10.1126/science.aav9527
* Dodet G, Piolle J-F, Quilfen Y, Abdalla S, Accensi M, Ardhuin F, et al. 2020. The sea state CCI dataset v1: Towards a sea state climate data record based on satellite observations. https://dx.doi.org/10.5194/essd-2019-253
* Hochet A, Dodet G, Sévellec F, Bouin M-N, Patra A, & Ardhuin F. 2023. Time of emergence for altimetry-based significant wave height changes in the North Atlantic. Geophysical Research Letters, 50, e2022GL102348. https://doi.org/10.1029/2022GL102348
* Hochet A, Dodet G, Ardhuin F, Hemer M, Young I. 2021. Sea State Decadal Variability in the North Atlantic: A Review. Climate 2021, 9, 173. https://doi.org/10.3390/cli9120173 Goda Y. 2010. Random seas and design of maritime structures. World scientific. https://doi.org/10.1142/7425.





<img src="https://documentation.marine.copernicus.eu/IMG/OMI_EXTREME_WAVE_NORTHWESTSHELF_swh_mean_and_anomaly_obs.png" width="300" height="200"/>



# North West Shelf significant wave height extreme variability mean and anomaly (observations)


**Product ID:** OMI_EXTREME_WAVE_NORTHWESTSHELF_swh_mean_and_anomaly_obs

**Description:** **DEFINITION**

The OMI_EXTREME_WAVE_NORTHWESTSHELF_swh_mean_and_anomaly_obs indicator is based on the computation of the 99th and the 1st percentiles from in situ data (observations). It is computed for the variable significant wave height (swh) measured by in situ buoys. The use of percentiles instead of annual maximum and minimum values, makes this extremes study less affected by individual data measurement errors. The percentiles are temporally averaged, and the spatial evolution is displayed, jointly with the anomaly in the target year. This study of extreme variability was first applied to sea level variable (Pérez Gómez et al 2016) and then extended to other essential variables, sea surface temperature and significant wave height (Pérez Gómez et al 2018). 

**CONTEXT**

Projections on Climate Change foresee a future with a greater frequency of extreme sea states (Stott, 2016; Mitchell, 2006). The damages caused by severe wave storms can be considerable not only in infrastructure and buildings but also in the natural habitat, crops and ecosystems affected by erosion and flooding aggravated by the extreme wave heights. In addition, wave storms strongly hamper the maritime activities, especially in harbours. These extreme phenomena drive complex hydrodynamic processes, whose understanding is paramount for proper infrastructure management, design and maintenance (Goda, 2010). In recent years, there have been several studies searching possible trends in wave conditions focusing on both mean and extreme values of significant wave height using a multi-source approach with model reanalysis information with high variability in the time coverage, satellite altimeter records covering the last 30 years and in situ buoy measured data since the 1980s decade but with sparse information and gaps in the time series (e.g. Dodet et al., 2020; Timmermans et al., 2020; Young & Ribal, 2019). These studies highlight a remarkable interannual, seasonal and spatial variability of wave conditions and suggest that the possible observed trends are not clearly associated with anthropogenic forcing (Hochet et al. 2021, 2023).
In the North Atlantic, the mean wave height shows some weak trends not very statistically significant. Young & Ribal (2019) found a mostly positive weak trend in the European Coasts while Timmermans et al.  (2020) showed a weak negative trend in high latitudes, including the North Sea and even more intense in the Norwegian Sea. For extreme values, some authors have found a clearer positive trend in high percentiles (90th-99th) (Young et al., 2011; Young & Ribal, 2019).

**COPERNICUS MARINE SERVICE KEY FINDINGS** 

The mean 99th percentiles showed in the area present a wide range from 2.5 meters in the English Channel with 0.3m of standard deviation (std), 3-5m in the southern and central North Sea with 0.3-0.6m of std, 4 meters in the Skagerrak Strait with 0.6m of std, 7.5m in the northern North Sea with 0.6m of std to 8 meters in the North of the British Isles with 0.6m of std. 
Results for this year show either low positive or negative anomalies between -0.6m and +0.4m, inside the margin of the standard deviation for all the stations. 

**DOI (product):** 
https://doi.org/10.48670/moi-00270

**References:**

* Dodet G, Piolle J-F, Quilfen Y, Abdalla S, Accensi M, Ardhuin F, et al. 2020. The sea state CCI dataset v1: Towards a sea state climate data record based on satellite observations. https://dx.doi.org/10.5194/essd-2019-253
* Mitchell JF, Lowe J, Wood RA, & Vellinga M. 2006. Extreme events due to human-induced climate change. Philosophical Transactions of the Royal Society A: Mathematical, Physical and Engineering Sciences, 364(1845), 2117-2133.
* Pérez-Gómez B, Álvarez-Fanjul E, She J, Pérez-González I, Manzano F. 2016. Extreme sea level events, Section 4.4, p:300. In: Von Schuckmann K, Le Traon PY, Alvarez-Fanjul E, Axell L, Balmaseda M, Breivik LA, Brewin RJW, Bricaud C, Drevillon M, Drillet Y, Dubois C , Embury O, Etienne H, García-Sotillo M, Garric G, Gasparin F, Gutknecht E, Guinehut S, Hernandez F, Juza M, Karlson B, Korres G, Legeais JF, Levier B, Lien VS, Morrow R, Notarstefano G, Parent L, Pascual A, Pérez-Gómez B, Perruche C, Pinardi N, Pisano A, Poulain PM , Pujol IM, Raj RP, Raudsepp U, Roquet H, Samuelsen A, Sathyendranath S, She J, Simoncelli S, Solidoro C, Tinker J, Tintoré J, Viktorsson L, Ablain M, Almroth-Rosell E, Bonaduce A, Clementi E, Cossarini G, Dagneaux Q, Desportes C, Dye S, Fratianni C, Good S, Greiner E, Gourrion J, Hamon M, Holt J, Hyder P, Kennedy J, Manzano-Muñoz F, Melet A, Meyssignac B, Mulet S, Nardelli BB, O’Dea E, Olason E, Paulmier A, Pérez-González I, Reid R, Racault MF, Raitsos DE, Ramos A, Sykes P, Szekely T, Verbrugge N. 2016. The Copernicus Marine Environment Monitoring Service Ocean State Report, Journal of Operational Oceanography. 9 (sup2): 235-320. http://dx.doi.org/10.1080/1755876X.2016.1273446
* Pérez Gómez B, De Alfonso M, Zacharioudaki A, Pérez González I, Álvarez Fanjul E, Müller M, Marcos M, Manzano F, Korres G, Ravdas M, Tamm S. 2018. Sea level, SST and waves: extremes variability. In: Copernicus Marine Service Ocean State Report, Issue 2, Journal of Operational Oceanography, 11:sup1, Chap. 3.1, s79–s88, DOI: https://doi.org/10.1080/1755876X.2018.1489208.
* Stott P. 2016. How climate change affects extreme weather events. Science, 352(6293), 1517-1518.
* Timmermans BW, Gommenginger CP, Dodet G, Bidlot JR. 2020. Global wave height trends and variability from new multimission satellite altimeter products, reanalyses, and wave buoys, Geophys. Res. Lett., № 47. https://doi.org/10.1029/2019GL086880
* Young IR, Zieger S, and Babanin AV. 2011. Global Trends in Wind Speed and Wave Height, Science, 332, 451–455, https://doi.org/10.1126/science.1197219
* Young IR & Ribal A. 2019. Multiplatform evaluation of global trends in wind speed and wave height. Science, 364, 548–552. https://doi.org/10.1126/science.aav9527
* Hochet A, Dodet G, Sévellec F, Bouin M-N, Patra A, & Ardhuin F. 2023. Time of emergence for altimetry-based significant wave height changes in the North Atlantic. Geophysical Research Letters, 50, e2022GL102348. https://doi.org/10.1029/2022GL102348
* Hochet A, Dodet G, Ardhuin F, Hemer M, Young I. 2021. Sea State Decadal Variability in the North Atlantic: A Review. Climate 2021, 9, 173. https://doi.org/10.3390/cli9120173 Goda Y. 2010. Random seas and design of maritime structures. World scientific. https://doi.org/10.1142/7425.





<img src="https://documentation.marine.copernicus.eu/IMG/OMI_HEALTH_CHL_ARCTIC_OCEANCOLOUR_area_averaged_mean.png" width="300" height="200"/>



# Arctic Ocean Chlorophyll-a time series and trend from Observations Reprocessing


**Product ID:** OMI_HEALTH_CHL_ARCTIC_OCEANCOLOUR_area_averaged_mean

**Description:** **DEFINITION**

The time series are derived from the regional chlorophyll reprocessed (REP) products as distributed by CMEMS which, in turn, result from the application of the regional chlorophyll algorithms to remote sensing reflectances (Rrs) provided by the ESA Ocean Colour Climate Change Initiative (ESA OC-CCI, Sathyendranath et al. 2019; Jackson 2020). Daily regional mean values are calculated by performing the average (weighted by pixel area) over the region of interest. A fixed annual cycle is extracted from the original signal, using the Census-I method as described in Vantrepotte et al. (2009). The deasonalised time series is derived by subtracting the seasonal cycle from the original time series, and then fitted to a linear regression to, finally, obtain the linear trend. 

**CONTEXT**

Phytoplankton – and chlorophyll concentration , which is  a measure of  phytoplankton concentration – respond rapidly to changes in environmental conditions. Chlorophyll concentration is highly seasonal in the Arctic Ocean region due to a strong dependency on light and nutrient availability, which in turn are driven by seasonal sunlight and sea-ice cover dynamics, as well as changes in mixed layer. In the past two decades, an increase in annual net primary production by Arctic Ocean phytoplankton has been observed and linked to sea-ice decline (Arrigo and van Dijken, 2015); in the same line Kahru et al. (2011) have showed that chlorophyll concentration peaks are appearing increasingly earlier in the year in parts of the Arctic. It is therefore of critical importance to monitor chlorophyll concentration at multiple temporal and spatial scales in the area, in order to be able to separate potential long-term climate signals from natural variability in the short term.

**CMEMS KEY FINDINGS**

While the overall trend average for the 1997-2021 period in the Arctic Sea is positive (0.86 ± 0.17 % per year), a continued plateau in the linear trend, initiated in 2013 is observed in the time series extension, with both the amplitude and the baseline of the cycle continuing to decrease during 2021 as reported for previous years (Sathyendranath et al., 2018). In particular, the annual average for the region in 2021 is 1.05 mg m-3 - a 30% reduction on 2020 values. There appears to be no appreciable changes in the timings or amplitude of the 2021 spring and autumn blooms. 

**DOI (product):** 
https://doi.org/10.48670/moi-00188

**References:**

* Arrigo, K. R., & van Dijken, G. L., 2015. Continued increases in Arctic Ocean primary production. Progress in Oceanography, 136, 60–70. doi: 10.1016/j.pocean.2015.05.002.
* Kahru, M., Brotas, V., Manzano‐Sarabia, M., Mitchell, B. G., 2011. Are phytoplankton blooms occurring earlier in the Arctic? Global Change Biology, 17(4), 1733–1739. doi:10.1111/j.1365‐2486.2010.02312.x.
* Jackson, T. (2020) OC-CCI Product User Guide (PUG). ESA/ESRIN Report. D4.2PUG, 2020-10-12. Issue:v4.2. https://docs.pml.space/share/s/okB2fOuPT7Cj2r4C5sppDg
* Sathyendranath, S., Pardo, S., Benincasa, M., Brando, V. E., Brewin, R. J.W., Mélin, F., Santoleri, R., 2018. 1.5. Essential Variables: Ocean Colour in Copernicus Marine Service Ocean State Report - Issue 2, Journal of Operational Oceanography, 11:sup1, 1-142, doi: 10.1080/1755876X.2018.1489208
* Sathyendranath, S, Brewin, RJW, Brockmann, C, Brotas, V, Calton, B, Chuprin, A, Cipollini, P, Couto, AB, Dingle, J, Doerffer, R, Donlon, C, Dowell, M, Farman, A, Grant, M, Groom, S, Horseman, A, Jackson, T, Krasemann, H, Lavender, S, Martinez-Vicente, V, Mazeran, C, Mélin, F, Moore, TS, Müller, D, Regner, P, Roy, S, Steele, CJ, Steinmetz, F, Swinton, J, Taberner, M, Thompson, A, Valente, A, Zühlke, M, Brando, VE, Feng, H, Feldman, G, Franz, BA, Frouin, R, Gould, Jr., RW, Hooker, SB, Kahru, M, Kratzer, S, Mitchell, BG, Muller-Karger, F, Sosik, HM, Voss, KJ, Werdell, J, and Platt, T (2019) An ocean-colour time series for use in climate studies: the experience of the Ocean-Colour Climate Change Initiative (OC-CCI). Sensors: 19, 4285. doi:10.3390/s19194285
* Vantrepotte, V., Mélin, F., 2009. Temporal variability of 10-year global SeaWiFS time series of phytoplankton chlorophyll-a concentration. ICES J. Mar. Sci., 66, 1547-1556. 10.1093/icesjms/fsp107.





<img src="https://documentation.marine.copernicus.eu/IMG/OMI_HEALTH_CHL_ATLANTIC_OCEANCOLOUR_area_averaged_mean.png" width="300" height="200"/>



# North Atlantic Ocean Chlorophyll-a time series and trend from Observations Reprocessing


**Product ID:** OMI_HEALTH_CHL_ATLANTIC_OCEANCOLOUR_area_averaged_mean

**Description:** **DEFINITION**

The time series are derived from the regional chlorophyll reprocessed (REP) products as distributed by CMEMS which, in turn, result from the application of the regional chlorophyll algorithms over remote sensing reflectances (Rrs) provided by the ESA Ocean Colour Climate Change Initiative (ESA OC-CCI, Sathyendranath et al. 2019; Jackson 2020). Daily regional mean values are calculated by performing the average (weighted by pixel area) over the region of interest. A fixed annual cycle is extracted from the original signal, using the Census-I method as described in Vantrepotte et al. (2009). The deseasonalised time series is derived by subtracting the mean seasonal cycle from the original time series, and then fitted to a linear regression to, finally, obtain the linear trend. 

**CONTEXT**

Phytoplankton – and chlorophyll concentration as a proxy for phytoplankton – respond rapidly to changes in environmental conditions, such as temperature, light and nutrients availability, and mixing. The response in the North Atlantic ranges from cyclical to decadal oscillations (Henson et al., 2009); it is therefore of critical importance to monitor chlorophyll concentration at multiple temporal and spatial scales, in order to be able to separate potential long-term climate signals from natural variability in the short term. In particular, phytoplankton in the North Atlantic are known to respond to climate variability associated with the North Atlantic Oscillation (NAO), with the initiation of the spring bloom showing a nominal correlation with sea surface temperature and the NAO index (Zhai et al., 2013).

**CMEMS KEY FINDINGS**

While the overall trend average for the 1997-2021 period in the North Atlantic Ocean is slightly positive (0.16 ± 0.12 % per year), an underlying low frequency harmonic signal can be seen in the deseasonalised data. The annual average for the region in 2021 is 0.25 mg m-3. Though no appreciable changes in the timing of the spring and autumn blooms have been observed during 2021, a lower peak chlorophyll concentration is observed in the timeseries extension. This decrease in peak concentration with respect to the previous year is contributing to the reduction trend.

**DOI (product):**
https://doi.org/10.48670/moi-00194

**References:**

* Henson, S. A., Dunne, J. P. , and Sarmiento, J. L., 2009, Decadal variability in North Atlantic phytoplankton blooms, J. Geophys. Res., 114, C04013, doi:10.1029/2008JC005139.
* Jackson, T. (2020) OC-CCI Product User Guide (PUG). ESA/ESRIN Report. D4.2PUG, 2020-10-12. Issue:v4.2. https://docs.pml.space/share/s/okB2fOuPT7Cj2r4C5sppDg
* Sathyendranath, S., Pardo, S., Benincasa, M., Brando, V. E., Brewin, R. J.W., Mélin, F., Santoleri, R., 2018, 1.5. Essential Variables: Ocean Colour in Copernicus Marine Service Ocean State Report - Issue 2, Journal of Operational Oceanography, 11:sup1, 1-142, doi: 10.1080/1755876X.2018.1489208
* Sathyendranath, S, Brewin, RJW, Brockmann, C, Brotas, V, Calton, B, Chuprin, A, Cipollini, P, Couto, AB, Dingle, J, Doerffer, R, Donlon, C, Dowell, M, Farman, A, Grant, M, Groom, S, Horseman, A, Jackson, T, Krasemann, H, Lavender, S, Martinez-Vicente, V, Mazeran, C, Mélin, F, Moore, TS, Müller, D, Regner, P, Roy, S, Steele, CJ, Steinmetz, F, Swinton, J, Taberner, M, Thompson, A, Valente, A, Zühlke, M, Brando, VE, Feng, H, Feldman, G, Franz, BA, Frouin, R, Gould, Jr., RW, Hooker, SB, Kahru, M, Kratzer, S, Mitchell, BG, Muller-Karger, F, Sosik, HM, Voss, KJ, Werdell, J, and Platt, T (2019) An ocean-colour time series for use in climate studies: the experience of the Ocean-Colour Climate Change Initiative (OC-CCI). Sensors: 19, 4285. doi:10.3390/s19194285
* Vantrepotte, V., Mélin, F., 2009. Temporal variability of 10-year global SeaWiFS time series of phytoplankton chlorophyll-a concentration. ICES J. Mar. Sci., 66, 1547-1556. doi: 10.1093/icesjms/fsp107.
* Zhai, L., Platt, T., Tang, C., Sathyendranath, S., Walne, A., 2013. The response of phytoplankton to climate variability associated with the North Atlantic Oscillation, Deep Sea Research Part II: Topical Studies in Oceanography, 93, 159-168, doi: 10.1016/j.dsr2.2013.04.009.





<img src="https://documentation.marine.copernicus.eu/IMG/OMI_HEALTH_CHL_ATLANTIC_OCEANCOLOUR_eutrophication.png" width="300" height="200"/>



# North Atlantic Ocean Eutrophication from Observations Reprocessing


**Product ID:** OMI_HEALTH_CHL_ATLANTIC_OCEANCOLOUR_eutrophication

**Description:** **DEFINITION**

We have derived an annual eutrophication and eutrophication indicator map for the North Atlantic Ocean using satellite-derived chlorophyll concentration. Using the satellite-derived chlorophyll products distributed in the regional North Atlantic CMEMS MY Ocean Colour dataset (OC- CCI), we derived P90 and P10 daily climatologies. The time period selected for the climatology was 1998-2017. For a given pixel, P90 and P10 were defined as dynamic thresholds such as 90% of the 1998-2017 chlorophyll values for that pixel were below the P90 value, and 10% of the chlorophyll values were below the P10 value.  To minimise the effect of gaps in the data in the computation of these P90 and P10 climatological values, we imposed a threshold of 25% valid data for the daily climatology. For the 20-year 1998-2017 climatology this means that, for a given pixel and day of the year, at least 5 years must contain valid data for the resulting climatological value to be considered significant. Pixels where the minimum data requirements were met were not considered in further calculations.
 We compared every valid daily observation over 2021 with the corresponding daily climatology on a pixel-by-pixel basis, to determine if values were above the P90 threshold, below the P10 threshold or within the [P10, P90] range. Values above the P90 threshold or below the P10 were flagged as anomalous. The number of anomalous and total valid observations were stored during this process. We then calculated the percentage of valid anomalous observations (above/below the P90/P10 thresholds) for each pixel, to create percentile anomaly maps in terms of % days per year. Finally, we derived an annual indicator map for eutrophication levels: if 25% of the valid observations for a given pixel and year were above the P90 threshold, the pixel was flagged as eutrophic. Similarly, if 25% of the observations for a given pixel were below the P10 threshold, the pixel was flagged as oligotrophic.

**CONTEXT**

Eutrophication is the process by which an excess of nutrients – mainly phosphorus and nitrogen – in a water body leads to increased growth of plant material in an aquatic body. Anthropogenic activities, such as farming, agriculture, aquaculture and industry, are the main source of nutrient input in problem areas (Jickells, 1998; Schindler, 2006; Galloway et al., 2008). Eutrophication is an issue particularly in coastal regions and areas with restricted water flow, such as lakes and rivers (Howarth and Marino, 2006; Smith, 2003). The impact of eutrophication on aquatic ecosystems is well known: nutrient availability boosts plant growth – particularly algal blooms – resulting in a decrease in water quality (Anderson et al., 2002; Howarth et al.; 2000). This can, in turn, cause death by hypoxia of aquatic organisms (Breitburg et al., 2018), ultimately driving changes in community composition (Van Meerssche et al., 2019). Eutrophication has also been linked to changes in the pH (Cai et al., 2011, Wallace et al. 2014) and depletion of inorganic carbon in the aquatic environment (Balmer and Downing, 2011). Oligotrophication is the opposite of eutrophication, where reduction in some limiting resource leads to a decrease in photosynthesis by aquatic plants, reducing the capacity of the ecosystem to sustain the higher organisms in it. 
Eutrophication is one of the more long-lasting water quality problems in Europe (OSPAR ICG-EUT, 2017), and is on the forefront of most European Directives on water-protection. Efforts to reduce anthropogenically-induced pollution resulted in the implementation of the Water Framework Directive (WFD) in 2000. 

**CMEMS KEY FINDINGS**

The coastal and shelf waters, especially between 30 and 400N that showed active oligotrophication flags for 2020 have reduced in 2021 and a reversal to eutrophic flags can be seen in places. Again, the eutrophication index is positive only for a small number of coastal locations just north of 40oN in 2021, however south of 40oN there has been a significant increase in eutrophic flags, particularly around the Azores.  In general, the 2021 indicator map showed an increase in oligotrophic areas in the Northern Atlantic and an increase in eutrophic areas in the Southern Atlantic. The Third Integrated Report on the Eutrophication Status of the OSPAR Maritime Area (OSPAR ICG-EUT, 2017) reported an improvement from 2008 to 2017 in eutrophication status across offshore and outer coastal waters of the Greater North Sea, with a decrease in the size of coastal problem areas in Denmark, France, Germany, Ireland, Norway and the United Kingdom.

**DOI (product):**
https://doi.org/10.48670/moi-00195

**References:**

* Anderson, D.M., Glibert, P.M. & Burkholder, J.M. (2002). Harmful algal blooms and eutrophication: Nutrient sources, composition, and consequences. Estuaries 25, 704–726 /10.1007/BF02804901.
* Balmer, M.B., Downing, J.A. (2011), Carbon dioxide concentrations in eutrophic lakes: undersaturation implies atmospheric uptake, Inland Waters, 1:2, 125-132, 10.5268/IW-1.2.366.
* Breitburg, D., Levin, L.A., Oschlies, A., Grégoire, M., Chavez, F.P., Conley, D.J., Garçon, V., Gilbert, D., Gutiérrez, D., Isensee, K. and Jacinto, G.S. (2018). Declining oxygen in the global ocean and coastal waters. Science, 359 (6371), p.eaam7240.
* Cai, W., Hu, X., Huang, W. (2011) Acidification of subsurface coastal waters enhanced by eutrophication. Nature Geosci 4, 766–770, 10.1038/ngeo1297.
* Galloway, J.N., Townsend, A.R., Erisman, J.W., Bekunda, M., Cai, Z., Freney, J. R., Martinelli, L. A., Seitzinger, S. P., Sutton, M. A. (2008). Transformation of the Nitrogen Cycle: Recent Trends, Questions, and Potential Solutions, Science 320, 5878, 889-892, 10.1126/science.1136674.
* Howarth, R.W., Anderson, D., Cloern, J., Elfring, C., Hopkinson, C., Lapointe, B., Malone, T., & Marcus, N., McGlathery, K., Sharpley, A., Walker, D. (2000). Nutrient pollution of coastal rivers, bays and seas. Issues in Ecology, 7.
* Howarth, R.W., Marino, R. (2006). Nitrogen as the limiting nutrient for eutrophication in coastal marine ecosystems: Evolving views over three decades, Limnology and Oceanography, 51(1, part 2), 10.4319/lo.2006.51.1_part_2.0364.
* Jickells, T. D. (1998). Nutrient biogeochemistry of the coastal zone. Science 281, 217–222. doi: 10.1126/science.281.5374.217
* OSPAR ICG-EUT. Axe, P., Clausen, U., Leujak, W., Malcolm, S., Ruiter, H., Prins, T., Harvey, E.T. (2017). Eutrophication Status of the OSPAR Maritime Area. Third Integrated Report on the Eutrophication Status of the OSPAR Maritime Area.
* Schindler, D. W. (2006) Recent advances in the understanding and management of eutrophication. Limnology and Oceanography, 51, 356-363.
* Smith, V.H. (2003). Eutrophication of freshwater and coastal marine ecosystems a global problem. Environmental Science and Pollution Research, 10, 126–139, 10.1065/espr2002.12.142.
* Van Meerssche, E., Pinckney, J.L. (2019) Nutrient Loading Impacts on Estuarine Phytoplankton Size and Community Composition: Community-Based Indicators of Eutrophication. Estuaries and Coasts 42, 504–512, 10.1007/s12237-018-0470-z.
* Wallace, R.B., Baumann, H., Grear, J.S., Aller, R.B., Gobler, C.J. (2014). Coastal ocean acidification: The other eutrophication problem, Estuarine, Coastal and Shelf Science, 148, 1-13, 10.1016/j.ecss.2014.05.027.





<img src="https://documentation.marine.copernicus.eu/IMG/OMI_HEALTH_CHL_BALTIC_area_averaged_mean.png" width="300" height="200"/>



# Baltic Sea Chlorophyll-a time series and trend from Observations Reprocessing


**Product ID:** OMI_HEALTH_CHL_BALTIC_OCEANCOLOUR_area_averaged_mean

**Description:** **DEFINITION**

The time series are derived from the regional chlorophyll reprocessed (MY) product as distributed by CMEMS which, in turn, result from the application of the regional chlorophyll algorithm over remote sensing reflectances (Rrs) provided by the Plymouth Marine Laboratory using an ad-hoc configuration for CMEMS of the ESA OC-CCI processor version 6 (OC-CCIv6) to merge at 1km resolution (rather than at 4km as for OC-CCI) MERIS, MODIS-AQUA, SeaWiFS, NPP-VIIRS and OLCI-A data. The chlorophyll product is derived from a Multi-Layer Perceptron neural-net (MLP) developed on field measurements collected within the BiOMaP program of JRC/EC (Zibordi et al., 2011). The algorithm is an ensemble of different MLPs that use Rrs at different wavelengths as input. The processing chain and the techniques used to develop the algorithm are detailed in Brando et al. (2021a; 2021b). 
Monthly regional mean values are calculated by performing the average of 2D monthly mean (weighted by pixel area) over the region of interest. The deseasonalized time series is obtained by applying the X-11 seasonal adjustment methodology on the original time series as described in Colella et al. (2016), and then the Mann-Kendall test (Mann, 1945; Kendall, 1975) and Sens’s method (Sen, 1968) are subsequently applied to obtain the magnitude of trend.

**CONTEXT**

Phytoplankton   and chlorophyll concentration as a proxy for phytoplankton   respond rapidly to changes in environmental conditions, such as light, temperature, nutrients and mixing (Gregg and Rousseaux, 2014). The character of the response in the Baltic Sea depends on the nature of the change drivers, and ranges from seasonal cycles to decadal oscillations (Kahru and Elmgren 2014). Therefore, it is of critical importance to monitor chlorophyll concentration at multiple temporal and spatial scales, in order to be able to separate potential long-term climate signals from natural variability in the short term. In particular, in the Baltic Sea phytoplankton is known to respond to the variations of SST in the basin associated with climate variability (Kabel et al. 2012).

**KEY FINDINGS**

The Baltic Sea shows a slight positive trend over the 1997-2023 period, with a slope of 0.30±0.49% per year, indicating a decrease compared to the previous release. The maxima and minima values are relatively consistent year-to-year, with the absolute maximum occurring in 2008 and the minima observed in 2004 and 2014. A decrease in the chlorophyll signal has been evident over the past two years.

**DOI (product):**
https://doi.org/10.48670/moi-00197

**References:**

* Brando, V.E., A. Di Cicco, M. Sammartino, S. Colella, D D’Alimonte, T. Kajiyama, S. Kaitala, J. Attila, 2021a. OCEAN COLOUR PRODUCTION CENTRE, Baltic Sea Observation Products. Copernicus Marine Environment Monitoring Centre. Quality Information Document (https://documentation.marine.copernicus.eu/QUID/CMEMS-OC-QUID-009-131to134.pdf).
* Brando, V.E.; Sammartino, M; Colella, S.; Bracaglia, M.; Di Cicco, A; D’Alimonte, D.; Kajiyama, T., Kaitala, S., Attila, J., 2021b (accepted). Phytoplankton Bloom Dynamics in the Baltic Sea Using a Consistently Reprocessed Time Series of Multi-Sensor Reflectance and Novel Chlorophyll-a Retrievals. Remote Sens. 2021, 13, x.
* Colella, S., Falcini, F., Rinaldi, E., Sammartino, M., & Santoleri, R. (2016). Mediterranean ocean colour chlorophyll trends. PloS one, 11(6).
* Gregg, W. W., and C. S. Rousseaux, 2014. Decadal Trends in Global Pelagic Ocean Chlorophyll: A New Assessment Integrating Multiple Satellites, in Situ Data, and Models. Journal of Geophysical Research Oceans 119. doi:10.1002/2014JC010158.
* Kabel K, Moros M, Porsche C, Neumann T, Adolphi F, Andersen TJ, Siegel H, Gerth M, Leipe T, Jansen E, Sinninghe Damsté JS. 2012. Impact of climate change on the health of the Baltic Sea ecosystem over the last 1000 years. Nat Clim Change. doi:10.1038/nclimate1595.
* Kahru, M. and Elmgren, R.: Multidecadal time series of satellite- detected accumulations of cyanobacteria in the Baltic Sea, Biogeosciences, 11, 3619 3633, doi:10.5194/bg-11-3619-2014, 2014.
* Kendall MG. 1975. Multivariate analysis. London: Charles Griffin & Co; p. 210, 43.
* Mann HB. 1945. Nonparametric tests against trend. Econometrica. 13:245 259. p. 42.
* Sathyendranath, S., et al., 2018. ESA Ocean Colour Climate Change Initiative (Ocean_Colour_cci): Version 3.1. Technical Report Centre for Environmental Data Analysis. doi:10.5285/9c334fbe6d424a708cf3c4cf0c6a53f5.
* Sen PK. 1968. Estimates of the regression coefficient based on Kendall’s tau. J Am Statist Assoc. 63:1379 1389.
* Zibordi, G., Berthon, J.-F., Mélin, F., and D’Alimonte, D.: Cross- site consistent in situ measurements for satellite ocean color ap- plications: the BiOMaP radiometric dataset, Rem. Sens. Environ., 115, 2104–2115, 2011.





<img src="https://documentation.marine.copernicus.eu/IMG/OMI_HEALTH_CHL_BALTIC_trend.png" width="300" height="200"/>



# Baltic Sea Chlorophyll-a trend map from Observations Reprocessing


**Product ID:** OMI_HEALTH_CHL_BALTIC_OCEANCOLOUR_trend

**Description:** **DEFINITION**

This product includes the Baltic Sea satellite chlorophyll trend map based on regional chlorophyll reprocessed (MY) product as distributed by CMEMS OC-TAC which, in turn, result from the application of the regional chlorophyll algorithms over remote sensing reflectances (Rrs) provided by the Plymouth Marine Laboratory (PML) using an ad-hoc configuration for CMEMS of the ESA OC-CCI processor version 6 (OC-CCIv6) to merge at 1km resolution (rather than at 4km as for OC-CCI) MERIS, MODIS-AQUA, SeaWiFS, NPP-VIIRS and OLCI-A data. The chlorophyll product is derived from a Multi Layer Perceptron neural-net (MLP) developed on field measurements collected within the BiOMaP program of JRC/EC (Zibordi et al., 2011). The algorithm is an ensemble of different MLPs that use Rrs at different wavelengths as input. The processing chain and the techniques used to develop the algorithm are detailed in Brando et al. (2021a; 2021b).
The trend map is obtained by applying Colella et al. (2016) methodology, where the Mann-Kendall test (Mann, 1945; Kendall, 1975) and Sens’s method (Sen, 1968) are applied on deseasonalized monthly time series, as obtained from the X-11 technique (see e. g. Pezzulli et al. 2005), to estimate, trend magnitude and its significance. The trend is expressed in % per year that represents the relative changes (i.e., percentage) corresponding to the dimensional trend [mg m-3 y-1] with respect to the reference climatology (1997-2014). Only significant trends (p < 0.05) are included.

**CONTEXT**

Phytoplankton are key actors in the carbon cycle and, as such, recognised as an Essential Climate Variable (ECV). Chlorophyll concentration - as a proxy for phytoplankton - respond rapidly to changes in environmental conditions, such as light, temperature, nutrients and mixing (Colella et al. 2016). The character of the response in the Baltic Sea depends on the nature of the change drivers, and ranges from seasonal cycles to decadal oscillations (Kahru and Elmgren 2014) and anthropogenic climate change. Eutrophication is one of the most important issues for the Baltic Sea (HELCOM, 2018), therefore the use of long-term time series of consistent, well-calibrated, climate-quality data record is crucial for detecting eutrophication. Furthermore, chlorophyll analysis also demands the use of robust statistical temporal decomposition techniques, in order to separate the long-term signal from the seasonal component of the time series.

**KEY FINDINGS**

On average, the trend for the Baltic Sea over the 1997-2023 period is relatively flat (0.08%). The pattern of positive and negative trends is quite similar to the previous release, indicating a general decrease in absolute values. This result aligns with the findings of Sathyendranath et al. (2018), which show an increasing trend in chlorophyll concentration in most of the European Seas.

**DOI (product):**
https://doi.org/10.48670/moi-00198

**References:**

* Brando, V.E., A. Di Cicco, M. Sammartino, S. Colella, D D’Alimonte, T. Kajiyama, S. Kaitala, J. Attila, 2021a. OCEAN COLOUR PRODUCTION CENTRE, Baltic Sea Observation Products. Copernicus Marine Environment Monitoring Centre. Quality Information Document (https://documentation.marine.copernicus.eu/QUID/CMEMS-OC-QUID-009-131to134.pdf).
* Brando, V.E.; Sammartino, M; Colella, S.; Bracaglia, M.; Di Cicco, A; D’Alimonte, D.; Kajiyama, T., Kaitala, S., Attila, J., 2021b. Phytoplankton bloom dynamics in the Baltic sea using a consistently reprocessed time series of multi-sensor reflectance and novel chlorophyll-a retrievals. Remote Sensing, 13(16), 3071.
* Colella, S., Falcini, F., Rinaldi, E., Sammartino, M., & Santoleri, R. (2016). Mediterranean ocean colour chlorophyll trends. PloS one, 11(6).
* HELCOM (2018): HELCOM Thematic assessment of eutrophication 2011-2016. Baltic Sea Environment Proceedings No. 156.
* Kahru, M. and Elmgren, R.: Multidecadal time series of satellite- detected accumulations of cyanobacteria in the Baltic Sea, Biogeosciences, 11, 3619 3633, doi:10.5194/bg-11-3619-2014, 2014.
* Kendall MG. 1975. Multivariate analysis. London: Charles Griffin & Co; p. 210, 43.
* Mann HB. 1945. Nonparametric tests against trend. Econometrica. 13:245–259. p. 42.
* Pezzulli S, Stephenson DB, Hannachi A. 2005. The Variability of Seasonality. J. Climate. 18:71–88. doi:10.1175/JCLI-3256.1.
* Sathyendranath, S., Pardo, S., Benincasa, M., Brando, V. E., Brewin, R. J.W., Mélin, F., Santoleri, R., 2018, 1.5. Essential Variables: Ocean Colour in Copernicus Marine Service Ocean State Report - Issue 2, Journal of Operational Oceanography, 11:sup1, 1-142, doi: 10.1080/1755876X.2018.1489208.
* Sen PK. 1968. Estimates of the regression coefficient based on Kendall’s tau. J Am Statist Assoc. 63:1379–1389.
* Zibordi, G., Berthon, J.-F., Mélin, F., and D’Alimonte, D.: Cross- site consistent in situ measurements for satellite ocean color ap- plications: the BiOMaP radiometric dataset, Rem. Sens. Environ., 115, 2104–2115, 2011.





<img src="https://documentation.marine.copernicus.eu/IMG/OMI_HEALTH_CHL_BLKSEA_area_averaged_mean.png" width="300" height="200"/>



# Black Sea Chlorophyll-a time series and trend from Observations Reprocessing


**Product ID:** OMI_HEALTH_CHL_BLKSEA_OCEANCOLOUR_area_averaged_mean

**Description:** **DEFINITION**

The time series are derived from the regional chlorophyll reprocessed (MY) product as distributed by CMEMS. This dataset, derived from multi-sensor (SeaStar-SeaWiFS, AQUA-MODIS, NOAA20-VIIRS, NPP-VIIRS, Envisat-MERIS and Sentinel3-OLCI) Rrs spectra produced by CNR using an in-house processing chain, is obtained by means of two different regional algorithms developed with the BiOMaP data set (Zibordi et al., 2011): a band-ratio algorithm (B/R) (Zibordi et al., 2015) and a Multilayer Perceptron (MLP) neural net algorithm based on Rrs values at three individual wavelengths (490, 510 and 555 nm) (Kajiyama et al., 2018). The processing chain and the techniques used for algorithms merging are detailed in Colella et al. (2023). Monthly regional mean values are calculated by performing the average of 2D monthly mean (weighted by pixel area) over the region of interest. The deseasonalized time series is obtained by applying the X-11 seasonal adjustment methodology on the original time series as described in Colella et al. (2016), and then the Mann-Kendall test (Mann, 1945; Kendall, 1975) and Sens’s method (Sen, 1968) are subsequently applied to obtain the magnitude of trend.

**CONTEXT**

Phytoplankton   and chlorophyll concentration as a proxy for phytoplankton   respond rapidly to changes in environmental conditions, such as light, temperature, nutrients and mixing (Gregg and Rousseaux, 2014, Colella et al. 2016). The character of the response depends on the nature of the change drivers, and ranges from seasonal cycles to decadal oscillations (Basterretxea et al. 2018). Therefore, it is of critical importance to monitor chlorophyll concentration at multiple temporal and spatial scales, in order to be able to separate potential long-term climate signals from natural variability in the short term. In particular, phytoplankton in the Black Sea is known to respond to climate variability associated with the North Atlantic Oscillation (NAO) (Oguz et al .2003).

**KEY FINDINGS**

In the Black Sea, the trend average for the 1997-2023 period is negative (-1.13±1.07% per year). Nevertheless, this negative trend is lower than the one estimated in the previous release (both 1997-2021 and 1997-2022). The negative trend is mainly due to the marked change on chlorophyll concentrations between 2002 and 2004. From 2004 onwards, minima and maxima are strongly variable year by year. However, on average, the minima/maxima variability can be considered quite constant with a continuous decrease of maxima from 2015 up to mid 2020 where signal seems to change again with relative high chlorophyll values in 2021, 2022 and especially in the last year (2023). The general negative trend in the Black Sea is also confirmed by the analysis of Sathyendranath et al. (2018), that reveals an increasing trend in chlorophyll concentration in all the European Seas, except for the Black Sea.
**DOI (product):** 
https://doi.org/10.48670/moi-00211

**References:**

* Basterretxea, G., Font-Muñoz, J. S., Salgado-Hernanz, P. M., Arrieta, J., & Hernández-Carrasco, I. (2018). Patterns of chlorophyll interannual variability in Mediterranean biogeographical regions. Remote Sensing of Environment, 215, 7-17.
* Colella, S., Falcini, F., Rinaldi, E., Sammartino, M., & Santoleri, R. (2016). Mediterranean ocean colour chlorophyll trends. PloS one, 11(6).
* Colella, S., Brando, V.E., Cicco, A.D., D’Alimonte, D., Forneris, V., Bracaglia, M., 2021. Quality Information Document. Copernicus Marine Service. OCEAN COLOUR PRODUCTION CENTRE, Ocean Colour Mediterranean and Black Sea Observation Product. (https://documentation.marine.copernicus.eu/QUID/CMEMS-OC-QUID-009-141to144-151to154.pdf)
* Gregg, W. W., and C. S. Rousseaux, 2014. Decadal Trends in Global Pelagic Ocean Chlorophyll: A New Assessment Integrating Multiple Satellites, in Situ Data, and Models. Journal of Geophysical Research Oceans 119. doi:10.1002/2014JC010158.
* Kajiyama T., D. D’Alimonte, and G. Zibordi, “Algorithms merging for the determination of Chlorophyll-a concentration in the Black Sea,” IEEE Geoscience and Remote Sensing Letters, 2018. [Online]. Available: https://-www.doi.org/¬10.1109/¬LGRS.2018.2883539
* Kendall MG. 1975. Multivariate analysis. London: Charles Griffin & Co; p. 210, 43.
* Mann HB. 1945. Nonparametric tests against trend. Econometrica. 13:245 259. p. 42.
* Oguz, T., Cokacar, T., Malanotte‐Rizzoli, P., & Ducklow, H. W. (2003). Climatic warming and accompanying changes in the ecological regime of the Black Sea during 1990s. Global Biogeochemical Cycles, 17(3).
* Sathyendranath, S., Pardo, S., Benincasa, M., Brando, V. E., Brewin, R. J.W., Mélin, F., Santoleri, R., 2018, 1.5. Essential Variables: Ocean Colour in Copernicus Marine Service Ocean State Report - Issue 2, Journal of Operational Oceanography, 11:sup1, 1-142, doi: 10.1080/1755876X.2018.1489208
* Sen PK. 1968. Estimates of the regression coefficient based on Kendall’s tau. J Am Statist Assoc. 63:1379 1389.
* Zibordi, G., Berthon, J.-F., Mélin, F., and D’Alimonte, D.: Cross- site consistent in situ measurements for satellite ocean color ap- plications: the BiOMaP radiometric dataset, Rem. Sens. Environ., 115, 2104–2115, 2011.
* Zibordi, G., F. Mélin, J.-F. Berthon, and M. Talone (2015). In situ autonomous optical radiometry measurements for satellite ocean color validation in the Western Black Sea. Ocean Sci., 11, 275–286, 2015.





<img src="https://documentation.marine.copernicus.eu/IMG/OMI_HEALTH_CHL_BLKSEA_trend.png" width="300" height="200"/>



# Black Sea Chlorophyll-a trend map from Observations Reprocessing


**Product ID:** OMI_HEALTH_CHL_BLKSEA_OCEANCOLOUR_trend

**Description:** **DEFINITION**

This product includes the Black Sea satellite chlorophyll trend map based on regional chlorophyll reprocessed (MY) product as distributed by CMEMS OC-TAC. This dataset, derived from multi-sensor (SeaStar-SeaWiFS, AQUA-MODIS, NOAA20-VIIRS, NPP-VIIRS, Envisat-MERIS and Sentinel3-OLCI) Rrs spectra produced by CNR using an in-house processing chain, is obtained by means of two different regional algorithms developed with the BiOMaP data set (Zibordi et al., 2011): a band-ratio algorithm (B/R) (Zibordi et al., 2015) and a Multilayer Perceptron (MLP) neural net algorithm based on Rrs values at three individual wavelengths (490, 510 and 555 nm) (Kajiyama et al., 2018). The processing chain and the techniques used for algorithms merging are detailed in Colella et al. (2023).  
The trend map is obtained by applying Colella et al. (2016) methodology, where the Mann-Kendall test (Mann, 1945; Kendall, 1975) and Sens’s method (Sen, 1968) are applied on deseasonalized monthly time series, as obtained from the X-11 technique (see e. g. Pezzulli et al. 2005), to estimate, trend magnitude and its significance. The trend is expressed in % per year that represents the relative changes (i.e., percentage) corresponding to the dimensional trend [mg m-3 y-1] with respect to the reference climatology (1997-2014). Only significant trends (p < 0.05) are included.

**CONTEXT**

Phytoplankton are key actors in the carbon cycle and, as such, recognised as an Essential Climate Variable (ECV). Chlorophyll concentration - as a proxy for phytoplankton - respond rapidly to changes in environmental conditions, such as light, temperature, nutrients and mixing (Colella et al. 2016). The character of the response depends on the nature of the change drivers, and ranges from seasonal cycles to decadal oscillations (Basterretxea et al. 2018). Therefore, it is of critical importance to monitor chlorophyll concentration at multiple temporal and spatial scales, in order to be able to separate potential long-term climate signals from natural variability in the short term. In particular, phytoplankton in the Black Sea is known to respond to climate variability associated with the North Atlantic Oscillation (NAO) (Oguz et al .2003). Furthermore, chlorophyll analysis also demands the use of robust statistical temporal decomposition techniques, in order to separate the long-term signal from the seasonal component of the time series.

**KEY FINDINGS**

The average Black Sea trend for the 1997-2023 period is absolutely similar to the previous ones (1997-2021 or 1997-2022) showing, on average, a trend of -1.24% per year. The trend is negative overall the basin, with weaker values in the central area, up to no significant trend percentages. The western side of the basin highlights markable negative trend. Negative values are shown in the Azov Sea with a strong inversion offshore the Don River. The overall negative trend in the map is in accordance with the results of Bengil and Mavruk (2018), that revealed a decreasing trend of chlorophyll during the post-eutrophication phase in the years 1997-2017.

**DOI (product):** 
https://doi.org/10.48670/moi-00212

**References:**

* Basterretxea, G., Font-Muñoz, J. S., Salgado-Hernanz, P. M., Arrieta, J., & Hernández-Carrasco, I. (2018). Patterns of chlorophyll interannual variability in Mediterranean biogeographical regions. Remote Sensing of Environment, 215, 7-17.
* Bengil, F., & Mavruk, S. (2018). Bio-optical trends of seas around Turkey: An assessment of the spatial and temporal variability. Oceanologia, 60(4), 488-499.
* Colella, S., Falcini, F., Rinaldi, E., Sammartino, M., & Santoleri, R. (2016). Mediterranean ocean colour chlorophyll trends. PloS one, 11(6).
* Colella, S., Brando, V.E., Cicco, A.D., D’Alimonte, D., Forneris, V., Bracaglia, M., 2021. OCEAN COLOUR PRODUCTION CENTRE, Ocean Colour Mediterranean and Black Sea Observation Product. Copernicus Marine Environment Monitoring Centre. Quality Information Document (https://documentation.marine.copernicus.eu/QUID/CMEMS-OC-QUID-009-141to144-151to154.pdf)
* Kajiyama T., D. D’Alimonte, and G. Zibordi, “Algorithms merging for the determination of Chlorophyll-a concentration in the Black Sea,” IEEE Geoscience and Remote Sensing Letters, 2018. [Online]. Available: https://-www.doi.org/¬10.1109/¬LGRS.2018.2883539
* Kendall MG. 1975. Multivariate analysis. London: Charles Griffin & Co; p. 210, 43.
* Mann HB. 1945. Nonparametric tests against trend. Econometrica. 13:245–259. p. 42.
* Oguz, T., Cokacar, T., Malanotte‐Rizzoli, P., & Ducklow, H. W. (2003). Climatic warming and accompanying changes in the ecological regime of the Black Sea during 1990s. Global Biogeochemical Cycles, 17(3).
* Pezzulli S, Stephenson DB, Hannachi A. 2005. The Variability of Seasonality. J. Climate. 18:71–88. doi:10.1175/JCLI-3256.1.
* Sathyendranath, S., Pardo, S., Benincasa, M., Brando, V. E., Brewin, R. J.W., Mélin, F., Santoleri, R., 2018, 1.5. Essential Variables: Ocean Colour in Copernicus Marine Service Ocean State Report - Issue 2, Journal of Operational Oceanography, 11:sup1, 1-142, doi: 10.1080/1755876X.2018.1489208.
* Sen PK. 1968. Estimates of the regression coefficient based on Kendall’s tau. J Am Statist Assoc. 63:1379–1389.
* Zibordi, G., Berthon, J.-F., Mélin, F., and D’Alimonte, D.: Cross- site consistent in situ measurements for satellite ocean color ap- plications: the BiOMaP radiometric dataset, Rem. Sens. Environ., 115, 2104–2115, 2011.
* Zibordi, G., F. Mélin, J.-F. Berthon, and M. Talone (2015). In situ autonomous optical radiometry measurements for satellite ocean color validation in the Western Black Sea. Ocean Sci., 11, 275–286, 2015.





<img src="https://documentation.marine.copernicus.eu/IMG/OMI_HEALTH_CHL_GLOBAL_OCEANCOLOUR_oligo_nag_area_mean.png" width="300" height="200"/>



# North Atlantic Gyre Area Chlorophyll-a time series and trend from Observations Reprocessing


**Product ID:** OMI_HEALTH_CHL_GLOBAL_OCEANCOLOUR_oligo_nag_area_mean

**Description:** **DEFINITION**

Oligotrophic subtropical gyres are regions of the ocean with low levels of nutrients required for phytoplankton growth and low levels of surface chlorophyll-a whose concentration can be quantified through satellite observations. The gyre boundary has been defined using a threshold value of 0.15 mg m-3 chlorophyll for the Atlantic gyres (Aiken et al. 2016), and 0.07 mg m-3 for the Pacific gyres (Polovina et al. 2008). The area inside the gyres for each month is computed using monthly chlorophyll data from which the monthly climatology is subtracted to compute anomalies.  A gap filling algorithm has been utilized to account for missing data. Trends in the area anomaly are then calculated for the entire study period (September 1997 to December 2021).

**CONTEXT**

Oligotrophic gyres of the oceans have been referred to as ocean deserts (Polovina et al. 2008). They are vast, covering approximately 50% of the Earth’s surface (Aiken et al. 2016). Despite low productivity, these regions contribute significantly to global productivity due to their immense size (McClain et al. 2004). Even modest changes in their size can have large impacts on a variety of global biogeochemical cycles and on trends in chlorophyll (Signorini et al. 2015). Based on satellite data, Polovina et al. (2008) showed that the areas of subtropical gyres were expanding. The Ocean State Report (Sathyendranath et al. 2018) showed that the trends had reversed in the Pacific for the time segment from January 2007 to December 2016. 

**CMEMS KEY FINDINGS**

The trend in the North Atlantic gyre area for the 1997 Sept – 2021 December period was positive, with a 0.14% year-1 increase in area relative to 2000-01-01 values. This trend has decreased compared with the 1997-2019 trend of 0.39%, and is no longer statistically significant (p>0.05). 
During the 1997 Sept – 2021 December period, the trend in chlorophyll concentration was negative (-0.21% year-1) inside the North Atlantic gyre relative to 2000-01-01 values. This is a slightly lower rate of change compared with the -0.24%  trend for the 1997-2020 period but is still statistically significant (p<0.05).

**DOI (product):** 
https://doi.org/10.48670/moi-00226

**References:**

* Aiken J, Brewin RJW, Dufois F, Polimene L, Hardman-Mountford NJ, Jackson T, Loveday B, Hoya SM, Dall’Olmo G, Stephens J, et al. 2016. A synthesis of the environmental response of the North and South Atlantic sub-tropical gyres during two decades of AMT. Prog Oceanogr. doi:10.1016/j.pocean.2016.08.004.
* McClain CR, Signorini SR, Christian JR 2004. Subtropical gyre variability observed by ocean-color satellites. Deep Sea Res Part II Top Stud Oceanogr. 51:281–301. doi:10.1016/j.dsr2.2003.08.002.
* Polovina JJ, Howell EA, Abecassis M 2008. Ocean’s least productive waters are expanding. Geophys Res Lett. 35:270. doi:10.1029/2007GL031745.
* Sathyendranath S, Pardo S, Brewin RJW. 2018. Oligotrophic gyres. In: Copernicus Marine Service Ocean State Report, Issue 2, Journal of Operational Oceanography, 11:sup1, s13–s16, DOI: 10.1080/1755876X.2018.1489208
* Signorini SR, Franz BA, McClain CR 2015. Chlorophyll variability in the oligotrophic gyres: mechanisms, seasonality and trends. Front Mar Sci. 2. doi:10.3389/fmars.2015.00001.





<img src="https://documentation.marine.copernicus.eu/IMG/OMI_HEALTH_CHL_GLOBAL_OCEANCOLOUR_oligo_npg_area_mean.png" width="300" height="200"/>



# North Pacific Gyre Area Chlorophyll-a time series and trend from Observations Reprocessing


**Product ID:** OMI_HEALTH_CHL_GLOBAL_OCEANCOLOUR_oligo_npg_area_mean

**Description:** **DEFINITION**

Oligotrophic subtropical gyres are regions of the ocean with low levels of nutrients required for phytoplankton growth and low levels of surface chlorophyll-a whose concentration can be quantified through satellite observations. The gyre boundary has been defined using a threshold value of 0.15 mg m-3 chlorophyll for the Atlantic gyres (Aiken et al. 2016), and 0.07 mg m-3 for the Pacific gyres (Polovina et al. 2008). The area inside the gyres for each month is computed using monthly chlorophyll data from which the monthly climatology is subtracted to compute anomalies. A gap filling algorithm has been utilized to account for missing data inside the gyre.  Trends in the area anomaly are then calculated for the entire study period (September 1997 to December 2021).

**CONTEXT**

Oligotrophic gyres of the oceans have been referred to as ocean deserts (Polovina et al. 2008). They are vast, covering approximately 50% of the Earth’s surface (Aiken et al. 2016). Despite low productivity, these regions contribute significantly to global productivity due to their immense size (McClain et al. 2004). Even modest changes in their size can have large impacts on a variety of global biogeochemical cycles and on trends in chlorophyll (Signorini et al 2015). Based on satellite data, Polovina et al. (2008) showed that the areas of subtropical gyres were expanding. The Ocean State Report (Sathyendranath et al. 2018) showed that the trends had reversed in the Pacific for the time segment from January 2007 to December 2016. 

**CMEMS KEY FINDINGS**

The trend in the North Pacific gyre area for the 1997 Sept – 2021 December period was positive, with a 1.75% increase in area relative to 2000-01-01 values. Note that this trend is lower than the 2.17% reported for the 1997-2020 period. The trend is statistically significant (p<0.05). 
During the 1997 Sept – 2021 December period, the trend in chlorophyll concentration was negative (-0.26% year-1) in the North Pacific gyre relative to 2000-01-01 values. This trend is slightly less negative than the trend of -0.31% year-1 for the 1997-2020 period, though the sign of the trend remains unchanged and is statistically significant (p<0.05).  It must be noted that the difference is small and within the uncertainty of the calculations, indicating that the trend is significant, however there may be no change associated with the timeseries extension.
For 2016, The Ocean State Report (Sathyendranath et al. 2018) reported a large increase in gyre area in the Pacific Ocean (both North and South Pacific gyres), probably linked with the 2016 ENSO event which saw large decreases in chlorophyll in the Pacific Ocean. 

**DOI (product):** 
https://doi.org/10.48670/moi-00227

**References:**

* Aiken J, Brewin RJW, Dufois F, Polimene L, Hardman-Mountford NJ, Jackson T, Loveday B, Hoya SM, Dall’Olmo G, Stephens J, et al. 2016. A synthesis of the environmental response of the North and South Atlantic sub-tropical gyres during two decades of AMT. Prog Oceanogr. doi:10.1016/j.pocean.2016.08.004.
* McClain CR, Signorini SR, Christian JR 2004. Subtropical gyre variability observed by ocean-color satellites. Deep Sea Res Part II Top Stud Oceanogr. 51:281–301. doi:10.1016/j.dsr2.2003.08.002.
* Polovina JJ, Howell EA, Abecassis M 2008. Ocean’s least productive waters are expanding. Geophys Res Lett. 35:270. doi:10.1029/2007GL031745.
* Sathyendranath S, Pardo S, Brewin RJW. 2018. Oligotrophic gyres. In: Copernicus Marine Service Ocean State Report, Issue 2, Journal of Operational Oceanography, 11:sup1, s13–s16, DOI: 10.1080/1755876X.2018.1489208
* Signorini SR, Franz BA, McClain CR 2015. Chlorophyll variability in the oligotrophic gyres: mechanisms, seasonality and trends. Front Mar Sci. 2. doi:10.3389/fmars.2015.00001.





<img src="https://documentation.marine.copernicus.eu/IMG/OMI_HEALTH_CHL_GLOBAL_OCEANCOLOUR_oligo_sag_area_mean.png" width="300" height="200"/>



# South Atlantic Gyre Area Chlorophyll-a time series and trend from Observations Reprocessing


**Product ID:** OMI_HEALTH_CHL_GLOBAL_OCEANCOLOUR_oligo_sag_area_mean

**Description:** **DEFINITION**

Oligotrophic subtropical gyres are regions of the ocean with low levels of nutrients required for phytoplankton growth and low levels of surface chlorophyll-a whose concentration can be quantified through satellite observations. The gyre boundary has been defined using a threshold value of 0.15 mg m-3 chlorophyll for the Atlantic gyres (Aiken et al. 2016), and 0.07 mg m-3 for the Pacific gyres (Polovina et al. 2008). The area inside the gyres for each month is computed using monthly chlorophyll data from which the monthly climatology is subtracted to compute anomalies.  A gap filling algorithm has been utilized to account for missing data inside the gyre. Trends in the area anomaly are then calculated for the entire study period (September 1997 to December 2021).

**CONTEXT**

Oligotrophic gyres of the oceans have been referred to as ocean deserts (Polovina et al. 2008). They are vast, covering approximately 50% of the Earth’s surface (Aiken et al. 2016). Despite low productivity, these regions contribute significantly to global productivity due to their immense size (McClain et al. 2004). Even modest changes in their size can have large impacts on a variety of global biogeochemical cycles and on trends in chlorophyll (Signorini et al 2015). Based on satellite data, Polovina et al. (2008) showed that the areas of subtropical gyres were expanding. The Ocean State Report (Sathyendranath et al. 2018) showed that the trends had reversed in the Pacific for the time segment from January 2007 to December 2016.  

**CMEMS KEY FINDINGS**

The trend in the South Altantic gyre area for the 1997 Sept – 2021 December period was positive, with a 0.01% increase in area relative to 2000-01-01 values. Note that this trend is lower than the 0.09% rate for the 1997-2020 trend (though within the uncertainties associated with the two estimates) and is not statistically significant (p>0.05). 
During the 1997 Sept – 2021 December period, the trend in chlorophyll concentration was positive (0.73% year-1) relative to 2000-01-01 values. This is a significant increase from the trend of 0.35% year-1 for the 1997-2020 period and is statistically significant (p<0.05).  The last two years of the timeseries show an increased deviation from the mean.

**DOI (product):** 
https://doi.org/10.48670/moi-00228

**References:**

* Aiken J, Brewin RJW, Dufois F, Polimene L, Hardman-Mountford NJ, Jackson T, Loveday B, Hoya SM, Dall’Olmo G, Stephens J, et al. 2016. A synthesis of the environmental response of the North and South Atlantic sub-tropical gyres during two decades of AMT. Prog Oceanogr. doi:10.1016/j.pocean.2016.08.004.
* McClain CR, Signorini SR, Christian JR 2004. Subtropical gyre variability observed by ocean-color satellites. Deep Sea Res Part II Top Stud Oceanogr. 51:281–301. doi:10.1016/j.dsr2.2003.08.002.
* Polovina JJ, Howell EA, Abecassis M 2008. Ocean’s least productive waters are expanding. Geophys Res Lett. 35:270. doi:10.1029/2007GL031745.
* Sathyendranath S, Pardo S, Brewin RJW. 2018. Oligotrophic gyres. In: Copernicus Marine Service Ocean State Report, Issue 2, Journal of Operational Oceanography, 11:sup1, s13–s16, DOI: 10.1080/1755876X.2018.1489208
* Signorini SR, Franz BA, McClain CR 2015. Chlorophyll variability in the oligotrophic gyres: mechanisms, seasonality and trends. Front Mar Sci. 2. doi:10.3389/fmars.2015.00001.





<img src="https://documentation.marine.copernicus.eu/IMG/OMI_HEALTH_CHL_GLOBAL_OCEANCOLOUR_oligo_spg_area_mean.png" width="300" height="200"/>



# South Pacific Gyre Area Chlorophyll-a time series and trend from Observations Reprocessing


**Product ID:** OMI_HEALTH_CHL_GLOBAL_OCEANCOLOUR_oligo_spg_area_mean

**Description:** **DEFINITION**

Oligotrophic subtropical gyres are regions of the ocean with low levels of nutrients required for phytoplankton growth and low levels of surface chlorophyll-a whose concentration can be quantified through satellite observations. The gyre boundary has been defined using a threshold value of 0.15 mg m-3 chlorophyll for the Atlantic gyres (Aiken et al. 2016), and 0.07 mg m-3 for the Pacific gyres (Polovina et al. 2008). The area inside the gyres for each month is computed using monthly chlorophyll data from which the monthly climatology is subtracted to compute anomalies. A gap filling algorithm has been utilized to account for missing data.  Trends in the area anomaly are then calculated for the entire study period (September 1997 to December 2021).

**CONTEXT**

Oligotrophic gyres of the oceans have been referred to as ocean deserts (Polovina et al. 2008). They are vast, covering approximately 50% of the Earth’s surface (Aiken et al. 2016). Despite low productivity, these regions contribute significantly to global productivity due to their immense size (McClain et al. 2004). Even modest changes in their size can have large impacts on a variety of global biogeochemical cycles and on trends in chlorophyll (Signorini et al 2015). Based on satellite data, Polovina et al. (2008) showed that the areas of subtropical gyres were expanding. The Ocean State Report (Sathyendranath et al. 2018) showed that the trends had reversed in the Pacific for the time segment from January 2007 to December 2016. 

**CMEMS KEY FINDINGS**

The trend in the South Pacific gyre area for the 1997 Sept – 2021 December period was positive, with a 0.04% increase in area relative to 2000-01-01 values. Note that this trend is lower than the 0.16% change for the 1997-2020 period, with the sign of the trend remaining unchanged and is not statistically significant (p<0.05).  An underlying low frequency signal is observed with a period of approximately a decade.
During the 1997 Sept – 2021 December period, the trend in chlorophyll concentration was positive (0.66% year-1) in the South Pacific gyre relative to 2000-01-01 values. This rate has increased compared to the rate of  0.45% year-1 for the 1997-2020 period and remains statistically significant (p<0.05).  In the last two years of the timeseries, an increase in the variation from the mean is observed.
For 2016, the Ocean State Report (Sathyendranath et al. 2018) reported a large increase in gyre area in the Pacific Ocean (both North and South Pacific gyres), probably linked with the 2016 ENSO event which saw large decreases in chlorophyll in the Pacific Ocean. 

**DOI (product):** 
https://doi.org/10.48670/moi-00229

**References:**

* Aiken J, Brewin RJW, Dufois F, Polimene L, Hardman-Mountford NJ, Jackson T, Loveday B, Hoya SM, Dall’Olmo G, Stephens J, et al. 2016. A synthesis of the environmental response of the North and South Atlantic sub-tropical gyres during two decades of AMT. Prog Oceanogr. doi:10.1016/j.pocean.2016.08.004.
* McClain CR, Signorini SR, Christian JR 2004. Subtropical gyre variability observed by ocean-color satellites. Deep Sea Res Part II Top Stud Oceanogr. 51:281–301. doi:10.1016/j.dsr2.2003.08.002.
* Polovina JJ, Howell EA, Abecassis M 2008. Ocean’s least productive waters are expanding. Geophys Res Lett. 35:270. doi:10.1029/2007GL031745.
* Sathyendranath S, Pardo S, Brewin RJW. 2018. Oligotrophic gyres. In: Copernicus Marine Service Ocean State Report, Issue 2, Journal of Operational Oceanography, 11:sup1, s13–s16, DOI: 10.1080/1755876X.2018.1489208
* Signorini SR, Franz BA, McClain CR 2015. Chlorophyll variability in the oligotrophic gyres: mechanisms, seasonality and trends. Front Mar Sci. 2. doi:10.3389/fmars.2015.00001.





<img src="https://documentation.marine.copernicus.eu/IMG/OMI_HEALTH_CHL_GLOBAL_OCEANCOLOUR_trend.png" width="300" height="200"/>



# Global Ocean Chlorophyll-a trend map from Observations Reprocessing


**Product ID:** OMI_HEALTH_CHL_GLOBAL_OCEANCOLOUR_trend

**Description:** **DEFINITION**

The trend map is derived from version 5 of the global climate-quality chlorophyll time series produced by the ESA Ocean Colour Climate Change Initiative (ESA OC-CCI, Sathyendranath et al. 2019; Jackson 2020) and distributed by CMEMS. The trend detection method is based on the Census-I algorithm as described by Vantrepotte et al. (2009), where the time series is decomposed as a fixed seasonal cycle plus a linear trend component plus a residual component. The linear trend is expressed in % year -1, and its level of significance (p) calculated using a t-test. Only significant trends (p < 0.05) are included. 

**CONTEXT**

Phytoplankton are key actors in the carbon cycle and, as such, recognised as an Essential Climate Variable (ECV). Chlorophyll concentration is the most widely used measure of the concentration of phytoplankton present in the ocean. Drivers for chlorophyll variability range from small-scale seasonal cycles to long-term climate oscillations and, most importantly, anthropogenic climate change. Due to such diverse factors, the detection of climate signals requires a long-term time series of consistent, well-calibrated, climate-quality data record. Furthermore, chlorophyll analysis also demands the use of robust statistical temporal decomposition techniques, in order to separate the long-term signal from the seasonal component of the time series.

**CMEMS KEY FINDINGS**

The average global trend for the 1997-2021 period was 0.51% per year, with a maximum value of 25% per year and a minimum value of -6.1% per year. Positive trends are pronounced in the high latitudes of both northern and southern hemispheres. The significant increases in chlorophyll reported in 2016-2017 (Sathyendranath et al., 2018b) for the Atlantic and Pacific oceans at high latitudes appear to be plateauing after the 2021 extension.  The negative trends shown in equatorial waters in 2020 appear to be remain consistent in 2021.  

**DOI (product):** 
https://doi.org/10.48670/moi-00230

**References:**

* Jackson, T. (2020) OC-CCI Product User Guide (PUG). ESA/ESRIN Report. D4.2PUG, 2020-10-12. Issue:v4.2. https://docs.pml.space/share/s/okB2fOuPT7Cj2r4C5sppDg
* Sathyendranath, S., Pardo, S., Benincasa, M., Brando, V. E., Brewin, R. J.W., Mélin, F., Santoleri, R., 2018b, 1.5. Essential Variables: Ocean Colour in Copernicus Marine Service Ocean State Report - Issue 2, Journal of Operational Oceanography, 11:sup1, 1-142, doi: 10.1080/1755876X.2018.1489208
* Sathyendranath, S, Brewin, RJW, Brockmann, C, Brotas, V, Calton, B, Chuprin, A, Cipollini, P, Couto, AB, Dingle, J, Doerffer, R, Donlon, C, Dowell, M, Farman, A, Grant, M, Groom, S, Horseman, A, Jackson, T, Krasemann, H, Lavender, S, Martinez-Vicente, V, Mazeran, C, Mélin, F, Moore, TS, Müller, D, Regner, P, Roy, S, Steele, CJ, Steinmetz, F, Swinton, J, Taberner, M, Thompson, A, Valente, A, Zühlke, M, Brando, VE, Feng, H, Feldman, G, Franz, BA, Frouin, R, Gould, Jr., RW, Hooker, SB, Kahru, M, Kratzer, S, Mitchell, BG, Muller-Karger, F, Sosik, HM, Voss, KJ, Werdell, J, and Platt, T (2019) An ocean-colour time series for use in climate studies: the experience of the Ocean-Colour Climate Change Initiative (OC-CCI). Sensors: 19, 4285. doi:10.3390/s19194285
* Vantrepotte, V., Mélin, F., 2009. Temporal variability of 10-year global SeaWiFS time series of phytoplankton chlorophyll-a concentration. ICES J. Mar. Sci., 66, 1547-1556. doi: 10.1093/icesjms/fsp107.





<img src="https://documentation.marine.copernicus.eu/IMG/OMI_HEALTH_CHL_MEDSEA_area_averaged_mean.png" width="300" height="200"/>



# Mediterranean Sea Chlorophyll-a time series and trend from Observations Reprocessing


**Product ID:** OMI_HEALTH_CHL_MEDSEA_OCEANCOLOUR_area_averaged_mean

**Description:** **DEFINITION**

The time series are derived from the regional chlorophyll reprocessed (MY) product as distributed by CMEMS. This dataset, derived from multi-sensor (SeaStar-SeaWiFS, AQUA-MODIS, NOAA20-VIIRS, NPP-VIIRS, Envisat-MERIS and Sentinel3-OLCI) Rrs spectra produced by CNR using an in-house processing chain, is obtained by means of the Mediterranean Ocean Colour regional algorithms: an updated version of the MedOC4 (Case 1 (off-shore) waters, Volpe et al., 2019, with new coefficients) and AD4 (Case 2 (coastal) waters, Berthon and Zibordi, 2004). The processing chain and the techniques used for algorithms merging are detailed in Colella et al. (2023). Monthly regional mean values are calculated by performing the average of 2D monthly mean (weighted by pixel area) over the region of interest. The deseasonalized time series is obtained by applying the X-11 seasonal adjustment methodology on the original time series as described in Colella et al. (2016), and then the Mann-Kendall test (Mann, 1945; Kendall, 1975) and Sens’s method (Sen, 1968) are subsequently applied to obtain the magnitude of trend.

**CONTEXT**

Phytoplankton   and chlorophyll concentration as a proxy for phytoplankton   respond rapidly to changes in environmental conditions, such as light, temperature, nutrients and mixing (Colella et al. 2016). The character of the response depends on the nature of the change drivers, and ranges from seasonal cycles to decadal oscillations (Basterretxea et al. 2018). Therefore, it is of critical importance to monitor chlorophyll concentration at multiple temporal and spatial scales, in order to be able to separate potential long-term climate signals from natural variability in the short term. In particular, phytoplankton in the Mediterranean Sea is known to respond to climate variability associated with the North Atlantic Oscillation (NAO) and El Niño Southern Oscillation (ENSO) (Basterretxea et al. 2018, Colella et al. 2016).

**KEY FINDINGS**

In the Mediterranean Sea, the trend average for the 1997-2023 period is slightly negative (-0.73±0.65% per year) emphasising the results obtained from previous release (1997-2022). This result is in contrast with the analysis of Sathyendranath et al. (2018) that reveals an increasing trend in chlorophyll concentration in all the European Seas. Starting from 2010-2011, except for 2018-2019, the decrease of chlorophyll concentrations is quite evident in the deseasonalized timeseries (in green), and in the maxima of the observations (grey line), starting from 2015. This attenuation of chlorophyll values of the last decade, results in an overall negative trend for the Mediterranean Sea.
**DOI (product):** 
https://doi.org/10.48670/moi-00259

**References:**

* Basterretxea, G., Font-Muñoz, J. S., Salgado-Hernanz, P. M., Arrieta, J., & Hernández-Carrasco, I. (2018). Patterns of chlorophyll interannual variability in Mediterranean biogeographical regions. Remote Sensing of Environment, 215, 7-17.
* Berthon, J.-F., Zibordi, G. (2004). Bio-optical relationships for the northern Adriatic Sea. Int. J. Remote Sens., 25, 1527-1532.
* Colella, S., Falcini, F., Rinaldi, E., Sammartino, M., Santoleri, R., 2016. Mediterranean ocean colour chlorophyll trends. PLoS One 11, 1 16. https://doi.org/10.1371/journal.pone.0155756.
* Colella, S., Brando, V.E., Cicco, A.D., D’Alimonte, D., Forneris, V., Bracaglia, M., 2021. Quality Information Document. Copernicus Marine Service. OCEAN COLOUR PRODUCTION CENTRE, Ocean Colour Mediterranean and Black Sea Observation Product. (https://documentation.marine.copernicus.eu/QUID/CMEMS-OC-QUID-009-141to144-151to154.pdf).
* Kendall MG. 1975. Multivariate analysis. London: Charles Griffin & Co; p. 210, 43.
* Mann HB. 1945. Nonparametric tests against trend. Econometrica. 13:245 259. p. 42.
* Sathyendranath, S., Pardo, S., Benincasa, M., Brando, V. E., Brewin, R. J.W., Mélin, F., Santoleri, R., 2018, 1.5. Essential Variables: Ocean Colour in Copernicus Marine Service Ocean State Report - Issue 2, Journal of Operational Oceanography, 11:sup1, 1-142, doi: 10.1080/1755876X.2018.1489208
* Sen PK. 1968. Estimates of the regression coefficient based on Kendall’s tau. J Am Statist Assoc. 63:1379 1389.
* Volpe, G., Colella, S., Brando, V. E., Forneris, V., Padula, F. L., Cicco, A. D., ... & Santoleri, R. (2019). Mediterranean ocean colour Level 3 operational multi-sensor processing. Ocean Science, 15(1), 127-146.





<img src="https://documentation.marine.copernicus.eu/IMG/OMI_HEALTH_CHL_MEDSEA_trend.png" width="300" height="200"/>



# Mediterranean Sea Chlorophyll-a trend map from Observations Reprocessing


**Product ID:** OMI_HEALTH_CHL_MEDSEA_OCEANCOLOUR_trend

**Description:** **DEFINITION**

This product includes the Mediterranean Sea satellite chlorophyll trend map based on regional chlorophyll reprocessed (MY) product as distributed by CMEMS OC-TAC. This dataset, derived from multi-sensor (SeaStar-SeaWiFS, AQUA-MODIS, NOAA20-VIIRS, NPP-VIIRS, Envisat-MERIS and Sentinel3-OLCI) (at 1 km resolution) Rrs spectra produced by CNR using an in-house processing chain, is obtained by means of the Mediterranean Ocean Colour regional algorithms: an updated version of the MedOC4 (Case 1 (off-shore) waters, Volpe et al., 2019, with new coefficients) and AD4 (Case 2 (coastal) waters, Berthon and Zibordi, 2004). The processing chain and the techniques used for algorithms merging are detailed in Colella et al. (2023). 
The trend map is obtained by applying Colella et al. (2016) methodology, where the Mann-Kendall test (Mann, 1945; Kendall, 1975) and Sens’s method (Sen, 1968) are applied on deseasonalized monthly time series, as obtained from the X-11 technique (see e. g. Pezzulli et al. 2005), to estimate, trend magnitude and its significance. The trend is expressed in % per year that represents the relative changes (i.e., percentage) corresponding to the dimensional trend [mg m-3 y-1] with respect to the reference climatology (1997-2014). Only significant trends (p < 0.05) are included.

**CONTEXT**

Phytoplankton are key actors in the carbon cycle and, as such, recognised as an Essential Climate Variable (ECV). Chlorophyll concentration - as a proxy for phytoplankton - respond rapidly to changes in environmental conditions, such as light, temperature, nutrients and mixing (Colella et al. 2016). The character of the response depends on the nature of the change drivers, and ranges from seasonal cycles to decadal oscillations (Basterretxea et al. 2018). The Mediterranean Sea is an oligotrophic basin, where chlorophyll concentration decreases following a specific gradient from West to East (Colella et al. 2016). The highest concentrations are observed in coastal areas and at the river mouths, where the anthropogenic pressure and nutrient loads impact on the eutrophication regimes (Colella et al. 2016). The the use of long-term time series of consistent, well-calibrated, climate-quality data record is crucial for detecting eutrophication. Furthermore, chlorophyll analysis also demands the use of robust statistical temporal decomposition techniques, in order to separate the long-term signal from the seasonal component of the time series.

**KEY FINDINGS**

Chlorophyll trend in the Mediterranean Sea, for the period 1997-2023, generally confirm trend results of the previous release with negative values over most of the basin. In Ligurian Sea, negative trend is slightly emphasized. As for the previous release, the southern part of the western Mediterranean basin, Rhode Gyre and in the northern coast of the Aegean Sea show weak positive trend areas but they seems weaker than previous ones. On average the trend in the Mediterranean Sea is about -0.83% per year, emphasizing the mean negative trend achieved in the previous release. Contrary to what shown by Salgado-Hernanz et al. (2019) in their analysis (related to 1998-2014 satellite observations), western and eastern part of the Mediterranean Sea do not show differences. In the Ligurian Sea, the trend switch to negative values, differing from the positive regime observed in the trend maps of both Colella et al. (2016) and Salgado-Hernanz et al. (2019), referred, respectively, to 1998-2009 and 1998-2014 period, respectively. The waters offshore the Po River mouth show weak negative trend values, partially differing from the markable negative regime observed in the 1998-2009 period (Colella et al., 2016), and definitely moving from the positive trend observed by Salgado-Hernanz et al. (2019).
**DOI (product):** 
https://doi.org/10.48670/moi-00260

**References:**

* Basterretxea, G., Font-Muñoz, J. S., Salgado-Hernanz, P. M., Arrieta, J., & Hernández-Carrasco, I. (2018). Patterns of chlorophyll interannual variability in Mediterranean biogeographical regions. Remote Sensing of Environment, 215, 7-17.
* Berthon, J.-F., Zibordi, G.: Bio-optical relationships for the northern Adriatic Sea. Int. J. Remote Sens., 25, 1527-1532, 200.
* Colella, S., Falcini, F., Rinaldi, E., Sammartino, M., & Santoleri, R. (2016). Mediterranean ocean colour chlorophyll trends. PloS one, 11(6).
* Colella, S., Brando, V.E., Cicco, A.D., D’Alimonte, D., Forneris, V., Bracaglia, M., 2021. OCEAN COLOUR PRODUCTION CENTRE, Ocean Colour Mediterranean and Black Sea Observation Product. Copernicus Marine Environment Monitoring Centre. Quality Information Document (https://documentation.marine.copernicus.eu/QUID/CMEMS-OC-QUID-009-141to144-151to154.pdf).
* Kendall MG. 1975. Multivariate analysis. London: Charles Griffin & Co; p. 210, 43.
* Mann HB. 1945. Nonparametric tests against trend. Econometrica. 13:245–259. p. 42.
* Pezzulli S, Stephenson DB, Hannachi A. 2005. The Variability of Seasonality. J. Climate. 18:71–88. doi:10.1175/JCLI-3256.1.
* Salgado-Hernanz, P. M., Racault, M. F., Font-Muñoz, J. S., & Basterretxea, G. (2019). Trends in phytoplankton phenology in the Mediterranean Sea based on ocean-colour remote sensing. Remote Sensing of Environment, 221, 50-64.
* Sen PK. 1968. Estimates of the regression coefficient based on Kendall’s tau. J Am Statist Assoc. 63:1379–1389.
* Volpe, G., Colella, S., Brando, V. E., Forneris, V., Padula, F. L., Cicco, A. D., ... & Santoleri, R. (2019). Mediterranean ocean colour Level 3 operational multi-sensor processing. Ocean Science, 15(1), 127-146.





<img src="https://documentation.marine.copernicus.eu/IMG/OMI_VAR_EXTREME_WMF_MEDSEA_area_averaged_mean.png" width="300" height="200"/>



# Mediterranean Water Mass Formation Rates from Reanalysis


**Product ID:** OMI_VAR_EXTREME_WMF_MEDSEA_area_averaged_mean

**Description:** **DEFINITION**

The Mediterranean water mass formation rates are evaluated in 4 areas as defined in the Ocean State Report issue 2  section 3.4 (Simoncelli and Pinardi, 2018) as shown in Figure 2: (1) the Gulf of Lions for the Western Mediterranean Deep Waters (WMDW); (2) the Southern Adriatic Sea Pit for the Eastern Mediterranean Deep Waters (EMDW); (3) the Cretan Sea for Cretan Intermediate Waters (CIW) and Cretan Deep Waters (CDW); (4) the Rhodes Gyre, the area of formation of the so-called Levantine Intermediate Waters (LIW) and Levantine Deep Waters (LDW).
Annual water mass formation rates have been computed using daily mixed layer depth estimates (density criteria Δσ = 0.01 kg/m3, 10 m reference level) considering the annual maximum volume of water above mixed layer depth with potential density within or higher the specific thresholds specified in Table 1 then divided by seconds per year.
Annual mean values are provided using the Mediterranean 1/24o eddy resolving reanalysis (Escudier et al. 2020, 2021).

**CONTEXT**

The formation of intermediate and deep water masses is one of the most important processes occurring in the Mediterranean Sea, being a component of its general overturning circulation. This circulation varies at interannual and multidecadal time scales and it is composed of an upper zonal cell (Zonal Overturning Circulation) and two main meridional cells in the western and eastern Mediterranean (Pinardi and Masetti 2000).
The objective is to monitor the main water mass formation events using the eddy resolving Mediterranean Sea Reanalysis (Escudier et al. 2020, 2021) and considering Pinardi et al. (2015) and Simoncelli and Pinardi (2018) as references for the methodology. The Mediterranean Sea Reanalysis can reproduce both Eastern Mediterranean Transient and Western Mediterranean Transition phenomena and catches the principal water mass formation events reported in the literature. This will permit constant monitoring of the open ocean deep convection process in the Mediterranean Sea and a better understanding of the multiple drivers of the general overturning circulation at interannual and multidecadal time scales. 
Deep and intermediate water formation events reveal themselves by a deep mixed layer depth distribution in four Mediterranean areas (Table 1 and Figure 2): Gulf of Lions, Southern Adriatic Sea Pit, Cretan Sea and Rhodes Gyre. 

**CMEMS KEY FINDINGS**

The Western Mediterranean Deep Water (WMDW) formation events in the Gulf of Lion appear to be larger after 1999 consistently with Schroeder et al. (2006, 2008) related to the Eastern Mediterranean Transient event. This modification of WMDW after 2005 has been called Western Mediterranean Transition. WMDW formation events are consistent with Somot et al. (2016) and the event in 2009 is also reported in Houpert et al. (2016). 
The Eastern Mediterranean Deep Water (EMDW) formation in the Southern Adriatic Pit region displays a period of water mass formation between 1988 and 1993, in agreement with Pinardi et al. (2015), in 1996, 1999 and 2000 as documented by Manca et al. (2002). Weak deep water formation in winter 2006 is confirmed by observations in Vilibić and Šantić (2008). An intense deep water formation event is detected in 2012-2013 (Gačić et al., 2014). Last years are characterized by large events starting from 2017 (Mihanovic et al., 2021).
Cretan Intermediate Water formation rates present larger peaks between 1989 and 1993 with the ones in 1992 and 1993 composing the Eastern Mediterranean Transient phenomena. The Cretan Deep Water formed in 1992 and 1993 is characterized by the highest densities of the entire period in accordance with Velaoras et al. (2014).
The Levantine Deep Water formation rate in the Rhode Gyre region presents the largest values between 1992 and 1993 in agreement with Kontoyiannis et al. (1999). 

**Figure caption**

Water mass formation rates [Sverdrup] computed in 4 regions: in the Gulf of Lion for the Western Mediterranean Deep Waters (WMDW); in the Southern Adriatic region for the Eastern Mediterranean Deep Waters (EMDW); in the Cretan Sea for the Cretan Intermediate Waters (CIW) and the Cretan Deep Waters (CDW); in the Rhode Gyre area for the Levantine Intermediate Waters (LIW) and the Levantine Deep Waters (LDW). Product used: MEDSEA_MULTIYEAR_PHY_006_004.

**DOI (product):** 
https://doi.org/10.48670/mds-00318

**References:**

* Escudier R., Clementi E., Cipollone A., Pistoia J., Drudi M., Grandi A., Lyubartsev V., Lecci R., Aydogdu A., Delrosso D., Omar M., Masina S., Coppini G., Pinardi N. 2021. A High Resolution Reanalysis for the Mediterranean Sea. Frontiers in Earth Science, Vol.9, pp.1060, DOI:10.3389/feart.2021.702285.
* Escudier, R., Clementi, E., Omar, M., Cipollone, A., Pistoia, J., Aydogdu, A., Drudi, M., Grandi, A., Lyubartsev, V., Lecci, R., Cretí, S., Masina, S., Coppini, G., & Pinardi, N. (2020). Mediterranean Sea Physical Reanalysis (CMEMS MED-Currents) (Version 1) set. Copernicus Monitoring Environment Marine Service (CMEMS). https://doi.org/10.25423/CMCC/MEDSEA_MULTIYEAR_PHY_006_004_E3R1
* Gačić, M., Civitarese, G., Kovačević, V., Ursella, L., Bensi, M., Menna, M., et al. 2014. Extreme winter 2012 in the Adriatic: an example of climatic effect on the BiOS rhythm. Ocean Sci. 10, 513–522. doi: 10.5194/os-10-513-2014
* Houpert, L., de Madron, X.D., Testor, P., Bosse, A., D’Ortenzio, F., Bouin, M.N., Dausse, D., Le Goff, H., Kunesch, S., Labaste, M., et al. 2016. Observations of open-ocean deep convection in the northwestern Mediterranean Sea: seasonal and inter- annual variability of mixing and deep water masses for the 2007-2013 period. J Geophys Res Oceans. 121:8139–8171. doi:10.1002/ 2016JC011857.
* Kontoyiannis, H., Theocharis, A., Nittis, K. 1999. Structures and characteristics of newly formed water masses in the NW levantine during 1986, 1992, 1995. In: Malanotte-Rizzoli P., Eremeev V.N., editor. The eastern Mediterranean as a laboratory basin for the assessment of contrasting ecosys- tems. NATO science series (series 2: environmental secur- ity), Vol. 51. Springer: Dordrecht.
* Manca, B., Kovacevic, V., Gačić, M., Viezzoli, D. 2002. Dense water formation in the Southern Adriatic Sea and spreading into the Ionian Sea in the period 1997–1999. J Mar Sys. 33/ 34:33–154.
* Mihanović, H., Vilibić, I., Šepić, J., Matić, F., Ljubešić, Z., Mauri, E., Gerin, R., Notarstefano, G., Poulain, P.-M.. 2021. Observation, preconditioning and recurrence of exceptionally high salinities in the Adriatic Sea. Frontiers in Marine Science, Vol. 8, https://www.frontiersin.org/article/10.3389/fmars.2021.672210
* Pinardi, N., Zavatarelli, M., Adani, M., Coppini, G., Fratianni, C., Oddo, P., ... & Bonaduce, A. 2015. Mediterranean Sea large-scale low-frequency ocean variability and water mass formation rates from 1987 to 2007: a retrospective analysis. Progress in Oceanography, 132, 318-332
* Schroeder, K., Gasparini, G.P., Tangherlini, M., Astraldi, M. 2006. Deep and intermediate water in the western Mediterranean under the influence of the eastern Mediterranean transient. Geophys Res Lett. 33. doi:10. 1028/2006GL02712.
* Schroeder, K., Ribotti, A., Borghini, M., Sorgente, R., Perilli, A., Gasparini, G.P. 2008. An extensive western Mediterranean deep water renewal between 2004 and 2006. Geophys Res Lett. 35(18):L18605. doi:10.1029/2008GL035146.
* Simoncelli, S. and Pinardi, N. 2018. Water mass formation processes in the Mediterranean sea over the past 30 years. In: Copernicus Marine Service Ocean State Report, Issue 2, Journal of Operational Oceanography, 11:sup1, s13–s16, DOI: 10.1080/1755876X.2018.1489208.
* Somot, S., Houpert, L., Sevault, F., Testor, P., Bosse, A., Taupier-Letage, I., Bouin, M.N., Waldman, R., Cassou, C., Sanchez-Gomez, E., et al. 2016. Characterizing, modelling and under- standing the climate variability of the deep water formation in the North-Western Mediterranean Sea. Clim Dyn. 1–32. doi:10.1007/s00382-016-3295-0.
* Velaoras, D., Krokos, G., Nittis, K., Theocharis, A. 2014. Dense intermediate water outflow from the Cretan Sea: a salinity driven, recurrent phenomenon, connected to thermohaline circulation changes. J Geophys Res Oceans. 119:4797–4820. doi:10.1002/2014JC009937.
* Vilibić, I., Šantić, D. 2008. Deep water ventilation traced by Synechococcus cyanobacteria. Ocean Dyn 58:119–125. doi:10.1007/s10236-008-0135-8.
* Von Schuckmann K. et al. (2018) Copernicus Marine Service Ocean State Report, Journal of Operational Oceanography, 11:sup1, S1-S142, DOI: 10.1080/1755876X.2018.1489208





<img src="https://mdl-metadata.s3.waw3-1.cloudferro.com/metadata/thumbnails/SEAICE_ANT_PHY_AUTO_L3_NRT_011_012.jpg" width="300" height="200"/>



# Antarctic Ocean - High Resolution Sea Ice Information


**Product ID:** SEAICE_ANT_PHY_AUTO_L3_NRT_011_012

**Description:** For the Antarctic Sea - A sea ice concentration product based on satellite SAR imagery and microwave radiometer data: The algorithm uses SENTINEL-1 SAR EW and IW mode dual-polarized HH/HV data combined with AMSR2 radiometer data.

**DOI (product):**   
https://doi.org/10.48670/mds-00320




<img src="https://mdl-metadata.s3.waw3-1.cloudferro.com/metadata/thumbnails/SEAICE_ANT_PHY_L3_MY_011_018.jpg" width="300" height="200"/>



# Antarctic Ocean Sea Ice Drift REPROCESSED


**Product ID:** SEAICE_ANT_PHY_L3_MY_011_018

**Description:** Antarctic sea ice displacement during winter from medium resolution sensors since 2002

**DOI (product):**  
https://doi.org/10.48670/moi-00120




<img src="https://mdl-metadata.s3.waw3-1.cloudferro.com/metadata/thumbnails/SEAICE_ARC_PHY_AUTO_L3_MYNRT_011_023.jpg" width="300" height="200"/>



# Arctic Ocean - High Resolution Sea Ice Information L3


**Product ID:** SEAICE_ARC_PHY_AUTO_L3_MYNRT_011_023

**Description:** Arctic L3 sea ice product providing concentration, stage-of-development and floe size information retrieved from Sentinel-1 SAR imagery and GCOM-W AMSR2 microwave radiometer data using a deep learning algorithm and delivered on a 0.5 km grid.

**DOI (product):**   
https://doi.org/10.48670/mds-00343




<img src="https://mdl-metadata.s3.waw3-1.cloudferro.com/metadata/thumbnails/SEAICE_ARC_PHY_AUTO_L4_MYNRT_011_024.jpg" width="300" height="200"/>



# Arctic Ocean - High Resolution Sea Ice Information L4


**Product ID:** SEAICE_ARC_PHY_AUTO_L4_MYNRT_011_024

**Description:** Arctic L4 sea ice concentration product based on a L3 sea ice concentration product retrieved from Sentinel-1 SAR imagery and GCOM-W AMSR2 microwave radiometer data using a deep learning algorithm (SEAICE_ARC_PHY_AUTO_L3_MYNRT_011_023), gap-filled with OSI SAF EUMETSAT sea ice concentration products and delivered on a 1 km grid. 

**DOI (product):**   
https://doi.org/10.48670/mds-00344




<img src="https://mdl-metadata.s3.waw3-1.cloudferro.com/metadata/thumbnails/SEAICE_ARC_PHY_AUTO_L4_NRT_011_015.jpg" width="300" height="200"/>



# Arctic Ocean - High resolution Sea Ice Concentration and Sea Ice Type


**Product ID:** SEAICE_ARC_PHY_AUTO_L4_NRT_011_015

**Description:** For the European Arctic Sea - A sea ice concentration product based on SAR data and microwave radiometer. The algorithm uses SENTINEL-1 SAR EW mode dual-polarized HH/HV data combined with AMSR2 radiometer data. A sea ice type product covering the same area is produced from SENTINEL-1 SAR EW mode dual-polarized HH/HV data.

**DOI (product):**   
https://doi.org/10.48670/moi-00122




<img src="https://mdl-metadata.s3.waw3-1.cloudferro.com/metadata/thumbnails/SEAICE_ARC_PHY_CLIMATE_L3_MY_011_021.jpg" width="300" height="200"/>



# Arctic Ocean - Sea and Ice Surface Temperature REPROCESSED


**Product ID:** SEAICE_ARC_PHY_CLIMATE_L3_MY_011_021

**Description:** Arctic Sea and Ice surface temperature
**Detailed description:** Arctic Sea and Ice surface temperature product based upon reprocessed AVHRR, (A)ATSR and SLSTR SST observations from the ESA CCI project, the Copernicus C3S project and the AASTI dataset. The product is a daily supercollated field using all available sensors with a 0.05 degrees resolution, and covers surface temperatures in the ocean, the sea ice and the marginal ice zone.

**DOI (product):**
https://doi.org/10.48670/moi-00315




<img src="https://mdl-metadata.s3.waw3-1.cloudferro.com/metadata/thumbnails/SEAICE_ARC_PHY_CLIMATE_L4_MY_011_016.jpg" width="300" height="200"/>



# Arctic Ocean - Sea and Ice Surface Temperature REPROCESSED


**Product ID:** SEAICE_ARC_PHY_CLIMATE_L4_MY_011_016

**Description:** Arctic Sea and Ice surface temperature

**Detailed description:**
Arctic Sea and Ice surface temperature product based upon reprocessed AVHRR, (A)ATSR and SLSTR SST observations from the ESA CCI project, the Copernicus C3S project and the AASTI dataset. The product is a daily interpolated field with a 0.05 degrees resolution, and covers surface temperatures in the ocean, the sea ice and the marginal ice zone.

**DOI (product):**   
https://doi.org/10.48670/moi-00123




<img src="https://mdl-metadata.s3.waw3-1.cloudferro.com/metadata/thumbnails/SEAICE_ARC_PHY_L3M_NRT_011_017.jpg" width="300" height="200"/>



# ARCTIC Ocean and Sea-Ice Sigma-Nought


**Product ID:** SEAICE_ARC_PHY_L3M_NRT_011_017

**Description:** For the Arctic Ocean  - multiple Sentinel-1 scenes, Sigma0 calibrated and noise-corrected, with individual geographical map projections over Svalbard and Greenland Sea regions.

**DOI (product):**   
https://doi.org/10.48670/moi-00124




<img src="https://mdl-metadata.s3.waw3-1.cloudferro.com/metadata/thumbnails/SEAICE_ARC_SEAICE_L3_REP_OBSERVATIONS_011_010.jpg" width="300" height="200"/>



# Arctic Ocean Sea Ice Drift REPROCESSED


**Product ID:** SEAICE_ARC_SEAICE_L3_REP_OBSERVATIONS_011_010

**Description:** Arctic sea ice drift dataset at 3, 6 and 30 day lag during winter. The Arctic low resolution sea ice drift products provided from IFREMER have a 62.5 km grid resolution. They are delivered as daily products at 3, 6 and 30 days for the cold season extended at fall and spring: from September until May, it is updated on a monthly basis. The data are Merged product from radiometer and scatterometer:
* SSM/I 85 GHz V & H Merged product (1992-1999)

**DOI (product):**  
https://doi.org/10.48670/moi-00126




<img src="https://mdl-metadata.s3.waw3-1.cloudferro.com/metadata/thumbnails/SEAICE_ARC_SEAICE_L4_NRT_OBSERVATIONS_011_002.jpg" width="300" height="200"/>



# Arctic Ocean - Sea Ice Concentration Charts - Svalbard and Greenland


**Product ID:** SEAICE_ARC_SEAICE_L4_NRT_OBSERVATIONS_011_002

**Description:** For the Arctic Ocean - The operational sea ice services at MET Norway and DMI provides ice charts of the Arctic area covering Baffin Bay, Greenland Sea, Fram Strait and Barents Sea. The charts show the ice concentration in WMO defined concentration intervals. The three different types of ice charts (datasets) are produced from twice to several times a week: MET charts are produced every weekday. DMI regional charts are produced at irregular intervals daily and a supplemental DMI overview chart is produced twice weekly.

**DOI (product):**  
https://doi.org/10.48670/moi-00128




<img src="https://mdl-metadata.s3.waw3-1.cloudferro.com/metadata/thumbnails/SEAICE_ARC_SEAICE_L4_NRT_OBSERVATIONS_011_007.jpg" width="300" height="200"/>



# SAR Sea Ice Berg Concentration and Individual Icebergs Observed with Sentinel-1


**Product ID:** SEAICE_ARC_SEAICE_L4_NRT_OBSERVATIONS_011_007

**Description:** The iceberg product contains 4 datasets (IW and EW modes and mosaic for the two modes) describing iceberg concentration as number of icebergs counted within 10x10 km grid cells. The iceberg concentration is derived by applying a Constant False Alarm Rate (CFAR) algorithm on data from Synthetic Aperture Radar (SAR) satellite sensors.

The iceberg product also contains two additional datasets of individual iceberg positions in Greenland-Newfoundland-Labrador Waters. These datasets are in shapefile format to allow the best representation of the icebergs (the 1st dataset contains the iceberg point observations, the 2nd dataset contains the polygonized satellite coverage). These are also derived by applying a Constant False Alarm Rate (CFAR) algorithm on Sentinel-1 SAR imagery.
Despite its precision (individual icebergs are proposed), this product is a generic and automated product and needs expertise to be correctly used. For all applications concerning marine navigation, please refer to the national Ice Service of the country concerned.

**DOI (product):**  
https://doi.org/10.48670/moi-00129




<img src="https://mdl-metadata.s3.waw3-1.cloudferro.com/metadata/thumbnails/SEAICE_ARC_SEAICE_L4_NRT_OBSERVATIONS_011_008.jpg" width="300" height="200"/>



# Arctic Ocean - Sea and Ice Surface Temperature


**Product ID:** SEAICE_ARC_SEAICE_L4_NRT_OBSERVATIONS_011_008

**Description:** Arctic Sea and Ice surface temperature product based upon observations from the Metop_A AVHRR instrument. The product is a daily interpolated field with a 0.05 degrees resolution, and covers surface temperatures in the ocean, the sea ice and the marginal ice zone.

**DOI (product):**   
https://doi.org/10.48670/moi-00130




<img src="https://mdl-metadata.s3.waw3-1.cloudferro.com/metadata/thumbnails/SEAICE_BAL_PHY_L4_MY_011_019.jpg" width="300" height="200"/>



# Baltic Sea ice concentration, extent, and classification time series


**Product ID:** SEAICE_BAL_PHY_L4_MY_011_019

**Description:** Gridded sea ice concentration, sea ice extent and classification based on the digitized Baltic ice charts produced by the FMI/SMHI ice analysts. It is produced daily in the afternoon, describing the ice situation daily at 14:00 EET. The nominal resolution is about 1km. The temporal coverage is from the beginning of the season 1980-1981 until today.

**DOI (product):**  
https://doi.org/10.48670/moi-00131




<img src="https://mdl-metadata.s3.waw3-1.cloudferro.com/metadata/thumbnails/SEAICE_BAL_SEAICE_L4_NRT_OBSERVATIONS_011_004.jpg" width="300" height="200"/>



# Baltic Sea - Sea Ice Concentration and Thickness Charts


**Product ID:** SEAICE_BAL_SEAICE_L4_NRT_OBSERVATIONS_011_004

**Description:** For the Baltic Sea- The operational sea ice service at FMI provides ice parameters over the Baltic Sea. The parameters are based on ice chart produced on daily basis during the Baltic Sea ice season and show the ice concentration in a 1 km grid. Ice thickness chart (ITC) is a product based on the most recent available ice chart (IC) and a SAR image. The SAR data is used to update the ice information in the IC. The ice regions in the IC are updated according to a SAR segmentation and new ice thickness values are assigned to each SAR segment based on the SAR backscattering and the ice IC thickness range at that location.

**DOI (product):**   
https://doi.org/10.48670/moi-00132

**References:**

* J. Karvonen, M. Simila, SAR-Based Estimation of the Baltic Sea Ice Motion, Proc. of the International Geoscience and Remote Sensing Symposium 2007 (IGARSS 07), pp. 2605-2608, 2007. (Unfortunately there is no publication of the new algorithm version yet).





<img src="https://mdl-metadata.s3.waw3-1.cloudferro.com/metadata/thumbnails/SEAICE_BAL_SEAICE_L4_NRT_OBSERVATIONS_011_011.jpg" width="300" height="200"/>



# Baltic Sea - SAR Sea Ice Thickness and Drift, Multisensor Sea Ice Concentration


**Product ID:** SEAICE_BAL_SEAICE_L4_NRT_OBSERVATIONS_011_011

**Description:** For the Baltic Sea - The operational sea ice service at FMI provides ice parameters over the Baltic Sea. The products are based on SAR images and are produced on pass-by-pass basis during the Baltic Sea ice season, and show the ice thickness and drift in a 500 m and 800m grid, respectively.  The Baltic sea ice concentration product is based on data from SAR and microwave radiometer. The algorithm uses SENTINEL-1 SAR EW mode dual-polarized HH/HV data combined with AMSR2 radiometer data.

**DOI (product):**   
https://doi.org/10.48670/moi-00133

**References:**

* J. Karvonen, Operational SAR-based sea ice drift monitoring over the Baltic Sea, Ocean Science, v. 8, pp. 473-483, (http://www.ocean-sci.net/8/473/2012/os-8-473-2012.html) 2012.





<img src="https://mdl-metadata.s3.waw3-1.cloudferro.com/metadata/thumbnails/SEAICE_GLO_PHY_CLIMATE_L3_MY_011_013.jpg" width="300" height="200"/>



# Arctic Ocean - Sea Ice Thickness REPROCESSED


**Product ID:** SEAICE_GLO_PHY_CLIMATE_L3_MY_011_013

**Description:** Arctic sea ice L3 data in separate monthly files. The time series is based on reprocessed radar altimeter satellite data from Envisat and CryoSat and is available in the freezing season between October and April. The product is brokered from the Copernicus Climate Change Service (C3S).

**DOI (product):**   
https://doi.org/10.48670/moi-00127




<img src="https://mdl-metadata.s3.waw3-1.cloudferro.com/metadata/thumbnails/SEAICE_GLO_PHY_L4_MY_011_020.jpg" width="300" height="200"/>



# Global Ocean - High Resolution SAR Sea Ice Drift Time Series


**Product ID:** SEAICE_GLO_PHY_L4_MY_011_020

**Description:** The  product contains a reprocessed multi year version of the daily composite dataset from SEAICE_GLO_SEAICE_L4_NRT_OBSERVATIONS_011_006 covering the Sentinel1 years from autumn 2014 until 1 year before present

**DOI (product):**   
https://doi.org/10.48670/mds-00328




<img src="https://mdl-metadata.s3.waw3-1.cloudferro.com/metadata/thumbnails/SEAICE_GLO_PHY_L4_NRT_011_014.jpg" width="300" height="200"/>



# Sea Ice Thickness derived from merging of L-Band radiometry and radar altimeter derived sea ice thickness


**Product ID:** SEAICE_GLO_PHY_L4_NRT_011_014

**Description:** Arctic sea ice thickness from merged  L-Band radiometer (SMOS ) and radar altimeter (CryoSat-2, Sentinel-3A/B) observations during freezing season between October and April in the northern hemisphere and Aprilt to October in the southern hemisphere. The SMOS mission provides L-band observations and the ice thickness-dependency of brightness temperature enables to estimate the sea-ice thickness for thin ice regimes. Radar altimeters measure the height of the ice surface above the water level, which can be converted into sea ice thickness assuming hydrostatic equilibrium. 

**DOI (product):**   
https://doi.org/10.48670/moi-00125




<img src="https://mdl-metadata.s3.waw3-1.cloudferro.com/metadata/thumbnails/SEAICE_GLO_SEAICE_L4_NRT_OBSERVATIONS_011_001.jpg" width="300" height="200"/>



# Global Ocean - Arctic and Antarctic - Sea Ice Concentration, Edge, Type and Drift (OSI-SAF)


**Product ID:** SEAICE_GLO_SEAICE_L4_NRT_OBSERVATIONS_011_001

**Description:** For the Global - Arctic and Antarctic - Ocean. The OSI SAF delivers five global sea ice products in operational mode: sea ice concentration, sea ice edge, sea ice type (OSI-401, OSI-402, OSI-403, OSI-405 and OSI-408). The sea ice concentration, edge and type products are delivered daily at 10km resolution and the sea ice drift in 62.5km resolution, all in polar stereographic projections covering the Northern Hemisphere and the Southern Hemisphere. The sea ice drift motion vectors have a time-span of 2 days. These are the Sea Ice operational nominal products for the Global Ocean.

**DOI (product):**   
https://doi.org/10.48670/moi-00134




<img src="https://mdl-metadata.s3.waw3-1.cloudferro.com/metadata/thumbnails/SEAICE_GLO_SEAICE_L4_NRT_OBSERVATIONS_011_006.jpg" width="300" height="200"/>



# Global Ocean - High Resolution SAR Sea Ice Drift


**Product ID:** SEAICE_GLO_SEAICE_L4_NRT_OBSERVATIONS_011_006

**Description:** DTU Space produces polar covering Near Real Time gridded ice displacement fields obtained by MCC processing of Sentinel-1 SAR, Envisat ASAR WSM swath data or RADARSAT ScanSAR Wide mode data . The nominal temporal span between processed swaths is 24hours, the nominal product grid resolution is a 10km.

**DOI (product):**   
https://doi.org/10.48670/moi-00135




<img src="https://mdl-metadata.s3.waw3-1.cloudferro.com/metadata/thumbnails/SEAICE_GLO_SEAICE_L4_REP_OBSERVATIONS_011_009.jpg" width="300" height="200"/>



# Global Ocean Sea Ice Concentration Time Series REPROCESSED (OSI-SAF)


**Product ID:** SEAICE_GLO_SEAICE_L4_REP_OBSERVATIONS_011_009

**Description:** The CDR and ICDR sea ice concentration dataset of the EUMETSAT OSI SAF (OSI-450-a and OSI-430-a), covering the period from October 1978 to present, with 16 days delay. It used passive microwave data from SMMR, SSM/I and SSMIS. Sea ice concentration is computed from atmospherically corrected PMW brightness temperatures, using a combination of state-of-the-art algorithms and dynamic tie points. It includes error bars for each grid cell (uncertainties). This version 3.0 of the CDR (OSI-450-a, 1978-2020) and ICDR (OSI-430-a, 2021-present with 16 days latency) was released in November 2022

**DOI (product):**   
https://doi.org/10.48670/moi-00136

**References:**

* [http://osisaf.met.no/docs/osisaf_cdop2_ss2_pum_sea-ice-conc-reproc_v2p2.pdf]





<img src="https://mdl-metadata.s3.waw3-1.cloudferro.com/metadata/thumbnails/SEALEVEL_BLK_PHY_MDT_L4_STATIC_008_067.jpg" width="300" height="200"/>



# BLACK SEA MEAN DYNAMIC TOPOGRAPHY


**Product ID:** SEALEVEL_BLK_PHY_MDT_L4_STATIC_008_067

**Description:** The Mean Dynamic Topography MDT-CMEMS_2020_BLK is an estimate of the mean over the 1993-2012 period of the sea surface height above geoid for the Black Sea. This is consistent with the reference time period also used in the SSALTO DUACS products

**DOI (product):**   
https://doi.org/10.48670/moi-00138




<img src="https://mdl-metadata.s3.waw3-1.cloudferro.com/metadata/thumbnails/SEALEVEL_EUR_PHY_L3_MY_008_061.jpg" width="300" height="200"/>



# EUROPEAN SEAS ALONG-TRACK L3 SEA SURFACE HEIGHTS REPROCESSED (1993-ONGOING) TAILORED FOR DATA ASSIMILATION


**Product ID:** SEALEVEL_EUR_PHY_L3_MY_008_061

**Description:** Altimeter satellite along-track sea surface heights anomalies (SLA) computed with respect to a twenty-year [1993, 2012] mean with a 1Hz (~7km) sampling. It serves in delayed-time applications.
This product is processed by the DUACS multimission altimeter data processing system. It processes data from all altimeter missions available (e.g. Sentinel-6A, Jason-3, Sentinel-3A, Sentinel-3B, Saral/AltiKa, Cryosat-2, Jason-1, Jason-2, Topex/Poseidon, ERS-1, ERS-2, Envisat, Geosat Follow-On, HY-2A, HY-2B, etc). The system exploits the most recent datasets available based on the enhanced GDR/NTC production. All the missions are homogenized with respect to a reference mission. Part of the processing is fitted to the European Sea area. (see QUID document or http://duacs.cls.fr [](http://duacs.cls.fr) pages for processing details). 
The product gives additional variables (e.g. Mean Dynamic Topography, Dynamic Atmospheric Correction, Ocean Tides, Long Wavelength Errors) that can be used to change the physical content for specific needs (see PUM document for details)

“’Associated products”’
A time invariant product https://resources.marine.copernicus.eu/product-detail/SEALEVEL_GLO_PHY_NOISE_L4_STATIC_008_033/INFORMATION describing the noise level of along-track measurements is available. It is associated to the sla_filtered variable. It is a gridded product. One file is provided for the global ocean and those values must be applied for Arctic and Europe products. For Mediterranean and Black seas, one value is given in the QUID document.

**DOI (product):**
https://doi.org/10.48670/moi-00139




<img src="https://mdl-metadata.s3.waw3-1.cloudferro.com/metadata/thumbnails/SEALEVEL_EUR_PHY_L3_NRT_008_059.jpg" width="300" height="200"/>



# EUROPEAN SEAS ALONG-TRACK L3 SEA LEVEL ANOMALIES NRT


**Product ID:** SEALEVEL_EUR_PHY_L3_NRT_008_059

**Description:** Altimeter satellite along-track sea surface heights anomalies (SLA) computed with respect to a twenty-year [1993, 2012] mean with a 1Hz (~7km) and 5Hz (~1km) sampling. It serves in near-real time applications.
This product is processed by the DUACS multimission altimeter data processing system. It processes data from all altimeter missions available (e.g. Sentinel-6A, Jason-3, Sentinel-3A, Sentinel-3B, Saral/AltiKa, Cryosat-2, HY-2B). The system exploits the most recent datasets available based on the enhanced OGDR/NRT+IGDR/STC production. All the missions are homogenized with respect to a reference mission. Part of the processing is fitted to the European Seas. (see QUID document or http://duacs.cls.fr [](http://duacs.cls.fr) pages for processing details). 
The product gives additional variables (e.g. Mean Dynamic Topography, Dynamic Atmospheric Correction, Ocean Tides, Long Wavelength Errors) that can be used to change the physical content for specific needs (see PUM document for details)

**Associated products**

A time invariant product http://marine.copernicus.eu/services-portfolio/access-to-products/?option=com_csw&view=details&product_id=SEALEVEL_GLO_PHY_NOISE_L4_STATIC_008_033 [](http://marine.copernicus.eu/services-portfolio/access-to-products/?option=com_csw&view=details&product_id=SEALEVEL_GLO_PHY_NOISE_L4_STATIC_008_033) describing the noise level of along-track measurements is available. It is associated to the sla_filtered variable. It is a gridded product. One file is provided for the global ocean and those values must be applied for Arctic and Europe products. For Mediterranean and Black seas, one value is given in the QUID document.

**DOI (product):**   
https://doi.org/10.48670/moi-00140




<img src="https://mdl-metadata.s3.waw3-1.cloudferro.com/metadata/thumbnails/SEALEVEL_EUR_PHY_L4_MY_008_068.jpg" width="300" height="200"/>



# EUROPEAN SEAS GRIDDED L4 SEA SURFACE HEIGHTS AND DERIVED VARIABLES REPROCESSED (1993-ONGOING)


**Product ID:** SEALEVEL_EUR_PHY_L4_MY_008_068

**Description:** Altimeter satellite gridded Sea Level Anomalies (SLA) computed with respect to a twenty-year [1993, 2012] mean. The SLA is estimated by Optimal Interpolation, merging the L3 along-track measurement from the different altimeter missions available. Part of the processing is fitted to the European Sea area. (see QUID document or http://duacs.cls.fr [](http://duacs.cls.fr) pages for processing details). The product gives additional variables (i.e. Absolute Dynamic Topography and geostrophic currents (absolute and anomalies)). It serves in delayed-time applications.
This product is processed by the DUACS multimission altimeter data processing system.

**DOI (product):**
https://doi.org/10.48670/moi-00141




<img src="https://mdl-metadata.s3.waw3-1.cloudferro.com/metadata/thumbnails/SEALEVEL_EUR_PHY_L4_NRT_008_060.jpg" width="300" height="200"/>



# EUROPEAN SEAS GRIDDED L4 SEA SURFACE HEIGHTS AND DERIVED VARIABLES NRT


**Product ID:** SEALEVEL_EUR_PHY_L4_NRT_008_060

**Description:** Altimeter satellite gridded Sea Level Anomalies (SLA) computed with respect to a twenty-year [1993, 2012] mean. The SLA is estimated by Optimal Interpolation, merging the L3 along-track measurement from the different altimeter missions available. Part of the processing is fitted to the European Sea area. (see QUID document or http://duacs.cls.fr [](http://duacs.cls.fr) pages for processing details). The product gives additional variables (i.e. Absolute Dynamic Topography and geostrophic currents (absolute and anomalies)). It serves in near-real time applications.
This product is processed by the DUACS multimission altimeter data processing system. 

**DOI (product):**   
https://doi.org/10.48670/moi-00142




<img src="https://mdl-metadata.s3.waw3-1.cloudferro.com/metadata/thumbnails/SEALEVEL_EUR_PHY_MDT_L4_STATIC_008_070.jpg" width="300" height="200"/>



# EUROPEAN SEAS MEAN DYNAMIC TOPOGRAPHY


**Product ID:** SEALEVEL_EUR_PHY_MDT_L4_STATIC_008_070

**Description:** The Mean Dynamic Topography MDT-CMEMS_2024_EUR is an estimate of the mean over the 1993-2012 period of the sea surface height above geoid for the European Seas. This is consistent with the reference time period also used in the SSALTO DUACS products

**DOI (product):**   
https://doi.org/10.48670/mds-00337




<img src="https://mdl-metadata.s3.waw3-1.cloudferro.com/metadata/thumbnails/SEALEVEL_GLO_PHY_CLIMATE_L4_MY_008_057.jpg" width="300" height="200"/>



# GLOBAL OCEAN GRIDDED L4 SEA SURFACE HEIGHTS AND DERIVED VARIABLES REPROCESSED (COPERNICUS CLIMATE SERVICE)


**Product ID:** SEALEVEL_GLO_PHY_CLIMATE_L4_MY_008_057

**Description:** DUACS delayed-time altimeter gridded maps of sea surface heights and derived variables over the global Ocean (https://cds.climate.copernicus.eu/cdsapp#!/dataset/satellite-sea-level-global?tab=overview). The processing focuses on the stability and homogeneity of the sea level record (based on a stable two-satellite constellation) and the product is dedicated to the monitoring of the sea level long-term evolution for climate applications and the analysis of Ocean/Climate indicators. These products are produced and distributed by the Copernicus Climate Change Service (C3S, https://climate.copernicus.eu/).

**DOI (product):**
https://doi.org/10.48670/moi-00145




<img src="https://mdl-metadata.s3.waw3-1.cloudferro.com/metadata/thumbnails/SEALEVEL_GLO_PHY_L3_MY_008_062.jpg" width="300" height="200"/>



# GLOBAL OCEAN ALONG-TRACK L3 SEA SURFACE HEIGHTS REPROCESSED (1993-ONGOING) TAILORED FOR DATA ASSIMILATION


**Product ID:** SEALEVEL_GLO_PHY_L3_MY_008_062

**Description:** Altimeter satellite along-track sea surface heights anomalies (SLA) computed with respect to a twenty-year [1993, 2012] mean with a 1Hz (~7km) sampling. It serves in delayed-time applications.
This product is processed by the DUACS multimission altimeter data processing system. It processes data from all altimeter missions available (e.g. Sentinel-6A, Jason-3, Sentinel-3A, Sentinel-3B, Saral/AltiKa, Cryosat-2, Jason-1, Jason-2, Topex/Poseidon, ERS-1, ERS-2, Envisat, Geosat Follow-On, HY-2A, HY-2B, etc.). The system exploits the most recent datasets available based on the enhanced GDR/NTC production. All the missions are homogenized with respect to a reference mission. Part of the processing is fitted to the Global ocean. (see QUID document or http://duacs.cls.fr [](http://duacs.cls.fr) pages for processing details). 
The product gives additional variables (e.g. Mean Dynamic Topography, Dynamic Atmospheric Correction, Ocean Tides, Long Wavelength Errors) that can be used to change the physical content for specific needs (see PUM document for details)

**Associated products**
A time invariant product https://resources.marine.copernicus.eu/product-detail/SEALEVEL_GLO_PHY_NOISE_L4_STATIC_008_033/INFORMATION describing the noise level of along-track measurements is available. It is associated to the sla_filtered variable. It is a gridded product. One file is provided for the global ocean and those values must be applied for Arctic and Europe products. For Mediterranean and Black seas, one value is given in the QUID document.

**DOI (product)**:
https://doi.org/10.48670/moi-00146




<img src="https://mdl-metadata.s3.waw3-1.cloudferro.com/metadata/thumbnails/SEALEVEL_GLO_PHY_L3_NRT_008_044.jpg" width="300" height="200"/>



# GLOBAL OCEAN ALONG-TRACK L3 SEA SURFACE HEIGHTS NRT


**Product ID:** SEALEVEL_GLO_PHY_L3_NRT_008_044

**Description:** Altimeter satellite along-track sea surface heights anomalies (SLA) computed with respect to a twenty-year [1993, 2012] mean with a 1Hz (~7km) and 5Hz (~1km) sampling. It serves in near-real time applications.
This product is processed by the DUACS multimission altimeter data processing system. It processes data from all altimeter missions available (e.g. Sentinel-6A, Jason-3, Sentinel-3A, Sentinel-3B, Saral/AltiKa, Cryosat-2, HY-2B). The system exploits the most recent datasets available based on the enhanced OGDR/NRT+IGDR/STC production. All the missions are homogenized with respect to a reference mission. Part of the processing is fitted to the Global Ocean. (see QUID document or http://duacs.cls.fr [](http://duacs.cls.fr) pages for processing details). 
The product gives additional variables (e.g. Mean Dynamic Topography, Dynamic Atmospheric Correction, Ocean Tides, Long Wavelength Errors) that can be used to change the physical content for specific needs (see PUM document for details)

**Associated products**
A time invariant product http://marine.copernicus.eu/services-portfolio/access-to-products/?option=com_csw&view=details&product_id=SEALEVEL_GLO_PHY_NOISE_L4_STATIC_008_033 [](http://marine.copernicus.eu/services-portfolio/access-to-products/?option=com_csw&view=details&product_id=SEALEVEL_GLO_PHY_NOISE_L4_STATIC_008_033) describing the noise level of along-track measurements is available. It is associated to the sla_filtered variable. It is a gridded product. One file is provided for the global ocean and those values must be applied for Arctic and Europe products. For Mediterranean and Black seas, one value is given in the QUID document.

**DOI (product)**:
https://doi.org/10.48670/moi-00147




<img src="https://mdl-metadata.s3.waw3-1.cloudferro.com/metadata/thumbnails/SEALEVEL_GLO_PHY_L4_MY_008_047.jpg" width="300" height="200"/>



# GLOBAL OCEAN GRIDDED L4 SEA SURFACE HEIGHTS AND DERIVED VARIABLES REPROCESSED (1993-ONGOING)


**Product ID:** SEALEVEL_GLO_PHY_L4_MY_008_047

**Description:** Altimeter satellite gridded Sea Level Anomalies (SLA) computed with respect to a twenty-year [1993, 2012] mean. The SLA is estimated by Optimal Interpolation, merging the L3 along-track measurement from the different altimeter missions available. Part of the processing is fitted to the Global ocean. (see QUID document or http://duacs.cls.fr [](http://duacs.cls.fr) pages for processing details). The product gives additional variables (i.e. Absolute Dynamic Topography and geostrophic currents (absolute and anomalies)). It serves in delayed-time applications.
This product is processed by the DUACS multimission altimeter data processing system.

**DOI (product):**   
https://doi.org/10.48670/moi-00148




<img src="https://mdl-metadata.s3.waw3-1.cloudferro.com/metadata/thumbnails/SEALEVEL_GLO_PHY_L4_NRT_008_046.jpg" width="300" height="200"/>



# GLOBAL OCEAN GRIDDED L4 SEA SURFACE HEIGHTS AND DERIVED VARIABLES NRT


**Product ID:** SEALEVEL_GLO_PHY_L4_NRT_008_046

**Description:** Altimeter satellite gridded Sea Level Anomalies (SLA) computed with respect to a twenty-year [1993, 2012] mean. The SLA is estimated by Optimal Interpolation, merging the L3 along-track measurement from the different altimeter missions available. Part of the processing is fitted to the Global Ocean. (see QUID document or http://duacs.cls.fr [](http://duacs.cls.fr) pages for processing details). The product gives additional variables (i.e. Absolute Dynamic Topography and geostrophic currents (absolute and anomalies)). It serves in near-real time applications.
This product is processed by the DUACS multimission altimeter data processing system. 

**DOI (product):**   
https://doi.org/10.48670/moi-00149




<img src="https://mdl-metadata.s3.waw3-1.cloudferro.com/metadata/thumbnails/SEALEVEL_GLO_PHY_MDT_008_063.jpg" width="300" height="200"/>



# GLOBAL OCEAN MEAN DYNAMIC TOPOGRAPHY


**Product ID:** SEALEVEL_GLO_PHY_MDT_008_063

**Description:** Mean Dynamic Topography that combines the global CNES-CLS-2022 MDT, the Black Sea CMEMS2020 MDT and the Med Sea CMEMS2020 MDT. It  is an estimate of the mean over the 1993-2012 period of the sea surface height above geoid. This is consistent with the reference time period also used in the  DUACS products

**DOI (product):**  
https://doi.org/10.48670/moi-00150




<img src="https://mdl-metadata.s3.waw3-1.cloudferro.com/metadata/thumbnails/SEALEVEL_GLO_PHY_NOISE_L4_STATIC_008_033.jpg" width="300" height="200"/>



# GLOBAL OCEAN GRIDDED NORMALIZED MEASUREMENT NOISE OF SEA LEVEL ANOMALIES


**Product ID:** SEALEVEL_GLO_PHY_NOISE_L4_STATIC_008_033

**Description:** In wavenumber spectra, the 1hz measurement error is the noise level estimated as the mean value of energy at high wavenumbers (below ~20km in term of wavelength). The 1hz noise level spatial distribution follows the instrumental white-noise linked to the Surface Wave Height but also connections with the backscatter coefficient. The full understanding of this hump of spectral energy (Dibarboure et al., 2013, Investigating short wavelength correlated errors on low-resolution mode altimetry, OSTST 2013 presentation) still remain to be achieved and overcome with new retracking, new editing strategy or new technology.

**DOI (product):**   
https://doi.org/10.48670/moi-00144




<img src="https://mdl-metadata.s3.waw3-1.cloudferro.com/metadata/thumbnails/SEALEVEL_MED_PHY_MDT_L4_STATIC_008_066.jpg" width="300" height="200"/>



# MEDITERRANEAN SEA MEAN DYNAMIC TOPOGRAPHY


**Product ID:** SEALEVEL_MED_PHY_MDT_L4_STATIC_008_066

**Description:** The Mean Dynamic Topography MDT-CMEMS_2020_MED is an estimate of the mean over the 1993-2012 period of the sea surface height above geoid for the Mediterranean Sea. This is consistent with the reference time period also used in the SSALTO DUACS products

**DOI (product):**   
https://doi.org/10.48670/moi-00151




<img src="https://mdl-metadata.s3.waw3-1.cloudferro.com/metadata/thumbnails/SST_ATL_PHY_L3S_MY_010_038.jpg" width="300" height="200"/>



# European North West Shelf/Iberia Biscay Irish Seas – High Resolution ODYSSEA Sea Surface Temperature Multi-sensor L3 Observations Reprocessed


**Product ID:** SST_ATL_PHY_L3S_MY_010_038

**Description:** For the NWS/IBI Ocean- Sea Surface Temperature L3 Observations . This product provides daily foundation sea surface temperature from multiple satellite sources. The data are intercalibrated. This product consists in a fusion of sea surface temperature observations from multiple satellite sensors, daily, over a 0.05° resolution grid. It includes observations by polar orbiting from the ESA CCI / C3S archive .  The L3S SST data are produced selecting only the highest quality input data from input L2P/L3P images within a strict temporal window (local nightime), to avoid diurnal cycle and cloud contamination. The observations of each sensor are intercalibrated prior to merging using a bias correction based on a multi-sensor median reference correcting the large-scale cross-sensor biases.

**DOI (product):**   
https://doi.org/10.48670/moi-00311




<img src="https://mdl-metadata.s3.waw3-1.cloudferro.com/metadata/thumbnails/SST_ATL_PHY_L3S_NRT_010_037.jpg" width="300" height="200"/>



# European North West Shelf/Iberia Biscay Irish Seas – High Resolution ODYSSEA Sea Surface Temperature Multi-sensor L3 Observations


**Product ID:** SST_ATL_PHY_L3S_NRT_010_037

**Description:** For the NWS/IBI Ocean- Sea Surface Temperature L3 Observations . This product provides daily foundation sea surface temperature from multiple satellite sources. The data are intercalibrated. This product consists in a fusion of sea surface temperature observations from multiple satellite sensors, daily, over a 0.02° resolution grid. It includes observations by polar orbiting and geostationary satellites .  The L3S SST data are produced selecting only the highest quality input data from input L2P/L3P images within a strict temporal window (local nightime), to avoid diurnal cycle and cloud contamination. The observations of each sensor are intercalibrated prior to merging using a bias correction based on a multi-sensor median reference correcting the large-scale cross-sensor biases. 3 more datasets are available that only contain "per sensor type" data: Polar InfraRed (PIR), Polar MicroWave (PMW), Geostationary InfraRed (GIR)

**DOI (product):**   
https://doi.org/10.48670/moi-00310




<img src="https://mdl-metadata.s3.waw3-1.cloudferro.com/metadata/thumbnails/SST_ATL_SST_L4_NRT_OBSERVATIONS_010_025.jpg" width="300" height="200"/>



# European North West Shelf/Iberia Biscay Irish Seas – High Resolution ODYSSEA L4 Sea Surface Temperature Analysis


**Product ID:** SST_ATL_SST_L4_NRT_OBSERVATIONS_010_025

**Description:** For the Atlantic European North West Shelf Ocean-European North West Shelf/Iberia Biscay Irish Seas. The ODYSSEA NW+IBI Sea Surface Temperature analysis aims at providing daily gap-free maps of sea surface temperature, referred as L4 product, at 0.02deg x 0.02deg horizontal resolution, using satellite data from both infra-red and micro-wave radiometers. It is the sea surface temperature operational nominal product for the Northwest Shelf Sea and Iberia Biscay Irish Seas.

**DOI (product):**   
https://doi.org/10.48670/moi-00152




<img src="https://mdl-metadata.s3.waw3-1.cloudferro.com/metadata/thumbnails/SST_ATL_SST_L4_REP_OBSERVATIONS_010_026.jpg" width="300" height="200"/>



# European North West Shelf/Iberia Biscay Irish Seas - High Resolution L4 Sea Surface Temperature Reprocessed


**Product ID:** SST_ATL_SST_L4_REP_OBSERVATIONS_010_026

**Description:** For the European North West Shelf Ocean Iberia Biscay Irish Seas. The IFREMER Sea Surface Temperature reprocessed analysis aims at providing daily gap-free maps of sea surface temperature, referred as L4 product, at 0.05deg. x 0.05deg. horizontal resolution, over the 1982-present period, using satellite data from the European Space Agency Sea Surface Temperature Climate Change Initiative (ESA SST CCI) L3 products (1982-2016) and from the Copernicus Climate Change Service (C3S) L3 product (2017-present). The gridded SST product is intended to represent a daily-mean SST field at 20 cm depth.

**DOI (product):**   
https://doi.org/10.48670/moi-00153




<img src="https://mdl-metadata.s3.waw3-1.cloudferro.com/metadata/thumbnails/SST_BAL_PHY_L3S_MY_010_040.jpg" width="300" height="200"/>



# Baltic Sea - L3S Sea Surface Temperature Reprocessed


**Product ID:** SST_BAL_PHY_L3S_MY_010_040

**Description:** For the Baltic Sea- The DMI Sea Surface Temperature reprocessed analysis provides daily Level 3 Super-Collated fields of the sea surface temperature fields, at 0.02deg. x 0.02deg. horizontal resolution. It is produced by the DMI Optimal Interpolation (DMIOI) system (Høyer and She, 2007) as the first step prior to providing the high resolution (1/50deg. - approx. 2km grid resolution) daily analysis of the daily average sea surface temperature (SST) at 20 cm depth. It uses satellite data from infra-red radiometers, from the ESA SST_cci v3.0 (Embury et al., 2024) and Copernicus C3S projects, namely L2P data from (A)ATSRs, SLSTR and AVHRR for the period 1982-2021, L3U data from SLSTR and AVHRR for 2022-July 19 2024 and L2P data from SLSTR and AVHRR from July 20 2024 onward. For the Sea Ice Concentration it uses the Baltic high resolution sea ice concentration data from the Copernicus Marine Service SI TAC (SEAICE_BAL_PHY_L4_MY_011_019). 

**DOI (product):**   
https://doi.org/10.48670/moi-00312

**References:**

* Høyer, J. L., Le Borgne, P. and Eastwood, S. 2014. A bias correction method for Arctic satellite sea surface temperature observations, Remote Sensing of Environment, https://doi.org/10.1016/j.rse.2013.04.020.
* Høyer, J. L. and She, J., Optimal interpolation of sea surface temperature for the North Sea and Baltic Sea, J. Mar. Sys., Vol 65, 1-4, pp., 2007.Høyer, J. L. and She, J., Optimal interpolation of sea surface temperature for the North Sea and Baltic Sea, J. Mar. Sys., Vol 65, 1-4, pp., 2007.
* Embury, O., Merchant, C.J., Good, S.A., Rayner, N.A., Høyer, J.L., Atkinson, C., Block, T., Alerskans, E., Pearson, K.J., Worsfold, M., McCarroll, N., Donlon, C. Satellite-based time-series of sea-surface temperature since 1980 for climate applications. Scientific Data 11, 326 (2024). https://doi.org/10.1038/s41597-024-03147-w"





<img src="https://mdl-metadata.s3.waw3-1.cloudferro.com/metadata/thumbnails/SST_BAL_PHY_SUBSKIN_L4_NRT_010_034.jpg" width="300" height="200"/>



# Baltic Sea - Diurnal Subskin Sea Surface Temperature Analysis


**Product ID:** SST_BAL_PHY_SUBSKIN_L4_NRT_010_034

**Description:** For the Baltic Sea - the DMI Sea Surface Temperature Diurnal Subskin L4 aims at providing hourly analysis of the diurnal subskin signal at 0.02deg. x 0.02deg. horizontal resolution, using the BAL L4 NRT product as foundation temperature and satellite data from infra-red radiometers. Uses SST satellite products from the sensors: Metop B AVHRR, Sentinel-3 A/B SLSTR, VIIRS SUOMI NPP & NOAA20 

**DOI (product):**   
https://doi.org/10.48670/moi-00309

**References:**

* Karagali I. and Høyer, J. L. (2014). Characterisation and quantification of regional diurnal cycles from SEVIRI. Ocean Science, 10 (5), 745-758.
* Høyer, J. L., Le Borgne, P. and Eastwood, S. 2014. A bias correction method for Arctic satellite sea surface temperature observations, Remote Sensing of Environment, https://doi.org/10.1016/j.rse.2013.04.020.
* Høyer, J. L. and She, J., Optimal interpolation of sea surface temperature for the North Sea and Baltic Sea, J. Mar. Sys., Vol 65, 1-4, pp., 2007.Høyer, J. L. and She, J., Optimal interpolation of sea surface temperature for the North Sea and Baltic Sea, J. Mar. Sys., Vol 65, 1-4, pp., 2007.





<img src="https://mdl-metadata.s3.waw3-1.cloudferro.com/metadata/thumbnails/SST_BAL_SST_L3S_NRT_OBSERVATIONS_010_032.jpg" width="300" height="200"/>



# North Sea/Baltic Sea - Sea Surface Temperature Analysis L3S


**Product ID:** SST_BAL_SST_L3S_NRT_OBSERVATIONS_010_032

**Description:** For the Baltic Sea- The DMI Sea Surface Temperature L3S aims at providing daily multi-sensor supercollated data at 0.03deg. x 0.03deg. horizontal resolution, using satellite data from infra-red radiometers. Uses SST satellite products from these sensors: NOAA AVHRRs 7, 9, 11, 14, 16, 17, 18 , Envisat ATSR1, ATSR2 and AATSR.

**DOI (product):**   
https://doi.org/10.48670/moi-00154

**References:**

* Høyer, J. L., Le Borgne, P. and Eastwood, S. 2014. A bias correction method for Arctic satellite sea surface temperature observations, Remote Sensing of Environment, https://doi.org/10.1016/j.rse.2013.04.020.
* Høyer, J. L. and She, J., Optimal interpolation of sea surface temperature for the North Sea and Baltic Sea, J. Mar. Sys., Vol 65, 1-4, pp., 2007.Høyer, J. L. and She, J., Optimal interpolation of sea surface temperature for the North Sea and Baltic Sea, J. Mar. Sys., Vol 65, 1-4, pp., 2007.





<img src="https://mdl-metadata.s3.waw3-1.cloudferro.com/metadata/thumbnails/SST_BAL_SST_L4_NRT_OBSERVATIONS_010_007_b.jpg" width="300" height="200"/>



# Baltic Sea- Sea Surface Temperature Analysis L4


**Product ID:** SST_BAL_SST_L4_NRT_OBSERVATIONS_010_007_b

**Description:** For the Baltic Sea- The DMI Sea Surface Temperature analysis aims at providing daily gap-free maps of sea surface temperature, referred as L4 product, at 0.02deg. x 0.02deg. horizontal resolution, using satellite data from infra-red and microwave radiometers. Uses SST nighttime satellite products from these sensors: NOAA AVHRR, Metop AVHRR, Terra MODIS, Aqua MODIS, Aqua AMSR-E, Envisat AATSR, MSG Seviri

**DOI (product):**   
https://doi.org/10.48670/moi-00155




<img src="https://mdl-metadata.s3.waw3-1.cloudferro.com/metadata/thumbnails/SST_BAL_SST_L4_REP_OBSERVATIONS_010_016.jpg" width="300" height="200"/>



# Baltic Sea- Sea Surface Temperature Reprocessed


**Product ID:** SST_BAL_SST_L4_REP_OBSERVATIONS_010_016

**Description:** For the Baltic Sea- The DMI Sea Surface Temperature reprocessed analysis provides daily gap-free sea surface temperature fields, referred as L4 product, at 0.02deg. x 0.02deg. horizontal resolution. It is produced by the DMI Optimal Interpolation (DMIOI) system (Høyer and She, 2007) to provide a high resolution (1/50deg. - approx. 2km grid resolution) daily analysis of the daily average sea surface temperature (SST) at 20 cm depth. It uses satellite data from infra-red radiometers, from the ESA SST_cci v3.0 (Embury et al., 2024) and Copernicus C3S projects, namely L2P data from (A)ATSRs, SLSTR and AVHRR for the period 1982-2021, L3U data from SLSTR and AVHRR for 2022-July 19 2024 and L2P data from SLSTR and AVHRR from July 20 2024 onward. For the Sea Ice Concentration it uses the Baltic high resolution sea ice concentration data from the Copernicus Marine Service SI TAC (SEAICE_BAL_PHY_L4_MY_011_019). 

**DOI (product):**   
https://doi.org/10.48670/moi-00156

**References:**

* Høyer, J. L., & Karagali, I. (2016). Sea surface temperature climate data record for the North Sea and Baltic Sea. Journal of Climate, 29(7), 2529-2541.
* Høyer, J. L. and She, J., Optimal interpolation of sea surface temperature for the North Sea and Baltic Sea, J. Mar. Sys., Vol 65, 1-4, pp., 2007.Høyer, J. L. and She, J., Optimal interpolation of sea surface temperature for the North Sea and Baltic Sea, J. Mar. Sys., Vol 65, 1-4, pp., 2007.
* Embury, O., Merchant, C.J., Good, S.A., Rayner, N.A., Høyer, J.L., Atkinson, C., Block, T., Alerskans, E., Pearson, K.J., Worsfold, M., McCarroll, N., Donlon, C. Satellite-based time-series of sea-surface temperature since 1980 for climate applications. Scientific Data 11, 326 (2024). https://doi.org/10.1038/s41597-024-03147-w"





<img src="https://mdl-metadata.s3.waw3-1.cloudferro.com/metadata/thumbnails/SST_BS_PHY_L3S_MY_010_041.jpg" width="300" height="200"/>



# Black Sea - High Resolution L3S Sea Surface Temperature Reprocessed


**Product ID:** SST_BS_PHY_L3S_MY_010_041

**Description:** The Reprocessed (REP) Black Sea (BS) dataset provides a stable and consistent long-term Sea Surface Temperature (SST) time series over the Black Sea developed for climate applications. This product consists of daily (nighttime), merged multi-sensor (L3S), satellite-based estimates of the foundation SST (namely, the temperature free, or nearly-free, of any diurnal cycle) at 0.05° resolution grid covering the period from 1st January 1981 to present (approximately one month before real time). The BS-REP-L3S product is built from a consistent reprocessing of the collated level-3 (merged single-sensor, L3C) climate data record (CDR) v.3.0, provided by the ESA Climate Change Initiative (CCI) and covering the period up to 2021, and its interim extension (ICDR) that allows the regular temporal extension for 2022 onwards. 

**DOI (product):**   
https://doi.org/10.48670/moi-00313

**References:**

* Merchant, C. J., Embury, O., Bulgin, C. E., Block, T., Corlett, G. K., Fiedler, E., ... & Eastwood, S. (2019). Satellite-based time-series of sea-surface temperature since 1981 for climate applications. Scientific data, 6(1), 1-18. Pisano, A., Buongiorno Nardelli, B., Tronconi, C. & Santoleri, R. (2016). The new Mediterranean optimally interpolated pathfinder AVHRR SST Dataset (1982–2012). Remote Sens. Environ. 176, 107–116.
* Saha, Korak; Zhao, Xuepeng; Zhang, Huai-min; Casey, Kenneth S.; Zhang, Dexin; Baker-Yeboah, Sheekela; Kilpatrick, Katherine A.; Evans, Robert H.; Ryan, Thomas; Relph, John M. (2018). AVHRR Pathfinder version 5.3 level 3 collated (L3C) global 4km sea surface temperature for 1981-Present. NOAA National Centers for Environmental Information. Dataset. https://doi.org/10.7289/v52j68xx





<img src="https://mdl-metadata.s3.waw3-1.cloudferro.com/metadata/thumbnails/SST_BS_PHY_SUBSKIN_L4_NRT_010_035.jpg" width="300" height="200"/>



# Black Sea - High Resolution Diurnal Subskin Sea Surface Temperature Analysis


**Product ID:** SST_BS_PHY_SUBSKIN_L4_NRT_010_035

**Description:** For the Black Sea - the CNR diurnal sub-skin Sea Surface Temperature product provides daily gap-free (L4) maps of hourly mean sub-skin SST at 1/16° (0.0625°) horizontal resolution over the CMEMS Black Sea (BS) domain, by combining infrared satellite and model data (Marullo et al., 2014). The implementation of this product takes advantage of the consolidated operational SST processing chains that provide daily mean SST fields over the same basin (Buongiorno Nardelli et al., 2013). The sub-skin temperature is the temperature at the base of the thermal skin layer and it is equivalent to the foundation SST at night, but during daytime it can be significantly different under favorable (clear sky and low wind) diurnal warming conditions. The sub-skin SST L4 product is created by combining geostationary satellite observations aquired from SEVIRI and model data (used as first-guess) aquired from the CMEMS BS Monitoring Forecasting Center (MFC). This approach takes advantage of geostationary satellite observations as the input signal source to produce hourly gap-free SST fields using model analyses as first-guess. The resulting SST anomaly field (satellite-model) is free, or nearly free, of any diurnal cycle, thus allowing to interpolate SST anomalies using satellite data acquired at different times of the day (Marullo et al., 2014).

**DOI (product):**   
https://doi.org/10.48670/moi-00157

**References:**

* Marullo, S., Santoleri, R., Ciani, D., Le Borgne, P., Péré, S., Pinardi, N., ... & Nardone, G. (2014). Combining model and geostationary satellite data to reconstruct hourly SST field over the Mediterranean Sea. Remote sensing of environment, 146, 11-23.
* Buongiorno Nardelli B., C.Tronconi, A. Pisano, R.Santoleri, 2013: High and Ultra-High resolution processing of satellite Sea Surface Temperature data over Southern European Seas in the framework of MyOcean project, Rem. Sens. Env., 129, 1-16, doi:10.1016/j.rse.2012.10.012.





<img src="https://mdl-metadata.s3.waw3-1.cloudferro.com/metadata/thumbnails/SST_BS_SST_L3S_NRT_OBSERVATIONS_010_013.jpg" width="300" height="200"/>



# Black Sea - High Resolution and Ultra High Resolution L3S Sea Surface Temperature


**Product ID:** SST_BS_SST_L3S_NRT_OBSERVATIONS_010_013

**Description:** For the Black Sea (BS), the CNR BS Sea Surface Temperature (SST) processing chain provides supercollated (merged multisensor, L3S) SST data remapped over the Black Sea at high (1/16°)  and Ultra High (0.01°) spatial resolution, representative of nighttime SST values (00:00 UTC). The L3S SST data are produced selecting only the highest quality input data from input L2P images within a strict temporal window (local nightime), to avoid diurnal cycle and cloud contamination. The main L2P data currently used include SLSTR-3A/3B, VIIRS-N20/NPP, Metop-B/C AVHRR and SEVIRI. Consequently, the L3S processing is run daily, but L3S files are produced only if valid SST measurements are present on the area considered. 

**DOI (product):**   
https://doi.org/10.48670/moi-00158

**References:**

* Buongiorno Nardelli B., C.Tronconi, A. Pisano, R.Santoleri, 2013: High and Ultra-High resolution processing of satellite Sea Surface Temperature data over Southern European Seas in the framework of MyOcean project, Rem. Sens. Env., 129, 1-16, doi:10.1016/j.rse.2012.10.012.





<img src="https://mdl-metadata.s3.waw3-1.cloudferro.com/metadata/thumbnails/SST_BS_SST_L4_NRT_OBSERVATIONS_010_006.jpg" width="300" height="200"/>



# Black Sea High Resolution and Ultra High Resolution Sea Surface Temperature Analysis


**Product ID:** SST_BS_SST_L4_NRT_OBSERVATIONS_010_006

**Description:** For the Black Sea (BS), the CNR BS Sea Surface Temperature (SST) processing chain providess daily gap-free (L4) maps at high (HR 0.0625°) and ultra-high (UHR 0.01°) spatial resolution over the Black Sea. Remotely-sensed L4 SST datasets are operationally produced and distributed in near-real time by the Consiglio Nazionale delle Ricerche - Gruppo di Oceanografia da Satellite (CNR-GOS). These SST products are based on the nighttime images collected by the infrared sensors mounted on different satellite platforms, and cover the Southern European Seas. The main upstream data currently used include SLSTR-3A/3B, VIIRS-N20/NPP, Metop-B/C AVHRR and SEVIRI. The CNR-GOS processing chain includes several modules, from the data extraction and preliminary quality control, to cloudy pixel removal and satellite images collating/merging. A two-step algorithm finally allows to interpolate SST data at high (HR 0.0625°) and ultra-high (UHR 0.01°) spatial resolution, applying statistical techniques. These L4 data are also used to estimate the SST anomaly with respect to a pentad climatology. The basic design and the main algorithms used are described in the following papers.

**DOI (product):**  
https://doi.org/10.48670/moi-00159

**References:**

* Buongiorno Nardelli B., S. Colella, R. Santoleri, M. Guarracino, A. Kholod, 2009: A re-analysis of Black Sea Surface Temperature, J. Mar. Sys.., doi:10.1016/j.jmarsys.2009.07.001
* Buongiorno Nardelli B., C.Tronconi, A. Pisano, R.Santoleri, 2013: High and Ultra-High resolution processing of satellite Sea Surface Temperature data over Southern European Seas in the framework of MyOcean project, Rem. Sens. Env., 129, 1-16, doi:10.1016/j.rse.2012.10.012.





<img src="https://mdl-metadata.s3.waw3-1.cloudferro.com/metadata/thumbnails/SST_BS_SST_L4_REP_OBSERVATIONS_010_022.jpg" width="300" height="200"/>



# Black Sea - High Resolution L4 Sea Surface Temperature Reprocessed


**Product ID:** SST_BS_SST_L4_REP_OBSERVATIONS_010_022

**Description:** The Reprocessed (REP) Black Sea (BS) dataset provides a stable and consistent long-term Sea Surface Temperature (SST) time series over the Black Sea developed for climate applications. This product consists of daily (nighttime), optimally interpolated (L4), satellite-based estimates of the foundation SST (namely, the temperature free, or nearly-free, of any diurnal cycle) at 0.05° resolution grid covering the period from 1st January 1981 to present (approximately one month before real time). The BS-REP-L4 product is built from a consistent reprocessing of the collated level-3 (merged single-sensor, L3C) climate data record (CDR) v.3.0, provided by the ESA Climate Change Initiative (CCI) and covering the period up to 2021, and its interim extension (ICDR) that allows the regular temporal extension for 2022 onwards. 

**DOI (product):**   
https://doi.org/10.48670/moi-00160

**References:**

* Pisano, A., Nardelli, B. B., Tronconi, C., & Santoleri, R. (2016). The new Mediterranean optimally interpolated pathfinder AVHRR SST Dataset (1982–2012). Remote Sensing of Environment, 176, 107-116. doi: https://doi.org/10.1016/j.rse.2016.01.019
* Embury, O., Merchant, C.J., Good, S.A., Rayner, N.A., Høyer, J.L., Atkinson, C., Block, T., Alerskans, E., Pearson, K.J., Worsfold, M., McCarroll, N., Donlon, C., (2024). Satellite-based time-series of sea-surface temperature since 1980 for climate applications. Sci Data 11, 326. doi: https://doi.org/10.1038/s41597-024-03147-w





<img src="https://mdl-metadata.s3.waw3-1.cloudferro.com/metadata/thumbnails/SST_GLO_PHY_L3S_MY_010_039.jpg" width="300" height="200"/>



# Global High Resolution ODYSSEA Sea Surface Temperature Multi-sensor L3 Observations


**Product ID:** SST_GLO_PHY_L3S_MY_010_039

**Description:** For the Global Ocean- Sea Surface Temperature L3 Observations . This product provides daily foundation sea surface temperature from multiple satellite sources. The data are intercalibrated. This product consists in a fusion of sea surface temperature observations from multiple satellite sensors, daily, over a 0.05° resolution grid. It includes observations by polar orbiting from the ESA CCI / C3S archive .  The L3S SST data are produced selecting only the highest quality input data from input L2P/L3P images within a strict temporal window (local nightime), to avoid diurnal cycle and cloud contamination. The observations of each sensor are intercalibrated prior to merging using a bias correction based on a multi-sensor median reference correcting the large-scale cross-sensor biases. 


**DOI (product):**   
https://doi.org/10.48670/mds-00329




<img src="https://mdl-metadata.s3.waw3-1.cloudferro.com/metadata/thumbnails/SST_GLO_PHY_L4_MY_010_044.jpg" width="300" height="200"/>



# Global Ocean ODYSSEA L4 Sea Surface Temperature


**Product ID:** SST_GLO_PHY_L4_MY_010_044

**Description:** For the global ocean. The IFREMER/ODYSSEA Sea Surface Temperature reprocessed analysis aims at providing daily gap-free maps of sea surface temperature, referred as L4 product, at 0.10deg. x 0.10deg. horizontal resolution, over the 1982-present period, using satellite data from the European Space Agency Sea Surface Temperature Climate Change Initiative (ESA SST CCI) L3 products (1982-2016) and from the Copernicus Climate Change Service (C3S) L3 product (2017-present). The gridded SST product is intended to represent a daily-mean SST field at 20 cm depth.

**DOI (product):**   
https://doi.org/10.48670/mds-00345




<img src="https://mdl-metadata.s3.waw3-1.cloudferro.com/metadata/thumbnails/SST_GLO_PHY_L4_NRT_010_043.jpg" width="300" height="200"/>



# ODYSSEA Global Sea Surface Temperature Gridded Level 4 Daily Multi-Sensor Observations


**Product ID:** SST_GLO_PHY_L4_NRT_010_043

**Description:** This dataset provide a times series of gap free map of Sea Surface Temperature (SST) foundation at high resolution on a 0.10 x 0.10 degree grid (approximately 10 x 10 km) for the Global Ocean, every 24 hours.

Whereas along swath observation data essentially represent the skin or sub-skin SST, the Level 4 SST product is defined to represent the SST foundation (SSTfnd). SSTfnd is defined within GHRSST as the temperature at the base of the diurnal thermocline. It is so named because it represents the foundation temperature on which the diurnal thermocline develops during the day. SSTfnd changes only gradually along with the upper layer of the ocean, and by definition it is independent of skin SST fluctuations due to wind- and radiation-dependent diurnal stratification or skin layer response. It is therefore updated at intervals of 24 hrs. SSTfnd corresponds to the temperature of the upper mixed layer which is the part of the ocean represented by the top-most layer of grid cells in most numerical ocean models. It is never observed directly by satellites, but it comes closest to being detected by infrared and microwave radiometers during the night, when the previous day's diurnal stratification can be assumed to have decayed.

The processing combines the observations of multiple polar orbiting and geostationary satellites, embedding infrared of microwave radiometers. All these sources are intercalibrated with each other before merging. A ranking procedure is used to select the best sensor observation for each grid point. An optimal interpolation is used to fill in where observations are missing.

**DOI (product):**   
https://doi.org/10.48670/mds-00321




<img src="https://mdl-metadata.s3.waw3-1.cloudferro.com/metadata/thumbnails/SST_GLO_SST_L3S_NRT_OBSERVATIONS_010_010.jpg" width="300" height="200"/>



# ODYSSEA Global Ocean - Sea Surface Temperature Multi-sensor L3 Observations


**Product ID:** SST_GLO_SST_L3S_NRT_OBSERVATIONS_010_010

**Description:** For the Global Ocean- Sea Surface Temperature L3 Observations . This product provides daily foundation sea surface temperature from multiple satellite sources. The data are intercalibrated. This product consists in a fusion of sea surface temperature observations from multiple satellite sensors, daily, over a 0.1° resolution global grid. It includes observations by polar orbiting (NOAA-18 & NOAAA-19/AVHRR, METOP-A/AVHRR, ENVISAT/AATSR, AQUA/AMSRE, TRMM/TMI) and geostationary (MSG/SEVIRI, GOES-11) satellites . The observations of each sensor are intercalibrated prior to merging using a bias correction based on a multi-sensor median reference correcting the large-scale cross-sensor biases.3 more datasets are available that only contain "per sensor type" data: Polar InfraRed (PIR), Polar MicroWave (PMW), Geostationary InfraRed (GIR)

**DOI (product):**   
https://doi.org/10.48670/moi-00164




<img src="https://mdl-metadata.s3.waw3-1.cloudferro.com/metadata/thumbnails/SST_GLO_SST_L4_NRT_OBSERVATIONS_010_001.jpg" width="300" height="200"/>



# Global Ocean OSTIA Sea Surface Temperature and Sea Ice Analysis


**Product ID:** SST_GLO_SST_L4_NRT_OBSERVATIONS_010_001

**Description:** For the Global Ocean- the OSTIA global foundation Sea Surface Temperature product provides daily gap-free maps of: Foundation Sea Surface Temperature at 0.05° x 0.05° horizontal grid resolution, using in-situ and satellite data from both infrared and microwave radiometers. 

The Operational Sea Surface Temperature and Ice Analysis (OSTIA) system is run by the UK's Met Office and delivered by IFREMER PU. OSTIA uses satellite data provided by the GHRSST project together with in-situ observations to determine the sea surface temperature.
A high resolution (1/20° - approx. 6 km) daily analysis of sea surface temperature (SST) is produced for the global ocean and some lakes.  

**DOI (product):**   
https://doi.org/10.48670/moi-00165

**References:**

* Good, S.; Fiedler, E.; Mao, C.; Martin, M.J.; Maycock, A.; Reid, R.; Roberts-Jones, J.; Searle, T.; Waters, J.; While, J.; Worsfold, M. The Current Configuration of the OSTIA System for Operational Production of Foundation Sea Surface Temperature and Ice Concentration Analyses. Remote Sens. 2020, 12, 720. doi: 10.3390/rs12040720
* Donlon, C.J., Martin, M., Stark, J., Roberts-Jones, J., Fiedler, E., and Wimmer, W., 2012, The Operational Sea Surface Temperature and Sea Ice Analysis (OSTIA) system. Remote Sensing of the Environment. doi: 10.1016/j.rse.2010.10.017 2011.
* John D. Stark, Craig J. Donlon, Matthew J. Martin and Michael E. McCulloch, 2007, OSTIA : An operational, high resolution, real time, global sea surface temperature analysis system., Oceans 07 IEEE Aberdeen, conference proceedings. Marine challenges: coastline to deep sea. Aberdeen, Scotland.IEEE.





<img src="https://mdl-metadata.s3.waw3-1.cloudferro.com/metadata/thumbnails/SST_GLO_SST_L4_REP_OBSERVATIONS_010_011.jpg" width="300" height="200"/>



# Global Ocean OSTIA Sea Surface Temperature and Sea Ice Reprocessed


**Product ID:** SST_GLO_SST_L4_REP_OBSERVATIONS_010_011

**Description:** The OSTIA (Good et al., 2020) global sea surface temperature reprocessed product provides daily gap-free maps of foundation sea surface temperature and ice concentration (referred to as an L4 product) at 0.05deg.x 0.05deg. horizontal grid resolution, using in-situ and satellite data. This product provides the foundation Sea Surface Temperature, which is the temperature free of diurnal variability.

**DOI (product):**   
https://doi.org/10.48670/moi-00168

**References:**

* Worsfold, M.; Good, S.; Atkinson, C.; Embury, O. Presenting a Long-Term, Reprocessed Dataset of Global Sea Surface Temperature Produced Using the OSTIA System. Remote Sens. 2024, 16, 3358.  https://doi.org/10.3390/rs16183358





<img src="https://mdl-metadata.s3.waw3-1.cloudferro.com/metadata/thumbnails/SST_GLO_SST_L4_REP_OBSERVATIONS_010_024.jpg" width="300" height="200"/>



# ESA SST CCI and C3S reprocessed sea surface temperature analyses


**Product ID:** SST_GLO_SST_L4_REP_OBSERVATIONS_010_024

**Description:** The C3S global Sea Surface and Sea Ice Temperature Reprocessed product provides gap-free maps of daily average SST at 20 cm depth and IST skin at 0.05deg. x 0.05deg. horizontal grid resolution, using satellite data from the ESA SST_cci v3.0 L3U data from (A)ATSRs, SLSTR and AVHRR, L2P data from the AMSRE and AMSR2 Passive Microwave Instruments (Embury et al., 2024) and L2P data from the AASTI and C3S IST CDR/ICDR v.1. The C3S level 4 SST/IST analyses were produced by running the DMI Optimal Interpolation (DMIOI) system (Høyer and She, 2007; Høyer et al., 2014; Nielsen-Englyst et al., 2023, Nielsen-Englyst et al., 2024) to provide a high resolution (1/20deg. - approx. 5km grid resolution) daily analysis of the daily average sea surface temperature (SST) at 20 cm depth and sea ice surface temperature (IST) at the surface skin to cover surface temperatures in the global ocean, the sea ice and the marginal ice zone. It uses a Multi-Source Composite Sea-Ice concentration dataset (from a combination of EUMETSAT OSI-SAF OSI-450a (Lavergne et al., 2019), OSI-458, ESA CCI Sea ice CDR, SICCI-HR-SIC, U.S. National Ice Centre’s (NIC) ice charts, Swedish Meteorological and Hydrological Institute (SHMI) and Finnish Meteorological Institute’s (FMI) ice charts used for the Baltic region) developed at DMI for the purpose of the CARRA2 project (Pan-Arctic) and extended to the South Hemisphere. 

**DOI (product):**   
https://doi.org/10.48670/moi-00169

**References:**

* Høyer, J. L. and She, J., Optimal interpolation of sea surface temperature for the North Sea and Baltic Sea, J. Mar. Sys., Vol 65, 1-4, pp., 2007.Høyer, J. L. and She, J., Optimal interpolation of sea surface temperature for the North Sea and Baltic Sea, J. Mar. Sys., Vol 65, 1-4, pp., 2007.
* Høyer, J. L., Le Borgne, P., & Eastwood, S. (2014) A bias correction method for Arctic satellite sea surface temperature observations. Remote Sensing of Environment, 146, 201-213.
* Nielsen-Englyst, P., Høyer, J. L., Kolbe, W. M., Dybkjar, G., Lavergne, T., Tonboe, R. T., Skarpalezos, S., Karagali, I. (2023) A combined sea and sea-ice surface temperature climate dataset of the Arctic, 1982–2021. Remote Sensing of Environment, 284, 113331, doi: https://doi.org/10.1016/j.rse.2022.113331.
* Nielsen-Englyst, P., Høyer, J. L., Karagali, I., Kolbe, W. M., Tonboe, R. T., Pedersen, L. T. (2024) Impact of passive microwave observations on the estimation of Arctic sea surface temperatures. Remote Sensing of Environment, 301, 113949, doi: https://doi.org/10.1016/j.rse.2023.113949.
* Embury, O., Merchant, C.J., Good, S.A., Rayner, N.A., Høyer, J.L., Atkinson, C., Block, T., Alerskans, E., Pearson, K.J., Worsfold, M., McCarroll, N., Donlon, C. Satellite-based time-series of sea-surface temperature since 1980 for climate applications. Scientific Data 11, 326 (2024). https://doi.org/10.1038/s41597-024-03147-w
* Lavergne, T., Sørensen, A. M., Kern, S., Tonboe, R., Notz, D., Aaboe, S., Bell, L., Dybkjær, G., Eastwood, S., Gabarro, C., Heygster, G., Killie, M. A., Brandt Kreiner, M., Lavelle, J., Saldo, R., Sandven, S., and Pedersen, L. T.: Version 2 of the EUMETSAT OSI SAF and ESA CCI sea-ice concentration climate data records, The Cryosphere, 13, 49-78, doi:10.5194/tc-13-49-2019, 2019. "





<img src="https://mdl-metadata.s3.waw3-1.cloudferro.com/metadata/thumbnails/SST_MED_PHY_L3S_MY_010_042.jpg" width="300" height="200"/>



# Mediterranean Sea - High Resolution L3S Sea Surface Temperature Reprocessed


**Product ID:** SST_MED_PHY_L3S_MY_010_042

**Description:** The Reprocessed (REP) Mediterranean (MED) dataset provides a stable and consistent long-term Sea Surface Temperature (SST) time series over the Mediterranean Sea (and the adjacent North Atlantic box) developed for climate applications. This product consists of daily (nighttime), merged multi-sensor (L3S), satellite-based estimates of the foundation SST (namely, the temperature free, or nearly-free, of any diurnal cycle) at 0.05° resolution grid covering the period from 1st January 1981 to present (approximately one month before real time). The MED-REP-L3S product is built from a consistent reprocessing of the collated level-3 (merged single-sensor, L3C) climate data record (CDR) v.3.0, provided by the ESA Climate Change Initiative (CCI) and covering the period up to 2021, and its interim extension (ICDR) that allows the regular temporal extension for 2022 onwards. 

**DOI (product):**   
https://doi.org/10.48670/moi-00314

**References:**

* Pisano, A., Nardelli, B. B., Tronconi, C., & Santoleri, R. (2016). The new Mediterranean optimally interpolated pathfinder AVHRR SST Dataset (1982–2012). Remote Sensing of Environment, 176, 107-116. doi: https://doi.org/10.1016/j.rse.2016.01.019
* Embury, O., Merchant, C.J., Good, S.A., Rayner, N.A., Høyer, J.L., Atkinson, C., Block, T., Alerskans, E., Pearson, K.J., Worsfold, M., McCarroll, N., Donlon, C., (2024). Satellite-based time-series of sea-surface temperature since 1980 for climate applications. Sci Data 11, 326. doi: https://doi.org/10.1038/s41597-024-03147-w





<img src="https://mdl-metadata.s3.waw3-1.cloudferro.com/metadata/thumbnails/SST_MED_PHY_SUBSKIN_L4_NRT_010_036.jpg" width="300" height="200"/>



# Mediterranean Sea - High Resolution Diurnal Subskin Sea Surface Temperature Analysis


**Product ID:** SST_MED_PHY_SUBSKIN_L4_NRT_010_036

**Description:** For the Mediterranean Sea - the CNR diurnal sub-skin Sea Surface Temperature (SST) product provides daily gap-free (L4) maps of hourly mean sub-skin SST at 1/16° (0.0625°) horizontal resolution over the CMEMS Mediterranean Sea (MED) domain, by combining infrared satellite and model data (Marullo et al., 2014). The implementation of this product takes advantage of the consolidated operational SST processing chains that provide daily mean SST fields over the same basin (Buongiorno Nardelli et al., 2013). The sub-skin temperature is the temperature at the base of the thermal skin layer and it is equivalent to the foundation SST at night, but during daytime it can be significantly different under favorable (clear sky and low wind) diurnal warming conditions. The sub-skin SST L4 product is created by combining geostationary satellite observations aquired from SEVIRI and model data (used as first-guess) aquired from the CMEMS MED Monitoring Forecasting Center (MFC). This approach takes advantage of geostationary satellite observations as the input signal source to produce hourly gap-free SST fields using model analyses as first-guess. The resulting SST anomaly field (satellite-model) is free, or nearly free, of any diurnal cycle, thus allowing to interpolate SST anomalies using satellite data acquired at different times of the day (Marullo et al., 2014).
 
[How to cite](https://help.marine.copernicus.eu/en/articles/4444611-how-to-cite-or-reference-copernicus-marine-products-and-services)

**DOI (product):**   
https://doi.org/10.48670/moi-00170

**References:**

* Marullo, S., Santoleri, R., Ciani, D., Le Borgne, P., Péré, S., Pinardi, N., ... & Nardone, G. (2014). Combining model and geostationary satellite data to reconstruct hourly SST field over the Mediterranean Sea. Remote sensing of environment, 146, 11-23.
* Buongiorno Nardelli B., C.Tronconi, A. Pisano, R.Santoleri, 2013: High and Ultra-High resolution processing of satellite Sea Surface Temperature data over Southern European Seas in the framework of MyOcean project, Rem. Sens. Env., 129, 1-16, doi:10.1016/j.rse.2012.10.012.





<img src="https://mdl-metadata.s3.waw3-1.cloudferro.com/metadata/thumbnails/SST_MED_SST_L3S_NRT_OBSERVATIONS_010_012.jpg" width="300" height="200"/>



# Mediterranean Sea - High Resolution and Ultra High Resolution L3S Sea Surface Temperature


**Product ID:** SST_MED_SST_L3S_NRT_OBSERVATIONS_010_012

**Description:** For the Mediterranean Sea (MED), the CNR MED Sea Surface Temperature (SST) processing chain provides supercollated (merged multisensor, L3S) SST data remapped over the Mediterranean Sea at high (1/16°)  and Ultra High (0.01°) spatial resolution, representative of nighttime SST values (00:00 UTC). The L3S SST data are produced selecting only the highest quality input data from input L2P images within a strict temporal window (local nightime), to avoid diurnal cycle and cloud contamination. The main L2P data currently used include SLSTR-3A/3B, VIIRS-N20/NPP, Metop-B/C AVHRR and SEVIRI. Consequently, the L3S processing is run daily, but L3S files are produced only if valid SST measurements are present on the area considered. 

**DOI (product):**   
https://doi.org/10.48670/moi-00171

**References:**

* Buongiorno Nardelli B., C.Tronconi, A. Pisano, R.Santoleri, 2013: High and Ultra-High resolution processing of satellite Sea Surface Temperature data over Southern European Seas in the framework of MyOcean project, Rem. Sens. Env., 129, 1-16, doi:10.1016/j.rse.2012.10.012.





<img src="https://mdl-metadata.s3.waw3-1.cloudferro.com/metadata/thumbnails/SST_MED_SST_L4_NRT_OBSERVATIONS_010_004.jpg" width="300" height="200"/>



# Mediterranean Sea High Resolution and Ultra High Resolution Sea Surface Temperature Analysis


**Product ID:** SST_MED_SST_L4_NRT_OBSERVATIONS_010_004

**Description:** For the Mediterranean Sea (MED), the CNR MED Sea Surface Temperature (SST) processing chain provides daily gap-free (L4) maps at high (HR 0.0625°) and ultra-high (UHR 0.01°) spatial resolution over the Mediterranean Sea. Remotely-sensed L4 SST datasets are operationally produced and distributed in near-real time by the Consiglio Nazionale delle Ricerche - Gruppo di Oceanografia da Satellite (CNR-GOS). These SST products are based on the nighttime images collected by the infrared sensors mounted on different satellite platforms, and cover the Southern European Seas. The main upstream data currently used include SLSTR-3A/3B, VIIRS-N20/NPP, Metop-B/C AVHRR and SEVIRI. The CNR-GOS processing chain includes several modules, from the data extraction and preliminary quality control, to cloudy pixel removal and satellite images collating/merging. A two-step algorithm finally allows to interpolate SST data at high (HR 0.0625°) and ultra-high (UHR 0.01°) spatial resolution, applying statistical techniques. Since November 2024, the L4 MED UHR processing chain makes use of an improved background field as initial guess for the Optimal Interpolation of this product. The improvement is obtained in terms of the effective spatial resolution via the application of a convolutional neural network (CNN). These L4 data are also used to estimate the SST anomaly with respect to a pentad climatology. The basic design and the main algorithms used are described in the following papers. 

**DOI (product):**   
https://doi.org/10.48670/moi-00172

**References:**

* Buongiorno Nardelli B., C.Tronconi, A. Pisano, R.Santoleri, 2013: High and Ultra-High resolution processing of satellite Sea Surface Temperature data over Southern European Seas in the framework of MyOcean project, Rem. Sens. Env., 129, 1-16, doi:10.1016/j.rse.2012.10.012.
* Fanelli, C., Ciani, D., Pisano, A., & Buongiorno Nardelli, B. (2024). Deep Learning for Super-Resolution of Mediterranean Sea Surface Temperature Fields. EGUsphere, 2024, 1-18 (pre-print)





<img src="https://mdl-metadata.s3.waw3-1.cloudferro.com/metadata/thumbnails/SST_MED_SST_L4_REP_OBSERVATIONS_010_021.jpg" width="300" height="200"/>



# Mediterranean Sea - High Resolution L4 Sea Surface Temperature Reprocessed


**Product ID:** SST_MED_SST_L4_REP_OBSERVATIONS_010_021

**Description:** The Reprocessed (REP) Mediterranean (MED) dataset provides a stable and consistent long-term Sea Surface Temperature (SST) time series over the Mediterranean Sea (and the adjacent North Atlantic box) developed for climate applications. This product consists of daily (nighttime), optimally interpolated (L4), satellite-based estimates of the foundation SST (namely, the temperature free, or nearly-free, of any diurnal cycle) at 0.05° resolution grid covering the period from 1st January 1981 to present (approximately one month before real time). The MED-REP-L4 product is built from a consistent reprocessing of the collated level-3 (merged single-sensor, L3C) climate data record (CDR) v.3.0, provided by the ESA Climate Change Initiative (CCI) and covering the period up to 2021, and its interim extension (ICDR) that allows the regular temporal extension for 2022 onwards. 

**DOI (product):**   
https://doi.org/10.48670/moi-00173

**References:**

* Pisano, A., Nardelli, B. B., Tronconi, C., & Santoleri, R. (2016). The new Mediterranean optimally interpolated pathfinder AVHRR SST Dataset (1982–2012). Remote Sensing of Environment, 176, 107-116. doi: https://doi.org/10.1016/j.rse.2016.01.019
* Embury, O., Merchant, C.J., Good, S.A., Rayner, N.A., Høyer, J.L., Atkinson, C., Block, T., Alerskans, E., Pearson, K.J., Worsfold, M., McCarroll, N., Donlon, C., (2024). Satellite-based time-series of sea-surface temperature since 1980 for climate applications. Sci Data 11, 326. doi: https://doi.org/10.1038/s41597-024-03147-w





<img src="https://mdl-metadata.s3.waw3-1.cloudferro.com/metadata/thumbnails/WAVE_GLO_WAV_L3_SPC_NRT_OBSERVATIONS_014_002.jpg" width="300" height="200"/>



# GLOBAL OCEAN L3 SPECTRAL PARAMETERS FROM NRT SATELLITE MEASUREMENTS


**Product ID:** WAVE_GLO_PHY_SPC-FWK_L3_NRT_014_002

**Description:** Near-Real-Time mono-mission satellite-based integral parameters derived from the directional wave spectra.
Using linear propagation wave model, only wave observations that can be back-propagated to wave converging regions are considered.
The dataset parameters includes partition significant wave height, partition peak period and partition peak or principal direction given along swell propagation path in space and time at a 3-hour timestep, from source to land. Validity flags are also included for each parameter and indicates the valid time steps along propagation (eg. no propagation for significant wave height close to the storm source or any integral parameter when reaching the land).
The integral parameters at observation point are also available together with a quality flag based on the consistency between each propagated observation and the overall swell field.
This product is processed by the WAVE-TAC multi-mission SAR data processing system.
It processes near-real-time data from the following missions: SAR (Sentinel-1A and Sentinel-1B) and CFOSAT/SWIM.
One file is produced for each mission and is available in two formats depending on the user needs: one gathering in one netcdf file all observations related to the same swell field, and for another all observations available in a 3-hour time range, and for both formats, propagated information from source to land.

**DOI (product):**   
https://doi.org/10.48670/moi-00178




<img src="https://mdl-metadata.s3.waw3-1.cloudferro.com/metadata/thumbnails/WAVE_GLO_PHY_SPC_L3_MY_014_006.jpg" width="300" height="200"/>



# GLOBAL OCEAN L3 SPECTRAL PARAMETERS FROM REPROCESSED SATELLITE MEASUREMENTS


**Product ID:** WAVE_GLO_PHY_SPC_L3_MY_014_006

**Description:** Multi-Year mono-mission satellite-based integral parameters derived from the directional wave spectra. Using linear propagation wave model, only wave observations that can be back-propagated to wave converging regions are considered. The dataset parameters includes partition significant wave height, partition peak period and partition peak or principal direction given along swell propagation path in space and time at a 3-hour timestep, from source to land. Validity flags are also included for each parameter and indicates the valid time steps along propagation (eg. no propagation for significant wave height close to the storm source or any integral parameter when reaching the land). The integral parameters at observation point are also available together with a quality flag based on the consistency between each propagated observation and the overall swell field.This product is processed by the WAVE-TAC multi-mission SAR data processing system. It processes data from the following SAR missions: Sentinel-1A and Sentinel-1B.One file is produced for each mission and is available in two formats: one gathering in one netcdf file all observations related to the same swell field, and for another all observations available in a 3-hour time range, and for both formats, propagated information from source to land.

**DOI (product):**   
https://doi.org/10.48670/moi-00174




<img src="https://mdl-metadata.s3.waw3-1.cloudferro.com/metadata/thumbnails/WAVE_GLO_PHY_SPC_L3_NRT_014_009.jpg" width="300" height="200"/>



# GLOBAL OCEAN L3 SPECTRAL PARAMETERS FROM NRT SATELLITE MEASUREMENTS


**Product ID:** WAVE_GLO_PHY_SPC_L3_NRT_014_009

**Description:** Near Real-Time mono-mission satellite-based 2D full wave spectral product. These very complete products enable to characterise spectrally the direction, wave length and multiple sea Sates along CFOSAT track (in boxes of 70km/90km left and right from the nadir pointing). The data format are 2D directionnal matrices. They also include integrated parameters (Hs, direction, wavelength) from the spectrum with and without partitions. 

**DOI (product):**   
N/A




<img src="https://mdl-metadata.s3.waw3-1.cloudferro.com/metadata/thumbnails/WAVE_GLO_PHY_SPC_L4_NRT_014_004.jpg" width="300" height="200"/>



# GLOBAL OCEAN L4 SPECTRAL PARAMETERS FROM NRT SATELLITE MEASUREMENTS


**Product ID:** WAVE_GLO_PHY_SPC_L4_NRT_014_004

**Description:** Near-Real-Time multi-mission global satellite-based spectral integral parameters. Only valid data are used, based on the L3 corresponding product. Included wave parameters are partition significant wave height, partition peak period and partition peak or principal direction. Those parameters are propagated in space and time at a 3-hour timestep and on a regular space grid, providing information of the swell propagation characteristics, from source to land. One file gathers one swell system, gathering observations originating from the same storm source. This product is processed by the WAVE-TAC multi-mission SAR data processing system to serve in near-real time the main operational oceanography and climate forecasting centers in Europe and worldwide. It processes data from the following SAR missions: Sentinel-1A and Sentinel-1B. All the spectral parameter measurements are optimally interpolated using swell observations belonging to the same swell field. The SAR data processing system produces wave integral parameters by partition (partition significant wave height, partition peak period and partition peak or principal direction) and the associated standard deviation and density of propagated observations. 

**DOI (product):**   
https://doi.org/10.48670/moi-00175




<img src="https://mdl-metadata.s3.waw3-1.cloudferro.com/metadata/thumbnails/WAVE_GLO_PHY_SWH_L3_MY_014_005.jpg" width="300" height="200"/>



# GLOBAL OCEAN L3 SIGNIFICANT WAVE HEIGHT FROM REPROCESSED SATELLITE MEASUREMENTS


**Product ID:** WAVE_GLO_PHY_SWH_L3_MY_014_005

**Description:** Multi-Year mono-mission satellite-based along-track significant wave height. Only valid data are included, based on a rigorous editing combining various criteria such as quality flags (surface flag, presence of ice) and thresholds on parameter values. Such thresholds are applied on parameters linked to significant wave height determination from retracking (e.g. SWH, sigma0, range, off nadir angle…). All the missions are homogenized with respect to a reference mission and in-situ buoy measurements. Finally, an along-track filter is applied to reduce the measurement noise.

This product is based on the ESA Sea State Climate Change Initiative data Level 3 product (version 2) and is formatted by the WAVE-TAC to be homogeneous with the CMEMS Level 3 Near-real-time product. It is based on the reprocessing of GDR data from the following altimeter missions: Jason-1, Jason-2, Envisat, Cryosat-2, SARAL/AltiKa and Jason-3. CFOSAT Multi-Year dataset is based on the reprocessing of CFOSAT Level-2P products (CNES/CLS), inter-calibrated on Jason-3 reference mission issued from the CCI Sea State dataset.

One file containing valid SWH is produced for each mission and for a 3-hour time window. It contains the filtered SWH (VAVH) and the unfiltered SWH (VAVH_UNFILTERED).

**DOI (product):**   
https://doi.org/10.48670/moi-00176




<img src="https://mdl-metadata.s3.waw3-1.cloudferro.com/metadata/thumbnails/WAVE_GLO_PHY_SWH_L3_NRT_014_001.jpg" width="300" height="200"/>



# GLOBAL OCEAN L3 SIGNIFICANT WAVE HEIGHT FROM NRT SATELLITE MEASUREMENTS


**Product ID:** WAVE_GLO_PHY_SWH_L3_NRT_014_001

**Description:** Near-Real-Time mono-mission satellite-based along-track significant wave height. Only valid data are included, based on a rigorous editing combining various criteria such as quality flags (surface flag, presence of ice) and thresholds on parameter values. Such thresholds are applied on parameters linked to significant wave height determination from retracking (e.g. SWH, sigma0, range, off nadir angle…). All the missions are homogenized with respect to a reference mission (Jason-3 until April 2022, Sentinel-6A afterwards) and calibrated on in-situ buoy measurements. Finally, an along-track filter is applied to reduce the measurement noise.

As a support of information to the significant wave height, wind speed measured by the altimeters is also processed and included in the files. Wind speed values are provided by upstream products (L2) for each mission and are based on different algorithms. Only valid data are included and all the missions are homogenized with respect to the reference mission.

This product is processed by the WAVE-TAC multi-mission altimeter data processing system. It serves in near-real time the main operational oceanography and climate forecasting centers in Europe and worldwide. It processes operational data (OGDR and NRT, produced in near-real-time) from the following altimeter missions: Sentinel-6A, Jason-3, Sentinel-3A, Sentinel-3B, Cryosat-2, SARAL/AltiKa, CFOSAT ; and interim data (IGDR, 1 to 2 days delay) from Hai Yang-2B mission.

One file containing valid SWH is produced for each mission and for a 3-hour time window. It contains the filtered SWH (VAVH), the unfiltered SWH (VAVH_UNFILTERED) and the wind speed (wind_speed).

**DOI (product):**   
https://doi.org/10.48670/moi-00179




<img src="https://mdl-metadata.s3.waw3-1.cloudferro.com/metadata/thumbnails/WAVE_GLO_PHY_SWH_L4_MY_014_007.jpg" width="300" height="200"/>



# GLOBAL OCEAN L4 SIGNIFICANT WAVE HEIGHT FROM REPROCESSED SATELLITE MEASUREMENTS


**Product ID:** WAVE_GLO_PHY_SWH_L4_MY_014_007

**Description:** Multi-Year gridded multi-mission merged satellite significant wave height  based on CMEMS Multi-Year level-3 SWH datasets itself based on the ESA Sea State Climate Change Initiative data Level 3 product (see the product WAVE_GLO_PHY_SWH_L3_MY_014_005). Only valid data are included. It merges along-track SWH data from the following missions: Jason-1, Jason-2, Envisat, Cryosat-2, SARAL/AltiKa, Jason-3 and CFOSAT. Different SWH fields are produced: VAVH_DAILY fields are daily statistics computed from all available level 3 along-track measurements from 00 UTC until 23:59 UTC on a 2° horizontal grid ; VAVH_INST field provides an estimate of the instantaneous wave field at 12:00UTC (noon) on a 0.5° horizontal grid, using all available Level 3 along-track measurements and accounting for their spatial and temporal proximity.

**DOI (product):**   
https://doi.org/10.48670/moi-00177




<img src="https://mdl-metadata.s3.waw3-1.cloudferro.com/metadata/thumbnails/WAVE_GLO_PHY_SWH_L4_NRT_014_003.jpg" width="300" height="200"/>



# GLOBAL OCEAN L4 SIGNIFICANT WAVE HEIGHT FROM NRT SATELLITE MEASUREMENTS


**Product ID:** WAVE_GLO_PHY_SWH_L4_NRT_014_003

**Description:** Near-Real-Time gridded multi-mission merged satellite significant wave height, based on CMEMS level-3 SWH datasets. Onyl valid data are included. It merges multiple along-track SWH data (Sentinel-6A,  Jason-3, Sentinel-3A, Sentinel-3B, SARAL/AltiKa, Cryosat-2, CFOSAT, SWOT-nadir, HaiYang-2B and HaiYang-2C) and produces daily gridded data at  a 2° horizontal resolution. Different SWH fields are produced: VAVH_DAILY fields are daily statistics computed from all available level 3 along-track measurements from 00 UTC until 23:59 UTC ; VAVH_INST field provides an estimate of the instantaneous wave field at 12:00UTC (noon), using all available Level 3 along-track measurements and accounting for their spatial and temporal proximity.

**DOI (product):**   
https://doi.org/10.48670/moi-00180




<img src="https://mdl-metadata.s3.waw3-1.cloudferro.com/metadata/thumbnails/WIND_ARC_PHY_HR_L3_MY_012_105.jpg" width="300" height="200"/>



# High-resolution L3 Sea Surface Wind from MY Satellite Measurements over the Arctic Sea


**Product ID:** WIND_ARC_PHY_HR_L3_MY_012_105

**Description:** For the Arctic Ocean - The product contains daily Level-3 sea surface wind with a 1km horizontal pixel spacing using Synthetic Aperture Radar (SAR) observations and their collocated European Centre for Medium-Range Weather Forecasts (ECMWF) model outputs. Products are processed homogeneously starting from the L2OCN products.

**DOI (product):**  
https://doi.org/10.48670/mds-00338




<img src="https://mdl-metadata.s3.waw3-1.cloudferro.com/metadata/thumbnails/WIND_ARC_PHY_HR_L3_NRT_012_100.jpg" width="300" height="200"/>



# High-resolution L3 Sea Surface Wind from NRT Satellite Measurements over the Arctic Sea


**Product ID:** WIND_ARC_PHY_HR_L3_NRT_012_100

**Description:** For the Arctic Ocean - The product contains daily Level-3 sea surface wind with a 1km horizontal pixel spacing using Near Real-Time Synthetic Aperture Radar (SAR) observations and their collocated European Centre for Medium-Range Weather Forecasts (ECMWF) model outputs. Products are updated several times daily to provide the best product timeliness.'

**DOI (product):**  
https://doi.org/10.48670/mds-00330




<img src="https://mdl-metadata.s3.waw3-1.cloudferro.com/metadata/thumbnails/WIND_ATL_PHY_HR_L3_MY_012_106.jpg" width="300" height="200"/>



# High-resolution L3 Sea Surface Wind from MY Satellite Measurements over the Atlantic Sea


**Product ID:** WIND_ATL_PHY_HR_L3_MY_012_106

**Description:** For the Atlantic Ocean - The product contains daily Level-3 sea surface wind with a 1km horizontal pixel spacing using Synthetic Aperture Radar (SAR) observations and their collocated European Centre for Medium-Range Weather Forecasts (ECMWF) model outputs. Products are processed homogeneously starting from the L2OCN products.

**DOI (product):**  
https://doi.org/10.48670/mds-00339




<img src="https://mdl-metadata.s3.waw3-1.cloudferro.com/metadata/thumbnails/WIND_ATL_PHY_HR_L3_NRT_012_101.jpg" width="300" height="200"/>



# High-resolution L3 Sea Surface Wind from NRT Satellite Measurements over the Atlantic Sea


**Product ID:** WIND_ATL_PHY_HR_L3_NRT_012_101

**Description:** For the Atlantic Ocean - The product contains daily Level-3 sea surface wind with a 1km horizontal pixel spacing using Near Real-Time Synthetic Aperture Radar (SAR) observations and their collocated European Centre for Medium-Range Weather Forecasts (ECMWF) model outputs. Products are updated several times daily to provide the best product timeliness.

**DOI (product):**  
https://doi.org/10.48670/mds-00331




<img src="https://mdl-metadata.s3.waw3-1.cloudferro.com/metadata/thumbnails/WIND_BAL_PHY_HR_L3_MY_012_107.jpg" width="300" height="200"/>



# High-resolution L3 Sea Surface Wind from MY Satellite Measurements over the Baltic Sea


**Product ID:** WIND_BAL_PHY_HR_L3_MY_012_107

**Description:** For the Baltic Sea - The product contains daily Level-3 sea surface wind with a 1km horizontal pixel spacing using Synthetic Aperture Radar (SAR) observations and their collocated European Centre for Medium-Range Weather Forecasts (ECMWF) model outputs. Products are processed homogeneously starting from the L2OCN products.

**DOI (product):**  
https://doi.org/10.48670/mds-00340




<img src="https://mdl-metadata.s3.waw3-1.cloudferro.com/metadata/thumbnails/WIND_BAL_PHY_HR_L3_NRT_012_102.jpg" width="300" height="200"/>



# High-resolution L3 Sea Surface Wind from NRT Satellite Measurements over the Baltic Sea


**Product ID:** WIND_BAL_PHY_HR_L3_NRT_012_102

**Description:** For the Baltic Sea - The product contains daily Level-3 sea surface wind with a 1km horizontal pixel spacing using Near Real-Time Synthetic Aperture Radar (SAR) observations and their collocated European Centre for Medium-Range Weather Forecasts (ECMWF) model outputs. Products are updated several times daily to provide the best product timeliness.

**DOI (product):**  
https://doi.org/10.48670/mds-00332




<img src="https://mdl-metadata.s3.waw3-1.cloudferro.com/metadata/thumbnails/WIND_BLK_PHY_HR_L3_MY_012_108.jpg" width="300" height="200"/>



# High-resolution L3 Sea Surface Wind from MY Satellite Measurements over the Black Sea


**Product ID:** WIND_BLK_PHY_HR_L3_MY_012_108

**Description:** For the Black Sea - The product contains daily Level-3 sea surface wind with a 1km horizontal pixel spacing using Synthetic Aperture Radar (SAR) observations and their collocated European Centre for Medium-Range Weather Forecasts (ECMWF) model outputs. Products are processed homogeneously starting from the L2OCN products.

**DOI (product):**  
https://doi.org/10.48670/mds-00341




<img src="https://mdl-metadata.s3.waw3-1.cloudferro.com/metadata/thumbnails/WIND_BLK_PHY_HR_L3_NRT_012_103.jpg" width="300" height="200"/>



# High-resolution L3 Sea Surface Wind from NRT Satellite Measurements over the Black Sea


**Product ID:** WIND_BLK_PHY_HR_L3_NRT_012_103

**Description:** For the Black Sea - The product contains daily Level-3 sea surface wind with a 1km horizontal pixel spacing using Near Real-Time Synthetic Aperture Radar (SAR) observations and their collocated European Centre for Medium-Range Weather Forecasts (ECMWF) model outputs. Products are updated several times daily to provide the best product timeliness.

**DOI (product):**  
https://doi.org/10.48670/mds-00333




<img src="https://mdl-metadata.s3.waw3-1.cloudferro.com/metadata/thumbnails/WIND_GLO_PHY_CLIMATE_L4_MY_012_003.jpg" width="300" height="200"/>



# Global Ocean Monthly Mean Sea Surface Wind and Stress from Scatterometer and Model


**Product ID:** WIND_GLO_PHY_CLIMATE_L4_MY_012_003

**Description:** For the Global Ocean - The product contains monthly Level-4 sea surface wind and stress fields at 0.25 degrees horizontal spatial resolution. The monthly averaged wind and stress fields are based on monthly average ECMWF ERA5 reanalysis fields, corrected for persistent biases using all available Level-3 scatterometer observations from the Metop-A, Metop-B and Metop-C ASCAT, QuikSCAT SeaWinds and ERS-1 and ERS-2 SCAT satellite instruments.  The applied bias corrections, the standard deviation of the differences and the number of observations used to calculate the monthly average persistent bias are included in the product.

**DOI (product):**   
https://doi.org/10.48670/moi-00181




<img src="https://mdl-metadata.s3.waw3-1.cloudferro.com/metadata/thumbnails/WIND_GLO_PHY_L3_MY_012_005.jpg" width="300" height="200"/>



# Global Ocean Daily Gridded Reprocessed L3 Sea Surface Winds from Scatterometer


**Product ID:** WIND_GLO_PHY_L3_MY_012_005

**Description:** For the Global Ocean - The product contains daily L3 gridded sea surface wind observations from available scatterometers with resolutions corresponding  to the L2 swath products:
*0.5 degrees grid for the 50 km scatterometer L2 inputs,
*0.25 degrees grid based on 25 km scatterometer swath observations,
*and 0.125 degrees based on 12.5 km scatterometer swath observations, i.e., from the coastal products. Data from ascending and descending passes are gridded separately. 

The product provides stress-equivalent wind and stress variables as well as their divergence and curl. The MY L3 products follow the availability of the reprocessed EUMETSAT OSI SAF L2 products and are available for: The ASCAT scatterometer on MetOp-A and Metop-B at 0.125 and 0.25 degrees; The Seawinds scatterometer on QuikSCAT at 0.25 and 0.5 degrees; The AMI scatterometer on ERS-1 and ERS-2 at 0.25 degrees; The OSCAT scatterometer on Oceansat-2 at 0.25 and 0.5 degrees;

**DOI (product):**   
https://doi.org/10.48670/moi-00183




<img src="https://mdl-metadata.s3.waw3-1.cloudferro.com/metadata/thumbnails/WIND_GLO_PHY_L3_NRT_012_002.jpg" width="300" height="200"/>



# Global Ocean Daily Gridded Sea Surface Winds from Scatterometer


**Product ID:** WIND_GLO_PHY_L3_NRT_012_002

**Description:** For the Global Ocean - The product contains daily L3 gridded sea surface wind observations from available scatterometers with resolutions corresponding to the L2 swath products:

*0.5 degrees grid for the 50 km scatterometer L2 inputs,
*0.25 degrees grid based on 25 km scatterometer swath observations,
*and 0.125 degrees based on 12.5 km scatterometer swath observations, i.e., from the coastal products.

Data from ascending and descending passes are gridded separately.
The product provides stress-equivalent wind and stress variables as well as their divergence and curl. The NRT L3 products follow the NRT availability of the EUMETSAT OSI SAF L2 products and are available for:
*The ASCAT scatterometers on Metop-A (discontinued on 15/11/2021), Metop-B and Metop-C at 0.125 and 0.25 degrees;
*The OSCAT scatterometer on Scatsat-1 (discontinued on 28/02/2021) and Oceansat-3 at 0.25 and 0.5 degrees; 
*The HSCAT scatterometer on HY-2B, HY-2C and HY-2D at 0.25 and 0.5 degrees

In addition, the product includes European Centre for Medium-Range Weather Forecasts (ECMWF) operational model forecast wind and stress variables collocated with the scatterometer observations at L2 and processed to L3 in exactly the same way as the scatterometer observations.

**DOI (product):**   
https://doi.org/10.48670/moi-00182




<img src="https://mdl-metadata.s3.waw3-1.cloudferro.com/metadata/thumbnails/WIND_GLO_PHY_L4_MY_012_006.jpg" width="300" height="200"/>



# Global Ocean Hourly Reprocessed Sea Surface Wind and Stress from Scatterometer and Model


**Product ID:** WIND_GLO_PHY_L4_MY_012_006

**Description:** For the Global Ocean - The product contains hourly Level-4 sea surface wind and stress fields at 0.125 and 0.25 degrees horizontal spatial resolution. Scatterometer observations and their collocated European Centre for Medium-Range Weather Forecasts (ECMWF) ERA5 reanalysis model variables are used to calculate temporally-averaged difference fields. These fields are used to correct for persistent biases in hourly ECMWF ERA5 model fields. Bias corrections are based on scatterometer observations from Metop-A, Metop-B, Metop-C ASCAT (0.125 degrees) and QuikSCAT SeaWinds, ERS-1 and ERS-2 SCAT (0.25 degrees). The product provides stress-equivalent wind and stress variables as well as their divergence and curl. The applied bias corrections, the standard deviation of the differences (for wind and stress fields) and difference of variances (for divergence and curl fields) are included in the product.

**DOI (product):**   
https://doi.org/10.48670/moi-00185




<img src="https://mdl-metadata.s3.waw3-1.cloudferro.com/metadata/thumbnails/WIND_GLO_PHY_L4_NRT_012_004.jpg" width="300" height="200"/>



# Global Ocean Hourly Sea Surface Wind and Stress from Scatterometer and Model


**Product ID:** WIND_GLO_PHY_L4_NRT_012_004

**Description:** For the Global Ocean - The product contains hourly Level-4 sea surface wind and stress fields at 0.125 degrees horizontal spatial resolution. Scatterometer observations for Metop-B and Metop-C ASCAT and their collocated European Centre for Medium-Range Weather Forecasts (ECMWF) operational model variables are used to calculate temporally-averaged difference fields. These fields are used to correct for persistent biases in hourly ECMWF operational model fields. The product provides stress-equivalent wind and stress variables as well as their divergence and curl. The applied bias corrections, the standard deviation of the differences (for wind and stress fields) and difference of variances (for divergence and curl fields) are included in the product.

**DOI (product):**   
https://doi.org/10.48670/moi-00305




<img src="https://mdl-metadata.s3.waw3-1.cloudferro.com/metadata/thumbnails/WIND_MED_PHY_HR_L3_MY_012_109.jpg" width="300" height="200"/>



# High-resolution L3 Sea Surface Wind from MY Satellite Measurements over the Mediterranean Sea


**Product ID:** WIND_MED_PHY_HR_L3_MY_012_109

**Description:** For the Mediterranean Sea - The product contains daily Level-3 sea surface wind with a 1km horizontal pixel spacing using Synthetic Aperture Radar (SAR) observations and their collocated European Centre for Medium-Range Weather Forecasts (ECMWF) model outputs. Products are processed homogeneously starting from the L2OCN products.

**DOI (product):**  
https://doi.org/10.48670/mds-00342




<img src="https://mdl-metadata.s3.waw3-1.cloudferro.com/metadata/thumbnails/WIND_MED_PHY_HR_L3_NRT_012_104.jpg" width="300" height="200"/>



# High-resolution L3 Sea Surface Wind from NRT Satellite Measurements over the Mediterranean Sea


**Product ID:** WIND_MED_PHY_HR_L3_NRT_012_104

**Description:** For the Mediterranean Sea - The product contains daily Level-3 sea surface wind with a 1km horizontal pixel spacing using Near Real-Time Synthetic Aperture Radar (SAR) observations and their collocated European Centre for Medium-Range Weather Forecasts (ECMWF) model outputs. Products are updated several times daily to provide the best product timeliness.

**DOI (product):**  
https://doi.org/10.48670/mds-00334




## Download data

In [19]:
import datetime as dt
# Product example: ARCTIC_ANALYSISFORECAST_BGC_002_004
DATASET_ID = "cmems_mod_arc_bgc_anfc_ecosmo_P1D-m"  #  ⟵ from product page (Select one of you choice)

# Small AOI covering Iceland and nearby Arctic (adjust as you wish)
bbox = dict(
    minimum_longitude=-40.0, maximum_longitude=40.0,
    minimum_latitude=60.0,  maximum_latitude=85.0
)

# Use "latest" day that is surely produced (yesterday is safe for NRT)
end = dt.datetime.utcnow().date()
start = end - dt.timedelta(days=1)
time_sel = dict(
    start_datetime=start.isoformat(),
    end_datetime=end.isoformat()
)

# Name of the file we download
out_nc = "arctic_bgc_chl.nc"

In [20]:
# We'll ask for "chl*" but stay robust: if variable name differs, we'll detect it later from the file itself.
# Request only surface layer (0 m). Many CMEMS BGC datasets use "depth" in meters.
cm.subset(
    dataset_id=DATASET_ID,
    variables=None,                      # keep None here; we’ll auto-pick "chl" variable after download
    **bbox,
    minimum_depth=0, maximum_depth=0,    # surface
    **time_sel,
    output_filename=out_nc
)

INFO - 2025-08-14T14:25:06Z - Selected dataset version: "202105"
INFO:copernicusmarine:Selected dataset version: "202105"
INFO - 2025-08-14T14:25:06Z - Selected dataset part: "default"
INFO:copernicusmarine:Selected dataset part: "default"
INFO - 2025-08-14T14:25:16Z - Starting download. Please wait...
INFO:copernicusmarine:Starting download. Please wait...


  0%|          | 0/54 [00:00<?, ?it/s]

INFO - 2025-08-14T14:25:51Z - Successfully downloaded to arctic_bgc_chl.nc
INFO:copernicusmarine:Successfully downloaded to arctic_bgc_chl.nc


ResponseSubset(file_path=PosixPath('arctic_bgc_chl.nc'), output_directory=PosixPath('.'), filename='arctic_bgc_chl.nc', file_size=42.86796183206107, data_transfer_size=531.1198778625953, variables=['chl', 'dissic', 'expc', 'kd', 'model_depth', 'no3', 'nppv', 'o2', 'ph', 'phyc', 'po4', 'si', 'spco2', 'zooc'], coordinates_extent=[GeographicalExtent(minimum=-39.999999999978115, maximum=39.92000000003438, unit='degrees_east', coordinate_id='longitude'), GeographicalExtent(minimum=60.0, maximum=85.0, unit='degrees_north', coordinate_id='latitude'), TimeExtent(minimum='2025-08-13T00:00:00+00:00', maximum='2025-08-14T00:00:00+00:00', unit='iso8601', coordinate_id='time'), GeographicalExtent(minimum=0.0, maximum=0.0, unit='m', coordinate_id='depth')], status='000', message='The request was successful.', file_status='DOWNLOADED')

## Convert the data to TIFF (to plot on map)

In [52]:
import xarray as xr
import rioxarray as rxr

ds = xr.open_dataset(out_nc, decode_times=True)

# ---- Find a chlorophyll-like variable (robust to name variations)
candidates = [v for v in ds.data_vars if ("chl" in v.lower()) or ("chlor" in v.lower())]
if not candidates:
    raise ValueError(f"No chlorophyll-like variable found in {list(ds.data_vars)}")
var = candidates[0]
da = ds[var]

# ---- Select a single time slice (nearest to end date)
if "time" in da.dims:
    da = da.sel(time=da["time"].max())

# ---- Some CMEMS grids are polar stereographic with x/y & grid_mapping.
# Try to attach CRS using the CF grid mapping, otherwise assume lat/lon.
def write_geotiff_from_da(da, tif_path="arctic_bgc_chl.tif"):
    # If we have lat/lon coords, reproject on-the-fly by assigning EPSG:4326
    if set(["lat", "latitude"]).intersection(da.coords) and set(["lon", "longitude"]).intersection(da.coords):
        # Normalize coordinate names
        lat_name = "lat" if "lat" in da.coords else "latitude"
        lon_name = "lon" if "lon" in da.coords else "longitude"
        da = da.rename({lat_name: "lat", lon_name: "lon"})
        # rioxarray expects y,x order named differently; use .rio.set_spatial_dims
        da = da.rio.set_spatial_dims(x_dim="lon", y_dim="lat", inplace=False)
        da = da.rio.write_crs(4326, inplace=False)
        da.rio.to_raster(tif_path)
        return tif_path

    # Else assume projected x/y with CF grid_mapping
    # rioxarray can parse CRS from the grid mapping variable when present.
    # Find 'x','y' or 'rlon','rlat' etc., normalize to x/y.
    spatial_dims = [d for d in da.dims if d.lower() in ("x","y","rlon","rlat","cols","rows")]
    if len(spatial_dims) >= 2:
        # Try to infer x/y order
        # Prefer something like ('x','y'); else sort by "x-like" first
        order = []
        for cand in ("x","rlon","cols"):
            if cand in da.dims: order.append(cand)
        for cand in ("y","rlat","rows"):
            if cand in da.dims: order.append(cand)
        # Fallback: keep existing
        if len(order) != 2: order = da.dims[:2]
        da = da.rename({order[0]: "x", order[1]: "y"})
        # Attach CRS from grid mapping if available
        gm = da.rio.grid_mapping
        if gm is None:
            # As a last resort, try common polar stereographic for ARC MFC
            # (Best effort; proper CRS comes from CF metadata.)
            da = da.rio.write_crs("EPSG:3857", inplace=False)  # Arctic Polar Stereographic
        # Ensure spatial dims are set
        da = da.rio.set_spatial_dims(x_dim="x", y_dim="y", inplace=False)
        da.rio.to_raster(tif_path)
        return tif_path

    raise RuntimeError("Could not determine geospatial coordinates/CRS to write GeoTIFF.")

tif_path = write_geotiff_from_da(da, "arctic_bgc_chl.tif")
tif_path

'arctic_bgc_chl.tif'

## Display the data on a map

In [53]:
import leafmap

# Initialize map
m = leafmap.Map(center=[70, -20], zoom=3)

# Add background layer
m.add_basemap("Esri.OceanBasemap")

# Add our downloaded raster
m.add_raster(
    tif_path,
    layer_name="Chlorophyll-a (surface)",
    zoom_to_layer=True,
    colormap="viridis",
    vmin=0, # Min value for color scale
    vmax=10 # Max value for color scale
)

# Display map
m

Map(center=[72.5, -0.03999999999999915], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_t…

## Display data directly from the web with no download

In [54]:
import leafmap

# The layer identifier you can find them all here: https://data.marine.copernicus.eu/products
LAYER_ID = ("ARCTIC_ANALYSISFORECAST_BGC_002_004/"
            "cmems_mod_arc_bgc_anfc_ecosmo_P1D-m_202105/chl")

# Pick any of the listed times;
ISO_TIME = "2025-07-01T00:00:00Z"

# Optional: elevation (depth, in meters). The default in the XML is ~ -0.494 m (surface layer in z*)
ELEVATION = "-0.4940253794193268"   # or omit the &ELEVATION param to use the default

# Choose a TileMatrixSet that matches your basemap (Web Mercator or WGS84)
TMS = "EPSG:3857"

# Server-side styling. If viridis isn’t available on this layer, switch to 'cmap:matter' or use 'logScale' variant.
STYLE = "cmap:viridis"      # fallback: "cmap:matter" or "cmap:matter,logScale"

# Make URL
WMTS_URL = (
    "https://wmts.marine.copernicus.eu/teroWmts"
    "?SERVICE=WMTS&REQUEST=GetTile&VERSION=1.0.0"
    f"&LAYER={LAYER_ID}"
    "&FORMAT=image/png"
    f"&TILEMATRIXSET={TMS}"
    "&TILEMATRIX={z}&TILEROW={y}&TILECOL={x}"
    f"&time={ISO_TIME}"
    f"&STYLE={STYLE}"
    f"&ELEVATION={ELEVATION}"
)

# Prepare map
m = leafmap.Map(center=[30, 0], zoom=2)

# Add our WMTS layer
m.add_tile_layer(
    url=WMTS_URL,
    name="CMEMS Zooplankton (monthly)",
    attribution="Copernicus Marine"
)
m

Map(center=[30, 0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_text…

The difference is that on the online data served as Web Map Tile Server (WMTS) it is not possible to process the data, while on the downloaded data it is possibe to combine it with other data or extract the single pixel values.